In [1]:
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# #Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
# !pip install ipython-sql

In [2]:
import pandas as pd
import numpy as np
import ibm_db,ibm_db_sa
import ibm_db_dbi


# Connecting to IBM CloudPak DB2 

In [3]:
%load_ext sql
# run custom file with IBM's CloudPak DB2 credentials - for privacy purposes
%run hide/IBM_Db2-credentials.py

In [4]:
# %reload_ext sql
# # run custom file with IBM's CloudPak DB2 credentials - for privacy purposes
# %run hide/IBM_Db2-credentials.py

In [5]:
%sql ibm_db_sa://{dsn_uid}:{dsn_pwd}@{dsn_hostname}:{dsn_port}/{dsn_database}?security={dsn_security}

In [6]:
#Create the dsn connection string

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
list_dsn_info = dsn.split(';')
index_nums = [0,1,2,3,4,5,7]
output = [list_dsn_info[val] for val in index_nums]
print(output)

['DRIVER={IBM DB2 ODBC DRIVER}', 'DATABASE=bludb', 'HOSTNAME=54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud', 'PORT=32733', 'PROTOCOL=TCPIP', 'UID=bvr08213', 'SECURITY=SSL']


In [7]:
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print (f"Connected to database: {dsn_database} \nAs user: {dsn_uid} \nOn host: {dsn_hostname}")

except:
    print (f"Unable to connect: {ibm_db.conn_errormsg()}" )


Connected to database: bludb 
As user: bvr08213 
On host: 54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


In [8]:
#Retrieve Metadata for the Database Server

server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.05.0600
DB_NAME:    BLUDB


In [9]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           DB2CLI.DLL
DRIVER_VER:            11.05.0500
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1252
CONN_CODEPAGE:         1208


# Loading data 

In [10]:
census_data = pd.read_csv('hide/data/ChicagoCensusData.csv', header=0, keep_default_na=False, na_values=['','nan','NDA',np.nan,None])
print('Chicago Census Data')
display(census_data)

crime_data = pd.read_csv('hide/data/ChicagoCrimeData.csv', header=0, keep_default_na=False, na_values=['','nan','NDA',np.nan,None])
print('Chicago Crime Data')
display(crime_data)

school_data = pd.read_csv('hide/data/ChicagoPublicSchools.csv', header=0, keep_default_na=False, na_values=['','nan','NDA',np.nan,None])
print('Chicago Public Schools Data')
display(school_data)

Chicago Census Data


,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6
...,...,...,...,...,...,...,...,...,...
72,73,Washington Height,1.1,16.9,20.8,13.7,42.6,19713,48
73,74,Mount Greenwood,1.0,3.4,8.7,4.3,36.8,34381,16
74,75,Morgan Park,0.8,13.2,15.0,10.8,40.3,27149,30
75,76,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24


Chicago Crime Data


,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,8/28/2004,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.807441,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,6/26/2004,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,...,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,4/4/2011,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,...,2,3.0,38.0,6,1177436.0,1876313.0,2011,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,12/30/2010,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,4,7.0,46.0,6,1194622.0,1850125.0,2010,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,2/2/2016,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,...,8,15.0,66.0,6,1155240.0,1860661.0,2016,41.773455,-87.706480,"(41.773455295, -87.706480471)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,10453948,HZ192829,3/1/2016,028XX E 79TH ST,1055,HUMAN TRAFFICKING,INVOLUNTARY SERVITUDE,APARTMENT,False,False,...,4,7.0,46.0,26,1196679.0,1853139.0,2016,41.751885,-87.554826,"(41.751885152, -87.554825997)"
529,10397129,HZ133234,1/29/2016,006XX W HARRISON ST,5114,NON - CRIMINAL,FOID - REVOCATION,STREET,False,False,...,1,2.0,28.0,26,1172257.0,1897564.0,2016,41.874363,-87.643013,"(41.874363279, -87.643013039)"
530,3269495,HJ747227,11/8/2003,012XX W 81ST ST,510,RITUALISM,AGG RIT MUT: HANDS/FIST/FEET SERIOUS INJURY,OTHER,False,False,...,6,21.0,71.0,04B,1169648.0,1851076.0,2003,41.746852,-87.653941,"(41.746852486, -87.653941385)"
531,10840565,JA143710,2/6/2017,009XX N KARLOV AVE,5073,NON-CRIMINAL (SUBJECT SPECIFIED),NOTIFICATION OF CIVIL NO CONTACT ORDER,RESIDENCE,False,True,...,11,37.0,23.0,26,1148881.0,1905963.0,2017,41.897895,-87.728622,"(41.89789489, -87.728622316)"


Chicago Public Schools Data


,School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,...,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NaN,1171699.458,1915829.428,41.924497,-87.644522,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressRep...,Skyway Elementary Network,...,NaN,1196129.985,1856209.466,41.760324,-87.556736,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressRep...,Midway Elementary Network,...,NaN,1148427.165,1851012.215,41.747111,-87.731702,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
3,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressRep...,Pershing Elementary Network,...,NaN,1164504.290,1873959.199,41.809757,-87.672145,61,NEW CITY,20,9,"(41.8097569, -87.6721446)"
4,610513,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,60609,(773) 535-1590,http://schoolreports.cps.edu/SchoolProgressRep...,Southwest Side High School Network,...,91.8,1175177.622,1880745.126,41.828146,-87.632794,34,ARMOUR SQUARE,11,9,"(41.82814609, -87.63279369)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,610172,William T Sherman Elementary School,ES,1000 W 52nd St,Chicago,IL,60609,(773) 535-1757,http://schoolreports.cps.edu/SchoolProgressRep...,AUSL Schools,...,NaN,1170500.817,1870373.159,41.799788,-87.650255,61,NEW CITY,16,9,"(41.79978772, -87.65025483)"
562,609844,William W Carter Elementary School,ES,5740 S Michigan Ave,Chicago,IL,60637,(773) 535-0860,http://schoolreports.cps.edu/SchoolProgressRep...,Burnham Park Elementary Network,...,NaN,1178101.365,1866810.123,41.789841,-87.622490,40,WASHINGTON PARK,20,2,"(41.78984129, -87.62248974)"
563,610088,Wolfgang A Mozart Elementary School,ES,2200 N Hamlin Ave,Chicago,IL,60647,(773) 534-4160,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NaN,1150644.396,1914368.955,41.920927,-87.721925,22,LOGAN SQUARE,35,25,"(41.92092734, -87.72192541)"
564,609977,Woodlawn Community Elementary School,ES,6657 S Kimbark Ave,Chicago,IL,60637,(773) 535-0801,http://schoolreports.cps.edu/SchoolProgressRep...,Burnham Park Elementary Network,...,NaN,1185825.188,1860883.579,41.773400,-87.594356,42,WOODLAWN,5,3,"(41.77339962, -87.59435584)"


In [11]:
#renaming columns for inserting ease later (columns containing spaces,commas, and starts with numbers)
school_data.rename(columns={'Elementary, Middle, or High School':'Elementary_Middle_or_High_School',
                            '9th Grade EXPLORE (2009)':'Gr_9_EXPLORE_2009',
                            '9th Grade EXPLORE (2010)':'Gr_9_EXPLORE_2010',
                            '10th Grade PLAN (2009)':'Gr_10_PLAN_2009',
                            '10th Grade PLAN (2010)':'Gr_10_PLAN_2010',
                            '11th Grade Average ACT (2011)':'Gr_11_Average_ACT_2011'
                            }, inplace= True)
school_data

,School_ID,NAME_OF_SCHOOL,Elementary_Middle_or_High_School,Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,...,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NaN,1171699.458,1915829.428,41.924497,-87.644522,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressRep...,Skyway Elementary Network,...,NaN,1196129.985,1856209.466,41.760324,-87.556736,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressRep...,Midway Elementary Network,...,NaN,1148427.165,1851012.215,41.747111,-87.731702,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
3,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressRep...,Pershing Elementary Network,...,NaN,1164504.290,1873959.199,41.809757,-87.672145,61,NEW CITY,20,9,"(41.8097569, -87.6721446)"
4,610513,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,60609,(773) 535-1590,http://schoolreports.cps.edu/SchoolProgressRep...,Southwest Side High School Network,...,91.8,1175177.622,1880745.126,41.828146,-87.632794,34,ARMOUR SQUARE,11,9,"(41.82814609, -87.63279369)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,610172,William T Sherman Elementary School,ES,1000 W 52nd St,Chicago,IL,60609,(773) 535-1757,http://schoolreports.cps.edu/SchoolProgressRep...,AUSL Schools,...,NaN,1170500.817,1870373.159,41.799788,-87.650255,61,NEW CITY,16,9,"(41.79978772, -87.65025483)"
562,609844,William W Carter Elementary School,ES,5740 S Michigan Ave,Chicago,IL,60637,(773) 535-0860,http://schoolreports.cps.edu/SchoolProgressRep...,Burnham Park Elementary Network,...,NaN,1178101.365,1866810.123,41.789841,-87.622490,40,WASHINGTON PARK,20,2,"(41.78984129, -87.62248974)"
563,610088,Wolfgang A Mozart Elementary School,ES,2200 N Hamlin Ave,Chicago,IL,60647,(773) 534-4160,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NaN,1150644.396,1914368.955,41.920927,-87.721925,22,LOGAN SQUARE,35,25,"(41.92092734, -87.72192541)"
564,609977,Woodlawn Community Elementary School,ES,6657 S Kimbark Ave,Chicago,IL,60637,(773) 535-0801,http://schoolreports.cps.edu/SchoolProgressRep...,Burnham Park Elementary Network,...,NaN,1185825.188,1860883.579,41.773400,-87.594356,42,WOODLAWN,5,3,"(41.77339962, -87.59435584)"


In [12]:
school_data.College_Enrollment_Rate__

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
561     NaN
562     NaN
563     NaN
564     NaN
565    48.1
Name: College_Enrollment_Rate__, Length: 566, dtype: float64

In [13]:
school_data.isnull().sum().sort_values()

School_ID                      0
Ward                           0
COMMUNITY_AREA_NAME            0
COMMUNITY_AREA_NUMBER          0
Longitude                      0
                            ... 
Gr_11_Average_ACT_2011       483
Net_Change_PLAN_and_ACT      484
Gr_10_PLAN_2009              484
Graduation_Rate__            487
College_Enrollment_Rate__    489
Length: 78, dtype: int64

In [14]:
from datetime import datetime
crime_data['DATE'] =  pd.to_datetime(crime_data['DATE'])

In [15]:
display("Chicago's Census Data",census_data.info())
display("Chicago's Crime Data",crime_data.info())
display("Chicago's School Data",school_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 9 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   COMMUNITY_AREA_NUMBER                         77 non-null     int64  
 1   COMMUNITY_AREA_NAME                           77 non-null     object 
 2   PERCENT_OF_HOUSING_CROWDED                    77 non-null     float64
 3   PERCENT_HOUSEHOLDS_BELOW_POVERTY              77 non-null     float64
 4   PERCENT_AGED_16__UNEMPLOYED                   77 non-null     float64
 5   PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA  77 non-null     float64
 6   PERCENT_AGED_UNDER_18_OR_OVER_64              77 non-null     float64
 7   PER_CAPITA_INCOME                             77 non-null     int64  
 8   HARDSHIP_INDEX                                77 non-null     int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 5.5+ KB


"Chicago's Census Data"

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     533 non-null    int64         
 1   CASE_NUMBER            533 non-null    object        
 2   DATE                   533 non-null    datetime64[ns]
 3   BLOCK                  533 non-null    object        
 4   IUCR                   533 non-null    object        
 5   PRIMARY_TYPE           533 non-null    object        
 6   DESCRIPTION            533 non-null    object        
 7   LOCATION_DESCRIPTION   533 non-null    object        
 8   ARREST                 533 non-null    bool          
 9   DOMESTIC               533 non-null    bool          
 10  BEAT                   533 non-null    int64         
 11  DISTRICT               533 non-null    int64         
 12  WARD                   490 non-null    float64       
 13  COMMU

"Chicago's Crime Data"

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 78 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   School_ID                                         566 non-null    int64  
 1   NAME_OF_SCHOOL                                    566 non-null    object 
 2   Elementary_Middle_or_High_School                  566 non-null    object 
 3   Street_Address                                    566 non-null    object 
 4   City                                              566 non-null    object 
 5   State                                             566 non-null    object 
 6   ZIP_Code                                          566 non-null    int64  
 7   Phone_Number                                      566 non-null    object 
 8   Link                                              565 non-null    object 
 9   Network_Manager      

"Chicago's School Data"

None

# Dropping tables

In [16]:
%%sql

DROP TABLE IF EXISTS CENSUS_DATA;

DROP TABLE IF EXISTS CRIME_DATA;

DROP TABLE IF EXISTS PUBLIC_SCHOOLS;


 * ibm_db_sa://bvr08213:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.
Done.
Done.


[]

# Creating Tables

In [17]:

%%sql


CREATE TABLE IF NOT EXISTS CENSUS_DATA (
                          
                          COMMUNITY_AREA_NUMBER INTEGER PRIMARY KEY NOT NULL,
                          COMMUNITY_AREA_NAME VARCHAR(30), 
                          PERCENT_OF_HOUSING_CROWDED REAL, 
                          PERCENT_HOUSEHOLDS_BELOW_POVERTY REAL, 
                          PERCENT_AGED_16__UNEMPLOYED REAL,
                          PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA REAL,
                          PERCENT_AGED_UNDER_18_OR_OVER_64 REAL,
                          PER_CAPITA_INCOME INTEGER,
                          HARDSHIP_INDEX INTEGER
                                      
                          );



CREATE TABLE IF NOT EXISTS CRIME_DATA (

                                 ID INTEGER PRIMARY KEY NOT NULL, 
                                 CASE_NUMBER VARCHAR(12), 
                                 DATE DATE, 
                                 BLOCK VARCHAR(40), 
                                 IUCR VARCHAR(5), 
                                 PRIMARY_TYPE VARCHAR(35), 
                                 DESCRIPTION VARCHAR(50), 
                                 LOCATION_DESCRIPTION VARCHAR(50), 
                                 ARREST BOOLEAN, 
                                 DOMESTIC BOOLEAN,
                                 BEAT INTEGER,
                                 DISTRICT INTEGER, 
                                 WARD INTEGER, 
                                 COMMUNITY_AREA_NUMBER INTEGER, 
                                 FBICODE VARCHAR(5), 
                                 X_COORDINATE INTEGER, 
                                 Y_COORDINATE INTEGER, 
                                 YEAR INTEGER, 
                                 LATITUDE REAL, 
                                 LONGITUDE REAL, 
                                 LOCATION VARCHAR(30)
                                                
                                 );

CREATE TABLE IF NOT EXISTS PUBLIC_SCHOOLS (

                                     School_ID INTEGER PRIMARY KEY NOT NULL, 
                                     NAME_OF_SCHOOL VARCHAR(100), 
                                     Elementary_Middle_or_High_School VARCHAR(3), 
                                     Street_Address VARCHAR(30), 
                                     City VARCHAR(8),
                                     State VARCHAR(3),
                                     ZIP_Code INTEGER,
                                     Phone_Number VARCHAR(16),
                                     Link VARCHAR(100),
                                     Network_Manager VARCHAR(40),
                                     Collaborative_Name VARCHAR(40),
                                     Adequate_Yearly_Progress_Made_ VARCHAR(4),
                                     Track_Schedule VARCHAR(16),
                                     CPS_Performance_Policy_Status VARCHAR(16),
                                     CPS_Performance_Policy_Level VARCHAR(20),
                                     HEALTHY_SCHOOL_CERTIFIED VARCHAR(4),
                                     Safety_Icon VARCHAR(12),
                                     SAFETY_SCORE INTEGER,
                                     Family_Involvement_Icon VARCHAR(12),
                                     Family_Involvement_Score INTEGER,
                                     Environment_Icon VARCHAR(12),
                                     Environment_Score INTEGER, 
                                     Instruction_Icon VARCHAR(12),
                                     Instruction_Score INTEGER, 
                                     Leaders_Icon VARCHAR(12),
                                     Leaders_Score INTEGER, 
                                     Teachers_Icon VARCHAR(12),
                                     Teachers_Score INTEGER, 
                                     Parent_Engagement_Icon VARCHAR(12),
                                     Parent_Engagement_Score INTEGER,
                                     Parent_Environment_Icon VARCHAR(12),
                                     Parent_Environment_Score INTEGER,
                                     AVERAGE_STUDENT_ATTENDANCE VARCHAR(7), 
                                     Rate_of_Misconducts__per_100_students_ REAL,
                                     Average_Teacher_Attendance VARCHAR(7),
                                     Individualized_Education_Program_Compliance_Rate VARCHAR(7), 
                                     Pk_2_Literacy__ REAL,
                                     Pk_2_Math__ REAL, 
                                     Gr3_5_Grade_Level_Math__ REAL, 
                                     Gr3_5_Grade_Level_Read__ REAL,
                                     Gr3_5_Keep_Pace_Read__ REAL, 
                                     Gr3_5_Keep_Pace_Math__ REAL,
                                     Gr6_8_Grade_Level_Math__ REAL, 
                                     Gr6_8_Grade_Level_Read__ REAL,
                                     Gr6_8_Keep_Pace_Math_ REAL, 
                                     Gr6_8_Keep_Pace_Read__ REAL,
                                     Gr_8_Explore_Math__ REAL, 
                                     Gr_8_Explore_Read__ REAL, 
                                     ISAT_Exceeding_Math__ REAL,
                                     ISAT_Exceeding_Reading__ REAL, 
                                     ISAT_Value_Add_Math REAL,
                                     ISAT_Value_Add_Read REAL, 
                                     ISAT_Value_Add_Color_Math VARCHAR(8),
                                     ISAT_Value_Add_Color_Read VARCHAR(8),
                                     Students_Taking__Algebra__ REAL,
                                     Students_Passing__Algebra__ REAL, 
                                     Gr_9_EXPLORE_2009 REAL,
                                     Gr_9_EXPLORE_2010 REAL, 
                                     Gr_10_PLAN_2009 REAL,
                                     Gr_10_PLAN_2010 REAL, 
                                     Net_Change_EXPLORE_and_PLAN REAL,
                                     Gr_11_Average_ACT_2011 REAL, 
                                     Net_Change_PLAN_and_ACT REAL,
                                     College_Eligibility__ REAL, 
                                     Graduation_Rate__ REAL,
                                     College_Enrollment_Rate__ REAL, 
                                     COLLEGE_ENROLLMENT INTEGER,
                                     General_Services_Route INTEGER, 
                                     Freshman_on_Track_Rate__ REAL, 
                                     X_COORDINATE VARCHAR(20), 
                                     Y_COORDINATE VARCHAR(20), 
                                     LATITUDE REAL,
                                     LONGITUDE REAL, 
                                     COMMUNITY_AREA_NUMBER INTEGER,
                                     COMMUNITY_AREA_NAME VARCHAR(30), 
                                     WARD INTEGER, 
                                     Police_District INTEGER, 
                                     LOCATION VARCHAR(30)
                                     
                                     );

 * ibm_db_sa://bvr08213:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.
Done.
Done.


[]

In [18]:
#function to auto create table <--- NICE BASE TO WORK FROM (Needs some manual tweaking but works very well)
def SQL_CREATE_STATEMENT_FROM_DATAFRAME(DATAFRAME, TABLE_NAME):

    import pandas as pd
    sql_text = pd.io.sql.get_schema(DATAFRAME.reset_index(), TABLE_NAME)   
    print(sql_text)

SQL_CREATE_STATEMENT_FROM_DATAFRAME(school_data,'PUBLIC_SCHOOLS')

CREATE TABLE "PUBLIC_SCHOOLS" (
"index" INTEGER,
  "School_ID" INTEGER,
  "NAME_OF_SCHOOL" TEXT,
  "Elementary_Middle_or_High_School" TEXT,
  "Street_Address" TEXT,
  "City" TEXT,
  "State" TEXT,
  "ZIP_Code" INTEGER,
  "Phone_Number" TEXT,
  "Link" TEXT,
  "Network_Manager" TEXT,
  "Collaborative_Name" TEXT,
  "Adequate_Yearly_Progress_Made_" TEXT,
  "Track_Schedule" TEXT,
  "CPS_Performance_Policy_Status" TEXT,
  "CPS_Performance_Policy_Level" TEXT,
  "HEALTHY_SCHOOL_CERTIFIED" TEXT,
  "Safety_Icon" TEXT,
  "SAFETY_SCORE" REAL,
  "Family_Involvement_Icon" TEXT,
  "Family_Involvement_Score" REAL,
  "Environment_Icon" TEXT,
  "Environment_Score" REAL,
  "Instruction_Icon" TEXT,
  "Instruction_Score" REAL,
  "Leaders_Icon" TEXT,
  "Leaders_Score" REAL,
  "Teachers_Icon" TEXT,
  "Teachers_Score" REAL,
  "Parent_Engagement_Icon" TEXT,
  "Parent_Engagement_Score" REAL,
  "Parent_Environment_Icon" TEXT,
  "Parent_Environment_Score" REAL,
  "AVERAGE_STUDENT_ATTENDANCE" TEXT,
  "Rate_of_Misco

In [19]:
SQL_CREATE_STATEMENT_FROM_DATAFRAME(crime_data ,'CRIME_SCHOOLS')

CREATE TABLE "CRIME_SCHOOLS" (
"index" INTEGER,
  "ID" INTEGER,
  "CASE_NUMBER" TEXT,
  "DATE" TIMESTAMP,
  "BLOCK" TEXT,
  "IUCR" TEXT,
  "PRIMARY_TYPE" TEXT,
  "DESCRIPTION" TEXT,
  "LOCATION_DESCRIPTION" TEXT,
  "ARREST" INTEGER,
  "DOMESTIC" INTEGER,
  "BEAT" INTEGER,
  "DISTRICT" INTEGER,
  "WARD" REAL,
  "COMMUNITY_AREA_NUMBER" REAL,
  "FBICODE" TEXT,
  "X_COORDINATE" REAL,
  "Y_COORDINATE" REAL,
  "YEAR" INTEGER,
  "LATITUDE" REAL,
  "LONGITUDE" REAL,
  "LOCATION" TEXT
)


# Insert

In [20]:
#function to auto create insert statement from dataframe

def SQL_INSERT_STATEMENT_FROM_DATAFRAME(DATAFRAME, TABLE_NAME):
    sql_texts = []
    for index, row in DATAFRAME.iterrows():       
        sql_texts.append('INSERT INTO '+TABLE_NAME+' ('+ str(', '.join(DATAFRAME.columns))+ ') VALUES '+ str(tuple(row.values)))        
    return " ".join(sql_texts)

census_data_insert = SQL_INSERT_STATEMENT_FROM_DATAFRAME(census_data,'CENSUS_DATA')
crime_data_insert = SQL_INSERT_STATEMENT_FROM_DATAFRAME(crime_data,'CRIME_DATA')
school_data_insert = SQL_INSERT_STATEMENT_FROM_DATAFRAME(school_data,'PUBLIC_SCHOOLS')

#print example
print(census_data_insert[:2000])

INSERT INTO CENSUS_DATA (COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_OF_HOUSING_CROWDED, PERCENT_HOUSEHOLDS_BELOW_POVERTY, PERCENT_AGED_16__UNEMPLOYED, PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA, PERCENT_AGED_UNDER_18_OR_OVER_64, PER_CAPITA_INCOME, HARDSHIP_INDEX) VALUES (1, 'Rogers Park', 7.7, 23.6, 8.7, 18.2, 27.5, 23939, 39) INSERT INTO CENSUS_DATA (COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_OF_HOUSING_CROWDED, PERCENT_HOUSEHOLDS_BELOW_POVERTY, PERCENT_AGED_16__UNEMPLOYED, PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA, PERCENT_AGED_UNDER_18_OR_OVER_64, PER_CAPITA_INCOME, HARDSHIP_INDEX) VALUES (2, 'West Ridge', 7.8, 17.2, 8.8, 20.8, 38.5, 23040, 46) INSERT INTO CENSUS_DATA (COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_OF_HOUSING_CROWDED, PERCENT_HOUSEHOLDS_BELOW_POVERTY, PERCENT_AGED_16__UNEMPLOYED, PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA, PERCENT_AGED_UNDER_18_OR_OVER_64, PER_CAPITA_INCOME, HARDSHIP_INDEX) VALUES (3, 'Uptown', 3.8, 24.0, 8.9, 11.8, 22.

In [21]:
# d = """INSERT INTO CENSUS_DATA (
#                          COMMUNITY_AREA_NUMBER, 
#                          COMMUNITY_AREA_NAME, 
#                          PERCENT_OF_HOUSING_CROWDED, 
#                          PERCENT_HOUSEHOLDS_BELOW_POVERTY, 
#                          PERCENT_AGED_16__UNEMPLOYED, 
#                          PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA, 
#                          PERCENT_AGED_UNDER_18_OR_OVER_64, 
#                          PER_CAPITA_INCOME, HARDSHIP_INDEX
#                         ) 

#  VALUES 
# (1, 'Rogers Park', 7.7, 23.6, 8.7, 18.2, 27.5, 23939, 39),\n (2, 'West Ridge', 7.8, 17.2, 8.8, 20.8, 38.5, 23040, 46),\n (3, 'Uptown', 3.8, 24.0, 8.9, 11.8, 22.2, 35787, 20),\n (4, 'Lincoln Square', 3.4, 10.9, 8.2, 13.4, 25.5, 37524, 17),\n (5, 'North Center', 0.3, 7.5, 5.2, 4.5, 26.2, 57123, 6),\n (6, 'Lake View', 1.1, 11.4, 4.7, 2.6, 17.0, 60058, 5),\n (7, 'Lincoln Park', 0.8, 12.3, 5.1, 3.6, 21.5, 71551, 2),\n (8, 'Near North Side', 1.9, 12.9, 7.0, 2.5, 22.6, 88669, 1),\n (9, 'Edison Park', 1.1, 3.3, 6.5, 7.4, 35.3, 40959, 8),\n (10, 'Norwood Park', 2.0, 5.4, 9.0, 11.5, 39.5, 32875, 21),\n (11, 'Jefferson Park', 2.7, 8.6, 12.4, 13.4, 35.5, 27751, 25),\n (12, 'Forest Glen', 1.1, 7.5, 6.8, 4.9, 40.5, 44164, 11),\n (13, 'North Park', 3.9, 13.2, 9.9, 14.4, 39.0, 26576, 33),\n (14, 'Albany Park', 11.3, 19.2, 10.0, 32.9, 32.0, 21323, 53),\n (15, 'Portage Park', 4.1, 11.6, 12.6, 19.3, 34.0, 24336, 35),\n (16, 'Irving Park', 6.3, 13.1, 10.0, 22.4, 31.6, 27249, 34),\n (17, 'Dunning', 5.2, 10.6, 10.0, 16.2, 33.6, 26282, 28),\n (18, 'Montclaire', 8.1, 15.3, 13.8, 23.5, 38.6, 22014, 50),\n (19, 'Belmont Cragin', 10.8, 18.7, 14.6, 37.3, 37.3, 15461, 70),\n (20, 'Hermosa', 6.9, 20.5, 13.1, 41.6, 36.4, 15089, 71),\n (21, 'Avondale', 6.0, 15.3, 9.2, 24.7, 31.0, 20039, 42),\n (22, 'Logan Square', 3.2, 16.8, 8.2, 14.8, 26.2, 31908, 23),\n (23, 'Humboldt park', 14.8, 33.9, 17.3, 35.4, 38.0, 13781, 85),\n (24, 'West Town', 2.3, 14.7, 6.6, 12.9, 21.7, 43198, 10),\n (25, 'Austin', 6.3, 28.6, 22.6, 24.4, 37.9, 15957, 73),\n (26, 'West Garfield Park', 9.4, 41.7, 25.8, 24.5, 43.6, 10934, 92),\n (27, 'East Garfield Park', 8.2, 42.4, 19.6, 21.3, 43.2, 12961, 83),\n (28, 'Near West Side', 3.8, 20.6, 10.7, 9.6, 22.2, 44689, 15),\n (29, 'North Lawndale', 7.4, 43.1, 21.2, 27.6, 42.7, 12034, 87),\n (30, 'South Lawndale', 15.2, 30.7, 15.8, 54.8, 33.8, 10402, 96),\n (31, 'Lower West Side', 9.6, 25.8, 15.8, 40.7, 32.6, 16444, 76),\n (32, 'Loop', 1.5, 14.7, 5.7, 3.1, 13.5, 65526, 3),\n (33, 'Near South Side', 1.3, 13.8, 4.9, 7.4, 21.8, 59077, 7),\n (34, 'Armour Square', 5.7, 40.1, 16.7, 34.5, 38.3, 16148, 82),\n (35, 'Douglas', 1.8, 29.6, 18.2, 14.3, 30.7, 23791, 47),\n (36, 'Oakland', 1.3, 39.7, 28.7, 18.4, 40.4, 19252, 78),\n (37, 'Fuller Park', 3.2, 51.2, 33.9, 26.6, 44.9, 10432, 97),\n (38, 'Grand Boulevard', 3.3, 29.3, 24.3, 15.9, 39.5, 23472, 57),\n (39, 'Kenwood', 2.4, 21.7, 15.7, 11.3, 35.4, 35911, 26),\n (40, 'Washington Park', 5.6, 42.1, 28.6, 25.4, 42.8, 13785, 88),\n (41, 'Hyde Park', 1.5, 18.4, 8.4, 4.3, 26.2, 39056, 14),\n (42, 'Woodlawn', 2.9, 30.7, 23.4, 16.5, 36.1, 18672, 58),\n (43, 'South Shore', 2.8, 31.1, 20.0, 14.0, 35.7, 19398, 55),\n (44, 'Chatham', 3.3, 27.8, 24.0, 14.5, 40.3, 18881, 60),\n (45, 'Avalon Park', 1.4, 17.2, 21.1, 10.6, 39.3, 24454, 41),\n (46, 'South Chicago', 4.7, 29.8, 19.7, 26.6, 41.1, 16579, 75),\n (47, 'Burnside', 6.8, 33.0, 18.6, 19.3, 42.7, 12515, 79),\n (48, 'Calumet Heights', 2.1, 11.5, 20.0, 11.0, 44.0, 28887, 38),\n (49, 'Roseland', 2.5, 19.8, 20.3, 16.9, 41.2, 17949, 52),\n (50, 'Pullman', 1.5, 21.6, 22.8, 13.1, 38.6, 20588, 51),\n (51, 'South Deering', 4.0, 29.2, 16.3, 21.0, 39.5, 14685, 65),\n (52, 'East Side', 6.8, 19.2, 12.1, 31.9, 42.8, 17104, 64),\n (53, 'West Pullman', 3.3, 25.9, 19.4, 20.5, 42.1, 16563, 62),\n (54, 'Riverdale', 5.8, 56.5, 34.6, 27.5, 51.5, 8201, 98),\n (55, 'Hegewisch', 3.3, 17.1, 9.6, 19.2, 42.9, 22677, 44),\n (56, 'Garfield Ridge', 2.6, 8.8, 11.3, 19.3, 38.1, 26353, 32),\n (57, 'Archer Heights', 8.5, 14.1, 16.5, 35.9, 39.2, 16134, 67),\n (58, 'Brighton Park', 14.4, 23.6, 13.9, 45.1, 39.3, 13089, 84),\n (59, 'McKinley Park', 7.2, 18.7, 13.4, 32.9, 35.6, 16954, 61),\n (60, 'Bridgeport', 4.5, 18.9, 13.7, 22.2, 31.3, 22694, 43),\n (61, 'New City', 11.9, 29.0, 23.0, 41.5, 38.9, 12765, 91),\n (62, 'West Elsdon', 11.1, 15.6, 16.7, 37.0, 37.7, 15754, 69),\n (63, 'Gage Park', 15.8, 23.4, 18.2, 51.5, 38.8, 12171, 93),\n (64, 'Clearing', 2.7, 8.9, 9.5, 18.8, 37.6, 25113, 29),\n (65, 'West Lawn', 5.8, 14.9, 9.6, 33.6, 39.6, 16907, 56),\n (66, 'Chicago Lawn', 7.6, 27.9, 17.1, 31.2, 40.6, 13231, 80),\n (67, 'West Englewood', 4.8, 34.4, 35.9, 26.3, 40.7, 11317, 89),\n (68, 'Englewood', 3.8, 46.6, 28.0, 28.5, 42.5, 11888, 94),\n (69, 'Greater Grand Crossing', 3.6, 29.6, 23.0, 16.5, 41.0, 17285, 66),\n (70, 'Ashburn', 4.0, 10.4, 11.7, 17.7, 36.9, 23482, 37),\n (71, 'Auburn Gresham', 4.0, 27.6, 28.3, 18.5, 41.9, 15528, 74),\n (72, 'Beverly', 0.9, 5.1, 8.0, 3.7, 40.5, 39523, 12),\n (73, 'Washington Height', 1.1, 16.9, 20.8, 13.7, 42.6, 19713, 48),\n (74, 'Mount Greenwood', 1.0, 3.4, 8.7, 4.3, 36.8, 34381, 16),\n (75, 'Morgan Park', 0.8, 13.2, 15.0, 10.8, 40.3, 27149, 30),\n (76, "O'Hare", 3.6, 15.4, 7.1, 10.9, 30.3, 25828, 24),\n (77, 'Edgewater', 4.1, 18.2, 9.2, 9.7, 23.8, 33385, 19)
# """
# print(d)

In [22]:
%%sql
INSERT INTO BVR08213.CENSUS_DATA 
                                (COMMUNITY_AREA_NUMBER, 
                                 COMMUNITY_AREA_NAME, 
                                 PERCENT_OF_HOUSING_CROWDED, 
                                 PERCENT_HOUSEHOLDS_BELOW_POVERTY, 
                                 PERCENT_AGED_16__UNEMPLOYED, 
                                 PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA, 
                                 PERCENT_AGED_UNDER_18_OR_OVER_64, 
                                 PER_CAPITA_INCOME, 
                                 HARDSHIP_INDEX) 

 VALUES 
       (1, 'Rogers Park', 7.7, 23.6, 8.7, 18.2, 27.5, 23939, 39), 
       (2, 'West Ridge', 7.8, 17.2, 8.8, 20.8, 38.5, 23040, 46),
       (3, 'Uptown', 3.8, 24.0, 8.9, 11.8, 22.2, 35787, 20),
       (4, 'Lincoln Square', 3.4, 10.9, 8.2, 13.4, 25.5, 37524, 17),
       (5, 'North Center', 0.3, 7.5, 5.2, 4.5, 26.2, 57123, 6),
       (6, 'Lake View', 1.1, 11.4, 4.7, 2.6, 17.0, 60058, 5),
       (7, 'Lincoln Park', 0.8, 12.3, 5.1, 3.6, 21.5, 71551, 2),
       (8, 'Near North Side', 1.9, 12.9, 7.0, 2.5, 22.6, 88669, 1),
       (9, 'Edison Park', 1.1, 3.3, 6.5, 7.4, 35.3, 40959, 8),
       (10, 'Norwood Park', 2.0, 5.4, 9.0, 11.5, 39.5, 32875, 21),
       (11, 'Jefferson Park', 2.7, 8.6, 12.4, 13.4, 35.5, 27751, 25),
       (12, 'Forest Glen', 1.1, 7.5, 6.8, 4.9, 40.5, 44164, 11),
       (13, 'North Park', 3.9, 13.2, 9.9, 14.4, 39.0, 26576, 33),
       (14, 'Albany Park', 11.3, 19.2, 10.0, 32.9, 32.0, 21323, 53),
       (15, 'Portage Park', 4.1, 11.6, 12.6, 19.3, 34.0, 24336, 35),
       (16, 'Irving Park', 6.3, 13.1, 10.0, 22.4, 31.6, 27249, 34),
       (17, 'Dunning', 5.2, 10.6, 10.0, 16.2, 33.6, 26282, 28),
       (18, 'Montclaire', 8.1, 15.3, 13.8, 23.5, 38.6, 22014, 50),
       (19, 'Belmont Cragin', 10.8, 18.7, 14.6, 37.3, 37.3, 15461, 70),
       (20, 'Hermosa', 6.9, 20.5, 13.1, 41.6, 36.4, 15089, 71),
       (21, 'Avondale', 6.0, 15.3, 9.2, 24.7, 31.0, 20039, 42),
       (22, 'Logan Square', 3.2, 16.8, 8.2, 14.8, 26.2, 31908, 23),
       (23, 'Humboldt park', 14.8, 33.9, 17.3, 35.4, 38.0, 13781, 85),
       (24, 'West Town', 2.3, 14.7, 6.6, 12.9, 21.7, 43198, 10),
       (25, 'Austin', 6.3, 28.6, 22.6, 24.4, 37.9, 15957, 73),
       (26, 'West Garfield Park', 9.4, 41.7, 25.8, 24.5, 43.6, 10934, 92),
       (27, 'East Garfield Park', 8.2, 42.4, 19.6, 21.3, 43.2, 12961, 83),
       (28, 'Near West Side', 3.8, 20.6, 10.7, 9.6, 22.2, 44689, 15),
       (29, 'North Lawndale', 7.4, 43.1, 21.2, 27.6, 42.7, 12034, 87),
       (30, 'South Lawndale', 15.2, 30.7, 15.8, 54.8, 33.8, 10402, 96),
       (31, 'Lower West Side', 9.6, 25.8, 15.8, 40.7, 32.6, 16444, 76),
       (32, 'Loop', 1.5, 14.7, 5.7, 3.1, 13.5, 65526, 3),
       (33, 'Near South Side', 1.3, 13.8, 4.9, 7.4, 21.8, 59077, 7),
       (34, 'Armour Square', 5.7, 40.1, 16.7, 34.5, 38.3, 16148, 82),
       (35, 'Douglas', 1.8, 29.6, 18.2, 14.3, 30.7, 23791, 47),
       (36, 'Oakland', 1.3, 39.7, 28.7, 18.4, 40.4, 19252, 78),
       (37, 'Fuller Park', 3.2, 51.2, 33.9, 26.6, 44.9, 10432, 97),
       (38, 'Grand Boulevard', 3.3, 29.3, 24.3, 15.9, 39.5, 23472, 57),
       (39, 'Kenwood', 2.4, 21.7, 15.7, 11.3, 35.4, 35911, 26),
       (40, 'Washington Park', 5.6, 42.1, 28.6, 25.4, 42.8, 13785, 88),
       (41, 'Hyde Park', 1.5, 18.4, 8.4, 4.3, 26.2, 39056, 14),
       (42, 'Woodlawn', 2.9, 30.7, 23.4, 16.5, 36.1, 18672, 58),
       (43, 'South Shore', 2.8, 31.1, 20.0, 14.0, 35.7, 19398, 55),
       (44, 'Chatham', 3.3, 27.8, 24.0, 14.5, 40.3, 18881, 60),
       (45, 'Avalon Park', 1.4, 17.2, 21.1, 10.6, 39.3, 24454, 41),
       (46, 'South Chicago', 4.7, 29.8, 19.7, 26.6, 41.1, 16579, 75),
       (47, 'Burnside', 6.8, 33.0, 18.6, 19.3, 42.7, 12515, 79),
       (48, 'Calumet Heights', 2.1, 11.5, 20.0, 11.0, 44.0, 28887, 38),
       (49, 'Roseland', 2.5, 19.8, 20.3, 16.9, 41.2, 17949, 52),
       (50, 'Pullman', 1.5, 21.6, 22.8, 13.1, 38.6, 20588, 51),
       (51, 'South Deering', 4.0, 29.2, 16.3, 21.0, 39.5, 14685, 65),
       (52, 'East Side', 6.8, 19.2, 12.1, 31.9, 42.8, 17104, 64),
       (53, 'West Pullman', 3.3, 25.9, 19.4, 20.5, 42.1, 16563, 62),
       (54, 'Riverdale', 5.8, 56.5, 34.6, 27.5, 51.5, 8201, 98),
       (55, 'Hegewisch', 3.3, 17.1, 9.6, 19.2, 42.9, 22677, 44),
       (56, 'Garfield Ridge', 2.6, 8.8, 11.3, 19.3, 38.1, 26353, 32),
       (57, 'Archer Heights', 8.5, 14.1, 16.5, 35.9, 39.2, 16134, 67),
       (58, 'Brighton Park', 14.4, 23.6, 13.9, 45.1, 39.3, 13089, 84),
       (59, 'McKinley Park', 7.2, 18.7, 13.4, 32.9, 35.6, 16954, 61),
       (60, 'Bridgeport', 4.5, 18.9, 13.7, 22.2, 31.3, 22694, 43),
       (61, 'New City', 11.9, 29.0, 23.0, 41.5, 38.9, 12765, 91),
       (62, 'West Elsdon', 11.1, 15.6, 16.7, 37.0, 37.7, 15754, 69),
       (63, 'Gage Park', 15.8, 23.4, 18.2, 51.5, 38.8, 12171, 93),
       (64, 'Clearing', 2.7, 8.9, 9.5, 18.8, 37.6, 25113, 29),
       (65, 'West Lawn', 5.8, 14.9, 9.6, 33.6, 39.6, 16907, 56),
       (66, 'Chicago Lawn', 7.6, 27.9, 17.1, 31.2, 40.6, 13231, 80),
       (67, 'West Englewood', 4.8, 34.4, 35.9, 26.3, 40.7, 11317, 89),
       (68, 'Englewood', 3.8, 46.6, 28.0, 28.5, 42.5, 11888, 94),
       (69, 'Greater Grand Crossing', 3.6, 29.6, 23.0, 16.5, 41.0, 17285, 66),
       (70, 'Ashburn', 4.0, 10.4, 11.7, 17.7, 36.9, 23482, 37),
       (71, 'Auburn Gresham', 4.0, 27.6, 28.3, 18.5, 41.9, 15528, 74),
       (72, 'Beverly', 0.9, 5.1, 8.0, 3.7, 40.5, 39523, 12),
       (73, 'Washington Height', 1.1, 16.9, 20.8, 13.7, 42.6, 19713, 48),
       (74, 'Mount Greenwood', 1.0, 3.4, 8.7, 4.3, 36.8, 34381, 16),
       (75, 'Morgan Park', 0.8, 13.2, 15.0, 10.8, 40.3, 27149, 30),
       (76, 'O Hare', 3.6, 15.4, 7.1, 10.9, 30.3, 25828, 24),
       (77, 'Edgewater', 4.1, 18.2, 9.2, 9.7, 23.8, 33385, 19);

 * ibm_db_sa://bvr08213:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
77 rows affected.


[]

In [23]:
%%sql
INSERT INTO BVR08213.CRIME_DATA (
                        ID, 
                        CASE_NUMBER, 
                        DATE, 
                        BLOCK, 
                        IUCR, 
                        PRIMARY_TYPE, 
                        DESCRIPTION, 
                        LOCATION_DESCRIPTION, 
                        ARREST, 
                        DOMESTIC, 
                        BEAT, 
                        DISTRICT, 
                        WARD, 
                        COMMUNITY_AREA_NUMBER, 
                        FBICODE, 
                        X_COORDINATE, 
                        Y_COORDINATE, 
                        YEAR, 
                        LATITUDE, 
                        LONGITUDE, 
                        LOCATION) 
VALUES 
(3512276, 'HK587712', '2004-08-28', '047XX S KEDZIE AVE', '890', 'THEFT', 'FROM BUILDING', 'SMALL RETAIL STORE', False, False, 911, 9, 14.0, 58.0, '6', 1155838.0, 1873050.0, 2004, 41.807440500000006, -87.70395585, '(41.8074405, -87.703955849)') ,
(3406613, 'HK456306', '2004-06-26', '009XX N CENTRAL PARK AVE', '820', 'THEFT', '$500 AND UNDER', 'OTHER', False, False, 1112, 11, 27.0, 23.0, '6', 1152206.0, 1906127.0, 2004, 41.89827996, -87.71640551, '(41.898279962, -87.716405505)') ,
(8002131, 'HT233595', '2011-04-04', '043XX S WABASH AVE', '820', 'THEFT', '$500 AND UNDER', 'NURSING HOME/RETIREMENT HOME', False, False, 221, 2, 3.0, 38.0, '6', 1177436.0, 1876313.0, 2011, 41.81593313, -87.62464213, '(41.815933131, -87.624642127)') ,
(7903289, 'HT133522', '2010-12-30', '083XX S KINGSTON AVE', '840', 'THEFT', 'FINANCIAL ID THEFT: OVER $300', 'RESIDENCE', False, False, 423, 4, 7.0, 46.0, '6', 1194622.0, 1850125.0, 2010, 41.74366532, -87.56246276, '(41.743665322, -87.562462756)') ,
(10402076, 'HZ138551', '2016-02-02', '033XX W 66TH ST', '820', 'THEFT', '$500 AND UNDER', 'ALLEY', False, False, 831, 8, 15.0, 66.0, '6', 1155240.0, 1860661.0, 2016, 41.773455299999995, -87.70648047, '(41.773455295, -87.706480471)') ,
(7732712, 'HS540106', '2010-09-29', '006XX W CHICAGO AVE', '810', 'THEFT', 'OVER $500', 'PARKING LOT/GARAGE(NON.RESID.)', False, False, 1323, 12, 27.0, 24.0, '6', 1171668.0, 1905607.0, 2010, 41.89644677, -87.64493868, '(41.896446772, -87.644938678)') ,
(10769475, 'HZ534771', '2016-11-30', '050XX N KEDZIE AVE', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 1713, 17, 33.0, 14.0, '6', 1154133.0, 1933314.0, 2016, 41.97284491, -87.70860008, '(41.972844913, -87.708600079)') ,
(4494340, 'HL793243', '2005-12-16', '005XX E PERSHING RD', '860', 'THEFT', 'RETAIL THEFT', 'GROCERY FOOD STORE', True, False, 213, 2, 3.0, 38.0, '6', 1180448.0, 1879234.0, 2005, 41.82387989, -87.61350386, '(41.823879885, -87.613503857)') ,
(3778925, 'HL149610', '2005-01-28', '100XX S WASHTENAW AVE', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 2211, 22, 19.0, 72.0, '6', 1160129.0, 1838040.0, 2005, 41.71128051, -87.68917909999999, '(41.711280513, -87.689179097)') ,
(3324217, 'HK361551', '2004-05-13', '033XX W BELMONT AVE', '820', 'THEFT', '$500 AND UNDER', 'SMALL RETAIL STORE', False, False, 1733, 17, 35.0, 21.0, '6', 1153590.0, 1921084.0, 2004, 41.93929582, -87.71092344, '(41.939295821, -87.710923442)') ,
(9902691, 'HX552797', '2014-12-23', '076XX S CICERO AVE', '860', 'THEFT', 'RETAIL THEFT', 'DEPARTMENT STORE', True, False, 833, 8, 13.0, 65.0, '6', 1145727.0, 1853720.0, 2014, 41.75459296, -87.74152854, '(41.754592961, -87.741528537)') ,
(7947964, 'HT180228', '2011-02-26', '012XX S HALSTED ST', '820', 'THEFT', '$500 AND UNDER', 'BAR OR TAVERN', False, False, 1232, 12, 25.0, 28.0, '6', 1171230.0, 1894421.0, 2011, 41.86576126, -87.64687598, '(41.865761261, -87.646875977)') ,
(8568365, 'HV243612', '2012-04-15', '108XX S EMERALD AVE', '820', 'THEFT', '$500 AND UNDER', 'RESIDENTIAL YARD (FRONT/BACK)', False, False, 2233, 22, 34.0, 49.0, '6', 1173239.0, 1832994.0, 2012, 41.69715437, -87.64131567, '(41.697154372, -87.641315669)') ,
(5343947, 'HN200365', '2007-02-26', '011XX S CANAL ST', '842', 'THEFT', 'AGG: FINANCIAL ID THEFT', 'OTHER', False, False, 131, 1, 2.0, 28.0, '6', 1173307.0, 1895342.0, 2007, 41.86824272, -87.63922391, '(41.868242722, -87.639223912)') ,
(3121541, 'HK108680', '2003-01-05', '038XX W FLOURNOY ST', '890', 'THEFT', 'FROM BUILDING', 'RESIDENCE PORCH/HALLWAY', False, False, 1133, 11, 24.0, 26.0, '6', 1150808.0, 1896753.0, 2003, 41.87258418, -87.72178565, '(41.872584178, -87.72178565)') ,
(10209850, 'HY396729', '2015-08-24', '026XX N ELSTON AVE', '820', 'THEFT', '$500 AND UNDER', 'PARKING LOT/GARAGE(NON.RESID.)', False, False, 1432, 14, 1.0, 22.0, '6', 1160867.0, 1917657.0, 2015, 41.92974382, -87.68427378, '(41.929743818, -87.684273777)') ,
(6866556, 'HR271603', '2009-04-16', '089XX S STONY ISLAND AVE', '860', 'THEFT', 'RETAIL THEFT', 'OTHER', True, False, 413, 4, 8.0, 48.0, '6', 1188475.0, 1846086.0, 2009, 41.73273081, -87.58511413, '(41.732730806, -87.585114125)') ,
(9681782, 'HX332132', '2014-07-04', '079XX S STATE ST', '820', 'THEFT', '$500 AND UNDER', 'CTA PLATFORM', False, False, 623, 6, 6.0, 44.0, '6', 1177660.0, 1852569.0, 2014, 41.75077211, -87.62453842, '(41.750772111, -87.624538423)') ,
(9166453, 'HW303994', '2013-06-04', '055XX W GLADYS AVE', '810', 'THEFT', 'OVER $500', 'VACANT LOT/LAND', False, False, 1522, 15, 29.0, 25.0, '6', 1139693.0, 1897797.0, 2013, 41.87565911, -87.76256886, '(41.875659108, -87.762568861)') ,
(7762423, 'HS569246', '2010-10-17', '058XX W ARTHINGTON ST', '880', 'THEFT', 'PURSE-SNATCHING', 'SIDEWALK', False, False, 1513, 15, 29.0, 25.0, '6', 1137635.0, 1895420.0, 2010, 41.8691736, -87.77018245, '(41.869173601, -87.770182447)') ,
(10046030, 'HY234744', '2015-04-24', '008XX N MICHIGAN AVE', '860', 'THEFT', 'RETAIL THEFT', 'DEPARTMENT STORE', True, False, 1833, 18, 42.0, 8.0, '6', 1177338.0, 1906213.0, 2015, 41.89798294, -87.62409563, '(41.897982937, -87.624095634)') ,
(6233210, 'HP320451', '2008-05-06', '0000X S STATE ST', '860', 'THEFT', 'RETAIL THEFT', 'DEPARTMENT STORE', False, False, 123, 1, 42.0, 32.0, '6', 1176390.0, 1900234.0, 2008, 41.8815977, -87.62775808, '(41.881597699, -87.627758082)') ,
(5677874, 'HN484464', '2007-07-22', '0000X E 69TH ST', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 322, 3, 20.0, 69.0, '6', 1178177.0, 1859290.0, 2007, 41.76920358, -87.62244037, '(41.769203583, -87.622440369)') ,
(11230490, 'JB151690', '2018-02-14', '036XX S ARCHER AVE', '860', 'THEFT', 'RETAIL THEFT', 'GROCERY FOOD STORE', True, False, 912, 9, 11.0, 59.0, '6', 1161614.0, 1880357.0, 2018, 41.82737375, -87.68256806, '(41.827373751, -87.682568055)') ,
(7172285, 'HR582990', '2009-10-11', '060XX S HALSTED ST', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 712, 7, 16.0, 68.0, '6', 1171999.0, 1865041.0, 2009, 41.78512289, -87.64491718, '(41.785122891, -87.644917177)') ,
(7125811, 'HR535632', '2009-09-13', '044XX N ELSTON AVE', '820', 'THEFT', '$500 AND UNDER', 'ATHLETIC CLUB', False, False, 1722, 17, 39.0, 14.0, '6', 1148062.0, 1929262.0, 2009, 41.96184525, -87.73102929999999, '(41.961845248, -87.731029303)') ,
(2219750, 'HH486939', '2002-07-03', '100XX W OHARE ST', '870', 'THEFT', 'POCKET-PICKING', 'AIRPORT/AIRCRAFT', False, False, 1651, 16, 41.0, 76.0, '6', 1100658.0, 1934241.0, 2002, 41.97629041, -87.90522722, '(41.976290414, -87.905227221)') ,
(9296677, 'HW441620', '2013-09-07', '066XX S WESTERN AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 832, 8, 15.0, 66.0, '6', 1161515.0, 1860616.0, 2013, 41.773204, -87.68347884, '(41.773203996, -87.68347884)') ,
(2889910, 'HJ561433', '2003-08-13', '104XX S HALSTED ST', '810', 'THEFT', 'OVER $500', 'PARKING LOT/GARAGE(NON.RESID.)', False, False, 2233, 22, 34.0, 73.0, '6', 1172824.0, 1835651.0, 2003, 41.70445472, -87.64275716, '(41.704454724, -87.642757155)') ,
(7173960, 'HR583513', '2009-10-11', '079XX S PARNELL AVE', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 621, 6, 17.0, 44.0, '6', 1174008.0, 1852295.0, 2009, 41.750102000000005, -87.63792916, '(41.750101999, -87.637929163)') ,
(9859793, 'HX509438', '2014-11-16', '002XX S MICHIGAN AVE', '890', 'THEFT', 'FROM BUILDING', 'OTHER', False, False, 113, 1, 42.0, 32.0, '6', 1177328.0, 1899333.0, 2014, 41.87910409, -87.62434115, '(41.879104092, -87.62434115)') ,
(7868443, 'HS681158', '2010-12-21', '036XX S GILES AVE', '810', 'THEFT', 'OVER $500', 'APARTMENT', False, False, 212, 2, 2.0, 35.0, '6', 1178860.0, 1880905.0, 2010, 41.8285016, -87.61927867, '(41.828501601, -87.61927867)') ,
(3314082, 'HK350670', '2004-05-07', '031XX N BROADWAY', '860', 'THEFT', 'RETAIL THEFT', 'SMALL RETAIL STORE', False, False, 2332, 19, 44.0, 6.0, '6', 1171693.0, 1921311.0, 2004, 41.93953877, -87.64438351, '(41.939538767, -87.644383505)') ,
(1516518, 'G256545', '2001-05-04', '045XX S DAMEN AV', '820', 'THEFT', '$500 AND UNDER', 'SMALL RETAIL STORE', True, False, 914, 9, NULL, NULL, '6', 1163610.0, 1874345.0, 2001, 41.81083442, -87.67541390000001, '(41.810834416, -87.675413895)') ,
(10952376, 'JA272438', '2017-05-20', '028XX W 21ST ST', '820', 'THEFT', '$500 AND UNDER', 'SIDEWALK', False, False, 1023, 10, 12.0, 30.0, '6', 1157493.0, 1889929.0, 2017, 41.85372509, -87.69742748, '(41.853725089, -87.697427476)') ,
(2000970, 'HH201585', '2002-02-22', '024XX N MILWAUKEE AV', '810', 'THEFT', 'OVER $500', 'BAR OR TAVERN', False, False, 1414, 14, NULL, NULL, '6', 1156341.0, 1915936.0, 2002, 41.92511408, -87.70095228, '(41.925114077, -87.700952275)') ,
(1476982, 'G212935', '2001-04-14', '007XX N GREEN ST', '850', 'THEFT', 'ATTEMPT THEFT', 'MOVIE HOUSE/THEATER', False, False, 1323, 12, NULL, NULL, '6', 1170623.0, 1905487.0, 2001, 41.89614043, -87.64878026, '(41.896140428, -87.648780261)') ,
(4273693, 'HL590300', '2005-08-31', '009XX N SACRAMENTO BLVD', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 1311, 12, 26.0, 23.0, '6', 1156004.0, 1906299.0, 2005, 41.89867615, -87.70245104, '(41.898676154, -87.702451037)') ,
(4067549, 'HL409680', '2005-06-09', '069XX S SANGAMON ST', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 733, 7, 17.0, 68.0, '6', 1171170.0, 1858915.0, 2005, 41.76833059, -87.64813561, '(41.768330593, -87.648135609)') ,
(4718689, 'HM324025', '2006-05-01', '0000X W ELM ST', '860', 'THEFT', 'RETAIL THEFT', 'GROCERY FOOD STORE', True, False, 1824, 18, 42.0, 8.0, '6', 1175504.0, 1908078.0, 2006, 41.903141999999995, -87.63077557, '(41.903141995, -87.63077557)') ,
(1845995, 'G662598', '2001-11-03', '0000X N STATE ST', '820', 'THEFT', '$500 AND UNDER', 'DEPARTMENT STORE', True, False, 122, 1, NULL, NULL, '6', 1176398.0, 1900691.0, 2001, 41.88285155, -87.62771491, '(41.882851552, -87.627714909)') ,
(8678872, 'HV353560', '2012-04-15', '072XX S PAXTON AVE', '890', 'THEFT', 'FROM BUILDING', 'RESIDENCE', False, False, 333, 3, 5.0, 43.0, '6', 1192140.0, 1857319.0, 2012, 41.76346691, -87.57132336, '(41.763466911, -87.571323359)') ,
(6855661, 'HR261581', '2009-04-10', '017XX W CERMAK RD', '860', 'THEFT', 'RETAIL THEFT', 'GROCERY FOOD STORE', True, False, 1034, 10, 25.0, 31.0, '6', 1165218.0, 1889459.0, 2009, 41.85227488, -87.66908723, '(41.852274879, -87.66908723)') ,
(4385612, 'HL680775', '2005-10-14', '017XX N HERMITAGE AVE', '820', 'THEFT', '$500 AND UNDER', 'VEHICLE NON-COMMERCIAL', False, False, 1433, 14, 32.0, 24.0, '6', 1164410.0, 1911688.0, 2005, 41.91329017, -87.67142355, '(41.913290173, -87.671423547)') ,
(4929211, 'HM543491', '2006-06-29', '021XX W ROSCOE ST', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 1913, 19, 32.0, 5.0, '6', 1161308.0, 1922552.0, 2006, 41.94316683, -87.68251667, '(41.943166826, -87.682516667)') ,
(9978032, 'HY167451', '2015-02-27', '063XX S YALE AVE', '820', 'THEFT', '$500 AND UNDER', 'SIDEWALK', False, False, 722, 7, 20.0, 68.0, '6', 1175585.0, 1863128.0, 2015, 41.77979388, -87.63182661, '(41.779793875, -87.631826613)') ,
(10190267, 'HY377901', '2015-08-10', '026XX N MILWAUKEE AVE', '810', 'THEFT', 'OVER $500', 'SIDEWALK', False, False, 1412, 14, 35.0, 22.0, '6', 1154504.0, 1917426.0, 2015, 41.92923973, -87.70766234, '(41.929239732, -87.707662339)') ,
(9575333, 'HX225704', '2014-04-16', '002XX W MONROE ST', '890', 'THEFT', 'FROM BUILDING', 'RESTAURANT', False, False, 122, 1, 2.0, 32.0, '6', 1174585.0, 1899875.0, 2014, 41.88065311, -87.63439665, '(41.880653112, -87.634396649)') ,
(2688426, 'HJ308632', '2003-04-17', '017XX N AUSTIN AVE', '810', 'THEFT', 'OVER $500', 'RESIDENCE-GARAGE', False, False, 2531, 25, 29.0, 25.0, '6', 1136138.0, 1910943.0, 2003, 41.91179756, -87.77530779, '(41.911797557, -87.775307788)') ,
(8854340, 'HV528008', '2012-10-20', '029XX N LAKE SHORE DR', '890', 'THEFT', 'FROM BUILDING', 'NURSING HOME/RETIREMENT HOME', False, False, 1934, 19, 44.0, 6.0, '6', 1173772.0, 1920055.0, 2012, 41.93604616, -87.63678016, '(41.936046155, -87.636780157)') ,
(3704662, 'HK801112', '2004-12-10', '055XX S WELLS ST', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 711, 7, 3.0, 68.0, '6', 1175549.0, 1868101.0, 2004, 41.79344111, -87.63180984, '(41.793441113, -87.631809838)') ,
(7724785, 'HS532107', '2010-09-11', '065XX S ST LAWRENCE AVE', '890', 'THEFT', 'FROM BUILDING', 'APARTMENT', False, False, 321, 3, 20.0, 42.0, '6', 1181360.0, 1861718.0, 2010, 41.77579347, -87.61069831, '(41.775793468, -87.61069831)') ,
(10943516, 'JA261898', '2017-05-13', '0000X N LATROBE AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 1522, 15, 28.0, 25.0, '6', 1141381.0, 1899903.0, 2017, 41.88140727, -87.75631908, '(41.88140727, -87.756319082)') ,
(11185089, 'JA559176', '2017-12-20', '019XX S MILLARD AVE', '810', 'THEFT', 'OVER $500', 'RESIDENCE', False, False, 1014, 10, 24.0, 29.0, '6', 1152346.0, 1890287.0, 2017, 41.85481053, -87.71630954, '(41.854810528, -87.716309542)') ,
(11072808, 'JA412895', '2017-08-31', '011XX N KARLOV AVE', '810', 'THEFT', 'OVER $500', 'RESIDENTIAL YARD (FRONT/BACK)', False, False, 1111, 11, 37.0, 23.0, '6', 1148843.0, 1907272.0, 2017, 41.90148766, -87.728728, '(41.901487656, -87.728728004)') ,
(10084732, 'HY273740', '2015-05-24', '004XX N STATE ST', '820', 'THEFT', '$500 AND UNDER', 'BAR OR TAVERN', False, False, 1831, 18, 42.0, 8.0, '6', 1176302.0, 1903096.0, 2015, 41.88945317, -87.62799483, '(41.889453169, -87.627994833)') ,
(10485657, 'HZ224258', '2016-04-12', '017XX N LUNA AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 2532, 25, 37.0, 25.0, '6', 1139131.0, 1911034.0, 2016, 41.91199331, -87.76431002, '(41.911993311, -87.764310023)') ,
(2963994, 'HJ652747', '2003-09-26', '022XX W HARRISON ST', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 1211, 12, 2.0, 28.0, '6', 1161653.0, 1897333.0, 2003, 41.87395682, -87.68195253, '(41.873956822, -87.681952527)') ,
(1340847, 'G040244', '2001-01-19', '063XX N NAGLE AV', '820', 'THEFT', '$500 AND UNDER', 'GROCERY FOOD STORE', True, False, 1611, 16, NULL, NULL, '6', 1132586.0, 1941599.0, 2001, 41.99598354, -87.78763989, '(41.99598354, -87.787639887)') ,
(2771112, 'HJ415128', '2003-06-08', '037XX S WOLCOTT AVE', '810', 'THEFT', 'OVER $500', 'VEHICLE NON-COMMERCIAL', False, False, 922, 9, 11.0, 59.0, '6', 1164300.0, 1879991.0, 2003, 41.82631315, -87.67272382, '(41.826313153, -87.67272382)') ,
(8861185, 'HV534721', '2012-10-25', '017XX W FULLERTON AVE', '820', 'THEFT', '$500 AND UNDER', 'DEPARTMENT STORE', False, False, 1811, 18, 32.0, 7.0, '6', 1164140.0, 1915989.0, 2012, 41.92509811, -87.67229367, '(41.925098106, -87.672293669)') ,
(11143628, 'JA503725', '2017-11-08', '058XX S ASHLAND AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', True, False, 713, 7, 15.0, 67.0, '6', 1166676.0, 1865801.0, 2017, 41.78732372, -87.66441195, '(41.787323718, -87.664411954)') ,
(5214424, 'HM803061', '2006-12-31', '029XX N ASHLAND AVE', '860', 'THEFT', 'RETAIL THEFT', 'GROCERY FOOD STORE', True, False, 1931, 19, 32.0, 6.0, '6', 1165142.0, 1919763.0, 2006, 41.93543292, -87.66850434, '(41.935432921, -87.668504341)') ,
(5931425, 'HN731220', '2007-11-21', '003XX S WACKER DR', '890', 'THEFT', 'FROM BUILDING', 'COMMERCIAL / BUSINESS OFFICE', False, False, 111, 1, 2.0, 32.0, '6', 1173916.0, 1898816.0, 2007, 41.87776207, -87.63688469, '(41.877762073, -87.63688469)') ,
(6622100, 'HP691879', '2008-11-17', '046XX S DREXEL BLVD', '820', 'THEFT', '$500 AND UNDER', 'APARTMENT', False, False, 2123, 2, 4.0, 39.0, '6', 1183100.0, 1874490.0, 2008, 41.81080062, -87.60392245, '(41.810800619, -87.603922445)') ,
(8863261, 'HV537004', '2012-10-25', '054XX N ASHLAND AVE', '810', 'THEFT', 'OVER $500', 'OTHER', False, True, 2012, 20, 40.0, 77.0, '6', 1164674.0, 1936421.0, 2012, 41.98115315, -87.6697502, '(41.981153152, -87.669750197)') ,
(6731976, 'HR149903', '2009-02-02', '010XX W BELMONT AVE', '890', 'THEFT', 'FROM BUILDING', 'TAVERN/LIQUOR STORE', False, False, 1924, 19, 44.0, 6.0, '6', 1168964.0, 1921423.0, 2009, 41.93990585, -87.65441005, '(41.939905846, -87.654410045)') ,
(9664739, 'HX314361', '2014-06-21', '018XX W 22ND PL', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 1034, 10, 25.0, 31.0, '6', 1164340.0, 1889153.0, 2014, 41.85145378, -87.67231838, '(41.851453777, -87.672318382)') ,
(8670090, 'HV345270', '2012-06-20', '013XX W 51ST ST', '890', 'THEFT', 'FROM BUILDING', 'CONSTRUCTION SITE', False, False, 933, 9, 16.0, 61.0, '6', 1168063.0, 1870939.0, 2012, 41.80139327, -87.65917869, '(41.801393265, -87.659178692)') ,
(7505841, 'HS308747', '2010-05-15', '004XX E 95TH ST', '890', 'THEFT', 'FROM BUILDING', 'GAS STATION', False, False, 633, 6, 9.0, 49.0, '6', 1180667.0, 1842071.0, 2010, 41.72189586, -87.61384102, '(41.721895856, -87.613841024)') ,
(3637126, 'HK728309', '2004-11-03', '008XX N KEDZIE AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 1121, 11, 27.0, 23.0, '6', 1154891.0, 1905271.0, 2004, 41.89587761, -87.70656663, '(41.895877613, -87.706566627)') ,
(4153041, 'HL005270', '2005-07-12', '008XX N CICERO AVE', '820', 'THEFT', '$500 AND UNDER', 'GAS STATION', False, False, 1111, 11, 37.0, 25.0, '6', 1144207.0, 1904963.0, 2005, 41.8952399, -87.74581479, '(41.895239896, -87.745814792)') ,
(10102551, 'HY291329', '2015-06-06', '028XX W DEVON AVE', '860', 'THEFT', 'RETAIL THEFT', 'DRUG STORE', False, False, 2413, 24, 50.0, 2.0, '6', 1156278.0, 1942331.0, 2015, 41.9975448, -87.70046744, '(41.997544797, -87.700467444)') ,
(2988938, 'HJ678713', '2003-10-07', '050XX S KEDZIE AVE', '810', 'THEFT', 'OVER $500', 'PARKING LOT/GARAGE(NON.RESID.)', False, False, 821, 8, 14.0, 63.0, '6', 1155893.0, 1871118.0, 2003, 41.80213772, -87.70380602, '(41.802137724, -87.703806024)') ,
(10225755, 'HY412959', '2015-09-05', '065XX S HOYNE AVE', '820', 'THEFT', '$500 AND UNDER', 'RESIDENCE', False, False, 726, 7, 15.0, 67.0, '6', 1163490.0, 1861218.0, 2015, 41.77481479, -87.67622207, '(41.774814785, -87.676222073)') ,
(9499852, 'HX155070', '2014-02-17', '002XX E MARQUETTE RD', '810', 'THEFT', 'OVER $500', 'PARKING LOT/GARAGE(NON.RESID.)', False, False, 322, 3, 20.0, 69.0, '6', 1178939.0, 1860641.0, 2014, 41.77289355, -87.61960616, '(41.772893552, -87.61960616)') ,
(8727329, 'HV402967', '2012-07-27', '062XX S MC VICKER AVE', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 812, 8, 23.0, 64.0, '6', 1137145.0, 1862622.0, 2012, 41.77917942, -87.77276701, '(41.779179421, -87.772767014)') ,
(9682442, 'HX332979', '2014-07-05', '042XX W 55TH ST', '820', 'THEFT', '$500 AND UNDER', 'APARTMENT', False, True, 813, 8, 13.0, 62.0, '6', 1148818.0, 1867790.0, 2014, 41.79314445, -87.72983895, '(41.793144448, -87.729838952)') ,
(3013050, 'HJ710634', '2003-10-22', '070XX S WESTERN AVE', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 832, 8, 17.0, 66.0, '6', 1161593.0, 1857866.0, 2003, 41.76565599, -87.6832691, '(41.765655988, -87.683269102)') ,
(9509922, 'HX165034', '2014-02-25', '109XX S CHURCH ST', '820', 'THEFT', '$500 AND UNDER', 'RESIDENCE', False, False, 2212, 22, 19.0, 75.0, '6', 1167242.0, 1831951.0, 2014, 41.69442233, -87.66330302, '(41.694422332, -87.663303015)') ,
(6980726, 'HR385117', '2009-06-20', '076XX S CICERO AVE', '860', 'THEFT', 'RETAIL THEFT', 'SMALL RETAIL STORE', True, False, 833, 8, 13.0, 65.0, '6', 1145727.0, 1853720.0, 2009, 41.75459296, -87.74152854, '(41.754592961, -87.741528537)') ,
(7428379, 'HS230912', '2010-03-29', '113XX S NORMAL AVE', '890', 'THEFT', 'FROM BUILDING', 'RESIDENCE PORCH/HALLWAY', False, False, 2233, 22, 34.0, 49.0, '6', 1174990.0, 1829768.0, 2010, 41.68826295, -87.63500028, '(41.688262954, -87.635000281)') ,
(3808095, 'HL177398', '2005-02-12', '026XX N ELSTON AVE', '860', 'THEFT', 'RETAIL THEFT', 'DEPARTMENT STORE', False, False, 1432, 14, 1.0, 22.0, '6', 1160867.0, 1917657.0, 2005, 41.92974382, -87.68427378, '(41.929743818, -87.684273777)') ,
(2405698, 'HH702524', '2002-10-09', '025XX W FLOURNOY ST', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 1135, 11, 2.0, 28.0, '6', 1159336.0, 1896944.0, 2002, 41.8729373, -87.69047017, '(41.872937304, -87.690470166)') ,
(11048967, 'JA384163', '2017-08-02', '028XX N Maplewood Ave', '820', 'THEFT', '$500 AND UNDER', 'RESIDENCE', False, False, 1411, 14, 1.0, 21.0, '6', 1158865.0, 1918857.0, 2017, 41.93307805, -87.69159758, '(41.933078052, -87.691597583)') ,
(2975480, 'HJ672058', '2003-10-01', '016XX N OAKLEY AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 1434, 14, 1.0, 24.0, '6', 1160770.0, 1911006.0, 2003, 41.91149502, -87.68481506, '(41.911495021, -87.684815055)') ,
(8134254, 'HT368471', '2011-06-26', '069XX N WOLCOTT AVE', '890', 'THEFT', 'FROM BUILDING', 'RESIDENCE PORCH/HALLWAY', False, False, 2424, 24, 49.0, 1.0, '6', 1162360.0, 1945827.0, 2011, 42.00701239, -87.67799561, '(42.007012388, -87.67799561)') ,
(10012707, 'HY202462', '2015-03-28', '056XX W BYRON ST', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 1633, 16, 38.0, 15.0, '6', 1138080.0, 1925360.0, 2015, 41.95132446, -87.76782401, '(41.95132446, -87.767824012)') ,
(9704121, 'HX354359', '2014-07-21', '032XX S KEDZIE AVE', '820', 'THEFT', '$500 AND UNDER', 'CTA BUS', False, False, 1033, 10, 12.0, 30.0, '6', 1155548.0, 1883013.0, 2014, 41.83478608, -87.70475217, '(41.834786077, -87.704752166)') ,
(10409937, 'HZ146741', '2016-02-09', '006XX W 87TH ST', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 622, 6, 21.0, 71.0, '6', 1173383.0, 1847259.0, 2016, 41.7362964, -87.64036804, '(41.736296397, -87.64036804)') ,
(1612358, 'G376790', '2001-06-28', '0000X S STATE ST', '820', 'THEFT', '$500 AND UNDER', 'DEPARTMENT STORE', True, False, 123, 1, NULL, NULL, '6', 1176418.0, 1900379.0, 2001, 41.88199496, -87.62765089, '(41.881994955, -87.62765089)') ,
(9612532, 'HX262353', '2014-05-14', '084XX W BRYN MAWR AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 1614, 16, 41.0, 76.0, '6', 1118919.0, 1936092.0, 2014, 41.98109971, -87.83803283, '(41.981099706, -87.838032833)') ,
(6231712, 'HP317516', '2008-05-05', '061XX S NARRAGANSETT AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 812, 8, 23.0, 64.0, '6', 1134794.0, 1863173.0, 2008, 41.78073320000001, -87.78137324, '(41.7807332, -87.781373244)') ,
(10585295, 'HZ335434', '2016-07-03', '040XX N KEDVALE AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 1722, 17, 39.0, 16.0, '6', 1148122.0, 1926708.0, 2016, 41.95483572, -87.73087472, '(41.95483572, -87.730874723)') ,
(7865597, 'HS678732', '2010-12-27', '063XX S HALSTED ST', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 723, 7, 20.0, 68.0, '6', 1172055.0, 1862918.0, 2010, 41.77929591, -87.64477419, '(41.779295906, -87.644774188)') ,
(4029579, 'HL382289', '2005-05-27', '001XX E GRAND AVE', '810', 'THEFT', 'OVER $500', 'STREET', False, False, 1834, 18, 42.0, 8.0, '6', 1177533.0, 1903942.0, 2005, 41.89174678, -87.62344844, '(41.891746779, -87.623448439)') ,
(4778195, 'HM392612', '2006-06-03', '057XX N MAGNOLIA AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 2013, 20, 48.0, 77.0, '6', 1166924.0, 1938218.0, 2006, 41.98603604, -87.66142359999999, '(41.986036039, -87.661423603)') ,
(2522757, 'HH869886', '2002-12-31', '048XX W NORTH AVE', '820', 'THEFT', '$500 AND UNDER', 'GROCERY FOOD STORE', True, False, 2533, 25, 37.0, 25.0, '6', 1143844.0, 1910199.0, 2002, 41.90961488, -87.74701658, '(41.909614875, -87.747016581)') ,
(7054414, 'HR460350', '2009-08-01', '062XX S PAULINA ST', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 714, 7, 15.0, 67.0, '6', 1166078.0, 1863385.0, 2009, 41.78070666, -87.66667324, '(41.780706656, -87.666673244)') ,
(4533063, 'HM117889', '2006-01-11', '010XX E 111TH ST', '810', 'THEFT', 'OVER $500', 'OTHER', False, False, 513, 5, 9.0, 50.0, '6', 1184891.0, 1831392.0, 2006, 41.69249346, -87.59870306, '(41.692493455, -87.598703055)') ,
(7166946, 'HR576816', '2009-10-07', '045XX N KEDZIE AVE', '810', 'THEFT', 'OVER $500', 'PARKING LOT/GARAGE(NON.RESID.)', False, False, 1724, 17, 33.0, 14.0, '6', 1154233.0, 1929942.0, 2009, 41.96358994, -87.70832281, '(41.963589941, -87.708322812)') ,
(10619033, 'HZ372381', '2016-07-25', '023XX W JACKSON BLVD', '820', 'THEFT', '$500 AND UNDER', 'VEHICLE NON-COMMERCIAL', False, False, 1225, 12, 2.0, 28.0, '6', 1160918.0, 1898643.0, 2016, 41.87756686, -87.68461475, '(41.877566857, -87.684614749)') ,
(10787573, 'HZ556726', '2016-12-19', '005XX W 95TH ST', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 2223, 22, 21.0, 73.0, '6', 1174212.0, 1841887.0, 2016, 41.72153655, -87.63748998, '(41.721536545, -87.637489981)') ,
(5613705, 'HN421595', '2007-06-13', '058XX S ARCHER AVE', '890', 'THEFT', 'FROM BUILDING', 'OTHER', False, False, 814, 8, 23.0, 56.0, '6', 1140413.0, 1868766.0, 2007, 41.79598053, -87.76063592, '(41.795980534, -87.760635922)') ,
(3398799, 'HK454598', '2004-06-25', '017XX N NATCHEZ AVE', '820', 'THEFT', '$500 AND UNDER', 'VEHICLE NON-COMMERCIAL', False, False, 2513, 25, 36.0, 25.0, '6', 1132589.0, 1910893.0, 2004, 41.91172297, -87.78834715, '(41.911722972, -87.788347149)') ,
(2355174, 'HH657809', '2002-09-18', '102XX S HOXIE AVE', '820', 'THEFT', '$500 AND UNDER', 'STREET', False, False, 431, 4, 10.0, 51.0, '6', 1195158.0, 1837678.0, 2002, 41.70949645, -87.56090809, '(41.709496448, -87.560908092)') ,
(7646435, 'HS451246', '2010-08-07', '023XX W ROSCOE ST', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', True, False, 1913, 19, 32.0, 5.0, '08B', 1160352.0, 1922529.0, 2010, 41.94312358, -87.68603108, '(41.943123577, -87.686031082)') ,
(2088541, 'HH316735', '2002-04-19', '031XX S ASHLAND AV', '460', 'BATTERY', 'SIMPLE', 'GROCERY FOOD STORE', False, False, 922, 9, NULL, NULL, '08B', 1166183.0, 1883676.0, 2002, 41.83638523, -87.66571041, '(41.836385231, -87.66571041)') ,
(1778867, 'G597730', '2001-10-05', '024XX W HADDON AV', '460', 'BATTERY', 'SIMPLE', 'STREET', False, True, 1312, 12, NULL, NULL, '08B', 1159562.0, 1907600.0, 2001, 41.90217367, -87.6893468, '(41.902173665, -87.689346803)') ,
(1887291, 'G736811', '2001-12-09', '035XX W DICKENS AV', '460', 'BATTERY', 'SIMPLE', 'RESIDENCE', False, True, 1413, 14, NULL, NULL, '08B', 1152633.0, 1913771.0, 2001, 41.91924736, -87.71463465, '(41.919247363, -87.714634645)') ,
(8418606, 'HT651635', '2011-12-29', '056XX W ROOSEVELT RD', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'PARKING LOT/GARAGE(NON.RESID.)', False, True, 1513, 15, 29.0, 25.0, '08B', 1139135.0, 1894124.0, 2011, 41.86559007, -87.764707, '(41.865590065, -87.764706995)') ,
(10679671, 'HZ432273', '2016-09-13', '035XX N ELSTON AVE', '460', 'BATTERY', 'SIMPLE', 'DRUG STORE', False, False, 1733, 17, 35.0, 21.0, '08B', 1154298.0, 1923613.0, 2016, 41.94622144, -87.70825356, '(41.946221444, -87.708253555)') ,
(1719296, 'G517474', '2001-08-29', '017XX W 59 ST', '460', 'BATTERY', 'SIMPLE', 'RESIDENCE', False, True, 714, 7, NULL, NULL, '08B', 1165757.0, 1865528.0, 2001, 41.78659414, -87.66778928, '(41.786594143, -87.66778928)') ,
(6673746, 'HP746347', '2008-12-22', '070XX N CALIFORNIA AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'VEHICLE NON-COMMERCIAL', False, False, 2411, 24, 50.0, 2.0, '08B', 1156438.0, 1946416.0, 2008, 42.00875095, -87.69976766, '(42.008750949, -87.699767662)') ,
(1353618, 'G056330', '2001-01-27', '078XX S SAWYER AV', '460', 'BATTERY', 'SIMPLE', 'RESIDENCE PORCH/HALLWAY', True, True, 835, 8, NULL, NULL, '08B', 1156032.0, 1852572.0, 2001, 41.75124194, -87.70379416, '(41.751241937, -87.703794164)') ,
(5979106, 'HN772519', '2007-12-22', '081XX S GREEN ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'SIDEWALK', False, True, 613, 6, 21.0, 71.0, '08B', 1172054.0, 1850881.0, 2007, 41.74626491, -87.64513091, '(41.746264912, -87.645130909)') ,
(10562980, 'HZ309936', '2016-06-16', '048XX S DREXEL BLVD', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', False, False, 222, 2, 4.0, 39.0, '08B', 1183133.0, 1873130.0, 2016, 41.80706791, -87.60384375, '(41.807067906, -87.603843747)') ,
(10179921, 'HY368284', '2015-08-04', '024XX S TROY ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', True, True, 1033, 10, 12.0, 30.0, '08B', 1155747.0, 1887668.0, 2015, 41.84755595, -87.7038968, '(41.847555951, -87.703896798)') ,
(4006321, 'HL353697', '2005-05-04', '077XX S BURNHAM AVE', '460', 'BATTERY', 'SIMPLE', 'SCHOOL, PUBLIC, GROUNDS', False, False, 421, 4, 7.0, 43.0, '08B', 1196066.0, 1854156.0, 2005, 41.75469107, -87.55703869, '(41.754691074, -87.557038686)') ,
(8142216, 'HT375934', '2011-07-01', '058XX S SANGAMON ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'SIDEWALK', False, True, 712, 7, 16.0, 68.0, '08B', 1170970.0, 1866261.0, 2011, 41.78849326, -87.64865431, '(41.788493256, -87.648654314)') ,
(10879266, 'JA188596', '2017-03-15', '013XX W 49TH PL', '454', 'BATTERY', 'AGG PO HANDS NO/MIN INJURY', 'APARTMENT', True, False, 933, 9, 16.0, 61.0, '08B', 1168264.0, 1871930.0, 2017, 41.80410835, -87.65841301, '(41.804108351, -87.658413013)') ,
(4430638, 'HL725506', '2005-11-09', '048XX N FRANCISCO AVE', '484', 'BATTERY', 'PRO EMP HANDS NO/MIN INJURY', 'SCHOOL, PUBLIC, BUILDING', True, False, 2031, 20, 40.0, 4.0, '08B', 1156348.0, 1932071.0, 2005, 41.96938944, -87.70048881, '(41.96938944, -87.700488807)') ,
(8718715, 'HV395700', '2012-07-22', '034XX W CERMAK RD', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 1024, 10, 24.0, 29.0, '08B', 1153675.0, 1889243.0, 2012, 41.85191936, -87.71145926, '(41.85191936, -87.711459258)') ,
(2391105, 'HH696901', '2002-10-06', '027XX E 75TH ST', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 334, 3, 7.0, 43.0, '08B', 1195735.0, 1856117.0, 2002, 41.76008039, -87.55818693, '(41.760080391, -87.558186925)') ,
(8155450, 'HT390647', '2011-07-10', '033XX W MAYPOLE AVE', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 1123, 11, 28.0, 27.0, '08B', 1154131.0, 1900784.0, 2011, 41.88358005, -87.70947777, '(41.883580046, -87.709477772)') ,
(3710918, 'HK815196', '2004-12-18', '023XX W OHIO ST', '420', 'BATTERY', 'AGGRAVATED:KNIFE/CUTTING INSTR', 'APARTMENT', False, False, 1313, 12, 26.0, 24.0, '04B', 1160505.0, 1903943.0, 2004, 41.89211908, -87.68598438, '(41.892119079, -87.685984378)') ,
(11133668, 'JA490957', '2017-10-29', '107XX S LAFAYETTE AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENTIAL YARD (FRONT/BACK)', False, True, 513, 5, 34.0, 49.0, '08B', 1177830.0, 1833729.0, 2017, 41.69906886, -87.62448396, '(41.699068864, -87.624483962)') ,
(3116759, 'HK103473', '2004-01-02', '028XX N MILWAUKEE AVE', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 1412, 14, 35.0, 21.0, '08B', 1152784.0, 1918805.0, 2004, 41.93305809, -87.71394625, '(41.933058087, -87.713946247)') ,
(4946261, 'HM556695', '2006-08-23', '009XX N CENTRAL PARK AVE', '430', 'BATTERY', 'AGGRAVATED: OTHER DANG WEAPON', 'SIDEWALK', False, False, 1112, 11, 27.0, 23.0, '04B', 1152207.0, 1906086.0, 2006, 41.89816743, -87.71640292, '(41.898167434, -87.716402915)') ,
(2281487, 'HH558381', '2002-08-04', '051XX W MADISON ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'SIDEWALK', False, True, 1533, 15, 28.0, 25.0, '08B', 1142023.0, 1899535.0, 2002, 41.88038556, -87.75397078, '(41.880385563, -87.753970777)') ,
(3434215, 'HK498590', '2004-07-15', '011XX N LA SALLE DR', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', True, False, 1824, 18, 43.0, 8.0, '08B', 1174938.0, 1907711.0, 2004, 41.90214763, -87.63286559, '(41.90214763, -87.632865587)') ,
(9806771, 'HX455839', '2014-10-05', '096XX S WOODLAWN AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 511, 5, 8.0, 50.0, '08B', 1185895.0, 1841338.0, 2014, 41.71976292, -87.59471496, '(41.719762924, -87.594714959)') ,
(7287599, 'HR703960', '2009-12-24', '047XX W SUPERIOR ST', '420', 'BATTERY', 'AGGRAVATED:KNIFE/CUTTING INSTR', 'RESIDENCE', False, False, 1111, 11, 28.0, 25.0, '04B', 1144674.0, 1904545.0, 2009, 41.89408407, -87.74411014, '(41.894084068, -87.744110141)') ,
(4349267, 'HL646682', '2005-10-01', '009XX W 103RD ST', '460', 'BATTERY', 'SIMPLE', 'CHURCH/SYNAGOGUE/PLACE OF WORSHIP', False, False, 2232, 22, 34.0, 73.0, '08B', 1171680.0, 1836526.0, 2005, 41.70688095, -87.64692076, '(41.706880954, -87.646920763)') ,
(8771351, 'HV445580', '2012-08-24', '034XX N HALSTED ST', '460', 'BATTERY', 'SIMPLE', 'RESTAURANT', True, False, 1925, 19, 44.0, 6.0, '08B', 1170309.0, 1923242.0, 2012, 41.94486792, -87.64941345, '(41.944867918, -87.649413445)') ,
(1850651, 'G688026', '2001-11-15', '069XX S STEWART AV', '430', 'BATTERY', 'AGGRAVATED: OTHER DANG WEAPON', 'SIDEWALK', False, False, 732, 7, NULL, NULL, '04B', 1174777.0, 1859152.0, 2001, 41.76890134, -87.63490722, '(41.768901335, -87.634907215)') ,
(2811745, 'HJ466766', '2003-07-01', '057XX S NORMAL BLVD', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 711, 7, 3.0, 68.0, '08B', 1173936.0, 1867042.0, 2003, 41.79057107, -87.63775596, '(41.790571073, -87.637755959)') ,
(4328714, 'HL632721', '2005-09-23', '020XX W 18TH ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', False, True, 1223, 12, 25.0, 31.0, '08B', 1163120.0, 1891403.0, 2005, 41.85765368, -87.67673292, '(41.85765368, -87.676732923)') ,
(6514506, 'HP579910', '2008-09-18', '067XX S EBERHART AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'STREET', False, True, 321, 3, 20.0, 42.0, '08B', 1180737.0, 1860533.0, 2008, 41.77255605, -87.61301853, '(41.772556054, -87.613018533)') ,
(4424181, 'HL718742', '2005-11-06', '007XX N LAWNDALE AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE PORCH/HALLWAY', False, True, 1112, 11, 27.0, 23.0, '08B', 1151588.0, 1904632.0, 2005, 41.89418971, -87.71871472, '(41.894189705, -87.718714722)') ,
(6644648, 'HP714065', '2008-12-03', '014XX N MENARD AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 2531, 25, 29.0, 25.0, '08B', 1137418.0, 1908965.0, 2008, 41.90634674, -87.77065305, '(41.906346741, -87.770653047)') ,
(6644618, 'HP716225', '2008-12-04', '030XX S DR MARTIN LUTHER KING JR DR', '460', 'BATTERY', 'SIMPLE', 'SCHOOL, PUBLIC, BUILDING', False, False, 2112, 1, 2.0, 35.0, '08B', 1179306.0, 1885032.0, 2008, 41.83981621, -87.61751617, '(41.839816207, -87.617516172)') ,
(9337353, 'HW481295', '2013-10-05', '016XX S ST LOUIS AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', True, True, 1021, 10, 24.0, 29.0, '08B', 1153305.0, 1891618.0, 2013, 41.85844398, -87.71275427, '(41.858443981, -87.712754265)') ,
(7759940, 'HS568476', '2010-10-17', '033XX W 28TH ST', '430', 'BATTERY', 'AGGRAVATED: OTHER DANG WEAPON', 'STREET', False, False, 1032, 10, 22.0, 30.0, '04B', 1154736.0, 1885210.0, 2010, 41.84083118, -87.70767292, '(41.840831181, -87.707672922)') ,
(11277173, 'JB213238', '2018-04-06', '022XX N PARKSIDE AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 2515, 25, 37.0, 19.0, '08B', 1138350.0, 1914356.0, 2018, 41.92112344, -87.76709866, '(41.92112344, -87.767098657)') ,
(7670858, 'HS475732', '2010-08-22', '047XX W ALTGELD ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, False, 2521, 25, 31.0, 19.0, '08B', 1144359.0, 1916264.0, 2010, 41.92624818, -87.74497181, '(41.926248183, -87.744971814)') ,
(7787295, 'HS595785', '2010-11-01', '079XX S ESSEX AVE', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 422, 4, 7.0, 46.0, '08B', 1194232.0, 1852910.0, 2010, 41.75131717, -87.56380046, '(41.751317167, -87.563800456)') ,
(7002482, 'HR408597', '2009-07-03', '030XX W JACKSON BLVD', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 1124, 11, 28.0, 27.0, '08B', 1156002.0, 1898529.0, 2009, 41.87735457, -87.70266813, '(41.877354573, -87.702668131)') ,
(8315579, 'HT542713', '2011-10-14', '034XX N PAULINA ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESTAURANT', False, True, 1924, 19, 32.0, 6.0, '08B', 1164380.0, 1922814.0, 2011, 41.94382121, -87.67121812, '(41.94382121, -87.671218121)') ,
(3867425, 'HL240985', '2005-03-18', '025XX N HALSTED ST', '460', 'BATTERY', 'SIMPLE', 'STREET', True, False, 1933, 19, 43.0, 7.0, '08B', 1170513.0, 1917018.0, 2005, 41.92778453, -87.64884628, '(41.927784528, -87.648846284)') ,
(2645724, 'HJ254782', '2003-03-23', '021XX N MAJOR AVE', '430', 'BATTERY', 'AGGRAVATED: OTHER DANG WEAPON', 'STREET', False, False, 2515, 25, 29.0, 19.0, '04B', 1138038.0, 1913774.0, 2003, 41.91953201, -87.76825913, '(41.919532014, -87.768259126)') ,
(3500168, 'HK561262', '2004-08-16', '038XX W FILLMORE ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, False, 1133, 11, 24.0, 29.0, '08B', 1151050.0, 1895100.0, 2004, 41.86804342, -87.72094045, '(41.868043424, -87.720940449)') ,
(9068374, 'HW213573', '2013-03-31', '072XX S STONY ISLAND AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'PARKING LOT/GARAGE(NON.RESID.)', True, False, 324, 3, 5.0, 43.0, '08B', 1188071.0, 1857123.0, 2013, 41.76302702, -87.58624307, '(41.763027016, -87.586243072)') ,
(1639049, 'G419948', '2001-07-17', '048XX W 45 ST', '460', 'BATTERY', 'SIMPLE', 'STREET', False, False, 814, 8, NULL, NULL, '08B', 1145097.0, 1874410.0, 2001, 41.81138164, -87.74331705, '(41.811381639, -87.743317053)') ,
(3812293, 'HL177008', '2005-02-11', '067XX S MORGAN ST', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 724, 7, 17.0, 68.0, '08B', 1170803.0, 1860231.0, 2005, 41.77194987, -87.64944248, '(41.771949874, -87.649442482)') ,
(2961173, 'HJ639869', '2003-09-19', '054XX S WENTWORTH AVE', '460', 'BATTERY', 'SIMPLE', 'DEPARTMENT STORE', False, False, 232, 2, 3.0, 37.0, '08B', 1175887.0, 1869033.0, 2003, 41.79599104, -87.63054249, '(41.795991039, -87.630542489)') ,
(4620332, 'HM217017', '2006-03-06', '067XX S EMERALD AVE', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 723, 7, 6.0, 68.0, '08B', 1172498.0, 1860115.0, 2006, 41.7715944, -87.64323257, '(41.771594402, -87.643232569)') ,
(4044375, 'HL388623', '2005-05-30', '008XX S SPRINGFIELD AVE', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 1133, 11, 24.0, 26.0, '08B', 1150520.0, 1895862.0, 2005, 41.8701448, -87.72286629999999, '(41.870144798, -87.7228663)') ,
(9999493, 'HY189363', '2015-03-18', '001XX N STATE ST', '460', 'BATTERY', 'SIMPLE', 'DEPARTMENT STORE', True, False, 111, 1, 42.0, 32.0, '08B', 1176338.0, 1901346.0, 2015, 41.88465026, -87.62791546, '(41.884650262, -87.627915459)') ,
(7692081, 'HS498285', '2010-09-04', '006XX W MELROSE ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', False, True, 2332, 19, 44.0, 6.0, '08B', 1171428.0, 1921885.0, 2010, 41.94111969, -87.64534052, '(41.941119685, -87.645340518)') ,
(5378309, 'HN224900', '2007-03-13', '023XX S WASHTENAW AVE', '041A', 'BATTERY', 'AGGRAVATED: HANDGUN', 'STREET', False, False, 1034, 10, 28.0, 30.0, '04B', 1158737.0, 1888170.0, 2007, 41.84887283, -87.69290967, '(41.848872832, -87.692909673)') ,
(6982403, 'HR387668', '2009-06-21', '021XX W 63RD ST', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', True, False, 726, 7, 15.0, 67.0, '08B', 1163273.0, 1862841.0, 2009, 41.77927306, -87.67697216, '(41.779273063, -87.676972155)') ,
(2679112, 'HJ294892', '2003-04-11', '048XX W HURON ST', '460', 'BATTERY', 'SIMPLE', 'RESIDENCE', False, False, 1532, 15, 28.0, 25.0, '08B', 1143908.0, 1904200.0, 2003, 41.89315175, -87.74693211, '(41.893151749, -87.746932109)') ,
(4448011, 'HL745027', '2005-11-19', '0000X E 69TH ST', '460', 'BATTERY', 'SIMPLE', 'STREET', True, False, 322, 3, 6.0, 69.0, '08B', 1177600.0, 1859277.0, 2005, 41.76918098, -87.62455576, '(41.769180977, -87.624555758)') ,
(1729799, 'G533278', '2001-09-06', '045XX S DREXEL BL', '041A', 'BATTERY', 'AGGRAVATED: HANDGUN', 'OTHER', False, False, 2123, 2, NULL, NULL, '04B', 1182923.0, 1874774.0, 2001, 41.81158406, -87.60456282, '(41.811584058, -87.604562822)') ,
(2495142, 'HH827769', '2002-12-09', '050XX W DIVISION ST', '460', 'BATTERY', 'SIMPLE', 'OTHER COMMERCIAL TRANSPORTATION', False, False, 2533, 25, 37.0, 25.0, '08B', 1142729.0, 1907525.0, 2002, 41.90229796, -87.75117933, '(41.902297963, -87.751179331)') ,
(2341955, 'HH639427', '2002-09-10', '005XX N WALLER AVE', '460', 'BATTERY', 'SIMPLE', 'SCHOOL, PUBLIC, BUILDING', False, False, 1512, 15, 29.0, 25.0, '08B', 1138238.0, 1903181.0, 2002, 41.89045993, -87.76778089, '(41.890459933, -87.767780886)') ,
(3672175, 'HK771992', '2004-11-25', '018XX S SAWYER AVE', '460', 'BATTERY', 'SIMPLE', 'APARTMENT', False, False, 1022, 10, 24.0, 29.0, '08B', 1154987.0, 1890842.0, 2004, 41.85628102, -87.70660101, '(41.856281024, -87.706601006)') ,
(2584554, 'HH867007', '2002-12-30', '040XX W IRVING PARK RD', '499', 'BATTERY', 'AGGRAVATED DOMESTIC BATTERY', 'RESIDENCE', True, True, 1732, 17, 39.0, 16.0, '04B', 1148845.0, 1926278.0, 2002, 41.95364179, -87.72822799, '(41.953641789, -87.728227991)') ,
(2903068, 'HJ575773', '2003-08-20', '016XX W MADISON ST', '460', 'BATTERY', 'SIMPLE', 'RESIDENCE PORCH/HALLWAY', True, False, 1211, 12, 2.0, 28.0, '08B', 1165476.0, 1900090.0, 2003, 41.88144179, -87.66783781, '(41.881441785, -87.667837814)') ,
(6720028, 'HR136920', '2009-01-24', '014XX S BLUE ISLAND AVE', '460', 'BATTERY', 'SIMPLE', 'CHA APARTMENT', False, False, 1231, 12, 2.0, 28.0, '08B', 1168361.0, 1893107.0, 2009, 41.86221801, -87.65744617, '(41.862218007, -87.657446166)') ,
(8223830, 'HT457930', '2011-08-20', '041XX W NORTH AVE', '460', 'BATTERY', 'SIMPLE', 'SMALL RETAIL STORE', False, False, 2534, 25, 30.0, 23.0, '08B', 1148672.0, 1910317.0, 2011, 41.90984675, -87.72927735, '(41.909846748, -87.72927735)') ,
(1363954, 'G070193', '2001-02-03', '004XX W WRIGHTWOOD AV', '460', 'BATTERY', 'SIMPLE', 'RESIDENCE', False, False, 2333, 19, NULL, NULL, '08B', 1172852.0, 1918278.0, 2001, 41.93119046, -87.640214, '(41.93119046, -87.640214004)') ,
(3020236, 'HJ724585', '2003-10-29', '015XX S HAMLIN AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', False, True, 1014, 10, 24.0, 29.0, '08B', 1151293.0, 1892263.0, 2003, 41.86025361, -87.72012274, '(41.860253606, -87.720122737)') ,
(6016285, 'HP121474', '2008-01-13', '026XX N HALSTED ST', '460', 'BATTERY', 'SIMPLE', 'STREET', False, False, 1933, 19, 43.0, 7.0, '08B', 1170490.0, 1917716.0, 2008, 41.92970038, -87.64891033, '(41.929700376, -87.648910326)') ,
(1465880, 'G198370', '2001-04-07', '0000X N PINE AV', '430', 'BATTERY', 'AGGRAVATED: OTHER DANG WEAPON', 'RESIDENCE', False, True, 1522, 15, NULL, NULL, '04B', 1139534.0, 1900041.0, 2001, 41.88181984, -87.76309792, '(41.881819836, -87.763097916)') ,
(3017838, 'HJ723825', '2003-10-28', '044XX S DREXEL BLVD', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', False, True, 2123, 2, 4.0, 39.0, '08B', 1182938.0, 1875792.0, 2003, 41.81437718, -87.60447613, '(41.814377178, -87.604476134)') ,
(2804858, 'HJ458342', '2003-06-28', '015XX N WELLS ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', False, True, 1821, 18, 43.0, 8.0, '08B', 1174421.0, 1910623.0, 2003, 41.91014987, -87.63467747, '(41.910149869, -87.634677466)') ,
(6481371, 'HP558812', '2008-09-07', '041XX W 57TH ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 813, 8, 13.0, 62.0, '08B', 1149766.0, 1866484.0, 2008, 41.78954227, -87.72639649999999, '(41.789542268, -87.726396498)') ,
(10003235, 'HY192694', '2015-03-21', '064XX S OAKLEY AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', False, False, 825, 8, 15.0, 66.0, '08B', 1162182.0, 1861826.0, 2015, 41.77651055, -87.68100012, '(41.776510546, -87.681000121)') ,
(5663639, 'HN472326', '2007-07-17', '010XX N KEYSTONE AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 1111, 11, 37.0, 23.0, '08B', 1149197.0, 1906636.0, 2007, 41.89973555, -87.72744421, '(41.899735553, -87.727444212)') ,
(3663835, 'HK756778', '2004-11-17', '054XX S LAFLIN ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'SIDEWALK', False, True, 932, 9, 16.0, 61.0, '08B', 1167261.0, 1868636.0, 2004, 41.79509079, -87.66218586, '(41.795090788, -87.662185864)') ,
(2423472, 'HH741961', '2002-10-27', '010XX N WALLER AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'STREET', False, True, 1511, 15, 29.0, 25.0, '08B', 1138162.0, 1906418.0, 2002, 41.89934404, -87.76798167, '(41.89934404, -87.767981666)') ,
(11110571, 'JA460432', '2017-10-05', '076XX S HOMAN AVE', '460', 'BATTERY', 'SIMPLE', 'SCHOOL, PUBLIC, GROUNDS', False, False, 835, 8, 18.0, 70.0, '08B', 1155024.0, 1853614.0, 2017, 41.75412154, -87.70746025, '(41.754121535, -87.707460248)') ,
(1419496, 'G140454', '2001-03-11', '077XX S SOUTH SHORE DR', '460', 'BATTERY', 'SIMPLE', 'APARTMENT', False, False, 421, 4, NULL, NULL, '08B', 1197205.0, 1854743.0, 2001, 41.75627357, -87.55284517, '(41.756273565, -87.552845167)') ,
(9589300, 'HX239866', '2014-04-28', '003XX S MOZART ST', '430', 'BATTERY', 'AGGRAVATED: OTHER DANG WEAPON', 'SIDEWALK', False, False, 1124, 11, 2.0, 27.0, '04B', 1157437.0, 1898367.0, 2014, 41.87688097, -87.6974036, '(41.876880974, -87.697403598)') ,
(4766490, 'HM378171', '2006-05-28', '044XX S PRINCETON AVE', '460', 'BATTERY', 'SIMPLE', 'STREET', True, False, 935, 9, 3.0, 37.0, '08B', 1175024.0, 1875681.0, 2006, 41.81425311, -87.63350864, '(41.814253111, -87.633508643)') ,
(9035595, 'HW182288', '2013-03-06', '046XX S LANGLEY AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 221, 2, 4.0, 38.0, '08B', 1181712.0, 1874461.0, 2013, 41.81075326, -87.60901434, '(41.810753256, -87.609014335)') ,
(10549382, 'HZ295843', '2016-06-06', '015XX S HOMAN AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'STREET', False, True, 1021, 10, 24.0, 29.0, '08B', 1153951.0, 1892227.0, 2016, 41.86010231, -87.71036679999999, '(41.860102309, -87.710366798)') ,
(5080475, 'HM673431', '2006-10-21', '094XX S WENTWORTH AVE', '420', 'BATTERY', 'AGGRAVATED:KNIFE/CUTTING INSTR', 'RESIDENCE', False, True, 634, 6, 21.0, 49.0, '04B', 1176612.0, 1842280.0, 2006, 41.72256142, -87.62868748, '(41.72256142, -87.628687477)') ,
(6270946, 'HP352546', '2008-05-23', '051XX S WOOD ST', '460', 'BATTERY', 'SIMPLE', 'SIDEWALK', False, False, 932, 9, 16.0, 61.0, '08B', 1165214.0, 1870708.0, 2008, 41.80082022, -87.66963358, '(41.800820216, -87.669633576)') ,
(6939024, 'HR286405', '2009-04-25', '052XX W CONGRESS PKWY', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', False, True, 1522, 15, 29.0, 25.0, '08B', 1141598.0, 1897182.0, 2009, 41.87393649, -87.75558951, '(41.873936494, -87.755589514)') ,
(6780581, 'HR195987', '2009-03-02', '001XX W 75TH ST', '497', 'BATTERY', 'AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON', 'SIDEWALK', True, False, 731, 7, 6.0, 69.0, '04B', 1176514.0, 1855256.0, 2009, 41.75817139, -87.62865724, '(41.758171386, -87.628657244)') ,
(9116765, 'HW261668', '2013-05-05', '003XX N PULASKI RD', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', False, True, 1114, 11, 28.0, 26.0, '08B', 1149673.0, 1901704.0, 2013, 41.88619239, -87.72582411, '(41.886192389, -87.725824111)') ,
(11137994, 'JA496716', '2017-11-02', '008XX N WASHTENAW AVE', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'RESIDENCE', True, True, 1211, 12, 26.0, 24.0, '08B', 1158244.0, 1905393.0, 2017, 41.89614452, -87.69424841, '(41.896144516, -87.694248407)') ,
(11014534, 'JA344492', '2017-07-12', '009XX E 132ND PL', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'APARTMENT', True, True, 533, 5, 9.0, 54.0, '08B', 1185380.0, 1817834.0, 2017, 41.65527698, -87.59733674, '(41.655276978, -87.597336736)') ,
(10584429, 'HZ334220', '2016-07-03', '023XX W PRATT BLVD', '460', 'BATTERY', 'SIMPLE', 'STREET', True, False, 2412, 24, 50.0, 2.0, '08B', 1159476.0, 1945090.0, 2016, 42.00505013, -87.68862676, '(42.005050125, -87.68862676)') ,
(6806337, 'HR217367', '2009-03-15', '051XX W KINZIE ST', '486', 'BATTERY', 'DOMESTIC BATTERY SIMPLE', 'SIDEWALK', False, True, 1532, 15, 28.0, 25.0, '08B', 1142136.0, 1902250.0, 2009, 41.88783376, -87.75348849, '(41.887833763, -87.753488486)') ,
(8594911, 'HV268964', '2012-05-02', '030XX S HAYNES CT', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 913, 9, 11.0, 60.0, '14', 1168714.0, 1884562.0, 2012, 41.83876214, -87.65639763, '(41.838762141, -87.65639763)') ,
(11127628, 'JA483294', '2017-10-23', '072XX S DAMEN AVE', '1345', 'CRIMINAL DAMAGE', 'TO CITY OF CHICAGO PROPERTY', 'STREET', False, False, 735, 7, 17.0, 67.0, '14', 1164274.0, 1856746.0, 2017, 41.76252653, -87.67347381, '(41.76252653, -87.673473808)') ,
(7937542, 'HT169840', '2011-02-18', '029XX W 36TH ST', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE-GARAGE', False, False, 913, 9, 12.0, 58.0, '14', 1157190.0, 1880621.0, 2011, 41.82818901, -87.698792, '(41.828189011, -87.698792003)') ,
(8388644, 'HT621306', '2011-12-06', '067XX W IMLAY ST', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 1611, 16, 41.0, 10.0, '14', 1130269.0, 1942692.0, 2011, 41.99902303, -87.796138, '(41.99902303, -87.796137998)') ,
(7468176, 'HS269736', '2010-04-22', '051XX S INGLESIDE AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, True, 2124, 2, 4.0, 41.0, '14', 1183412.0, 1871257.0, 2010, 41.80192174, -87.60287886, '(41.801921744, -87.602878856)') ,
(3225932, 'HK243330', '2004-03-16', '106XX S EWING AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'APARTMENT', False, False, 432, 4, 10.0, 52.0, '14', 1202181.0, 1834924.0, 2004, 41.70176349, -87.53528303, '(41.70176349, -87.535283026)') ,
(2048711, 'HH262924', '2002-03-24', '055XX W HARRISON ST', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'SIDEWALK', False, True, 1522, 15, NULL, NULL, '14', 1139534.0, 1896770.0, 2002, 41.87284378, -87.76317771, '(41.872843784, -87.763177706)') ,
(4303010, 'HL613433', '2005-09-15', '023XX W AUGUSTA BLVD', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 1312, 12, 32.0, 24.0, '14', 1160685.0, 1906600.0, 2005, 41.89940638, -87.68524961, '(41.899406378, -87.685249607)') ,
(10079280, 'HY267896', '2015-05-20', '115XX S STEWART AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'RESIDENTIAL YARD (FRONT/BACK)', False, True, 522, 5, 34.0, 53.0, '14', 1175695.0, 1828405.0, 2015, 41.68450696, -87.63245993, '(41.684506962, -87.632459932)') ,
(5758092, 'HN568531', '2007-09-04', '052XX S CARPENTER ST', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, True, 934, 9, 16.0, 61.0, '14', 1170202.0, 1870108.0, 2007, 41.79906661, -87.65135838, '(41.799066605, -87.651358383)') ,
(7881407, 'HT112422', '2011-01-09', '039XX S ROCKWELL ST', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 914, 9, 12.0, 58.0, '14', 1159685.0, 1878264.0, 2011, 41.82167019, -87.68970283, '(41.821670189, -87.689702831)') ,
(8805149, 'HV479133', '2012-09-17', '014XX W OAKDALE AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'APARTMENT', False, False, 1932, 19, 32.0, 6.0, '14', 1166189.0, 1919683.0, 2012, 41.93519105, -87.66465887, '(41.935191052, -87.664658867)') ,
(8921352, 'HV592346', '2012-12-07', '016XX N KILDARE AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 2534, 25, 30.0, 23.0, '14', 1147474.0, 1910521.0, 2012, 41.91042961, -87.73367311, '(41.910429612, -87.733673107)') ,
(2325074, 'HH619797', '2002-09-01', '019XX S MAY ST', '1345', 'CRIMINAL DAMAGE', 'TO CITY OF CHICAGO PROPERTY', 'STREET', False, False, 1233, 12, 25.0, 31.0, '14', 1169104.0, 1890750.0, 2002, 41.85573412, -87.65478711, '(41.855734119, -87.654787112)') ,
(10624311, 'HZ376709', '2016-08-03', '079XX S INGLESIDE AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'APARTMENT', False, False, 624, 6, 8.0, 44.0, '14', 1183941.0, 1852494.0, 2016, 41.75042201, -87.60152439, '(41.750422009, -87.601524389)') ,
(8899397, 'HV573337', '2012-11-23', '013XX S CHRISTIANA AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'APARTMENT', False, False, 1021, 10, 24.0, 29.0, '14', 1154243.0, 1893666.0, 2012, 41.86404527, -87.70925654, '(41.864045266, -87.70925654)') ,
(3906764, 'HL269210', '2005-04-01', '033XX W DICKENS AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 1413, 14, 26.0, 22.0, '14', 1153647.0, 1913754.0, 2005, 41.91918058, -87.71090952, '(41.919180579, -87.710909516)') ,
(4249096, 'HL567458', '2005-08-11', '066XX S CALIFORNIA AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', True, False, 831, 8, 15.0, 66.0, '14', 1158855.0, 1860454.0, 2005, 41.77281419, -87.69323424, '(41.772814187, -87.693234244)') ,
(7260480, 'HR674429', '2009-12-04', '056XX S RICHMOND ST', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 824, 8, 14.0, 63.0, '14', 1157675.0, 1867038.0, 2009, 41.79090565, -87.69738135, '(41.790905647, -87.697381345)') ,
(4606754, 'HM200888', '2006-02-25', '069XX W HIGGINS AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 1613, 16, 41.0, 10.0, '14', 1129154.0, 1935641.0, 2006, 41.97969354, -87.80040126, '(41.979693542, -87.800401259)') ,
(9817983, 'HX467661', '2014-10-14', '013XX S HOMAN AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', True, True, 1021, 10, 24.0, 29.0, '14', 1153916.0, 1893520.0, 2014, 41.86365114, -87.71046084, '(41.863651143, -87.71046084)') ,
(9801245, 'HX448587', '2014-09-28', '113XX S COTTAGE GROVE AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 531, 5, 9.0, 50.0, '14', 1181588.0, 1829877.0, 2014, 41.68841275, -87.6108423, '(41.688412751, -87.610842301)') ,
(7399281, 'HS200939', '2010-03-10', '053XX W CONGRESS PKWY', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'SCHOOL, PUBLIC, GROUNDS', False, False, 1522, 15, 29.0, 25.0, '14', 1140822.0, 1897164.0, 2010, 41.873901399999994, -87.75843909999999, '(41.873901397, -87.758439102)') ,
(5567780, 'HN335611', '2007-05-11', '035XX N CICERO AVE', '1305', 'CRIMINAL DAMAGE', 'CRIMINAL DEFACEMENT', 'RESTAURANT', False, False, 1634, 16, 38.0, 15.0, '14', 1143747.0, 1923278.0, 2007, 41.94550677, -87.74704439, '(41.945506767, -87.747044393)') ,
(10770065, 'HZ535494', '2016-11-30', '033XX W 38TH ST', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 911, 9, 12.0, 58.0, '14', 1154734.0, 1879205.0, 2016, 41.82435275, -87.70784072, '(41.824352752, -87.707840716)') ,
(9499538, 'HX154767', '2014-02-17', '053XX N RAVENSWOOD AVE', '1305', 'CRIMINAL DAMAGE', 'CRIMINAL DEFACEMENT', 'RESIDENCE-GARAGE', False, False, 2012, 20, 40.0, 4.0, '14', 1163235.0, 1935314.0, 2014, 41.97814597, -87.67507366, '(41.978145973, -87.675073656)') ,
(4728934, 'HM337445', '2006-05-07', '028XX N LARAMIE AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, True, 2514, 25, 31.0, 19.0, '14', 1141225.0, 1918275.0, 2006, 41.93182501, -87.75643821, '(41.931825006, -87.756438211)') ,
(4646705, 'HM244120', '2006-03-21', '016XX W TOUHY AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 2423, 24, 49.0, 1.0, '14', 1164202.0, 1947913.0, 2006, 42.01269752, -87.67115923, '(42.012697523, -87.671159228)') ,
(6937224, 'HR342598', '2009-05-26', '022XX W 21ST ST', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 1223, 12, 25.0, 31.0, '14', 1161667.0, 1890025.0, 2009, 41.85390268, -87.68210466, '(41.853902678, -87.68210466)') ,
(3823238, 'HL187666', '2005-02-17', '014XX W CHICAGO AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'BAR OR TAVERN', True, False, 1323, 12, 27.0, 24.0, '14', 1166403.0, 1905446.0, 2005, 41.89611927, -87.66428058, '(41.89611927, -87.664280577)') ,
(9305467, 'HW449676', '2013-09-13', '039XX W GLADYS AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'APARTMENT', False, True, 1132, 11, 24.0, 26.0, '14', 1150166.0, 1898065.0, 2013, 41.87619698, -87.72410855, '(41.876196982, -87.724108549)') ,
(7781193, 'HS589302', '2010-10-29', '095XX S CONSTANCE AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, True, 431, 4, 7.0, 51.0, '14', 1190183.0, 1842108.0, 2010, 41.72177386, -87.57898476, '(41.721773858, -87.578984755)') ,
(9491052, 'HX145326', '2014-02-09', '027XX S SACRAMENTO AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 1033, 10, 12.0, 30.0, '14', 1156803.0, 1885759.0, 2014, 41.84229613, -87.70007290000001, '(41.842296132, -87.700072904)') ,
(5744223, 'HN553354', '2007-08-27', '056XX S PRINCETON AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 711, 7, 3.0, 68.0, '14', 1175250.0, 1867502.0, 2007, 41.79180409, -87.63292413, '(41.791804086, -87.632924134)') ,
(6007606, 'HN784049', '2007-12-29', '026XX W 84TH PL', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, True, 835, 8, 18.0, 70.0, '14', 1160125.0, 1848565.0, 2007, 41.74016294, -87.68890519, '(41.740162942, -87.688905189)') ,
(2862972, 'HJ527694', '2003-07-29', '042XX N SACRAMENTO AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 1724, 17, 33.0, 16.0, '14', 1155622.0, 1928028.0, 2003, 41.95830989, -87.70326764, '(41.958309892, -87.703267639)') ,
(4700014, 'HM303461', '2006-04-20', '040XX W CULLOM AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'VEHICLE NON-COMMERCIAL', False, False, 1722, 17, 39.0, 16.0, '14', 1148801.0, 1928273.0, 2006, 41.95911707, -87.72833797, '(41.959117073, -87.728337966)') ,
(4920006, 'HM535632', '2006-08-10', '055XX N MC VICKER AVE', '1305', 'CRIMINAL DAMAGE', 'CRIMINAL DEFACEMENT', 'RESIDENCE-GARAGE', False, False, 1622, 16, 45.0, 11.0, '14', 1134904.0, 1936485.0, 2006, 41.98190944, -87.7792346, '(41.981909438, -87.779234603)') ,
(8119054, 'HT353855', '2011-06-18', '069XX S ADA ST', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 734, 7, 17.0, 67.0, '14', 1168524.0, 1858841.0, 2011, 41.76818496, -87.6578366, '(41.768184963, -87.657836599)') ,
(1414626, 'G134016', '2001-03-01', '055XX S NOTTINGHAM AV', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 811, 8, NULL, NULL, '14', 1130022.0, 1866716.0, 2001, 41.7905386, -87.79878798, '(41.790538595, -87.79878798)') ,
(6713046, 'HR125162', '2009-01-17', '054XX S KARLOV AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 815, 8, 23.0, 62.0, '14', 1149956.0, 1868311.0, 2009, 41.79455215, -87.72565247, '(41.794552154, -87.725652465)') ,
(1720702, 'G496261', '2001-08-20', '057XX S HAMILTON AV', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'WAREHOUSE', True, False, 715, 7, NULL, NULL, '14', 1162968.0, 1866799.0, 2001, 41.79014072, -87.67797972, '(41.790140719, -87.677979724)') ,
(3108542, 'HJ842548', '2003-12-28', '015XX W AUGUSTA BLVD', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 1323, 12, 27.0, 24.0, '14', 1165882.0, 1906757.0, 2003, 41.89972788, -87.66615665, '(41.89972788, -87.666156654)') ,
(5450649, 'HN280980', '2007-04-12', '036XX N LINCOLN AVE', '1305', 'CRIMINAL DAMAGE', 'CRIMINAL DEFACEMENT', 'RESIDENCE', False, False, 1923, 19, 47.0, 5.0, '14', 1163573.0, 1924158.0, 2007, 41.94752628, -87.67414628, '(41.947526283, -87.674146281)') ,
(11185730, 'JA560123', '2017-12-22', '031XX N NEWCASTLE AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', True, False, 2511, 25, 36.0, 18.0, '14', 1130206.0, 1920118.0, 2017, 41.93707873, -87.79688972, '(41.937078733, -87.796889721)') ,
(10218934, 'HY405637', '2015-09-01', '057XX S CAMPBELL AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'APARTMENT', False, True, 824, 8, 16.0, 63.0, '14', 1160682.0, 1866595.0, 2015, 41.78962843, -87.68636753, '(41.789628428, -87.686367529)') ,
(6620347, 'HP689703', '2008-11-18', '081XX S LAFAYETTE AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 622, 6, 21.0, 44.0, '14', 1177357.0, 1850979.0, 2008, 41.74641581, -87.62569667, '(41.746415805, -87.625696672)') ,
(6940315, 'HR345366', '2009-05-28', '008XX W SUPERIOR ST', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'RESTAURANT', False, False, 1323, 12, 27.0, 24.0, '14', 1170504.0, 1905264.0, 2009, 41.89553111, -87.64922385, '(41.895531107, -87.649223851)') ,
(10501619, 'HZ242802', '2016-04-27', '046XX W 83RD ST', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE-GARAGE', False, False, 834, 8, 13.0, 70.0, '14', 1147330.0, 1849154.0, 2016, 41.74203263, -87.73577014, '(41.742032629, -87.735770141)') ,
(5053244, 'HM661326', '2006-10-15', '035XX N NAGLE AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 1632, 16, 36.0, 17.0, '14', 1132735.0, 1922890.0, 2006, 41.94464161, -87.78753023, '(41.944641609, -87.787530225)') ,
(2709601, 'HJ336995', '2003-05-01', '089XX S UNIVERSITY AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'RESIDENCE', False, False, 413, 4, 8.0, 47.0, '14', 1185441.0, 1845914.0, 2003, 41.73233066, -87.5962343, '(41.732330658, -87.596234295)') ,
(3607386, 'HK694552', '2004-10-18', '072XX S BENNETT AVE', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 333, 3, 5.0, 43.0, '14', 1189958.0, 1857438.0, 2004, 41.76384622, -87.57931687, '(41.763846222, -87.579316872)') ,
(3047178, 'HJ760939', '2003-11-15', '0000X E OHIO ST', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'PARKING LOT/GARAGE(NON.RESID.)', False, False, 1834, 18, 42.0, 8.0, '14', 1176443.0, 1904200.0, 2003, 41.89247942, -87.62744368, '(41.892479421, -87.627443675)') ,
(8690635, 'HV366181', '2012-07-04', '021XX E 87TH ST', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 412, 4, 8.0, 48.0, '14', 1192134.0, 1847716.0, 2012, 41.73711561, -87.57165693, '(41.737115609, -87.57165693)') ,
(2891261, 'HJ563089', '2003-08-15', '109XX S MICHIGAN AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'APARTMENT', True, False, 513, 5, 9.0, 49.0, '14', 1178855.0, 1832369.0, 2003, 41.69531362, -87.6207721, '(41.695313622, -87.620772103)') ,
(2290580, 'HH576932', '2002-08-10', '080XX S MORGAN ST', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 621, 6, 21.0, 71.0, '14', 1171044.0, 1851456.0, 2002, 41.747864899999996, -87.64881501, '(41.747864896, -87.648815009)') ,
(6903733, 'HR308948', '2009-05-07', '005XX N LAVERGNE AVE', '1310', 'CRIMINAL DAMAGE', 'TO PROPERTY', 'SIDEWALK', False, False, 1532, 15, 28.0, 25.0, '14', 1142928.0, 1903317.0, 2009, 41.89074702, -87.75055336, '(41.890747016, -87.750553356)') ,
(3114653, 'HK100758', '2003-12-31', '026XX N WHIPPLE ST', '1320', 'CRIMINAL DAMAGE', 'TO VEHICLE', 'STREET', False, False, 1411, 14, 35.0, 22.0, '14', 1155588.0, 1917493.0, 2003, 41.92940181, -87.70367714, '(41.92940181, -87.703677142)') ,
(8652732, 'HV328581', '2012-06-10', '137XX S BRAINARD AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'OTHER COMMERCIAL TRANSPORTATION', True, False, 433, 4, 10.0, 55.0, '18', 1200405.0, 1814512.0, 2012, 41.64579585, -87.54247201, '(41.645795847, -87.542472012)') ,
(8592856, 'HV267193', '2012-05-02', '001XX W 87TH ST', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'PARKING LOT/GARAGE(NON.RESID.)', True, False, 634, 6, 21.0, 44.0, '18', 1176671.0, 1847269.0, 2012, 41.73625056, -87.62832169, '(41.736250559, -87.628321692)') ,
(2973642, 'HJ620738', '2003-09-10', '048XX N WINTHROP AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'SIDEWALK', True, False, 2033, 20, 46.0, 3.0, '18', 1168000.0, 1932319.0, 2003, 41.96982579, -87.65763728, '(41.969825789, -87.657637278)') ,
(2910492, 'HJ557529', '2003-08-12', '004XX N LARAMIE AVE', '2024', 'NARCOTICS', 'POSS: HEROIN(WHITE)', 'SIDEWALK', True, False, 1532, 15, 28.0, 25.0, '18', 1141629.0, 1902341.0, 2003, 41.88809286, -87.75534813, '(41.888092864, -87.755348129)') ,
(7692354, 'HS498920', '2010-09-04', '071XX S STONY ISLAND AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'STREET', True, False, 324, 3, 5.0, 43.0, '18', 1188152.0, 1858101.0, 2010, 41.7657088, -87.58591505, '(41.765708803, -87.58591505)') ,
(4389244, 'HL608698', '2005-09-13', '002XX S WHIPPLE ST', '2024', 'NARCOTICS', 'POSS: HEROIN(WHITE)', 'SIDEWALK', True, False, 1124, 11, 28.0, 27.0, '18', 1156082.0, 1898741.0, 2005, 41.87793471, -87.70236867, '(41.877934709, -87.702368669)') ,
(1577428, 'G323273', '2001-06-04', '007XX S CALIFORNIA AV', '2024', 'NARCOTICS', 'POSS: HEROIN(WHITE)', 'SIDEWALK', True, False, 1135, 11, NULL, NULL, '18', 1157863.0, 1896427.0, 2001, 41.87154875, -87.69589235, '(41.871548751, -87.695892351)') ,
(3530721, 'HK577020', '2004-08-23', '016XX W JONQUIL TER', '2024', 'NARCOTICS', 'POSS: HEROIN(WHITE)', 'SCHOOL, PUBLIC, GROUNDS', True, False, 2422, 24, 49.0, 1.0, '18', 1163893.0, 1951001.0, 2004, 42.0211776, -87.67220843, '(42.021177601, -87.67220843)') ,
(10177662, 'HY365876', '2015-08-02', '041XX W 87TH ST', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'STREET', True, False, 834, 8, 18.0, 70.0, '18', 1150504.0, 1846544.0, 2015, 41.73480925, -87.72420818, '(41.734809253, -87.724208178)') ,
(7502426, 'HS305355', '2010-05-13', '035XX S WASHTENAW AVE', '1821', 'NARCOTICS', 'MANU/DEL:CANNABIS 10GM OR LESS', 'SCHOOL, PUBLIC, BUILDING', True, False, 913, 9, 12.0, 58.0, '18', 1158944.0, 1880896.0, 2010, 41.82890791, -87.69234919, '(41.828907913, -87.692349187)') ,
(3225630, 'HK162357', '2004-02-03', '047XX W FULTON ST', '2014', 'NARCOTICS', 'MANU/DELIVER: HEROIN (WHITE)', 'ALLEY', True, False, 1113, 11, 28.0, 25.0, '18', 1144810.0, 1901373.0, 2004, 41.88537717, -87.74369071, '(41.88537717, -87.743690705)') ,
(5989011, 'HN778969', '2007-12-26', '100XX W OHARE ST', '2027', 'NARCOTICS', 'POSS: CRACK', 'AIRPORT/AIRCRAFT', True, False, 1651, 16, 41.0, 76.0, '18', 1100658.0, 1934241.0, 2007, 41.97629041, -87.90522722, '(41.976290414, -87.905227221)') ,
(8508682, 'HV185969', '2012-03-06', '003XX S WESTERN AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'SIDEWALK', True, False, 1125, 11, 2.0, 28.0, '18', 1160452.0, 1898418.0, 2012, 41.87695909, -87.68633201, '(41.876959088, -87.686332007)') ,
(2364511, 'HH657604', '2002-09-19', '040XX W WILCOX ST', '2027', 'NARCOTICS', 'POSS: CRACK', 'SIDEWALK', True, False, 1115, 11, 28.0, 26.0, '18', 1149571.0, 1899032.0, 2002, 41.87886211, -87.72626809, '(41.878862107, -87.726268091)') ,
(9416016, 'HW555338', '2013-12-02', '033XX W FILLMORE ST', '2093', 'NARCOTICS', 'FOUND SUSPECT NARCOTICS', 'POLICE FACILITY/VEH PARKING LOT', True, False, 1134, 11, 24.0, 29.0, '26', 1154228.0, 1895173.0, 2013, 41.86818094, -87.70927139, '(41.868180939, -87.709271389)') ,
(8594622, 'HV268708', '2012-05-02', '042XX N MARINE DR', '2032', 'NARCOTICS', 'MANU/DELIVER: METHAMPHETAMINES', 'APARTMENT', True, False, 1915, 19, 46.0, 3.0, '18', 1170986.0, 1928779.0, 2012, 41.96004678, -87.64676205, '(41.960046783, -87.646762046)') ,
(1448800, 'G170513', '2001-03-25', '074XX S PAXTON AV', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'SIDEWALK', True, False, 333, 3, NULL, NULL, '18', 1192135.0, 1855700.0, 2001, 41.75902437, -87.57139424, '(41.759024368, -87.571394244)') ,
(9012649, 'HW160463', '2013-02-17', '071XX S MICHIGAN AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'SIDEWALK', True, False, 323, 3, 6.0, 69.0, '18', 1178399.0, 1857737.0, 2013, 41.76493695, -87.62167373, '(41.764936945, -87.621673727)') ,
(5547866, 'HN355411', '2007-05-21', '004XX N PINE AVE', '2027', 'NARCOTICS', 'POSS: CRACK', 'SIDEWALK', True, False, 1523, 15, 28.0, 25.0, '18', 1139465.0, 1902293.0, 2007, 41.88800087, -87.76329636, '(41.888000869, -87.763296356)') ,
(3248961, 'HK190723', '2004-02-18', '055XX S WELLS ST', '2024', 'NARCOTICS', 'POSS: HEROIN(WHITE)', 'STREET', True, False, 711, 7, 3.0, 68.0, '18', 1175549.0, 1868127.0, 2004, 41.79351246, -87.63180906, '(41.79351246, -87.63180906)') ,
(8784049, 'HV457847', '2012-09-02', '013XX S CHRISTIANA AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'SIDEWALK', True, False, 1021, 10, 24.0, 29.0, '18', 1154245.0, 1893613.0, 2012, 41.86389979, -87.70925061, '(41.863899789, -87.709250612)') ,
(9434484, 'HW566855', '2013-12-11', '002XX S WESTERN AVE', '2014', 'NARCOTICS', 'MANU/DELIVER: HEROIN (WHITE)', 'SIDEWALK', False, False, 1225, 12, 2.0, 28.0, '18', 1160442.0, 1898894.0, 2013, 41.87826548, -87.68635555, '(41.878265483, -87.686355549)') ,
(3830058, 'HL141298', '2005-01-24', '011XX W WILSON AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'OTHER', True, False, 2311, 19, 46.0, 3.0, '18', 1167916.0, 1930703.0, 2005, 41.96539325, -87.65799298, '(41.965393249, -87.657992984)') ,
(6884465, 'HR281750', '2009-04-22', '084XX S SAGINAW AVE', '2027', 'NARCOTICS', 'POSS: CRACK', 'VEHICLE NON-COMMERCIAL', True, False, 423, 4, 7.0, 46.0, '18', 1195298.0, 1849515.0, 2009, 41.74197479, -87.56000598, '(41.741974788, -87.560005978)') ,
(4074763, 'HL342355', '2005-05-08', '038XX S ROCKWELL ST', '2022', 'NARCOTICS', 'POSS: COCAINE', 'STREET', True, False, 913, 9, 12.0, 58.0, '18', 1159670.0, 1878992.0, 2005, 41.82366822, -87.68973785, '(41.823668216, -87.689737854)') ,
(9458866, 'HX112183', '2014-01-12', '056XX S ARTESIAN AVE', '2027', 'NARCOTICS', 'POSS: CRACK', 'ALLEY', True, False, 824, 8, 16.0, 63.0, '18', 1160988.0, 1867300.0, 2014, 41.79155672, -87.68522603, '(41.791556717, -87.685226026)') ,
(2973220, 'HJ624999', '2003-09-12', '006XX E 51ST ST', '2014', 'NARCOTICS', 'MANU/DELIVER: HEROIN (WHITE)', 'SIDEWALK', True, False, 223, 2, 4.0, 38.0, '18', 1181337.0, 1871334.0, 2003, 41.80218118, -87.61048625, '(41.802181175, -87.610486247)') ,
(3085845, 'HJ618818', '2003-09-13', '031XX W FIFTH AVE', '2095', 'NARCOTICS', 'ATTEMPT POSSESSION NARCOTICS', 'STREET', True, False, 1124, 11, 28.0, 27.0, '18', 1155247.0, 1898955.0, 2003, 41.87853875, -87.70542885, '(41.878538751, -87.705428851)') ,
(7185163, 'HR597126', '2009-10-19', '059XX S CAMPBELL AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'RESIDENTIAL YARD (FRONT/BACK)', True, False, 824, 8, 16.0, 66.0, '18', 1160724.0, 1865135.0, 2009, 41.78562112, -87.68625382, '(41.785621124, -87.686253817)') ,
(5753121, 'HN558484', '2007-08-29', '079XX S ELIZABETH ST', '2022', 'NARCOTICS', 'POSS: COCAINE', 'STREET', True, False, 612, 6, 21.0, 71.0, '18', 1169373.0, 1852008.0, 2007, 41.74941598, -87.65492213, '(41.749415975, -87.65492213)') ,
(7097303, 'HR505781', '2009-08-27', '008XX N HOMAN AVE', '2092', 'NARCOTICS', 'SOLICIT NARCOTICS ON PUBLICWAY', 'ALLEY', True, False, 1121, 11, 27.0, 23.0, '26', 1153551.0, 1905610.0, 2009, 41.89683462, -87.71147916, '(41.896834618, -87.711479159)') ,
(5715871, 'HN522324', '2007-08-11', '051XX S EMERALD AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'SIDEWALK', True, False, 934, 9, 3.0, 61.0, '18', 1172176.0, 1870863.0, 2007, 41.801095200000006, -87.64409708, '(41.801095201, -87.644097082)') ,
(5844831, 'HN647052', '2007-10-14', '062XX S DREXEL AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'GAS STATION', True, False, 313, 3, 20.0, 42.0, '18', 1183081.0, 1863726.0, 2007, 41.78126375, -87.60432695, '(41.781263748, -87.60432695)') ,
(8089415, 'HT323138', '2011-05-31', '005XX E MARQUETTE RD', '2024', 'NARCOTICS', 'POSS: HEROIN(WHITE)', 'STREET', True, False, 321, 3, 20.0, 42.0, '18', 1180907.0, 1861368.0, 2011, 41.77484347, -87.61236971, '(41.774843468, -87.612369706)') ,
(9419053, 'HW562748', '2013-12-07', '051XX W MADISON ST', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'PARKING LOT/GARAGE(NON.RESID.)', True, False, 1533, 15, 28.0, 25.0, '18', 1142285.0, 1899542.0, 2013, 41.88039991, -87.75300855, '(41.880399914, -87.753008553)') ,
(5925338, 'HN719188', '2007-11-20', '048XX S WOOD ST', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'STREET', True, False, 931, 9, 20.0, 61.0, '18', 1165167.0, 1872502.0, 2007, 41.80574416, -87.66975515, '(41.805744159, -87.669755148)') ,
(8006283, 'HT237921', '2011-04-06', '017XX S HAMILTON AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'STREET', True, False, 1223, 12, 25.0, 31.0, '18', 1162315.0, 1891573.0, 2011, 41.85813703, -87.679683, '(41.858137031, -87.679683)') ,
(10025545, 'HY215032', '2015-04-08', '039XX W LEXINGTON ST', '2014', 'NARCOTICS', 'MANU/DELIVER: HEROIN (WHITE)', 'SIDEWALK', True, False, 1132, 11, 24.0, 26.0, '18', 1150205.0, 1896407.0, 2015, 41.87164648, -87.72400857, '(41.871646482, -87.724008569)') ,
(3349359, 'HK369511', '2004-05-17', '013XX W 13TH ST', '2095', 'NARCOTICS', 'ATTEMPT POSSESSION NARCOTICS', 'STREET', True, False, 1231, 12, 2.0, 28.0, '18', 1167661.0, 1894186.0, 2004, 41.86519397, -87.65998469, '(41.865193967, -87.659984688)') ,
(4483436, 'HL703172', '2005-10-29', '008XX N KEELER AVE', '2092', 'NARCOTICS', 'SOLICIT NARCOTICS ON PUBLICWAY', 'SIDEWALK', True, False, 1111, 11, 37.0, 23.0, '26', 1148227.0, 1905349.0, 2005, 41.89622263, -87.73104025, '(41.896222633, -87.731040246)') ,
(5372669, 'HM762800', '2006-12-08', '119XX S PARNELL AVE', '2092', 'NARCOTICS', 'SOLICIT NARCOTICS ON PUBLICWAY', 'SIDEWALK', True, False, 524, 5, 34.0, 53.0, '26', 1174782.0, 1825817.0, 2006, 41.67742542, -87.63587885, '(41.677425416, -87.635878846)') ,
(5561014, 'HN370211', '2007-05-28', '008XX W EASTWOOD AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'STREET', True, False, 2312, 19, 46.0, 3.0, '18', 1169806.0, 1931096.0, 2007, 41.96643056, -87.65103242, '(41.966430564, -87.651032417)') ,
(6780235, 'HR195962', '2009-03-02', '039XX W 19TH ST', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'SIDEWALK', True, False, 1014, 10, 24.0, 29.0, '18', 1150239.0, 1890342.0, 2009, 41.85500274, -87.72404177, '(41.855002742, -87.724041772)') ,
(4746425, 'HM284854', '2006-04-11', '006XX N RIDGEWAY AVE', '2024', 'NARCOTICS', 'POSS: HEROIN(WHITE)', 'SIDEWALK', True, False, 1122, 11, 27.0, 23.0, '18', 1151272.0, 1904037.0, 2006, 41.89256317, -87.71989092, '(41.89256317, -87.719890923)') ,
(4031070, 'HL319228', '2005-04-26', '009XX N HAMLIN AVE', '2017', 'NARCOTICS', 'MANU/DELIVER:CRACK', 'STREET', True, False, 1112, 11, 27.0, 23.0, '18', 1150883.0, 1905924.0, 2005, 41.89774891, -87.72127014, '(41.897748913, -87.721270135)') ,
(7041136, 'HR447925', '2009-07-25', '014XX N AVERS AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'APARTMENT', True, False, 2535, 25, 30.0, 23.0, '18', 1150449.0, 1909225.0, 2009, 41.90681567, -87.72277789, '(41.906815669, -87.722777891)') ,
(5226574, 'HM590293', '2006-09-09', '007XX N MONTICELLO AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'STREET', True, False, 1112, 11, 27.0, 23.0, '18', NULL, NULL, 2006, NULL, NULL, nan) ,
(6441121, 'HP513177', '2008-08-14', '053XX W ADDISON ST', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'STREET', True, False, 1634, 16, 38.0, 15.0, '18', 1140135.0, 1923418.0, 2008, 41.94595797, -87.76031752, '(41.945957969, -87.760317517)') ,
(4044725, 'HL331611', '2005-05-02', '009XX N LOREL AVE', '2014', 'NARCOTICS', 'MANU/DELIVER: HEROIN (WHITE)', 'SIDEWALK', True, False, 1524, 15, 37.0, 25.0, '18', 1140504.0, 1905660.0, 2005, 41.89722132, -87.75939804, '(41.897221324, -87.759398038)') ,
(5638846, 'HN448271', '2007-07-05', '049XX W MAYPOLE AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'SIDEWALK', True, False, 1532, 15, 28.0, 25.0, '18', 1143579.0, 1901014.0, 2007, 41.88441515, -87.74822021, '(41.884415152, -87.748220206)') ,
(2159382, 'HH402660', '2002-05-28', '012XX N LOCKWOOD AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'STREET', True, False, 2532, 25, 37.0, 25.0, '18', 1140793.0, 1907963.0, 2002, 41.90353572, -87.75827987, '(41.903535717, -87.758279867)') ,
(5625404, 'HN422493', '2007-06-22', '078XX S COTTAGE GROVE AVE', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'PARKING LOT/GARAGE(NON.RESID.)', True, False, 624, 6, 6.0, 69.0, '18', 1182924.0, 1853196.0, 2007, 41.75237205, -87.60522934, '(41.752372047, -87.605229336)') ,
(6186267, 'HP263901', '2008-04-05', '033XX W 38TH ST', '1811', 'NARCOTICS', 'POSS: CANNABIS 30GMS OR LESS', 'STREET', True, False, 913, 9, 12.0, 58.0, '18', 1154727.0, 1879205.0, 2008, 41.82435289, -87.70786640000001, '(41.824352892, -87.707866397)') ,
(2403623, 'G577663', '2001-09-26', '062XX S KEDZIE AVE', '2027', 'NARCOTICS', 'POSS: CRACK', 'STREET', True, False, 823, 8, 15.0, 66.0, '18', 1156090.0, 1862921.0, 2001, 41.77964003, -87.70330384, '(41.779640028, -87.70330384)') ,
(8300838, 'HT535175', '2011-10-08', '037XX W WILSON AVE', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'APARTMENT', False, True, 1723, 17, 39.0, 14.0, '26', 1150552.0, 1930307.0, 2011, 41.96466441, -87.72184715, '(41.964664408, -87.721847151)') ,
(10811959, 'JA110097', '2017-01-10', '019XX N HAMLIN AVE', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'RESIDENCE', False, True, 2535, 25, 26.0, 22.0, '26', 1150735.0, 1912671.0, 2017, 41.91626623, -87.72163701, '(41.916266226, -87.721637009)') ,
(4750131, 'HM360756', '2006-05-19', '071XX N PAULINA ST', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'RESIDENCE', False, True, 2423, 24, 49.0, 1.0, '26', 1163884.0, 1947582.0, 2006, 42.011796000000004, -87.67233871, '(42.011795997, -87.67233871)') ,
(5448716, 'HN275969', '2007-04-10', '035XX W GRAND AVE', '2830', 'OTHER OFFENSE', 'OBSCENE TELEPHONE CALLS', 'DRUG STORE', False, False, 1121, 11, 26.0, 23.0, '17', 1152431.0, 1907428.0, 2007, 41.90184559, -87.71554468, '(41.901845588, -87.715544681)') ,
(10779118, 'HZ544441', '2016-12-08', '010XX W 51ST ST', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'APARTMENT', False, True, 933, 9, 16.0, 61.0, '26', 1170415.0, 1871001.0, 2016, 41.80151246, -87.65055125, '(41.801512455, -87.650551251)') ,
(4374455, 'HL660560', '2005-10-08', '058XX S KEELER AVE', '4387', 'OTHER OFFENSE', 'VIOLATE ORDER OF PROTECTION', 'RESIDENCE', True, True, 813, 8, 13.0, 62.0, '26', 1149366.0, 1865564.0, 2005, 41.78702538, -87.72788695, '(41.787025384, -87.727886949)') ,
(7363357, 'HS164946', '2010-02-13', '005XX W BELMONT AVE', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE', 'RESIDENCE', False, False, 2332, 19, 44.0, 6.0, '26', 1172211.0, 1921511.0, 2010, 41.94007614, -87.6424738, '(41.940076135, -87.642473797)') ,
(6993831, 'HR399710', '2009-06-23', '017XX W 18TH ST', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE', 'RESIDENCE', False, True, 1223, 12, 25.0, 31.0, '26', 1164905.0, 1891447.0, 2009, 41.85773678, -87.67017968, '(41.857736775, -87.670179675)') ,
(6155786, 'HP243837', '2008-03-25', '004XX W EVERGREEN AVE', '4510', 'OTHER OFFENSE', 'PROBATION VIOLATION', 'APARTMENT', False, False, 1821, 18, 27.0, 8.0, '26', 1172934.0, 1909278.0, 2008, 41.90649223, -87.64017999, '(41.906492227, -87.640179991)') ,
(6054992, 'HP157774', '2008-02-03', '047XX W BELDEN AVE', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'APARTMENT', False, True, 2522, 25, 31.0, 19.0, '26', 1144244.0, 1914869.0, 2008, 41.92242233, -87.74542953, '(41.922422328, -87.745429533)') ,
(6252456, 'HP340132', '2008-05-16', '048XX S LAKE PARK AVE', '2826', 'OTHER OFFENSE', 'HARASSMENT BY ELECTRONIC MEANS', 'APARTMENT', False, True, 2132, 2, 4.0, 39.0, '26', NULL, NULL, 2008, NULL, NULL, nan) ,
(2748713, 'HJ388435', '2003-05-24', '015XX E 65TH ST', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE', 'APARTMENT', False, False, 321, 3, 5.0, 42.0, '26', 1187366.0, 1862190.0, 2003, 41.77694808, -87.58866619, '(41.776948076, -87.58866619)') ,
(10931066, 'JA246453', '2017-04-30', '034XX W BEACH AVE', '2826', 'OTHER OFFENSE', 'HARASSMENT BY ELECTRONIC MEANS', 'APARTMENT', False, False, 1422, 14, 26.0, 23.0, '26', 1153216.0, 1909426.0, 2017, 41.90731274, -87.71260816, '(41.907312737, -87.712608155)') ,
(5601454, 'HN407116', '2007-06-15', '047XX W IRVING PARK RD', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'OTHER', False, True, 1722, 17, 45.0, 15.0, '26', 1143937.0, 1926169.0, 2007, 41.95343636, -87.74627323, '(41.953436358, -87.746273229)') ,
(10808622, 'JA107722', '2017-01-07', '068XX W WRIGHTWOOD AVE', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'RESIDENCE', False, False, 2512, 25, 36.0, 18.0, '26', 1130226.0, 1916527.0, 2017, 41.92722425, -87.79689879, '(41.927224253, -87.796898791)') ,
(2866020, 'HJ532105', '2003-07-31', '028XX W HENDERSON ST', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE', 'APARTMENT', False, False, 1733, 17, 33.0, 21.0, '26', 1156775.0, 1922046.0, 2003, 41.94187156, -87.69919149, '(41.941871562, -87.699191492)') ,
(2412250, 'HH729144', '2002-10-21', '062XX N SACRAMENTO AVE', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE', 'APARTMENT', False, False, 2413, 24, 50.0, 2.0, '26', 1155218.0, 1941267.0, 2002, 41.99464657, -87.70439555, '(41.994646565, -87.704395549)') ,
(10015483, 'HY204891', '2015-03-31', '011XX N PULASKI RD', '4625', 'OTHER OFFENSE', 'PAROLE VIOLATION', 'SIDEWALK', True, False, 1112, 11, 27.0, 23.0, '26', 1149510.0, 1907365.0, 2015, 41.90172993, -87.7262756, '(41.901729932, -87.726275604)') ,
(10488465, 'HZ228967', '2016-04-16', '016XX N MENARD AVE', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'APARTMENT', False, False, 2531, 25, 29.0, 25.0, '26', 1137505.0, 1910283.0, 2016, 41.90996192, -87.77030168, '(41.909961924, -87.770301678)') ,
(10197369, 'HY385205', '2015-08-16', '013XX W 56TH ST', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'APARTMENT', False, True, 713, 7, 16.0, 67.0, '26', 1168181.0, 1867622.0, 2015, 41.79228849, -87.65884139, '(41.792288494, -87.658841391)') ,
(3305754, 'HK328402', '2004-04-27', '026XX E 103RD ST', '502P', 'OTHER OFFENSE', 'FALSE/STOLEN/ALTERED TRP', 'VEHICLE NON-COMMERCIAL', True, False, 431, 4, 10.0, 51.0, '26', 1160319.0, 1836243.0, 2004, 41.70634533, -87.68853255, '(41.70634533, -87.688532547)') ,
(11018984, 'JA349450', '2017-07-15', '020XX E 79TH ST', '502P', 'OTHER OFFENSE', 'FALSE/STOLEN/ALTERED TRP', 'PARKING LOT/GARAGE(NON.RESID.)', True, False, 414, 4, 8.0, 46.0, '26', 1191073.0, 1852990.0, 2017, 41.75161364, -87.57537386, '(41.751613643, -87.575373855)') ,
(3646946, 'HK743113', '2004-11-10', '080XX S MARYLAND AVE', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE', 'APARTMENT', False, True, 631, 6, 8.0, 44.0, '26', 1183297.0, 1851935.0, 2004, 41.74890306, -87.60390164, '(41.74890306, -87.603901641)') ,
(6769056, 'HR185873', '2009-02-24', '069XX S WOODLAWN AVE', '2820', 'OTHER OFFENSE', 'TELEPHONE THREAT', 'RESIDENCE', False, True, 321, 3, 5.0, 69.0, '26', 1185601.0, 1859225.0, 2009, 41.76885361, -87.59522983, '(41.76885361, -87.595229828)') ,
(2415583, 'HH692211', '2002-10-04', '007XX S KENNETH AVE', '5002', 'OTHER OFFENSE', 'OTHER VEHICLE OFFENSE', 'STREET', True, False, 1131, 11, 24.0, 26.0, '26', 1146814.0, 1896113.0, 2002, 41.87090509, -87.73646590000001, '(41.87090509, -87.736465901)') ,
(11122168, 'JA470839', '2017-10-13', '014XX E 83RD ST', '5000', 'OTHER OFFENSE', 'OTHER CRIME AGAINST PERSON', 'RESIDENCE', False, False, 412, 4, 8.0, 45.0, '26', 1187086.0, 1850233.0, 2017, 41.74414362, -87.59007138, '(41.744143621, -87.590071382)') ,
(9993135, 'HY182950', '2014-11-01', '024XX W OHIO ST', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE', 'APARTMENT', False, False, 1221, 12, 26.0, 24.0, '26', 1160216.0, 1903936.0, 2014, 41.89210585, -87.68704595, '(41.892105849, -87.687045945)') ,
(7255767, 'HR668156', '2009-11-30', '006XX W 61ST ST', '4387', 'OTHER OFFENSE', 'VIOLATE ORDER OF PROTECTION', 'RESIDENCE', True, True, 711, 7, 20.0, 68.0, '26', 1173080.0, 1864440.0, 2009, 41.78344986, -87.64097151, '(41.783449861, -87.640971514)') ,
(10493762, 'HZ234518', '2016-04-21', '031XX W 41ST PL', '4651', 'OTHER OFFENSE', 'SEX OFFENDER: FAIL REG NEW ADD', 'APARTMENT', False, False, 921, 9, 14.0, 58.0, '26', 1156058.0, 1876941.0, 2016, 41.81811349, -87.70304427, '(41.818113493, -87.703044274)') ,
(9426342, 'HW569852', '2013-12-11', '0000X N WALLER AVE', '5002', 'OTHER OFFENSE', 'OTHER VEHICLE OFFENSE', 'RESIDENCE', True, True, 1513, 15, 29.0, 25.0, '26', 1138158.0, 1899781.0, 2013, 41.88113134, -87.76815693, '(41.88113134, -87.768156926)') ,
(7511512, 'HS314108', '2010-05-18', '027XX W ARDMORE AVE', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE', 'APARTMENT', False, False, 2011, 20, 40.0, 2.0, '26', 1157082.0, 1938409.0, 2010, 41.98676632, -87.69761692, '(41.986766317, -87.697616923)') ,
(2200712, 'HH463517', '2002-06-24', '008XX N MICHIGAN AVE', '2825', 'OTHER OFFENSE', 'HARASSMENT BY TELEPHONE', 'COMMERCIAL / BUSINESS OFFICE', False, False, 1833, 18, 42.0, 8.0, '26', 1177330.0, 1906499.0, 2002, 41.89876792, -87.62411633, '(41.898767916, -87.624116333)') ,
(5322667, 'HN182437', '2007-02-17', '091XX S COTTAGE GROVE AVE', '560', 'ASSAULT', 'SIMPLE', 'SIDEWALK', False, True, 633, 6, 8.0, 44.0, '08A', 1183165.0, 1844391.0, 2007, 41.72820456, -87.60461941, '(41.728204555, -87.604619408)') ,
(9348957, 'HW493063', '2013-10-14', '011XX N LOREL AVE', '530', 'ASSAULT', 'AGGRAVATED: OTHER DANG WEAPON', 'ALLEY', False, False, 1524, 15, 37.0, 25.0, '04A', 1140468.0, 1907012.0, 2013, 41.90093203, -87.75949705, '(41.900932034, -87.759497048)') ,
(2948095, 'HJ637197', '2003-08-30', '011XX N KEYSTONE AVE', '560', 'ASSAULT', 'SIMPLE', 'SIDEWALK', False, False, 1111, 11, 27.0, 23.0, '08A', 1149178.0, 1907288.0, 2003, 41.90152508, -87.72749709, '(41.901525076, -87.727497088)') ,
(2492280, 'HH829588', '2002-12-10', '050XX W NORTH AVE', '051A', 'ASSAULT', 'AGGRAVATED: HANDGUN', 'STREET', False, False, 2533, 25, 37.0, 25.0, '04A', 1142629.0, 1910170.0, 2002, 41.909558000000004, -87.75148075, '(41.909558, -87.75148075)') ,
(10574810, 'HZ322965', '2016-06-22', '025XX S DR MARTIN LUTHER KING JR DR', '530', 'ASSAULT', 'AGGRAVATED: OTHER DANG WEAPON', 'PARKING LOT/GARAGE(NON.RESID.)', False, False, 133, 1, 2.0, 33.0, '04A', 1179337.0, 1887599.0, 2016, 41.84685952, -87.61732389, '(41.846859523, -87.617323885)') ,
(9271865, 'HW415978', '2013-08-20', '050XX W JACKSON BLVD', '051A', 'ASSAULT', 'AGGRAVATED: HANDGUN', 'SIDEWALK', False, True, 1533, 15, 28.0, 25.0, '04A', 1143027.0, 1898205.0, 2013, 41.87671722, -87.75031732, '(41.87671722, -87.750317319)') ,
(3943561, 'HL315870', '2005-04-24', '015XX W 18TH PL', '530', 'ASSAULT', 'AGGRAVATED: OTHER DANG WEAPON', 'SIDEWALK', True, False, 1222, 12, 25.0, 31.0, '04A', 1166407.0, 1891160.0, 2005, 41.85691726, -87.66467465, '(41.856917255, -87.664674654)') ,
(2620109, 'HJ222529', '2003-03-06', '103XX S DOTY AVE E', '553', 'ASSAULT', 'AGGRAVATED PO: OTHER DANG WEAP', 'OTHER', True, False, 434, 4, 10.0, 51.0, '04A', 1189757.0, 1836996.0, 2003, 41.70775621, -87.58070877, '(41.707756212, -87.580708771)') ,
(4921404, 'HM535930', '2006-08-12', '0000X W 95TH ST', '560', 'ASSAULT', 'SIMPLE', 'STREET', False, False, 634, 6, 21.0, 49.0, '08A', 1177518.0, 1841973.0, 2006, 41.72169857, -87.62537818, '(41.721698573, -87.625378177)') ,
(10502099, 'HZ242203', '2016-04-27', '028XX W ARTHINGTON ST', '520', 'ASSAULT', 'AGGRAVATED:KNIFE/CUTTING INSTR', 'APARTMENT', False, True, 1135, 11, 2.0, 27.0, '04A', 1157555.0, 1895912.0, 2016, 41.87014181, -87.69703716, '(41.87014181, -87.697037158)') ,
(5360488, 'HN213069', '2007-03-07', '009XX W MARQUETTE RD', '560', 'ASSAULT', 'SIMPLE', 'RESIDENCE', True, False, 723, 7, 17.0, 68.0, '08A', 1171038.0, 1860403.0, 2007, 41.77241673, -87.64857603, '(41.772416732, -87.648576027)') ,
(2109180, 'HH343366', '2002-04-26', '038XX N BROADWAY', '560', 'ASSAULT', 'SIMPLE', 'RESIDENCE PORCH/HALLWAY', False, False, 2324, 19, 46.0, 6.0, '08A', 1170231.0, 1925799.0, 2002, 41.95188613, -87.64962517, '(41.951886129, -87.649625167)') ,
(10950230, 'JA269772', '2017-05-18', '038XX W MONROE ST', '051A', 'ASSAULT', 'AGGRAVATED: HANDGUN', 'STREET', True, True, 1122, 11, 28.0, 26.0, '04A', 1150782.0, 1899386.0, 2017, 41.87980993, -87.72181223, '(41.879809931, -87.721812233)') ,
(2065876, 'HH106347', '2002-01-04', '047XX S FORRESTVILLE AV', '560', 'ASSAULT', 'SIMPLE', 'STREET', True, False, 222, 2, NULL, NULL, '08A', 1180725.0, 1873755.0, 2002, 41.8088387, -87.61265623, '(41.808838704, -87.612656231)') ,
(4892475, 'HM507529', '2006-07-28', '074XX S MORGAN ST', '560', 'ASSAULT', 'SIMPLE', 'RESIDENCE', False, True, 733, 7, 17.0, 68.0, '08A', 1170929.0, 1855598.0, 2006, 41.75923359, -87.6491157, '(41.759233594, -87.649115701)') ,
(6170783, 'HP258815', '2008-04-03', '023XX W 95TH ST', '560', 'ASSAULT', 'SIMPLE', 'COMMERCIAL / BUSINESS OFFICE', False, False, 2221, 22, 19.0, 72.0, '08A', 1162615.0, 1841590.0, 2008, 41.72097091, -87.67997607, '(41.720970905, -87.679976067)') ,
(3741570, 'HL111533', '2005-01-07', '005XX E 51ST ST', '560', 'ASSAULT', 'SIMPLE', 'HOSPITAL BUILDING/GROUNDS', False, False, 223, 2, 3.0, 38.0, '08A', 1180467.0, 1871309.0, 2005, 41.80213261, -87.61367763, '(41.802132606, -87.613677631)') ,
(8533129, 'HS479338', '2010-08-24', '084XX S CHAPPEL AVE', '560', 'ASSAULT', 'SIMPLE', 'CHA APARTMENT', False, False, 412, 4, 8.0, 45.0, '08A', 1191324.0, 1849177.0, 2010, 41.74114436, -87.57457729, '(41.741144361, -87.574577285)') ,
(10653179, 'HZ403432', '2016-08-23', '008XX E 65TH ST', '051A', 'ASSAULT', 'AGGRAVATED: HANDGUN', 'APARTMENT', True, True, 312, 3, 20.0, 42.0, '04A', 1183153.0, 1862099.0, 2016, 41.77679744, -87.60411357, '(41.776797435, -87.604113569)') ,
(8789058, 'HV462975', '2012-09-05', '057XX S ROCKWELL ST', '560', 'ASSAULT', 'SIMPLE', 'STREET', False, True, 824, 8, 16.0, 63.0, '08A', 1160020.0, 1866486.0, 2012, 41.78934296, -87.68879789, '(41.789342962, -87.688797892)') ,
(9269662, 'HW414519', '2013-08-19', '007XX E 43RD ST', '560', 'ASSAULT', 'SIMPLE', 'SIDEWALK', False, False, 221, 2, 4.0, 38.0, '08A', 1181988.0, 1876665.0, 2013, 41.81679481, -87.60793377, '(41.816794814, -87.607933765)') ,
(4681320, 'HL414306', '2005-06-11', '076XX S PAULINA ST', '051A', 'ASSAULT', 'AGGRAVATED: HANDGUN', 'APARTMENT', False, False, 611, 6, 17.0, 71.0, '04A', 1166338.0, 1854024.0, 2005, 41.75501329, -87.66598631, '(41.755013292, -87.665986312)') ,
(8082600, 'HT315369', '2011-05-26', '032XX W ADAMS ST', '545', 'ASSAULT', 'PRO EMP HANDS NO/MIN INJURY', 'SCHOOL, PUBLIC, GROUNDS', False, False, 1124, 11, 28.0, 27.0, '08A', 1154752.0, 1898890.0, 2011, 41.87837031, -87.70724814, '(41.878370307, -87.707248137)') ,
(3970108, 'HL330223', '2005-05-02', '0000X W CHICAGO AVE', '560', 'ASSAULT', 'SIMPLE', 'OTHER', True, False, 1832, 18, 42.0, 8.0, '08A', 1175959.0, 1905721.0, 2005, 41.89666404, -87.62917534, '(41.896664035, -87.629175344)') ,
(3661130, 'HK751265', '2004-11-15', '084XX S ELIZABETH ST', '530', 'ASSAULT', 'AGGRAVATED: OTHER DANG WEAPON', 'STREET', False, False, 613, 6, 21.0, 71.0, '04A', 1169464.0, 1848751.0, 2004, 41.74047635, -87.6546828, '(41.740476349, -87.654682797)') ,
(8306557, 'HT540548', '2011-10-13', '007XX W 76TH ST', '560', 'ASSAULT', 'SIMPLE', 'RESIDENCE', False, False, 621, 6, 17.0, 68.0, '08A', 1172760.0, 1854473.0, 2011, 41.7561063, -87.64243828, '(41.756106295, -87.642438283)') ,
(9748574, 'HX398190', '2014-08-22', '111XX S VERNON AVE', '560', 'ASSAULT', 'SIMPLE', 'SIDEWALK', False, False, 531, 5, 9.0, 49.0, '08A', 1181228.0, 1831035.0, 2014, 41.69159874, -87.61212473, '(41.691598742, -87.612124734)') ,
(3313069, 'HK348713', '2004-05-07', '078XX S SOUTH SHORE DR', '560', 'ASSAULT', 'SIMPLE', 'OTHER', False, False, 421, 4, 7.0, 43.0, '08A', 1197870.0, 1853997.0, 2004, 41.75420991, -87.55043302, '(41.754209909, -87.550433015)') ,
(6793833, 'HR206683', '2009-03-09', '049XX W WASHINGTON BLVD', '560', 'ASSAULT', 'SIMPLE', 'RESIDENCE PORCH/HALLWAY', False, False, 1532, 15, 28.0, 25.0, '08A', 1143459.0, 1900050.0, 2009, 41.88177206, -87.74868498, '(41.881772064, -87.748684983)') ,
(1962624, 'HH154448', '2002-01-28', '064XX S PEORIA DR', '560', 'ASSAULT', 'SIMPLE', 'RESIDENCE', True, True, 723, 7, NULL, NULL, '08A', 1171787.0, 1862205.0, 2002, 41.77734524, -87.6457776, '(41.777345235, -87.645777602)') ,
(11111810, 'JA462321', '2017-10-07', '064XX S MORGAN ST', '520', 'ASSAULT', 'AGGRAVATED:KNIFE/CUTTING INSTR', 'APARTMENT', True, True, 724, 7, 16.0, 68.0, '04A', 1170753.0, 1862070.0, 2017, 41.7769974, -87.64957217, '(41.776997403, -87.649572168)') ,
(8557804, 'HV233824', '2012-04-08', '018XX W MONROE ST', '560', 'ASSAULT', 'SIMPLE', 'OTHER', False, False, 1211, 12, 2.0, 28.0, '08A', 1164254.0, 1899585.0, 2012, 41.88008194, -87.67233922, '(41.880081938, -87.672339218)') ,
(6900303, 'HR307075', '2009-05-05', '017XX N MASON AVE', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'APARTMENT', False, False, 2531, 25, 29.0, 25.0, '5', 1136478.0, 1911018.0, 2009, 41.91199729, -87.77405691, '(41.911997289, -87.774056913)') ,
(10399689, 'HZ134318', '2016-01-25', '001XX W 108TH ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 513, 5, 34.0, 49.0, '5', 1177107.0, 1833334.0, 2016, 41.69800123, -87.6271431, '(41.698001226, -87.627143101)') ,
(4826872, 'HM437597', '2006-06-24', '031XX E 87TH ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'FACTORY/MANUFACTURING BUILDING', False, False, 424, 4, 7.0, 46.0, '5', 1198380.0, 1847881.0, 2006, 41.73741441, -87.54876843, '(41.737414412, -87.54876843)') ,
(9088959, 'HW233866', '2013-03-19', '019XX S LAWNDALE AVE', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'APARTMENT', False, False, 1014, 10, 24.0, 29.0, '5', 1152009.0, 1890137.0, 2013, 41.85440555, -87.71755043, '(41.854405548, -87.717550434)') ,
(1825533, 'G659211', '2001-11-01', '019XX N WILMOT AV', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'RESIDENCE PORCH/HALLWAY', False, False, 1434, 14, NULL, NULL, '5', 1160401.0, 1912953.0, 2001, 41.91684538, -87.68611667, '(41.916845382, -87.686116668)') ,
(9129599, 'HW274134', '2013-05-11', '050XX W OAKDALE AVE', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 2521, 25, 31.0, 19.0, '5', 1142202.0, 1919143.0, 2013, 41.93418879, -87.75282624, '(41.934188789, -87.752826241)') ,
(9383468, 'HW526270', '2013-11-07', '011XX W CHESTNUT ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 1213, 12, 27.0, 24.0, '5', 1168755.0, 1906292.0, 2013, 41.8983901, -87.65561766, '(41.898390095, -87.655617655)') ,
(1603886, 'G375927', '2001-06-28', '069XX S EAST END AV', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'RESIDENCE-GARAGE', False, False, 332, 3, NULL, NULL, '5', 1189024.0, 1858986.0, 2001, 41.76811648, -87.58269062, '(41.76811648, -87.582690622)') ,
(6239811, 'HP325677', '2008-05-08', '068XX W NORTH AVE', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESTAURANT', False, False, 2513, 25, 36.0, 25.0, '5', 1130295.0, 1909886.0, 2008, 41.90899932, -87.79679791, '(41.908999321, -87.796797911)') ,
(3929767, 'HL300450', '2005-04-16', '003XX W 118TH ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'CONSTRUCTION SITE', False, False, 522, 5, 34.0, 53.0, '5', 1176271.0, 1826678.0, 2005, 41.67975493, -87.63040295, '(41.67975493, -87.630402949)') ,
(5401018, 'HN246122', '2007-03-24', '045XX S ASHLAND AVE', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'CHURCH/SYNAGOGUE/PLACE OF WORSHIP', False, False, 921, 9, 3.0, 61.0, '5', 1166433.0, 1874662.0, 2007, 41.81164453, -87.66505034, '(41.811644528, -87.665050339)') ,
(1959865, 'HH149573', '2002-01-26', '016XX W 79 ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'SMALL RETAIL STORE', False, False, 611, 6, NULL, NULL, '5', 1166792.0, 1852325.0, 2002, 41.75034132, -87.66437094, '(41.750341323, -87.664370937)') ,
(3395645, 'HK450946', '2004-06-23', '026XX N SEMINARY AVE', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', True, False, 1933, 19, 32.0, 7.0, '5', 1168505.0, 1917892.0, 2004, 41.93022658, -87.6561995, '(41.930226577, -87.656199503)') ,
(8200440, 'HT434553', '2011-08-05', '059XX S KOLIN AVE', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE-GARAGE', False, False, 813, 8, 13.0, 65.0, '5', 1148385.0, 1864881.0, 2011, 41.78517002, -87.73150141, '(41.785170023, -87.731501411)') ,
(4223368, 'HL548996', '2005-08-12', '071XX S PRAIRIE AVE', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'APARTMENT', False, False, 323, 3, 6.0, 69.0, '5', 1179273.0, 1857836.0, 2005, 41.76518872, -87.61846728, '(41.765188724, -87.618467275)') ,
(7281380, 'HR696852', '2009-12-18', '109XX S AVENUE D', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE-GARAGE', False, False, 432, 4, 10.0, 52.0, '5', 1204126.0, 1833120.0, 2009, 41.69676349, -87.52822298, '(41.696763489, -87.528222976)') ,
(3420516, 'HK481967', '2004-07-07', '041XX W FULLERTON AVE', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'COMMERCIAL / BUSINESS OFFICE', False, False, 2524, 25, 31.0, 20.0, '5', 1148209.0, 1915631.0, 2004, 41.92443781, -87.73084109, '(41.92443781, -87.730841092)') ,
(7991098, 'HT202584', '2011-03-14', '013XX E 69TH ST', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'APARTMENT', False, False, 321, 3, 5.0, 69.0, '5', 1186023.0, 1859609.0, 2011, 41.76989739, -87.5936709, '(41.769897392, -87.593670899)') ,
(2766777, 'HJ409442', '2003-06-05', '023XX S MILLARD AVE', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'CHURCH/SYNAGOGUE/PLACE OF WORSHIP', False, False, 1013, 10, 22.0, 30.0, '5', 1152403.0, 1888309.0, 2003, 41.84938154, -87.71615251, '(41.849381535, -87.716152507)') ,
(8020403, 'HT251336', '2011-04-15', '094XX S PLEASANT AVE', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 2221, 22, 19.0, 72.0, '5', 1165402.0, 1842077.0, 2011, 41.72224881, -87.66975409, '(41.722248814, -87.669754093)') ,
(8901424, 'HV575905', '2012-11-21', '062XX W MELROSE ST', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'RESIDENCE-GARAGE', False, False, 1633, 16, 36.0, 17.0, '5', 1134147.0, 1920947.0, 2012, 41.93928502, -87.78238611, '(41.939285016, -87.78238611)') ,
(7083209, 'HR489129', '2009-08-17', '039XX N NEW ENGLAND AVE', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'RESIDENCE', False, False, 1632, 16, 38.0, 17.0, '5', 1129694.0, 1925351.0, 2009, 41.95144746, -87.79865147, '(41.951447464, -87.798651469)') ,
(8193625, 'HT427568', '2011-08-01', '014XX W 103RD ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'CLEANING STORE', False, False, 2213, 22, 19.0, 72.0, '5', 1168331.0, 1836447.0, 2011, 41.70673677, -87.65918703, '(41.706736767, -87.659187025)') ,
(7136001, 'HR539548', '2009-09-16', '087XX S BLACKSTONE AVE', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'RESIDENCE-GARAGE', False, False, 412, 4, 8.0, 48.0, '5', 1187647.0, 1847282.0, 2009, 41.73603247, -87.58810948, '(41.736032465, -87.588109483)') ,
(7623578, 'HS426544', '2010-07-23', '071XX S EMERALD AVE', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'APARTMENT', False, False, 732, 7, 6.0, 68.0, '5', 1172577.0, 1857615.0, 2010, 41.76473236, -87.64301653, '(41.764732362, -87.643016534)') ,
(8321464, 'HT556032', '2011-10-23', '029XX E 78TH ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 421, 4, 7.0, 43.0, '5', 1197014.0, 1853942.0, 2011, 41.75408032, -87.55357173, '(41.754080317, -87.553571733)') ,
(3880644, 'HL254965', '2005-03-25', '083XX S ABERDEEN ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 613, 6, 21.0, 71.0, '5', 1170436.0, 1849550.0, 2005, 41.74264783, -87.65109829, '(41.742647825, -87.651098285)') ,
(10904332, 'JA216127', '2017-04-07', '066XX S BISHOP ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 725, 7, 17.0, 67.0, '5', 1167806.0, 1860811.0, 2017, 41.77360634, -87.66041191, '(41.77360634, -87.660411912)') ,
(9204493, 'HW350078', '2013-07-05', '070XX S MAY ST', '610', 'BURGLARY', 'FORCIBLE ENTRY', 'RESIDENCE', False, False, 733, 7, 17.0, 68.0, '5', 1169869.0, 1858086.0, 2013, 41.76608406, -87.65292843, '(41.766084055, -87.652928432)') ,
(6901330, 'HR308245', '2009-04-22', '051XX W WASHINGTON BLVD', '620', 'BURGLARY', 'UNLAWFUL ENTRY', 'APARTMENT', False, False, 1533, 15, 28.0, 25.0, '5', 1142219.0, 1900020.0, 2009, 41.88171283, -87.75323904, '(41.88171283, -87.753239038)') ,
(10268156, 'HY455377', '2015-08-31', '072XX S WOOD ST', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 735, 7, 17.0, 67.0, '7', 1165601.0, 1856776.0, 2015, 41.76258082, -87.66860929, '(41.762580818, -87.66860929)') ,
(6672913, 'HP747058', '2008-12-22', '065XX S NORMAL BLVD', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 722, 7, 20.0, 68.0, '7', 1174217.0, 1861584.0, 2008, 41.77558748, -87.63688771, '(41.775587482, -87.636887705)') ,
(5835389, 'HN645348', '2007-10-12', '032XX W CULLOM AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 1724, 17, 33.0, 16.0, '7', 1153808.0, 1928395.0, 2007, 41.95935337, -87.70992679, '(41.95935337, -87.709926786)') ,
(3969617, 'HL334216', '2005-05-03', '085XX S CRANDON AVE', '930', 'MOTOR VEHICLE THEFT', 'THEFT/RECOVERY: AUTOMOBILE', 'STREET', False, False, 412, 4, 8.0, 46.0, '7', 1192986.0, 1848936.0, 2005, 41.74044266, -87.56849579, '(41.740442663, -87.568495789)') ,
(7427845, 'HS230740', '2010-03-19', '038XX N LAWNDALE AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 1732, 17, 39.0, 16.0, '7', 1151043.0, 1925228.0, 2010, 41.95071764, -87.72017548, '(41.950717641, -87.72017548)') ,
(8921455, 'HV593288', '2012-12-03', '001XX S BELL AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 1211, 12, 2.0, 28.0, '7', 1161406.0, 1899247.0, 2012, 41.87921415, -87.68280613, '(41.879214151, -87.682806132)') ,
(3310641, 'HK341632', '2004-04-30', '092XX S MERRILL AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 413, 4, 7.0, 48.0, '7', 1192116.0, 1844223.0, 2004, 41.72753093, -87.5718361, '(41.727530927, -87.571836097)') ,
(2514430, 'HH859922', '2002-12-25', '055XX S LAKE PARK AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 2132, 2, 5.0, 41.0, '7', 1187590.0, 1868643.0, 2002, 41.79465029, -87.58763986, '(41.794650287, -87.587639858)') ,
(1790731, 'G611731', '2001-10-11', '010XX N ORLEANS ST', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 1823, 18, NULL, NULL, '7', 1173677.0, 1907427.0, 2001, 41.90139648, -87.63750584, '(41.90139648, -87.637505838)') ,
(5161792, 'HM757133', '2006-12-04', '020XX S JEFFERSON ST', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 1233, 12, 25.0, 31.0, '7', 1172647.0, 1890608.0, 2006, 41.85526689, -87.64178688, '(41.855266893, -87.641786875)') ,
(3072896, 'HJ786817', '2003-11-28', '032XX W PALMER ST', '930', 'MOTOR VEHICLE THEFT', 'THEFT/RECOVERY: AUTOMOBILE', 'STREET', False, False, 1413, 14, 26.0, 22.0, '7', 1154005.0, 1914431.0, 2003, 41.92103119, -87.70957608, '(41.921031186, -87.709576082)') ,
(8681629, 'HV356669', '2012-06-27', '060XX S CHAMPLAIN AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 313, 3, 20.0, 42.0, '7', 1181604.0, 1865055.0, 2012, 41.78494488, -87.60970089, '(41.784944881, -87.609700893)') ,
(9556192, 'HX207970', '2014-04-01', '001XX N KARLOV AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 1114, 11, 28.0, 26.0, '7', 1149045.0, 1900297.0, 2014, 41.8823436, -87.72816673, '(41.882343602, -87.728166727)') ,
(5136468, 'HM733422', '2006-11-21', '002XX N WOLCOTT AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 1333, 12, 27.0, 28.0, '7', 1163701.0, 1901624.0, 2006, 41.88568879, -87.67431227, '(41.885688792, -87.674312269)') ,
(2116573, 'HH352862', '2002-05-05', '071XX S MICHIGAN AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'OTHER', False, False, 323, 3, 6.0, 69.0, '7', 1178397.0, 1857805.0, 2002, 41.76512359, -87.621679, '(41.76512359, -87.621678995)') ,
(1819413, 'G652158', '2001-10-29', '035XX W FRANKLIN BL', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', True, False, 1121, 11, NULL, NULL, '7', 1152790.0, 1903147.0, 2001, 41.890091, -87.71433947, '(41.890091004, -87.714339469)') ,
(9651438, 'HX301095', '2014-06-12', '056XX S PEORIA ST', '920', 'MOTOR VEHICLE THEFT', 'ATT: AUTOMOBILE', 'STREET', False, False, 712, 7, 20.0, 68.0, '7', 1171269.0, 1867439.0, 2014, 41.79171928, -87.64752354, '(41.791719281, -87.647523539)') ,
(9974424, 'HY163644', '2015-02-25', '031XX W 66TH ST', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 831, 8, 15.0, 66.0, '7', 1156680.0, 1860698.0, 2015, 41.77352791, -87.70120071, '(41.773527905, -87.701200713)') ,
(6723496, 'HR140766', '2009-01-26', '054XX W POTOMAC AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 2532, 25, 37.0, 25.0, '7', 1139723.0, 1908121.0, 2009, 41.903988899999995, -87.76220641, '(41.903988903, -87.762206409)') ,
(2316019, 'HH605275', '2002-08-25', '054XX S SPAULDING AVE', '915', 'MOTOR VEHICLE THEFT', 'TRUCK, BUS, MOTOR HOME', 'STREET', False, False, 822, 8, 14.0, 63.0, '7', 1155306.0, 1868363.0, 2002, 41.7945894, -87.70603255, '(41.794589396, -87.706032546)') ,
(2859254, 'HJ513676', '2003-07-23', '078XX S ASHLAND AVE', '930', 'MOTOR VEHICLE THEFT', 'THEFT/RECOVERY: AUTOMOBILE', 'ALLEY', False, False, 612, 6, 17.0, 71.0, '7', 1167037.0, 1852681.0, 2003, 41.75131301, -87.66346299, '(41.751313006, -87.663462987)') ,
(2103277, 'HH337319', '2002-04-28', '023XX N KEATING AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 2522, 25, 31.0, 19.0, '7', 1144344.0, 1915147.0, 2002, 41.92318331, -87.74505509, '(41.923183306, -87.745055091)') ,
(2169996, 'HH423186', '2002-06-06', '046XX S CHRISTIANA AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 821, 8, 14.0, 58.0, '7', 1154837.0, 1873772.0, 2002, 41.80944182, -87.70760798, '(41.809441824, -87.707607982)') ,
(5733472, 'HN543219', '2007-08-21', '061XX S MAPLEWOOD AVE', '910', 'MOTOR VEHICLE THEFT', 'AUTOMOBILE', 'STREET', False, False, 825, 8, 15.0, 66.0, '7', 1160422.0, 1863958.0, 2007, 41.78239751, -87.68739352, '(41.782397506, -87.687393515)') ,
(10707046, 'HZ463811', '2016-10-06', '0000X W CTA OHARE PLATFORM ST', '1150', 'DECEPTIVE PRACTICE', 'CREDIT CARD FRAUD', 'CTA TRAIN', False, False, 1654, 16, 41.0, 76.0, '11', 1100955.0, 1934905.0, 2016, 41.97810845, -87.90412276, '(41.978108446, -87.904122758)') ,
(5904817, 'HN702776', '2007-11-09', '005XX N KINGSBURY ST', '1150', 'DECEPTIVE PRACTICE', 'CREDIT CARD FRAUD', 'RESTAURANT', False, False, 1831, 18, 42.0, 8.0, '11', 1173193.0, 1903598.0, 2007, 41.89090025, -87.63939736, '(41.890900247, -87.639397357)') ,
(7195847, 'HR607189', '2009-10-25', '059XX N LINCOLN AVE', '1210', 'DECEPTIVE PRACTICE', 'THEFT OF LABOR/SERVICES', 'RESTAURANT', False, False, 2011, 20, 40.0, 2.0, '11', 1155105.0, 1939170.0, 2009, 41.98889458, -87.70486783, '(41.988894576, -87.704867826)') ,
(1826250, 'G654957', '2001-10-20', '016XX E 95 ST', '1120', 'DECEPTIVE PRACTICE', 'FORGERY', 'BANK', False, False, 431, 4, NULL, NULL, '10', 1189065.0, 1842275.0, 2001, 41.72225893, -87.58307441, '(41.722258934, -87.583074406)') ,
(3761051, 'HL131736', '2005-01-13', '078XX S WINCHESTER AVE', '1150', 'DECEPTIVE PRACTICE', 'CREDIT CARD FRAUD', 'RESIDENCE', False, True, 611, 6, 18.0, 71.0, '11', 1164718.0, 1852772.0, 2005, 41.75161195, -87.67195845, '(41.751611954, -87.671958451)') ,
(5713696, 'HN519755', '2007-08-10', '023XX W GRAND AVE', '1220', 'DECEPTIVE PRACTICE', 'THEFT OF LOST/MISLAID PROP', 'STREET', True, False, 1313, 12, 26.0, 24.0, '11', 1160756.0, 1903423.0, 2007, 41.89068696, -87.68507699, '(41.890686955, -87.685076991)') ,
(1803248, 'G629024', '2001-06-10', '046XX N DELPHIA AV', '1130', 'DECEPTIVE PRACTICE', 'FRAUD OR CONFIDENCE GAME', 'OTHER', False, False, 1614, 16, NULL, NULL, '11', 1117909.0, 1930160.0, 2001, 41.96483755, -87.84187255, '(41.964837552, -87.841872545)') ,
(3864164, 'HL239939', '2004-12-09', '004XX N STATE ST', '1206', 'DECEPTIVE PRACTICE', 'THEFT BY LESSEE,MOTOR VEH', 'STREET', True, False, 1834, 18, 42.0, 8.0, '11', 1176302.0, 1903096.0, 2004, 41.88945317, -87.62799483, '(41.889453169, -87.627994833)') ,
(3300223, 'HK328522', '2004-04-27', '016XX W HOWARD ST', '1220', 'DECEPTIVE PRACTICE', 'THEFT OF LOST/MISLAID PROP', 'CTA PLATFORM', True, False, 2422, 24, 49.0, 1.0, '11', 1163750.0, 1950346.0, 2004, 42.0193833, -87.67275327, '(42.019383301, -87.67275327)') ,
(2723161, 'HJ349966', '2003-03-20', '035XX W 77TH PL', '1150', 'DECEPTIVE PRACTICE', 'CREDIT CARD FRAUD', 'RESIDENCE', False, False, 835, 8, 18.0, 70.0, '11', 1154152.0, 1852985.0, 2003, 41.75241282, -87.71067256, '(41.752412818, -87.710672559)') ,
(1580040, 'G345035', '2001-06-11', '087XX S DR MARTN LUTHR KING JR DR', '1110', 'DECEPTIVE PRACTICE', 'BOGUS CHECK', 'BANK', False, False, 632, 6, NULL, NULL, '11', 1180395.0, 1847374.0, 2001, 41.73645417, -87.61467513, '(41.736454169, -87.614675134)') ,
(11057640, 'JA394755', '2017-08-15', '021XX W CRYSTAL ST', '1150', 'DECEPTIVE PRACTICE', 'CREDIT CARD FRAUD', 'RESIDENCE', False, False, 1424, 14, 32.0, 24.0, '11', 1161955.0, 1908307.0, 2017, 41.9040641, -87.68053721, '(41.904064096, -87.680537205)') ,
(10327846, 'HY518743', '2015-11-04', '079XX S MORGAN ST', '1153', 'DECEPTIVE PRACTICE', 'FINANCIAL IDENTITY THEFT OVER $ 300', 'RESIDENCE', False, False, 621, 6, 17.0, 71.0, '11', 1171025.0, 1852140.0, 2015, 41.749742299999994, -87.64886469, '(41.749742298, -87.648864693)') ,
(11178752, 'JA550741', '2017-12-14', '029XX W BERTEAU AVE', '1153', 'DECEPTIVE PRACTICE', 'FINANCIAL IDENTITY THEFT OVER $ 300', 'OTHER', False, False, 1724, 17, 33.0, 16.0, '11', 1156097.0, 1927766.0, 2017, 41.95758135, -87.70152846, '(41.957581352, -87.70152846)') ,
(2225484, 'HH493814', '2002-07-07', '034XX N CLARK ST', '1210', 'DECEPTIVE PRACTICE', 'THEFT OF LABOR/SERVICES', 'RESTAURANT', True, False, 1924, 19, 44.0, 6.0, '11', 1169100.0, 1922885.0, 2002, 41.94391468, -87.65386761, '(41.94391468, -87.653867614)') ,
(10760731, 'HZ523997', '2014-11-21', '0000X S MAYFIELD AVE', '1153', 'DECEPTIVE PRACTICE', 'FINANCIAL IDENTITY THEFT OVER $ 300', 'APARTMENT', False, False, 1513, 15, 29.0, 25.0, '11', NULL, NULL, 2014, NULL, NULL, nan) ,
(1438213, 'G161722', '2001-03-21', '011XX W WILSON AV', '1210', 'DECEPTIVE PRACTICE', 'THEFT OF LABOR/SERVICES', 'CTA PLATFORM', True, False, 2311, 19, NULL, NULL, '11', 1167954.0, 1930663.0, 2001, 41.96528267, -87.65785443, '(41.965282666, -87.657854427)') ,
(10268721, 'HY455698', '2015-10-09', '046XX W DIVERSEY AVE', '1153', 'DECEPTIVE PRACTICE', 'FINANCIAL IDENTITY THEFT OVER $ 300', 'BANK', False, False, 2521, 25, 31.0, 20.0, '11', 1144894.0, 1918211.0, 2015, 41.93158085, -87.74295666, '(41.931580848, -87.742956658)') ,
(2003966, 'HH208082', '2002-01-29', '057XX S CICERO AV', '1130', 'DECEPTIVE PRACTICE', 'FRAUD OR CONFIDENCE GAME', 'AIRPORT/AIRCRAFT', True, False, 813, 8, NULL, NULL, '11', 1145654.0, 1866253.0, 2002, 41.78898704, -87.74147999, '(41.788987036, -87.74147999)') ,
(8210615, 'HT404945', '2011-07-14', '080XX S ESCANABA AVE', '1152', 'DECEPTIVE PRACTICE', 'ILLEGAL USE CASH CARD', 'RESIDENCE', False, False, 422, 4, 7.0, 46.0, '11', 1196894.0, 1852256.0, 2011, 41.74945679, -87.55406744, '(41.749456794, -87.554067444)') ,
(3842013, 'HL213843', '2005-03-03', '004XX E 40TH ST', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'STREET', False, False, 213, 2, 3.0, 38.0, '3', 1179914.0, 1878240.0, 2005, 41.82116453, -87.61549337, '(41.821164532, -87.615493365)') ,
(4810471, 'HM422467', '2006-06-18', '027XX W ARDMORE AVE', '320', 'ROBBERY', 'STRONGARM - NO WEAPON', 'STREET', True, False, 2011, 20, 40.0, 2.0, '3', 1156808.0, 1938405.0, 2006, 41.98676092, -87.69862481, '(41.986760917, -87.698624808)') ,
(10882712, 'JA192460', '2017-03-19', '036XX W ROOSEVELT RD', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'STREET', False, False, 1133, 11, 24.0, 29.0, '3', 1152475.0, 1894470.0, 2017, 41.86628662, -87.71572561, '(41.866286618, -87.715725607)') ,
(4110831, 'HL446401', '2005-06-26', '042XX W 26TH ST', '325', 'ROBBERY', 'VEHICULAR HIJACKING', 'STREET', False, False, 1031, 10, 22.0, 30.0, '3', 1148444.0, 1886380.0, 2005, 41.84416527, -87.73073233, '(41.844165269, -87.730732326)') ,
(7578055, 'HS330202', '2010-05-27', '119XX S MICHIGAN AVE', '320', 'ROBBERY', 'STRONGARM - NO WEAPON', 'STREET', False, False, 532, 5, 9.0, 53.0, '3', 1178981.0, 1825999.0, 2010, 41.67783055, -87.62050376, '(41.677830552, -87.620503762)') ,
(6588112, 'HP661088', '2008-11-02', '043XX W WALTON ST', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'STREET', False, False, 1111, 11, 37.0, 23.0, '3', 1147268.0, 1905976.0, 2008, 41.89796161, -87.73454643, '(41.897961608, -87.734546427)') ,
(2666808, 'HJ281736', '2003-04-05', '084XX S MAY ST', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'STREET', False, False, 613, 6, 21.0, 71.0, '3', 1170122.0, 1848931.0, 2003, 41.74095603, -87.65226674, '(41.74095603, -87.652266736)') ,
(5260944, 'HN136243', '2007-01-21', '036XX N WESTERN AVE', '320', 'ROBBERY', 'STRONGARM - NO WEAPON', 'RESTAURANT', True, False, 1912, 19, 47.0, 5.0, '3', 1159731.0, 1924038.0, 2007, 41.94727721, -87.68827183, '(41.947277209, -87.688271827)') ,
(2906269, 'HJ580490', '2003-08-22', '036XX W DIVISION ST', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'PARKING LOT/GARAGE(NON.RESID.)', False, False, 1112, 11, 27.0, 23.0, '3', 1151715.0, 1907749.0, 2003, 41.90274056, -87.7181662, '(41.902740562, -87.718166202)') ,
(8536808, 'HV213648', '2012-03-25', '055XX S JUSTINE ST', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'RESIDENCE', False, False, 713, 7, 16.0, 67.0, '3', 1166954.0, 1867942.0, 2012, 41.79319294, -87.66333147, '(41.793192941, -87.663331471)') ,
(8841511, 'HV514287', '2012-10-11', '046XX N SACRAMENTO AVE', '320', 'ROBBERY', 'STRONGARM - NO WEAPON', 'SIDEWALK', True, False, 1713, 17, 33.0, 14.0, '3', 1155536.0, 1930847.0, 2012, 41.96604713, -87.70350763, '(41.966047133, -87.703507631)') ,
(3046354, 'HJ753917', '2003-11-12', '052XX W ADDISON ST', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'GAS STATION', False, False, 1634, 16, 38.0, 15.0, '3', 1140892.0, 1923436.0, 2003, 41.94599344, -87.75753456, '(41.945993442, -87.757534561)') ,
(10831592, 'JA133264', '2017-01-28', '009XX N MASSASOIT AVE', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'STREET', False, False, 1511, 15, 29.0, 25.0, '3', 1137854.0, 1905671.0, 2017, 41.89729974, -87.76913101, '(41.897299743, -87.769131006)') ,
(2953056, 'HJ642080', '2003-09-20', '071XX W HIGGINS AVE', '312', 'ROBBERY', 'ARMED:KNIFE/CUTTING INSTRUMENT', 'STREET', False, False, 1613, 16, 41.0, 10.0, '3', 1127782.0, 1936028.0, 2003, 41.98077882, -87.80543826, '(41.980778822, -87.80543826)') ,
(11184062, 'JA557966', '2017-12-21', '001XX S OAKLEY BLVD', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'VEHICLE NON-COMMERCIAL', False, False, 1223, 12, 2.0, 28.0, '3', 1161099.0, 1899441.0, 2017, 41.87975288, -87.68392799, '(41.879752884, -87.683927994)') ,
(8875058, 'HV548880', '2012-11-05', '067XX S STATE ST', '330', 'ROBBERY', 'AGGRAVATED', 'GAS STATION', False, False, 322, 3, 6.0, 69.0, '3', 1177438.0, 1860407.0, 2012, 41.77228548, -87.62511546, '(41.772285481, -87.625115462)') ,
(7410895, 'HR480844', '2009-08-13', '126XX S EMERALD AVE', '326', 'ROBBERY', 'AGGRAVATED VEHICULAR HIJACKING', 'STREET', False, False, 523, 5, 34.0, 53.0, '3', 1173624.0, 1821070.0, 2009, 41.66442451, -87.64025723, '(41.66442451, -87.640257226)') ,
(9425197, 'HW568748', '2013-12-12', '081XX S PERRY AVE', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'RESIDENTIAL YARD (FRONT/BACK)', False, False, 622, 6, 21.0, 44.0, '3', 1177022.0, 1851111.0, 2013, 41.74678558, -87.62692021, '(41.746785583, -87.626920212)') ,
(2164207, 'HH405177', '2002-05-29', '041XX W GLADYS AVE', '312', 'ROBBERY', 'ARMED:KNIFE/CUTTING INSTRUMENT', 'STREET', False, False, 1132, 11, 28.0, 26.0, '3', 1148751.0, 1898031.0, 2002, 41.87613113, -87.72930487, '(41.876131125, -87.729304868)') ,
(4136958, 'HL472862', '2005-07-09', '107XX S LA SALLE ST', '031A', 'ROBBERY', 'ARMED: HANDGUN', 'ALLEY', False, False, 513, 5, 34.0, 49.0, '3', 1177175.0, 1833834.0, 2005, 41.69937177, -87.62687911, '(41.699371768, -87.626879105)') ,
(1570987, 'G326063', '2001-06-05', '029XX S DEARBORN ST', '1350', 'CRIMINAL TRESPASS', 'TO STATE SUP LAND', 'CHA PARKING LOT/GROUNDS', True, False, 2113, 2, NULL, NULL, '26', 1176499.0, 1884965.0, 2001, 41.83969609, -87.62781857, '(41.839696087, -87.627818571)') ,
(4084479, 'HL424420', '2005-06-16', '027XX S STATE ST', '1350', 'CRIMINAL TRESPASS', 'TO STATE SUP LAND', 'CHA PARKING LOT/GROUNDS', True, False, 2113, 1, 3.0, 35.0, '26', 1176730.0, 1886544.0, 2005, 41.84402377, -87.62692325, '(41.844023772, -87.626923253)') ,
(11056591, 'JA393243', '2017-08-16', '094XX S HARVARD AVE', '1365', 'CRIMINAL TRESPASS', 'TO RESIDENCE', 'RESIDENCE', True, False, 634, 6, 21.0, 49.0, '26', 1175675.0, 1842516.0, 2017, 41.72323003, -87.63211253, '(41.723230031, -87.632112534)') ,
(7174283, 'HR585012', '2009-10-13', '043XX W 79TH ST', '1330', 'CRIMINAL TRESPASS', 'TO LAND', 'SCHOOL, PUBLIC, GROUNDS', True, False, 834, 8, 13.0, 70.0, '26', 1148764.0, 1851854.0, 2009, 41.74941446, -87.7304466, '(41.749414464, -87.730446597)') ,
(11106461, 'JA455629', '2017-10-02', '079XX S HALSTED ST', '1330', 'CRIMINAL TRESPASS', 'TO LAND', 'RESTAURANT', True, False, 621, 6, 17.0, 71.0, '26', 1172343.0, 1852285.0, 2017, 41.75011132, -87.64403074, '(41.750111323, -87.644030737)') ,
(8693981, 'HV369974', '2012-07-06', '062XX S TALMAN AVE', '1330', 'CRIMINAL TRESPASS', 'TO LAND', 'ALLEY', True, False, 825, 8, 15.0, 66.0, '26', 1159778.0, 1863228.0, 2012, 41.78040754, -87.68977465, '(41.780407535, -87.689774647)') ,
(5827212, 'HN637742', '2007-10-09', '025XX N PULASKI RD', '1330', 'CRIMINAL TRESPASS', 'TO LAND', 'GAS STATION', True, False, 2524, 25, 31.0, 22.0, '26', 1149288.0, 1916658.0, 2007, 41.92723512, -87.72684967, '(41.927235118, -87.726849666)') ,
(9413737, 'HW557360', '2013-12-03', '004XX S STATE ST', '1330', 'CRIMINAL TRESPASS', 'TO LAND', 'LIBRARY', True, False, 113, 1, 2.0, 32.0, '26', 1176436.0, 1898416.0, 2013, 41.87660796, -87.62764406, '(41.876607964, -87.627644063)') ,
(3064175, 'HJ776916', '2003-11-23', '003XX S MAPLEWOOD AVE', '1350', 'CRIMINAL TRESPASS', 'TO STATE SUP LAND', 'CHA HALLWAY/STAIRWELL/ELEVATOR', True, False, 1125, 11, 2.0, 28.0, '26', 1159433.0, 1898385.0, 2003, 41.87688955, -87.69007438, '(41.876889551, -87.690074383)') ,
(2774880, 'HJ419340', '2003-06-10', '031XX S DEARBORN ST', '1330', 'CRIMINAL TRESPASS', 'TO LAND', 'COLLEGE/UNIVERSITY GROUNDS', True, False, 2113, 2, 3.0, 35.0, '26', 1176477.0, 1884226.0, 2003, 41.83766871, -87.62792157, '(41.83766871, -87.627921572)') ,
(2100965, 'HH329720', '2002-04-25', '005XX E 36TH PL', '1350', 'CRIMINAL TRESPASS', 'TO STATE SUP LAND', 'CHA PARKING LOT/GROUNDS', True, False, 212, 2, 4.0, 35.0, '26', 1180555.0, 1880852.0, 2002, 41.82831734, -87.61306155, '(41.828317337, -87.613061551)') ,
(5635192, 'HM802052', '2006-12-30', '039XX S COTTAGE GROVE AVE', '1360', 'CRIMINAL TRESPASS', 'TO VEHICLE', 'STREET', False, False, 213, 2, 4.0, 38.0, '26', 1182267.0, 1878667.0, 2006, 41.82228198, -87.60684825, '(41.822281982, -87.606848245)') ,
(3081572, 'HJ804627', '2003-12-07', '002XX W ADAMS ST', '1330', 'CRIMINAL TRESPASS', 'TO LAND', 'BANK', True, False, 112, 1, 2.0, 32.0, '26', 1174618.0, 1899411.0, 2003, 41.87937913, -87.63428935, '(41.87937913, -87.634289354)') ,
(6777303, 'HR190881', '2009-02-27', '047XX N WESTERN AVE', '1330', 'CRIMINAL TRESPASS', 'TO LAND', 'BANK', False, False, 1911, 19, 47.0, 4.0, '26', 1159513.0, 1931422.0, 2009, 41.96754381, -87.68886902, '(41.96754381, -87.688869019)') ,
(1762743, 'G575031', '2001-09-25', '0000X W 87 ST', '1330', 'CRIMINAL TRESPASS', 'TO LAND', 'PARKING LOT/GARAGE(NON.RESID.)', True, False, 634, 6, NULL, NULL, '26', 1177043.0, 1847234.0, 2001, 41.73614614, -87.62695987, '(41.736146144, -87.626959873)') ,
(1427912, 'G122095', '2001-03-02', '039XX N ASHLAND AV', '1505', 'PROSTITUTION', 'CALL OPERATION', 'RESIDENCE', True, False, 1923, 19, NULL, NULL, '16', 1164982.0, 1926580.0, 2001, 41.95414251, -87.66889815, '(41.954142513, -87.668898147)') ,
(5784844, 'HN595319', '2007-09-17', '070XX N CLARK ST', '1506', 'PROSTITUTION', 'SOLICIT ON PUBLIC WAY', 'SIDEWALK', True, False, 2424, 24, 49.0, 1.0, '16', 1163398.0, 1946852.0, 2007, 42.00980315, -87.6741476, '(42.009803145, -87.6741476)') ,
(1553689, 'G302291', '2001-05-25', '026XX W MADISON ST', '1506', 'PROSTITUTION', 'SOLICIT ON PUBLIC WAY', 'STREET', True, False, 1125, 11, NULL, NULL, '16', 1158862.0, 1899877.0, 2001, 41.88099546, -87.69213, '(41.880995464, -87.692129998)') ,
(1367327, 'G057394', '2001-01-28', '046XX S CICERO AV', '1513', 'PROSTITUTION', 'SOLICIT FOR BUSINESS', 'STREET', True, False, 814, 8, NULL, NULL, '16', 1145110.0, 1873073.0, 2001, 41.80771246, -87.74330304, '(41.807712461, -87.743303038)') ,
(5228794, 'HM590468', '2006-09-09', '010XX N STATE ST', '1506', 'PROSTITUTION', 'SOLICIT ON PUBLIC WAY', 'STREET', True, False, 1824, 18, 42.0, 8.0, '16', 1176182.0, 1907168.0, 2006, 41.90062965, -87.62831264, '(41.900629649, -87.628312638)') ,
(8789017, 'HV463112', '2012-09-05', '0000X N KENTON AVE', '1506', 'PROSTITUTION', 'SOLICIT ON PUBLIC WAY', 'STREET', True, False, 1113, 11, 28.0, 25.0, '16', 1145800.0, 1899764.0, 2012, 41.88094316, -87.74009605, '(41.880943161, -87.740096048)') ,
(3680034, 'HK749575', '2004-11-14', '002XX N LARAMIE AVE', '143A', 'WEAPONS VIOLATION', 'UNLAWFUL POSS OF HANDGUN', 'RESIDENCE', False, False, 1532, 15, 28.0, 25.0, '15', 1141670.0, 1901145.0, 2004, 41.88481014, -87.75522715, '(41.884810136, -87.75522715)') ,
(7357439, 'HS100098', '2010-01-01', '070XX S ROCKWELL ST', '143B', 'WEAPONS VIOLATION', 'UNLAWFUL POSS OTHER FIREARM', 'RESIDENCE', True, False, 831, 8, 18.0, 66.0, '15', 1160255.0, 1857948.0, 2010, 41.76590865, -87.68817109, '(41.765908652, -87.688171087)') ,
(9432406, 'HW576249', '2013-12-18', '012XX S KEDVALE AVE', '143A', 'WEAPONS VIOLATION', 'UNLAWFUL POSS OF HANDGUN', 'RESIDENCE', True, False, 1011, 10, 24.0, 29.0, '15', 1148913.0, 1894068.0, 2013, 41.86525306, -87.72881253, '(41.865253059, -87.728812529)') ,
(3426362, 'HK485601', '2004-07-10', '067XX S THROOP ST', '143A', 'WEAPONS VIOLATION', 'UNLAWFUL POSS OF HANDGUN', 'SIDEWALK', True, False, 724, 7, 17.0, 67.0, '15', 1168821.0, 1860034.0, 2004, 41.7714523, -87.65671355, '(41.7714523, -87.656713552)') ,
(2269553, 'HH460583', '2002-06-22', '021XX N ST LOUIS AVE', '143A', 'WEAPONS VIOLATION', 'UNLAWFUL POSS OF HANDGUN', 'STREET', True, False, 1413, 14, 26.0, 22.0, '15', 1152922.0, 1913821.0, 2002, 41.91937884, -87.71357149, '(41.919378841, -87.713571489)') ,
(4965099, 'HM576559', '2006-09-02', '087XX S ESCANABA AVE', '143A', 'WEAPONS VIOLATION', 'UNLAWFUL POSS OF HANDGUN', 'RESIDENCE', True, False, 423, 4, 10.0, 46.0, '15', 1196907.0, 1847528.0, 2006, 41.73648247, -87.55417668, '(41.736482473, -87.554176677)') ,
(4317595, 'HL611297', '2005-09-14', '055XX S SACRAMENTO AVE', '470', 'PUBLIC PEACE VIOLATION', 'RECKLESS CONDUCT', 'SIDEWALK', False, False, 824, 8, 14.0, 63.0, '24', 1157318.0, 1867853.0, 2005, 41.79314936, -87.69866833, '(41.79314936, -87.69866833)') ,
(8681171, 'HV356389', '2012-06-27', '046XX S LEAMINGTON AVE', '470', 'PUBLIC PEACE VIOLATION', 'RECKLESS CONDUCT', 'STREET', True, False, 814, 8, 23.0, 56.0, '24', 1142805.0, 1873435.0, 2012, 41.80874896, -87.75174831, '(41.80874896, -87.75174831)') ,
(9551405, 'HX203242', '2014-03-29', '063XX S EBERHART AVE', '470', 'PUBLIC PEACE VIOLATION', 'RECKLESS CONDUCT', 'SIDEWALK', True, False, 312, 3, 20.0, 42.0, '24', 1180666.0, 1863182.0, 2014, 41.779826799999995, -87.61319748, '(41.779826802, -87.613197481)') ,
(2075734, 'HH292682', '2002-04-05', '016XX W BRYN MAWR AV', '2850', 'PUBLIC PEACE VIOLATION', 'BOMB THREAT', 'SCHOOL, PRIVATE, BUILDING', False, False, 2012, 20, NULL, NULL, '26', NULL, NULL, 2002, NULL, NULL, nan) ,
(1806896, 'G635735', '2001-10-22', '0000X N HERMITAGE AV', '2850', 'PUBLIC PEACE VIOLATION', 'BOMB THREAT', 'SCHOOL, PUBLIC, BUILDING', True, False, 1333, 12, NULL, NULL, '26', 1164687.0, 1900646.0, 2001, 41.88298425, -87.67071922, '(41.882984246, -87.670719218)') ,
(5766654, 'HN567387', '2007-09-02', '026XX W BELMONT AVE', '1754', 'OFFENSE INVOLVING CHILDREN', 'AGG SEX ASSLT OF CHILD FAM MBR', 'CHURCH/SYNAGOGUE/PLACE OF WORSHIP', False, False, 1411, 14, 1.0, 21.0, '2', 1158166.0, 1921161.0, 2007, 41.93941471, -87.69410322, '(41.939414712, -87.694103222)') ,
(6986273, 'HR391350', '2009-06-23', '015XX S KOLIN AVE', '1753', 'OFFENSE INVOLVING CHILDREN', 'SEX ASSLT OF CHILD BY FAM MBR', 'APARTMENT', False, True, 1012, 10, 24.0, 29.0, '2', 1147638.0, 1892092.0, 2009, 41.85985521, -87.73354382, '(41.859855212, -87.733543816)') ,
(5176248, 'HM768251', '2006-12-11', '076XX S CAMPBELL AVE', '1751', 'OFFENSE INVOLVING CHILDREN', 'CRIM SEX ABUSE BY FAM MEMBER', 'RESIDENCE', True, False, 835, 8, 18.0, 70.0, '20', 1161039.0, 1853860.0, 2006, 41.75467441, -87.6854103, '(41.754674405, -87.685410299)') ,
(8159639, 'HT394616', '2011-07-01', '077XX S SOUTH SHORE DR', '1752', 'OFFENSE INVOLVING CHILDREN', 'AGG CRIM SEX ABUSE FAM MEMBER', 'RESIDENCE', False, False, 421, 4, 7.0, 43.0, '20', 1197175.0, 1854844.0, 2011, 41.75655146, -87.55295175, '(41.756551463, -87.552951751)') ,
(2382981, 'HH688935', '2002-10-03', '057XX S HALSTED ST', '291', 'CRIM SEXUAL ASSAULT', 'ATTEMPT NON-AGGRAVATED', 'ALLEY', False, False, 712, 7, 16.0, 68.0, '2', 1171947.0, 1866895.0, 2002, 41.79021162, -87.64505342, '(41.790211616, -87.645053415)') ,
(3588432, 'HK677648', '2004-10-10', '022XX N KOSTNER AVE', '281', 'CRIM SEXUAL ASSAULT', 'NON-AGGRAVATED', 'APARTMENT', True, False, 2522, 25, 31.0, 20.0, '2', 1146686.0, 1914577.0, 2004, 41.92157476, -87.73646425, '(41.92157476, -87.736464249)') ,
(1455002, 'G158614', '2001-03-18', '059XX S GREEN ST', '261', 'CRIM SEXUAL ASSAULT', 'AGGRAVATED: HANDGUN', 'ALLEY', False, False, 712, 7, NULL, NULL, '2', 1171608.0, 1865691.0, 2001, 41.78691515, -87.64633171, '(41.786915146, -87.646331709)') ,
(3280399, 'HK310265', '2004-04-18', '049XX W FOSTER AVE', '1562', 'SEX OFFENSE', 'AGG CRIMINAL SEXUAL ABUSE', 'CHURCH/SYNAGOGUE/PLACE OF WORSHIP', False, False, 1623, 16, 45.0, 12.0, '17', 1142286.0, 1934118.0, 2004, 41.97527996, -87.75214432, '(41.975279956, -87.752144317)') ,
(6906555, 'HR314298', '2009-05-11', '007XX W 49TH PL', '1585', 'SEX OFFENSE', 'OTHER', 'STREET', True, False, 935, 9, 3.0, 61.0, '17', 1172196.0, 1872045.0, 2009, 41.80433829, -87.64398897, '(41.804338291, -87.64398897)') ,
(8169210, 'HT404326', '2011-07-18', '023XX N MASON AVE', '1562', 'SEX OFFENSE', 'AGG CRIMINAL SEXUAL ABUSE', 'RESIDENCE PORCH/HALLWAY', True, False, 2515, 25, 37.0, 19.0, '17', 1136345.0, 1914858.0, 2011, 41.92253708, -87.77445359, '(41.922537078, -87.774453593)') ,
(10070513, 'HY259084', '2015-05-13', '056XX S WOOD ST', '3710', 'INTERFERENCE WITH PUBLIC OFFICER', 'RESIST/OBSTRUCT/DISARM OFFICER', 'STREET', False, False, 715, 7, 15.0, 67.0, '24', 1165309.0, 1867385.0, 2015, 41.79169949, -87.66937929999999, '(41.791699487, -87.669379304)') ,
(4072241, 'HL415544', '2005-06-12', '031XX W FLOURNOY ST', '3750', 'INTERFERENCE WITH PUBLIC OFFICER', 'ESCAPE', 'POLICE FACILITY/VEH PARKING LOT', True, False, 1134, 11, 24.0, 27.0, '24', 1155475.0, 1896862.0, 2005, 41.87279076, -87.70464797, '(41.872790764, -87.70464797)') ,
(6183080, 'HP265256', '2008-04-06', '013XX N HUDSON AVE', '1661', 'GAMBLING', 'GAME/DICE', 'SIDEWALK', True, False, 1821, 18, 27.0, 8.0, '19', 1173068.0, 1909509.0, 2008, 41.90712313, -87.6396809, '(41.907123131, -87.639680899)') ,
(6987751, 'HR392537', '2009-06-24', '070XX S TALMAN AVE', '1661', 'GAMBLING', 'GAME/DICE', 'SIDEWALK', True, False, 831, 8, 18.0, 66.0, '19', 1159923.0, 1857877.0, 2009, 41.76572065, -87.68938994, '(41.765720645, -87.689389937)') ,
(3987219, 'HL266884', '2005-03-31', '024XX N CLARK ST', '2210', 'LIQUOR LAW VIOLATION', 'SELL/GIVE/DEL LIQUOR TO MINOR', 'CONVENIENCE STORE', True, False, 2333, 19, 43.0, 7.0, '22', 1172680.0, 1916483.0, 2005, 41.92626872, -87.64089934, '(41.926268719, -87.640899336)') ,
(3266814, 'HK238408', '2004-03-13', '093XX S STONY ISLAND AVE', '2230', 'LIQUOR LAW VIOLATION', 'ILLEGAL CONSUMPTION BY MINOR', 'ALLEY', True, False, 413, 4, 8.0, 48.0, '22', 1188539.0, 1843379.0, 2004, 41.72530099, -87.58496589, '(41.72530099, -87.584965887)') ,
(9692995, 'HX343113', '2014-07-13', '132XX S RIVERDALE AVE', '1025', 'ARSON', 'AGGRAVATED', 'RESIDENCE', False, False, 533, 5, 9.0, 54.0, '9', 1182053.0, 1817235.0, 2014, 41.65371054, -87.60952871, '(41.653710537, -87.609528714)') ,
(4607081, 'HM195342', '2006-02-22', '001XX W 118TH ST', '1020', 'ARSON', 'BY FIRE', 'RESIDENCE', False, False, 522, 5, 34.0, 53.0, '9', 1177509.0, 1826708.0, 2006, 41.67980945, -87.62587044, '(41.679809448, -87.625870438)') ,
(21149, 'HW519443', '2013-11-03', '044XX S RICHMOND ST', '110', 'HOMICIDE', 'FIRST DEGREE MURDER', 'HOUSE', True, True, 922, 9, 14.0, 58.0, '01A', 1157439.0, 1875086.0, 2013, 41.81299523, -87.69802859, '(41.812995227, -87.698028592)') ,
(23469, 'JA359626', '2017-07-23', '015XX E 82ND ST', '110', 'HOMICIDE', 'FIRST DEGREE MURDER', 'STREET', False, False, 411, 4, 8.0, 45.0, '01A', 1188090.0, 1850923.0, 2017, 41.74601319, -87.58637073, '(41.746013191, -87.58637073)') ,
(5276766, 'HN144152', '2007-01-26', '050XX W VAN BUREN ST', '1792', 'KIDNAPPING', 'CHILD ABDUCTION/STRANGER', 'STREET', False, False, 1533, 15, 29.0, 25.0, '20', 1143050.0, 1897546.0, 2007, 41.87490841, -87.75024931, '(41.874908413, -87.750249307)') ,
(5738893, 'HN544458', '2007-08-22', '016XX E 87TH ST', '3960', 'INTIMIDATION', 'INTIMIDATION', 'SMALL RETAIL STORE', False, False, 412, 4, 8.0, 45.0, '26', 1188659.0, 1847618.0, 2007, 41.73693038, -87.58439122, '(41.736930377, -87.584391221)') ,
(9094512, 'HW238778', '2013-04-15', '038XX N SPAULDING AVE', '583', 'STALKING', 'CYBERSTALKING', 'RESIDENCE', False, True, 1733, 17, 33.0, 16.0, '08A', 1153706.0, 1925344.0, 2013, 41.95098326, -87.71038332, '(41.950983256, -87.71038332)') ,
(11233321, 'JB155558', '2018-02-16', '036XX W 81ST ST', '1540', 'OBSCENITY', 'OBSCENE MATTER', 'RESIDENCE', True, True, 834, 8, 18.0, 70.0, '17', 1153482.0, 1850636.0, 2018, 41.74598005, -87.71318989, '(41.745980049, -87.713189894)') ,
(10270593, 'HY458571', '2015-10-12', '059XX W CHICAGO AVE', '1480', 'CONCEALED CARRY LICENSE VIOLATION', 'OTHER', 'SIDEWALK', True, False, 1511, 15, 29.0, 25.0, '15', 1136633.0, 1904728.0, 2015, 41.89473398, -87.77363824, '(41.894733978, -87.773638244)') ,
(10626372, 'HZ378869', '2016-08-05', '008XX W 76TH ST', '1536', 'PUBLIC INDECENCY', 'LICENSED PREMISE', 'SIDEWALK', True, False, 621, 6, 17.0, 71.0, '17', 1172201.0, 1854465.0, 2016, 41.75609664, -87.64448713, '(41.756096643, -87.644487127)') ,
(10625887, 'HZ378355', '2016-08-02', '054XX N ELSTON AVE', '5093', 'NON-CRIMINAL', 'LOST PASSPORT', 'GOVERNMENT BUILDING/PROPERTY', False, False, 1621, 16, 45.0, 11.0, '26', 1140460.0, 1935678.0, 2016, 41.97959452, -87.75882076, '(41.97959452, -87.758820763)') ,
(9608166, 'HX258445', '2014-05-12', '010XX N WELLS ST', '1900', 'OTHER NARCOTIC VIOLATION', 'INTOXICATING COMPOUNDS', 'SIDEWALK', True, False, 1823, 18, 27.0, 8.0, '18', 1174535.0, 1907194.0, 2014, 41.90073798, -87.63436132, '(41.900737976, -87.634361322)') ,
(10453948, 'HZ192829', '2016-03-01', '028XX E 79TH ST', '1055', 'HUMAN TRAFFICKING', 'INVOLUNTARY SERVITUDE', 'APARTMENT', False, False, 422, 4, 7.0, 46.0, '26', 1196679.0, 1853139.0, 2016, 41.75188515, -87.554826, '(41.751885152, -87.554825997)') ,
(10397129, 'HZ133234', '2016-01-29', '006XX W HARRISON ST', '5114', 'NON - CRIMINAL', 'FOID - REVOCATION', 'STREET', False, False, 124, 1, 2.0, 28.0, '26', 1172257.0, 1897564.0, 2016, 41.87436328, -87.64301304, '(41.874363279, -87.643013039)') ,
(3269495, 'HJ747227', '2003-11-08', '012XX W 81ST ST', '510', 'RITUALISM', 'AGG RIT MUT: HANDS/FIST/FEET SERIOUS INJURY', 'OTHER', False, False, 612, 6, 21.0, 71.0, '04B', 1169648.0, 1851076.0, 2003, 41.74685249, -87.65394139, '(41.746852486, -87.653941385)') ,
(10840565, 'JA143710', '2017-02-06', '009XX N KARLOV AVE', '5073', 'NON-CRIMINAL (SUBJECT SPECIFIED)', 'NOTIFICATION OF CIVIL NO CONTACT ORDER', 'RESIDENCE', False, True, 1111, 11, 37.0, 23.0, '26', 1148881.0, 1905963.0, 2017, 41.89789489, -87.72862232, '(41.89789489, -87.728622316)') ,
(1326195, 'G021609', '2001-01-11', '087XX S ESCANABA AV', '9901', 'DOMESTIC VIOLENCE', 'DOMESTIC VIOLENCE', 'APARTMENT', True, True, 423, 4, NULL, NULL, '08B', 1196869.0, 1847416.0, 2001, 41.73617608, -87.55431961, '(41.73617608, -87.554319607)');

 * ibm_db_sa://bvr08213:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
533 rows affected.


[]

In [30]:
#print sample insert statement
print(school_data_insert[:4000])

INSERT INTO PUBLIC_SCHOOLS (School_ID, NAME_OF_SCHOOL, Elementary_Middle_or_High_School, Street_Address, City, State, ZIP_Code, Phone_Number, Link, Network_Manager, Collaborative_Name, Adequate_Yearly_Progress_Made_, Track_Schedule, CPS_Performance_Policy_Status, CPS_Performance_Policy_Level, HEALTHY_SCHOOL_CERTIFIED, Safety_Icon, SAFETY_SCORE, Family_Involvement_Icon, Family_Involvement_Score, Environment_Icon, Environment_Score, Instruction_Icon, Instruction_Score, Leaders_Icon, Leaders_Score, Teachers_Icon, Teachers_Score, Parent_Engagement_Icon, Parent_Engagement_Score, Parent_Environment_Icon, Parent_Environment_Score, AVERAGE_STUDENT_ATTENDANCE, Rate_of_Misconducts__per_100_students_, Average_Teacher_Attendance, Individualized_Education_Program_Compliance_Rate, Pk_2_Literacy__, Pk_2_Math__, Gr3_5_Grade_Level_Math__, Gr3_5_Grade_Level_Read__, Gr3_5_Keep_Pace_Read__, Gr3_5_Keep_Pace_Math__, Gr6_8_Grade_Level_Math__, Gr6_8_Grade_Level_Read__, Gr6_8_Keep_Pace_Math_, Gr6_8_Keep_Pace_R

In [31]:
# print sample insert statement with corrections and line spacing for readability
d="""INSERT INTO PUBLIC_SCHOOLS (School_ID, NAME_OF_SCHOOL, Elementary_Middle_or_High_School, Street_Address, City, State, ZIP_Code, Phone_Number, Link, Network_Manager, Collaborative_Name, Adequate_Yearly_Progress_Made_, Track_Schedule, CPS_Performance_Policy_Status, CPS_Performance_Policy_Level, HEALTHY_SCHOOL_CERTIFIED, Safety_Icon, SAFETY_SCORE, Family_Involvement_Icon, Family_Involvement_Score, Environment_Icon, Environment_Score, Instruction_Icon, Instruction_Score, Leaders_Icon, Leaders_Score, Teachers_Icon, Teachers_Score, Parent_Engagement_Icon, Parent_Engagement_Score, Parent_Environment_Icon, Parent_Environment_Score, AVERAGE_STUDENT_ATTENDANCE, Rate_of_Misconducts__per_100_students_, Average_Teacher_Attendance, Individualized_Education_Program_Compliance_Rate, Pk_2_Literacy__, Pk_2_Math__, Gr3_5_Grade_Level_Math__, Gr3_5_Grade_Level_Read__, Gr3_5_Keep_Pace_Read__, Gr3_5_Keep_Pace_Math__, Gr6_8_Grade_Level_Math__, Gr6_8_Grade_Level_Read__, Gr6_8_Keep_Pace_Math_, Gr6_8_Keep_Pace_Read__, Gr_8_Explore_Math__, Gr_8_Explore_Read__, ISAT_Exceeding_Math__, ISAT_Exceeding_Reading__, ISAT_Value_Add_Math, ISAT_Value_Add_Read, ISAT_Value_Add_Color_Math, ISAT_Value_Add_Color_Read, Students_Taking__Algebra__, Students_Passing__Algebra__, Gr_9_EXPLORE_2009, Gr_9_EXPLORE_2010, Gr_10_PLAN_2009, Gr_10_PLAN_2010, Net_Change_EXPLORE_and_PLAN, Gr_11_Average_ACT_2011, Net_Change_PLAN_and_ACT, College_Eligibility__, Graduation_Rate__, College_Enrollment_Rate__, COLLEGE_ENROLLMENT, General_Services_Route, Freshman_on_Track_Rate__, X_COORDINATE, Y_COORDINATE, Latitude, Longitude, COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, Ward, Police_District, Location) \n VALUES (610038, 'Abraham Lincoln Elementary School', 'ES', '615 W Kemper Pl', 'Chicago', 'IL', 60614, '(773) 534-5720', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'Yes', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Strong', 74.0, 'Strong', 66.0, 'Weak', 65.0, 'Strong', 70.0, 'Strong', 56.0, 'Average', 47.0, '96.00%', 2.0, '96.40%', '95.80%', 80.1, 43.3, 89.6, 84.9, 60.7, 62.6, 81.9, 85.2, 52.0, 62.4, 66.3, 77.9, 69.7, 64.4, 0.2, 0.9, 'Yellow', 'Green', 67.1, 54.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 813, 33, nan, 1171699.4579999999, 1915829.428, 41.92449696, -87.64452163, 7, 'LINCOLN PARK', 43, 18, '(41.92449696, -87.64452163)') ,\n(610281, 'Adam Clayton Powell Paideia Community Academy Elementary School', 'ES', '7511 S South Shore Dr', 'Chicago', 'IL', 60649, '(773) 535-6650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 54.0, 'Strong', 66.0, 'Strong', 74.0, 'Very Strong', 84.0, 'Weak', 63.0, 'Strong', 76.0, 'Weak', 46.0, 'Average', 50.0, '95.60%', 15.7, '95.30%', '100.00%', 62.4, 51.7, 21.9, 15.1, 29.0, 42.8, 38.5, 27.4, 44.8, 42.7, 14.1, 34.4, 16.8, 16.5, 0.7, 1.4, 'Green', 'Green', 17.2, 27.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 521, 46, nan, 1196129.985, 1856209.466, 41.76032435, -87.55673627, 43, 'SOUTH SHORE', 7, 4, '(41.76032435, -87.55673627)') ,\n(610185, 'Adlai E Stevenson Elementary School', 'ES', '8010 S Kostner Ave', 'Chicago', 'IL', 60652, '(773) 535-2280', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610185.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 61.0, nan, nan, 'Average', 50.0, 'Weak', 36.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Weak', 41.0, '95.70%', 2.3, '94.70%', '98.30%', 53.7, 26.6, 38.3, 34.7, 43.7, 57.3, 48.8, 39.2, 46.8, 44.0, 7.5, 21.9, 18.3, 15.5, -0.9, -1.0, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1324, 44, nan, 1148427.165, 1851012.215, 41.74711093, -87.73170248, 70, 'ASHBURN', 13, 8, '(41.74711093, -87.73170248)') ,\n(609993, 'Agustin Lara Elementary Academy', 'ES', '4619 S Wolcott Ave', 'Chicago', 'IL', 60609, '(773) 535-4389', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609993.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 56.0, 'Average', 44.0, 'Average', 45.0, 'Weak', 37.0, 'Weak', 65.0, 'Average', 48.0, 'Average', 53.0, 'Strong', 58.0, '95.50%', 10.4, '95.80%', '100.00%', 76.9, nan, 26.0, 24.7, 61.8, 49.7, 39.2, 27.2, 69.7, 60.6, 9.1, 18.2, 11.1, 9.6, 0.9, 2.4, 'Green', 'Green', 42.9, 25.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 556, 42, nan, 1164504.29, 1873959.199, 41.8097569, -87.6721446, 61, 'NEW CITY', 20, 9, '(41.8097569, -87.6721446)') ,\n(610513, 'Air Force Academy High School', 'HS', '3630 S Wells St', 'Chicago', 'IL', 60609, '(773) 535-1590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610513.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', nan, 'Standard', 'Not on Probation', 'Not Enough Data', 'Yes', 'Average', 49.0, 'Strong', 60.0, 'Strong', 60.0, 'Average', 55.0, 'Weak', 45.0, 'Average', 54.0, 'Average', 53.0, 'Average', 49.0, '93.30%', 15.6, '96.90%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.6, 14.8, nan, 16.0, 1.4, nan, nan, nan, nan, nan, 302, 40, 91.8, 1175177.622, 1880745.126, 41.82814609, -87.63279369, 34, 'ARMOUR SQUARE', 11, 9, '(41.82814609, -87.63279369)') ,\n(610212, 'Albany Park Multicultural Academy', 'MS', '4929 N Sawyer Ave', 'Chicago', 'IL', 60625, '(773) 534-5108', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610212.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, 'Weak', 37.0, 'Strong', 66.0, 'Strong', 71.0, 'Weak', 43.0, 'Average', 50.0, 'Weak', 46.0, 'Average', 51.0, '97.00%', 2.3, '96.90%', '100.00%', nan, nan, nan, nan, nan, nan, 60.7, 39.8, 53.7, 59.8, 17.5, 20.8, 34.5, 15.6, 0.2, 0.3, 'Yellow', 'Yellow', 29.2, 50.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 266, 31, nan, 1153858.196, 1932691.8909999998, 41.971143299999994, -87.70962725, 14, 'ALBANY PARK', 39, 17, '(41.9711433, -87.70962725)') ,\n(609720, 'Albert G Lane Technical High School', 'HS', '2501 W Addison St', 'Chicago', 'IL', 60618, '(773) 534-5400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609720.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 88.0, nan, nan, 'Strong', 62.0, 'Average', 52.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '96.30%', 2.1, '96.20%', '99.40%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 19.1, 19.5, 19.9, 20.1, 1.0, 23.4, 3.5, 67.9, 92.2, 79.8, 4368, 35, 90.7, 1158975.392, 1923791.705, 41.94661693, -87.69105603, 5, 'NORTH CENTER', 47, 19, '(41.94661693, -87.69105603)') ,\n(610342, 'Albert R Sabin Elementary Magnet School', 'ES', '2216 W Hirsch St', 'Chicago', 'IL', 60622, '(773) 534-4491', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610342.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 67.0, nan, nan, 'Weak', 30.0, 'Very Weak', 18.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.70%', 28.1, '95.00%', '100.00%', 56.3, nan, 33.5, 33.5, 49.5, 49.5, 27.4, 39.3, 35.5, 44.4, 25.0, 23.4, 18.0, 12.8, -1.8, 0.1, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 620, 35, nan, 1161265.299, 1909314.592, 41.90684338, -87.68304259, 24, 'WEST TOWN', 1, 14, '(41.90684338, -87.68304259)') ,\n(610524, 'Alcott High School for the Humanities', 'HS', '2957 N Hoyne Ave', 'Chicago', 'IL', 60618, '(773) 534-5979', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610524.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', nan, 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Strong', 70.0, nan, nan, 'Strong', 67.0, 'Average', 51.0, 'Weak', nan, nan, nan, 'Strong', 57.0, 'Weak', 43.0, '92.70%', 7.1, '96.90%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.7, 13.7, nan, 16.0, 1.3, nan, nan, nan, nan, nan, 232, 33, 87.6, 1161870.5559999999, 1919857.44, 41.93576106, -87.68052441, 5, 'NORTH CENTER', 1, 19, '(41.93576106, -87.68052441)') ,\n(610209, 'Alessandro Volta Elementary School', 'ES', '4950 N Avers Ave', 'Chicago', 'IL', 60625, '(773) 534-5080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610209.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 43.0, 'Strong', 61.0, 'Weak', 28.0, 'Weak', 37.0, 'Weak', 62.0, 'Average', 56.0, 'Average', 51.0, 'Average', 53.0, '96.40%', 22.5, '95.90%', '100.00%', 63.9, 43.2, 51.3, 32.9, 50.7, 70.0, 52.5, 31.3, 59.8, 57.6, 16.5, 24.7, 19.9, 14.2, 0.3, -0.4, 'Yellow', 'Yellow', 31.6, 65.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1023, 31, nan, 1149774.095, 1932831.1509999998, 41.97160605, -87.72464139, 14, 'ALBANY PARK', 39, 17, '(41.97160605, -87.72464139)') ,\n(609799, 'Alexander Graham Bell Elementary School', 'ES', '3730 N Oakley Ave', 'Chicago', 'IL', 60618, '(773) 534-5150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609799.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 88.0, 'Strong', 64.0, 'Average', 46.0, 'Weak', 51.0, 'Average', 51.0, nan, nan, nan, nan, '96.30%', 6.3, '95.90%', '99.30%', 91.9, 67.3, 79.2, 77.4, 53.3, 54.7, 84.2, 83.0, 49.8, 53.6, 62.6, 71.7, 64.0, 57.9, 0.0, 0.3, 'Yellow', 'Yellow', 58.1, 65.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 998, 35, nan, 1160327.881, 1924862.7219999998, 41.94952795, -87.68605496, 5, 'NORTH CENTER', 47, 19, '(41.94952795, -87.68605496)') ,\n(609947, 'Alexander Graham Elementary School', 'ES', '4436 S Union Ave', 'Chicago', 'IL', 60609, '(773) 535-1308', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609947.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 49.0, 'Weak', 30.0, 'Weak', 31.0, 'Weak', 33.0, 'Weak', 30.0, 'Average', 41.0, 'Average', 48.0, 'Average', 51.0, '92.50%', 27.4, '95.00%', '92.10%', 76.7, 47.4, 59.4, 38.8, 51.6, 76.2, 44.4, 31.3, 66.7, 49.3, 5.9, 35.3, 20.8, 11.1, 1.7, -0.5, 'Green', 'Yellow', 45.3, 31.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 476, 42, nan, 1172335.702, 1875308.997, 41.81329195, -87.64338051, 61, 'NEW CITY', 11, 9, '(41.81329195, -87.64338051)') ,\n(609963, 'Alexander Hamilton Elementary School', 'ES', '1650 W Cornelia Ave', 'Chicago', 'IL', 60657, '(773) 534-5484', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609963.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 73.0, 'Strong', 64.0, 'Strong', 60.0, 'Average', 59.0, 'Weak', 59.0, 'Average', 48.0, 'Strong', 57.0, 'Strong', 54.0, '95.30%', 12.5, '97.40%', '97.40%', 70.3, 47.2, 34.4, 48.4, 56.5, 62.9, 31.9, 46.8, 48.9, 51.1, 14.3, 35.7, 13.8, 21.3, 0.6, 0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 307, 33, nan, 1164474.274, 1923334.414, 41.94524725, -87.67085683, 6, 'LAKE VIEW', 32, 19, '(41.94524725, -87.67085683)') ,\n(610210, 'Alexander von Humboldt Elementary School', 'ES', '2620 W Hirsch St', 'Chicago', 'IL', 60622, '(773) 534-4480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610210.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 31.0, nan, nan, 'Weak', 32.0, 'Average', 45.0, 'Weak', nan, nan, nan, 'Weak', 43.0, 'Weak', 44.0, '92.50%', 185.5, '96.00%', '100.00%', nan, 32.6, 23.6, 24.0, 56.3, 54.0, 31.8, 19.7, 60.7, 39.7, 9.1, 20.5, 8.6, 5.7, -0.4, -1.2, 'Yellow', 'Red', 14.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 398, 34, nan, 1158554.232, 1909250.021, 41.90672216, -87.69300325, 24, 'WEST TOWN', 26, 14, '(41.90672216, -87.69300325)') ,\n(609808, 'Alex Haley Elementary Academy', 'ES', '11411 S Eggleston Ave', 'Chicago', 'IL', 60628, '(773) 535-5340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609808.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 28.0, nan, nan, 'Average', 58.0, 'Strong', 60.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 49.0, '94.90%', 2.9, '94.70%', '94.70%', 51.6, 31.4, 17.3, 13.6, 43.7, 41.4, 36.3, 24.5, 64.8, 53.9, 1.8, 16.1, 11.6, 1.9, 0.5, -1.5, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 525, 48, nan, 1175378.7140000002, 1829178.573, 41.68663682, -87.63359476, 49, 'ROSELAND', 34, 22, '(41.68663682, -87.63359476)') ,\n(610028, 'Alfred David Kohn Elementary School', 'ES', '10414 S State St', 'Chicago', 'IL', 60628, '(773) 535-5489', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610028.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 19.0, 'Weak', 26.0, 'Weak', 22.0, 'Very Weak', 13.0, 'Weak', 21.0, 'Very Weak', 19.0, 'Average', 47.0, 'Weak', 45.0, '90.10%', 31.2, '94.20%', '96.40%', 41.3, 31.9, 34.0, 22.2, 42.8, 56.2, 29.6, 22.8, 80.0, 61.7, 4.3, 19.6, 5.0, 5.6, 0.5, -0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 428, 48, nan, 1178053.498, 1835837.822, 41.70485072, -87.62360199, 49, 'ROSELAND', 34, 5, '(41.70485072, -87.62360199)') ,\n(610098, 'Alfred Nobel Elementary School', 'ES', '4127 W Hirsch St', 'Chicago', 'IL', 60651, '(773) 534-4365', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610098.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 37.0, 'Weak', 34.0, 'Weak', 37.0, 'Weak', 35.0, 'Weak', 48.0, 'Weak', 36.0, 'Average', 48.0, 'Average', 51.0, '94.60%', 24.8, '95.20%', '100.00%', 55.4, 33.2, 26.7, 23.2, 47.3, 50.0, 39.2, 28.1, 59.5, 51.0, 15.2, 16.7, 9.7, 7.1, -0.3, 0.7, 'Yellow', 'Yellow', 23.4, 44.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 801, 34, nan, 1148503.002, 1908960.619, 41.90612797, -87.72993324, 23, 'HUMBOLDT PARK', 37, 25, '(41.90612797, -87.72993324)') ,\n(609788, 'Alice L Barnard Computer Math & Science Center Elementary School', 'ES', '10354 S Charles St', 'Chicago', 'IL', 60643, '(773) 535-2625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609788.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 46.0, nan, nan, 'Weak', 39.0, 'Average', 58.0, 'Weak', nan, nan, nan, 'Strong', 69.0, 'Average', 51.0, '95.60%', 28.5, '95.00%', '100.00%', 62.1, 38.5, 43.0, 38.8, 47.4, 61.8, 45.6, 40.4, 49.4, 61.4, 18.5, 29.6, 18.0, 15.7, 1.9, 1.9, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 274, 49, nan, 1168627.899, 1835867.534, 41.70514024, -87.65811642, 72, 'BEVERLY', 19, 22, '(41.70514024, -87.65811642)') ,\n(610334, 'Al Raby High School', 'HS', '3545 W Fulton Blvd', 'Chicago', 'IL', 60624, '(773) 534-6755', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610334.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 52.0, nan, nan, 'Average', 51.0, 'Average', 53.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '80.30%', 14.0, '95.10%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.8, 13.0, 14.1, 14.0, 1.2, 15.2, 1.1, 13.5, 72.2, 64.2, 551, 34, 74.3, 1152512.295, 1901720.85, 41.886183, -87.71539705, 27, 'EAST GARFIELD PARK', 28, 11, '(41.886183, -87.71539705)') ,\n(610131, 'Ambrose Plamondon Elementary School', 'ES', '2642 W 15th Pl', 'Chicago', 'IL', 60608, '(773) 534-1789', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610131.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 45.0, nan, nan, 'Weak', 32.0, 'Weak', 28.0, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Weak', 41.0, '96.90%', 0.6, '96.60%', '100.00%', 61.9, 14.3, 27.3, 37.9, 35.0, 54.8, 43.5, 25.0, 74.4, 40.0, 16.7, 25.0, 12.3, 11.4, -1.5, -1.3, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 171, 39, nan, 1158816.3020000001, 1892329.996, 41.86028668, -87.69250468, 29, 'NORTH LAWNDALE', 28, 10, '(41.86028668, -87.69250468)') ,\n(610316, 'Amelia Earhart Options for Knowledge Elementary School', 'ES', '1710 E 93rd St', 'Chicago', 'IL', 60617, '(773) 535-6416', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610316.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Very Strong', 81.0, 'Strong', 63.0, 'Strong', 60.0, 'Strong', 73.0, 'Weak', 34.0, 'Average', 54.0, nan, nan, nan, nan, '96.30%', 33.4, '95.40%', '100.00%', 76.7, nan, 58.1, 59.1, 66.7, 58.1, 48.8, 67.5, 51.3, 62.5, nan, nan, 27.5, 26.4, 0.1, 1.0, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 264, 47, nan, 1189297.5890000002, 1843689.5269999998, 41.72613498, -87.58217727, 48, 'CALUMET HEIGHTS', 8, 4, '(41.72613498, -87.58217727)') ,\n(609780, 'Ames Middle School', 'MS', '1920 N Hamlin Ave', 'Chicago', 'IL', 60647, '(773) 534-4970', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609780.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 38.0, 'Very Weak', 17.0, 'Weak', 27.0, 'Weak', 35.0, 'Weak', 26.0, 'Weak', 26.0, 'Average', 50.0, 'Average', 50.0, '94.30%', 5.3, '96.00%', '100.00%', nan, nan, nan, nan, nan, nan, 35.1, 27.9, 76.8, 51.1, 9.2, 18.4, 14.5, 3.0, -1.9, -1.5, 'Red', 'Red', 28.8, 55.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 586, 34, nan, 1150696.779, 1912593.823, 41.91605519, -87.72177945, 22, 'LOGAN SQUARE', 26, 25, '(41.91605519, -87.72177945)') ,\n(610339, 'Amos Alonzo Stagg Elementary School', 'ES', '7424 S Morgan St', 'Chicago', 'IL', 60621, '(773) 535-3565', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610339.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 23.0, 'Weak', 35.0, 'Weak', 35.0, 'Average', 42.0, 'Weak', 25.0, 'Average', 42.0, 'Weak', 44.0, 'Average', 50.0, '91.40%', 34.3, '95.50%', '100.00%', nan, nan, 35.3, 22.4, 47.6, 58.2, 20.1, 26.8, 49.2, 43.9, 8.0, 23.5, 7.7, 7.8, -0.4, -0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 539, 45, nan, 1170892.32, 1855485.8669999999, 41.75892669, -87.6492534, 68, 'ENGLEWOOD', 17, 7, '(41.75892669, -87.6492534)') ,\n(610320, 'Ana Roque de Duprey Elementary School', 'ES', '2620 W Hirsch St', 'Chicago', 'IL', 60622, '(773) 534-4230', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610320.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'Yes', 'Average', 57.0, nan, nan, 'Very Weak', 12.0, 'Very Weak', 14.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Strong', 57.0, '95.40%', 77.5, '95.60%', '100.00%', 32.1, 23.3, 56.8, 32.4, 75.8, 75.8, 34.9, 34.1, 53.8, 52.5, 7.1, 28.6, 11.1, 6.2, 0.7, 0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 114, 34, nan, 1158554.232, 1909250.021, 41.90672216, -87.69300325, 24, 'WEST TOWN', 26, 14, '(41.90672216, -87.69300325)') ,\n(609837, 'Andrew Carnegie Elementary School', 'ES', '1414 E 61st Pl', 'Chicago', 'IL', 60637, '(773) 535-0530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609837.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 48.0, 'Average', 51.0, 'Weak', 37.0, 'Strong', 63.0, 'Weak', 42.0, 'Weak', 31.0, 'Average', 49.0, 'Average', 49.0, '94.60%', 6.2, '94.40%', '100.00%', nan, nan, 40.2, 48.3, 40.8, 52.3, 29.8, 44.7, 32.6, 36.2, 18.4, 38.8, 21.4, 20.4, -1.2, 0.1, 'Red', 'Yellow', 48.1, 8.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 730, 46, nan, 1186764.646, 1864534.4730000002, 41.78339577, -87.59079649, 42, 'WOODLAWN', 20, 3, '(41.78339577, -87.59079649)') ,\n(610060, 'Andrew Jackson Elementary Language Academy', 'ES', '1340 W Harrison St', 'Chicago', 'IL', 60607, '(773) 534-7000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610060.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 86.0, nan, nan, 'Average', 57.0, 'Average', 46.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '97.40%', 0.7, '96.10%', '100.00%', 88.7, nan, 90.3, 90.9, 70.6, 81.2, 85.7, 90.7, 64.8, 62.1, 72.6, 87.1, 70.4, 62.7, 0.6, 0.8, 'Green', 'Green', 48.4, 80.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 556, 38, nan, 1167364.912, 1897512.032, 41.87432724, -87.66097599, 28, 'NEAR WEST SIDE', 2, 12, '(41.87432724, -87.66097599)') ,\n(609869, 'Anna R. Langford Community Academy', 'ES', '6010 S Throop St', 'Chicago', 'IL', 60636, '(773) 535-9180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609869.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 31.0, 'Weak', 31.0, 'Average', 54.0, 'Weak', 32.0, 'Weak', 49.0, 'Average', 50.0, 'Average', 53.0, 'Average', 52.0, '91.80%', 29.0, '95.40%', '100.00%', 58.8, nan, 40.0, 14.0, 52.8, 59.8, 31.9, 17.0, 51.3, 44.9, 7.7, 11.5, 12.6, 7.5, 0.5, 0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 339, 43, nan, 1168649.029, 1864844.4630000002, 41.78465652, -87.6572053, 67, 'WEST ENGLEWOOD', 16, 7, '(41.78465652, -87.6572053)') ,\n(610084, 'Annie Keller Elementary Gifted Magnet School', 'ES', '3020 W 108th St', 'Chicago', 'IL', 60655, '(773) 535-2636', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610084.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 97.0, 'Very Strong', 85.0, 'Very Strong', 82.0, 'Weak', 94.0, 'Very Strong', 82.0, 'Strong', 68.0, 'Strong', 60.0, '97.50%', 4.9, '96.50%', '100.00%', 100.0, nan, 100.0, 100.0, 63.4, 74.7, 96.5, 97.7, 61.2, 76.7, 89.3, 100.0, 92.8, 92.3, 1.6, 2.3, 'Green', 'Green', 51.5, 70.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 245, 49, nan, 1157959.455, 1832892.0669999998, 41.69719792, -87.69726379999999, 74, 'MOUNT GREENWOOD', 19, 22, '(41.69719792, -87.6972638)') ,\n(610229, 'A.N. Pritzker School', 'ES', '2009 W Schiller St', 'Chicago', 'IL', 60622, '(773) 534-4415', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610229.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 59.0, nan, nan, 'Average', 58.0, 'Strong', 61.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.60%', 36.5, '95.60%', '98.50%', 56.8, 11.8, 58.2, 58.5, 55.0, 55.6, 62.9, 54.9, 57.6, 55.1, 38.7, 50.0, 37.2, 34.5, -0.2, 0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 669, 35, nan, 1162693.096, 1909391.648, 41.907025, -87.67779555, 24, 'WEST TOWN', 1, 14, '(41.907025, -87.67779555)') ,\n(610277, 'Anthony Overton Elementary School', 'ES', '221 E 49th St', 'Chicago', 'IL', 60615, '(773) 535-1430', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610277.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 32.0, nan, nan, 'Strong', 64.0, 'Strong', 76.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Strong', 55.0, '90.50%', 24.9, '94.90%', '100.00%', 52.0, 54.8, 19.6, 16.2, 41.1, 40.0, 20.4, 25.0, 45.1, 46.2, 0.0, 8.3, 8.2, 4.3, 1.2, 0.7, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 436, 42, nan, 1178699.353, 1872549.0259999998, 41.80557578, -87.62012252, 38, 'GRAND BOULEVARD', 3, 2, '(41.80557578, -87.62012252)') ,\n(609951, 'Ariel Elementary Community Academy', 'ES', '1119 E 46th St', 'Chicago', 'IL', 60653, '(773) 535-1996', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609951.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 51.0, 'Average', 50.0, '95.50%', 5.4, '96.00%', '100.00%', 70.8, 34.3, 48.4, 43.0, 62.0, 58.6, 40.0, 47.1, 58.6, 49.3, 14.6, 36.6, 24.8, 14.4, 0.0, -1.6, 'Yellow', 'Red', 35.0, 28.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 550, 42, nan, 1184576.517, 1874697.119, 41.81133445, -87.5985003, 39, 'KENWOOD', 4, 2, '(41.81133445, -87.5985003)') ,\n(610161, 'Arna Wendell Bontemps Elementary School', 'ES', '1241 W 58th St', 'Chicago', 'IL', 60636, '(773) 535-9175', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610161.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 33.0, nan, nan, 'Average', 48.0, 'Average', 59.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Average', 52.0, '94.20%', 31.2, '95.80%', '100.00%', 36.4, 26.9, 25.7, 20.8, 64.3, 64.8, 28.7, 22.1, 54.3, 51.9, 0.0, 18.8, 7.5, 4.2, -1.1, -2.5, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 345, 43, nan, 1168872.4209999999, 1866273.4440000001, 41.78857298, -87.65634499, 67, 'WEST ENGLEWOOD', 16, 7, '(41.78857298, -87.65634499)') ,\n(610171, 'Arnold Mireles Elementary Academy', 'ES', '9000 S Exchange Ave', 'Chicago', 'IL', 60617, '(773) 535-6360', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610171.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 27.0, nan, nan, 'Average', 43.0, 'Average', 54.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 51.0, '92.80%', 11.8, '95.20%', '98.90%', 44.0, 27.5, 21.9, 15.2, 45.4, 45.7, 23.0, 19.4, 46.9, 51.0, 6.4, 18.2, 7.6, 5.4, -1.1, -0.7, 'Red', 'Red', 9.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 905, 47, nan, 1197292.129, 1845782.158, 41.73168217, -87.55282373, 46, 'SOUTH CHICAGO', 10, 4, '(41.73168217, -87.55282373)') ,\n(610037, 'Arthur A Libby Elementary School', 'ES', '5300 S Loomis Blvd', 'Chicago', 'IL', 60609, '(773) 535-9050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610037.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, nan, nan, 'Strong', 60.0, 'Strong', 60.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 52.0, '90.80%', 21.6, '95.10%', '100.00%', 31.1, 37.3, 24.7, 12.0, 43.2, 48.3, 32.0, 17.7, 55.3, 48.3, 4.2, 14.6, 4.8, 3.9, -0.4, -2.8, 'Yellow', 'Red', 18.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 539, 43, nan, 1167858.322, 1869572.5019999999, 41.79764784, -87.65996858, 61, 'NEW CITY', 16, 9, '(41.79764784, -87.65996858)') ,\n(609887, 'Arthur Dixon Elementary School', 'ES', '8306 S Saint Lawrence Ave', 'Chicago', 'IL', 60619, '(773) 535-3834', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609887.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 47.0, nan, nan, 'Average', 47.0, 'Average', 52.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.80%', 32.2, '95.60%', '100.00%', 69.7, 35.5, 42.8, 37.8, 51.8, 70.8, 43.5, 44.5, 51.3, 49.4, 22.1, 31.6, 22.0, 15.7, -0.3, 0.7, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 652, 45, nan, 1181644.805, 1850009.5769999998, 41.74365776, -87.61001515, 44, 'CHATHAM', 6, 6, '(41.74365776, -87.61001515)') ,\n(609836, 'Arthur E Canty Elementary School', 'ES', '3740 N Panama Ave', 'Chicago', 'IL', 60634, '(773) 534-1238', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609836.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, 'Strong', 64.0, 'Strong', 70.0, 'Strong', 64.0, 'Weak', 49.0, 'Average', 47.0, 'Strong', 66.0, 'Average', 53.0, '96.10%', 2.3, '96.10%', '99.10%', 75.3, 60.6, 77.2, 58.3, 52.9, 62.0, 81.3, 71.3, 61.4, 59.7, 41.3, 60.0, 47.4, 34.2, 0.3, 0.1, 'Yellow', 'Yellow', 29.8, 100.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 784, 30, nan, 1121355.6979999999, 1924041.4119999998, 41.94799245, -87.8293319, 17, 'DUNNING', 36, 16, '(41.94799245, -87.8293319)') ,\n(610268, 'Arthur R Ashe Elementary School', 'ES', '8505 S Ingleside Ave', 'Chicago', 'IL', 60619, '(773) 535-3550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610268.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Weak', 39.0, 'Weak', 46.0, '87.90%', 41.6, '93.40%', '100.00%', 30.1, 25.5, 20.3, 14.3, 47.4, 54.8, 43.4, 23.6, 61.0, 58.7, 7.9, 15.8, 9.2, 2.8, 1.1, -0.9, 'Green', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 459, 45, nan, 1184078.6609999998, 1848752.991, 41.74015307, -87.60113659, 44, 'CHATHAM', 8, 6, '(41.74015307, -87.60113659)') ,\n(609941, 'Asa Philip Randolph Elementary School', 'ES', '7316 S Hoyne Ave', 'Chicago', 'IL', 60636, '(773) 535-9015', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609941.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 15.0, nan, nan, 'Average', 41.0, 'Average', 48.0, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Average', 49.0, '92.10%', 43.5, '95.30%', '95.60%', 52.9, 52.6, 20.2, 11.3, 40.1, 48.1, 44.2, 21.3, 63.0, 50.8, 12.1, 12.1, 13.8, 4.5, 0.3, -1.3, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 573, 43, nan, 1163589.689, 1856037.765, 41.76059741, -87.67600175, 67, 'WEST ENGLEWOOD', 17, 7, '(41.76059741, -87.67600175)') ,\n(610287, 'Ashburn Community Elementary School', 'ES', '8300 S Saint Louis Ave', 'Chicago', 'IL', 60652, '(773) 535-7860', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610287.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 48.0, 'Strong', 76.0, 'Average', 52.0, 'Average', 40.0, 'Weak', 68.0, 'Strong', 70.0, 'Average', 51.0, 'Average', 50.0, '95.50%', 28.5, '97.10%', '100.00%', 70.6, 61.7, 40.4, 33.3, 55.5, 56.2, 45.8, 36.8, 69.9, 45.5, 22.4, 30.6, 15.1, 9.5, 0.4, -0.6, 'Yellow', 'Yellow', 40.8, 40.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 490, 44, nan, 1154458.76, 1849313.602, 41.74233179, -87.70964592, 70, 'ASHBURN', 18, 8, '(41.74233179, -87.70964592)') ,\n(609820, 'Augustus H Burley Elementary School', 'ES', '1630 W Barry Ave', 'Chicago', 'IL', 60657, '(773) 534-5475', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609820.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, nan, nan, 'Strong', 78.0, 'Strong', 65.0, 'Weak', nan, nan, nan, 'Strong', 59.0, 'Average', 49.0, '96.50%', 0.7, '95.00%', '97.90%', 69.4, 47.0, 64.5, 70.2, 51.2, 66.7, 69.4, 75.0, 62.9, 67.3, 50.0, 69.6, 54.2, 53.3, 1.3, 2.1, 'Green', 'Green', 35.7, 80.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 572, 33, nan, 1164768.164, 1920682.831, 41.93796493, -87.66985204, 6, 'LAKE VIEW', 32, 19, '(41.93796493, -87.66985204)') ,\n(400018, 'Austin Business and Entrepreneurship Academy High School', 'HS', '231 N Pine Ave', 'Chicago', 'IL', 60644, '(773) 534-6316', nan, 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Non_Standard', 'Probation', 'Level 3', 'No', 'Weak', 38.0, nan, nan, 'Average', 41.0, 'Weak', 33.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '76.00%', 12.4, '0.00%', '98.20%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.0, 12.0, 14.3, 12.9, 0.9, 14.4, 0.1, 12.2, 65.6, 51.3, 316, 36, nan, 1139494.763, 1901274.2580000001, 41.88520477, -87.76321191, 25, 'AUSTIN', 28, 15, '(41.88520477, -87.76321191)') ,\n(610169, 'Austin O Sexton Elementary School', 'ES', '6020 S Langley Ave', 'Chicago', 'IL', 60637, '(773) 535-0640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610169.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 23.0, nan, nan, 'Weak', 38.0, 'Weak', 34.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Weak', 44.0, '90.50%', 8.6, '94.10%', '100.00%', 48.8, nan, 12.7, 13.5, 56.6, 51.9, 26.4, 33.6, 53.1, 66.7, 3.7, 11.1, 6.1, 5.2, 0.2, -0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 337, 42, nan, 1181884.66, 1865139.761, 41.78517099, -87.60866927, 42, 'WOODLAWN', 20, 3, '(41.78517099, -87.60866927)') ,\n(610501, 'Austin Polytechnical Academy High School', 'HS', '231 N Pine Ave', 'Chicago', 'IL', 60644, '(773) 534-6300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610501.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 38.0, nan, nan, 'Average', 45.0, 'Average', 53.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '70.10%', 21.5, '95.20%', '98.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.0, 12.1, 13.6, 13.1, 1.1, 14.8, 1.2, 8.5, nan, nan, 250, 36, 56.1, 1139494.763, 1901274.2580000001, 41.88520477, -87.76321191, 25, 'AUSTIN', 28, 15, '(41.88520477, -87.76321191)') ,\n(609786, 'Avalon Park Elementary School', 'ES', '8045 S Kenwood Ave', 'Chicago', 'IL', 60619, '(773) 535-6615', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609786.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 32.0, 'Weak', 24.0, 'Weak', 38.0, 'Weak', 32.0, 'Weak', 32.0, 'Very Weak', 16.0, 'Weak', 46.0, 'Weak', 45.0, '92.10%', 52.1, '96.40%', '97.70%', 47.8, 15.4, 11.9, 22.3, 41.7, 40.9, 14.8, 20.6, 42.7, 44.1, 2.2, 24.4, 7.9, 5.1, 1.1, 1.2, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 398, 47, nan, 1186651.081, 1851731.65, 41.74826636, -87.59161761, 45, 'AVALON PARK', 8, 4, '(41.74826636, -87.59161761)') ,\n(609871, 'Barbara Vick Early Childhood & Family Center', 'ES', '2554 W 113th St', 'Chicago', 'IL', 60655, '(773) 535-2671', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609871.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', nan, 'Track_E', nan, nan, 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Strong', 66.0, 'Strong', 60.0, '60.90%', 0.0, '95.50%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 204, 49, nan, 1161152.12, 1829643.669, 41.68821843, -87.68566340000001, 75, 'MORGAN PARK', 19, 22, '(41.68821843, -87.6856634)') ,\n(609922, 'Belmont-Cragin Elementary School', 'ES', '2456 N Mango Ave', 'Chicago', 'IL', 60639, '(773) 534-2900', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609922.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 75.0, 'Very Strong', 81.0, 'Strong', 67.0, 'Strong', 66.0, 'Weak', 45.0, 'Average', 53.0, 'Strong', 55.0, 'Strong', 54.0, '95.30%', 1.3, '96.80%', '98.90%', 60.7, 31.8, 22.5, 21.3, 51.8, 55.3, 42.9, 39.0, 61.7, 53.2, 0.0, 6.9, 11.0, 11.0, 0.2, 1.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 634, 29, nan, 1137592.32, 1915980.764, 41.92559567, -87.76984337, 19, 'BELMONT CRAGIN', 30, 25, '(41.92559567, -87.76984337)') ,\n(609764, 'Benito Juarez Community Academy High School', 'HS', '2150 S Laflin St', 'Chicago', 'IL', 60608, '(773) 534-7030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609764.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 46.0, 'Average', 53.0, 'Average', 44.0, 'Average', 45.0, 'Weak', 49.0, 'Average', 48.0, nan, nan, nan, nan, '83.70%', 17.2, '95.80%', '99.50%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.2, 13.7, 14.6, 14.5, 1.3, 15.8, 1.2, 14.8, 56.2, 48.8, 1683, 39, 71.1, 1166665.696, 1889615.3480000002, 41.85267306, -87.66376931, 31, 'LOWER WEST SIDE', 25, 12, '(41.85267306, -87.66376931)') ,\n(610265, 'Benjamin Banneker Elementary School', 'ES', '6656 S Normal Blvd', 'Chicago', 'IL', 60621, '(773) 535-3020', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610265.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, nan, nan, 'Weak', 35.0, 'Average', 58.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '91.30%', 26.9, '93.90%', '100.00%', nan, 31.1, 10.9, 11.8, 40.0, 34.4, 34.4, 31.1, 55.7, 60.5, 25.9, 30.8, 7.9, 1.5, 0.7, 0.9, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 302, 45, nan, 1174114.357, 1860550.7240000002, 41.77275433, -87.63729464, 68, 'ENGLEWOOD', 6, 7, '(41.77275433, -87.63729464)') ,\n(610290, 'Benjamin E Mays Elementary Academy', 'ES', '838 W Marquette Rd', 'Chicago', 'IL', 60621, '(773) 535-3892', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610290.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 27.0, 'Weak', 23.0, 'Average', 55.0, 'Average', 48.0, 'Weak', 41.0, 'Weak', 37.0, nan, nan, nan, nan, '95.50%', 0.3, '96.60%', '100.00%', nan, 60.7, 45.9, 26.8, 47.2, 60.2, 37.2, 24.7, 60.7, 57.8, 17.1, 31.4, 17.4, 9.6, 0.1, -0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 301, 45, nan, 1171705.53, 1860460.8280000002, 41.77256081, -87.64612737, 68, 'ENGLEWOOD', 16, 7, '(41.77256081, -87.64612737)') ,\n(610076, 'Bernhard Moos Elementary School', 'ES', '1711 N California Ave', 'Chicago', 'IL', 60647, '(773) 534-4340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610076.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 50.0, 'Weak', 34.0, 'Average', 47.0, 'Weak', 32.0, 'Weak', 34.0, 'Weak', 31.0, nan, nan, nan, nan, '93.00%', 45.5, '96.30%', '98.50%', 52.7, 52.4, 43.0, 26.5, 60.8, 61.6, 60.6, 30.5, 70.8, 61.3, 10.9, 12.7, 24.0, 7.5, 1.4, 0.2, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 429, 34, nan, 1157476.303, 1911358.8969999999, 41.912531099999995, -87.69690545, 24, 'WEST TOWN', 1, 14, '(41.9125311, -87.69690545)') ,\n(610150, 'Betsy Ross Elementary School', 'ES', '6059 S Wabash Ave', 'Chicago', 'IL', 60637, '(773) 535-0650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610150.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 39.0, nan, nan, 'Average', 51.0, 'Average', 50.0, 'Weak', nan, nan, nan, 'Weak', 37.0, 'Weak', 44.0, '88.90%', 64.1, '93.90%', '97.90%', 25.3, 31.0, 10.3, 8.2, 31.5, 22.8, 24.4, 20.2, 56.3, 38.1, 0.0, 5.3, 2.5, 2.0, -0.1, -1.1, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 350, 42, nan, 1177801.979, 1864618.42, 41.78383382, -87.62365383, 40, 'WASHINGTON PARK', 20, 3, '(41.78383382, -87.62365383)') ,\n(610175, 'Beulah Shoesmith Elementary School', 'ES', '1330 E 50th St', 'Chicago', 'IL', 60615, '(773) 535-1764', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610175.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 45.0, 'Average', 54.0, 'Average', 40.0, 'Weak', 39.0, 'Weak', 37.0, 'Average', 41.0, 'Average', 49.0, 'Weak', 45.0, '94.70%', 0.3, '94.80%', '100.00%', 65.0, 12.0, 32.5, 28.4, 54.4, 61.3, 56.4, 43.6, 83.3, 66.7, nan, nan, 10.8, 19.6, 0.0, 1.6, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 325, 46, nan, 1185846.553, 1872152.239, 41.80432122, -87.59392226, 39, 'KENWOOD', 4, 2, '(41.80432122, -87.59392226)') ,\n(610087, 'Blair Early Childhood Center', 'ES', '6751 W 63rd Pl', 'Chicago', 'IL', 60638, '(773) 535-2076', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610087.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', nan, 'Track_E', nan, nan, 'No', nan, nan, 'Strong', 64.0, nan, nan, nan, nan, 'Weak', 61.0, 'Strong', 66.0, 'Strong', 58.0, 'Strong', 65.0, '80.20%', 0.8, '94.80%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 137, 44, nan, 1132295.745, 1861625.19, 41.77652935, -87.79056836, 64, 'CLEARING', 23, 8, '(41.77652935, -87.79056836)') ,\n(609969, 'Bret Harte Elementary School', 'ES', '1556 E 56th St', 'Chicago', 'IL', 60637, '(773) 535-0870', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609969.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 63.0, nan, nan, 'Strong', 77.0, 'Strong', 76.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Weak', 44.0, '94.20%', 17.9, '95.40%', '98.50%', 57.1, nan, 52.8, 60.0, 66.1, 60.8, 43.8, 60.4, 60.4, 64.6, nan, nan, 27.5, 21.9, 0.4, 0.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 340, 46, nan, 1187805.4840000002, 1868216.424, 41.7934746, -87.58686328, 41, 'HYDE PARK', 5, 2, '(41.7934746, -87.58686328)') ,\n(610106, 'Brian Piccolo Elementary Specialty School', 'ES', '1040 N Keeler Ave', 'Chicago', 'IL', 60651, '(773) 534-4425', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610106.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 27.0, nan, nan, 'Weak', 29.0, 'Weak', 31.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '90.60%', 65.8, '95.00%', '92.90%', 29.2, 19.6, 9.7, 15.2, 40.9, 43.0, 20.6, 16.5, 51.3, 48.1, 6.8, 5.1, 3.4, 1.6, -0.2, -1.4, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 549, 34, nan, 1148146.922, 1906778.22, 41.9001461, -87.73129753, 23, 'HUMBOLDT PARK', 37, 11, '(41.9001461, -87.73129753)') ,\n(610317, 'Brighton Park Elementary School', 'ES', '3825 S Washtenaw Ave', 'Chicago', 'IL', 60632, '(773) 535-7237', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610317.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 55.0, nan, nan, 'Average', 49.0, 'Average', 43.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '96.30%', 2.6, '95.90%', '100.00%', 64.5, 50.0, 37.7, 32.2, 51.2, 58.8, 46.1, 39.7, 64.7, 53.9, 6.0, 20.9, 18.6, 11.4, 0.8, 0.2, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 691, 39, nan, 1159041.6609999998, 1879076.7769999998, 41.82391375, -87.69204069, 58, 'BRIGHTON PARK', 12, 9, '(41.82391375, -87.69204069)') ,\n(610381, 'Bronzeville Scholastic Academy High School', 'HS', '4934 S Wabash Ave', 'Chicago', 'IL', 60615, '(773) 535-1150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610381.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 41.0, 'Weak', 38.0, 'Average', 42.0, 'Average', 43.0, 'Weak', 32.0, 'Very Weak', 15.0, 'Average', 51.0, 'Weak', 43.0, '92.30%', 16.2, '95.60%', '97.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.3, 13.8, 15.3, 15.7, 1.4, 16.3, 1.0, 14.7, 85.3, 73.2, 550, 42, 71.1, 1177513.255, 1872170.4440000001, 41.80456384, -87.62448408, 38, 'GRAND BOULEVARD', 3, 2, '(41.80456384, -87.62448408)') ,\n(609821, 'Burnham Elementary Inclusive Academy', 'ES', '1903 E 96th St', 'Chicago', 'IL', 60617, '(773) 535-6530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609821.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 61.0, 'Strong', 77.0, 'Very Strong', 85.0, 'Very Strong', 99.0, 'Weak', 65.0, 'Strong', 71.0, 'Strong', 54.0, 'Average', 47.0, '95.50%', 29.4, '96.90%', '96.00%', 75.9, 65.1, 59.1, 43.6, 61.1, 79.6, 77.2, 58.7, 57.5, 65.5, 8.7, 26.1, 36.5, 22.0, 1.3, 0.6, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 286, 47, nan, 1190566.496, 1841647.9530000002, 41.72050221, -87.57759489, 51, 'SOUTH DEERING', 7, 4, '(41.72050221, -87.57759489)') ,\n(609827, 'Burnside Elementary Scholastic Academy', 'ES', '650 E 91st Pl', 'Chicago', 'IL', 60619, '(773) 535-3300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609827.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 51.0, nan, nan, 'Average', 58.0, 'Strong', 70.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 47.0, '94.90%', 8.9, '94.90%', '100.00%', 81.7, 55.8, 31.1, 38.5, 49.3, 39.9, 40.6, 33.0, 52.4, 49.4, 18.6, 35.6, 15.5, 14.3, 0.0, -0.4, 'Yellow', 'Yellow', 20.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 766, 45, nan, 1182385.6509999998, 1844494.844, 41.72850759, -87.60747108, 44, 'CHATHAM', 6, 6, '(41.72850759, -87.60747108)') ,\n(610353, 'Calmeca Academy of Fine Arts and Dual Language', 'ES', '3456 W 38th St', 'Chicago', 'IL', 60632, '(773) 535-7000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610353.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', 'Average', 58.0, 'Strong', 67.0, 'Average', 59.0, 'Strong', 72.0, 'Weak', 64.0, 'Average', 55.0, 'Average', 51.0, 'Strong', 57.0, '96.50%', 1.9, '96.90%', '100.00%', 47.0, nan, 44.2, 23.9, 52.8, 64.5, 35.8, 26.6, 49.8, 49.3, 16.1, 21.0, 13.9, 10.3, -0.7, 1.5, 'Red', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 792, 39, nan, 1153706.842, 1879216.756, 41.82440547, -87.71160875, 58, 'BRIGHTON PARK', 12, 9, '(41.82440547, -87.71160875)') ,\n(609729, 'Carl Schurz High School', 'HS', '3601 N Milwaukee Ave', 'Chicago', 'IL', 60641, '(773) 534-3420', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609729.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 48.0, 'Weak', 36.0, 'Weak', 36.0, 'Weak', 29.0, 'Weak', 30.0, 'Weak', 31.0, nan, nan, nan, nan, '81.20%', 13.8, '96.00%', '98.90%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.1, 12.9, 14.3, 14.0, 0.9, 15.6, 1.3, 9.7, 62.1, 40.4, 2366, 29, 61.6, 1146851.1979999999, 1923627.843, 41.94640794, -87.73562542, 16, 'IRVING PARK', 38, 17, '(41.94640794, -87.73562542)') ,\n(610039, 'Carl von Linne Elementary School', 'ES', '3221 N Sacramento Ave', 'Chicago', 'IL', 60618, '(773) 534-5262', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610039.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 68.0, 'Average', 49.0, 'Strong', 66.0, 'Strong', 75.0, 'Weak', 37.0, 'Average', 44.0, 'Average', 50.0, 'Strong', 55.0, '95.80%', 1.8, '96.30%', '100.00%', 66.1, 41.2, 40.2, 29.7, 51.3, 72.0, 49.4, 44.2, 64.4, 67.6, 20.4, 40.7, 24.8, 15.1, 1.2, 0.9, 'Green', 'Green', 48.2, 24.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 644, 31, nan, 1155853.119, 1921364.031, 41.94001885, -87.70259825, 21, 'AVONDALE', 33, 17, '(41.94001885, -87.70259825)') ,\n(610238, 'Carrie Jacobs Bond Elementary School', 'ES', '7050 S May St', 'Chicago', 'IL', 60621, '(773) 535-3480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610238.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 54.0, 'Average', 49.0, 'Strong', 67.0, 'Strong', 63.0, 'Weak', 39.0, 'Weak', 32.0, 'Average', 53.0, 'Average', 52.0, '91.70%', 18.8, '94.30%', '96.90%', nan, 56.5, 34.0, 16.1, 41.7, 48.3, 41.3, 23.8, 65.8, 52.8, 7.7, 11.5, 7.9, 5.6, -0.2, 0.9, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 343, 45, nan, 1169836.4440000001, 1857831.503, 41.76538639, -87.65305514, 68, 'ENGLEWOOD', 17, 7, '(41.76538639, -87.65305514)') ,\n(609839, 'Carroll-Rosenwald Specialty Elementary School', 'ES', '2929 W 83rd St', 'Chicago', 'IL', 60652, '(773) 535-9414', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609839.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, nan, nan, 'Very Weak', 12.0, 'Very Weak', 12.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 47.0, '94.80%', 27.8, '95.60%', '100.00%', 63.1, 47.5, 26.7, 29.4, 52.0, 38.4, 37.1, 27.7, 45.1, 43.5, 13.8, 30.0, 11.8, 9.1, -0.6, 0.6, 'Red', 'Yellow', 10.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 537, 43, nan, 1158181.605, 1849398.28, 41.74248931, -87.696003, 70, 'ASHBURN', 18, 8, '(41.74248931, -87.696003)') ,\n(610345, 'Carter G Woodson South Elementary School', 'ES', '4414 S Evans', 'Chicago', 'IL', 60653, '(773) 535-1280', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610345.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 26.0, 'Weak', 27.0, 'Very Weak', 2.0, 'Very Weak', 1.0, 'Weak', 30.0, 'Average', 41.0, 'Weak', 44.0, 'Average', 47.0, '91.90%', 70.5, '94.70%', '98.40%', 65.3, 50.8, 9.8, 6.3, 31.7, 33.3, 9.1, 18.2, 33.7, 40.4, 8.0, 16.0, 4.3, 2.4, -0.7, -2.4, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 329, 40, nan, 1181853.018, 1875820.129, 41.81447955, -87.60845505, 38, 'GRAND BOULEVARD', 3, 2, '(41.81447955, -87.60845505)') ,\n(610148, 'Cesar E Chavez Multicultural Academic Center Elementary School', 'ES', '4747 S Marshfield Ave', 'Chicago', 'IL', 60609, '(773) 535-4600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610148.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 54.0, 'Average', 52.0, 'Strong', 63.0, 'Average', 52.0, 'Weak', 57.0, 'Average', 50.0, 'Weak', 45.0, 'Average', 51.0, '95.10%', 11.1, '96.10%', '100.00%', 59.6, 37.8, 37.7, 16.5, 49.8, 74.2, 62.7, 28.0, 80.7, 63.3, 9.2, 10.8, 19.8, 8.4, 2.0, 0.0, 'Green', 'Yellow', 44.6, 18.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 915, 42, nan, 1166186.662, 1873024.0790000001, 41.80715514, -87.66600055, 61, 'NEW CITY', 20, 9, '(41.80715514, -87.66600055)') ,\n(609679, 'Charles Allen Prosser Career Academy High School', 'HS', '2148 N Long Ave', 'Chicago', 'IL', 60639, '(773) 534-3200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609679.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, 'Average', 47.0, 'Average', 53.0, 'Average', 51.0, 'Weak', 37.0, 'Average', 44.0, 'Weak', 46.0, 'Weak', 45.0, '86.00%', 9.7, '94.40%', '99.40%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 15.5, 15.1, 16.3, 16.2, 0.7, 17.9, 1.6, 27.7, 78.7, 58.0, 1415, 29, 79.4, 1139989.443, 1913970.649, 41.92003613, -87.76108432, 19, 'BELMONT CRAGIN', 37, 25, '(41.92003613, -87.76108432)') ,\n(610005, 'Charles Evans Hughes Elementary School', 'ES', '4247 W 15th St', 'Chicago', 'IL', 60623, '(773) 534-1762', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610005.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, nan, nan, 'Very Strong', 82.0, 'Strong', 72.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.40%', 0.0, '94.70%', '100.00%', 69.2, 63.3, 32.6, 16.9, 40.3, 76.0, 54.8, 20.8, 69.6, 45.8, 5.9, 23.5, 12.2, 4.7, 0.5, -1.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 315, 36, nan, 1148118.259, 1892326.805, 41.86049033, -87.73177486, 29, 'NORTH LAWNDALE', 24, 10, '(41.86049033, -87.73177486)') ,\n(609879, 'Charles Gates Dawes Elementary School', 'ES', '3810 W 81st Pl', 'Chicago', 'IL', 60652, '(773) 535-2350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609879.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 44.0, 'Weak', 38.0, 'Weak', 27.0, 'Weak', 30.0, 'Weak', 60.0, 'Average', 54.0, 'Average', 50.0, 'Average', 48.0, '95.70%', 35.6, '95.60%', '99.50%', 54.3, 24.0, 39.9, 42.9, 66.9, 76.5, 52.3, 56.2, 70.1, 65.3, 19.8, 35.5, 23.1, 13.6, 0.5, 0.3, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1032, 44, nan, 1152339.16, 1850308.795, 41.74510467, -87.71738613, 70, 'ASHBURN', 18, 8, '(41.74510467, -87.71738613)') ,\n(609966, 'Charles G Hammond Elementary School', 'ES', '2819 W 21st Pl', 'Chicago', 'IL', 60623, '(773) 535-4580', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609966.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 40.0, nan, nan, 'Average', 41.0, 'Average', 43.0, 'Weak', nan, nan, nan, 'Weak', 43.0, 'Weak', 45.0, '95.30%', 8.5, '95.20%', '92.00%', 77.4, nan, 27.4, 24.8, 62.4, 53.7, 38.1, 28.8, 60.2, 59.8, 12.8, 5.3, 10.1, 5.8, 0.5, 0.2, 'Yellow', 'Yellow', 25.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 446, 39, nan, 1157809.015, 1889554.6230000001, 41.85269133, -87.69627777, 30, 'SOUTH LAWNDALE', 12, 10, '(41.85269133, -87.69627777)') ,\n(610366, 'Charles H Wacker Elementary School', 'ES', '9746 S Morgan St', 'Chicago', 'IL', 60643, '(773) 535-2821', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610366.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 32.0, nan, nan, 'Weak', 34.0, 'Weak', 32.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Weak', 41.0, '93.40%', 11.5, '93.40%', '100.00%', 64.5, 53.9, 20.8, 31.0, 34.8, 40.6, 66.7, 60.0, 85.7, 67.2, 7.7, 23.1, 17.4, 15.9, 1.8, 0.0, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 246, 49, nan, 1171336.009, 1839924.624, 41.716214799999996, -87.6480814, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.7162148, -87.6480814)') ,\n(610030, 'Charles Kozminski Elementary Community Academy', 'ES', '936 E 54th St', 'Chicago', 'IL', 60615, '(773) 535-0980', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610030.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, nan, nan, 'Weak', 35.0, 'Average', 40.0, 'Weak', nan, nan, nan, 'Weak', 39.0, 'Weak', 42.0, '92.00%', 55.6, '94.40%', '100.00%', 60.0, 34.9, 27.5, 24.6, 36.9, 33.9, 14.7, 23.8, 37.9, 43.4, 6.5, 18.8, 5.0, 6.1, -1.2, -1.1, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 403, 46, nan, 1183537.482, 1869839.6509999998, 41.7980295, -87.60246287, 41, 'HYDE PARK', 4, 2, '(41.7980295, -87.60246287)') ,\n(609996, 'Charles N Holden Elementary School', 'ES', '1104 W 31st St', 'Chicago', 'IL', 60608, '(773) 535-7200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609996.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 53.0, 'Average', 44.0, 'Average', 50.0, 'Weak', 35.0, 'Weak', 53.0, 'Average', 51.0, 'Strong', 54.0, 'Average', 50.0, '94.90%', 4.7, '96.00%', '100.00%', 53.2, nan, 33.9, 26.1, 49.2, 45.7, 56.8, 37.4, 57.3, 58.2, 28.0, 37.5, 16.7, 11.5, -0.8, 0.0, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 600, 40, nan, 1169423.887, 1884309.813, 41.83805473, -87.65380002, 60, 'BRIDGEPORT', 11, 9, '(41.83805473, -87.65380002)') ,\n(609833, 'Charles P Caldwell Academy of Math & Science Elementary School', 'ES', '8546 S Cregier', 'Chicago', 'IL', 60617, '(773) 535-6300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609833.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 25.0, 'Weak', 31.0, 'Average', 40.0, 'Average', 44.0, 'Weak', 28.0, 'Weak', 25.0, 'Weak', 44.0, 'Average', 47.0, '92.70%', 30.7, '94.70%', '100.00%', 60.6, 67.6, 25.3, 23.4, 54.0, 38.2, 42.7, 26.5, 52.8, 48.1, 4.8, 14.3, 10.9, 6.8, 0.9, -0.1, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 291, 47, nan, 1189641.205, 1848463.6269999999, 41.73922735, -87.58076569, 45, 'AVALON PARK', 8, 4, '(41.73922735, -87.58076569)') ,\n(609732, 'Charles P Steinmetz Academic Centre High School', 'HS', '3030 N Mobile Ave', 'Chicago', 'IL', 60634, '(773) 534-3030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609732.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 26.0, 'Weak', 35.0, 'Weak', 29.0, 'Weak', 34.0, 'Weak', 36.0, 'Weak', 32.0, nan, nan, nan, nan, '79.20%', 30.4, '96.20%', '99.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.0, 13.8, 13.8, 14.3, 1.3, 16.1, 2.3, 16.3, 56.7, 47.0, 1890, 30, 70.3, 1133838.348, 1919613.6940000001, 41.93563171, -87.78355194, 19, 'BELMONT CRAGIN', 36, 25, '(41.93563171, -87.78355194)') ,\n(609875, 'Charles R Darwin Elementary School', 'ES', '3116 W Belden Ave', 'Chicago', 'IL', 60647, '(773) 534-4110', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609875.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, 'Average', 40.0, 'Average', 44.0, 'Weak', 34.0, 'Weak', 31.0, 'Average', 46.0, 'Average', 49.0, 'Strong', 54.0, '95.60%', 33.8, '96.20%', '100.00%', 50.3, 35.6, 37.7, 33.5, 59.6, 68.5, 29.6, 36.1, 60.6, 51.2, 16.7, 19.2, 16.0, 6.9, -0.2, -0.8, 'Yellow', 'Red', 21.0, 50.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 622, 35, nan, 1155126.902, 1915284.2030000002, 41.92334998, -87.70543099, 22, 'LOGAN SQUARE', 26, 14, '(41.92334998, -87.70543099)') ,\n(609986, 'Charles R Henderson Elementary School', 'ES', '5650 S Wolcott Ave', 'Chicago', 'IL', 60636, '(773) 535-9080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609986.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 16.0, nan, nan, 'Weak', 32.0, 'Weak', 38.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Weak', 44.0, '89.20%', 80.6, '93.60%', '100.00%', 38.8, 23.1, 21.6, 18.0, 52.8, 55.9, 18.8, 14.4, 60.6, 45.3, 4.4, 17.8, 4.8, 2.6, -0.1, -0.6, 'Yellow', 'Yellow', 8.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 407, 43, nan, 1164617.918, 1866985.2580000001, 41.79061716, -87.67192466, 67, 'WEST ENGLEWOOD', 15, 7, '(41.79061716, -87.67192466)') ,\n(609813, 'Charles S Brownell Elementary School', 'ES', '6741 S Michigan Ave', 'Chicago', 'IL', 60637, '(773) 535-3030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609813.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 20.0, 'Very Weak', 16.0, 'Strong', 79.0, 'Very Strong', 82.0, 'Weak', 20.0, 'Very Weak', 19.0, 'Average', 48.0, 'Weak', 46.0, '92.80%', 14.1, '95.50%', '100.00%', nan, nan, 24.3, 17.0, 38.8, 63.1, 36.8, 32.4, 67.6, 47.2, nan, nan, 12.9, 8.8, 1.5, -0.4, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 310, 42, nan, 1178368.986, 1860177.542, 41.77163474, -87.62170972, 69, 'GREATER GRAND CROSSING', 20, 3, '(41.77163474, -87.62170972)') ,\n(609883, 'Charles S Deneen Elementary School', 'ES', '7240 S Wabash Ave', 'Chicago', 'IL', 60619, '(773) 535-3035', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609883.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 55.0, 'Weak', 33.0, 'Average', 49.0, 'Average', 51.0, 'Weak', 15.0, 'Average', 49.0, nan, nan, nan, nan, '93.80%', 39.5, '97.10%', '100.00%', 45.9, nan, 20.6, 18.4, 51.8, 44.3, 20.0, 26.8, 61.3, 54.4, 2.2, 4.4, 9.4, 6.8, 2.8, -0.5, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 551, 46, nan, 1177940.398, 1856846.4530000002, 41.76250359, -87.62338156, 69, 'GREATER GRAND CROSSING', 6, 3, '(41.76250359, -87.62338156)') ,\n(610194, 'Charles Sumner  Math & Science Community Acad Elementary School', 'ES', '4320 W 5th Ave', 'Chicago', 'IL', 60624, '(773) 534-6730', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610194.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 41.0, 'Weak', 27.0, 'Average', 44.0, 'Average', 48.0, 'Weak', 30.0, 'Weak', 34.0, 'Average', 48.0, 'Weak', 46.0, '93.90%', 12.3, '96.90%', '100.00%', 49.0, nan, 28.4, 33.1, 50.0, 54.7, 31.9, 36.3, 59.3, 47.4, 15.6, 11.1, 13.1, 9.7, -1.0, 0.1, 'Red', 'Yellow', 48.9, 23.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 430, 36, nan, 1147607.2, 1896098.17, 41.87084922, -87.73355414, 26, 'WEST GARFIELD PARK', 24, 11, '(41.87084922, -87.73355414)') ,\n(609897, 'Charles W Earle Elementary School', 'ES', '6121 S Hermitage Ave', 'Chicago', 'IL', 60636, '(773) 535-9130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609897.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 24.0, nan, nan, 'Strong', 62.0, 'Strong', 71.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Average', 48.0, '93.80%', 16.9, '94.60%', '100.00%', 37.5, 36.8, 44.1, 20.8, 56.1, 73.8, 38.6, 25.0, 59.4, 50.7, 12.0, 20.0, 9.8, 5.6, 1.1, -0.6, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 331, 43, nan, 1165772.423, 1863964.0380000002, 41.78230211, -87.66777712, 67, 'WEST ENGLEWOOD', 15, 7, '(41.78230211, -87.66777712)') ,\n(610248, 'Chicago Academy Elementary School', 'ES', '3400 N Austin Ave', 'Chicago', 'IL', 60634, '(773) 534-0146', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610248.pdf', 'AUSL Schools', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, nan, nan, 'Average', 40.0, 'Weak', 22.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 51.0, '96.30%', 6.0, '96.10%', '100.00%', nan, nan, 66.3, 62.6, 66.7, 68.8, 62.9, 64.9, 57.5, 62.4, 26.7, 46.7, 41.9, 30.3, -0.1, 1.8, 'Yellow', 'Green', 50.0, 53.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 601, 30, nan, 1135740.091, 1922002.9409999999, 41.94215439, -87.77650575, 17, 'DUNNING', 38, 16, '(41.94215439, -87.77650575)') ,\n(610340, 'Chicago Academy High School', 'HS', '3400 N Austin Ave', 'Chicago', 'IL', 60634, '(773) 534-0146', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610340.pdf', 'AUSL Schools', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, 'Weak', 35.0, 'Average', 57.0, 'Average', 53.0, 'Weak', 56.0, 'Average', 57.0, nan, nan, nan, nan, '93.10%', 6.3, '96.10%', '97.10%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.6, 14.4, 16.3, 16.1, 1.5, 18.4, 2.1, 23.9, 70.1, 64.2, 502, 30, 80.6, 1135740.091, 1922002.9409999999, 41.94215439, -87.77650575, 17, 'DUNNING', 38, 16, '(41.94215439, -87.77650575)') ,\n(609753, 'Chicago High School for Agricultural Sciences', 'HS', '3857 W 111th St', 'Chicago', 'IL', 60655, '(773) 535-2500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609753.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, 'Strong', 77.0, 'Average', 49.0, 'Average', 47.0, 'Weak', 57.0, 'Average', 40.0, 'Average', 52.0, 'Strong', 55.0, '92.80%', 5.2, '95.20%', '96.80%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 16.1, 16.5, 16.9, 17.4, 1.3, 20.4, 3.5, 30.7, 70.7, 79.6, 579, 49, 88.1, 1152383.425, 1830663.6469999999, 41.69119388, -87.71773863, 74, 'MOUNT GREENWOOD', 19, 22, '(41.69119388, -87.71773863)') ,\n(609754, 'Chicago Military Academy High School', 'HS', '3519 S Giles Ave', 'Chicago', 'IL', 60653, '(773) 534-9750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609754.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 32.0, nan, nan, 'Weak', 23.0, 'Weak', 35.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '90.30%', 28.2, '95.60%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.8, 14.4, 15.2, 15.6, 0.8, 17.4, 2.2, 24.4, 83.6, 69.3, 483, 40, 88.1, 1178881.169, 1881647.166, 41.83053768, -87.61917837, 35, 'DOUGLAS', 2, 2, '(41.83053768, -87.61917837)') ,\n(609674, 'Chicago Vocational Career Academy High School', 'HS', '2100 E 87th St', 'Chicago', 'IL', 60617, '(773) 535-6100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609674.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 27.0, 'Average', 43.0, 'Weak', 21.0, 'Weak', 30.0, 'Weak', 34.0, 'Weak', 39.0, nan, nan, nan, nan, '68.80%', 33.3, '94.60%', '99.10%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.5, 12.6, 13.8, 13.2, 0.7, 14.3, 0.5, 7.0, 49.5, 54.0, 833, 47, 44.1, 1191700.5929999999, 1847743.551, 41.73720173, -87.57324389, 45, 'AVALON PARK', 8, 4, '(41.73720173, -87.57324389)') ,\n(609899, 'Christian Ebinger Elementary School', 'ES', '7350 W Pratt Ave', 'Chicago', 'IL', 60631, '(773) 534-1070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609899.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 78.0, 'Strong', 72.0, 'Strong', 66.0, 'Average', 50.0, 'Weak', 26.0, 'Strong', 62.0, 'Average', 52.0, 'Weak', 43.0, '95.90%', 9.3, '95.20%', '97.00%', 84.7, 61.5, 81.2, 68.5, 63.3, 82.2, 77.4, 69.4, 56.8, 70.1, 34.5, 53.4, 50.6, 40.2, 0.5, 1.4, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 650, 30, nan, 1126172.537, 1944639.095, 42.00443557, -87.81116435, 9, 'EDISON PARK', 41, 16, '(42.00443557, -87.81116435)') ,\n(609705, 'Christian Fenger Academy High School', 'HS', '11220 S Wallace St', 'Chicago', 'IL', 60628, '(773) 535-5430', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609705.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, 'Weak', 28.0, 'Average', 41.0, 'Weak', 32.0, 'Weak', 46.0, 'Average', 42.0, nan, nan, nan, nan, '74.40%', 49.8, '95.40%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.0, 12.1, 12.9, 13.3, 1.3, 14.0, 1.1, 11.3, 34.8, 36.7, 688, 48, 68.2, 1174272.711, 1830372.91, 41.68993885, -87.63760833, 49, 'ROSELAND', 34, 22, '(41.68993885, -87.63760833)') ,\n(609863, 'Christopher Columbus Elementary School', 'ES', '1003 N Leavitt St', 'Chicago', 'IL', 60622, '(773) 534-4350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609863.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, nan, nan, 'Average', 57.0, 'Average', 55.0, 'Weak', nan, nan, nan, 'Strong', 54.0, 'Average', 48.0, '95.50%', 2.4, '96.90%', '92.60%', 77.0, 72.3, 59.2, 48.7, 57.7, 62.5, 76.2, 56.9, 78.8, 63.0, 40.0, 45.0, 42.0, 30.5, 1.1, 1.6, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 309, 35, nan, 1161595.965, 1906670.676, 41.89958139, -87.68190167, 24, 'WEST TOWN', 32, 13, '(41.89958139, -87.68190167)') ,\n(609790, 'Clara Barton Elementary School', 'ES', '7650 S Wolcott Ave', 'Chicago', 'IL', 60620, '(773) 535-3260', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609790.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 25.0, 'Average', 42.0, 'Weak', 28.0, 'Weak', 34.0, 'Weak', 77.0, 'Average', 59.0, 'Average', 51.0, 'Average', 49.0, '91.70%', 25.3, '96.40%', '100.00%', 64.3, nan, 27.0, 22.8, 51.8, 55.3, 27.4, 22.4, 45.3, 52.9, 6.5, 13.0, 8.1, 5.7, -0.5, -0.3, 'Yellow', 'Yellow', 60.0, 3.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 602, 43, nan, 1164981.896, 1853724.2969999998, 41.75421962, -87.67096453, 71, 'AUBURN GRESHAM', 18, 6, '(41.75421962, -87.67096453)') ,\n(610347, 'Claremont Academy Elementary School', 'ES', '2300 W 64th St', 'Chicago', 'IL', 60636, '(773) 535-8110', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610347.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 36.0, 'Weak', 33.0, 'Average', 50.0, 'Average', 51.0, 'Weak', 62.0, 'Average', 59.0, 'Weak', 46.0, 'Average', 52.0, '92.50%', 64.2, '96.20%', '100.00%', 58.7, 60.5, 40.9, 22.5, 52.3, 66.9, 46.0, 31.7, 72.5, 61.0, 14.9, 14.9, 12.1, 8.1, 0.3, 0.7, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 531, 43, nan, 1162155.645, 1862200.052, 41.77753754, -87.68108634, 66, 'CHICAGO LAWN', 15, 8, '(41.77753754, -87.68108634)') ,\n(610499, 'Collins Academy High School', 'HS', '1313 S Sacramento Dr', 'Chicago', 'IL', 60623, '(773) 534-1840', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610499.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 60.0, 'Average', 42.0, 'Average', 51.0, 'Average', 46.0, 'Weak', 47.0, 'Average', 48.0, 'Strong', 54.0, 'Strong', 56.0, '91.40%', 2.9, '96.60%', '98.70%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.5, 12.5, 14.2, 14.1, 1.6, 15.6, 1.4, 20.5, nan, nan, 409, 37, 70.4, 1156217.24, 1893722.861, 41.86416169, -87.70200761, 29, 'NORTH LAWNDALE', 24, 10, '(41.86416169, -87.70200761)') ,\n(610170, 'Columbia Explorers Elementary Academy', 'ES', '4520 S Kedzie Ave', 'Chicago', 'IL', 60632, '(773) 535-4050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610170.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, 'Strong', 64.0, 'Average', 59.0, 'Strong', 63.0, 'Weak', 65.0, 'Strong', 72.0, 'Average', 52.0, 'Strong', 55.0, '95.40%', 8.9, '97.10%', '100.00%', 67.6, 30.5, 27.4, 32.5, 44.4, 48.7, 41.6, 49.3, 52.8, 59.6, 17.7, 27.8, 17.1, 13.8, 0.8, 1.5, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1038, 39, nan, 1155752.301, 1874367.2340000002, 41.81105689, -87.70423481, 58, 'BRIGHTON PARK', 14, 8, '(41.81105689, -87.70423481)') ,\n(610004, 'Countee Cullen Elementary School', 'ES', '10650 S Eberhart Ave', 'Chicago', 'IL', 60628, '(773) 535-5375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610004.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 14.0, nan, nan, 'Weak', 28.0, 'Weak', 34.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.10%', 36.9, '96.30%', '100.00%', 71.9, 71.4, 24.4, 28.9, 55.3, 68.2, 26.8, 25.9, 56.2, 45.8, 9.5, 4.8, 9.7, 8.1, -1.8, -1.7, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 233, 48, nan, 1181431.47, 1834229.814, 41.70036107, -87.61128176, 49, 'ROSELAND', 9, 5, '(41.70036107, -87.61128176)') ,\n(609781, 'Crispus Attucks Elementary School', 'ES', '5055 S State St', 'Chicago', 'IL', 60609, '(773) 535-1270', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609781.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 45.0, 'Strong', 76.0, 'Strong', 64.0, 'Average', 57.0, 'Weak', 81.0, 'Very Strong', 85.0, nan, nan, nan, nan, '95.20%', 12.5, '93.50%', '100.00%', 64.7, 66.7, 25.3, 9.9, 42.4, 53.5, 16.0, 16.5, 53.3, 32.4, 3.1, 9.4, 7.6, 10.0, -0.3, -2.0, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 274, 42, nan, 1177176.077, 1871285.644, 41.80214349, -87.62574740000001, 38, 'GRAND BOULEVARD', 3, 2, '(41.80214349, -87.6257474)') ,\n(609873, 'Crown Community Academy of Fine Arts Center Elementary School', 'ES', '2128 S Saint Louis Ave', 'Chicago', 'IL', 60623, '(773) 534-1680', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609873.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 27.0, 'Weak', 39.0, 'Average', 51.0, 'Average', 52.0, 'Weak', 45.0, 'Weak', 31.0, 'Average', 49.0, 'Average', 51.0, '92.50%', 100.5, '96.70%', '100.00%', 51.6, 38.0, 12.5, 16.1, 35.2, 38.7, 36.7, 28.1, 64.3, 56.6, 3.4, 6.9, 3.9, 8.8, -0.3, 1.0, 'Yellow', 'Yellow', 13.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 298, 37, nan, 1153387.649, 1889511.3869999999, 41.85266155, -87.7125068, 29, 'NORTH LAWNDALE', 24, 10, '(41.85266155, -87.7125068)') ,\n(610063, 'Cyrus H McCormick Elementary School', 'ES', '2712 S Sawyer Ave', 'Chicago', 'IL', 60623, '(773) 535-7252', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610063.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'Yes', nan, nan, 'Strong', 60.0, nan, nan, nan, nan, 'Weak', 61.0, 'Average', 54.0, 'Average', 52.0, 'Strong', 56.0, '95.90%', 4.7, '95.90%', '100.00%', 55.3, 39.4, 42.1, 22.6, 43.9, 54.2, nan, nan, nan, nan, nan, nan, 25.2, 10.3, -0.2, 0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 894, 37, nan, 1155096.537, 1885727.878, 41.84224508, -87.706336, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84224508, -87.706336)') ,\n(609804, 'Daniel Boone Elementary School', 'ES', '6710 N Washtenaw Ave', 'Chicago', 'IL', 60645, '(773) 534-2160', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609804.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 52.0, 'Average', 50.0, 'Average', 47.0, 'Average', 48.0, 'Weak', 45.0, 'Average', 53.0, 'Average', 50.0, 'Average', 49.0, '95.50%', 10.5, '95.40%', '97.60%', 65.0, 43.0, 39.7, 33.2, 55.4, 61.8, 43.1, 35.2, 56.3, 42.3, 22.5, 35.0, 22.0, 17.6, 1.1, 1.9, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 752, 32, nan, 1157068.6570000001, 1944483.4719999998, 42.00343519, -87.69750011, 2, 'WEST RIDGE', 50, 24, '(42.00343519, -87.69750011)') ,\n(610083, 'Daniel C Beard Elementary School', 'ES', '6445 W Strong St', 'Chicago', 'IL', 60656, '(773) 534-1228', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610083.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', nan, 'Track_E', 'Not Applicable', 'Not Enough Data', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Strong', 57.0, 'Strong', 58.0, '91.90%', 0.0, '94.30%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 160, 30, nan, 1132103.024, 1932162.514, 41.97009738, -87.78963691, 10, 'NORWOOD PARK', 41, 16, '(41.97009738, -87.78963691)') ,\n(610380, 'Daniel Hale Williams Prep School of Medicine', 'HS', '4934 S Wabash Ave', 'Chicago', 'IL', 60615, '(773) 535-1120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610380.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 50.0, nan, nan, 'Average', 45.0, 'Average', 54.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '91.30%', 5.4, '96.70%', '100.00%', nan, nan, nan, nan, nan, nan, 54.5, 63.8, 55.8, 58.5, 21.4, 33.3, 23.7, 8.5, -0.9, -1.8, 'Yellow', 'Red', 64.1, 25.0, 15.0, 15.1, 15.8, 15.4, 0.4, 17.5, 1.7, 32.7, nan, nan, 324, 42, 90.1, 1177513.255, 1872170.4440000001, 41.80456384, -87.62448408, 38, 'GRAND BOULEVARD', 3, 2, '(41.80456384, -87.62448408)') ,\n(609870, 'Daniel J Corkery Elementary School', 'ES', '2510 S Kildare Ave', 'Chicago', 'IL', 60623, '(773) 534-1650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609870.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 39.0, nan, nan, 'Weak', 31.0, 'Weak', 29.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.70%', 18.1, '97.20%', '100.00%', 64.1, 35.3, 35.7, 22.6, 48.4, 61.1, 45.8, 28.8, 47.5, 53.0, 21.6, 19.6, 13.6, 8.2, 0.1, 0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 631, 37, nan, 1148304.879, 1886901.414, 41.84559878, -87.73122946, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84559878, -87.73122946)') ,\n(609835, 'Daniel R Cameron Elementary School', 'ES', '1234 N Monticello Ave', 'Chicago', 'IL', 60651, '(773) 534-4290', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609835.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, nan, nan, 'Average', 49.0, 'Average', 50.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Weak', 46.0, '92.60%', 153.9, '96.20%', '98.40%', 47.8, 21.1, 16.4, 16.1, 31.3, 34.7, 19.2, 15.6, 33.5, 35.0, 9.2, 7.1, 10.3, 5.9, -1.0, -1.4, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 828, 34, nan, 1151767.5459999999, 1908130.0680000002, 41.90378521, -87.71796315, 23, 'HUMBOLDT PARK', 26, 25, '(41.90378521, -87.71796315)') ,\n(610223, 'Daniel S Wentworth Elementary School', 'ES', '6950 S Sangamon St', 'Chicago', 'IL', 60621, '(773) 535-3394', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610223.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 30.0, 'Weak', 32.0, 'Weak', 34.0, 'Weak', 36.0, 'Weak', 31.0, 'Weak', 32.0, nan, nan, nan, nan, '91.60%', 73.8, '95.90%', '96.20%', nan, 24.7, 17.9, 14.2, 48.5, 51.0, 40.2, 18.8, 74.4, 38.8, 10.5, 10.5, 5.5, 1.5, 1.0, -0.6, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 365, 45, nan, 1171140.355, 1858532.5580000002, 41.76728177, -87.64825544, 68, 'ENGLEWOOD', 17, 7, '(41.76728177, -87.64825544)') ,\n(610221, 'Daniel Webster Elementary School', 'ES', '4055 W Arthington St', 'Chicago', 'IL', 60624, '(773) 534-6925', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610221.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 48.0, 'Average', 56.0, 'Very Strong', 81.0, 'Strong', 66.0, 'Weak', 46.0, 'Average', 44.0, 'Weak', 45.0, 'Strong', 55.0, '94.80%', 5.7, '97.20%', '100.00%', 81.6, 45.9, 37.0, 37.0, 56.6, 63.3, 44.2, 26.9, 57.8, 58.8, 2.6, 37.5, 25.9, 21.0, 2.0, 0.7, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 370, 36, nan, 1149263.6090000002, 1895759.896, 41.86988904, -87.72748159, 26, 'WEST GARFIELD PARK', 24, 11, '(41.86988904, -87.72748159)') ,\n(609704, 'David G Farragut Career Academy High School', 'HS', '2345 S Christiana Ave', 'Chicago', 'IL', 60623, '(773) 534-1300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609704.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 43.0, nan, nan, 'Average', 42.0, 'Weak', 32.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '86.10%', 18.6, '95.20%', '99.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.6, 12.9, 14.4, 13.9, 1.3, 16.0, 1.6, 13.4, 43.2, 31.3, 1456, 37, 68.6, 1154443.895, 1888031.41, 41.8485793, -87.70866954, 30, 'SOUTH LAWNDALE', 22, 10, '(41.8485793, -87.70866954)') ,\n(610402, 'DeVry University Advantage Academy High School', 'HS', '3300 N Campbell', 'Chicago', 'IL', 60618, '(773) 697-2216', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610402.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Strong', 58.0, 'Strong', 55.0, '96.70%', 0.0, '94.50%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 20.7, nan, 40.6, nan, 63.3, 200, 35, nan, 1159059.4109999998, 1921900.745, 41.94142629, -87.6907993, 5, 'NORTH CENTER', 47, 19, '(41.94142629, -87.6907993)') ,\n(609885, 'Dewey Elementary Academy of Fine Arts', 'ES', '5415 S Union Ave', 'Chicago', 'IL', 60609, '(773) 535-1666', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609885.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 41.0, nan, nan, 'Average', 42.0, 'Weak', 35.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Strong', 58.0, '90.70%', 26.3, '95.50%', '100.00%', 64.7, 56.1, 12.2, 11.8, 30.9, 44.2, 10.4, 4.3, 40.3, 23.1, 3.6, 0.0, 3.3, 0.5, 0.4, -1.2, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 357, 45, nan, 1172596.05, 1868881.791, 41.795649299999994, -87.642615, 61, 'NEW CITY', 3, 9, '(41.7956493, -87.642615)') ,\n(609859, 'DeWitt Clinton Elementary School', 'ES', '6110 N Fairfield Ave', 'Chicago', 'IL', 60659, '(773) 534-2025', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609859.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 47.0, 'Average', 41.0, 'Average', 47.0, 'Average', 52.0, 'Weak', 41.0, 'Weak', 36.0, nan, nan, nan, nan, '95.30%', 14.8, '95.40%', '100.00%', 68.8, 44.9, 45.8, 40.1, 55.6, 63.7, 64.7, 39.7, 70.6, 54.2, 24.3, 25.2, 33.5, 21.8, 0.6, -0.5, 'Green', 'Yellow', 23.6, 46.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1139, 32, nan, 1156854.125, 1940520.2119999998, 41.99256421, -87.69839748, 2, 'WEST RIDGE', 50, 24, '(41.99256421, -87.69839748)') ,\n(610515, 'Disney II Magnet School', 'ES', '3815 N Kedvale Ave', 'Chicago', 'IL', 60641, '(773) 534-3750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610515.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', nan, nan, 'Very Strong', 99.0, nan, nan, nan, nan, 'Weak', 75.0, 'Average', 53.0, 'Strong', 62.0, 'Strong', 58.0, '97.30%', 0.0, '96.50%', '100.00%', 84.6, nan, 76.5, 79.4, 63.4, 63.7, nan, nan, nan, nan, nan, nan, 57.8, 52.0, 0.6, 2.1, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 363, 29, nan, 1148206.405, 1925121.479, 41.95048055, -87.73060545, 16, 'IRVING PARK', 38, 17, '(41.95048055, -87.73060545)') ,\n(610077, 'Donald Morrill Math & Science Elementary School', 'ES', '6011 S Rockwell St', 'Chicago', 'IL', 60629, '(773) 535-9288', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610077.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Weak', 25.0, 'Weak', 32.0, 'Weak', 39.0, 'Weak', 35.0, 'Weak', 32.0, 'Weak', 44.0, 'Average', 49.0, '93.40%', 19.6, '94.80%', '97.30%', 64.6, 46.3, 20.0, 15.3, 46.6, 42.6, 30.9, 22.9, 54.5, 50.3, 12.1, 26.2, 10.0, 5.7, -0.8, -1.2, 'Red', 'Red', 38.9, 50.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 823, 43, nan, 1160111.7959999999, 1864645.2980000002, 41.78428993, -87.68851192, 66, 'CHICAGO LAWN', 15, 8, '(41.78428993, -87.68851192)') ,\n(610198, 'Douglas Taylor Elementary School', 'ES', '9912 S Avenue H', 'Chicago', 'IL', 60617, '(773) 535-6240', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610198.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 34.0, nan, nan, 'Weak', 20.0, 'Very Weak', 1.0, 'Weak', nan, nan, nan, 'Weak', 42.0, 'Weak', 45.0, '95.10%', 16.6, '94.30%', '100.00%', 48.9, 34.4, 28.5, 22.3, 44.4, 44.9, 39.7, 33.5, 56.6, 48.9, 15.7, 18.6, 12.4, 9.9, -0.5, -0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 632, 47, nan, 1202756.0929999999, 1839915.4880000001, 41.71544589, -87.5330074, 52, 'EAST SIDE', 10, 4, '(41.71544589, -87.5330074)') ,\n(610533, 'Dr Jorge Prieto Math and Science', 'ES', '2231 N Central Ave', 'Chicago', 'IL', 60639, '(773) 534-0210', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610533.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 52.0, 'Strong', 58.0, '94.70%', 7.0, '96.90%', '100.00%', 48.4, 19.5, 33.8, 19.9, 43.1, 55.0, 44.2, 30.0, 56.9, 47.8, 12.1, 28.9, 14.6, 8.6, -0.7, -0.7, 'Red', 'Red', 17.5, 52.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1001, 29, nan, 1138718.257, 1914429.033, 41.92131717, -87.7657438, 19, 'BELMONT CRAGIN', 37, 25, '(41.92131717, -87.7657438)') ,\n(609751, 'Dr  Martin Luther King  Jr  College Prep High School', 'HS', '4445 S Drexel Blvd', 'Chicago', 'IL', 60653, '(773) 535-1180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609751.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Strong', 56.0, 'Average', 52.0, '92.90%', 4.4, '96.30%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 17.4, 17.7, 17.7, 18.1, 0.7, 20.5, 2.8, 31.9, 75.4, 85.1, 915, 40, 91.1, 1183114.175, 1875544.7869999998, 41.81369471, -87.60383761, 39, 'KENWOOD', 4, 2, '(41.81369471, -87.60383761)') ,\n(610266, 'Dumas Technology Academy', 'ES', '6650 S Ellis Ave', 'Chicago', 'IL', 60637, '(773) 535-0750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610266.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 19.0, nan, nan, 'Weak', 25.0, 'Average', 49.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 53.0, '92.50%', 38.4, '95.60%', '98.10%', 63.6, 12.0, 22.0, 18.8, 61.6, 63.7, 20.0, 22.4, 61.0, 53.7, 15.9, 40.9, 8.3, 5.8, -0.2, -0.7, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 381, 46, nan, 1183997.399, 1860905.9440000001, 41.77350389, -87.60105531, 42, 'WOODLAWN', 5, 3, '(41.77350389, -87.60105531)') ,\n(610188, 'Dunne Technology Academy', 'ES', '10845 S Union Ave', 'Chicago', 'IL', 60628, '(773) 535-5517', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610188.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 46.0, 'Average', 53.0, 'Strong', 65.0, 'Strong', 71.0, 'Weak', 51.0, 'Average', 59.0, 'Average', 49.0, 'Average', 47.0, '94.60%', 22.8, '96.40%', '100.00%', 53.6, 38.8, 36.3, 31.4, 55.8, 71.1, 39.4, 37.7, 57.0, 49.5, 9.6, 25.0, 24.7, 17.2, 2.8, 1.8, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 310, 49, nan, 1173617.217, 1832764.2859999998, 41.69651565, -87.63993763, 49, 'ROSELAND', 34, 22, '(41.69651565, -87.63993763)') ,\n(610352, 'Durkin Park Elementary School', 'ES', '8445 S Kolin Ave', 'Chicago', 'IL', 60652, '(773) 535-2322', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610352.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 61.0, 'Strong', 74.0, 'Average', 57.0, 'Average', 58.0, 'Weak', 88.0, 'Strong', 74.0, 'Average', 53.0, 'Weak', 38.0, '95.40%', 0.7, '95.80%', '100.00%', 75.6, 58.1, 36.6, 32.3, 55.8, 65.6, 53.4, 40.4, 61.8, 61.5, 22.4, 40.8, 22.0, 16.3, 0.8, 1.5, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 543, 44, nan, 1148934.857, 1848010.352, 41.73886352, -87.72991927, 70, 'ASHBURN', 18, 8, '(41.73886352, -87.72991927)') ,\n(610254, 'Dvorak Technology Academy', 'ES', '3615 W 16th St', 'Chicago', 'IL', 60623, '(773) 534-1690', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610254.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 46.0, 'Weak', 22.0, 'Strong', 66.0, 'Strong', 69.0, 'Weak', 4.0, 'Very Weak', 13.0, nan, nan, nan, nan, '94.20%', 4.4, '95.40%', '100.00%', 49.0, 42.3, 20.5, 22.4, 45.8, 43.1, 25.1, 26.3, 38.9, 53.6, 3.6, 30.4, 11.0, 10.4, -1.5, 0.4, 'Red', 'Yellow', 20.3, 16.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 556, 37, nan, 1152488.851, 1891762.257, 41.85885599, -87.71574626, 29, 'NORTH LAWNDALE', 24, 10, '(41.85885599, -87.71574626)') ,\n(609736, 'Dyett High School', 'HS', '555 E 51st St', 'Chicago', 'IL', 60615, '(773) 535-1825', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609736.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 27.0, 'Very Weak', 18.0, 'Weak', 35.0, 'Average', 47.0, 'Weak', 35.0, 'Weak', 25.0, nan, nan, nan, nan, '62.50%', 24.4, '93.50%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.6, 11.7, 13.0, 12.9, 1.3, 14.0, 1.0, 3.3, 33.7, 51.0, 318, 42, 37.1, 1180944.201, 1871282.832, 41.80204982, -87.61192836, 40, 'WASHINGTON PARK', 4, 2, '(41.80204982, -87.61192836)') ,\n(610132, 'Edgar Allan Poe Elementary Classical School', 'ES', '10538 S Langley Ave', 'Chicago', 'IL', 60628, '(773) 535-5525', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610132.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Weak', 33.0, 'Strong', 66.0, 'Very Strong', 88.0, 'Weak', 27.0, 'Average', 41.0, 'Strong', 55.0, 'Average', 47.0, '97.60%', 0.0, '97.20%', '100.00%', 100.0, 78.3, 94.0, 91.6, 51.8, 65.5, 78.6, 92.9, 53.6, 60.7, nan, nan, 78.8, 79.6, 1.7, -0.2, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 192, 48, nan, 1182742.505, 1835063.9, 41.70261965, -87.60645552, 50, 'PULLMAN', 9, 5, '(41.70261965, -87.60645552)') ,\n(609901, 'Edgebrook Elementary School', 'ES', '6525 N Hiawatha Ave', 'Chicago', 'IL', 60646, '(773) 534-1194', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609901.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Average', 51.0, 'Average', 53.0, 'Weak', 66.0, 'Strong', 70.0, 'Strong', 56.0, 'Weak', 39.0, '96.90%', 3.4, '96.60%', '100.00%', 96.3, 47.6, 88.1, 74.1, 55.2, 75.2, 80.0, 79.1, 54.5, 60.0, 50.0, 71.4, 60.2, 56.8, 0.0, 0.5, 'Yellow', 'Yellow', 83.3, 64.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 468, 30, nan, 1139595.896, 1942911.685, 41.99946016, -87.76182109999999, 12, 'FOREST GLEN', 41, 16, '(41.99946016, -87.7618211)') ,\n(610523, 'Edison Park Elementary School', 'ES', '6220 N Olcott Ave', 'Chicago', 'IL', 60631, '(773) 534-0960', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610523.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 95.0, nan, nan, 'Average', 52.0, 'Average', 49.0, 'Weak', nan, nan, nan, 'Strong', 56.0, 'Weak', 46.0, '96.10%', 6.9, '96.60%', '98.60%', 78.1, nan, 64.2, 62.6, 54.7, 49.1, 59.4, 61.4, 38.8, 48.5, 40.0, 55.0, 27.7, 32.6, -0.6, -0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 431, 30, nan, 1125476.537, 1940915.071, 41.99422812, -87.81380814, 10, 'NORWOOD PARK', 41, 16, '(41.99422812, -87.81380814)') ,\n(609819, 'Edmond Burke Elementary School', 'ES', '5356 S King Dr', 'Chicago', 'IL', 60615, '(773) 535-1325', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609819.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 1.0, nan, nan, 'Very Weak', 13.0, 'Weak', 22.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 48.0, '89.30%', 82.0, '95.50%', '96.70%', 44.4, 57.4, 30.3, 16.2, 40.7, 40.4, 37.6, 24.7, 63.3, 45.5, 3.6, 14.3, 7.9, 4.3, -1.4, -1.9, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 244, 42, nan, 1179801.469, 1869357.523, 41.79679285, -87.61617809, 40, 'WASHINGTON PARK', 3, 2, '(41.79679285, -87.61617809)') ,\n(609815, 'Edward A Bouchet Math & Science Academy Elementary School', 'ES', '7355 S Jeffery Blvd', 'Chicago', 'IL', 60649, '(773) 535-0501', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609815.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 33.0, 'Average', 57.0, 'Average', 55.0, 'Average', 50.0, 'Weak', 53.0, 'Average', 58.0, 'Average', 50.0, 'Average', 48.0, '92.90%', 28.0, '95.90%', '98.30%', 57.6, 42.4, 13.7, 21.7, 51.6, 44.8, 15.7, 34.7, 48.2, 66.5, 0.0, 23.7, 5.6, 7.6, -0.2, 0.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 783, 46, nan, 1190861.978, 1856358.0380000002, 41.76086093, -87.57603849, 43, 'SOUTH SHORE', 5, 3, '(41.76086093, -87.57603849)') ,\n(610246, 'Edward Beasley Elementary Magnet Academic Center', 'ES', '5255 S State St', 'Chicago', 'IL', 60609, '(773) 535-1230', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610246.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 47.0, nan, nan, 'Weak', 34.0, 'Average', 48.0, 'Weak', nan, nan, nan, 'Weak', 42.0, 'Weak', 45.0, '95.00%', 10.3, '94.70%', '100.00%', 74.0, 54.1, 39.8, 38.7, 43.6, 47.5, 53.8, 55.4, 57.3, 54.2, 27.0, 43.0, 26.4, 21.3, 0.3, 0.3, 'Yellow', 'Yellow', 38.6, 76.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1365, 42, nan, 1177212.26, 1869948.081, 41.79847228, -87.62565509, 40, 'WASHINGTON PARK', 3, 2, '(41.79847228, -87.62565509)') ,\n(609881, 'Edward C Delano Elementary School', 'ES', '3937 W Wilcox St', 'Chicago', 'IL', 60624, '(773) 534-6620', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609881.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 38.0, 'Strong', 64.0, 'Strong', 66.0, 'Average', 52.0, 'Weak', 63.0, 'Strong', 79.0, 'Strong', 55.0, 'Strong', 58.0, '93.40%', 20.3, '94.70%', '94.70%', 88.4, 76.7, 26.7, 18.0, 62.6, 62.0, 41.5, 32.0, 67.1, 69.3, 3.4, 6.9, 6.8, 10.9, 1.0, 1.7, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 414, 34, nan, 1150012.228, 1899002.166, 41.87877166, -87.72464875, 26, 'WEST GARFIELD PARK', 28, 11, '(41.87877166, -87.72464875)') ,\n(609862, 'Edward Coles Elementary Language Academy', 'ES', '8441 S Yates Blvd', 'Chicago', 'IL', 60617, '(773) 535-6550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609862.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 22.0, 'Average', 43.0, 'Average', 40.0, 'Average', 54.0, 'Weak', 52.0, 'Average', 53.0, 'Average', 50.0, 'Average', 51.0, '95.10%', 24.9, '95.00%', '98.70%', 76.2, 60.0, 44.6, 26.4, 54.7, 72.5, 45.5, 22.4, 72.1, 52.6, 0.0, 11.4, 16.7, 5.7, 1.0, -0.1, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 554, 47, nan, 1193682.146, 1849300.8630000001, 41.74142687, -87.56593332, 46, 'SOUTH CHICAGO', 7, 4, '(41.74142687, -87.56593332)') ,\n(609909, 'Edward Everett Elementary School', 'ES', '3419 S Bell Ave', 'Chicago', 'IL', 60608, '(773) 535-4550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609909.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', nan, nan, 'Strong', 66.0, nan, nan, nan, nan, 'Weak', 47.0, 'Average', 45.0, nan, nan, nan, nan, '95.30%', 2.4, '97.70%', '93.80%', 64.4, 47.0, 37.2, 35.0, 56.9, 52.2, nan, nan, nan, nan, nan, nan, 13.6, 13.6, 0.6, 1.3, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 321, 39, nan, 1161955.375, 1881878.553, 41.83154196, -87.68127322, 59, 'MCKINLEY PARK', 11, 9, '(41.83154196, -87.68127322)') ,\n(610012, 'Edward Jenner Elementary Academy of the Arts', 'ES', '1119 N Cleveland Ave', 'Chicago', 'IL', 60610, '(773) 534-8440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610012.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 50.0, 'Weak', 34.0, 'Strong', 62.0, 'Average', 41.0, 'Weak', 20.0, 'Weak', 26.0, 'Average', 49.0, 'Average', 53.0, '88.90%', 16.0, '94.80%', '98.70%', 34.8, 21.3, 21.9, 10.7, 42.3, 57.7, 37.1, 14.5, 69.4, 46.7, 4.3, 25.5, 8.2, 1.3, 2.0, 0.2, 'Green', 'Yellow', 45.8, 31.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 314, 33, nan, 1172718.668, 1907660.3390000002, 41.90205805, -87.64101894, 8, 'NEAR NORTH SIDE', 27, 18, '(41.90205805, -87.64101894)') ,\n(609904, 'Edward K Ellington Elementary School', 'ES', '243 N Parkside Ave', 'Chicago', 'IL', 60644, '(773) 534-6361', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609904.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 30.0, nan, nan, 'Weak', 37.0, 'Average', 55.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Average', 50.0, '93.30%', 17.4, '97.30%', '98.90%', 56.6, 46.9, 29.5, 33.3, 60.6, 64.5, 43.6, 29.9, 69.3, 60.8, 11.1, 14.8, 16.1, 8.0, 1.9, 0.4, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 377, 36, nan, 1138584.642, 1901378.0019999999, 41.885506, -87.76655157, 25, 'AUSTIN', 29, 15, '(41.885506, -87.76655157)') ,\n(610006, 'Edward N Hurley Elementary School', 'ES', '3849 W 69th Pl', 'Chicago', 'IL', 60629, '(773) 535-2068', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610006.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 77.0, 'Average', 56.0, 'Average', 50.0, 'Average', 49.0, 'Weak', 43.0, 'Average', 54.0, 'Average', 49.0, 'Weak', 46.0, '95.40%', 12.2, '94.90%', '96.60%', 74.8, 51.7, 47.6, 32.1, 56.6, 58.4, 47.0, 39.6, 54.0, 60.7, 19.0, 26.7, 16.4, 11.7, 0.0, 0.8, 'Yellow', 'Green', nan, 30.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 830, 44, nan, 1151703.8020000001, 1858195.0380000002, 41.7667583, -87.71950804, 65, 'WEST LAWN', 13, 8, '(41.7667583, -87.71950804)') ,\n(609735, 'Edward Tilden Career Community Academy High School', 'HS', '4747 S Union Ave', 'Chicago', 'IL', 60609, '(773) 535-1625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609735.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 34.0, 'Average', 57.0, 'Average', 45.0, 'Weak', 37.0, 'Weak', 54.0, 'Weak', 35.0, 'Average', 51.0, 'Average', 47.0, '78.40%', 13.0, '95.20%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.6, 12.0, 12.9, 12.9, 1.3, 13.5, 0.6, 10.5, 41.1, 37.0, 461, 42, 86.5, 1172475.4740000002, 1873203.826, 41.80751207, -87.64292985, 61, 'NEW CITY', 11, 9, '(41.80751207, -87.64292985)') ,\n(610315, 'Edward White Elementary Career Academy', 'ES', '1136 W 122nd St', 'Chicago', 'IL', 60643, '(773) 535-5672', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610315.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 27.0, nan, nan, 'Average', 44.0, 'Average', 48.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Weak', 46.0, '93.80%', 31.0, '96.70%', '100.00%', 64.9, 35.1, 16.3, 32.6, 67.6, 47.4, 31.6, 34.2, 57.6, 81.8, 14.3, 28.6, 7.4, 7.4, 1.9, 1.2, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 138, 49, nan, 1170787.9, 1823930.602, 41.67233661, -87.65055321, 53, 'WEST PULLMAN', 34, 5, '(41.67233661, -87.65055321)') ,\n(609708, 'Edwin G Foreman High School', 'HS', '3235 N LeClaire Ave', 'Chicago', 'IL', 60641, '(773) 534-3400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609708.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 26.0, nan, nan, 'Weak', 28.0, 'Weak', 23.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '84.30%', 10.3, '94.80%', '99.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.0, 12.7, 14.1, 13.7, 0.7, 15.3, 1.2, 12.5, 59.7, 40.9, 1716, 29, 75.6, 1141833.3590000002, 1921189.138, 41.93981043, -87.7541302, 15, 'PORTAGE PARK', 30, 16, '(41.93981043, -87.7541302)') ,\n(609913, 'Elaine O Goodlow Elementary Magnet School', 'ES', '2040 W 62nd St', 'Chicago', 'IL', 60636, '(773) 535-9365', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609913.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 44.0, 'Average', 42.0, 'Average', 52.0, 'Average', 56.0, 'Weak', 35.0, 'Weak', 28.0, 'Weak', 46.0, 'Weak', 46.0, '91.60%', 47.5, '96.00%', '100.00%', 77.8, 67.9, 40.5, 26.0, 50.4, 67.7, 35.4, 29.2, 61.0, 50.0, 4.0, 8.0, 9.8, 8.6, -0.2, -0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 419, 43, nan, 1163653.744, 1863565.274, 41.78125258, -87.67555599, 67, 'WEST ENGLEWOOD', 15, 7, '(41.78125258, -87.67555599)') ,\n(610233, 'Elihu Yale Elementary School', 'ES', '7025 S Princeton Ave', 'Chicago', 'IL', 60621, '(773) 535-3190', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610233.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 29.0, 'Weak', 35.0, 'Average', 43.0, 'Average', 50.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Average', 47.0, '89.70%', 67.2, '97.00%', '100.00%', 16.4, 6.9, 23.3, 19.1, 39.1, 38.2, 31.9, 15.7, 62.5, 50.0, 0.0, 12.9, 4.5, 3.2, -0.1, -1.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 182, 45, nan, 1175551.272, 1858262.1940000001, 41.76644232, -87.63209571, 69, 'GREATER GRAND CROSSING', 6, 7, '(41.76644232, -87.63209571)') ,\n(610227, 'Eli Whitney Elementary School', 'ES', '2815 S Komensky Ave', 'Chicago', 'IL', 60623, '(773) 534-1560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610227.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', 'Average', 44.0, nan, nan, 'Average', 49.0, 'Average', 53.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.80%', 11.6, '95.20%', '97.60%', 68.7, 26.5, 30.3, 26.1, 54.5, 60.8, 41.5, 29.2, 55.3, 54.3, 6.8, 12.6, 15.0, 9.4, -0.1, 0.7, 'Yellow', 'Yellow', 9.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1133, 37, nan, 1149877.98, 1884894.804, 41.84006197, -87.72550840000001, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84006197, -87.7255084)') ,\n(610195, 'Elizabeth H Sutherland Elementary School', 'ES', '10015 S Leavitt St', 'Chicago', 'IL', 60643, '(773) 535-2580', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610195.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Very Strong', 86.0, nan, nan, 'Strong', 64.0, 'Average', 59.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Weak', 40.0, '95.60%', 6.8, '96.20%', '96.00%', 87.7, nan, 56.6, 72.8, 66.5, 67.6, 66.9, 84.7, 70.2, 68.1, 29.3, 60.9, 37.2, 35.7, 0.0, -0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 762, 49, nan, 1163512.956, 1838118.8909999998, 41.71142687, -87.67678398, 72, 'BEVERLY', 19, 22, '(41.71142687, -87.67678398)') ,\n(610119, 'Elizabeth Peabody Elementary School', 'ES', '1444 W Augusta Blvd', 'Chicago', 'IL', 60622, '(773) 534-4170', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610119.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 51.0, 'Average', 58.0, 'Average', 50.0, 'Strong', 61.0, 'Weak', 57.0, 'Average', 55.0, 'Average', 52.0, 'Average', 49.0, '95.20%', 7.6, '97.00%', '100.00%', 66.2, nan, 18.4, 10.5, 32.4, 43.5, 45.0, 30.4, 56.4, 46.7, 14.3, 28.6, 9.1, 5.9, -1.1, -0.8, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 289, 35, nan, 1166416.965, 1906812.638, 41.89986912, -87.66419014, 24, 'WEST TOWN', 27, 13, '(41.89986912, -87.66419014)') ,\n(609852, 'Eliza Chappell Elementary School', 'ES', '2135 W Foster Ave', 'Chicago', 'IL', 60625, '(773) 534-2390', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609852.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 70.0, 'Strong', 65.0, 'Average', 53.0, 'Average', 51.0, 'Weak', 56.0, 'Average', 48.0, 'Average', 50.0, 'Average', 52.0, '95.10%', 2.9, '96.70%', '98.90%', 72.6, 57.0, 56.0, 54.4, 55.2, 69.0, 53.2, 63.9, 60.0, 59.8, 12.8, 41.0, 19.6, 17.6, 0.4, 1.0, 'Yellow', 'Green', 27.5, 63.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 451, 31, nan, 1161016.902, 1934466.545, 41.975867, -87.68325438, 4, 'LINCOLN SQUARE', 47, 20, '(41.975867, -87.68325438)') ,\n(610235, 'Ella Flagg Young Elementary School', 'ES', '1434 N Parkside Ave', 'Chicago', 'IL', 60651, '(773) 534-6200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610235.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 31.0, nan, nan, 'Average', 44.0, 'Average', 50.0, 'Weak', nan, nan, nan, 'Weak', 44.0, 'Average', 47.0, '94.30%', 3.0, '95.40%', '100.00%', 55.4, 29.6, 21.3, 24.9, 44.4, 44.3, 32.9, 28.2, 61.5, 54.9, 11.5, 16.1, 9.1, 8.0, 0.1, 0.8, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1111, 36, nan, 1138362.5359999998, 1909130.405, 41.90678358, -87.76717934, 25, 'AUSTIN', 29, 25, '(41.90678358, -87.76717934)') ,\n(609682, 'Ellen H Richards Career Academy High School', 'HS', '5009 S Laflin St', 'Chicago', 'IL', 60609, '(773) 535-4945', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609682.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 30.0, 'Weak', 23.0, 'Weak', 32.0, 'Very Weak', 19.0, 'Weak', 42.0, 'Weak', 34.0, 'Weak', 43.0, 'Average', 47.0, '90.60%', 17.3, '95.70%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.8, 12.0, 13.1, 12.8, 1.0, 14.0, 0.9, 5.5, 57.3, 43.1, 473, 42, 63.5, 1167224.006, 1871465.305, 41.80285552, -87.66224052, 61, 'NEW CITY', 16, 9, '(41.80285552, -87.66224052)') ,\n(610073, 'Ellen Mitchell Elementary School', 'ES', '2233 W Ohio St', 'Chicago', 'IL', 60612, '(773) 534-7655', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610073.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Strong', 64.0, 'Very Strong', 95.0, 'Very Strong', 95.0, 'Weak', 81.0, 'Very Strong', 90.0, 'Strong', 56.0, 'Strong', 54.0, '95.50%', 12.2, '97.70%', '100.00%', 46.7, 41.9, 53.1, 43.9, 72.4, 75.5, 35.5, 51.3, 55.3, 55.3, 21.1, 26.3, 30.1, 27.3, -0.5, -0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 344, 35, nan, 1161269.097, 1903925.361, 41.89205482, -87.68317867, 24, 'WEST TOWN', 26, 13, '(41.89205482, -87.68317867)') ,\n(609712, 'Emil G Hirsch Metropolitan High School', 'HS', '7740 S Ingleside Ave', 'Chicago', 'IL', 60619, '(773) 535-3100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609712.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 13.0, 'Average', 46.0, 'Weak', 28.0, 'Weak', 28.0, 'Weak', 52.0, 'Average', 50.0, nan, nan, nan, nan, '84.80%', 47.1, '95.40%', '88.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.5, 11.2, 12.9, 12.4, 0.9, 14.1, 1.2, 10.8, 36.2, 45.1, 458, 46, 69.5, 1183865.127, 1853710.216, 41.7537612, -87.60176452, 69, 'GREATER GRAND CROSSING', 8, 6, '(41.7537612, -87.60176452)') ,\n(609973, 'Emiliano Zapata Elementary Academy', 'ES', '2728 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 534-1390', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609973.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 60.0, 'Average', 52.0, 'Average', 43.0, 'Average', 42.0, 'Weak', 73.0, 'Strong', 79.0, 'Weak', 46.0, 'Average', 49.0, '96.60%', 2.8, '96.80%', '100.00%', nan, nan, 51.6, 40.3, 68.9, 76.2, 44.5, 45.5, 60.5, 64.3, 15.5, 35.7, 21.6, 13.7, -0.3, 1.2, 'Yellow', 'Green', 20.2, 52.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 988, 37, nan, 1147465.325, 1885362.468, 41.84139181, -87.73434997, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84139181, -87.73434997)') ,\n(610065, 'Emmett Louis Till Math and Science Academy', 'ES', '6543 S Champlain Ave', 'Chicago', 'IL', 60637, '(773) 535-0570', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610065.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Weak', 31.0, 'Average', 47.0, 'Average', 40.0, 'Weak', 49.0, 'Average', 42.0, 'Average', 47.0, 'Average', 48.0, '88.00%', 95.7, '93.30%', '90.50%', 34.3, nan, 15.8, 7.9, 39.4, 43.1, 15.1, 15.5, 43.6, 52.5, 4.5, 23.3, 3.2, 3.5, -0.4, -1.6, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 471, 46, nan, 1181736.7820000001, 1861592.586, 41.77544062, -87.60932095, 42, 'WOODLAWN', 20, 3, '(41.77544062, -87.60932095)') ,\n(609916, 'Enrico Fermi Elementary School', 'ES', '1415 E 70th St', 'Chicago', 'IL', 60637, '(773) 535-0540', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609916.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 30.0, 'Average', 42.0, 'Strong', 65.0, 'Strong', 63.0, 'Weak', 42.0, 'Average', 47.0, nan, nan, nan, nan, '89.90%', 101.2, '95.40%', '100.00%', 43.8, 8.3, 10.7, 18.1, 35.9, 26.6, 6.3, 7.9, 32.7, 32.7, 0.0, 11.5, 0.6, 2.6, -1.0, -1.0, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 238, 46, nan, 1186924.7859999998, 1858826.102, 41.76772772, -87.59039017, 43, 'SOUTH SHORE', 5, 3, '(41.76772772, -87.59039017)') ,\n(610203, 'Enrico Tonti Elementary School', 'ES', '5815 S Homan Ave', 'Chicago', 'IL', 60629, '(773) 535-9280', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610203.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', nan, nan, 'Average', 44.0, nan, nan, nan, nan, 'Weak', 47.0, 'Average', 54.0, 'Average', 49.0, 'Average', 49.0, '95.80%', 2.1, '97.20%', '100.00%', 73.4, 35.7, 37.9, 24.8, 47.9, 59.5, nan, nan, nan, nan, nan, nan, 17.5, 13.0, 0.7, 0.3, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1021, 44, nan, 1154749.55, 1865777.301, 41.78750499, -87.70814201, 63, 'GAGE PARK', 14, 8, '(41.78750499, -87.70814201)') ,\n(610543, 'Eric Solorio Academy High School', 'HS', '5400 S St Louis Ave', 'Chicago', 'IL', 60632, '(773) 535-9070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610543.pdf', 'AUSL Schools', 'SOUTHWEST SIDE COLLABORATIVE', nan, 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Strong', 67.0, 'Average', 54.0, 'Average', 53.0, 'Strong', 63.0, 'Weak', 62.0, 'Strong', 64.0, 'Average', 53.0, 'Strong', 56.0, '92.20%', 3.0, '0.00%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.9, nan, nan, nan, nan, nan, nan, nan, nan, 581, 44, 85.6, 1153924.117, 1868568.611, 41.79518118, -87.71109447, 63, 'GAGE PARK', 14, 8, '(41.79518118, -87.71109447)') ,\n(610137, 'Ernst Prussing Elementary School', 'ES', '4650 N Menard Ave', 'Chicago', 'IL', 60630, '(773) 534-3460', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610137.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 71.0, nan, nan, 'Average', 49.0, 'Average', 42.0, 'Weak', nan, nan, nan, 'Weak', 43.0, 'Weak', 44.0, '95.40%', 23.4, '96.00%', '98.50%', 84.6, nan, 55.4, 54.0, 48.3, 58.2, 56.0, 57.4, 52.3, 46.3, 47.2, 75.5, 31.4, 24.5, -0.8, -0.1, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 672, 30, nan, 1136787.381, 1930557.622, 41.96561054, -87.77245066, 15, 'PORTAGE PARK', 45, 16, '(41.96561054, -87.77245066)') ,\n(609908, 'Esmond Elementary School', 'ES', '1865 W Montvale', 'Chicago', 'IL', 60643, '(773) 535-2650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609908.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 36.0, nan, nan, 'Average', 49.0, 'Average', 43.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Average', 50.0, '92.90%', 34.0, '95.70%', '100.00%', 77.6, 38.8, 33.3, 26.3, 52.0, 47.5, 32.0, 22.1, 55.2, 37.5, 8.6, 17.1, 8.9, 7.7, 0.9, -0.4, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 342, 49, nan, 1165656.136, 1830105.142, 41.68939069, -87.66916143, 75, 'MORGAN PARK', 19, 22, '(41.68939069, -87.66916143)') ,\n(609918, 'Eugene Field Elementary School', 'ES', '7019 N Ashland Ave', 'Chicago', 'IL', 60626, '(773) 534-2030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609918.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 42.0, 'Weak', 33.0, 'Average', 47.0, 'Weak', 37.0, 'Weak', 21.0, 'Very Weak', 8.0, 'Weak', 44.0, 'Average', 53.0, '96.80%', 10.0, '96.70%', '100.00%', nan, nan, 23.9, 24.3, 49.4, 51.4, 48.4, 36.4, 70.3, 53.2, 7.4, 13.0, 9.8, 7.9, -0.2, 0.1, 'Yellow', 'Yellow', 45.6, 20.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 375, 32, nan, 1164539.561, 1946810.3380000002, 42.00966462, -87.66994862, 1, 'ROGERS PARK', 49, 24, '(42.00966462, -87.66994862)') ,\n(609874, 'Everett McKinley Dirksen Elementary School', 'ES', '8601 W Foster Ave', 'Chicago', 'IL', 60656, '(773) 534-1090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609874.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 44.0, nan, nan, 'Very Weak', 12.0, 'Very Weak', 1.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.90%', 2.7, '96.20%', '93.80%', 69.9, 45.8, 65.1, 56.9, 55.5, 55.2, 51.9, 45.1, 45.6, 40.7, 22.5, 35.2, 32.8, 25.5, -0.5, -0.1, 'Yellow', 'Yellow', 6.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 786, 30, nan, 1118113.731, 1933386.1509999998, 41.97368728, -87.84105162, 76, 'OHARE', 41, 16, '(41.97368728, -87.84105162)') ,\n(610319, 'Evergreen Academy Middle School', 'MS', '3537 S Paulina St', 'Chicago', 'IL', 60609, '(773) 535-4836', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610319.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 60.0, 'Average', 58.0, 'Average', 47.0, 'Average', 44.0, 'Weak', 59.0, 'Strong', 65.0, 'Weak', 43.0, 'Average', 48.0, '96.10%', 5.5, '97.40%', '100.00%', nan, nan, nan, nan, nan, nan, 37.6, 31.6, 54.4, 57.5, 13.3, 21.7, 11.5, 9.3, -0.3, -0.5, 'Yellow', 'Yellow', 37.8, 43.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 389, 39, nan, 1165638.817, 1881113.479, 41.829365, -87.66778007, 59, 'MCKINLEY PARK', 11, 9, '(41.829365, -87.66778007)') ,\n(610057, 'Fairfield Elementary Academy', 'ES', '6201 S Fairfield Ave', 'Chicago', 'IL', 60629, '(773) 535-9500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610057.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Weak', 24.0, 'Average', 41.0, 'Weak', 34.0, 'Weak', 45.0, 'Weak', 25.0, nan, nan, nan, nan, '93.40%', 43.4, '93.30%', '98.60%', 49.0, 36.8, 15.9, 11.1, 27.3, 42.0, 20.5, 17.8, 44.7, 35.8, 0.0, 12.5, 4.1, 4.3, 0.1, -0.7, 'Yellow', 'Yellow', 16.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 604, 43, nan, 1159148.629, 1863391.0569999998, 41.78086789, -87.69207758, 66, 'CHICAGO LAWN', 15, 8, '(41.78086789, -87.69207758)') ,\n(610541, 'Federico Garcia Lorca Elementary School', 'ES', '3231 N Springfield Ave', 'Chicago', 'IL', 60618, '(773) 534-0950', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610541.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 53.0, 'Average', 58.0, 'Average', 53.0, 'Average', 58.0, 'Weak', 74.0, 'Very Strong', 82.0, 'Average', 51.0, 'Strong', 59.0, '96.80%', 43.8, '0.00%', '100.00%', 63.0, 41.7, 39.1, 25.6, 51.5, 63.8, 37.8, 27.3, 72.2, 52.3, 7.7, 25.0, 8.9, 7.5, -1.0, -0.1, 'Red', 'Yellow', 39.4, 42.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 860, 29, nan, 1149828.0590000001, 1921320.477, 41.94001885, -87.72474354, 21, 'AVONDALE', 30, 17, '(41.94001885, -87.72474354)') ,\n(610308, 'Wilma Rudolph Elementary Learning Center', 'ES', '110 N Paulina St', 'Chicago', 'IL', 60612, '(773) 534-7460', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610308.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', nan, 'Standard', 'Not Applicable', 'Not Enough Data', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Strong', 57.0, 'Strong', 60.0, '80.00%', 0.0, '95.60%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 100, 38, nan, 1165013.632, 1900863.727, 41.88357478, -87.66951363, 28, 'NEAR WEST SIDE', 27, 13, '(41.88357478, -87.66951363)') ,\n(610120, 'Ferdinand Peck Elementary School', 'ES', '3826 W 58th St', 'Chicago', 'IL', 60629, '(773) 535-2450', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610120.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 51.0, 'Average', 43.0, 'Average', 43.0, 'Average', 47.0, 'Weak', 32.0, 'Weak', 37.0, 'Weak', 44.0, 'Weak', 43.0, '96.30%', 1.2, '97.20%', '96.90%', 74.4, 59.0, 50.8, 31.3, 53.3, 69.5, 44.0, 40.2, 71.4, 64.1, 24.1, 38.6, 23.2, 13.8, 0.7, 1.1, 'Green', 'Green', 17.8, 87.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1560, 44, nan, 1151726.72, 1865909.055, 41.78792632, -87.71922211, 62, 'WEST ELSDON', 13, 8, '(41.78792632, -87.71922211)') ,\n(609917, 'Fernwood Elementary School', 'ES', '10041 S Union Ave', 'Chicago', 'IL', 60628, '(773) 535-2700', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609917.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 29.0, 'Weak', 37.0, 'Very Weak', 17.0, 'Average', 43.0, 'Weak', 39.0, 'Average', 50.0, 'Average', 48.0, 'Average', 53.0, '93.40%', 29.0, '96.10%', '100.00%', 70.5, 43.3, 22.5, 12.7, 35.2, 55.4, 31.1, 20.4, 38.7, 34.8, 15.6, 34.4, 12.4, 7.8, -0.7, -1.7, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 344, 49, nan, 1173454.065, 1838113.389, 41.71119799, -87.6403774, 73, 'WASHINGTON HEIGHTS', 34, 22, '(41.71119799, -87.6403774)') ,\n(610258, 'Florence B Price Elementary School', 'ES', '4351 S Drexel Blvd', 'Chicago', 'IL', 60653, '(773) 535-1300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610258.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 20.0, nan, nan, 'Weak', 37.0, 'Average', 46.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Average', 47.0, '90.00%', 156.6, '95.20%', '100.00%', nan, nan, 10.0, 13.5, 46.9, 41.7, 7.8, 27.3, 44.6, 50.7, 0.0, 6.9, 0.0, 0.7, -1.6, -0.2, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 112, 40, nan, 1183099.293, 1876145.197, 41.81534262, -87.60387349999999, 39, 'KENWOOD', 4, 2, '(41.81534262, -87.6038735)') ,\n(610096, 'Florence Nightingale Elementary School', 'ES', '5250 S Rockwell St', 'Chicago', 'IL', 60632, '(773) 535-9270', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610096.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, 'Average', 54.0, 'Average', 40.0, 'Average', 48.0, 'Weak', 68.0, 'Strong', 72.0, 'Average', 48.0, 'Average', 48.0, '95.60%', 9.4, '95.80%', '99.40%', 57.1, 35.1, 38.7, 25.6, 48.7, 61.6, 32.4, 29.1, 47.7, 54.1, 10.7, 17.4, 11.3, 8.2, -0.8, -1.2, 'Red', 'Red', 23.3, 32.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1382, 43, nan, 1159893.892, 1869545.12, 41.79774019, -87.68917624, 63, 'GAGE PARK', 14, 9, '(41.79774019, -87.68917624)') ,\n(609924, 'Fort Dearborn Elementary School', 'ES', '9025 S Throop St', 'Chicago', 'IL', 60620, '(773) 535-2680', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609924.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 34.0, nan, nan, 'Strong', 61.0, 'Strong', 71.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 50.0, '93.40%', 24.5, '95.90%', '100.00%', 60.1, 48.3, 36.6, 35.8, 52.0, 64.5, 69.4, 44.2, 74.7, 70.2, 7.0, 33.3, 25.2, 14.1, 1.4, 0.1, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 478, 49, nan, 1169287.183, 1844810.5380000002, 41.72966698, -87.65544435, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.72966698, -87.65544435)') ,\n(609927, 'Foster Park Elementary School', 'ES', '8530 S Wood St', 'Chicago', 'IL', 60620, '(773) 535-2725', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609927.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 11.0, 'Average', 41.0, 'Weak', 20.0, 'Weak', 22.0, 'Weak', 50.0, 'Average', 46.0, 'Weak', 46.0, 'Average', 48.0, '93.70%', 33.6, '94.70%', '97.10%', nan, 59.1, 23.4, 17.1, 41.2, 40.3, 29.2, 15.9, 45.1, 29.3, 4.0, 25.5, 8.0, 5.7, -1.1, -2.1, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 418, 43, nan, 1165798.179, 1847975.2780000002, 41.738426200000006, -87.66813597, 71, 'AUBURN GRESHAM', 18, 6, '(41.7384262, -87.66813597)') ,\n(610215, 'Francisco I Madero Middle School', 'MS', '3202 W 28th St', 'Chicago', 'IL', 60623, '(773) 535-4466', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610215.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'Yes', 'Average', 49.0, 'Weak', 32.0, 'Average', 48.0, 'Average', 47.0, 'Weak', 49.0, 'Weak', 34.0, 'Average', 47.0, 'Average', 52.0, '95.10%', 7.1, '97.00%', '100.00%', nan, nan, nan, nan, nan, nan, 28.5, 28.4, 34.3, 54.5, 10.5, 25.3, 15.1, 4.9, -1.1, -0.7, 'Red', 'Yellow', 16.5, 50.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 314, 37, nan, 1155440.547, 1885270.824, 41.84098398, -87.70508585, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84098398, -87.70508585)') ,\n(610067, 'Francis M McKay Elementary School', 'ES', '6901 S Fairfield Ave', 'Chicago', 'IL', 60629, '(773) 535-9340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610067.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 23.0, 'Average', 48.0, 'Weak', 37.0, 'Average', 54.0, 'Weak', 67.0, 'Strong', 61.0, 'Average', 48.0, 'Average', 51.0, '93.50%', 9.1, '95.10%', '100.00%', 51.4, 34.2, 16.0, 13.0, 33.2, 47.6, 23.4, 21.6, 59.1, 47.5, 5.1, 13.9, 5.7, 5.7, -0.6, 0.2, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 891, 43, nan, 1159272.821, 1858741.621, 41.76810662, -87.69174944, 66, 'CHICAGO LAWN', 18, 8, '(41.76810662, -87.69174944)') ,\n(610114, 'Francis Parkman Elementary School', 'ES', '245 W 51st St', 'Chicago', 'IL', 60609, '(773) 535-1739', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610114.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 38.0, 'Weak', 25.0, 'Average', 45.0, 'Weak', 39.0, 'Weak', 20.0, 'Weak', 28.0, nan, nan, nan, nan, '90.60%', 20.7, '95.30%', '100.00%', 62.2, 35.9, 15.1, 5.9, 48.8, 34.1, 14.8, 15.8, 39.3, 43.4, 6.3, 12.5, 4.6, 3.1, -0.3, -0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 224, 42, nan, 1175207.047, 1871107.604, 41.80169919, -87.63297392, 37, 'FULLER PARK', 3, 9, '(41.80169919, -87.63297392)') ,\n(610020, 'Francis Scott Key Elementary School', 'ES', '517 N Parkside Ave', 'Chicago', 'IL', 60644, '(773) 534-6230', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610020.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 37.0, nan, nan, 'Average', 49.0, 'Strong', 64.0, 'Weak', nan, nan, nan, 'Weak', 44.0, 'Weak', 45.0, '91.90%', 39.6, '94.00%', '100.00%', 54.5, 44.4, 27.2, 26.3, 59.8, 55.1, 26.9, 25.9, 69.4, 63.3, 7.5, 7.5, 11.3, 5.9, 0.9, 1.5, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 301, 36, nan, 1138633.137, 1902956.198, 41.88983589, -87.76633519, 25, 'AUSTIN', 29, 15, '(41.88983589, -87.76633519)') ,\n(610112, 'Francis W Parker Elementary Community Academy', 'ES', '6800 S Stewart Ave', 'Chicago', 'IL', 60621, '(773) 535-3375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610112.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 30.0, nan, nan, 'Average', 53.0, 'Average', 57.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 51.0, '91.90%', 39.0, '94.90%', '98.60%', nan, 30.9, 25.6, 14.2, 39.6, 55.7, 25.3, 27.3, 60.0, 60.0, 7.7, 14.1, 8.4, 6.5, 0.4, 0.0, 'Yellow', 'Yellow', 10.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 808, 45, nan, 1174661.5459999999, 1859841.533, 41.77079606, -87.63530990000001, 68, 'ENGLEWOOD', 6, 7, '(41.77079606, -87.6353099)') ,\n(609800, 'Frank I Bennett Elementary School', 'ES', '10115 S Prairie Ave', 'Chicago', 'IL', 60628, '(773) 535-5460', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609800.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 46.0, 'Average', 46.0, 'Average', 55.0, 'Strong', 66.0, 'Weak', 55.0, 'Average', 56.0, 'Average', 47.0, 'Weak', 45.0, '92.50%', 6.5, '95.90%', '100.00%', 57.2, 41.9, 35.3, 27.4, 56.2, 56.7, 55.2, 34.2, 60.5, 58.6, 10.0, 40.0, 13.7, 8.4, 0.7, 0.1, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 504, 48, nan, 1179742.916, 1837880.076, 41.71041656, -87.61735342, 49, 'ROSELAND', 9, 5, '(41.71041656, -87.61735342)') ,\n(609939, 'Frank L Gillespie Elementary School', 'ES', '9301 S State St', 'Chicago', 'IL', 60619, '(773) 535-5065', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609939.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 27.0, nan, nan, 'Weak', 30.0, 'Average', 45.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '91.90%', 2.1, '96.30%', '98.80%', 55.1, 33.6, 24.7, 24.0, 50.7, 58.7, 36.8, 33.8, 53.2, 52.8, 8.1, 16.2, 7.6, 5.6, 0.2, 0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 609, 48, nan, 1177970.337, 1843287.805, 41.72529635, -87.62368165, 49, 'ROSELAND', 6, 6, '(41.72529635, -87.62368165)') ,\n(609926, 'Franklin Elementary Fine Arts Center', 'ES', '225 W Evergreen Ave', 'Chicago', 'IL', 60610, '(773) 534-8510', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609926.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, nan, nan, 'Strong', 60.0, 'Average', 55.0, 'Weak', nan, nan, nan, 'Strong', 58.0, 'Average', 48.0, '95.90%', 5.6, '95.30%', '100.00%', 72.7, 42.6, 74.1, 66.9, 50.0, 53.1, 70.2, 71.3, 56.4, 54.8, 33.3, 63.6, 55.1, 50.2, 0.8, 1.4, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 363, 33, nan, 1174204.012, 1909317.563, 41.90657253, -87.6355136, 8, 'NEAR NORTH SIDE', 43, 18, '(41.90657253, -87.6355136)') ,\n(609958, 'Frank W Gunsaulus Elementary Scholastic Academy', 'ES', '4420 S Sacramento Ave', 'Chicago', 'IL', 60632, '(773) 535-7215', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609958.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 46.0, 'Weak', 38.0, 'Weak', 30.0, 'Average', 45.0, 'Weak', 34.0, 'Very Weak', 14.0, 'Average', 49.0, 'Average', 48.0, '96.60%', 87.3, '96.00%', '100.00%', 72.8, 61.1, 44.0, 32.7, 48.2, 59.2, 56.3, 51.1, 56.5, 59.6, 27.4, 42.4, 24.0, 18.1, 0.6, 1.7, 'Green', 'Green', 22.4, 100.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 867, 39, nan, 1157065.0259999998, 1875083.27, 41.81299531, -87.69940042, 58, 'BRIGHTON PARK', 14, 9, '(41.81299531, -87.69940042)') ,\n(610144, 'Frank W Reilly Elementary School', 'ES', '3650 W School St', 'Chicago', 'IL', 60618, '(773) 534-5250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610144.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 48.0, nan, nan, 'Weak', 33.0, 'Weak', 34.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 51.0, '97.00%', 8.7, '96.70%', '100.00%', 77.8, 40.1, 39.5, 27.0, 63.2, 62.7, 48.7, 33.2, 62.7, 59.2, 11.6, 30.3, 15.5, 9.8, 0.2, 0.5, 'Yellow', 'Yellow', 22.5, 69.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1195, 29, nan, 1151243.469, 1921732.4130000002, 41.94112155, -87.71953058, 21, 'AVONDALE', 35, 17, '(41.94112155, -87.71953058)') ,\n(610165, 'Franz Peter Schubert Elementary School', 'ES', '2727 N Long Ave', 'Chicago', 'IL', 60639, '(773) 534-3080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610165.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, 'Weak', 31.0, nan, nan, nan, nan, 'Weak', 32.0, 'Weak', 26.0, 'Average', 47.0, 'Average', 49.0, '95.80%', 2.1, '97.00%', '100.00%', 73.2, 35.9, 37.4, 27.3, 49.5, 56.4, nan, nan, nan, nan, nan, nan, 10.6, 10.2, -0.2, -1.1, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 949, 29, nan, 1139945.571, 1917742.0019999999, 41.93038593, -87.76115305, 19, 'BELMONT CRAGIN', 31, 25, '(41.93038593, -87.76115305)') ,\n(610503, 'Frazier Prospective IB Magnet Elementary School', 'ES', '4027 W Grenshaw St', 'Chicago', 'IL', 60624, '(773) 534-6880', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610503.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 60.0, 'Average', 55.0, 'Very Strong', 84.0, 'Strong', 72.0, 'Weak', 42.0, 'Average', 55.0, 'Strong', 66.0, 'Strong', 65.0, '96.80%', 7.2, '96.60%', '100.00%', 81.3, 68.8, 71.6, 48.6, 64.9, 71.6, 76.2, 61.9, 87.3, 82.5, 5.0, 30.0, 37.6, 25.4, 1.6, 1.9, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 220, 36, nan, 1149588.241, 1894694.332, 41.86695871, -87.72631742, 29, 'NORTH LAWNDALE', 24, 11, '(41.86695871, -87.72631742)') ,\n(609854, 'Frederic Chopin Elementary School', 'ES', '2450 W Rice St', 'Chicago', 'IL', 60622, '(773) 534-4080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609854.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 86.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Weak', 97.0, 'Very Strong', 99.0, 'Average', 49.0, 'Average', 49.0, '96.90%', 1.7, '96.00%', '100.00%', 67.5, 59.7, 59.8, 41.5, 59.8, 72.0, 59.3, 56.3, 67.4, 70.1, 60.6, 45.5, 57.6, 47.6, 3.1, 4.9, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 269, 35, nan, 1159698.085, 1905621.0559999999, 41.89674048, -87.68890154, 24, 'WEST TOWN', 1, 13, '(41.89674048, -87.68890154)') ,\n(610245, 'Frederick A Douglass Academy High School', 'HS', '543 N Waller Ave', 'Chicago', 'IL', 60644, '(773) 534-6176', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610245.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 44.0, nan, nan, 'Weak', 39.0, 'Weak', 37.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Weak', 44.0, '79.20%', 28.0, '94.70%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.0, 12.3, 13.8, 13.5, 0.5, 14.8, 1.0, 5.1, 52.8, 39.5, 354, 36, 77.1, 1138278.715, 1903151.591, 41.89037849, -87.76763207, 25, 'AUSTIN', 29, 15, '(41.89037849, -87.76763207)') ,\n(609930, 'Frederick Funston Elementary School', 'ES', '2010 N Central Park', 'Chicago', 'IL', 60647, '(773) 534-4125', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609930.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 16.0, nan, nan, 'Very Weak', 14.0, 'Very Weak', 16.0, 'Weak', nan, nan, nan, 'Weak', 40.0, 'Average', 47.0, '94.70%', 16.8, '95.40%', '100.00%', 65.1, 40.2, 18.2, 17.4, 32.6, 48.3, 26.1, 26.8, 46.4, 41.8, 18.2, 31.8, 12.2, 5.8, -2.1, -1.1, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 619, 34, nan, 1152012.963, 1913173.956, 41.91762127, -87.71692852, 22, 'LOGAN SQUARE', 26, 25, '(41.91762127, -87.71692852)') ,\n(610163, 'Frederick Stock Elementary School', 'ES', '7507 W Birchwood Ave', 'Chicago', 'IL', 60631, '(773) 534-1215', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610163.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', nan, 'Standard', nan, nan, 'No', nan, nan, 'Very Strong', 99.0, nan, nan, nan, nan, 'Weak', 84.0, 'Very Strong', 81.0, 'Strong', 69.0, 'Strong', 63.0, '72.40%', 0.0, '95.70%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 260, 30, nan, 1125318.193, 1949163.0259999998, 42.0168639, -87.81420664, 9, 'EDISON PARK', 41, 16, '(42.0168639, -87.81420664)') ,\n(610010, 'Friedrich Ludwig Jahn Elementary School', 'ES', '3149 N Wolcott Ave', 'Chicago', 'IL', 60657, '(773) 534-5500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610010.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 72.0, nan, nan, 'Strong', 60.0, 'Average', 59.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Weak', 46.0, '94.70%', 19.9, '94.70%', '100.00%', 59.4, 36.2, 41.4, 40.6, 47.9, 48.9, 30.2, 45.5, 40.6, 58.7, 13.5, 32.4, 23.9, 15.2, 0.3, 0.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 495, 33, nan, 1163153.6709999999, 1921136.53, 41.93924404, -87.67577282, 5, 'NORTH CENTER', 32, 19, '(41.93924404, -87.67577282)') ,\n(609737, 'Friedrich W von Steuben Metropolitan Science High School', 'HS', '5039 N Kimball Ave', 'Chicago', 'IL', 60625, '(773) 534-5100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609737.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 70.0, 'Weak', 39.0, 'Average', 42.0, 'Average', 51.0, 'Weak', 26.0, 'Weak', 29.0, nan, nan, nan, nan, '87.00%', 4.0, '95.60%', '99.20%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 15.9, 15.7, 17.3, 16.7, 0.8, 20.4, 3.1, 34.6, 88.2, 78.5, 1636, 31, 76.8, 1152840.522, 1933431.315, 41.973192600000004, -87.71334969, 13, 'NORTH PARK', 39, 17, '(41.9731926, -87.71334969)') ,\n(609709, 'Gage Park High School', 'HS', '5630 S Rockwell St', 'Chicago', 'IL', 60629, '(773) 535-9230', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609709.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 14.0, 'Average', 43.0, 'Weak', 29.0, 'Weak', 37.0, 'Weak', 38.0, 'Weak', 35.0, nan, nan, nan, nan, '72.50%', 19.5, '94.00%', '99.10%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.4, 12.0, 13.8, 13.6, 1.2, 14.1, 0.3, 7.9, 52.8, 40.3, 1052, 43, 46.0, 1159962.875, 1867094.3469999998, 41.79101352, -87.68899064, 63, 'GAGE PARK', 16, 8, '(41.79101352, -87.68899064)') ,\n(610009, 'Galileo Math & Science Scholastic Academy Elementary School', 'ES', '820 S Carpenter St', 'Chicago', 'IL', 60607, '(773) 534-7070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610009.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 65.0, 'Average', 52.0, 'Average', 40.0, 'Average', 42.0, 'Weak', 56.0, 'Weak', 33.0, 'Average', 49.0, 'Average', 50.0, '95.00%', 3.1, '97.00%', '100.00%', 77.3, 60.5, 54.7, 55.4, 68.7, 68.2, 55.6, 59.5, 66.3, 56.6, 33.3, 41.7, 26.5, 22.3, -1.0, -0.8, 'Red', 'Red', 33.3, 78.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 606, 38, nan, 1169436.562, 1896405.606, 41.87124634, -87.65340205, 28, 'NEAR WEST SIDE', 25, 12, '(41.87124634, -87.65340205)') ,\n(610072, 'Garrett A Morgan Elementary School', 'ES', '8407 S Kerfoot Ave', 'Chicago', 'IL', 60620, '(773) 535-3366', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610072.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Weak', 38.0, 'Average', 58.0, 'Average', 55.0, 'Weak', 46.0, 'Weak', 36.0, 'Average', 47.0, 'Weak', 46.0, '92.00%', 10.0, '94.80%', '96.80%', 71.2, 39.6, 28.3, 26.2, 56.3, 64.6, 19.2, 30.9, 53.1, 55.8, 8.3, 21.7, 7.9, 3.2, -0.7, 0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 237, 45, nan, 1173449.365, 1849063.053, 41.74124549, -87.64007166, 71, 'AUBURN GRESHAM', 21, 6, '(41.74124549, -87.64007166)') ,\n(610293, 'Genevieve Melody Elementary School', 'ES', '412 S Keeler Ave', 'Chicago', 'IL', 60624, '(773) 534-6850', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610293.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Strong', 61.0, nan, nan, 'Strong', 65.0, 'Strong', 77.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '92.10%', 26.0, '96.00%', '94.50%', 56.0, nan, 11.1, 19.2, 43.3, 37.1, 26.9, 29.6, 55.8, 76.9, 6.3, 12.5, 4.2, 6.3, -0.3, 0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 315, 36, nan, 1148424.226, 1897525.231, 41.87474954, -87.73051772, 26, 'WEST GARFIELD PARK', 24, 11, '(41.87474954, -87.73051772)') ,\n(609779, 'George Armstrong International Studies Elementary School', 'ES', '2110 W Greenleaf Ave', 'Chicago', 'IL', 60645, '(773) 534-2150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609779.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 59.0, 'Average', 57.0, 'Average', 59.0, 'Strong', 64.0, 'Weak', 53.0, 'Average', 45.0, nan, nan, nan, nan, '96.30%', 8.1, '96.30%', '99.40%', 66.5, 45.5, 47.5, 38.8, 55.7, 65.5, 59.7, 51.5, 72.9, 54.4, 28.2, 43.5, 30.4, 19.5, 0.7, -0.2, 'Green', 'Yellow', 30.8, 60.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1509, 32, nan, 1160591.979, 1946966.544, 42.01017628, -87.68446868, 2, 'WEST RIDGE', 50, 24, '(42.01017628, -87.68446868)') ,\n(610062, 'George B McClellan Elementary School', 'ES', '3527 S Wallace St', 'Chicago', 'IL', 60609, '(773) 535-1732', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610062.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 49.0, nan, nan, 'Weak', 33.0, 'Average', 40.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Average', 47.0, '93.20%', 7.8, '96.00%', '98.60%', 40.5, 17.4, 35.2, 26.8, 65.2, 55.1, 33.9, 39.7, 57.6, 58.3, 27.8, 22.2, 22.7, 21.3, 0.0, 0.9, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 283, 40, nan, 1172917.689, 1881398.625, 41.82998962, -87.64106579, 60, 'BRIDGEPORT', 11, 9, '(41.82998962, -87.64106579)') ,\n(610196, 'George B Swift Elementary Specialty School', 'ES', '5900 N Winthrop Ave', 'Chicago', 'IL', 60660, '(773) 534-2695', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610196.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 61.0, 'Average', 53.0, 'Strong', 62.0, 'Strong', 60.0, 'Weak', 61.0, 'Average', 59.0, 'Average', 49.0, 'Strong', 54.0, '96.30%', 6.0, '95.80%', '98.80%', 69.2, 50.4, 49.4, 33.9, 55.5, 63.3, 52.9, 36.3, 58.2, 51.2, 23.9, 23.9, 25.4, 16.2, -0.8, 1.1, 'Red', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 709, 32, nan, 1167755.176, 1939362.0269999998, 41.98915734, -87.65833342, 77, 'EDGEWATER', 48, 20, '(41.98915734, -87.65833342)') ,\n(609849, 'George F Cassell Elementary School', 'ES', '11314 S Spaulding Ave', 'Chicago', 'IL', 60655, '(773) 535-2640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609849.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 76.0, 'Strong', 67.0, 'Average', 57.0, 'Weak', 36.0, 'Weak', 66.0, 'Strong', 61.0, 'Average', 52.0, 'Weak', 46.0, '94.40%', 13.8, '95.10%', '98.90%', 62.6, 40.0, 48.5, 54.7, 57.5, 66.3, 43.6, 61.0, 57.3, 63.6, 20.5, 54.5, 24.9, 24.5, -0.6, -0.8, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 358, 49, nan, 1156350.416, 1829287.52, 41.68733881, -87.70325179, 74, 'MOUNT GREENWOOD', 19, 22, '(41.68733881, -87.70325179)') ,\n(609761, 'George H Corliss High School', 'HS', '821 E 103rd St', 'Chicago', 'IL', 60628, '(773) 535-5115', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609761.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 33.0, 'Average', 52.0, 'Average', 40.0, 'Average', 48.0, 'Weak', 48.0, 'Average', 54.0, nan, nan, nan, nan, '71.30%', 23.4, '95.20%', '98.50%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.0, 11.4, 13.0, 12.9, 0.9, 14.3, 1.3, 10.6, 42.8, 44.0, 621, 48, 76.2, 1183649.971, 1836810.314, 41.70739096, -87.60307842, 50, 'PULLMAN', 9, 5, '(41.70739096, -87.60307842)') ,\n(610305, 'George Leland Elementary School', 'ES', '5221 W Congress Pkwy', 'Chicago', 'IL', 60644, '(773) 534-6340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610305.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, 'Strong', 71.0, nan, nan, nan, nan, 'Weak', 68.0, 'Strong', 77.0, 'Strong', 60.0, 'Average', 53.0, '92.60%', 3.3, '94.90%', '100.00%', 67.8, 59.1, 30.8, 23.1, 46.2, 61.5, nan, nan, nan, nan, nan, nan, 46.2, 20.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 182, 36, nan, 1141563.256, 1897140.916, 41.87382439, -87.75571809, 25, 'AUSTIN', 29, 15, '(41.87382439, -87.75571809)') ,\n(610048, 'George Manierre Elementary School', 'ES', '1420 N Hudson Ave', 'Chicago', 'IL', 60610, '(773) 534-8456', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610048.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 43.0, nan, nan, 'Average', 49.0, 'Average', 40.0, 'Weak', nan, nan, nan, 'Weak', 42.0, 'Weak', 44.0, '91.00%', 44.0, '94.80%', '100.00%', 44.4, nan, 23.7, 13.9, 44.5, 52.8, 26.5, 17.1, 49.1, 41.7, 0.0, 11.1, 4.6, 4.7, -1.1, -1.2, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 396, 33, nan, 1173014.972, 1909920.5680000002, 41.90825367, -87.63986347, 8, 'NEAR NORTH SIDE', 27, 18, '(41.90825367, -87.63986347)') ,\n(610139, 'George M Pullman Elementary School', 'ES', '11311 S Forrestville Ave', 'Chicago', 'IL', 60628, '(773) 535-5395', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610139.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, nan, nan, 'Strong', 66.0, 'Strong', 75.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 49.0, '93.50%', 14.6, '96.20%', '100.00%', 63.8, 49.4, 32.2, 41.7, 69.5, 67.2, 39.7, 32.9, 67.2, 54.3, 4.0, 16.0, 14.8, 17.8, 0.7, 0.7, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 259, 48, nan, 1181982.459, 1830019.85, 41.68879566, -87.60939383, 50, 'PULLMAN', 9, 5, '(41.68879566, -87.60939383)') ,\n(609795, 'George Rogers Clark Elementary School', 'ES', '1045 S Monitor Ave', 'Chicago', 'IL', 60644, '(773) 534-6225', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609795.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Weak', 39.0, nan, nan, 'Weak', 32.0, 'Weak', 36.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 51.0, '93.90%', 14.4, '96.40%', '100.00%', 80.0, 74.1, 56.8, 37.5, 60.7, 71.1, 60.5, 46.6, 57.5, 56.1, 30.0, 33.3, 22.0, 15.3, 1.2, 0.7, 'Green', 'Yellow', 29.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 297, 36, nan, 1137583.9679999999, 1894930.8769999999, 41.867832299999996, -87.77038158, 25, 'AUSTIN', 29, 15, '(41.8678323, -87.77038158)') ,\n(609760, 'George Washington Carver Military Academy High School', 'HS', '13100 S Doty Ave', 'Chicago', 'IL', 60627, '(773) 535-5250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609760.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 53.0, nan, nan, 'Weak', 37.0, 'Weak', 34.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '91.60%', 8.9, '96.40%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.5, 14.2, 15.0, 14.9, 0.4, 16.8, 1.8, 21.2, 78.7, 60.5, 532, 48, 79.5, 1187202.39, 1818214.083, 41.65627709, -87.5906566, 54, 'RIVERDALE', 9, 5, '(41.65627709, -87.5906566)') ,\n(609845, 'George Washington Carver Primary School', 'ES', '901 E 133rd Pl', 'Chicago', 'IL', 60827, '(773) 535-5674', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609845.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 25.0, nan, nan, 'Weak', 33.0, 'Weak', 30.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Strong', 57.0, '90.90%', 15.6, '94.70%', '100.00%', 37.4, 36.6, 23.8, 19.7, 38.8, 45.8, 22.0, 17.1, 27.9, 43.1, 4.5, 22.7, 9.4, 8.0, -0.8, -0.6, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 546, 48, nan, 1184786.467, 1817244.896, 41.65367428, -87.59952683, 54, 'RIVERDALE', 9, 5, '(41.65367428, -87.59952683)') ,\n(610219, 'George Washington Elementary School', 'ES', '3611 E 114th St', 'Chicago', 'IL', 60617, '(773) 535-5010', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610219.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 58.0, 'Strong', 61.0, 'Weak', 33.0, 'Average', 43.0, 'Weak', 80.0, 'Strong', 76.0, 'Average', 52.0, 'Average', 51.0, '95.50%', 5.6, '95.90%', '99.00%', 69.3, 56.4, 53.8, 59.6, 70.0, 76.7, 54.2, 58.1, 69.6, 68.3, 23.0, 37.5, 30.4, 24.7, 0.4, 0.8, 'Yellow', 'Green', 36.8, 25.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 813, 47, nan, 1202028.594, 1829944.81, 41.68810403, -87.53600985, 52, 'EAST SIDE', 10, 4, '(41.68810403, -87.53600985)') ,\n(609739, 'George Washington High School', 'HS', '3535 E 114th St', 'Chicago', 'IL', 60617, '(773) 535-5725', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609739.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 46.0, 'Average', 53.0, 'Weak', 36.0, 'Weak', 36.0, 'Weak', 52.0, 'Average', 46.0, nan, nan, nan, nan, '80.20%', 12.7, '95.10%', '99.50%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.4, 13.6, 14.3, 14.1, 0.7, 15.5, 1.2, 14.0, 71.6, 50.9, 1524, 47, 78.6, 1201607.873, 1829942.879, 41.688109399999995, -87.53755009999999, 52, 'EAST SIDE', 10, 4, '(41.6881094, -87.5375501)') ,\n(609900, 'George W Curtis Elementary School', 'ES', '32 E 115th St', 'Chicago', 'IL', 60628, '(773) 535-5050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609900.pdf', 'AUSL Schools', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 23.0, nan, nan, 'Average', 43.0, 'Average', 58.0, 'Weak', nan, nan, nan, 'Strong', 68.0, 'Strong', 70.0, '93.80%', 18.2, '98.10%', '97.90%', 51.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, 9.1, 18.8, 10.1, 3.8, 0.9, 0.9, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 504, 48, nan, 1178641.811, 1828771.2619999999, 41.68544575, -87.62166148, 49, 'ROSELAND', 9, 5, '(41.68544575, -87.62166148)') ,\n(609693, 'George Westinghouse High School', 'HS', '3223 W Franklin Blvd', 'Chicago', 'IL', 60624, '(773) 534-6400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609693.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', nan, 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Strong', 74.0, 'Strong', 72.0, 'Strong', 77.0, 'Very Strong', 80.0, 'Weak', 86.0, 'Strong', 71.0, 'Strong', 54.0, 'Strong', 60.0, '93.60%', 2.0, '96.80%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 16.1, 16.9, nan, 17.1, 1.0, nan, nan, nan, nan, nan, 828, 34, 90.5, 1154695.243, 1903109.304, 41.88994963, -87.70734353, 23, 'HUMBOLDT PARK', 28, 11, '(41.88994963, -87.70734353)') ,\n(610202, 'George W Tilton Elementary School', 'ES', '223 N Keeler Ave', 'Chicago', 'IL', 60624, '(773) 534-6746', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610202.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 6.0, 'Weak', 25.0, 'Weak', 30.0, 'Average', 41.0, 'Weak', 25.0, 'Average', 44.0, 'Average', 51.0, 'Average', 49.0, '91.40%', 65.5, '95.10%', '100.00%', 51.1, 39.6, 22.5, 10.8, 29.0, 36.8, 29.1, 12.0, 46.2, 21.4, 14.3, 8.8, 10.0, 3.5, -0.7, -2.6, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 321, 36, nan, 1148397.097, 1900931.417, 41.88409703, -87.73052949, 26, 'WEST GARFIELD PARK', 28, 11, '(41.88409703, -87.73052949)') ,\n(609920, 'Gerald Delgado Kanoon Elementary Magnet School', 'ES', '2233 S Kedzie Ave', 'Chicago', 'IL', 60623, '(773) 534-1736', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609920.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 44.0, 'Weak', 37.0, 'Average', 48.0, 'Average', 48.0, 'Weak', 61.0, 'Average', 43.0, 'Average', 47.0, 'Average', 49.0, '95.90%', 3.3, '95.60%', '100.00%', 76.2, 35.2, 16.9, 18.3, 51.7, 51.2, 25.7, 28.6, 51.5, 60.0, 16.0, 22.0, 8.0, 6.6, -0.4, -0.6, 'Yellow', 'Yellow', 21.6, 45.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 713, 37, nan, 1155417.273, 1888838.9309999999, 41.85077575, -87.70507547, 30, 'SOUTH LAWNDALE', 24, 10, '(41.85077575, -87.70507547)') ,\n(610187, 'Graeme Stewart Elementary School', 'ES', '4525 N Kenmore Ave', 'Chicago', 'IL', 60640, '(773) 534-2640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610187.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 49.0, nan, nan, 'Average', 46.0, 'Average', 57.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 52.0, '95.20%', 77.0, '96.10%', '100.00%', 47.4, 44.2, 23.6, 25.8, 66.3, 52.9, 31.7, 46.0, 59.3, 55.9, 9.1, 40.9, 11.9, 9.7, 0.5, 1.1, 'Yellow', 'Yellow', 30.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 280, 32, nan, 1168446.748, 1930325.571, 41.96434608, -87.65605252, 3, 'UPTOWN', 46, 19, '(41.96434608, -87.65605252)') ,\n(610285, 'Granville T Woods Math & Science Academy Elementary School', 'ES', '6206 S Racine Ave', 'Chicago', 'IL', 60636, '(773) 535-9250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610285.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, nan, nan, 'Average', 43.0, 'Weak', 31.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 49.0, '91.90%', 41.5, '95.20%', '100.00%', nan, 61.7, 25.8, 11.4, 35.1, 53.8, 24.2, 21.6, 50.9, 46.3, 5.4, 17.5, 11.1, 6.5, 0.0, -0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 402, 43, nan, 1169345.488, 1863577.552, 41.7811649, -87.65468845, 67, 'WEST ENGLEWOOD', 16, 7, '(41.7811649, -87.65468845)') ,\n(610383, 'Greater Lawndale High School For Social Justice', 'HS', '3120 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 535-4300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610383.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 43.0, 'Strong', 72.0, 'Average', 46.0, 'Average', 44.0, 'Weak', 81.0, 'Strong', 65.0, 'Average', 53.0, 'Weak', 46.0, '87.50%', 16.4, '95.10%', '95.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.6, 14.0, 14.5, 14.4, 0.8, 15.7, 1.2, 14.9, 67.0, 57.4, 371, 37, 87.4, 1147521.3020000001, 1883405.128, 41.83601953, -87.73419465, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83601953, -87.73419465)') ,\n(609857, 'Grover Cleveland Elementary School', 'ES', '3121 W Byron St', 'Chicago', 'IL', 60618, '(773) 534-5130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609857.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, 'Average', 43.0, 'Average', 58.0, 'Average', 47.0, 'Weak', 51.0, 'Average', 44.0, 'Average', 49.0, 'Average', 48.0, '95.00%', 6.1, '96.30%', '100.00%', 72.5, 38.7, 53.6, 37.3, 49.7, 49.5, 47.9, 45.7, 50.5, 58.3, 21.2, 36.5, 23.4, 15.8, -0.3, -0.5, 'Yellow', 'Yellow', 27.6, 58.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 688, 31, nan, 1154778.842, 1925727.705, 41.95201470000001, -87.70642927, 16, 'IRVING PARK', 33, 17, '(41.9520147, -87.70642927)') ,\n(610241, 'Guglielmo Marconi Elementary Community Academy', 'ES', '230 N Kolmar Ave', 'Chicago', 'IL', 60624, '(773) 534-6210', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610241.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 26.0, nan, nan, 'Strong', 65.0, 'Strong', 76.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Average', 52.0, '91.50%', 58.8, '96.50%', '100.00%', 64.7, 65.3, 28.8, 23.4, 51.7, 70.0, 37.7, 31.5, 64.6, 65.7, 0.0, 21.1, 4.0, 7.3, -1.6, -1.3, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 245, 36, nan, 1145991.517, 1900994.657, 41.88431659, -87.73936153, 26, 'WEST GARFIELD PARK', 28, 11, '(41.88431659, -87.73936153)') ,\n(609741, 'Gurdon S Hubbard High School', 'HS', '6200 S Hamlin Ave', 'Chicago', 'IL', 60629, '(773) 535-2200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609741.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 42.0, nan, nan, 'Weak', 34.0, 'Weak', 29.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '89.30%', 4.3, '96.40%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.4, 14.2, 15.3, 15.3, 0.9, 17.3, 2.0, 30.1, 57.2, 51.3, 1672, 44, 64.5, 1152065.741, 1863208.025, 41.78050763, -87.71804991, 65, 'WEST LAWN', 13, 8, '(41.78050763, -87.71804991)') ,\n(609726, 'Gwendolyn Brooks College Preparatory Academy High School', 'HS', '250 E 111th St', 'Chicago', 'IL', 60628, '(773) 535-9930', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609726.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, nan, nan, 'Average', 44.0, 'Average', 49.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 49.0, '87.10%', 0.1, '95.20%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 18.0, 18.2, 18.7, 18.7, 0.7, 21.5, 2.8, 51.1, 88.1, 87.9, 783, 48, 90.3, 1180061.781, 1831459.2130000002, 41.69278956, -87.61638144, 49, 'ROSELAND', 9, 5, '(41.69278956, -87.61638144)') ,\n(610182, 'Hannah G Solomon Elementary School', 'ES', '6206 N Hamlin Ave', 'Chicago', 'IL', 60659, '(773) 534-5226', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610182.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 78.0, 'Very Strong', 96.0, 'Strong', 67.0, 'Strong', 62.0, 'Weak', 75.0, 'Strong', 62.0, 'Average', 52.0, 'Average', 51.0, '95.80%', 0.8, '95.70%', '99.00%', 71.2, 51.1, 74.4, 63.6, 59.1, 70.0, 73.9, 72.8, 75.0, 67.4, 45.5, 63.6, 52.4, 42.8, 0.6, -0.2, 'Yellow', 'Yellow', 45.5, 93.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 363, 31, nan, 1149868.813, 1940981.795, 41.99397007, -87.72407991, 13, 'NORTH PARK', 39, 17, '(41.99397007, -87.72407991)') ,\n(610068, 'Hanson Park Elementary School', 'ES', '5411 W Fullerton Ave', 'Chicago', 'IL', 60639, '(773) 534-3100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610068.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, nan, nan, 'Weak', 34.0, 'Weak', 33.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 47.0, '95.00%', 24.4, '94.90%', '100.00%', 48.5, 42.6, 31.0, 20.6, 45.9, 51.5, 42.0, 29.2, 50.7, 41.2, 12.4, 22.1, 14.4, 11.0, -0.8, -0.5, 'Red', 'Yellow', 14.0, 88.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1499, 29, nan, 1139861.158, 1915387.949, 41.92392771, -87.76152094, 19, 'BELMONT CRAGIN', 37, 25, '(41.92392771, -87.76152094)') ,\n(610124, 'Harold Washington Elementary School', 'ES', '9130 S University Ave', 'Chicago', 'IL', 60619, '(773) 535-6225', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610124.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 28.0, nan, nan, 'Average', 45.0, 'Strong', 63.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 49.0, '96.10%', 3.1, '96.20%', '100.00%', 67.6, 55.6, 27.8, 39.0, 53.2, 53.6, 32.5, 37.6, 45.0, 46.6, 9.5, 28.6, 16.0, 13.1, -0.2, 0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 549, 48, nan, 1185436.746, 1844530.731, 41.72853492, -87.59629327, 47, 'BURNSIDE', 8, 4, '(41.72853492, -87.59629327)') ,\n(610192, 'Harriet Beecher Stowe Elementary School', 'ES', '3444 W Wabansia Ave', 'Chicago', 'IL', 60647, '(773) 534-4175', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610192.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 28.0, nan, nan, 'Weak', 26.0, 'Weak', 31.0, 'Weak', nan, nan, nan, 'Weak', 44.0, 'Strong', 54.0, '93.80%', 13.7, '94.70%', '98.70%', 39.9, 32.2, 11.1, 12.0, 36.9, 48.6, 16.7, 18.0, 49.4, 48.6, 2.1, 9.5, 3.1, 2.5, -0.2, -0.8, 'Yellow', 'Red', 19.1, 11.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 893, 34, nan, 1153019.365, 1911120.2580000001, 41.91196584, -87.7132855, 23, 'HUMBOLDT PARK', 26, 14, '(41.91196584, -87.7132855)') ,\n(610158, 'Harriet E Sayre Elementary Language Academy', 'ES', '1850 N Newland Ave', 'Chicago', 'IL', 60707, '(773) 534-3351', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610158.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 68.0, nan, nan, 'Average', 41.0, 'Average', 50.0, 'Weak', nan, nan, nan, 'Weak', 44.0, 'Average', 49.0, '96.20%', 2.6, '96.60%', '100.00%', 69.1, nan, 37.6, 42.5, 44.8, 37.1, 41.7, 48.1, 47.6, 42.8, 20.0, 38.3, 19.6, 17.3, -0.5, 0.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 605, 29, nan, 1129750.187, 1911749.959, 41.9141236, -87.79875667, 25, 'AUSTIN', 36, 25, '(41.9141236, -87.79875667)') ,\n(609974, 'Hawthorne Elementary Scholastic Academy', 'ES', '3319 N Clifton Ave', 'Chicago', 'IL', 60657, '(773) 534-5550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609974.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'Yes', 'Very Strong', 92.0, 'Very Strong', 88.0, 'Strong', 61.0, 'Average', 56.0, 'Weak', 90.0, 'Very Strong', 84.0, 'Strong', 58.0, 'Average', 52.0, '96.90%', 1.2, '96.50%', '100.00%', 85.3, 56.9, 82.7, 83.3, 58.6, 62.5, 89.1, 87.4, 65.1, 62.9, 79.0, 80.6, 75.1, 66.0, 1.8, 2.2, 'Green', 'Green', 98.4, 91.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 569, 33, nan, 1168069.5259999998, 1922284.759, 41.94228994, -87.65767253, 6, 'LAKE VIEW', 44, 19, '(41.94228994, -87.65767253)') ,\n(609985, 'Helen M Hefferan Elementary School', 'ES', '4409 W Wilcox St', 'Chicago', 'IL', 60624, '(773) 534-6192', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609985.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 29.0, 'Weak', 39.0, 'Weak', 26.0, 'Strong', 63.0, 'Weak', 58.0, 'Average', 52.0, 'Average', 51.0, 'Strong', 56.0, '93.80%', 13.4, '95.90%', '100.00%', 72.4, 64.9, 36.5, 34.1, 45.7, 37.3, 61.4, 46.7, 61.6, 51.1, 28.0, 44.0, 25.6, 19.2, 0.5, 0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 259, 36, nan, 1146965.713, 1898931.63, 41.87863685, -87.73583687, 26, 'WEST GARFIELD PARK', 28, 11, '(41.87863685, -87.73583687)') ,\n(610122, 'Helen Peirce International Studies Elementary School', 'ES', '1423 W Bryn Mawr Ave', 'Chicago', 'IL', 60660, '(773) 534-2440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610122.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, nan, nan, 'Weak', 32.0, 'Weak', 22.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 47.0, '96.70%', 5.8, '95.80%', '100.00%', 57.2, 58.2, 56.4, 47.2, 52.4, 63.3, 52.8, 49.1, 49.1, 57.3, 25.9, 35.2, 25.5, 20.5, -1.9, -1.4, 'Red', 'Red', 100.0, 5.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 977, 32, nan, 1165723.804, 1937258.243, 41.98342819, -87.66586537, 77, 'EDGEWATER', 48, 20, '(41.98342819, -87.66586537)') ,\n(609972, 'Helge A Haugan Elementary School', 'ES', '4540 N Hamlin Ave', 'Chicago', 'IL', 60625, '(773) 534-5040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609972.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 43.0, 'Weak', 39.0, 'Average', 57.0, 'Average', 49.0, 'Weak', 33.0, 'Weak', 30.0, 'Weak', 46.0, 'Average', 51.0, '96.20%', 23.4, '94.60%', '100.00%', 68.9, 37.2, 31.5, 24.3, 53.4, 58.5, 32.5, 29.3, 48.0, 32.9, nan, nan, 15.2, 10.2, -0.3, 0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1299, 31, nan, 1150195.236, 1930075.176, 41.96403525, -87.72316496, 14, 'ALBANY PARK', 39, 17, '(41.96403525, -87.72316496)') ,\n(609856, 'Henry Clay Elementary School', 'ES', '13231 S Burley Ave', 'Chicago', 'IL', 60633, '(773) 535-5600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609856.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 38.0, 'Weak', 39.0, 'Weak', 30.0, 'Weak', 30.0, 'Weak', 24.0, 'Weak', 31.0, nan, nan, nan, nan, '95.20%', 6.0, '96.10%', '100.00%', 52.4, 47.0, 34.4, 27.1, 42.0, 48.0, 39.3, 29.6, 54.8, 37.1, 10.2, 25.4, 17.1, 12.3, -0.3, -0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 633, 47, nan, 1199800.451, 1817700.031, 41.65455937, -87.544577, 55, 'HEGEWISCH', 10, 4, '(41.65455937, -87.544577)') ,\n(610040, 'Henry D Lloyd Elementary School', 'ES', '2103 N Lamon Ave', 'Chicago', 'IL', 60639, '(773) 534-3070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610040.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 51.0, '96.20%', 0.0, '97.20%', '97.00%', nan, 32.9, 34.1, 18.3, 46.5, 53.3, nan, nan, nan, nan, nan, nan, 23.5, 10.5, -0.8, -0.3, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1334, 29, nan, 1143423.139, 1913554.3590000002, 41.91883022, -87.74847861, 19, 'BELMONT CRAGIN', 31, 25, '(41.91883022, -87.74847861)') ,\n(610092, 'Henry H Nash Elementary School', 'ES', '4837 W Erie St', 'Chicago', 'IL', 60644, '(773) 534-6125', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610092.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 41.0, nan, nan, 'Strong', 68.0, 'Strong', 76.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.10%', 27.9, '95.80%', '100.00%', 48.6, 46.4, 22.4, 18.9, 60.4, 59.1, 27.1, 19.5, 74.6, 60.7, 5.1, 7.5, 7.1, 6.0, 0.6, -0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 393, 36, nan, 1143836.768, 1903833.4530000002, 41.89214724, -87.74720292, 25, 'AUSTIN', 28, 15, '(41.89214724, -87.74720292)') ,\n(610279, 'Henry O Tanner Elementary School', 'ES', '7350 S Evans Ave', 'Chicago', 'IL', 60619, '(773) 535-3870', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610279.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 33.0, nan, nan, 'Average', 42.0, 'Average', 55.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '91.80%', 43.1, '95.60%', '98.00%', 61.3, 35.3, 19.5, 13.3, 52.6, 52.3, 22.1, 21.1, 56.6, 57.4, 5.3, 8.1, 6.9, 4.6, 1.9, 0.3, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 448, 46, nan, 1182462.062, 1856226.416, 41.76069854, -87.60682834, 69, 'GREATER GRAND CROSSING', 6, 3, '(41.76069854, -87.60682834)') ,\n(609861, 'Henry R Clissold Elementary School', 'ES', '2350 W 110th Pl', 'Chicago', 'IL', 60643, '(773) 535-2560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609861.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 45.0, 'Strong', 61.0, 'Weak', 39.0, 'Strong', 61.0, 'Weak', 38.0, 'Average', 51.0, 'Average', 52.0, 'Weak', 46.0, '95.10%', 8.4, '96.90%', '94.80%', 75.5, nan, 58.5, 64.0, 68.8, 62.8, 46.7, 63.0, 53.3, 58.7, 33.8, 49.2, 27.7, 26.7, -0.7, 0.5, 'Red', 'Yellow', 42.2, 44.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 588, 49, nan, 1162450.244, 1831415.5990000002, 41.69305404, -87.68086190000001, 75, 'MORGAN PARK', 19, 22, '(41.69305404, -87.6808619)') ,\n(610297, 'High School of Leadership at South Shore', 'HS', '7627 S Constance Ave', 'Chicago', 'IL', 60649, '(773) 535-6190', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610297.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', nan, nan, 'Weak', 30.0, nan, nan, nan, nan, 'Weak', 16.0, 'Weak', 28.0, 'Weak', 42.0, 'Weak', 31.0, '72.00%', 43.4, '95.20%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.7, 11.7, 13.1, 13.1, 1.4, 14.4, 1.3, 12.2, 36.6, 68.8, 731, 46, 48.8, 1189903.206, 1854648.757, 41.75619362, -87.57960727, 43, 'SOUTH SHORE', 8, 4, '(41.75619362, -87.57960727)') ,\n(609798, 'Hiram H Belding Elementary School', 'ES', '4257 N Tripp Ave', 'Chicago', 'IL', 60641, '(773) 534-3590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609798.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 64.0, 'Strong', 63.0, 'Strong', 67.0, 'Strong', 69.0, 'Weak', 49.0, 'Average', 51.0, 'Average', 53.0, 'Average', 52.0, '95.00%', 21.3, '96.20%', '98.60%', 80.0, 78.6, 46.2, 40.4, 50.3, 45.7, 55.6, 54.6, 54.5, 54.2, 36.8, 49.1, 27.3, 21.4, -0.8, -0.5, 'Red', 'Yellow', 46.4, 57.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 672, 31, nan, 1147266.366, 1928184.928, 41.95890499, -87.73398223, 16, 'IRVING PARK', 38, 17, '(41.95890499, -87.73398223)') ,\n(609768, 'Hope College Preparatory High School', 'HS', '5515 S Lowe Ave', 'Chicago', 'IL', 60621, '(773) 535-3160', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609768.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Average', 51.0, 'Weak', 37.0, 'Average', 44.0, 'Weak', 56.0, 'Weak', 35.0, nan, nan, nan, nan, '72.20%', 24.0, '94.30%', '99.10%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.3, 13.0, 13.7, 13.5, 1.2, 15.0, 1.3, 16.7, 55.2, 58.2, 561, 45, 63.6, 1172948.54, 1868120.666, 41.79355292, -87.64134488, 68, 'ENGLEWOOD', 3, 7, '(41.79355292, -87.64134488)') ,\n(609850, 'Horace Greeley Elementary School', 'ES', '832 W Sheridan Rd', 'Chicago', 'IL', 60613, '(773) 534-5800', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609850.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 65.0, nan, nan, 'Average', 48.0, 'Weak', 38.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Average', 52.0, '95.90%', 6.0, '96.00%', '100.00%', 64.5, 35.5, 46.7, 37.4, 48.8, 52.8, 65.6, 47.9, 53.1, 49.7, 43.1, 50.0, 26.9, 16.3, 0.2, -0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 611, 32, nan, 1169860.926, 1926142.5780000002, 41.95283701, -87.65097549, 6, 'LAKE VIEW', 46, 19, '(41.95283701, -87.65097549)') ,\n(610052, 'Horace Mann Elementary School', 'ES', '8050 S Chappel Ave', 'Chicago', 'IL', 60617, '(773) 535-6640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610052.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Weak', 31.0, 'Weak', 31.0, 'Weak', 30.0, 'Weak', 31.0, 'Weak', 24.0, 'Weak', 44.0, 'Weak', 46.0, '92.60%', 44.4, '94.50%', '100.00%', 46.9, 30.2, 12.8, 16.5, 49.7, 46.2, 23.3, 23.3, 57.2, 51.0, 6.4, 27.7, 3.4, 3.4, -0.7, 0.1, 'Red', 'Yellow', 19.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 472, 47, nan, 1191225.84, 1851783.833, 41.74830012, -87.57485274, 46, 'SOUTH CHICAGO', 8, 4, '(41.74830012, -87.57485274)') ,\n(610058, 'Horatio May Elementary Community Academy', 'ES', '512 S Lavergne Ave', 'Chicago', 'IL', 60644, '(773) 534-6140', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610058.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, nan, nan, 'Average', 47.0, 'Average', 56.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '91.90%', 40.0, '96.40%', '98.20%', 63.3, 58.8, 17.5, 17.5, 54.0, 51.4, 27.1, 25.0, 58.6, 58.6, 4.6, 13.8, 9.4, 5.6, -0.4, -0.8, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 458, 36, nan, 1143308.514, 1897064.832, 41.87358321, -87.74931216, 25, 'AUSTIN', 29, 15, '(41.87358321, -87.74931216)') ,\n(609713, 'Hyde Park Academy High School', 'HS', '6220 S Stony Island Ave', 'Chicago', 'IL', 60637, '(773) 535-0880', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609713.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 29.0, 'Weak', 39.0, 'Weak', 34.0, 'Average', 43.0, 'Weak', 35.0, 'Average', 41.0, nan, nan, nan, nan, '70.50%', 18.5, '94.20%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.0, 12.8, 14.4, 14.1, 1.1, 15.6, 1.2, 11.1, 51.1, 57.6, 1518, 46, 63.7, 1187898.897, 1864121.427, 41.78223539, -87.58665115, 42, 'WOODLAWN', 5, 3, '(41.78223539, -87.58665115)') ,\n(610390, 'Hyman G Rickover Naval Academy High School', 'HS', '5900 N Glenwood Ave', 'Chicago', 'IL', 60660, '(773) 534-2890', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610390.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 64.0, nan, nan, 'Strong', 64.0, 'Average', 42.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Average', 48.0, '92.90%', 15.4, '95.90%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 15.2, 15.4, 16.5, 15.8, 0.6, 18.7, 2.2, 23.4, 63.0, 63.5, 423, 32, 90.7, 1165871.81, 1939308.369, 41.98905063, -87.66526222, 77, 'EDGEWATER', 48, 20, '(41.98905063, -87.66526222)') ,\n(610110, 'Ida B Wells Preparatory Elementary Academy', 'ES', '244 E Pershing Rd', 'Chicago', 'IL', 60653, '(773) 535-1204', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610110.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 49.0, nan, nan, 'Strong', 64.0, 'Very Strong', 93.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '93.20%', 9.5, '92.80%', '100.00%', 80.9, nan, 24.7, 26.3, 41.3, 60.5, 49.3, 42.3, 58.8, 57.4, 8.3, 37.5, 15.3, 10.1, 0.3, -0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 215, 40, nan, 1178735.106, 1879229.78, 41.82390751, -87.61978794, 35, 'DOUGLAS', 3, 2, '(41.82390751, -87.61978794)') ,\n(610273, 'Ignance Paderewski Elementary Learning Academy', 'ES', '2221 S Lawndale Ave', 'Chicago', 'IL', 60623, '(773) 534-1821', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610273.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 31.0, nan, nan, 'Strong', 63.0, 'Very Strong', 98.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Average', 47.0, '92.10%', 20.9, '95.20%', '100.00%', 42.3, 40.9, 19.1, 19.1, 50.0, 64.3, 27.5, 17.0, 44.2, 58.7, 0.0, 0.0, 2.0, 2.0, 0.2, -0.8, 'Yellow', 'Yellow', 93.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 130, 37, nan, 1152095.48, 1888877.695, 41.85094816, -87.71726618, 30, 'SOUTH LAWNDALE', 22, 10, '(41.85094816, -87.71726618)') ,\n(610384, 'Infinity Math Science and Technology High School', 'HS', '3120 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 535-4225', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610384.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, nan, nan, 'Strong', 66.0, 'Strong', 65.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Average', 48.0, '92.20%', 7.5, '96.00%', '85.40%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.1, 15.0, 14.8, 15.1, 1.0, 17.2, 2.4, 29.0, 84.4, 55.7, 388, 37, 98.1, 1147521.3020000001, 1883405.128, 41.83601953, -87.73419465, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83601953, -87.73419465)') ,\n(610078, 'Inter-American Elementary Magnet School', 'ES', '851 W Waveland Ave', 'Chicago', 'IL', 60613, '(773) 534-5490', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610078.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, nan, nan, 'Average', 50.0, 'Average', 51.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Average', 47.0, '96.20%', 3.0, '96.20%', '100.00%', 68.2, 31.2, 46.6, 38.7, 51.3, 49.5, 52.3, 50.0, 54.8, 54.1, 34.5, 29.6, 27.6, 21.3, 1.1, 1.9, 'Green', 'Green', 32.1, 83.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 690, 33, nan, 1169719.256, 1924734.562, 41.94897646, -87.65153745, 6, 'LAKE VIEW', 44, 19, '(41.94897646, -87.65153745)') ,\n(609848, 'Ira F Aldridge Elementary School', 'ES', '630 E 131st St', 'Chicago', 'IL', 60827, '(773) 535-5614', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609848.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 39.0, nan, nan, 'Weak', 36.0, 'Weak', 39.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Weak', 46.0, '92.90%', 49.0, '96.30%', '100.00%', 62.7, 60.9, 20.7, 20.2, 46.7, 40.9, 21.2, 29.8, 37.4, 60.2, 2.5, 20.0, 7.7, 7.2, -1.5, -0.8, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 285, 48, nan, 1182870.294, 1818598.178, 41.65743243, -87.60649623, 54, 'RIVERDALE', 9, 5, '(41.65743243, -87.60649623)') ,\n(610532, 'Irene C. Hernandez Middle School for the Advancement of Science', 'MS', '3510 W 55th St', 'Chicago', 'IL', 60632, '(773) 535-8850', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610532.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Not Enough Data', 'No', 'Weak', 34.0, 'Weak', 34.0, 'Weak', 27.0, 'Weak', 27.0, 'Weak', 28.0, 'Weak', 27.0, nan, nan, nan, nan, '95.50%', 14.8, '96.20%', '100.00%', nan, nan, nan, nan, nan, nan, 33.7, 28.9, 34.2, 46.6, 12.5, 18.0, 11.6, 9.2, -1.8, -0.8, 'Red', 'Red', 18.2, 40.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 970, 45, nan, 1153845.758, 1867957.685, 41.79350627, -87.71139803, 63, 'GAGE PARK', 14, 8, '(41.79350627, -87.71139803)') ,\n(610125, 'Irma C Ruiz Elementary School', 'ES', '2410 S Leavitt St', 'Chicago', 'IL', 60608, '(773) 535-4825', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610125.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'Yes', 'Average', 57.0, 'Average', 59.0, 'Average', 57.0, 'Strong', 62.0, 'Weak', 51.0, 'Average', 56.0, 'Average', 50.0, 'Average', 51.0, '95.50%', 15.9, '95.00%', '95.20%', 55.8, 40.7, 19.4, 26.4, 44.0, 41.9, 38.9, 33.6, 57.5, 48.5, 10.7, 27.2, 10.7, 12.8, -0.4, 0.4, 'Red', 'Yellow', 28.8, 16.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 916, 39, nan, 1162043.649, 1887912.321, 41.84809742, -87.68078114, 31, 'LOWER WEST SIDE', 25, 10, '(41.84809742, -87.68078114)') ,\n(610276, 'Irvin C Mollison Elementary School', 'ES', '4415 S Dr Martin L King Jr Dr', 'Chicago', 'IL', 60653, '(773) 535-1804', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610276.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 44.0, nan, nan, 'Average', 46.0, 'Weak', 38.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Average', 50.0, '91.10%', 36.1, '94.30%', '94.40%', 49.3, 36.8, 27.4, 19.8, 48.2, 41.9, 34.4, 26.4, 57.9, 41.9, 11.1, 25.9, 4.8, 4.3, 0.2, 0.2, 'Yellow', 'Yellow', 20.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 244, 40, nan, 1179638.34, 1875750.718, 41.81434006, -87.61658079, 38, 'GRAND BOULEVARD', 3, 2, '(41.81434006, -87.61658079)') ,\n(610103, 'Isabelle C O Keeffe Elementary School', 'ES', '6940 S Merrill Ave', 'Chicago', 'IL', 60649, '(773) 535-0600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610103.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 17.0, nan, nan, 'Weak', 28.0, 'Weak', 29.0, 'Weak', nan, nan, nan, 'Weak', 40.0, 'Weak', 43.0, '92.10%', 24.6, '94.60%', '100.00%', 52.0, 54.0, 13.8, 16.4, 42.8, 29.1, 17.9, 28.6, 43.3, 60.0, 1.7, 19.0, 3.4, 6.7, -0.4, 0.7, 'Yellow', 'Yellow', 17.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 575, 46, nan, 1191708.826, 1859191.4640000002, 41.76861557, -87.57284299, 43, 'SOUTH SHORE', 5, 3, '(41.76861557, -87.57284299)') ,\n(610256, 'Jackie Robinson Elementary School', 'ES', '4225 S Lake Park Ave', 'Chicago', 'IL', 60653, '(773) 535-1777', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610256.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Average', 47.0, '90.50%', 47.9, '93.80%', '100.00%', 52.0, 30.6, 10.6, 21.3, 39.1, 26.1, nan, nan, nan, nan, nan, nan, 6.5, 6.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 140, 40, nan, 1184412.88, 1876890.9740000002, 41.81735839, -87.59903174, 36, 'OAKLAND', 4, 2, '(41.81735839, -87.59903174)') ,\n(609797, 'Jacob Beidler Elementary School', 'ES', '3151 W Walnut St', 'Chicago', 'IL', 60612, '(773) 534-6811', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609797.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 63.0, nan, nan, 'Strong', 70.0, 'Average', 58.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.40%', 0.7, '96.60%', '100.00%', 70.7, 54.1, 30.0, 17.3, 55.4, 70.7, 24.8, 24.3, 50.7, 56.2, 3.8, 7.5, 14.3, 5.1, 0.5, 0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 411, 34, nan, 1155122.862, 1901415.832, 41.88529401, -87.70581859, 27, 'EAST GARFIELD PARK', 27, 13, '(41.88529401, -87.70581859)') ,\n(609766, 'Jacqueline B Vaughn Occupational High School', 'HS', '4355 N Linder Ave', 'Chicago', 'IL', 60641, '(773) 534-3600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609766.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', nan, 'Track_E', 'Not Applicable', 'Not Enough Data', 'No', 'Average', 57.0, 'Strong', 72.0, 'Weak', 33.0, 'Weak', 20.0, 'Weak', 67.0, 'Strong', 69.0, 'Strong', 63.0, 'Strong', 57.0, '87.60%', 3.7, '95.50%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 55.6, 65.3, nan, 203, 30, nan, 1138928.091, 1928655.205, 41.96035141, -87.76462602, 15, 'PORTAGE PARK', 38, 16, '(41.96035141, -87.76462602)') ,\n(609912, 'James B Farnsworth Elementary School', 'ES', '5414 N Linder Ave', 'Chicago', 'IL', 60630, '(773) 534-3535', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609912.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, 'Strong', 62.0, 'Average', 40.0, 'Weak', 38.0, 'Weak', 63.0, 'Average', 59.0, 'Strong', 55.0, 'Average', 50.0, '94.20%', 1.7, '95.60%', '100.00%', 68.4, 33.1, 50.3, 60.8, 62.7, 55.0, 52.1, 57.6, 59.2, 60.4, 30.2, 41.8, 30.7, 25.6, 1.0, 0.3, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 574, 30, nan, 1138478.05, 1935743.025, 41.9798092, -87.76610809, 11, 'JEFFERSON PARK', 45, 16, '(41.9798092, -87.76610809)') ,\n(610070, 'James B McPherson Elementary School', 'ES', '4728 N Wolcott Ave', 'Chicago', 'IL', 60640, '(773) 534-2625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610070.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 71.0, nan, nan, 'Average', 48.0, 'Weak', 38.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Strong', 54.0, '96.50%', 1.2, '96.30%', '100.00%', 69.7, 41.9, 36.7, 39.7, 47.4, 51.7, 30.4, 42.0, 34.1, 43.4, 10.2, 37.3, 15.4, 15.9, -0.8, -0.6, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 695, 31, nan, 1162923.519, 1931558.605, 41.96784758, -87.67632505, 4, 'LINCOLN SQUARE', 47, 19, '(41.96784758, -87.67632505)') ,\n(610066, 'James E McDade Elementary Classical School', 'ES', '8801 S Indiana Ave', 'Chicago', 'IL', 60619, '(773) 535-3669', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610066.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, nan, nan, 'Average', 57.0, 'Average', 52.0, 'Weak', nan, nan, nan, 'Strong', 61.0, 'Average', 52.0, '96.20%', 0.0, '94.30%', '100.00%', nan, nan, 87.7, 82.2, 50.7, 48.1, nan, 90.0, nan, 45.0, nan, nan, 73.3, 74.1, -1.1, -0.7, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 199, 45, nan, 1179172.217, 1846656.745, 41.73451387, -87.61917677, 44, 'CHATHAM', 6, 6, '(41.73451387, -87.61917677)') ,\n(609803, 'James G Blaine Elementary School', 'ES', '1420 W Grace St', 'Chicago', 'IL', 60613, '(773) 534-5750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609803.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, nan, nan, 'Strong', 76.0, 'Strong', 74.0, 'Weak', nan, nan, nan, 'Weak', 40.0, 'Weak', 46.0, '96.40%', 2.1, '96.00%', '88.40%', 87.5, 62.6, 88.6, 78.5, 61.7, 62.8, 81.9, 79.9, 56.3, 63.6, 53.6, 73.2, 61.3, 50.1, 0.9, -0.4, 'Green', 'Yellow', 50.0, 63.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 923, 33, nan, 1166063.1670000001, 1925373.2680000002, 41.95080812, -87.66495825, 6, 'LAKE VIEW', 44, 19, '(41.95080812, -87.66495825)') ,\n(609983, 'James Hedges Elementary School', 'ES', '4747 S Winchester Ave', 'Chicago', 'IL', 60609, '(773) 535-7360', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609983.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 22.0, nan, nan, 'Weak', 22.0, 'Weak', 24.0, 'Weak', nan, nan, nan, 'Weak', 41.0, 'Weak', 45.0, '95.40%', 48.8, '96.50%', '100.00%', 66.3, 41.1, 34.0, 19.6, 46.3, 50.4, 25.1, 25.2, 57.0, 55.1, 13.2, 17.1, 12.6, 6.8, -0.1, 0.5, 'Yellow', 'Yellow', 16.5, 61.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 845, 42, nan, 1164199.04, 1873001.0469999998, 41.80713406, -87.67329121, 61, 'NEW CITY', 20, 9, '(41.80713406, -87.67329121)') ,\n(610047, 'James Madison Elementary School', 'ES', '7433 S Dorchester Ave', 'Chicago', 'IL', 60619, '(773) 535-0551', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610047.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 39.0, nan, nan, 'Weak', 32.0, 'Weak', 28.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.90%', 12.4, '97.70%', '100.00%', 60.0, 60.7, 21.8, 24.3, 52.0, 49.0, 33.3, 19.3, 48.8, 38.6, 3.1, 28.1, 14.1, 5.1, 0.7, -0.9, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 317, 46, nan, 1186872.582, 1855846.3469999998, 41.75955224, -87.59067582, 43, 'SOUTH SHORE', 5, 3, '(41.75955224, -87.59067582)') ,\n(610074, 'James Monroe Elementary School', 'ES', '3651 W Schubert Ave', 'Chicago', 'IL', 60647, '(773) 534-4155', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610074.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 45.0, nan, nan, 'Weak', 26.0, 'Weak', 22.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 47.0, '95.10%', 22.1, '95.80%', '98.10%', 66.5, 25.8, 37.9, 27.0, 46.5, 55.1, 31.6, 32.2, 50.3, 45.3, 12.5, 24.0, 15.5, 11.6, -0.6, -0.4, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1125, 29, nan, 1151359.1509999998, 1917666.2340000002, 41.92996135, -87.71921246, 22, 'LOGAN SQUARE', 35, 25, '(41.92996135, -87.71921246)') ,\n(610200, 'James N Thorp Elementary School', 'ES', '8914 S Buffalo Ave', 'Chicago', 'IL', 60617, '(773) 535-6250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610200.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 32.0, nan, nan, 'Weak', 39.0, 'Weak', 39.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '91.80%', 45.4, '95.90%', '100.00%', 51.0, 24.3, 9.1, 10.4, 33.6, 31.0, 27.3, 30.5, 52.9, 43.8, 3.6, 20.0, 8.7, 3.6, -0.4, -1.2, 'Yellow', 'Red', 14.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 449, 47, nan, 1199554.7, 1846396.13, 41.73331035, -87.54451462, 46, 'SOUTH CHICAGO', 10, 4, '(41.73331035, -87.54451462)') ,\n(610107, 'James Otis Elementary School', 'ES', '525 N Armour St', 'Chicago', 'IL', 60622, '(773) 534-7665', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610107.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 1', 'No', 'Very Strong', 83.0, 'Very Strong', 92.0, 'Strong', 67.0, 'Average', 58.0, 'Weak', 92.0, 'Very Strong', 94.0, 'Strong', 54.0, 'Strong', 54.0, '95.50%', 3.2, '96.50%', '100.00%', 50.9, 47.1, 35.9, 46.6, 53.4, 63.1, 51.4, 55.0, 61.3, 57.7, 25.8, 19.4, 21.8, 11.1, 0.7, -0.2, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 491, 35, nan, 1166113.4109999998, 1903868.626, 41.89179703, -87.66538928, 24, 'WEST TOWN', 27, 13, '(41.89179703, -87.66538928)') ,\n(609891, 'James R Doolittle Jr Elementary School', 'ES', '535 E 35th St', 'Chicago', 'IL', 60616, '(773) 535-1040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609891.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 30.0, nan, nan, 'Weak', 25.0, 'Average', 46.0, 'Weak', nan, nan, nan, 'Weak', 44.0, 'Average', 47.0, '91.40%', 134.8, '94.70%', '97.70%', 55.6, 50.0, 15.9, 3.7, 40.6, 32.4, 7.4, 24.5, 39.8, 54.9, 11.8, 35.3, 2.2, 1.1, -1.4, -0.2, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 306, 40, nan, 1180586.11, 1881863.446, 41.831092100000006, -87.61291629, 35, 'DOUGLAS', 4, 2, '(41.8310921, -87.61291629)') ,\n(610044, 'James Russell Lowell Elementary School', 'ES', '3320 W Hirsch St', 'Chicago', 'IL', 60651, '(773) 534-4300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610044.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 39.0, nan, nan, 'Weak', 39.0, 'Average', 40.0, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Average', 52.0, '93.20%', 11.2, '95.70%', '100.00%', 52.7, 40.9, 20.6, 18.5, 39.5, 50.3, 42.3, 28.4, 66.4, 52.2, 8.0, 26.0, 10.2, 8.2, 0.2, 0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 593, 34, nan, 1153864.623, 1909151.595, 41.90654684, -87.71023279, 23, 'HUMBOLDT PARK', 26, 14, '(41.90654684, -87.71023279)') ,\n(610174, 'James Shields Elementary School', 'ES', '4250 S Rockwell St', 'Chicago', 'IL', 60632, '(773) 535-7285', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610174.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 38.0, nan, nan, 'Average', 57.0, 'Average', 58.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 50.0, '97.00%', 13.9, '97.00%', '99.60%', 54.3, nan, 37.2, 35.1, 59.8, 70.8, 41.9, 44.9, 63.2, 66.8, 23.3, 25.6, 18.3, 12.7, 1.1, 1.5, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1834, 39, nan, 1159705.642, 1876174.2680000002, 41.81593528, -87.68968453, 58, 'BRIGHTON PARK', 12, 9, '(41.81593528, -87.68968453)') ,\n(610213, 'James Wadsworth Elementary School', 'ES', '6420 S University Ave', 'Chicago', 'IL', 60637, '(773) 535-0730', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610213.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 49.0, '92.20%', 8.0, '95.50%', '100.00%', 78.0, 45.1, 13.4, 22.3, 60.6, 35.6, nan, nan, nan, nan, nan, nan, 5.4, 5.4, -1.3, -0.6, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 277, 46, nan, 1184773.873, 1862597.144, 41.77812652, -87.59815597, 42, 'WOODLAWN', 20, 3, '(41.77812652, -87.59815597)') ,\n(610217, 'James Ward Elementary School', 'ES', '2701 S Shields Ave', 'Chicago', 'IL', 60616, '(773) 534-9050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610217.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 49.0, 'Strong', 65.0, 'Average', 50.0, 'Average', 46.0, 'Weak', 63.0, 'Average', 56.0, 'Average', 48.0, 'Weak', 43.0, '97.80%', 1.3, '96.90%', '100.00%', 74.8, 63.8, 71.6, 57.0, 68.1, 80.6, 77.8, 53.3, 79.3, 51.9, 44.2, 34.6, 51.1, 23.8, -0.1, -1.0, 'Yellow', 'Red', 43.6, 50.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 492, 40, nan, 1174442.789, 1886499.005, 41.84395162, -87.63531815, 34, 'ARMOUR SQUARE', 11, 9, '(41.84395162, -87.63531815)') ,\n(610274, 'James Weldon Johnson Elementary School', 'ES', '1420 S Albany Ave', 'Chicago', 'IL', 60623, '(773) 534-1829', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610274.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Strong', 67.0, 'Weak', 38.0, 'Strong', 61.0, 'Strong', 75.0, 'Weak', 63.0, 'Strong', 66.0, 'Average', 51.0, 'Strong', 54.0, '94.70%', 1.4, '97.50%', '100.00%', 64.3, nan, 21.3, 19.8, 54.3, 48.8, 17.3, 28.0, 59.3, 50.0, 3.8, 30.8, 8.9, 5.0, -0.4, -1.0, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 365, 37, nan, 1155905.3020000001, 1892874.4430000002, 41.86183983, -87.7031756, 29, 'NORTH LAWNDALE', 24, 10, '(41.86183983, -87.7031756)') ,\n(609772, 'Jane Addams Elementary School', 'ES', '10810 S Avenue H', 'Chicago', 'IL', 60617, '(773) 535-6210', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609772.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 56.0, 'Average', 50.0, 'Average', 44.0, 'Average', 46.0, 'Weak', 47.0, 'Average', 44.0, 'Average', 53.0, 'Average', 53.0, '96.00%', 5.0, '96.80%', '100.00%', 71.4, nan, 32.5, 40.6, 60.7, 63.6, 53.9, 57.7, 77.4, 66.8, 19.5, 36.8, 19.3, 15.9, 0.5, -0.4, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 928, 47, nan, 1202811.36, 1833818.906, 41.69871497, -87.53301251, 52, 'EAST SIDE', 10, 4, '(41.69871497, -87.53301251)') ,\n(610093, 'Jane A Neil Elementary School', 'ES', '8555 S Michigan Ave', 'Chicago', 'IL', 60619, '(773) 535-3000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610093.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 42.0, 'Average', 41.0, 'Weak', 27.0, 'Weak', 28.0, 'Weak', 40.0, 'Weak', 36.0, 'Average', 52.0, 'Average', 53.0, '88.90%', 5.0, '94.60%', '100.00%', nan, 47.4, 25.0, 32.3, 51.7, 60.4, 38.1, 41.7, 62.7, 69.9, 3.4, 13.8, 11.4, 7.1, -0.7, 1.0, 'Red', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 306, 45, nan, 1178690.672, 1848082.986, 41.7384386, -87.62089767, 44, 'CHATHAM', 6, 6, '(41.7384386, -87.62089767)') ,\n(609796, 'Jean Baptiste Beaubien Elementary School', 'ES', '5025 N Laramie Ave', 'Chicago', 'IL', 60630, '(773) 534-3500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609796.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 63.0, nan, nan, 'Average', 49.0, 'Average', 47.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.70%', 7.4, '96.30%', '98.40%', 81.6, nan, 66.8, 68.9, 67.4, 69.1, 77.0, 73.4, 70.6, 60.1, 45.1, 56.6, 49.1, 41.8, 0.2, 0.2, 'Yellow', 'Yellow', 36.8, 72.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1181, 31, nan, 1140829.242, 1933051.044, 41.97237914, -87.75752776, 11, 'JEFFERSON PARK', 45, 16, '(41.97237914, -87.75752776)') ,\n(610031, 'Jean D Lafayette Elementary School', 'ES', '2714 W Augusta Blvd', 'Chicago', 'IL', 60622, '(773) 534-4326', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610031.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 44.0, 'Average', 53.0, 'Average', 43.0, 'Weak', 38.0, 'Weak', 80.0, 'Strong', 72.0, 'Average', 52.0, 'Average', 52.0, '94.10%', 14.3, '96.30%', '98.60%', 54.6, 12.0, 25.3, 23.3, 44.9, 45.6, 26.0, 22.7, 48.3, 46.7, 11.3, 26.4, 8.3, 6.2, -0.3, -1.3, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 474, 34, nan, 1158045.392, 1906579.3480000002, 41.89940402, -87.69494543, 24, 'WEST TOWN', 1, 13, '(41.89940402, -87.69494543)') ,\n(610271, 'Jensen Elementary Scholastic Academy', 'ES', '3030 W Harrison St', 'Chicago', 'IL', 60612, '(773) 534-6840', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610271.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 35.0, 'Average', 41.0, 'Average', 42.0, 'Strong', 67.0, 'Weak', 43.0, 'Average', 40.0, 'Weak', 46.0, 'Average', 47.0, '93.10%', 17.2, '95.30%', '100.00%', 81.3, 75.5, 61.4, 37.3, 58.9, 78.6, 48.8, 37.3, 63.4, 63.2, 11.6, 23.3, 20.2, 11.7, 2.9, 0.9, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 364, 37, nan, 1156127.601, 1897250.02, 41.87384239, -87.70224149, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87384239, -87.70224149)') ,\n(609932, 'Jesse Owens Elementary Community Academy', 'ES', '12450 S State St', 'Chicago', 'IL', 60628, '(773) 535-5661', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609932.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 51.0, 'Strong', 55.0, '91.30%', 4.9, '93.50%', '100.00%', 57.1, 37.9, 11.9, 28.4, 42.2, 42.2, nan, nan, nan, nan, nan, nan, 50.0, 13.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 340, 48, nan, 1178488.2140000002, 1822184.0590000001, 41.66737294, -87.62242272, 53, 'WEST PULLMAN', 9, 5, '(41.66737294, -87.62242272)') ,\n(610173, 'Jesse Sherwood Elementary School', 'ES', '245 W 57th St', 'Chicago', 'IL', 60621, '(773) 535-0829', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610173.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 38.0, nan, nan, 'Average', 47.0, 'Average', 58.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Strong', 56.0, '92.00%', 3.9, '95.50%', '100.00%', 50.9, 41.5, 17.6, 17.1, 51.5, 47.5, 55.2, 40.0, 79.6, 65.2, nan, nan, 25.7, 12.9, 3.6, 3.2, 'Green', 'Green', 43.1, 71.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 355, 42, nan, 1175428.3909999998, 1867127.635, 41.79077279999999, -87.63228120000001, 68, 'ENGLEWOOD', 20, 7, '(41.7907728, -87.6322812)') ,\n(609942, 'Johann W von Goethe Elementary School', 'ES', '2236 N Rockwell St', 'Chicago', 'IL', 60647, '(773) 534-4135', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609942.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 75.0, 'Strong', 61.0, 'Strong', 68.0, 'Average', 52.0, 'Weak', 62.0, 'Strong', 69.0, 'Strong', 57.0, 'Average', 53.0, '95.80%', 5.6, '97.00%', '99.10%', 72.7, 55.1, 40.6, 42.4, 68.6, 80.2, 57.6, 52.7, 71.2, 59.6, 21.4, 31.4, 24.9, 17.6, -0.8, -0.7, 'Red', 'Red', 20.5, 29.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 786, 35, nan, 1158528.612, 1915119.2130000002, 41.9228282, -87.69293637, 22, 'LOGAN SQUARE', 1, 14, '(41.9228282, -87.69293637)') ,\n(610216, 'John A Walsh Elementary School', 'ES', '2015 S Peoria St', 'Chicago', 'IL', 60608, '(773) 534-7950', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610216.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', 'Strong', 60.0, nan, nan, 'Strong', 70.0, 'Very Strong', 91.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 53.0, '95.10%', 2.9, '95.30%', '97.00%', 64.4, nan, 37.6, 37.0, 49.6, 61.5, 47.1, 44.5, 60.4, 63.9, 12.1, 24.2, 17.4, 12.5, 0.2, 0.3, 'Yellow', 'Yellow', 40.0, 42.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 501, 39, nan, 1170942.07, 1890439.543, 41.85484214, -87.64804964, 31, 'LOWER WEST SIDE', 25, 12, '(41.85484214, -87.64804964)') ,\n(609789, 'John Barry Elementary School', 'ES', '2828 N Kilbourn Ave', 'Chicago', 'IL', 60641, '(773) 534-3455', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609789.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, 'Average', 49.0, 'Strong', 70.0, 'Very Strong', 81.0, 'Weak', 34.0, 'Average', 41.0, 'Average', 51.0, 'Average', 49.0, '95.10%', 5.6, '96.10%', '97.70%', 76.4, 55.0, 41.2, 28.2, 44.9, 49.2, 63.6, 32.5, 81.2, 64.8, nan, nan, 15.2, 12.4, 0.7, 0.5, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 800, 29, nan, 1145852.597, 1918556.6519999998, 41.93251118, -87.73942514, 20, 'HERMOSA', 31, 25, '(41.93251118, -87.73942514)') ,\n(609894, 'John B Drake Elementary School', 'ES', '2722 S King Dr', 'Chicago', 'IL', 60616, '(773) 534-9129', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609894.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 59.0, 'Weak', 20.0, 'Average', 57.0, 'Average', 46.0, 'Weak', 43.0, 'Very Weak', 16.0, 'Average', 50.0, 'Average', 49.0, '93.80%', 43.3, '95.90%', '100.00%', 63.0, nan, 15.7, 29.8, 67.1, 60.0, 41.8, 50.0, 83.1, 78.5, 7.7, 26.9, 7.7, 9.5, -0.2, 0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 249, 40, nan, 1179225.388, 1886481.0890000002, 41.84379445, -87.61776768, 35, 'DOUGLAS', 4, 1, '(41.84379445, -87.61776768)') ,\n(610089, 'John B Murphy Elementary School', 'ES', '3539 W Grace St', 'Chicago', 'IL', 60618, '(773) 534-5223', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610089.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 80.0, nan, nan, 'Strong', 75.0, 'Average', 59.0, 'Weak', nan, nan, nan, 'Strong', 58.0, 'Strong', 54.0, '95.60%', 1.9, '96.30%', '100.00%', 61.3, nan, 43.4, 30.5, 40.6, 45.9, 64.3, 44.8, 46.7, 40.7, nan, nan, 22.0, 20.7, -0.5, -0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 638, 31, nan, 1151954.94, 1924991.364, 41.95005034, -87.71682949, 16, 'IRVING PARK', 35, 17, '(41.95005034, -87.71682949)') ,\n(610243, 'John Calhoun North Elementary School', 'ES', '2833 W Adams St', 'Chicago', 'IL', 60612, '(773) 534-6940', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610243.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 52.0, 'Average', 51.0, 'Average', 44.0, 'Average', 54.0, 'Weak', 79.0, 'Very Strong', 88.0, 'Weak', 43.0, 'Average', 48.0, '95.90%', 4.6, '96.50%', '98.10%', 71.3, 48.8, 37.4, 28.3, 52.2, 61.5, 50.5, 37.9, 62.5, 69.3, 12.5, 25.0, 32.8, 14.4, 3.5, 3.5, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 328, 38, nan, 1157392.716, 1898855.0559999999, 41.87822115, -87.69755292, 27, 'EAST GARFIELD PARK', 2, 11, '(41.87822115, -87.69755292)') ,\n(609829, 'John C Burroughs Elementary School', 'ES', '3542 S Washtenaw Ave', 'Chicago', 'IL', 60632, '(773) 535-7226', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609829.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', 'Average', 50.0, 'Average', 55.0, 'Weak', 39.0, 'Weak', 38.0, 'Weak', 71.0, 'Average', 53.0, 'Average', 50.0, 'Weak', 45.0, '95.60%', 0.2, '96.10%', '100.00%', 67.3, 27.9, 47.7, 43.4, 55.9, 45.1, 44.2, 57.4, 51.9, 64.3, 25.0, 15.9, 25.9, 16.0, 0.1, -0.1, 'Yellow', 'Yellow', 42.2, 21.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 510, 39, nan, 1158904.877, 1880879.379, 41.8288631, -87.69249318, 58, 'BRIGHTON PARK', 12, 9, '(41.8288631, -87.69249318)') ,\n(609866, 'John C Coonley Elementary School', 'ES', '4046 N Leavitt St', 'Chicago', 'IL', 60618, '(773) 534-5140', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609866.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, 'Very Strong', 90.0, 'Strong', 65.0, 'Weak', 39.0, 'Weak', 65.0, 'Average', 56.0, 'Strong', 58.0, 'Strong', 55.0, '95.90%', 1.4, '96.50%', '100.00%', 90.1, 52.9, 72.6, 66.7, 61.4, 65.8, 64.8, 64.8, 53.8, 63.5, 27.8, 50.0, 45.3, 39.0, -0.7, 0.9, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 668, 31, nan, 1160924.137, 1927061.47, 41.95554905, -87.68380195, 5, 'NORTH CENTER', 47, 19, '(41.95554905, -87.68380195)') ,\n(609893, 'John C Dore Elementary School', 'ES', '6108 S Natoma Ave', 'Chicago', 'IL', 60638, '(773) 535-2080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609893.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 74.0, 'Strong', 67.0, 'Strong', 69.0, 'Strong', 79.0, 'Weak', 57.0, 'Average', 51.0, 'Strong', 58.0, 'Weak', 46.0, '95.60%', 10.8, '96.70%', '96.20%', 81.8, 52.9, 58.4, 62.7, 70.5, 68.1, 76.9, 74.0, 85.0, 69.4, 58.0, 60.0, 43.8, 25.3, 2.6, 0.5, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 590, 44, nan, 1133084.3, 1863232.77, 41.78092716, -87.78764009, 64, 'CLEARING', 23, 8, '(41.78092716, -87.78764009)') ,\n(609959, 'John Charles Haines Elementary School', 'ES', '247 W 23rd Pl', 'Chicago', 'IL', 60616, '(773) 534-9200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609959.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Weak', 32.0, 'Average', 47.0, 'Weak', 37.0, 'Weak', 37.0, 'Weak', 42.0, 'Average', 48.0, 'Weak', 43.0, 'Weak', 43.0, '98.40%', 0.3, '96.20%', '96.90%', 70.2, 47.6, 69.3, 57.6, 69.0, 69.8, 73.2, 53.6, 74.3, 63.6, 35.9, 32.0, 45.0, 25.7, 1.0, 0.5, 'Green', 'Yellow', 25.9, 93.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 664, 40, nan, 1174842.529, 1888681.3280000002, 41.84993116, -87.63378596, 34, 'ARMOUR SQUARE', 25, 9, '(41.84993116, -87.63378596)') ,\n(610176, 'John D Shoop Math-Science Technical Academy Elementary School', 'ES', '11140 S Bishop St', 'Chicago', 'IL', 60643, '(773) 535-2715', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610176.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 32.0, 'Average', 47.0, 'Average', 43.0, 'Average', 51.0, 'Weak', 53.0, 'Average', 42.0, 'Average', 52.0, 'Strong', 54.0, '93.20%', 40.8, '96.30%', '86.90%', 75.9, 67.7, 34.5, 27.9, 50.0, 52.2, 44.2, 34.2, 50.6, 48.3, 13.8, 13.6, 12.9, 8.3, 0.6, -0.2, 'Green', 'Yellow', 4.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 553, 49, nan, 1168507.416, 1830689.468, 41.69093337, -87.65870614, 75, 'MORGAN PARK', 34, 22, '(41.69093337, -87.65870614)') ,\n(609898, 'John F Eberhart Elementary School', 'ES', '3400 W 65th Pl', 'Chicago', 'IL', 60629, '(773) 535-9190', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609898.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 42.0, 'Average', 41.0, 'Weak', 33.0, 'Weak', 34.0, 'Weak', 42.0, 'Weak', 32.0, nan, nan, nan, nan, '95.30%', 13.1, '95.60%', '100.00%', 48.2, 27.3, 28.2, 31.4, 52.4, 49.5, 32.8, 39.9, 50.4, 51.3, 10.2, 20.9, 8.9, 9.6, -1.5, -0.7, 'Red', 'Red', 15.9, 17.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1777, 44, nan, 1154822.365, 1861019.3830000001, 41.7744471, -87.70800189, 66, 'CHICAGO LAWN', 15, 8, '(41.7744471, -87.70800189)') ,\n(609919, 'John Fiske Elementary School', 'ES', '6145 S Ingleside Ave', 'Chicago', 'IL', 60637, '(773) 535-0990', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609919.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 13.0, 'Weak', 24.0, 'Weak', 37.0, 'Average', 55.0, 'Weak', 40.0, 'Weak', 35.0, 'Average', 47.0, 'Average', 50.0, '92.90%', 37.1, '91.70%', '91.40%', 87.1, nan, 15.7, 14.6, 58.0, 54.2, 30.1, 30.1, 69.7, 61.8, 32.1, 28.6, 4.0, 4.0, -1.3, -1.5, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 254, 46, nan, 1183547.778, 1864329.648, 41.78290934, -87.60259685, 42, 'WOODLAWN', 20, 3, '(41.78290934, -87.60259685)') ,\n(609718, 'John F Kennedy High School', 'HS', '6325 W 56th St', 'Chicago', 'IL', 60638, '(773) 535-2325', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609718.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 53.0, 'Weak', 30.0, 'Weak', 32.0, 'Weak', 27.0, 'Weak', 22.0, 'Weak', 24.0, nan, nan, nan, nan, '83.00%', 22.0, '94.70%', '99.30%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.8, 14.1, 15.1, 15.6, 0.8, 16.4, 1.3, 24.4, 53.3, 52.9, 1589, 44, 59.7, 1135044.301, 1866707.447, 41.79042796, -87.78037224, 56, 'GARFIELD RIDGE', 23, 8, '(41.79042796, -87.78037224)') ,\n(610263, 'John Foster Dulles Elementary School', 'ES', '6311 S Calumet Ave', 'Chicago', 'IL', 60637, '(773) 535-0690', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610263.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 42.0, 'Weak', 22.0, 'Average', 53.0, 'Average', 42.0, 'Weak', 32.0, 'Weak', 21.0, 'Average', 51.0, 'Strong', 55.0, '94.10%', 28.2, '97.30%', '100.00%', 91.7, nan, 16.9, 19.9, 42.3, 47.4, 15.5, 13.1, 61.9, 48.8, 2.5, 5.1, 9.1, 3.8, 1.4, 0.1, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 537, 42, nan, 1179581.237, 1863162.082, 41.779796999999995, -87.61717494, 69, 'GREATER GRAND CROSSING', 20, 3, '(41.779797, -87.61717494)') ,\n(610228, 'John Greenleaf Whittier Elementary School', 'ES', '1900 W 23rd St', 'Chicago', 'IL', 60608, '(773) 535-4590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610228.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 65.0, nan, nan, 'Average', 45.0, 'Weak', 24.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.90%', 3.8, '96.50%', '100.00%', 50.4, nan, 13.3, 17.3, 35.9, 36.2, 22.3, 21.5, 50.0, 45.8, 6.4, 25.5, 6.3, 3.9, -0.9, -0.2, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 409, 39, nan, 1164042.174, 1888805.999, 41.85050786, -87.67342126, 31, 'LOWER WEST SIDE', 25, 10, '(41.85050786, -87.67342126)') ,\n(609694, 'John Hancock College Preparatory High School', 'HS', '4034 W 56th St', 'Chicago', 'IL', 60629, '(773) 535-2410', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609694.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 51.0, 'Average', 44.0, 'Average', 44.0, 'Average', 41.0, 'Weak', 37.0, 'Average', 42.0, 'Average', 50.0, 'Weak', 44.0, '78.20%', 7.5, '95.90%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.6, 13.8, 14.7, 15.0, 0.4, 15.8, 1.1, 22.6, 56.1, 45.9, 967, 44, 87.4, 1150280.45, 1867197.8269999998, 41.79149114, -87.72449161, 62, 'WEST ELSDON', 13, 8, '(41.79149114, -87.72449161)') ,\n(609971, 'John Harvard Elementary School of Excellence', 'ES', '7525 S Harvard', 'Chicago', 'IL', 60620, '(773) 535-3045', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609971.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 51.0, 'Weak', 24.0, 'Strong', 61.0, 'Strong', 61.0, 'Weak', 43.0, 'Weak', 36.0, 'Average', 47.0, 'Strong', 55.0, '93.00%', 8.7, '96.00%', '97.80%', nan, nan, 25.6, 28.2, 42.7, 46.8, 50.0, 38.9, 58.8, 53.1, 14.6, 23.1, 13.8, 7.7, 1.2, 0.4, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 476, 45, nan, 1175413.4270000001, 1854942.1169999999, 41.75733472, -87.63270008, 69, 'GREATER GRAND CROSSING', 17, 6, '(41.75733472, -87.63270008)') ,\n(609975, 'John Hay Elementary Community Academy', 'ES', '1018 N Laramie Ave', 'Chicago', 'IL', 60651, '(773) 534-6000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609975.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 44.0, nan, nan, 'Average', 49.0, 'Strong', 61.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Strong', 55.0, '92.90%', 30.8, '97.10%', '100.00%', 53.8, 51.5, 45.8, 26.8, 58.0, 68.4, 34.3, 34.1, 68.7, 65.2, 0.0, 5.4, 11.9, 10.0, 0.0, 0.9, 'Yellow', 'Green', 10.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 554, 36, nan, 1141465.048, 1906381.947, 41.89918473, -87.75585034, 25, 'AUSTIN', 37, 15, '(41.89918473, -87.75585034)') ,\n(609964, 'John H Hamline Elementary School', 'ES', '4747 S Bishop St', 'Chicago', 'IL', 60609, '(773) 535-4565', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609964.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 33.0, 'Weak', 24.0, 'Weak', 29.0, 'Average', 43.0, 'Weak', 17.0, 'Weak', 33.0, 'Weak', 44.0, 'Average', 47.0, '94.40%', 7.5, '95.70%', '100.00%', 61.7, 48.9, 39.1, 19.5, 50.8, 67.9, 58.0, 26.5, 64.5, 56.5, 9.1, 18.2, 20.1, 6.1, 1.0, -0.2, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 679, 42, nan, 1167511.897, 1873073.8830000001, 41.80726347, -87.66113856, 61, 'NEW CITY', 20, 9, '(41.80726347, -87.66113856)') ,\n(610026, 'John H Kinzie Elementary School', 'ES', '5625 S Mobile Ave', 'Chicago', 'IL', 60638, '(773) 535-2425', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610026.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, nan, nan, 'Average', 49.0, 'Average', 54.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 48.0, '95.40%', 14.9, '95.30%', '99.10%', 62.0, nan, 62.6, 42.6, 51.6, 67.6, 68.3, 50.2, 60.0, 56.4, 36.7, 40.5, 32.6, 19.7, 0.4, -0.3, 'Yellow', 'Yellow', 26.5, 63.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 733, 44, nan, 1135391.774, 1866445.86, 41.78970398, -87.7791043, 56, 'GARFIELD RIDGE', 23, 8, '(41.78970398, -87.7791043)') ,\n(610207, 'John H Vanderpoel Elementary Magnet School', 'ES', '9510 S Prospect Ave', 'Chicago', 'IL', 60643, '(773) 535-2690', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610207.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 74.0, nan, nan, 'Weak', 24.0, 'Weak', 31.0, 'Weak', nan, nan, nan, 'Strong', 55.0, 'Average', 50.0, '97.20%', 0.3, '96.20%', '100.00%', 81.6, 69.8, 51.7, 47.7, 53.5, 59.8, 42.7, 54.5, 35.7, 55.9, 11.1, 38.9, 17.7, 24.1, -0.3, 1.0, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 317, 49, nan, 1166544.213, 1841551.135, 41.72078152, -87.66558526, 72, 'BEVERLY', 19, 22, '(41.72078152, -87.66558526)') ,\n(609782, 'John J Audubon Elementary School', 'ES', '3500 N Hoyne Ave', 'Chicago', 'IL', 60618, '(773) 534-5470', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609782.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 95.0, 'Strong', 73.0, 'Strong', 75.0, 'Strong', 60.0, 'Weak', 60.0, 'Strong', 65.0, 'Strong', 61.0, 'Average', 47.0, '96.20%', 4.5, '96.70%', '98.60%', nan, 54.3, 72.1, 68.9, 65.5, 76.7, 67.5, 70.3, 82.6, 65.3, 31.6, 36.8, 45.3, 34.8, 1.2, 1.1, 'Green', 'Green', 47.4, 61.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 580, 33, nan, 1161693.811, 1923258.727, 41.94509808, -87.68107885, 5, 'NORTH CENTER', 47, 19, '(41.94509808, -87.68107885)') ,\n(610126, 'John J Pershing Elementary Humanities Magnet', 'ES', '3113 S Rhodes Ave', 'Chicago', 'IL', 60616, '(773) 534-9272', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610126.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', nan, nan, 'Very Strong', 90.0, nan, nan, nan, nan, 'Weak', 70.0, 'Strong', 78.0, nan, nan, nan, nan, '95.50%', 19.2, '96.70%', '100.00%', nan, 92.3, 56.3, 56.3, 77.1, 80.9, nan, nan, nan, nan, nan, nan, 54.9, 39.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 258, 40, nan, 1180449.2040000001, 1884392.3159999999, 41.83803465, -87.61334083, 35, 'DOUGLAS', 4, 2, '(41.83803465, -87.61334083)') ,\n(610054, 'John L Marsh Elementary School', 'ES', '9822 S Exchange Ave', 'Chicago', 'IL', 60617, '(773) 535-6430', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610054.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Strong', 72.0, 'Average', 53.0, 'Average', 53.0, 'Weak', 39.0, 'Weak', 55.0, 'Average', 48.0, 'Average', 50.0, 'Strong', 55.0, '95.70%', 5.4, '95.50%', '99.10%', 55.8, 37.6, 34.5, 34.9, 53.2, 58.7, 33.5, 44.9, 50.8, 52.2, 11.1, 31.1, 18.9, 14.5, -0.9, -0.8, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 841, 47, nan, 1197424.97, 1840279.493, 41.71657908, -87.55251993, 51, 'SOUTH DEERING', 10, 4, '(41.71657908, -87.55251993)') ,\n(609723, 'John Marshall Metropolitan High School', 'HS', '3250 W Adams St', 'Chicago', 'IL', 60624, '(773) 534-6455', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609723.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 51.0, 'Average', 52.0, 'Average', 45.0, 'Average', 46.0, 'Weak', 49.0, 'Average', 42.0, nan, nan, nan, nan, '74.10%', 42.9, '95.80%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.7, 11.8, 13.4, 13.2, 1.5, 14.6, 1.2, 6.5, 36.8, 45.2, 726, 38, 55.2, 1154542.5829999999, 1898925.511, 41.87847194, -87.70801612, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87847194, -87.70801612)') ,\n(609710, 'John M Harlan Community Academy High School', 'HS', '9652 S Michigan Ave', 'Chicago', 'IL', 60628, '(773) 535-5400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609710.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, nan, nan, nan, nan, '81.20%', 21.6, '94.40%', '95.70%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 16.0, 40.0, 17.8, 15.6, -2.9, -0.8, 'Red', 'Yellow', 100.0, 40.0, 13.2, 12.9, 14.1, 13.7, 0.5, 14.7, 0.6, 5.9, 47.9, 55.7, 1245, 48, 64.4, 1178815.804, 1840781.935, 41.71840075, -87.62066069, 49, 'ROSELAND', 6, 5, '(41.71840075, -87.62066069)') ,\n(609954, 'John Milton Gregory Elementary School', 'ES', '3715 W Polk St', 'Chicago', 'IL', 60624, '(773) 534-6820', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609954.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 44.0, 'Average', 43.0, 'Weak', 39.0, 'Average', 43.0, 'Weak', 66.0, 'Average', 59.0, 'Average', 49.0, 'Strong', 55.0, '95.20%', 3.2, '94.60%', '100.00%', 61.6, 45.5, 36.8, 17.9, 62.3, 70.8, 44.3, 37.7, 69.1, 77.9, 6.3, 41.7, 24.2, 19.5, 1.5, 2.1, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 323, 37, nan, 1151660.565, 1896077.812, 41.87071468, -87.71867323, 27, 'EAST GARFIELD PARK', 24, 11, '(41.87071468, -87.71867323)') ,\n(610180, 'John M Smyth Elementary School', 'ES', '1059 W 13th St', 'Chicago', 'IL', 60608, '(773) 534-7180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610180.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 37.0, nan, nan, 'Average', 54.0, 'Strong', 64.0, 'Weak', nan, nan, nan, 'Weak', 43.0, 'Average', 52.0, '92.40%', 20.1, '93.30%', '100.00%', 67.8, nan, 17.6, 10.6, 38.1, 46.0, 24.8, 12.3, 54.2, 38.8, 1.4, 1.4, 6.3, 3.2, 1.1, -0.9, 'Green', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 475, 38, nan, 1169435.063, 1894247.175, 41.86532347, -87.65347031, 28, 'NEAR WEST SIDE', 2, 12, '(41.86532347, -87.65347031)') ,\n(610199, 'Johnnie Colemon Elementary Academy', 'ES', '1441 W 119th St', 'Chicago', 'IL', 60643, '(773) 535-3975', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610199.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 60.0, nan, nan, 'Average', 56.0, 'Strong', 63.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Strong', 57.0, '95.90%', 23.4, '95.40%', '100.00%', 75.3, 53.4, 24.4, 22.1, 47.6, 46.9, 47.2, 44.4, 37.7, 50.7, 12.5, 16.7, 14.6, 18.4, 0.2, 0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 266, 49, nan, 1168705.904, 1825787.71, 41.67747786, -87.65812011, 53, 'WEST PULLMAN', 34, 5, '(41.67747786, -87.65812011)') ,\n(610111, 'John Palmer Elementary School', 'ES', '5051 N Kenneth Ave', 'Chicago', 'IL', 60630, '(773) 534-3704', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610111.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 75.0, nan, nan, 'Strong', 61.0, 'Strong', 68.0, 'Weak', nan, nan, nan, 'Strong', 65.0, 'Strong', 61.0, '96.30%', 12.8, '94.80%', '99.20%', 72.2, 42.4, 47.7, 38.0, 58.9, 58.5, 48.9, 45.3, 52.5, 57.3, 24.8, 37.6, 25.3, 15.1, 0.5, 0.3, 'Green', 'Yellow', 26.5, 74.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 919, 31, nan, 1145817.577, 1933407.476, 41.97326374, -87.73917547, 14, 'ALBANY PARK', 39, 17, '(41.97326374, -87.73917547)') ,\n(609775, 'John P Altgeld Elementary School', 'ES', '1340 W 71st St', 'Chicago', 'IL', 60636, '(773) 535-3250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609775.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 48.0, 'Weak', 37.0, 'Strong', 64.0, 'Average', 58.0, 'Weak', 27.0, 'Weak', 35.0, 'Weak', 46.0, 'Average', 48.0, '91.90%', 32.9, '95.50%', '100.00%', 42.1, nan, 30.1, 34.4, 67.1, 52.4, 37.2, 37.2, 56.0, 56.8, 8.8, 40.4, 16.4, 20.9, -0.8, 3.0, 'Red', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 527, 43, nan, 1168430.269, 1857717.233, 41.76510322, -87.6582125, 67, 'WEST ENGLEWOOD', 17, 7, '(41.76510322, -87.6582125)') ,\n(610184, 'John Spry Elementary Community School', 'ES', '2400 S Marshall Blvd', 'Chicago', 'IL', 60623, '(773) 534-1700', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610184.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 66.0, 'Average', 59.0, 'Strong', 70.0, 'Strong', 67.0, 'Weak', 52.0, 'Average', 43.0, 'Weak', 46.0, 'Average', 48.0, '96.20%', 5.9, '97.40%', '99.00%', 44.4, 12.8, nan, nan, nan, nan, nan, nan, nan, nan, 18.7, 13.3, 17.9, 7.3, 1.7, 1.1, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 809, 39, nan, 1157044.9109999998, 1887895.675, 41.84815451, -87.69912725, 30, 'SOUTH LAWNDALE', 12, 10, '(41.84815451, -87.69912725)') ,\n(610269, 'John T McCutcheon Elementary School', 'ES', '4865 N Sheridan Rd', 'Chicago', 'IL', 60640, '(773) 534-2680', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610269.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 56.0, 'Weak', 36.0, 'Strong', 67.0, 'Average', 41.0, 'Weak', 32.0, 'Average', 42.0, 'Average', 52.0, 'Average', 53.0, '94.00%', 8.8, '94.80%', '98.10%', 62.4, 66.3, 31.3, 30.3, 46.7, 48.3, 34.1, 40.5, 48.1, 58.4, 9.1, 13.6, 19.5, 10.8, 1.0, -0.2, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 355, 32, nan, 1168789.535, 1932738.4619999998, 41.97095968, -87.65472193, 3, 'UPTOWN', 48, 20, '(41.97095968, -87.65472193)') ,\n(610130, 'John T Pirie Fine Arts & Academic Center Elementary School', 'ES', '650 E 85th St', 'Chicago', 'IL', 60619, '(773) 535-3435', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610130.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 26.0, 'Average', 44.0, 'Strong', 63.0, 'Strong', 62.0, 'Weak', 43.0, 'Average', 46.0, nan, nan, nan, nan, '93.50%', 20.8, '95.40%', '100.00%', 74.5, 49.6, 31.1, 37.3, 55.8, 49.7, 44.2, 50.0, 71.8, 73.7, nan, nan, 10.7, 20.9, 0.1, 1.6, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 384, 45, nan, 1182225.898, 1848815.302, 41.74036711, -87.60792289, 44, 'CHATHAM', 6, 6, '(41.74036711, -87.60792289)') ,\n(609864, 'John W Cook Elementary School', 'ES', '8150 S Bishop St', 'Chicago', 'IL', 60620, '(773) 535-3315', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609864.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Average', 47.0, '94.00%', 26.0, '94.00%', '100.00%', 57.1, 36.0, 21.5, 12.7, 42.4, 55.4, 41.2, 31.8, 68.7, 68.2, 1.7, 6.8, 6.4, 4.6, 0.8, 0.6, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 482, 49, nan, 1168050.5320000001, 1850484.1530000002, 41.74526284, -87.65981190000001, 71, 'AUBURN GRESHAM', 21, 6, '(41.74526284, -87.6598119)') ,\n(609937, 'John W Garvy Elementary School', 'ES', '5225 N Oak Park Ave', 'Chicago', 'IL', 60656, '(773) 534-1185', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609937.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 60.0, 'Strong', 70.0, 'Average', 50.0, 'Average', 51.0, 'Weak', 51.0, 'Average', 55.0, 'Strong', 56.0, 'Average', 49.0, '95.30%', 10.2, '96.50%', '97.30%', 83.4, 43.6, 63.9, 57.8, 58.1, 63.9, 60.6, 59.4, 51.4, 55.3, 38.4, 49.3, 34.4, 33.5, -0.1, 0.2, 'Yellow', 'Yellow', 44.6, 45.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 720, 30, nan, 1130304.414, 1934430.6940000001, 41.97635262, -87.79619832, 10, 'NORWOOD PARK', 41, 16, '(41.97635262, -87.79619832)') ,\n(610225, 'John Whistler Elementary School', 'ES', '11533 S Ada St', 'Chicago', 'IL', 60643, '(773) 535-5560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610225.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 45.0, 'Weak', 32.0, 'Average', 41.0, 'Weak', 28.0, 'Weak', 20.0, 'Weak', 21.0, 'Average', 52.0, 'Strong', 55.0, '92.40%', 11.1, '97.00%', '100.00%', 74.7, 60.2, 34.0, 27.4, 49.0, 61.2, 31.5, 28.0, 50.4, 51.2, 6.0, 8.2, 5.4, 4.1, -0.9, 0.5, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 375, 49, nan, 1169444.9340000001, 1828137.9440000001, 41.68391137, -87.65534733, 53, 'WEST PULLMAN', 34, 5, '(41.68391137, -87.65534733)') ,\n(609828, 'Jonathan Burr Elementary School', 'ES', '1621 W Wabansia Ave', 'Chicago', 'IL', 60622, '(773) 534-4090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609828.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 71.0, 'Strong', 66.0, 'Strong', 68.0, 'Average', 51.0, 'Weak', 53.0, 'Strong', 63.0, 'Strong', 56.0, 'Average', 49.0, '95.90%', 4.3, '96.90%', '100.00%', 78.5, 53.1, 53.8, 43.8, 51.9, 51.9, 71.3, 57.7, 69.2, 58.4, 14.3, 25.0, 29.1, 22.2, 0.4, 0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 348, 35, nan, 1165168.4640000002, 1911344.1809999999, 41.91233061, -87.66864691, 24, 'WEST TOWN', 1, 14, '(41.91233061, -87.66864691)') ,\n(610159, 'Jonathan Y Scammon Elementary School', 'ES', '4201 W Henderson St', 'Chicago', 'IL', 60641, '(773) 534-3475', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610159.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 44.0, nan, nan, 'Weak', 21.0, 'Weak', 20.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Weak', 44.0, '95.90%', 8.0, '95.30%', '100.00%', 69.6, 39.3, 32.4, 27.7, 50.9, 52.2, 39.1, 33.0, 61.5, 60.5, 14.7, 33.7, 14.8, 11.9, -0.6, 0.2, 'Red', 'Yellow', 19.4, 100.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 937, 29, nan, 1147787.325, 1921900.2959999999, 41.94164944, -87.73222901, 16, 'IRVING PARK', 30, 17, '(41.94164944, -87.73222901)') ,\n(609865, 'Jordan Elementary Community School', 'ES', '7414 N Wolcott Ave', 'Chicago', 'IL', 60626, '(773) 534-2220', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609865.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 61.0, 'Weak', 31.0, 'Average', 57.0, 'Average', 49.0, 'Weak', 17.0, 'Very Weak', 15.0, 'Average', 52.0, 'Average', 51.0, '95.60%', 16.4, '95.10%', '100.00%', 60.1, 42.0, 36.0, 21.1, 41.4, 54.1, 26.3, 19.1, 43.2, 34.7, 4.8, 15.9, 16.9, 7.1, 0.3, -1.2, 'Yellow', 'Red', 37.7, 23.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 753, 32, nan, 1162385.2240000002, 1949501.716, 42.01709536, -87.67779928, 1, 'ROGERS PARK', 49, 24, '(42.01709536, -87.67779928)') ,\n(610313, 'Jose De Diego Elementary Community Academy', 'ES', '1313 N Claremont Ave', 'Chicago', 'IL', 60622, '(773) 534-4451', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610313.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'Yes', 'Average', 57.0, 'Average', 53.0, 'Weak', 37.0, 'Average', 40.0, 'Weak', 60.0, 'Strong', 64.0, 'Average', 53.0, 'Strong', 54.0, '95.00%', 29.0, '96.80%', '100.00%', 75.0, 51.4, 29.9, 32.4, 49.2, 44.4, 45.5, 45.1, 49.1, 51.3, 17.5, 33.3, 16.7, 11.3, -0.3, -1.1, 'Yellow', 'Red', 49.2, 23.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 869, 35, nan, 1160540.669, 1908752.314, 41.90531549, -87.68572005, 24, 'WEST TOWN', 1, 14, '(41.90531549, -87.68572005)') ,\n(609950, 'Josefa Ortiz De Dominguez Elementary School', 'ES', '3000 S Lawndale Ave', 'Chicago', 'IL', 60623, '(773) 534-1600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609950.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', nan, nan, 'No', nan, nan, 'Strong', 69.0, nan, nan, nan, nan, 'Weak', 48.0, 'Average', 54.0, 'Average', 51.0, 'Strong', 55.0, '95.80%', 3.6, '97.00%', '99.00%', 72.4, 51.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 793, 37, nan, 1152138.268, 1884434.021, 41.83875333, -87.71722614, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83875333, -87.71722614)') ,\n(610242, 'Joseph Brennemann Elementary School', 'ES', '4251 N Clarendon Ave', 'Chicago', 'IL', 60613, '(773) 534-5766', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610242.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 50.0, 'Average', 56.0, 'Average', 55.0, 'Average', 59.0, 'Weak', 41.0, 'Average', 54.0, 'Average', 53.0, 'Strong', 59.0, '96.50%', 13.7, '97.70%', '96.70%', 79.8, 47.2, 32.4, 30.8, 50.5, 65.7, 48.8, 30.0, 66.2, 50.0, 8.3, 25.0, 18.1, 10.9, 1.0, 1.0, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 362, 32, nan, 1170183.793, 1928747.67, 41.95997841, -87.64971223, 3, 'UPTOWN', 46, 19, '(41.95997841, -87.64971223)') ,\n(609938, 'Joseph E Gary Elementary School', 'ES', '3740 W 31st St', 'Chicago', 'IL', 60623, '(773) 534-1455', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609938.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 32.0, 'Weak', 27.0, 'Weak', 29.0, 'Weak', 28.0, 'Weak', 29.0, 'Very Weak', 17.0, 'Weak', 45.0, 'Average', 48.0, '96.10%', 16.8, '96.20%', '100.00%', nan, nan, 34.5, 21.8, 50.5, 55.3, 45.5, 27.7, 58.4, 41.7, 12.3, 21.3, 14.3, 7.6, 0.4, 0.1, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1204, 37, nan, 1151739.739, 1883825.92, 41.83709246, -87.71870452, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83709246, -87.71870452)') ,\n(610041, 'Josephine C Locke Elementary School', 'ES', '2828 N Oak Park Ave', 'Chicago', 'IL', 60634, '(773) 534-3300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610041.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 57.0, 'Average', 47.0, 'Average', 49.0, 'Average', 56.0, 'Weak', 44.0, 'Average', 45.0, 'Average', 47.0, 'Average', 50.0, '95.50%', 2.2, '95.10%', '100.00%', 63.4, 39.5, 36.1, 30.5, 55.3, 63.0, 58.7, 46.0, 64.0, 70.5, 26.1, 33.8, 21.1, 15.6, 0.7, 1.7, 'Green', 'Green', 23.9, 45.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1317, 29, nan, 1130557.942, 1918183.07, 41.93176301, -87.79564085, 18, 'MONTCLARE', 36, 25, '(41.93176301, -87.79564085)') ,\n(610015, 'Joseph Jungman Elementary School', 'ES', '1746 S Miller St', 'Chicago', 'IL', 60608, '(773) 534-7375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610015.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 36.0, 'Weak', 34.0, 'Average', 56.0, 'Average', 53.0, 'Weak', 20.0, 'Average', 43.0, 'Average', 48.0, 'Average', 49.0, '96.40%', 5.3, '96.30%', '93.80%', 75.5, 34.7, 32.5, 20.0, 59.2, 64.4, 25.0, 34.9, 51.3, 71.3, 6.9, 34.5, 7.8, 6.6, 0.1, -0.7, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 286, 39, nan, 1169764.005, 1891617.595, 41.85810053, -87.65233933, 31, 'LOWER WEST SIDE', 25, 12, '(41.85810053, -87.65233933)') ,\n(609925, 'Joseph Kellman Corporate Community Elementary School', 'ES', '751 S Sacramento Blvd', 'Chicago', 'IL', 60612, '(773) 534-6602', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609925.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, 'Strong', 68.0, 'Average', 42.0, 'Average', 45.0, 'Weak', 37.0, 'Average', 50.0, 'Strong', 59.0, 'Strong', 57.0, '95.20%', 4.4, '95.80%', '100.00%', 84.7, 55.3, 37.0, 32.3, 48.9, 56.0, 34.5, 43.2, 56.5, 55.2, 0.0, 11.5, 13.3, 10.5, -0.3, -0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 293, 37, nan, 1156531.906, 1896287.544, 41.87119309, -87.70078311, 27, 'EAST GARFIELD PARK', 2, 11, '(41.87119309, -87.70078311)') ,\n(610043, 'Joseph Lovett Elementary School', 'ES', '6333 W Bloomingdale Ave', 'Chicago', 'IL', 60639, '(773) 534-3130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610043.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 43.0, nan, nan, 'Weak', 32.0, 'Weak', 22.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Weak', 45.0, '95.40%', 16.1, '95.40%', '100.00%', 68.5, 62.7, 23.5, 33.1, 51.2, 51.2, 32.9, 37.3, 40.3, 65.5, 7.8, 27.5, 9.1, 8.3, -0.2, -0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 490, 29, nan, 1133787.949, 1911253.2, 41.91269042, -87.78393402, 25, 'AUSTIN', 29, 25, '(41.91269042, -87.78393402)') ,\n(610189, 'Joseph Stockton Elementary School', 'ES', '4420 N Beacon St', 'Chicago', 'IL', 60640, '(773) 534-2450', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610189.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 37.0, nan, nan, 'Average', 51.0, 'Strong', 68.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.90%', 11.0, '95.30%', '100.00%', 53.2, 47.2, 35.3, 28.2, 41.2, 50.0, 54.7, 56.4, 51.1, 50.5, 21.1, 44.7, 18.3, 12.4, -0.5, -0.1, 'Yellow', 'Yellow', 30.8, 75.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 462, 32, nan, 1166321.035, 1929570.108, 41.9623189, -87.66388983, 3, 'UPTOWN', 46, 19, '(41.9623189, -87.66388983)') ,\n(610218, 'Joseph Warren Elementary School', 'ES', '9239 S Jeffery Ave', 'Chicago', 'IL', 60617, '(773) 535-6625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610218.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 33.0, 'Average', 55.0, 'Average', 49.0, 'Average', 40.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Average', 50.0, '95.00%', 21.6, '95.30%', '100.00%', 38.6, 20.0, 26.1, 21.5, 55.3, 46.4, 33.3, 27.0, 45.6, 35.0, 8.6, 25.7, 13.7, 14.7, -1.0, -0.4, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 328, 47, nan, 1191168.455, 1843939.0669999998, 41.72677474, -87.5753162, 48, 'CALUMET HEIGHTS', 8, 4, '(41.72677474, -87.5753162)') ,\n(610019, 'Joshua D Kershaw Elementary School', 'ES', '6450 S Lowe Ave', 'Chicago', 'IL', 60621, '(773) 535-3050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610019.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 62.0, 'Strong', 63.0, 'Strong', 69.0, 'Average', 56.0, 'Weak', 50.0, 'Average', 59.0, 'Strong', 58.0, 'Strong', 63.0, '94.80%', 8.2, '96.00%', '100.00%', nan, nan, 55.2, 46.3, 71.6, 71.6, 67.2, 58.1, 83.3, 78.7, 5.6, 38.9, 22.3, 16.2, 1.9, 1.1, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 262, 45, nan, 1173111.97, 1861899.898, 41.77647883, -87.64092929, 68, 'ENGLEWOOD', 20, 7, '(41.77647883, -87.64092929)') ,\n(610129, 'Josiah Pickard Elementary School', 'ES', '2301 W 21st Pl', 'Chicago', 'IL', 60608, '(773) 535-7280', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610129.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 50.0, 'Weak', 36.0, 'Weak', 35.0, 'Weak', 37.0, 'Weak', 32.0, 'Weak', 34.0, 'Average', 48.0, 'Strong', 54.0, '95.80%', 5.1, '96.40%', '100.00%', 76.4, nan, 48.9, 25.6, 55.0, 78.6, 42.5, 30.2, 70.0, 69.8, 6.6, 8.2, 16.1, 11.0, 1.7, 0.0, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 656, 39, nan, 1161333.371, 1889647.219, 41.85287295, -87.6833397, 31, 'LOWER WEST SIDE', 25, 12, '(41.85287295, -87.6833397)') ,\n(610022, 'Joyce Kilmer Elementary School', 'ES', '6700 N Greenview Ave', 'Chicago', 'IL', 60626, '(773) 534-2115', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610022.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', nan, nan, 'Average', 43.0, nan, nan, nan, nan, 'Weak', 43.0, 'Average', 42.0, 'Average', 49.0, 'Average', 51.0, '96.00%', 15.6, '96.90%', '100.00%', 70.0, 51.2, 40.5, 28.6, 53.4, 61.2, 39.4, 33.7, 57.8, 52.8, 22.7, 27.3, 22.8, 15.0, -0.2, -0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 886, 32, nan, 1165015.365, 1944631.6819999998, 42.00367622, -87.66826026, 1, 'ROGERS PARK', 40, 24, '(42.00367622, -87.66826026)') ,\n(610253, 'Julia C Lathrop Elementary School', 'ES', '1440 S Christiana Ave', 'Chicago', 'IL', 60623, '(773) 534-1812', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610253.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 67.0, nan, nan, 'Average', 58.0, 'Weak', 26.0, 'Weak', nan, nan, nan, 'Weak', 40.0, 'Average', 50.0, '90.50%', 113.5, '95.50%', '100.00%', 38.5, nan, 19.5, 13.2, 45.5, 34.1, 18.4, 34.7, 35.0, 61.1, 7.7, 3.8, 5.9, 4.0, -1.7, -0.6, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 83, 37, nan, 1154232.147, 1892627.1719999998, 41.86119482, -87.70932409999999, 29, 'NORTH LAWNDALE', 24, 10, '(41.86119482, -87.7093241)') ,\n(610000, 'Julia Ward Howe Elementary School of Excellence', 'ES', '720 N Lorel Ave', 'Chicago', 'IL', 60644, '(773) 534-6060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610000.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 1', 'No', 'Strong', 67.0, 'Average', 46.0, 'Very Strong', 90.0, 'Strong', 69.0, 'Weak', 59.0, 'Strong', 67.0, 'Strong', 57.0, 'Strong', 64.0, '96.00%', 7.7, '97.10%', '100.00%', nan, nan, 36.9, 35.6, 63.1, 75.2, 38.5, 39.1, 67.5, 56.4, 7.4, 14.8, 20.6, 9.3, 0.8, -0.8, 'Green', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 576, 36, nan, 1140502.548, 1904349.511, 41.89362521, -87.75943557, 25, 'AUSTIN', 37, 15, '(41.89362521, -87.75943557)') ,\n(610280, 'Kate S Buckingham Special Education Center', 'ES', '9207 S Phillips Ave', 'Chicago', 'IL', 60617, '(773) 535-6422', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610280.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', nan, 'Standard', 'Not Applicable', 'Level 3', 'No', 'Weak', 31.0, nan, nan, 'Weak', 24.0, 'Weak', 20.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '83.40%', 116.9, '94.40%', '100.00%', nan, nan, 20.0, 20.0, 80.0, 70.0, 0.0, 4.3, 45.0, 59.1, nan, nan, 0.0, 0.0, -0.4, -0.8, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 26, 47, nan, 1194146.741, 1844346.869, 41.72782129, -87.56439326, 48, 'CALUMET HEIGHTS', 7, 4, '(41.72782129, -87.56439326)') ,\n(610016, 'Kate S Kellogg Elementary School', 'ES', '9241 S Leavitt St', 'Chicago', 'IL', 60620, '(773) 535-2590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610016.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 76.0, nan, nan, 'Average', 57.0, 'Strong', 63.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Weak', 46.0, '96.30%', 12.0, '95.40%', '91.80%', 78.8, 10.0, 49.4, 67.8, 65.5, 65.5, 52.0, 77.5, 49.0, 64.7, 17.5, 47.5, 24.2, 30.9, -0.4, 0.7, 'Yellow', 'Yellow', 28.2, 36.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 283, 49, nan, 1163362.1840000001, 1843156.174, 41.72525315, -87.67719555, 72, 'BEVERLY', 19, 22, '(41.72525315, -87.67719555)') ,\n(609716, 'Kelvyn Park High School', 'HS', '4343 W Wrightwood Ave', 'Chicago', 'IL', 60639, '(773) 534-4200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609716.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 32.0, 'Weak', 28.0, 'Weak', 32.0, 'Weak', 28.0, 'Weak', 19.0, 'Average', 43.0, nan, nan, nan, nan, '74.40%', 12.9, '94.50%', '99.50%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.8, 12.2, 13.8, 13.8, 1.0, 14.9, 1.1, 16.7, 54.1, 35.0, 1252, 29, 56.9, 1146795.845, 1916879.493, 41.92789092, -87.73600171, 20, 'HERMOSA', 31, 25, '(41.92789092, -87.73600171)') ,\n(609746, 'Kenwood Academy High School', 'HS', '5015 S Blackstone Ave', 'Chicago', 'IL', 60615, '(773) 535-1350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609746.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 63.0, nan, nan, 'Average', 48.0, 'Average', 56.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '88.40%', 5.9, '95.30%', '99.40%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 60.2, 75.0, 69.5, 31.0, -1.1, -0.5, 'Red', 'Yellow', 81.2, 49.0, 15.6, 15.1, 16.8, 16.9, 1.3, 19.1, 2.3, 26.8, 69.5, 79.7, 1852, 46, 70.3, 1186813.047, 1871954.1519999998, 41.80375479, -87.59038395, 39, 'KENWOOD', 4, 2, '(41.80375479, -87.59038395)') ,\n(609719, 'Lake View High School', 'HS', '4015 N Ashland Ave', 'Chicago', 'IL', 60613, '(773) 534-5440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609719.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 64.0, nan, nan, 'Average', 52.0, 'Average', 43.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Weak', 43.0, '87.70%', 8.5, '96.20%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 15.3, 15.8, 16.1, 16.7, 1.4, 17.8, 1.7, 36.6, 81.4, 62.8, 1500, 33, 87.3, 1164975.217, 1926813.581, 41.95478361, -87.66891643, 6, 'LAKE VIEW', 47, 19, '(41.95478361, -87.66891643)') ,\n(610368, 'Langston Hughes Elementary School', 'ES', '240 W 104th St', 'Chicago', 'IL', 60628, '(773) 535-5075', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610368.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Average', 54.0, 'Very Weak', 16.0, 'Weak', 30.0, 'Weak', 62.0, 'Average', 48.0, 'Weak', 46.0, 'Strong', 56.0, '90.00%', 27.5, '95.50%', '100.00%', 44.7, 67.0, 19.3, 16.8, 46.8, 54.5, 22.7, 18.7, 57.0, 43.7, 0.0, 9.5, 8.2, 6.1, 0.3, -0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 424, 48, nan, 1176345.568, 1836013.039, 41.70536999, -87.62985093, 49, 'ROSELAND', 34, 5, '(41.70536999, -87.62985093)') ,\n(610033, 'LaSalle Elementary Language Academy', 'ES', '1734 N Orleans St', 'Chicago', 'IL', 60614, '(773) 534-8470', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610033.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Strong', 79.0, 'Strong', 62.0, 'Average', 52.0, 'Weak', 39.0, 'Average', 43.0, 'Average', 53.0, 'Average', 48.0, '96.80%', 7.0, '97.60%', '100.00%', 86.0, 47.4, 81.4, 78.3, 67.7, 76.4, 79.2, 84.7, 73.2, 76.5, 61.0, 71.2, 55.5, 49.5, 0.3, -0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 576, 33, nan, 1173614.17, 1911976.635, 41.9138823, -87.63760107, 7, 'LINCOLN PARK', 43, 18, '(41.9138823, -87.63760107)') ,\n(610520, 'LaSalle II Magnet Elementary School', 'ES', '1148 N Honore St', 'Chicago', 'IL', 60622, '(773) 534-0490', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610520.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, 'Very Strong', 81.0, 'Weak', 36.0, 'Average', 52.0, 'Weak', 83.0, 'Strong', 79.0, 'Strong', 55.0, 'Average', 51.0, '95.10%', 5.6, '94.80%', '100.00%', nan, nan, 61.1, 69.9, 82.5, 80.2, 61.1, 76.4, 70.4, 85.5, nan, nan, 33.8, 38.0, 1.1, 2.0, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 609, 35, nan, 1163810.597, 1907900.166, 41.90290876, -87.67373263, 24, 'WEST TOWN', 1, 13, '(41.90290876, -87.67373263)') ,\n(609910, 'Laughlin Falconer Elementary School', 'ES', '3020 N Lamon Ave', 'Chicago', 'IL', 60641, '(773) 534-3560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609910.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 54.0, 'Weak', 24.0, 'Average', 48.0, 'Weak', 27.0, 'Weak', 22.0, 'Weak', 30.0, 'Average', 49.0, 'Average', 53.0, '96.10%', 5.7, '97.00%', '99.50%', 71.7, 42.9, 47.7, 28.5, 46.9, 63.4, 72.0, 33.3, 77.2, 59.1, nan, nan, 21.0, 14.2, 1.1, 0.8, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1520, 29, nan, 1143139.285, 1919727.764, 41.93577597, -87.74936705, 19, 'BELMONT CRAGIN', 31, 25, '(41.93577597, -87.74936705)') ,\n(610133, 'Laura S Ward Elementary School', 'ES', '410 N Monticello Ave', 'Chicago', 'IL', 60624, '(773) 534-6440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610133.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, nan, nan, nan, nan, '92.90%', 3.9, '97.50%', '100.00%', 72.6, 54.0, 35.2, 13.0, 43.5, 60.7, 41.7, 27.5, 44.0, 50.4, 14.3, 47.1, 14.2, 6.9, -3.5, -5.0, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 429, 34, nan, 1151927.188, 1902542.7869999998, 41.88845002, -87.71752405, 23, 'HUMBOLDT PARK', 27, 11, '(41.88845002, -87.71752405)') ,\n(610034, 'Lawndale Elementary Community Academy', 'ES', '3500 W Douglas Blvd', 'Chicago', 'IL', 60623, '(773) 534-1635', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610034.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, nan, nan, nan, nan, '91.00%', 18.8, '94.80%', '98.00%', 43.4, nan, 17.2, 10.0, 34.0, 44.0, 21.1, 15.9, 45.2, 47.0, 0.0, 2.0, 3.3, 4.0, -2.1, -2.2, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 508, 37, nan, 1153224.4209999999, 1893272.908, 41.86298683, -87.71300615, 29, 'NORTH LAWNDALE', 24, 10, '(41.86298683, -87.71300615)') ,\n(610024, 'Lazaro Cardenas Elementary School', 'ES', '2345 S Millard Ave', 'Chicago', 'IL', 60623, '(773) 534-1465', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610024.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', nan, nan, 'Average', 58.0, nan, nan, nan, nan, 'Weak', 62.0, 'Average', 58.0, 'Average', 52.0, 'Strong', 57.0, '96.30%', 3.2, '98.50%', '100.00%', 77.2, 59.6, 31.9, 26.5, 41.4, 57.7, nan, nan, nan, nan, nan, nan, 26.3, 15.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 654, 37, nan, 1152451.967, 1887990.6930000002, 41.8485071, -87.71598119, 30, 'SOUTH LAWNDALE', 22, 10, '(41.8485071, -87.71598119)') ,\n(609907, 'Leif Ericson Elementary Scholastic Academy', 'ES', '3600 W 5th Ave', 'Chicago', 'IL', 60624, '(773) 534-6660', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609907.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 33.0, nan, nan, 'Average', 49.0, 'Average', 52.0, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Weak', 45.0, '94.70%', 20.8, '95.10%', '100.00%', 76.1, 70.9, 41.3, 22.4, 43.1, 58.5, 43.8, 25.4, 60.9, 40.9, 10.4, 20.8, 13.3, 7.7, 2.0, 1.0, 'Green', 'Green', 51.1, 4.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 518, 34, nan, 1152417.4440000001, 1897922.54, 41.87576191, -87.71584573, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87576191, -87.71584573)') ,\n(610298, 'Lenart Elementary Regional Gifted Center', 'ES', '8101 S LaSalle St', 'Chicago', 'IL', 60620, '(773) 535-0040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610298.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 79.0, 'Average', 58.0, 'Average', 51.0, 'Strong', 67.0, 'Weak', 50.0, 'Strong', 63.0, 'Average', 52.0, 'Average', 52.0, '97.40%', 0.3, '97.80%', '100.00%', nan, nan, 93.9, 96.9, 69.4, 46.9, 93.5, 96.8, 57.6, 55.9, 86.7, 93.3, 88.0, 86.4, 1.0, 0.5, 'Green', 'Yellow', 100.0, 70.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 328, 45, nan, 1176725.617, 1851241.455, 41.74715024, -87.62800231, 44, 'CHATHAM', 21, 6, '(41.74715024, -87.62800231)') ,\n(610036, 'Leslie Lewis Elementary School', 'ES', '1431 N Leamington Ave', 'Chicago', 'IL', 60651, '(773) 534-3060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610036.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 21.0, 'Weak', 37.0, 'Weak', 36.0, 'Strong', 64.0, 'Weak', 30.0, 'Average', 48.0, 'Weak', 42.0, 'Weak', 43.0, '91.80%', 37.7, '95.30%', '98.60%', 67.8, 58.4, 20.3, 19.8, 51.3, 40.1, 15.8, 22.5, 53.5, 58.3, 2.3, 4.5, 4.0, 5.0, -0.5, -0.1, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 565, 36, nan, 1141810.2920000001, 1909168.612, 41.90682528, -87.75451319, 25, 'AUSTIN', 37, 25, '(41.90682528, -87.75451319)') ,\n(609738, 'Lincoln Park High School', 'HS', '2001 N Orchard St', 'Chicago', 'IL', 60614, '(773) 534-8130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609738.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 65.0, 'Strong', 61.0, 'Average', 44.0, 'Average', 49.0, 'Weak', 45.0, 'Average', 45.0, nan, nan, nan, nan, '84.60%', 9.2, '96.00%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 17.7, 18.0, 18.6, 19.0, 1.3, 21.6, 3.0, 53.6, 77.9, 75.5, 2342, 33, 74.4, 1171322.336, 1913569.392, 41.91830362, -87.64597389, 7, 'LINCOLN PARK', 43, 18, '(41.91830362, -87.64597389)') ,\n(609807, 'Lionel Hampton Fine & Performing Arts Elementary School', 'ES', '3434 W 77th St', 'Chicago', 'IL', 60652, '(773) 535-4030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609807.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 42.0, nan, nan, 'Weak', 34.0, 'Weak', 28.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Average', 51.0, '95.00%', 94.9, '96.60%', '100.00%', 59.6, 40.0, 35.3, 28.6, 57.1, 56.1, 37.6, 33.8, 52.8, 54.6, 9.0, 37.2, 9.4, 9.4, -1.0, 0.0, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 658, 44, nan, 1154672.983, 1853379.243, 41.75348432, -87.70875287, 70, 'ASHBURN', 18, 8, '(41.75348432, -87.70875287)') ,\n(609834, 'Little Village Elementary School', 'ES', '2620 S Lawndale Ave', 'Chicago', 'IL', 60623, '(773) 534-1880', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609834.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 45.0, 'Average', 43.0, 'Average', 52.0, 'Strong', 66.0, 'Weak', 45.0, 'Average', 43.0, 'Average', 47.0, 'Average', 50.0, '96.70%', 3.2, '96.70%', '93.90%', 69.0, 56.0, 26.0, 25.1, 49.8, 52.2, 42.1, 32.6, 63.2, 56.3, 17.6, 19.4, 11.3, 11.4, 0.6, 1.5, 'Green', 'Green', 30.9, 66.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 842, 37, nan, 1152090.217, 1886231.1730000002, 41.84368588, -87.71735516, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84368588, -87.71735516)') ,\n(610325, 'Logandale Middle School', 'MS', '3212 W George St', 'Chicago', 'IL', 60618, '(773) 534-5350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610325.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 56.0, nan, nan, 'Weak', 32.0, 'Weak', 28.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 51.0, '95.10%', 5.1, '95.60%', '100.00%', nan, nan, nan, nan, nan, nan, 39.1, 31.0, 56.9, 55.1, 20.0, 22.6, 11.6, 7.8, -0.6, -0.3, 'Yellow', 'Yellow', 34.8, 37.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 941, 29, nan, 1154387.612, 1919159.019, 41.933997600000005, -87.70804359, 21, 'AVONDALE', 35, 14, '(41.9339976, -87.70804359)') ,\n(609809, 'Lorenz Brentano Math & Science Academy Elementary School', 'ES', '2723 N Fairfield Ave', 'Chicago', 'IL', 60647, '(773) 534-4100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609809.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 57.0, 'Average', 47.0, 'Average', 43.0, 'Average', 46.0, 'Weak', 30.0, 'Weak', 32.0, nan, nan, nan, nan, '94.70%', 9.2, '97.70%', '100.00%', 48.5, 42.6, 44.2, 30.8, 50.9, 56.1, 54.4, 30.9, 48.9, 46.6, 20.8, 28.3, 18.4, 8.2, -1.6, -1.0, 'Red', 'Red', 48.1, 3.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 455, 31, nan, 1157602.725, 1918079.5369999998, 41.93097047, -87.69625758, 22, 'LOGAN SQUARE', 35, 14, '(41.93097047, -87.69625758)') ,\n(609773, 'Louis A Agassiz Elementary School', 'ES', '2851 N Seminary Ave', 'Chicago', 'IL', 60657, '(773) 534-5725', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609773.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 53.0, nan, nan, 'Weak', 23.0, 'Weak', 21.0, 'Weak', nan, nan, nan, 'Strong', 56.0, 'Average', 47.0, '95.20%', 32.6, '96.80%', '98.10%', 68.5, 39.0, 44.1, 49.5, 44.9, 49.5, 43.6, 50.0, 43.6, 43.6, 30.0, 53.3, 27.5, 27.5, 0.2, -0.9, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 432, 33, nan, 1168495.925, 1919310.804, 41.93412004, -87.65619167, 6, 'LAKE VIEW', 44, 19, '(41.93412004, -87.65619167)') ,\n(609774, 'Louisa May Alcott Elementary School', 'ES', '2625 N Orchard St', 'Chicago', 'IL', 60614, '(773) 534-5460', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609774.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', nan, nan, 'Strong', 66.0, nan, nan, nan, nan, 'Weak', 56.0, 'Average', 57.0, 'Strong', 60.0, 'Weak', 41.0, '96.60%', 3.7, '97.10%', '93.20%', 85.1, 75.1, 72.6, 76.9, 56.4, 55.2, 73.1, 70.7, 69.2, 59.8, 43.5, 47.8, 52.6, 46.1, 0.4, 0.3, 'Yellow', 'Yellow', 50.0, 73.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 481, 35, nan, 1171189.134, 1917774.756, 41.92984626, -87.64633949, 7, 'LINCOLN PARK', 43, 19, '(41.92984626, -87.64633949)') ,\n(610156, 'Louis Armstrong Math & Science Elementary School', 'ES', '5345 W Congress Pkwy', 'Chicago', 'IL', 60644, '(773) 534-6365', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610156.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, nan, nan, 'Average', 49.0, 'Strong', 62.0, 'Weak', nan, nan, nan, 'Strong', 59.0, 'Average', 53.0, '92.60%', 83.4, '97.90%', '100.00%', nan, nan, 20.0, 16.5, 36.4, 31.6, 41.4, 34.5, 57.1, 60.7, nan, nan, 1.7, 7.8, -0.8, 0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 93, 36, nan, 1140614.251, 1897119.257, 41.87378243, -87.75920297, 25, 'AUSTIN', 29, 15, '(41.87378243, -87.75920297)') ,\n(610094, 'Louis Nettelhorst Elementary School', 'ES', '3252 N Broadway St', 'Chicago', 'IL', 60657, '(773) 534-5810', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610094.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 58.0, nan, nan, 'Weak', 34.0, 'Weak', 30.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.50%', 5.3, '95.90%', '97.90%', 85.4, 52.2, 67.6, 71.2, 61.6, 61.4, 56.8, 66.7, 45.1, 51.0, 29.6, 29.6, 38.2, 42.4, 0.2, 0.0, 'Yellow', 'Yellow', 53.6, 33.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 729, 33, nan, 1171628.585, 1922072.58, 41.94162999, -87.64459776, 6, 'LAKE VIEW', 44, 19, '(41.94162999, -87.64459776)') ,\n(610117, 'Louis Pasteur Elementary School', 'ES', '5825 S Kostner Ave', 'Chicago', 'IL', 60629, '(773) 535-2270', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610117.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 50.0, nan, nan, 'Average', 42.0, 'Weak', 38.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.00%', 5.0, '95.80%', '97.30%', 45.0, 19.2, 39.1, 43.4, 53.6, 57.4, 39.8, 50.8, 46.0, 51.5, 17.0, 28.1, 15.9, 13.7, -3.1, -0.6, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1173, 44, nan, 1148072.928, 1865505.479, 41.78688968, -87.73262962, 62, 'WEST ELSDON', 13, 8, '(41.78688968, -87.73262962)') ,\n(610237, 'Ludwig Van Beethoven Elementary School', 'ES', '25 W 47th St', 'Chicago', 'IL', 60609, '(773) 535-1480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610237.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 40.0, nan, nan, 'Average', 58.0, 'Average', 59.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Average', 52.0, '93.70%', 11.4, '94.70%', '98.70%', 58.3, 72.6, 18.7, 18.2, 43.1, 44.2, 33.8, 12.5, 54.5, 39.2, nan, nan, 16.5, 4.0, 0.7, -0.8, 'Green', 'Yellow', 39.1, 70.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 411, 42, nan, 1176782.61, 1873820.331, 41.80910779, -87.62711401, 38, 'GRAND BOULEVARD', 3, 2, '(41.80910779, -87.62711401)') ,\n(610108, 'Luke O Toole Elementary School', 'ES', '6550 S Seeley Ave', 'Chicago', 'IL', 60636, '(773) 535-9040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610108.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 5.0, nan, nan, 'Weak', 33.0, 'Average', 40.0, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Average', 47.0, '89.20%', 26.7, '94.60%', '100.00%', 63.5, nan, 21.2, 17.1, 39.4, 49.6, 26.0, 21.1, 55.1, 50.3, 2.9, 13.4, 7.8, 4.6, -0.3, -0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 511, 43, nan, 1163788.5520000001, 1860992.429, 41.77418952, -87.67513395, 67, 'WEST ENGLEWOOD', 15, 7, '(41.77418952, -87.67513395)') ,\n(609818, 'Luther Burbank Elementary School', 'ES', '2035 N Mobile Ave', 'Chicago', 'IL', 60639, '(773) 534-3000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609818.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 37.0, nan, nan, 'Average', 42.0, 'Weak', 34.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 49.0, '95.20%', 9.8, '95.50%', '100.00%', 60.8, 45.2, 42.8, 34.9, 61.2, 63.8, 41.0, 34.4, 55.0, 54.5, 13.1, 17.2, 20.6, 11.3, -0.2, 0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1139, 29, nan, 1134122.521, 1913042.0459999999, 41.91759334, -87.78266267, 19, 'BELMONT CRAGIN', 29, 25, '(41.91759334, -87.78266267)') ,\n(609817, 'Lyman A Budlong Elementary School', 'ES', '2701 W Foster Ave', 'Chicago', 'IL', 60625, '(773) 534-2591', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609817.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, 'Average', 56.0, 'Weak', 33.0, 'Weak', 38.0, 'Weak', 44.0, 'Average', 41.0, 'Average', 50.0, 'Average', 49.0, '95.40%', 3.9, '95.70%', '98.70%', 77.1, 54.4, 44.9, 53.5, 48.1, 57.9, 36.3, 44.0, 44.7, 36.4, 9.1, 23.6, 26.6, 23.7, -1.7, -0.4, 'Red', 'Yellow', 48.1, 16.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 864, 31, nan, 1157387.173, 1934388.0869999998, 41.97572652, -87.69660443, 4, 'LINCOLN SQUARE', 40, 20, '(41.97572652, -87.69660443)') ,\n(610205, 'Lyman Trumbull Elementary School', 'ES', '5200 N Ashland Ave', 'Chicago', 'IL', 60640, '(773) 534-2430', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610205.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 56.0, nan, nan, 'Average', 47.0, 'Weak', 25.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Strong', 55.0, '94.30%', 17.8, '95.20%', '97.70%', 55.1, 25.7, 21.6, 29.0, 37.5, 38.0, 39.0, 42.2, 30.4, 31.8, 21.1, 36.8, 14.2, 13.5, -1.8, -1.5, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 438, 32, nan, 1164684.835, 1934622.0119999999, 41.97621644, -87.66976159999999, 77, 'EDGEWATER', 40, 20, '(41.97621644, -87.6697616)') ,\n(610369, 'Mahalia Jackson Elementary School', 'ES', '917 W 88th St', 'Chicago', 'IL', 60620, '(773) 535-3341', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610369.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 38.0, nan, nan, 'Average', 54.0, 'Strong', 66.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Average', 51.0, '92.50%', 35.7, '94.60%', '96.60%', 57.8, 44.4, 14.8, 24.4, 47.8, 44.6, 24.4, 31.1, 54.2, 48.1, 6.7, 6.9, 7.6, 9.2, -0.3, 0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 323, 49, nan, 1171645.6809999999, 1846444.0159999998, 41.73409818, -87.64675679, 71, 'AUBURN GRESHAM', 21, 22, '(41.73409818, -87.64675679)') ,\n(610197, 'Mancel Talcott Elementary School', 'ES', '1840 W Ohio St', 'Chicago', 'IL', 60622, '(773) 534-7130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610197.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 63.0, 'Average', 58.0, 'Average', 59.0, 'Strong', 71.0, 'Weak', 53.0, 'Average', 55.0, 'Average', 51.0, 'Average', 49.0, '95.80%', 3.2, '96.90%', '100.00%', 68.2, nan, 43.2, 27.0, 49.6, 55.0, 70.3, 53.5, 68.6, 62.7, 17.0, 34.0, 25.6, 20.8, 0.2, 0.8, 'Yellow', 'Yellow', 37.3, 57.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 580, 35, nan, 1163847.473, 1904082.62, 41.89243235, -87.67370499, 24, 'WEST TOWN', 1, 13, '(41.89243235, -87.67370499)') ,\n(609722, 'Manley Career Academy High School', 'HS', '2935 W Polk St', 'Chicago', 'IL', 60612, '(773) 534-6900', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609722.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 41.0, 'Weak', 39.0, 'Average', 43.0, 'Weak', 31.0, 'Weak', 19.0, 'Weak', 32.0, nan, nan, nan, nan, '66.80%', 19.7, '95.40%', '98.40%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.2, 11.9, 13.3, 13.0, 0.8, 13.8, 0.5, 6.7, 49.0, 51.9, 599, 37, 59.3, 1156776.858, 1896186.78, 41.87091163, -87.69988652, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87091163, -87.69988652)') ,\n(609872, 'Manuel Perez Elementary School', 'ES', '1241 W 19th St', 'Chicago', 'IL', 60608, '(773) 534-7650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609872.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'Yes', 'Average', 45.0, nan, nan, 'Average', 42.0, 'Average', 45.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Average', 50.0, '95.90%', 7.3, '96.60%', '100.00%', 67.7, nan, 37.9, 33.6, 61.1, 62.3, 45.9, 26.7, 71.8, 51.8, 14.0, 14.0, 14.8, 8.7, 0.0, 1.1, 'Yellow', 'Green', 35.6, 18.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 426, 39, nan, 1168231.034, 1890840.4219999998, 41.85600113, -87.6579887, 31, 'LOWER WEST SIDE', 25, 12, '(41.85600113, -87.6579887)') ,\n(610128, 'Marcus Moziah Garvey Elementary School', 'ES', '10309 S Morgan St', 'Chicago', 'IL', 60643, '(773) 535-2763', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610128.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 50.0, nan, nan, 'Average', 50.0, 'Average', 48.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Average', 51.0, '94.00%', 20.5, '96.00%', '100.00%', 69.3, 62.7, 40.4, 47.4, 60.2, 59.8, 55.4, 52.9, 73.1, 63.9, 10.3, 48.7, 22.4, 18.5, 0.1, 0.9, 'Yellow', 'Green', 30.8, 69.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 356, 49, nan, 1171511.646, 1836420.6709999999, 41.7065956, -87.64754034, 73, 'WASHINGTON HEIGHTS', 34, 22, '(41.7065956, -87.64754034)') ,\n(610544, 'Mariano Azuela Elementary School', 'ES', '4707 W Marquette Rd', 'Chicago', 'IL', 60629, '(773) 535-7395', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610544.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 54.0, 'Very Strong', 96.0, 'Strong', 61.0, 'Very Strong', 81.0, 'Weak', 73.0, 'Very Strong', 84.0, 'Average', 47.0, 'Strong', 58.0, '95.50%', 19.7, '0.00%', '98.70%', 69.5, 42.9, 28.7, 26.1, 48.2, 32.1, 46.9, 34.9, 55.2, 57.4, nan, nan, 13.0, 11.9, -1.3, 0.2, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 873, 44, nan, 1146123.64, 1859720.388, 41.77105158, -87.73992325, 65, 'WEST LAWN', 13, 8, '(41.77105158, -87.73992325)') ,\n(610017, 'Maria Saucedo Elementary Scholastic Academy', 'ES', '2850 W 24th Blvd', 'Chicago', 'IL', 60623, '(773) 534-1770', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610017.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 60.0, nan, nan, 'Average', 55.0, 'Strong', 64.0, 'Weak', nan, nan, nan, 'Weak', 43.0, 'Weak', 42.0, '96.80%', 19.4, '96.90%', '98.90%', 72.3, 35.3, 38.6, 35.8, 54.9, 57.5, 51.8, 39.8, 56.3, 52.8, 15.3, 27.0, 24.3, 16.0, 2.0, 1.7, 'Green', 'Green', 10.8, 91.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1306, 39, nan, 1157546.774, 1887831.632, 41.84796859, -87.69728712, 30, 'SOUTH LAWNDALE', 12, 10, '(41.84796859, -87.69728712)') ,\n(609756, 'Marie Sklodowska Curie Metropolitan High School', 'HS', '4959 S Archer Ave', 'Chicago', 'IL', 60632, '(773) 535-2100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609756.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 43.0, 'Weak', 35.0, 'Weak', 37.0, 'Average', 45.0, 'Weak', 33.0, 'Weak', 39.0, nan, nan, nan, nan, '85.00%', 9.1, '95.00%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.2, 14.2, 15.6, 15.1, 0.9, 16.9, 1.3, 22.9, 76.1, 61.3, 3320, 37, 61.0, 1150928.132, 1871413.2030000002, 41.80304615, -87.72200673, 57, 'ARCHER HEIGHTS', 14, 8, '(41.80304615, -87.72200673)') ,\n(610502, 'Marine Military Math and Science Academy', 'HS', '145 S Campbell Ave', 'Chicago', 'IL', 60612, '(773) 534-7818', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610502.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 41.0, 'Average', 49.0, 'Weak', 38.0, 'Weak', 34.0, 'Weak', 53.0, 'Weak', 35.0, 'Strong', 54.0, 'Average', 49.0, '91.30%', 4.5, '95.30%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.0, 15.0, 15.9, 15.3, 1.3, 17.7, 1.8, 37.7, nan, nan, 366, 38, 86.3, 1159815.389, 1899102.4919999999, 41.87885054, -87.68865058, 28, 'NEAR WEST SIDE', 2, 11, '(41.87885054, -87.68865058)') ,\n(610081, 'Mark Sheridan Elementary Math & Science Academy', 'ES', '533 W 27th St', 'Chicago', 'IL', 60616, '(773) 534-9120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610081.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 74.0, 'Average', 59.0, 'Average', 53.0, 'Strong', 63.0, 'Weak', 59.0, 'Strong', 63.0, 'Strong', 55.0, 'Average', 51.0, '96.80%', 1.3, '97.00%', '95.90%', 85.5, nan, 62.7, 74.0, 72.3, 77.4, 59.4, 62.4, 67.8, 61.3, 35.9, 42.2, 47.2, 38.3, 1.5, 1.5, 'Green', 'Green', 50.0, 65.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 535, 40, nan, 1172807.818, 1886639.3080000002, 41.84437293, -87.64131399, 60, 'BRIDGEPORT', 11, 9, '(41.84437293, -87.64131399)') ,\n(610177, 'Mark Skinner Elementary School', 'ES', '1260 W Adams St', 'Chicago', 'IL', 60607, '(773) 534-7790', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610177.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, nan, nan, 'Average', 48.0, 'Average', 53.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '97.10%', 0.0, '94.90%', '96.70%', nan, nan, 92.5, 96.8, 61.5, 59.1, 89.4, 97.8, 56.5, 56.8, 81.6, 91.8, 77.2, 73.0, 1.0, 0.6, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 784, 38, nan, 1167788.281, 1899265.832, 41.87913069, -87.65937102, 28, 'NEAR WEST SIDE', 27, 12, '(41.87913069, -87.65937102)') ,\n(610206, 'Mark Twain Elementary School', 'ES', '5134 S Lotus Ave', 'Chicago', 'IL', 60638, '(773) 535-2290', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610206.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, 'Strong', 76.0, 'Average', 46.0, 'Average', 44.0, 'Weak', 84.0, 'Strong', 71.0, 'Strong', 60.0, 'Strong', 60.0, '96.30%', 5.3, '97.80%', '98.30%', 84.6, 61.1, 44.4, 48.5, 53.2, 63.3, 49.1, 58.1, 62.2, 55.8, 32.1, 43.8, 32.3, 19.8, 0.2, -0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1161, 44, nan, 1140861.0829999999, 1869888.283, 41.79905206, -87.75896524, 56, 'GARFIELD RIDGE', 23, 8, '(41.79905206, -87.75896524)') ,\n(610053, 'Marquette Elementary School', 'ES', '6550 S Richmond St', 'Chicago', 'IL', 60629, '(773) 535-9260', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610053.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 15.0, nan, nan, 'Very Weak', 17.0, 'Very Weak', 17.0, 'Weak', nan, nan, nan, 'Weak', 43.0, 'Weak', 46.0, '93.10%', 64.6, '95.30%', '99.30%', 54.4, 21.8, 22.6, 16.2, 42.0, 40.0, 14.2, 16.2, 31.2, 40.2, 4.2, 13.2, 6.8, 4.6, -1.4, -0.6, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1390, 44, nan, 1157806.476, 1860844.388, 41.77390682, -87.6970673, 66, 'CHICAGO LAWN', 15, 8, '(41.77390682, -87.6970673)') ,\n(610152, 'Martha Ruggles Elementary School', 'ES', '7831 S Prairie Ave', 'Chicago', 'IL', 60619, '(773) 535-3085', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610152.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 43.0, nan, nan, 'Weak', 37.0, 'Average', 41.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Weak', 46.0, '93.90%', 30.9, '96.70%', '98.00%', nan, 49.3, 23.6, 21.9, 55.0, 57.6, 26.8, 24.4, 51.3, 56.5, 8.8, 26.5, 8.7, 5.5, 0.2, 1.0, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 417, 46, nan, 1179432.46, 1853011.9219999998, 41.75194729, -87.61802983, 69, 'GREATER GRAND CROSSING', 6, 6, '(41.75194729, -87.61802983)') ,\n(610154, 'Martin A Ryerson Elementary School', 'ES', '646 N Lawndale Ave', 'Chicago', 'IL', 60624, '(773) 534-6700', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610154.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 40.0, 'Weak', 32.0, 'Average', 48.0, 'Average', 43.0, 'Weak', 41.0, 'Weak', 37.0, 'Weak', 45.0, 'Average', 48.0, '95.50%', 95.1, '96.80%', '100.00%', 63.6, 48.6, 33.3, 19.2, 48.9, 72.9, 59.2, 26.4, 93.3, 67.5, 13.6, 11.4, 23.1, 17.1, 2.8, 0.2, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 445, 34, nan, 1151558.259, 1904275.283, 41.89321142, -87.71883334, 23, 'HUMBOLDT PARK', 27, 11, '(41.89321142, -87.71883334)') ,\n(610539, 'Marvin Camras Elementary School', 'ES', '3000 N Mango Ave', 'Chicago', 'IL', 60634, '(773) 534-2960', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610539.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 54.0, 'Average', 58.0, 'Weak', 37.0, 'Average', 41.0, 'Weak', 83.0, 'Very Strong', 88.0, 'Average', 51.0, 'Strong', 55.0, '95.10%', 4.3, '0.00%', '97.70%', 47.5, 39.9, 30.8, 27.8, 47.8, 38.6, 41.1, 38.5, 49.2, 55.4, 15.3, 30.5, 11.2, 12.0, -1.8, -0.3, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 826, 30, nan, 1137482.2959999999, 1919394.9, 41.93496641, -87.77016525, 19, 'BELMONT CRAGIN', 30, 25, '(41.93496641, -87.77016525)') ,\n(610355, 'Mary E Courtenay Elementary Language Arts Center', 'ES', '1726 W Berteau Ave', 'Chicago', 'IL', 60613, '(773) 534-5790', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610355.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Very Strong', 99.0, nan, nan, 'Very Strong', 95.0, 'Very Strong', 80.0, 'Weak', nan, nan, nan, 'Strong', 59.0, 'Average', 49.0, '94.90%', 10.8, '94.20%', '100.00%', 90.5, 92.9, 51.0, 38.5, 62.0, 59.2, 47.7, 57.8, 45.5, 37.8, 16.7, 55.6, 21.2, 28.6, -1.2, -0.8, 'Red', 'Yellow', 19.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 257, 33, nan, 1164156.393, 1927985.5380000002, 41.9580169, -87.6718933, 6, 'LAKE VIEW', 47, 19, '(41.9580169, -87.6718933)') ,\n(610312, 'Mary E McDowell Elementary School', 'ES', '1419 E 89th St', 'Chicago', 'IL', 60619, '(773) 535-6404', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610312.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', nan, nan, 'Average', 49.0, nan, nan, nan, nan, 'Weak', 74.0, 'Average', 53.0, nan, nan, nan, nan, '95.50%', 3.1, '96.10%', '100.00%', 79.3, 53.8, 39.0, 54.2, 50.8, 64.4, nan, nan, nan, nan, nan, nan, 21.0, 16.1, 0.0, -0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 188, 47, nan, 1187249.1409999998, 1846211.629, 41.733104700000005, -87.58960096, 48, 'CALUMET HEIGHTS', 8, 4, '(41.7331047, -87.58960096)') ,\n(610127, 'Mary Gage Peterson Elementary School', 'ES', '5510 N Christiana Ave', 'Chicago', 'IL', 60625, '(773) 534-5070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610127.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 62.0, nan, nan, 'Average', 49.0, 'Average', 58.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Average', 48.0, '95.50%', 10.3, '96.70%', '100.00%', 72.7, 37.7, 53.1, 47.1, 51.6, 52.1, 59.7, 52.2, 49.2, 51.9, 34.0, 44.7, 35.5, 27.5, -0.3, 0.0, 'Yellow', 'Yellow', 33.3, 83.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 860, 31, nan, 1153016.828, 1936489.312, 41.98158041, -87.71261988, 13, 'NORTH PARK', 40, 17, '(41.98158041, -87.71261988)') ,\n(610046, 'Mary Lyon Elementary School', 'ES', '2941 N McVicker Ave', 'Chicago', 'IL', 60634, '(773) 534-3120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610046.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, nan, nan, 'Average', 42.0, 'Weak', 38.0, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Average', 51.0, '95.90%', 3.7, '97.00%', '99.30%', 72.0, nan, 42.9, 51.1, 68.1, 70.2, 61.6, 59.0, 62.9, 55.6, 37.6, 36.4, 29.5, 23.7, -0.3, 0.4, 'Yellow', 'Yellow', 24.3, 100.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1423, 30, nan, 1135587.211, 1919095.574, 41.93417898, -87.77713699, 19, 'BELMONT CRAGIN', 29, 25, '(41.93417898, -87.77713699)') ,\n(609888, 'Mary Mapes Dodge Elementary Renaissance Academy', 'ES', '2651 W Washington Blvd', 'Chicago', 'IL', 60612, '(773) 534-6640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609888.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 57.0, 'Strong', 61.0, 'Strong', 68.0, 'Strong', 73.0, 'Weak', 77.0, 'Very Strong', 83.0, 'Strong', 56.0, 'Strong', 57.0, '94.40%', 12.7, '97.10%', '100.00%', nan, nan, 51.4, 49.7, 53.1, 55.2, 53.2, 54.0, 59.0, 51.1, 12.0, 22.0, 29.4, 16.0, 0.4, -1.6, 'Yellow', 'Red', 43.1, 19.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 434, 38, nan, 1158615.705, 1900540.3059999999, 41.88282068, -87.6930162, 27, 'EAST GARFIELD PARK', 2, 13, '(41.88282068, -87.6930162)') ,\n(610365, 'Mary McLeod Bethune Elementary School', 'ES', '3030 W Arthington St', 'Chicago', 'IL', 60612, '(773) 534-6890', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610365.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 53.0, 'Average', 47.0, 'Average', 53.0, 'Weak', 39.0, 'Weak', 31.0, 'Average', 46.0, 'Strong', 56.0, 'Strong', 58.0, '93.30%', 25.8, '97.60%', '98.00%', 50.0, nan, 13.8, 13.8, 44.0, 49.1, 22.1, 24.1, 57.0, 46.0, 6.7, 16.7, 6.1, 4.4, 1.3, -0.4, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 362, 37, nan, 1156159.096, 1895921.1430000002, 41.87019518, -87.70216173, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87019518, -87.70216173)') ,\n(610535, 'Mason High School', 'HS', '4217 W 18th St', 'Chicago', 'IL', 60623, '(773) 534-1530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610535.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', nan, 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 50.0, nan, nan, 'Average', 59.0, 'Average', 59.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 47.0, '90.40%', 44.3, '0.00%', '72.70%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.6, 13.5, nan, 13.9, 1.3, nan, nan, nan, nan, nan, 48, 36, 40.9, 1148477.255, 1891003.4409999999, 41.85685194, -87.73049117, 29, 'NORTH LAWNDALE', 24, 10, '(41.85685194, -87.73049117)') ,\n(610240, 'Matthew A Henson Elementary School', 'ES', '1326 S Avers Ave', 'Chicago', 'IL', 60623, '(773) 534-1804', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610240.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 37.0, nan, nan, 'Weak', 34.0, 'Weak', 22.0, 'Weak', nan, nan, nan, 'Strong', 55.0, 'Strong', 63.0, '90.90%', 13.2, '95.70%', '97.90%', 70.1, 46.8, 19.0, 8.6, 48.6, 52.1, 15.7, 17.1, 30.3, 34.7, 3.6, 32.1, 3.5, 1.7, -0.8, -1.9, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 271, 36, nan, 1150885.71, 1893465.136, 41.86356038, -87.72158636, 29, 'NORTH LAWNDALE', 24, 10, '(41.86356038, -87.72158636)') ,\n(609935, 'Matthew Gallistel Elementary Language Academy', 'ES', '10347 S Ewing Ave', 'Chicago', 'IL', 60617, '(773) 535-6540', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609935.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 49.0, 'Average', 45.0, 'Weak', 37.0, 'Weak', 32.0, 'Weak', 49.0, 'Average', 40.0, 'Average', 50.0, 'Weak', 44.0, '95.10%', 5.9, '96.30%', '100.00%', 66.7, 53.6, 40.1, 23.7, 48.7, 62.2, 46.7, 34.0, 62.8, 58.6, 17.4, 27.7, 17.8, 12.9, -0.6, 0.9, 'Red', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1408, 47, nan, 1202203.353, 1836767.747, 41.70682232, -87.53513861, 52, 'EAST SIDE', 10, 4, '(41.70682232, -87.53513861)') ,\n(610362, 'Medgar Evers Elementary School', 'ES', '9811 S Lowe Ave', 'Chicago', 'IL', 60628, '(773) 535-2565', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610362.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Weak', 27.0, nan, nan, 'Weak', 35.0, 'Average', 49.0, 'Weak', nan, nan, nan, 'Strong', 56.0, 'Strong', 55.0, '94.90%', 18.1, '96.10%', '100.00%', 66.4, 46.7, 48.3, 39.7, 59.1, 71.9, 67.2, 43.9, 54.1, 51.2, 17.6, 21.6, 17.3, 14.8, 1.0, 0.4, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 367, 49, nan, 1173743.636, 1839743.2980000002, 41.7156643, -87.63926881, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.7156643, -87.63926881)') ,\n(609928, 'Melville W Fuller Elementary School', 'ES', '4214 S Saint Lawrence Ave', 'Chicago', 'IL', 60653, '(773) 535-1687', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609928.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 22.0, nan, nan, 'Weak', 32.0, 'Weak', 33.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Strong', 55.0, '91.80%', 40.2, '93.90%', '100.00%', 40.9, 21.4, 10.4, 2.4, 50.0, 40.0, 6.8, 10.9, 22.4, 34.8, 6.3, 12.5, 3.6, 1.4, -1.3, -1.3, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 241, 40, nan, 1180924.721, 1877097.4640000002, 41.8180061, -87.61182077, 38, 'GRAND BOULEVARD', 4, 2, '(41.8180061, -87.61182077)') ,\n(610055, 'Michael Faraday Elementary School', 'ES', '3250 W Monroe St', 'Chicago', 'IL', 60624, '(773) 534-6670', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610055.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, nan, nan, 'Strong', 65.0, 'Strong', 66.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 50.0, '93.30%', 39.4, '94.60%', '100.00%', 56.4, 34.0, 34.4, 17.2, 53.2, 73.3, 59.8, 22.9, 80.6, 63.5, 17.1, 17.1, 17.9, 7.0, 1.7, -1.1, 'Green', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 194, 38, nan, 1154521.277, 1899387.886, 41.87974117, -87.70808199, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87974117, -87.70808199)') ,\n(609832, 'Michael M Byrne Elementary School', 'ES', '5329 S Oak Park Ave', 'Chicago', 'IL', 60638, '(773) 535-2170', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609832.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 63.0, 'Strong', 61.0, 'Weak', 35.0, 'Weak', 38.0, 'Weak', 59.0, 'Average', 57.0, 'Average', 49.0, 'Weak', 45.0, '94.60%', 51.8, '95.80%', '94.00%', 74.3, 69.6, 50.2, 42.9, 43.7, 67.6, 44.9, 48.1, 45.8, 36.5, 21.5, 34.2, 23.1, 24.8, -1.2, -1.1, 'Red', 'Red', 19.3, 56.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 705, 44, nan, 1131997.021, 1868365.169, 41.79503029, -87.79150778, 56, 'GARFIELD RIDGE', 23, 8, '(41.79503029, -87.79150778)') ,\n(610244, 'Michele Clark Academic Prep Magnet High School', 'HS', '5101 W Harrison St', 'Chicago', 'IL', 60644, '(773) 534-6250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610244.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 48.0, 'Weak', 36.0, '79.10%', 24.4, '95.40%', '100.00%', nan, nan, nan, nan, nan, nan, 19.4, 22.8, 36.5, 52.7, 3.6, 10.7, 1.7, 2.6, -1.2, -0.7, 'Red', 'Yellow', nan, nan, 14.0, 13.3, 14.4, 14.5, 0.5, 15.7, 1.3, 13.0, 87.4, 60.4, 726, 36, 78.2, 1142209.317, 1896792.791, 41.87285714, -87.75335467, 25, 'AUSTIN', 24, 15, '(41.87285714, -87.75335467)') ,\n(610208, 'Mildred I Lavizzo Elementary School', 'ES', '138 W 109th St', 'Chicago', 'IL', 60628, '(773) 535-5300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610208.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 15.0, 'Weak', 35.0, 'Average', 41.0, 'Average', 46.0, 'Weak', 39.0, 'Weak', 30.0, 'Average', 48.0, 'Average', 51.0, '91.50%', 39.1, '95.00%', '95.70%', 49.6, 33.3, 5.7, 15.6, 35.7, 39.8, 24.3, 30.1, 65.6, 68.9, 4.3, 17.0, 6.1, 4.6, 0.6, 0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 394, 48, nan, 1177114.736, 1832710.465, 41.69628998, -87.62713349, 49, 'ROSELAND', 34, 5, '(41.69628998, -87.62713349)') ,\n(609830, 'Milton Brunson Math & Science Specialty Elementary School', 'ES', '932 N Central Ave', 'Chicago', 'IL', 60651, '(773) 534-6025', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609830.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 30.0, nan, nan, 'Weak', 30.0, 'Average', 45.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Strong', 55.0, '91.30%', 16.6, '95.00%', '100.00%', 64.4, 43.9, 21.1, 18.7, 44.4, 36.4, 22.6, 19.0, 33.6, 45.2, 3.4, 8.6, 8.8, 5.4, -0.3, -0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 658, 36, nan, 1138802.71, 1905814.957, 41.89767761, -87.76564297, 25, 'AUSTIN', 29, 15, '(41.89767761, -87.76564297)') ,\n(610011, 'Minnie Mars Jamieson Elementary School', 'ES', '5650 N Mozart St', 'Chicago', 'IL', 60659, '(773) 534-2395', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610011.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, 'Average', 50.0, 'Strong', 62.0, 'Strong', 66.0, 'Weak', 55.0, 'Average', 41.0, 'Average', 52.0, 'Average', 53.0, '95.70%', 8.1, '96.20%', '100.00%', 79.1, nan, 63.8, 58.0, 70.1, 72.1, 57.6, 67.2, 66.1, 62.7, 30.5, 35.4, 37.5, 23.7, 0.3, 0.6, 'Yellow', 'Yellow', 38.1, 42.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 811, 32, nan, 1156286.231, 1937618.645, 41.98461372, -87.70056525, 2, 'WEST RIDGE', 40, 20, '(41.98461372, -87.70056525)') ,\n(610018, 'Miriam G Canter Middle School', 'MS', '4959 S Blackstone Ave', 'Chicago', 'IL', 60615, '(773) 535-1410', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610018.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 57.0, 'Average', 55.0, 'Weak', 31.0, 'Weak', 37.0, 'Weak', 61.0, 'Strong', 62.0, nan, nan, nan, nan, '91.40%', 3.4, '95.70%', '100.00%', nan, nan, nan, nan, nan, nan, 33.3, 39.6, 53.9, 46.5, 17.6, 31.5, 21.6, 7.8, -0.2, -1.2, 'Yellow', 'Red', 24.1, 76.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 222, 46, nan, 1186808.1909999999, 1872169.101, 41.80434474, -87.59039494, 39, 'KENWOOD', 4, 2, '(41.80434474, -87.59039494)') ,\n(609725, 'Morgan Park High School', 'HS', '1744 W Pryor Ave', 'Chicago', 'IL', 60643, '(773) 535-2550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609725.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 40.0, nan, nan, 'Weak', 25.0, 'Weak', 38.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '84.20%', 11.1, '94.60%', '99.50%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 60.0, 80.0, 52.0, 38.2, -1.8, -1.0, 'Red', 'Yellow', 98.5, 64.1, 14.7, 14.6, 15.6, 15.9, 1.2, 17.8, 2.2, 33.6, 80.0, 78.5, 1584, 49, 73.3, 1166609.57, 1831085.382, 41.69206041, -87.66564309, 75, 'MORGAN PARK', 19, 22, '(41.69206041, -87.66564309)') ,\n(610257, 'Morton School of Excellence', 'ES', '431 N Troy St', 'Chicago', 'IL', 60612, '(773) 534-6791', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610257.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 1', 'No', 'Average', 53.0, 'Average', 49.0, 'Strong', 66.0, 'Strong', 63.0, 'Weak', 86.0, 'Very Strong', 95.0, 'Average', 52.0, 'Strong', 59.0, '93.60%', 73.9, '97.10%', '100.00%', nan, nan, 43.2, 36.8, 57.9, 62.2, 38.6, 36.0, 65.3, 49.0, 7.1, 7.1, 16.3, 8.4, 2.8, 0.5, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 349, 34, nan, 1155338.246, 1902872.4030000002, 41.88928666, -87.7049885, 23, 'HUMBOLDT PARK', 27, 13, '(41.88928666, -87.7049885)') ,\n(610075, 'Moses Montefiore Special Elementary School', 'ES', '1310 S Ashland Ave', 'Chicago', 'IL', 60608, '(773) 534-7825', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610075.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', nan, 'Track_E', 'Not Applicable', 'Level 3', 'No', 'Weak', 28.0, nan, nan, 'Average', 42.0, 'Very Weak', 14.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Strong', 54.0, '80.40%', 251.6, '93.50%', '100.00%', nan, nan, nan, nan, nan, nan, 0.0, 2.9, 35.5, 32.3, 0.0, 0.0, 0.0, 0.0, -0.6, -0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 21, 38, nan, 1165892.869, 1893999.3069999998, 41.86471954, -87.66648084, 28, 'NEAR WEST SIDE', 2, 12, '(41.86471954, -87.66648084)') ,\n(610082, 'Mount Greenwood Elementary School', 'ES', '10841 S Homan Ave', 'Chicago', 'IL', 60655, '(773) 535-2786', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610082.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 84.0, nan, nan, 'Average', 58.0, 'Weak', 38.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '96.00%', 2.9, '96.10%', '98.50%', 85.8, nan, 64.6, 71.2, 66.9, 63.1, 72.0, 71.6, 63.1, 51.6, 41.3, 65.3, 39.5, 39.2, -0.1, 0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 909, 49, nan, 1155639.216, 1832346.852, 41.69574845, -87.70577383, 74, 'MOUNT GREENWOOD', 19, 22, '(41.69574845, -87.70577383)') ,\n(610086, 'Mount Vernon Elementary School', 'ES', '10540 S Morgan St', 'Chicago', 'IL', 60643, '(773) 535-2825', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610086.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 31.0, 'Weak', 26.0, 'Average', 42.0, 'Average', 44.0, 'Weak', 14.0, 'Very Weak', 19.0, 'Average', 47.0, 'Average', 52.0, '93.30%', 76.6, '94.10%', '100.00%', 41.2, 36.5, 18.8, 17.1, 37.5, 52.5, 29.3, 40.5, 74.5, 75.0, 0.0, 4.3, 14.6, 9.8, 1.0, 0.7, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 290, 49, nan, 1171484.32, 1834733.756, 41.70196704, -87.64768959, 73, 'WASHINGTON HEIGHTS', 34, 22, '(41.70196704, -87.64768959)') ,\n(610385, 'Multicultural Academy of Scholarship', 'HS', '3120 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 535-4242', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610385.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 48.0, nan, nan, 'Average', 47.0, 'Average', 42.0, 'Weak', nan, nan, nan, 'Strong', 56.0, 'Average', 52.0, '89.20%', 22.4, '96.70%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.0, 13.0, 14.4, 14.1, 1.1, 15.6, 1.2, 21.1, 58.0, 58.5, 324, 37, 78.7, 1147521.3020000001, 1883405.128, 41.83601953, -87.73419465, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83601953, -87.73419465)') ,\n(609806, 'Myra Bradwell Communications Arts & Sciences Elementary School', 'ES', '7736 S Burnham Ave', 'Chicago', 'IL', 60649, '(773) 535-6600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609806.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Weak', 35.0, 'Average', 41.0, 'Average', 50.0, 'Weak', 52.0, 'Average', 54.0, 'Average', 49.0, 'Strong', 54.0, '92.10%', 75.9, '96.40%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1.4, 10.8, 6.3, 4.3, 1.2, -1.2, 'Green', 'Red', nan, nan, 11.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, 775, 46, nan, 1196028.811, 1854039.898, 41.7543734, -87.55717881, 43, 'SOUTH SHORE', 7, 4, '(41.7543734, -87.55717881)') ,\n(609952, 'Nathanael Greene Elementary School', 'ES', '3525 S Honore', 'Chicago', 'IL', 60609, '(773) 535-4560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609952.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 48.0, 'Strong', 54.0, '95.50%', 3.8, '96.20%', '98.60%', nan, nan, 43.2, 44.7, 58.3, 49.7, nan, nan, nan, nan, nan, nan, 23.2, 20.2, -1.2, 0.2, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 631, 39, nan, 1164636.2759999998, 1881214.535, 41.82966356, -87.67145551, 59, 'MCKINLEY PARK', 11, 9, '(41.82966356, -87.67145551)') ,\n(609960, 'Nathan Hale Elementary School', 'ES', '6140 S Melvina', 'Chicago', 'IL', 60638, '(773) 535-2265', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609960.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, 'Strong', 63.0, 'Average', 51.0, 'Average', 56.0, 'Weak', 75.0, 'Strong', 65.0, 'Strong', 55.0, 'Average', 52.0, '94.50%', 9.8, '96.20%', '96.50%', 53.8, 34.4, 43.3, 49.0, 55.2, 56.2, 35.1, 54.6, 53.6, 55.1, 12.5, 29.5, 19.3, 15.6, -0.5, 0.4, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 872, 44, nan, 1136092.7920000001, 1862988.841, 41.78020486, -87.77661589, 64, 'CLEARING', 23, 8, '(41.78020486, -87.77661589)') ,\n(610134, 'Nathaniel Pope Elementary School', 'ES', '1852 S Albany Ave', 'Chicago', 'IL', 60623, '(773) 534-1795', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610134.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 48.0, nan, nan, 'Strong', 73.0, 'Very Strong', 83.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '91.60%', 54.2, '95.80%', '93.50%', 50.0, nan, 19.3, 17.6, 40.0, 60.9, 33.3, 21.9, 64.8, 70.2, 0.0, 10.5, 4.5, 6.8, 0.2, -0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 200, 37, nan, 1155971.872, 1890640.0680000002, 41.85570712, -87.70299144, 29, 'NORTH LAWNDALE', 24, 10, '(41.85570712, -87.70299144)') ,\n(610348, 'Nathan R Goldblatt Elementary School', 'ES', '4257 W Adams St', 'Chicago', 'IL', 60624, '(773) 534-6860', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610348.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 49.0, nan, nan, 'Very Strong', 80.0, 'Very Strong', 99.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Strong', 58.0, '92.90%', 57.7, '95.60%', '100.00%', 66.7, 56.0, 44.7, 17.2, 56.8, 69.3, 55.1, 23.5, 67.7, 31.3, nan, nan, 11.7, 7.5, 0.7, -1.1, 'Green', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 268, 36, nan, 1147813.932, 1898625.5969999998, 41.87778081, -87.73273021, 26, 'WEST GARFIELD PARK', 28, 11, '(41.87778081, -87.73273021)') ,\n(609876, 'Nathan S Davis Elementary School', 'ES', '3014 W 39th Pl', 'Chicago', 'IL', 60632, '(773) 535-4540', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609876.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 43.0, nan, nan, 'Average', 41.0, 'Average', 46.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 50.0, '95.10%', 2.4, '95.90%', '94.00%', 42.1, nan, 35.4, 20.7, 44.4, 55.3, 39.2, 24.3, 61.4, 48.2, 18.6, 26.7, 16.2, 9.2, 0.5, 1.0, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1032, 39, nan, 1156815.412, 1878318.629, 41.821878600000005, -87.70022859, 58, 'BRIGHTON PARK', 14, 9, '(41.8218786, -87.70022859)') ,\n(610231, 'National Teachers Elementary Academy', 'ES', '55 W Cermak Rd', 'Chicago', 'IL', 60616, '(773) 534-9970', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610231.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, 'Strong', 70.0, 'Strong', 65.0, 'Strong', 60.0, 'Weak', 59.0, 'Average', 46.0, 'Strong', 56.0, 'Strong', 61.0, '94.70%', 9.2, '96.30%', '100.00%', 56.8, 36.7, 45.6, 48.5, 62.5, 70.4, 25.6, 42.9, 57.8, 61.5, 10.0, 10.0, 16.5, 12.7, 0.4, 1.1, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 446, 40, nan, 1176061.392, 1889726.757, 41.85277255, -87.62928114, 33, 'NEAR SOUTH SIDE', 3, 1, '(41.85277255, -87.62928114)') ,\n(609692, 'Neal F Simeon Career Academy High School', 'HS', '8147 S Vincennes Ave', 'Chicago', 'IL', 60620, '(773) 535-3200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609692.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 52.0, nan, nan, 'Weak', 33.0, 'Average', 42.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '78.60%', 2.1, '94.30%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.3, 14.1, 15.3, 14.8, 0.5, 16.2, 0.9, 14.8, 75.2, 74.0, 1535, 45, 68.5, 1174719.074, 1850730.1919999998, 41.74579217, -87.63537, 44, 'CHATHAM', 21, 6, '(41.74579217, -87.63537)') ,\n(610085, 'Near North Elementary School', 'ES', '739 N Ada St', 'Chicago', 'IL', 60642, '(773) 534-7845', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610085.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', nan, 'Standard', 'Not Applicable', 'Level 3', 'No', 'Average', 50.0, nan, nan, 'Weak', 27.0, 'Weak', 23.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '80.20%', 58.3, '94.80%', '98.10%', nan, nan, 0.0, 0.0, 41.2, 21.7, 0.0, 3.8, 39.6, 31.3, 0.0, 0.0, 1.3, 0.0, -1.8, -1.6, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 91, 35, nan, 1167232.603, 1905230.9330000002, 41.89551132, -87.66123981, 24, 'WEST TOWN', 27, 13, '(41.89551132, -87.66123981)') ,\n(610284, 'New Field Elementary School', 'ES', '1707 W Morse Ave', 'Chicago', 'IL', 60626, '(773) 534-2760', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610284.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 51.0, 'Strong', 56.0, '96.20%', 3.1, '95.60%', '99.00%', 74.2, 52.7, 32.1, 25.2, 40.4, 55.9, nan, nan, nan, nan, nan, nan, 23.2, 10.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 696, 32, nan, 1163673.054, 1946078.6819999998, 42.00767533, -87.67315753, 1, 'ROGERS PARK', 49, 24, '(42.00767533, -87.67315753)') ,\n(610323, 'New Millennium High School of Health at Bowen', 'HS', '2710 E 89th St', 'Chicago', 'IL', 60617, '(773) 535-7650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610323.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 17.0, 'Weak', 23.0, 'Weak', 37.0, 'Weak', 25.0, 'Weak', 29.0, 'Very Weak', 6.0, nan, nan, nan, nan, '75.60%', 61.8, '94.70%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.1, 12.3, 13.9, 13.2, 1.1, 14.7, 0.8, 16.7, 67.2, 52.4, 701, 47, 62.1, 1195939.937, 1846527.5280000002, 41.73376107, -87.55775263, 46, 'SOUTH CHICAGO', 7, 4, '(41.73376107, -87.55775263)') ,\n(609792, 'Newton Bateman Elementary School', 'ES', '4220 N Richmond St', 'Chicago', 'IL', 60618, '(773) 534-5055', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609792.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, 'Weak', 36.0, 'Average', 50.0, 'Average', 47.0, 'Weak', 25.0, 'Weak', 34.0, 'Average', 52.0, 'Average', 52.0, '96.20%', 0.2, '96.70%', '99.10%', 68.0, 33.4, 35.8, 36.6, 54.2, 49.6, 52.6, 47.1, 65.1, 57.6, 12.5, 34.5, 21.7, 16.1, -0.8, -0.1, 'Red', 'Yellow', 27.4, 67.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1015, 31, nan, 1155915.8490000002, 1928001.2119999998, 41.95823045, -87.70218806, 16, 'IRVING PARK', 33, 17, '(41.95823045, -87.70218806)') ,\n(609730, 'Nicholas Senn High School', 'HS', '5900 N Glenwood Ave', 'Chicago', 'IL', 60660, '(773) 534-2365', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609730.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 48.0, nan, nan, 'Average', 45.0, 'Average', 42.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '81.70%', 41.3, '96.40%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.8, 14.3, 14.3, 13.9, 1.1, 15.8, 1.5, 17.5, 40.1, 54.4, 1053, 32, 79.8, 1165871.81, 1939308.369, 41.98905063, -87.66526222, 77, 'EDGEWATER', 48, 20, '(41.98905063, -87.66526222)') ,\n(609793, 'Nicholson Technology Academy', 'ES', '6006 S Peoria St', 'Chicago', 'IL', 60621, '(773) 535-3285', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609793.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 48.0, 'Weak', 37.0, 'Strong', 74.0, 'Strong', 73.0, 'Weak', 42.0, 'Weak', 39.0, nan, nan, nan, nan, '93.40%', 6.0, '97.00%', '100.00%', nan, nan, 39.1, 18.6, 54.1, 58.5, 64.3, 34.5, 69.4, 56.7, 21.2, 21.2, 22.6, 8.9, 1.9, -0.2, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 464, 45, nan, 1171298.247, 1864964.2680000002, 41.7849277, -87.64748868, 68, 'ENGLEWOOD', 16, 7, '(41.7849277, -87.64748868)') ,\n(609961, 'Ninos Heroes Elementary Academic Center', 'ES', '8344 S Commercial Ave', 'Chicago', 'IL', 60617, '(773) 535-6694', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609961.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 36.0, nan, nan, 'Strong', 61.0, 'Average', 55.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 49.0, '93.90%', 25.9, '95.00%', '100.00%', 52.5, 35.7, 33.9, 28.1, 52.8, 62.1, 44.2, 27.9, 66.3, 59.5, 3.6, 16.4, 14.7, 9.4, 0.9, 0.5, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 435, 47, nan, 1197504.847, 1850181.7480000001, 41.74374969, -87.55189813, 46, 'SOUTH CHICAGO', 10, 4, '(41.74374969, -87.55189813)') ,\n(609810, 'Norman A Bridge Elementary School', 'ES', '3800 N New England Ave', 'Chicago', 'IL', 60634, '(773) 534-3718', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609810.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 67.0, nan, nan, 'Average', 49.0, 'Weak', 35.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Average', 48.0, '95.60%', 6.6, '96.60%', '98.40%', 73.0, 43.0, 56.5, 45.8, 55.9, 74.0, nan, nan, nan, nan, 33.0, 38.6, 27.8, 18.9, -0.3, -0.9, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1004, 30, nan, 1129684.0559999999, 1924513.9319999998, 41.94915063, -87.79870722, 17, 'DUNNING', 38, 16, '(41.94915063, -87.79870722)') ,\n(609691, 'North-Grand High School', 'HS', '4338 W Wabansia Ave', 'Chicago', 'IL', 60639, '(773) 534-8520', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609691.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 42.0, nan, nan, 'Weak', 39.0, 'Weak', 25.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '88.60%', 15.3, '95.60%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.3, 12.8, 14.1, 14.2, 0.9, 15.4, 1.3, 15.8, 69.7, 41.3, 883, 34, 81.3, 1147041.135, 1910981.274, 41.91170094, -87.73525151, 23, 'HUMBOLDT PARK', 31, 25, '(41.91170094, -87.73525151)') ,\n(610354, 'North River Elementary School', 'ES', '4416 N Troy St', 'Chicago', 'IL', 60625, '(773) 534-0590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610354.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 67.0, 'Strong', 60.0, 'Average', 53.0, 'Average', 51.0, 'Weak', 47.0, 'Average', 47.0, 'Average', 50.0, 'Average', 51.0, '95.60%', 20.1, '96.00%', '100.00%', 46.2, 55.8, 47.9, 39.4, 56.3, 64.1, 27.8, 35.1, 35.7, 54.9, 22.2, 40.7, 19.6, 13.7, -0.2, -0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 359, 31, nan, 1154546.547, 1929268.9209999999, 41.96173669, -87.70718808, 14, 'ALBANY PARK', 33, 17, '(41.96173669, -87.70718808)') ,\n(609749, 'Northside College Preparatory High School', 'HS', '5501 N Kedzie Ave', 'Chicago', 'IL', 60625, '(773) 534-3954', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609749.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, nan, nan, 'Very Strong', 99.0, 'Very Strong', 88.0, 'Weak', nan, nan, nan, 'Strong', 57.0, 'Strong', 62.0, '95.70%', 2.8, '96.80%', '98.40%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 22.4, 22.2, 24.5, 24.7, 2.3, 28.8, 4.3, 98.0, 97.6, 90.7, 1053, 31, 95.9, 1154090.8590000002, 1936413.866, 41.98135196, -87.70867192, 13, 'NORTH PARK', 40, 17, '(41.98135196, -87.70867192)') ,\n(609762, 'Percy L Julian High School', 'HS', '10330 S Elizabeth St', 'Chicago', 'IL', 60643, '(773) 535-5170', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609762.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Average', 50.0, 'Weak', 30.0, 'Weak', 35.0, 'Weak', 53.0, 'Average', 51.0, nan, nan, nan, nan, '74.60%', 33.4, '94.20%', '98.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.1, 12.8, 14.2, 13.9, 0.8, 14.8, 0.6, 8.3, 60.3, 54.9, 1226, 49, 65.3, 1169808.363, 1836139.004, 41.70585971, -87.65378585, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.70585971, -87.65378585)') ,\n(609744, 'Northside Learning Center High School', 'HS', '3730 W Bryn Mawr Ave', 'Chicago', 'IL', 60659, '(773) 534-5180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609744.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', nan, 'Standard', 'Not Applicable', 'Not Enough Data', 'No', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 81.0, 'Weak', 34.0, 'Weak', 75.0, 'Very Strong', 84.0, 'Strong', 62.0, 'Strong', 59.0, '89.80%', 5.3, '95.60%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 66.7, 0.0, nan, 298, 31, nan, 1150357.037, 1936987.496, 41.98299994, -87.72238881, 13, 'NORTH PARK', 39, 17, '(41.98299994, -87.72238881)') ,\n(610051, 'Northwest Middle School', 'MS', '5252 W Palmer St', 'Chicago', 'IL', 60639, '(773) 534-3250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610051.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 34.0, nan, nan, 'Very Weak', 17.0, 'Very Weak', 13.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '93.40%', 2.2, '95.10%', '98.90%', nan, nan, nan, nan, nan, nan, 29.5, 25.1, 41.4, 41.1, 10.3, 14.8, 11.3, 5.9, -0.9, -1.4, 'Red', 'Red', 10.5, 62.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 756, 29, nan, 1140773.188, 1914158.9440000001, 41.92053845, -87.75820003, 19, 'BELMONT CRAGIN', 37, 25, '(41.92053845, -87.75820003)') ,\n(610099, 'Norwood Park Elementary School', 'ES', '5900 N Nina Ave', 'Chicago', 'IL', 60631, '(773) 534-1198', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610099.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 86.0, 'Strong', 69.0, 'Strong', 68.0, 'Weak', 31.0, 'Weak', 32.0, 'Strong', 57.0, 'Average', 52.0, '95.50%', 0.3, '95.60%', '100.00%', 75.2, 57.8, 69.3, 65.8, 56.3, 52.7, 90.3, 77.7, 57.3, 61.2, 64.1, 69.2, 50.5, 39.0, -1.8, -0.9, 'Red', 'Red', 94.7, 69.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 359, 30, nan, 1128430.406, 1938729.626, 41.98818136, -87.80299207, 10, 'NORWOOD PARK', 41, 16, '(41.98818136, -87.80299207)') ,\n(610529, 'Ogden International High School', 'HS', '1250 W Erie St', 'Chicago', 'IL', 60622, '(773) 534-0866', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610529.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 56.0, nan, nan, 'Weak', 39.0, 'Weak', 24.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.20%', 13.7, '0.00%', '100.00%', nan, nan, nan, nan, nan, nan, 61.9, 71.6, 48.1, 61.6, 47.9, 69.2, 42.6, 38.9, -0.6, -0.6, 'Red', 'Yellow', 58.7, 29.0, 14.5, 14.7, nan, 16.0, 1.5, nan, nan, nan, nan, nan, 558, 35, 93.3, 1167748.1, 1904529.595, 41.89357570000001, -87.65936676, 24, 'WEST TOWN', 27, 13, '(41.8935757, -87.65936676)') ,\n(610201, 'Ole A Thorp Elementary Scholastic Academy', 'ES', '6024 W Warwick Ave', 'Chicago', 'IL', 60634, '(773) 534-3640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610201.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 70.0, 'Strong', 61.0, 'Average', 47.0, 'Average', 40.0, 'Weak', 47.0, 'Average', 57.0, 'Average', 50.0, 'Average', 50.0, '96.40%', 25.8, '95.80%', '100.00%', 80.6, 62.3, 62.3, 65.0, 56.8, 55.9, 73.9, 66.5, 64.8, 61.5, 46.8, 58.5, 48.8, 38.7, 0.8, -0.5, 'Green', 'Yellow', 33.7, 100.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 834, 30, nan, 1135416.207, 1924349.243, 41.94859865, -87.77764028, 17, 'DUNNING', 38, 16, '(41.94859865, -87.77764028)') ,\n(610300, 'Oliver S Westcott Elementary School', 'ES', '409 W 80th St', 'Chicago', 'IL', 60620, '(773) 535-3090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610300.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 30.0, 'Average', 41.0, 'Average', 48.0, 'Average', 54.0, 'Weak', 47.0, 'Average', 47.0, 'Weak', 46.0, 'Average', 48.0, '92.60%', 31.5, '94.80%', '100.00%', nan, 14.7, 36.4, 27.9, 50.4, 48.5, 27.9, 31.3, 43.5, 51.5, 6.0, 20.0, 7.0, 6.0, 0.3, 0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 413, 45, nan, 1174868.475, 1851842.144, 41.74884018, -87.63478949, 44, 'CHATHAM', 21, 6, '(41.74884018, -87.63478949)') ,\n(609997, 'Oliver Wendell Holmes Elementary School', 'ES', '955 W Garfield Blvd', 'Chicago', 'IL', 60621, '(773) 535-9025', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609997.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 21.0, nan, nan, 'Weak', 23.0, 'Average', 41.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 48.0, '87.00%', 139.8, '92.40%', '97.90%', 40.0, 30.5, 17.7, 8.0, 28.8, 33.1, 20.8, 8.8, 41.8, 20.2, 3.1, 9.4, 5.0, 3.0, -0.8, -2.3, 'Red', 'Red', 9.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 334, 45, nan, 1170648.336, 1868177.8909999998, 41.79376044, -87.64977785, 68, 'ENGLEWOOD', 16, 7, '(41.79376044, -87.64977785)') ,\n(610105, 'Oriole Park Elementary School', 'ES', '5424 N Oketo Ave', 'Chicago', 'IL', 60656, '(773) 534-1201', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610105.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, nan, nan, 'Very Strong', 88.0, 'Strong', 71.0, 'Weak', nan, nan, nan, 'Strong', 60.0, 'Weak', 46.0, '96.50%', 2.3, '95.50%', '98.40%', 84.4, nan, 86.0, 78.1, 59.6, 77.5, 91.4, 85.1, 81.1, 67.0, 69.6, 72.9, 75.1, 54.7, 1.1, -0.2, 'Green', 'Yellow', 87.1, 53.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 648, 30, nan, 1125978.285, 1935191.9819999998, 41.97851501, -87.81209057, 10, 'NORWOOD PARK', 41, 16, '(41.97851501, -87.81209057)') ,\n(610329, 'Orozco Fine Arts & Sciences Elementary School', 'ES', '1940 W 18th St', 'Chicago', 'IL', 60608, '(773) 534-7215', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610329.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 55.0, nan, nan, 'Weak', 34.0, 'Average', 44.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 53.0, '97.60%', 1.4, '96.40%', '100.00%', 92.9, 78.6, 82.8, 67.0, 73.9, 77.0, 50.1, 45.0, 51.0, 52.0, 18.8, 33.8, 30.2, 17.4, -0.6, -0.1, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 636, 39, nan, 1163539.274, 1891453.1469999999, 41.85778248, -87.67519253, 31, 'LOWER WEST SIDE', 25, 12, '(41.85778248, -87.67519253)') ,\n(610389, 'Orr Academy High School', 'HS', '730 N Pulaski Rd', 'Chicago', 'IL', 60624, '(773) 534-6500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610389.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, nan, nan, nan, nan, '66.30%', 10.2, '95.00%', '99.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.7, 12.0, 13.6, 13.3, 1.6, 15.1, 1.5, 5.8, nan, 51.9, 831, 34, 59.5, 1149548.53, 1904711.708, 41.89444828, -87.72620305, 23, 'HUMBOLDT PARK', 28, 11, '(41.89444828, -87.72620305)') ,\n(609811, 'Orville T Bright Elementary School', 'ES', '10740 S Calhoun Ave', 'Chicago', 'IL', 60617, '(773) 535-6215', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609811.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 18.0, nan, nan, 'Weak', 28.0, 'Weak', 27.0, 'Weak', nan, nan, nan, 'Weak', 41.0, 'Weak', 42.0, '92.20%', 55.7, '95.30%', '100.00%', 50.5, 19.0, 30.4, 28.2, 65.1, 64.8, 35.8, 32.3, 74.1, 61.3, 0.0, 17.9, 8.2, 5.9, 2.4, 0.6, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 361, 47, nan, 1194858.644, 1834119.7780000002, 41.69973969, -87.56212104, 51, 'SOUTH DEERING', 10, 4, '(41.69973969, -87.56212104)') ,\n(610367, 'Oscar DePriest Elementary School', 'ES', '139 S Parkside Ave', 'Chicago', 'IL', 60644, '(773) 534-6800', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610367.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 35.0, 'Weak', 39.0, 'Average', 47.0, 'Average', 53.0, 'Weak', 51.0, 'Weak', 39.0, 'Average', 47.0, 'Strong', 54.0, '92.70%', 20.0, '95.60%', '94.10%', 66.4, 62.0, 26.8, 18.4, 42.9, 43.2, 35.8, 45.9, 78.2, 76.9, 0.0, 9.1, 4.8, 9.8, 0.9, 2.3, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 524, 36, nan, 1138681.703, 1898643.0369999998, 41.87799914, -87.7662615, 25, 'AUSTIN', 29, 15, '(41.87799914, -87.7662615)') ,\n(610059, 'Oscar F Mayer Elementary School', 'ES', '2250 N Clifton Ave', 'Chicago', 'IL', 60614, '(773) 534-5535', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610059.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 67.0, nan, nan, 'Weak', 35.0, 'Average', 46.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.00%', 11.4, '95.20%', '100.00%', 79.6, nan, 47.9, 36.8, 53.1, 59.8, 26.9, 37.4, 37.8, 46.8, 16.4, 32.7, 18.1, 17.0, -0.6, 0.1, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 605, 33, nan, 1168219.452, 1915320.55, 41.92317657, -87.65732331, 7, 'LINCOLN PARK', 32, 18, '(41.92317657, -87.65732331)') ,\n(610021, 'Pablo Casals Elementary School', 'ES', '3501 W Potomac Ave', 'Chicago', 'IL', 60651, '(773) 534-4444', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610021.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Weak', 42.0, 'Weak', 43.0, '92.00%', 47.1, '95.90%', '97.20%', 45.1, 32.1, 23.6, 14.6, 46.7, 39.7, 24.6, 25.8, 45.4, 67.2, 8.2, 18.4, 9.6, 5.0, -1.1, -0.3, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 505, 34, nan, 1152770.808, 1908388.615, 41.90447488, -87.71427105, 23, 'HUMBOLDT PARK', 26, 14, '(41.90447488, -87.71427105)') ,\n(610115, 'Park Manor Elementary School', 'ES', '7037 S Rhodes Ave', 'Chicago', 'IL', 60637, '(773) 535-3070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610115.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Average', 42.0, 'Average', 42.0, 'Average', 48.0, 'Weak', 69.0, 'Strong', 67.0, 'Strong', 55.0, 'Strong', 54.0, '94.90%', 16.6, '96.30%', '100.00%', nan, nan, 17.6, 16.0, 43.3, 45.3, 28.9, 26.5, 75.0, 66.2, 3.6, 3.6, 6.2, 5.7, 1.7, 1.2, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 352, 46, nan, 1181167.289, 1858324.511, 41.76648584, -87.61150917, 69, 'GREATER GRAND CROSSING', 6, 3, '(41.76648584, -87.61150917)') ,\n(610116, 'Parkside Elementary Community Academy', 'ES', '6938 S East End Ave', 'Chicago', 'IL', 60649, '(773) 535-0940', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610116.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Weak', 42.0, 'Weak', 43.0, '89.80%', 90.7, '94.90%', '100.00%', 57.3, 33.6, 17.3, 7.8, 45.2, 44.0, 19.0, 22.6, 41.7, 53.3, 9.5, 4.8, 5.6, 3.5, -0.2, -0.1, 'Yellow', 'Yellow', nan, nan, 11.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, 348, 46, nan, 1188940.5420000001, 1859149.081, 41.76856599, -87.58299131, 43, 'SOUTH SHORE', 5, 3, '(41.76856599, -87.58299131)') ,\n(609988, 'Patrick Henry Elementary School', 'ES', '4250 N Saint Louis Ave', 'Chicago', 'IL', 60618, '(773) 534-5060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609988.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 45.0, 'Average', 56.0, 'Average', 46.0, 'Average', 43.0, 'Weak', 68.0, 'Average', 45.0, 'Average', 52.0, 'Average', 49.0, '95.10%', 4.3, '95.90%', '100.00%', 56.6, 35.5, 42.1, 38.1, 68.2, 61.3, 30.4, 48.2, 44.6, 75.0, nan, nan, 12.5, 12.3, 0.6, 1.0, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 650, 31, nan, 1152247.412, 1928240.9980000001, 41.95896179, -87.71566829, 16, 'IRVING PARK', 33, 17, '(41.95896179, -87.71566829)') ,\n(610003, 'Paul Cuffe Math-Science Technology Academy Elementary School', 'ES', '8324 S Racine Ave', 'Chicago', 'IL', 60620, '(773) 535-8250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610003.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 51.0, 'Strong', 60.0, 'Average', 45.0, 'Average', 54.0, 'Weak', 66.0, 'Very Strong', 83.0, 'Average', 48.0, 'Average', 51.0, '93.60%', 46.9, '95.40%', '98.50%', 72.4, 72.1, 44.3, 44.9, 63.0, 68.8, 54.2, 44.3, 54.8, 52.4, 23.1, 40.0, 20.6, 15.6, -0.5, -1.1, 'Red', 'Red', 28.8, 21.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 512, 45, nan, 1169734.759, 1849470.5, 41.74244489, -87.65366994, 71, 'AUBURN GRESHAM', 21, 6, '(41.74244489, -87.65366994)') ,\n(609676, 'Paul Laurence Dunbar Career Academy High School', 'HS', '3000 S King Dr', 'Chicago', 'IL', 60616, '(773) 534-9000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609676.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 29.0, 'Average', 57.0, 'Weak', 31.0, 'Weak', 37.0, 'Weak', 57.0, 'Average', 49.0, nan, nan, nan, nan, '76.90%', 21.5, '96.00%', '98.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.8, 12.7, 13.9, 13.8, 1.0, 14.7, 0.8, 6.9, 61.7, 51.5, 1456, 40, 65.7, 1179262.097, 1885167.032, 41.84018775, -87.61767315, 35, 'DOUGLAS', 2, 1, '(41.84018775, -87.61767315)') ,\n(610146, 'Paul Revere Elementary School', 'ES', '1010 E 72nd St', 'Chicago', 'IL', 60619, '(773) 535-0618', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610146.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, nan, nan, 'Average', 42.0, 'Average', 56.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Average', 47.0, '92.40%', 33.3, '94.60%', '89.30%', 46.5, 38.4, 10.7, 16.4, 55.0, 49.1, 15.7, 25.7, 50.6, 61.1, 6.7, 16.7, 3.5, 3.4, -2.4, -0.4, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 320, 46, nan, 1184267.268, 1857526.2380000001, 41.76422335, -87.60017165, 69, 'GREATER GRAND CROSSING', 5, 3, '(41.76422335, -87.60017165)') ,\n(609707, 'Paul Robeson High School', 'HS', '6835 S Normal Blvd', 'Chicago', 'IL', 60621, '(773) 535-3800', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609707.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 29.0, nan, nan, 'Weak', 33.0, 'Average', 40.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '73.50%', 26.4, '94.70%', '97.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.6, 11.0, 12.7, 12.7, 1.1, 13.6, 0.9, 2.9, 40.6, 42.2, 733, 45, 53.3, 1174171.884, 1859450.314, 41.76973341, -87.63711641, 68, 'ENGLEWOOD', 6, 7, '(41.76973341, -87.63711641)') ,\n(609791, 'Perkins Bass Elementary School', 'ES', '1140 W 66th St', 'Chicago', 'IL', 60621, '(773) 535-3275', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609791.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Average', 50.0, 'Average', 47.0, 'Average', 57.0, 'Weak', 52.0, 'Average', 42.0, 'Average', 47.0, 'Average', 49.0, '92.10%', 28.9, '96.40%', '100.00%', 61.7, 42.2, 15.5, 15.9, 45.3, 31.5, 23.9, 18.6, 53.6, 38.8, 7.3, 11.9, 5.0, 6.0, -0.1, 0.9, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 369, 45, nan, 1169677.933, 1861068.9430000002, 41.77427377, -87.65354234, 68, 'ENGLEWOOD', 17, 7, '(41.77427377, -87.65354234)') ,\n(610395, 'Pershing West Middle School', 'MS', '3200 S Calumet Ave', 'Chicago', 'IL', 60616, '(773) 534-9240', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610395.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 46.0, 'Average', 53.0, 'Weak', 34.0, 'Average', 55.0, 'Weak', 76.0, 'Average', 58.0, nan, nan, nan, nan, '96.20%', 3.7, '97.60%', '100.00%', nan, nan, 52.7, 45.0, 40.6, 60.0, 43.7, 54.1, 48.7, 52.9, 15.4, 28.8, 17.5, 21.4, -0.9, -0.5, 'Red', 'Yellow', 40.7, 50.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 269, 40, nan, 1179082.41, 1883818.0119999999, 41.83649005, -87.61837372, 35, 'DOUGLAS', 2, 2, '(41.83649005, -87.61837372)') ,\n(610145, 'Peter A Reinberg Elementary School', 'ES', '3425 N Major Ave', 'Chicago', 'IL', 60634, '(773) 534-3465', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610145.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 60.0, 'Average', 51.0, 'Weak', 35.0, 'Weak', 30.0, 'Weak', 60.0, 'Average', 54.0, 'Average', 47.0, 'Weak', 45.0, '95.80%', 8.2, '95.70%', '99.50%', 70.9, 46.3, 50.6, 39.6, 51.3, 66.0, 45.0, 39.1, 56.3, 49.6, 19.8, 25.7, 24.0, 14.7, -0.3, 0.5, 'Yellow', 'Yellow', 25.0, 80.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 832, 30, nan, 1137794.92, 1922326.021, 41.94300407, -87.76894541, 15, 'PORTAGE PARK', 38, 16, '(41.94300407, -87.76894541)') ,\n(609867, 'Peter Cooper Elementary Dual Language Academy', 'ES', '1624 W 19th St', 'Chicago', 'IL', 60608, '(773) 534-7205', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609867.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, nan, nan, nan, nan, '96.20%', 5.7, '96.30%', '100.00%', 62.7, 28.7, 37.1, 26.1, 43.5, 44.6, nan, nan, nan, nan, nan, nan, 9.6, 10.7, 0.5, 1.3, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 709, 39, nan, 1165708.517, 1890841.583, 41.85605839, -87.66724753, 31, 'LOWER WEST SIDE', 25, 12, '(41.85605839, -87.66724753)') ,\n(610147, 'Philip Rogers Elementary School', 'ES', '7345 N Washtenaw Ave', 'Chicago', 'IL', 60645, '(773) 534-2125', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610147.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 81.0, 'Strong', 76.0, 'Strong', 68.0, 'Strong', 62.0, 'Weak', 75.0, 'Strong', 64.0, 'Average', 53.0, 'Average', 51.0, '96.40%', 3.0, '95.10%', '100.00%', 77.3, 50.9, 49.8, 47.2, 60.9, 61.9, 73.0, 52.7, 75.1, 53.6, 37.3, 52.5, 34.3, 28.3, 0.7, 0.7, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 735, 32, nan, 1157132.164, 1948795.4740000002, 42.01526618, -87.69714858, 2, 'WEST RIDGE', 50, 24, '(42.01526618, -87.69714858)') ,\n(609777, 'Phillip D Armour Elementary School', 'ES', '950 W 33rd Pl', 'Chicago', 'IL', 60608, '(773) 535-4530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609777.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 36.0, nan, nan, 'Weak', 34.0, 'Very Weak', 14.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Average', 51.0, '93.90%', 16.0, '95.80%', '98.00%', 67.7, 26.5, 16.5, 24.0, 37.8, 43.6, 29.8, 24.0, 51.5, 44.9, 3.8, 11.5, 10.1, 8.7, -0.5, 0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 365, 40, nan, 1170302.817, 1882796.1940000001, 41.8338821, -87.65061898, 60, 'BRIDGEPORT', 11, 9, '(41.8338821, -87.65061898)') ,\n(610090, 'Phillip Murray Elementary Language Academy', 'ES', '5335 S Kenwood Ave', 'Chicago', 'IL', 60615, '(773) 535-0585', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610090.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 66.0, nan, nan, 'Weak', 34.0, 'Weak', 37.0, 'Weak', nan, nan, nan, 'Strong', 56.0, 'Average', 53.0, '95.60%', 3.5, '95.90%', '100.00%', 70.2, nan, 66.9, 67.4, 55.1, 61.8, 43.6, 58.0, 53.0, 42.0, nan, nan, 35.0, 36.0, -0.4, 0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 533, 46, nan, 1186033.436, 1870041.58, 41.79852501, -87.59330352, 41, 'HYDE PARK', 4, 2, '(41.79852501, -87.59330352)') ,\n(609981, 'Phobe Apperson Hearst Elementary School', 'ES', '4640 S Lamon Ave', 'Chicago', 'IL', 60638, '(773) 535-2376', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609981.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 33.0, nan, nan, 'Weak', 31.0, 'Weak', 27.0, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Weak', 44.0, '92.10%', 25.5, '95.80%', '97.90%', 52.2, 48.8, 14.9, 10.7, 29.7, 40.0, 29.5, 22.2, 50.0, 47.1, 7.1, 10.7, 4.9, 6.7, -0.2, -1.6, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 364, 44, nan, 1144438.111, 1873241.7659999998, 41.80818821, -87.74576315, 56, 'GARFIELD RIDGE', 23, 8, '(41.80818821, -87.74576315)') ,\n(610304, 'Phoenix Military Academy High School', 'HS', '145 S Campbell Ave', 'Chicago', 'IL', 60612, '(773) 534-7275', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610304.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 53.0, nan, nan, 'Average', 59.0, 'Strong', 78.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Average', 48.0, '92.10%', 4.3, '97.40%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.7, 14.9, 15.9, 15.9, 1.2, 18.6, 2.7, 33.9, 74.8, 51.3, 461, 38, 74.1, 1159815.389, 1899102.4919999999, 41.87885054, -87.68865058, 28, 'NEAR WEST SIDE', 2, 11, '(41.87885054, -87.68865058)') ,\n(610013, 'Pilsen Elementary Community Academy', 'ES', '1420 W 17th St', 'Chicago', 'IL', 60608, '(773) 534-7675', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610013.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 43.0, 'Average', 40.0, 'Average', 49.0, 'Average', 55.0, 'Weak', 16.0, 'Weak', 37.0, nan, nan, nan, nan, '96.00%', 9.4, '96.70%', '100.00%', 58.1, 36.1, 23.7, 19.4, 46.3, 46.7, 35.7, 33.3, 50.0, 51.2, nan, nan, 18.0, 12.8, -1.0, 0.3, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 447, 39, nan, 1167068.653, 1891878.989, 41.85887606, -87.66222540000001, 31, 'LOWER WEST SIDE', 25, 12, '(41.85887606, -87.6622254)') ,\n(610135, 'Portage Park Elementary School', 'ES', '5330 W Berteau Ave', 'Chicago', 'IL', 60641, '(773) 534-3576', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610135.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 62.0, nan, nan, 'Average', 46.0, 'Weak', 32.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 53.0, '94.20%', 3.7, '95.90%', '100.00%', 67.3, 33.8, 53.7, 41.6, 45.9, 61.1, 62.8, 55.3, 51.2, 44.1, 27.1, 37.4, 22.9, 21.1, -0.2, 0.8, 'Yellow', 'Green', 22.0, 66.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1139, 30, nan, 1139920.815, 1927445.996, 41.9570151, -87.76100593, 15, 'PORTAGE PARK', 38, 16, '(41.9570151, -87.76100593)') ,\n(610138, 'Pulaski International Academy Elmentary School', 'ES', '2230 W McLean Ave', 'Chicago', 'IL', 60647, '(773) 534-4391', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610138.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 68.0, 'Average', 45.0, 'Average', 54.0, 'Average', 57.0, 'Weak', 26.0, 'Weak', 23.0, 'Average', 47.0, 'Average', 47.0, '96.10%', 1.7, '96.20%', '100.00%', 75.2, 48.5, 38.3, 38.5, 54.5, 58.3, 56.7, 46.0, 71.1, 56.7, 33.3, 44.0, 24.2, 19.4, 0.5, 1.2, 'Green', 'Green', 37.3, 53.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 864, 35, nan, 1161026.753, 1913565.719, 41.91851374, -87.68380061, 22, 'LOGAN SQUARE', 32, 14, '(41.91851374, -87.68380061)') ,\n(609842, 'Rachel Carson Elementary School', 'ES', '5516 S Maplewood Ave', 'Chicago', 'IL', 60629, '(773) 535-9222', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609842.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 38.0, 'Weak', 36.0, 'Average', 45.0, 'Average', 56.0, 'Weak', 26.0, 'Weak', 36.0, 'Average', 50.0, 'Average', 51.0, '97.60%', 3.9, '97.70%', '100.00%', 68.5, nan, 44.0, 35.0, 63.2, 75.8, 40.8, 45.5, 59.8, 59.5, 18.1, 24.4, 19.4, 12.2, -0.4, 0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1222, 43, nan, 1160271.679, 1867913.54, 41.79325514, -87.68783576, 63, 'GAGE PARK', 16, 8, '(41.79325514, -87.68783576)') ,\n(609902, 'Ralph H Metcalfe Elementary Community Academy', 'ES', '12339 S Normal Ave', 'Chicago', 'IL', 60628, '(773) 535-5590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609902.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 50.0, '94.90%', 22.3, '95.50%', '100.00%', 43.2, 20.9, 27.5, 26.2, 47.3, 42.7, 51.1, 34.6, 60.5, 57.6, 13.6, 40.7, 18.2, 12.6, -0.7, -0.3, 'Red', 'Yellow', 24.6, 84.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 489, 48, nan, 1175245.267, 1822916.582, 41.6694559, -87.63426928, 53, 'WEST PULLMAN', 34, 5, '(41.6694559, -87.63426928)') ,\n(610141, 'Ravenswood Elementary School', 'ES', '4332 N Paulina St', 'Chicago', 'IL', 60613, '(773) 534-5525', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610141.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 46.0, nan, nan, 'Average', 42.0, 'Average', 41.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Weak', 44.0, '94.90%', 8.1, '95.70%', '100.00%', 78.7, nan, 46.9, 45.9, 62.2, 74.5, 39.8, 51.9, 67.6, 59.4, 4.3, 34.8, 18.9, 13.7, -0.4, -0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 465, 33, nan, 1164402.827, 1928980.2780000002, 41.96074129, -87.67095905, 6, 'LAKE VIEW', 47, 19, '(41.96074129, -87.67095905)') ,\n(609769, 'Ray Graham Training Center High School', 'HS', '2347 S Wabash Ave', 'Chicago', 'IL', 60616, '(773) 534-9257', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609769.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', nan, 'Standard', 'Not Applicable', 'Not Enough Data', 'No', 'Very Strong', 90.0, 'Very Strong', 82.0, 'Average', 52.0, 'Weak', 39.0, 'Weak', 63.0, 'Average', 52.0, 'Average', 53.0, 'Strong', 58.0, '86.30%', 3.7, '94.30%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 18.8, 10.3, nan, 137, 40, nan, 1177161.058, 1888615.131, 41.84969736, -87.6252787, 33, 'NEAR SOUTH SIDE', 2, 1, '(41.84969736, -87.6252787)') ,\n(609903, 'Richard Edwards Elementary School', 'ES', '4815 S Karlov Ave', 'Chicago', 'IL', 60632, '(773) 535-4875', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609903.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 48.0, 'Average', 48.0, 'Weak', 38.0, 'Average', 41.0, 'Weak', 49.0, 'Average', 49.0, 'Average', 49.0, 'Average', 47.0, '95.30%', 10.3, '96.80%', '96.10%', 73.3, 42.8, 46.5, 26.3, 52.2, 62.5, 32.7, 27.6, 44.1, 50.8, 11.0, 28.4, 17.6, 11.4, -0.7, -0.1, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1503, 44, nan, 1149882.354, 1872308.4740000002, 41.80552325, -87.72581893, 57, 'ARCHER HEIGHTS', 14, 8, '(41.80552325, -87.72581893)') ,\n(610291, 'Richard Henry Lee Elementary School', 'ES', '6448 S Tripp Ave', 'Chicago', 'IL', 60629, '(773) 535-2255', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610291.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, nan, nan, 'Average', 46.0, 'Weak', 35.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.00%', 21.8, '95.30%', '96.10%', 55.7, 29.4, 46.9, 28.5, 51.2, 60.6, 56.3, 30.7, 60.1, 43.7, 30.1, 22.1, 15.6, 7.6, 0.0, -0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 832, 44, nan, 1149113.752, 1861299.6930000002, 41.77532831, -87.72892174, 65, 'WEST LAWN', 13, 8, '(41.77532831, -87.72892174)') ,\n(610239, 'Richard J Daley Elementary Academy', 'ES', '5024 S Wolcott Ave', 'Chicago', 'IL', 60609, '(773) 535-9091', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610239.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 29.0, 'Average', 40.0, 'Weak', 37.0, 'Average', 46.0, 'Weak', 50.0, 'Average', 42.0, 'Weak', 46.0, 'Average', 47.0, '94.90%', 44.2, '96.00%', '98.40%', 50.8, 26.4, 36.5, 13.3, 52.6, 63.9, 43.4, 25.5, 70.4, 55.6, 16.7, 5.6, 14.4, 5.3, 1.0, 0.5, 'Green', 'Yellow', 13.2, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 797, 42, nan, 1164498.2179999999, 1871257.706, 41.80234381, -87.67224309, 61, 'NEW CITY', 16, 9, '(41.80234381, -87.67224309)') ,\n(610102, 'Richard J Oglesby Elementary School', 'ES', '7646 S Green St', 'Chicago', 'IL', 60620, '(773) 535-3060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610102.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 44.0, 'Average', 40.0, 'Average', 49.0, 'Average', 49.0, 'Weak', 43.0, 'Weak', 32.0, 'Weak', 46.0, 'Weak', 46.0, '94.40%', 25.4, '96.90%', '100.00%', nan, nan, 26.9, 18.8, 46.4, 48.2, 32.5, 30.7, 58.1, 56.7, 14.3, 22.9, 5.1, 7.5, 0.7, 0.3, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 402, 45, nan, 1171926.914, 1853950.5359999998, 41.75469091, -87.64550666, 71, 'AUBURN GRESHAM', 17, 6, '(41.75469091, -87.64550666)') ,\n(609702, 'Richard T Crane Technical Preparatory High School', 'HS', '2245 W Jackson Blvd', 'Chicago', 'IL', 60612, '(773) 534-7550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609702.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 43.0, nan, nan, 'Average', 50.0, 'Average', 48.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Weak', 46.0, '57.90%', 19.9, '93.90%', '99.10%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.0, 11.9, 13.2, 13.6, 1.6, 14.0, 0.8, 3.7, 48.0, 32.6, 478, 38, 40.9, 1161290.138, 1898611.918, 41.87747384, -87.68324922, 28, 'NEAR WEST SIDE', 2, 12, '(41.87747384, -87.68324922)') ,\n(610234, 'Richard Yates Elementary School', 'ES', '1839 N Richmond St', 'Chicago', 'IL', 60647, '(773) 534-4550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610234.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, nan, nan, 'Strong', 66.0, 'Strong', 73.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Strong', 55.0, '93.70%', 5.6, '96.20%', '98.60%', 60.4, 55.8, 41.3, 18.0, 42.9, 53.5, 42.2, 20.2, 41.3, 51.7, 21.8, 9.1, 15.2, 5.4, -0.2, -1.0, 'Yellow', 'Red', 28.6, 77.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 690, 34, nan, 1156561.0829999999, 1912262.2580000001, 41.91502858, -87.70024324, 22, 'LOGAN SQUARE', 35, 14, '(41.91502858, -87.70024324)') ,\n(609695, 'Roald Amundsen High School', 'HS', '5110 N Damen Ave', 'Chicago', 'IL', 60625, '(773) 534-2320', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609695.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 51.0, 'Average', 44.0, 'Average', 43.0, 'Average', 42.0, 'Weak', 27.0, 'Average', 41.0, 'Weak', 43.0, 'Weak', 41.0, '81.10%', 8.7, '95.90%', '99.20%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.8, 13.9, 14.9, 14.8, 1.0, 16.7, 1.8, 16.0, 64.6, 49.1, 1532, 31, 72.2, 1162034.1570000001, 1934187.196, 41.97507922, -87.67952139, 4, 'LINCOLN SQUARE', 47, 20, '(41.97507922, -87.67952139)') ,\n(610350, 'Robert A Black Magnet Elementary School', 'ES', '9101 S Euclid Ave', 'Chicago', 'IL', 60619, '(773) 535-6390', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610350.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 68.0, nan, nan, 'Strong', 63.0, 'Average', 58.0, 'Weak', nan, nan, nan, 'Strong', 55.0, 'Weak', 46.0, '96.00%', 0.4, '93.70%', '100.00%', nan, nan, 46.1, 59.9, 61.7, 73.1, 37.4, 58.4, 54.0, 55.4, 18.5, 33.3, 20.1, 21.0, 0.0, 0.0, 'Yellow', 'Yellow', 20.4, 36.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 474, 47, nan, 1190807.321, 1844986.0380000002, 41.72965645, -87.57660534, 48, 'CALUMET HEIGHTS', 8, 4, '(41.72965645, -87.57660534)') ,\n(609906, 'Robert Emmet Elementary School', 'ES', '5500 W Madison St', 'Chicago', 'IL', 60644, '(773) 534-6050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609906.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 37.0, nan, nan, 'Strong', 64.0, 'Strong', 71.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Strong', 54.0, '89.80%', 30.1, '94.80%', '100.00%', 68.1, 67.4, 28.8, 17.0, 48.0, 67.3, 15.1, 27.4, 56.9, 63.4, 8.9, 20.0, 12.9, 13.3, 1.0, 1.2, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 460, 36, nan, 1139489.278, 1899514.7030000002, 41.88037642, -87.76327497, 25, 'AUSTIN', 29, 15, '(41.88037642, -87.76327497)') ,\n(609929, 'Robert Fulton Elementary School', 'ES', '5300 S Hermitage Ave', 'Chicago', 'IL', 60609, '(773) 535-9000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609929.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 46.0, 'Average', 41.0, 'Strong', 63.0, 'Strong', 63.0, 'Weak', 53.0, 'Weak', 39.0, 'Weak', 45.0, 'Average', 49.0, '95.10%', 10.1, '96.80%', '100.00%', 40.7, nan, 17.9, 10.3, 41.2, 40.9, 50.4, 7.9, 70.8, 49.6, 10.6, 6.4, 9.6, 1.1, 0.9, -0.9, 'Green', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 466, 43, nan, 1165540.244, 1869516.208, 41.79754288, -87.66847093, 61, 'NEW CITY', 16, 9, '(41.79754288, -87.66847093)') ,\n(609979, 'Robert Healy Elementary School', 'ES', '3010 S Parnell Ave', 'Chicago', 'IL', 60616, '(773) 534-9190', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609979.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 63.0, nan, nan, 'Average', 54.0, 'Weak', 36.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '96.60%', 11.3, '97.10%', '99.10%', 84.0, nan, 72.1, 62.8, 63.4, 80.0, 64.3, 68.5, 61.4, 69.3, 38.6, 52.9, 45.1, 29.9, -0.3, 0.6, 'Yellow', 'Green', 18.9, 80.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1384, 40, nan, 1173077.2920000001, 1884891.36, 41.83957045, -87.64037683, 60, 'BRIDGEPORT', 11, 9, '(41.83957045, -87.64037683)') ,\n(610045, 'Robert H Lawrence Elementary School', 'ES', '9928 S Crandon Ave', 'Chicago', 'IL', 60617, '(773) 535-6320', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610045.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 19.0, 'Weak', 29.0, 'Average', 40.0, 'Average', 51.0, 'Weak', 35.0, 'Weak', 30.0, nan, nan, nan, nan, '90.70%', 14.0, '95.30%', '100.00%', 56.7, 35.9, 9.0, 14.6, 41.2, 31.0, 8.0, 15.8, 37.3, 40.6, 5.3, 15.8, 2.7, 3.4, -1.4, -1.3, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 371, 47, nan, 1193433.0520000001, 1839450.176, 41.71440176, -87.56716721, 51, 'SOUTH DEERING', 7, 4, '(41.71440176, -87.56716721)') ,\n(609956, 'Robert L Grimes Elementary School', 'ES', '5450 W 64th Pl', 'Chicago', 'IL', 60638, '(773) 535-2364', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609956.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 79.0, 'Strong', 64.0, 'Average', 56.0, 'Average', 59.0, 'Weak', 72.0, 'Strong', 71.0, 'Average', 53.0, 'Average', 49.0, '95.20%', 5.0, '95.60%', '100.00%', 58.8, 39.4, 37.4, 39.7, 50.7, 50.3, 45.5, 57.3, 51.0, 58.7, 17.4, 32.6, 22.1, 18.8, -0.4, 0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 486, 44, nan, 1140971.91, 1861303.67, 41.77549238, -87.75876919, 64, 'CLEARING', 13, 8, '(41.77549238, -87.75876919)') ,\n(610391, 'Robert Lindblom Math & Science Academy High School', 'HS', '6130 S Wolcott Ave', 'Chicago', 'IL', 60636, '(773) 535-9300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610391.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, nan, nan, 'Strong', 75.0, 'Very Strong', 85.0, 'Weak', nan, nan, nan, 'Strong', 55.0, 'Average', 51.0, '92.70%', 2.1, '95.90%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 49.1, 70.2, 61.7, 37.6, -1.2, -0.8, 'Red', 'Yellow', 61.1, 78.2, 18.0, 18.5, 19.1, 19.0, 1.0, 22.2, 3.1, 29.1, 68.9, 85.9, 916, 43, 83.3, 1164703.614, 1863861.294, 41.78204279999999, -87.67169858, 67, 'WEST ENGLEWOOD', 15, 7, '(41.7820428, -87.67169858)') ,\n(610252, 'Robert Nathaniel Dett Elementary School', 'ES', '2306 W Maypole Ave', 'Chicago', 'IL', 60612, '(773) 534-7160', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610252.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 48.0, nan, nan, 'Average', 51.0, 'Strong', 65.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Strong', 57.0, '94.40%', 6.9, '94.80%', '100.00%', 84.3, 58.8, 29.4, 16.2, 37.9, 55.4, 31.4, 26.7, 56.3, 29.6, 8.0, 12.0, 24.4, 15.4, 0.4, 1.0, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 250, 35, nan, 1160870.308, 1900992.919, 41.88401623, -87.68472465, 28, 'NEAR WEST SIDE', 2, 13, '(41.88401623, -87.68472465)') ,\n(609759, 'Roberto Clemente Community Academy High School', 'HS', '1147 N Western Ave', 'Chicago', 'IL', 60622, '(773) 534-4000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609759.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 44.0, 'Weak', 31.0, 'Weak', 39.0, 'Weak', 34.0, 'Weak', 28.0, 'Weak', 25.0, nan, nan, nan, nan, '69.60%', 20.6, '94.20%', '99.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.6, 12.3, 14.0, 13.3, 0.7, 15.4, 1.4, 12.5, 49.5, 41.5, 1016, 35, 58.9, 1160235.206, 1907768.8769999999, 41.90262318, -87.68686934, 24, 'WEST TOWN', 1, 13, '(41.90262318, -87.68686934)') ,\n(610029, 'Rodolfo Lozano Bilingual & International Center Elementary School', 'ES', '1501 N Greenview Ave', 'Chicago', 'IL', 60622, '(773) 534-4750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610029.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, nan, nan, 'Weak', 39.0, 'Weak', 37.0, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Average', 53.0, '94.80%', 0.9, '96.20%', '100.00%', 70.0, 38.7, 31.0, 29.5, 44.7, 54.3, 38.9, 34.7, 55.6, 50.0, 15.0, 20.0, 9.3, 11.3, -0.4, -0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 319, 35, nan, 1166155.136, 1910106.654, 41.90891372, -87.66505758, 24, 'WEST TOWN', 32, 14, '(41.90891372, -87.66505758)') ,\n(609733, 'Roger C Sullivan High School', 'HS', '6631 N Bosworth Ave', 'Chicago', 'IL', 60626, '(773) 534-2000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609733.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 30.0, 'Average', 44.0, 'Weak', 34.0, 'Weak', 34.0, 'Weak', 42.0, 'Weak', 33.0, 'Weak', 44.0, 'Weak', 45.0, '81.60%', 14.7, '95.80%', '94.40%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.6, 13.2, 13.8, 13.5, 0.9, 15.4, 1.6, 14.8, 39.3, 52.1, 826, 32, 68.6, 1164774.883, 1944269.292, 42.00268694, -87.66915533, 1, 'ROGERS PARK', 40, 24, '(42.00268694, -87.66915533)') ,\n(610091, 'Ronald Brown Elementary Community Academy', 'ES', '12607 S Union Ave', 'Chicago', 'IL', 60628, '(773) 535-5385', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610091.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 31.0, 'Weak', 26.0, 'Average', 52.0, 'Weak', 33.0, 'Weak', 48.0, 'Average', 45.0, 'Average', 52.0, 'Strong', 55.0, '93.80%', 35.6, '96.30%', '98.10%', 62.7, 32.0, 28.0, 33.7, 57.1, 47.4, 37.9, 33.0, 62.4, 66.7, 4.0, 32.0, 13.4, 12.5, 0.6, 1.4, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 283, 48, nan, 1173988.072, 1821226.195, 41.66484509, -87.63892027, 53, 'WEST PULLMAN', 9, 5, '(41.66484509, -87.63892027)') ,\n(610282, 'Ronald E McNair Elementary School', 'ES', '4820 W Walton St', 'Chicago', 'IL', 60651, '(773) 534-8980', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610282.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Weak', 30.0, 'Average', 56.0, 'Average', 59.0, 'Weak', 32.0, 'Weak', 20.0, 'Average', 47.0, 'Average', 53.0, '90.70%', 52.8, '96.50%', '100.00%', 65.4, 47.7, 36.6, 27.0, 47.0, 55.1, 36.1, 26.8, 45.7, 54.9, 4.5, 9.1, 9.8, 5.6, -1.2, -1.7, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 422, 36, nan, 1143946.854, 1905911.072, 41.8978464, -87.74674645, 25, 'AUSTIN', 37, 15, '(41.8978464, -87.74674645)') ,\n(609826, 'Rosario Castellanos Elementary School', 'ES', '2524 S Central Park Ave', 'Chicago', 'IL', 60623, '(773) 534-1620', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609826.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 36.0, 'Weak', 24.0, 'Weak', 34.0, 'Weak', 20.0, 'Weak', 17.0, 'Weak', 31.0, 'Weak', 46.0, 'Average', 48.0, '96.30%', 24.3, '96.90%', '100.00%', nan, nan, 34.8, 18.8, 49.8, 58.5, 33.3, 17.2, 61.0, 48.5, 7.5, 15.1, 8.0, 6.2, -0.8, -0.6, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 525, 37, nan, 1152737.279, 1886876.128, 41.84544296, -87.71496352, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84544296, -87.71496352)') ,\n(610056, 'Roswell B Mason Elementary School', 'ES', '4217 W 18th St', 'Chicago', 'IL', 60623, '(773) 534-1530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610056.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 37.0, 'Average', 47.0, 'Average', 44.0, 'Average', 58.0, 'Weak', 50.0, 'Weak', 36.0, 'Average', 51.0, 'Strong', 54.0, '92.40%', 31.0, '95.00%', '94.50%', 48.4, nan, 8.1, 20.3, 34.2, 31.2, 12.9, 28.4, 24.3, 39.8, 5.7, 26.9, 4.2, 5.8, -2.1, -0.8, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 479, 36, nan, 1148477.255, 1891003.4409999999, 41.85685194, -87.73049117, 29, 'NORTH LAWNDALE', 24, 10, '(41.85685194, -87.73049117)') ,\n(610027, 'Rudyard Kipling Elementary School', 'ES', '9351 S Lowe Ave', 'Chicago', 'IL', 60620, '(773) 535-3151', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610027.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 52.0, nan, nan, 'Average', 42.0, 'Average', 42.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Average', 51.0, '94.70%', 23.4, '96.00%', '97.80%', 76.2, 52.5, 49.7, 44.4, 64.5, 58.2, 51.6, 46.3, 57.1, 50.3, 13.7, 31.4, 20.9, 21.2, -0.6, 0.2, 'Red', 'Yellow', 16.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 445, 49, nan, 1173661.215, 1842654.148, 41.72365391, -87.63948475, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.72365391, -87.63948475)') ,\n(610250, 'Rueben Salazar Elementary Bilingual Center', 'ES', '160 W Wendell St', 'Chicago', 'IL', 60610, '(773) 534-8310', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610250.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 64.0, nan, nan, 'Average', 54.0, 'Average', 53.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Average', 47.0, '95.10%', 44.6, '95.80%', '100.00%', 68.3, 33.1, 61.7, 50.8, 53.3, 75.7, 43.6, 60.2, 54.3, 65.6, 22.6, 64.5, 20.5, 15.5, 0.9, 0.3, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 444, 33, nan, 1174702.288, 1907452.8190000001, 41.90144445, -87.63373912, 8, 'NEAR NORTH SIDE', 43, 18, '(41.90144445, -87.63373912)') ,\n(609995, 'Rufus M Hitch Elementary School', 'ES', '5625 N McVicker Ave', 'Chicago', 'IL', 60646, '(773) 534-1189', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609995.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'Yes', 'Strong', 64.0, 'Strong', 77.0, 'Average', 55.0, 'Average', 58.0, 'Weak', 65.0, 'Strong', 70.0, 'Strong', 55.0, 'Strong', 54.0, '95.10%', 3.7, '96.60%', '98.60%', 73.7, 27.6, 62.2, 51.7, 50.0, 67.8, 56.5, 54.8, 52.0, 63.3, 22.9, 37.5, 29.5, 24.7, -0.1, 0.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 542, 30, nan, 1134940.195, 1937129.571, 41.98367756, -87.77908614, 10, 'NORWOOD PARK', 45, 16, '(41.98367756, -87.77908614)') ,\n(609853, 'Salmon P Chase Elementary School', 'ES', '2021 N Point St', 'Chicago', 'IL', 60647, '(773) 534-4185', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609853.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, 'Weak', 28.0, 'Average', 59.0, 'Weak', 39.0, 'Weak', 25.0, 'Weak', 27.0, nan, nan, nan, nan, '95.40%', 18.5, '95.80%', '98.60%', 63.0, 26.7, 40.6, 25.0, 57.3, 72.1, 47.1, 36.8, 70.2, 56.8, 23.1, 23.1, 17.1, 11.0, -0.6, 0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 504, 35, nan, 1158104.87, 1913362.1619999998, 41.91801539, -87.69454144, 22, 'LOGAN SQUARE', 1, 14, '(41.91801539, -87.69454144)') ,\n(609943, 'Samuel Gompers Fine Arts Options Elementary School', 'ES', '12302 S State St', 'Chicago', 'IL', 60628, '(773) 535-5475', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609943.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 34.0, 'Average', 42.0, 'Average', 48.0, 'Average', 52.0, 'Weak', 57.0, 'Average', 59.0, 'Average', 49.0, 'Average', 47.0, '93.60%', 54.2, '96.00%', '100.00%', nan, nan, 36.9, 30.1, 57.9, 56.5, 36.8, 29.5, 67.5, 56.3, 3.9, 15.7, 13.5, 8.5, 0.4, -0.7, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 285, 48, nan, 1178435.326, 1823353.395, 41.67058298, -87.62258098, 53, 'WEST PULLMAN', 9, 5, '(41.67058298, -87.62258098)') ,\n(610155, 'Sauganash Elementary School', 'ES', '6040 N Kilpatrick Ave', 'Chicago', 'IL', 60646, '(773) 534-3470', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610155.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, nan, nan, 'Strong', 76.0, 'Strong', 61.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '96.30%', 1.5, '95.40%', '92.90%', 85.6, 50.4, 68.5, 75.7, 66.9, 70.2, 81.4, 80.1, 58.7, 57.8, 57.1, 69.6, 49.1, 34.5, 2.2, 0.3, 'Green', 'Yellow', 35.7, 78.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 549, 31, nan, 1143919.392, 1939921.031, 41.99117332, -87.74599137, 12, 'FOREST GLEN', 39, 17, '(41.99117332, -87.74599137)') ,\n(609805, 'Scott Joplin Elementary School', 'ES', '7931 S Honore St', 'Chicago', 'IL', 60620, '(773) 535-3425', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609805.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 35.0, 'Weak', 29.0, 'Average', 44.0, 'Average', 53.0, 'Weak', 45.0, 'Weak', 34.0, 'Average', 52.0, 'Average', 50.0, '95.00%', 44.5, '94.60%', '97.70%', nan, nan, 27.8, 27.4, 66.9, 49.4, 33.1, 29.6, 49.7, 61.6, 11.8, 31.4, 9.8, 12.2, 0.2, 0.7, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 538, 43, nan, 1165441.875, 1851949.948, 41.74934082, -87.66932903, 71, 'AUBURN GRESHAM', 18, 6, '(41.74934082, -87.66932903)') ,\n(609968, 'Sharon Christa McAuliffe Elementary School', 'ES', '1841 N Springfield Ave', 'Chicago', 'IL', 60647, '(773) 534-4400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609968.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'Yes', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 52.0, 'Strong', 56.0, '93.90%', 14.2, '95.70%', '98.90%', 64.7, 25.1, 34.4, 24.4, 39.5, 56.1, 27.5, 21.6, 37.5, 44.7, nan, nan, 10.4, 11.0, -1.0, -0.9, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 862, 34, nan, 1150124.467, 1912135.0019999999, 41.91480732, -87.72389409, 20, 'HERMOSA', 30, 25, '(41.91480732, -87.72389409)') ,\n(610157, 'Sidney Sawyer Elementary School', 'ES', '5248 S Sawyer Ave', 'Chicago', 'IL', 60632, '(773) 535-0440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610157.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 57.0, 'Weak', 30.0, 'Average', 41.0, 'Weak', 39.0, 'Weak', 24.0, 'Weak', 29.0, 'Average', 49.0, 'Average', 52.0, '96.80%', 7.4, '96.40%', '100.00%', 67.8, nan, 37.0, 34.9, 58.5, 65.8, 36.8, 44.2, 57.5, 54.1, 19.1, 27.0, 18.7, 12.7, -0.4, 0.4, 'Red', 'Yellow', 17.6, 82.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1846, 44, nan, 1155567.996, 1869433.427, 41.79752155, -87.70504311, 63, 'GAGE PARK', 14, 8, '(41.79752155, -87.70504311)') ,\n(610283, 'Simon Guggenheim Elementary School', 'ES', '7141 S Morgan St', 'Chicago', 'IL', 60621, '(773) 535-3587', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610283.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 30.0, nan, nan, 'Weak', 38.0, 'Weak', 22.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 47.0, '90.90%', 29.5, '93.10%', '100.00%', nan, nan, 3.5, 9.8, 33.3, 31.6, 16.2, 15.3, 38.6, 40.9, 4.5, 13.6, 4.2, 6.6, -1.7, 0.9, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 291, 45, nan, 1170922.983, 1857302.22, 41.76391032, -87.64908807, 68, 'ENGLEWOOD', 17, 7, '(41.76391032, -87.64908807)') ,\n(610521, 'Sir Miles Davis Magnet Elementary Academy', 'ES', '6730 S Paulina St', 'Chicago', 'IL', 60636, '(773) 535-9120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610521.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 35.0, nan, nan, 'Average', 51.0, 'Average', 54.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '93.00%', 57.8, '94.50%', '100.00%', 63.9, 62.3, 24.8, 14.1, 43.1, 52.0, 22.6, 17.1, 49.1, 45.5, 4.4, 13.3, 5.9, 3.2, 0.0, -0.9, 'Yellow', 'Red', 25.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 354, 43, nan, 1166133.893, 1859929.0180000002, 41.77122181, -87.66656657, 67, 'WEST ENGLEWOOD', 15, 7, '(41.77122181, -87.66656657)') ,\n(610534, 'Skinner North', 'ES', '640 W Scott St', 'Chicago', 'IL', 60610, '(773) 534-8500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610534.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Strong', 64.0, 'Average', 52.0, '96.40%', 0.0, '97.80%', '100.00%', 83.3, 83.0, 100.0, 100.0, 74.1, 76.0, nan, nan, nan, nan, nan, nan, 100.0, 100.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 242, 33, nan, 1171670.9740000002, 1908773.84, 41.9051367, -87.64483441, 8, 'NEAR NORTH SIDE', 27, 18, '(41.9051367, -87.64483441)') ,\n(610226, 'Socorro Sandoval Elementary School', 'ES', '5534 S Saint Louis Ave', 'Chicago', 'IL', 60629, '(773) 535-0457', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610226.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Weak', 44.0, 'Average', 50.0, '95.80%', 3.7, '96.50%', '98.90%', 77.3, 47.9, 44.1, 23.0, 53.0, 61.5, nan, nan, nan, nan, nan, nan, 15.9, 9.1, -0.5, -0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1186, 44, nan, 1153950.752, 1867537.733, 41.79235177, -87.71102417, 63, 'GAGE PARK', 14, 8, '(41.79235177, -87.71102417)') ,\n(610160, 'Songhai Elementary Learning Institute', 'ES', '11725 S Perry Ave', 'Chicago', 'IL', 60628, '(773) 535-5547', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610160.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 17.0, nan, nan, 'Average', 47.0, 'Average', 54.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Weak', 46.0, '89.50%', 84.7, '95.40%', '100.00%', 24.8, nan, 13.8, 16.3, 35.8, 36.8, 25.6, 22.5, 70.0, 67.2, 7.4, 25.9, 3.6, 3.6, 0.1, -1.4, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 352, 48, nan, 1177727.869, 1827095.66, 41.68086831, -87.62505762, 53, 'WEST PULLMAN', 34, 5, '(41.68086831, -87.62505762)') ,\n(609990, 'South Loop Elementary School', 'ES', '1212 S Plymouth Ct', 'Chicago', 'IL', 60605, '(773) 534-8690', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609990.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 84.0, 'Strong', 66.0, 'Weak', 34.0, 'Weak', 36.0, 'Weak', 57.0, 'Average', 46.0, 'Strong', 54.0, 'Weak', 46.0, '95.30%', 12.7, '97.20%', '100.00%', 89.0, 70.1, 81.7, 80.1, 63.5, 66.8, 72.0, 68.7, 70.9, 71.6, 27.3, 40.9, 54.2, 48.8, 1.0, 1.5, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 795, 38, nan, 1176190.878, 1894860.267, 41.86685634, -87.62865124, 33, 'NEAR SOUTH SIDE', 2, 1, '(41.86685634, -87.62865124)') ,\n(610530, 'South Shore Fine Arts Academy', 'ES', '1415 E 70th St', 'Chicago', 'IL', 60637, '(773) 535-8340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610530.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Strong', 56.0, 'Strong', 55.0, '93.60%', 22.4, '96.30%', '100.00%', 77.2, 46.5, 29.2, 20.8, 40.0, 54.5, nan, nan, nan, nan, nan, nan, 20.8, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 255, 46, nan, 1186924.7859999998, 1858826.102, 41.76772772, -87.59039017, 43, 'SOUTH SHORE', 5, 3, '(41.76772772, -87.59039017)') ,\n(609745, 'Southside Occupational Academy High School', 'HS', '7342 S Hoyne Ave', 'Chicago', 'IL', 60636, '(773) 535-9100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609745.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', nan, 'Track_E', 'Not Applicable', 'Not Enough Data', 'No', nan, nan, 'Average', 52.0, nan, nan, nan, nan, 'Weak', 47.0, 'Strong', 60.0, 'Strong', 63.0, 'Strong', 60.0, '85.50%', 13.1, '95.20%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 40.0, 71.7, nan, 201, 43, nan, 1163596.617, 1855772.0590000001, 41.75986813, -87.6759838, 67, 'WEST ENGLEWOOD', 17, 7, '(41.75986813, -87.6759838)') ,\n(610183, 'Spencer Technology Academy', 'ES', '214 N Lavergne Ave', 'Chicago', 'IL', 60644, '(773) 534-6150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610183.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, 'Average', 50.0, 'Average', 57.0, 'Strong', 71.0, 'Weak', 66.0, 'Average', 56.0, 'Average', 50.0, 'Strong', 55.0, '92.50%', 24.1, '96.00%', '100.00%', 55.3, 44.9, 26.1, 21.5, 56.0, 50.0, 36.0, 29.4, 56.6, 61.4, 9.4, 21.9, 10.7, 11.0, 0.7, 1.7, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 811, 36, nan, 1142962.029, 1901180.1369999999, 41.88488257, -87.75048169, 25, 'AUSTIN', 28, 15, '(41.88488257, -87.75048169)') ,\n(610357, 'Spry Community Links High School', 'HS', '2400 S Marshall Blvd', 'Chicago', 'IL', 60623, '(773) 534-1997', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610357.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 70.0, nan, nan, 'Very Strong', 92.0, 'Very Strong', 87.0, 'Weak', nan, nan, nan, 'Strong', 56.0, 'Average', 51.0, '95.60%', 9.7, '95.70%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.3, 14.6, 15.5, 15.9, 1.6, 16.9, 1.4, 35.0, 73.7, 65.2, 205, 39, 92.8, 1157044.9109999998, 1887895.675, 41.84815451, -87.69912725, 30, 'SOUTH LAWNDALE', 12, 10, '(41.84815451, -87.69912725)') ,\n(609880, 'Stephen Decatur Classical Elementary School', 'ES', '7030 N Sacramento Ave', 'Chicago', 'IL', 60645, '(773) 534-2201', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609880.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Weak', 99.0, 'Very Strong', 99.0, 'Average', 52.0, 'Weak', 46.0, '96.20%', 2.8, '95.40%', '100.00%', 100.0, 97.2, 98.6, 98.6, 50.7, 61.3, 100.0, 100.0, 48.1, 78.6, nan, nan, 93.6, 93.6, 0.2, -1.0, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 289, 32, nan, 1155108.239, 1946608.822, 42.00930696, -87.70465502, 2, 'WEST RIDGE', 50, 24, '(42.00930696, -87.70465502)') ,\n(609933, 'Stephen F Gale Elementary Community Academy', 'ES', '1631 W Jonquil Ter', 'Chicago', 'IL', 60626, '(773) 534-2100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609933.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 39.0, 'Weak', 31.0, 'Weak', 35.0, 'Weak', 27.0, 'Weak', 33.0, 'Weak', 29.0, 'Weak', 45.0, 'Weak', 43.0, '92.10%', 34.8, '95.20%', '100.00%', 40.1, 15.5, 18.7, 17.9, 49.3, 39.3, 32.4, 24.8, 46.2, 48.9, 7.5, 22.5, 7.6, 6.9, -0.5, 0.0, 'Yellow', 'Yellow', 20.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 532, 32, nan, 1163959.517, 1950960.205, 42.02106425, -87.67196481, 1, 'ROGERS PARK', 49, 24, '(42.02106425, -87.67196481)') ,\n(609976, 'Stephen K Hayt Elementary School', 'ES', '1518 W Granville Ave', 'Chicago', 'IL', 60660, '(773) 534-2040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609976.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 72.0, 'Strong', 61.0, 'Average', 52.0, 'Average', 44.0, 'Weak', 48.0, 'Strong', 66.0, nan, nan, nan, nan, '95.60%', 55.6, '96.00%', '99.30%', 78.2, 59.2, 50.8, 46.9, 56.9, 66.5, 55.5, 46.3, 61.5, 57.5, 26.7, 39.5, 28.6, 23.2, 0.3, 0.1, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1000, 32, nan, 1164969.8020000001, 1941312.085, 41.99456814, -87.66852270000001, 77, 'EDGEWATER', 40, 24, '(41.99456814, -87.6685227)') ,\n(609724, 'Stephen T Mather High School', 'HS', '5835 N Lincoln Ave', 'Chicago', 'IL', 60659, '(773) 534-2350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609724.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, 'Average', 44.0, 'Average', 48.0, 'Average', 47.0, 'Weak', 50.0, 'Average', 56.0, nan, nan, nan, nan, '87.60%', 7.6, '96.50%', '97.90%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 14.5, 14.4, 15.0, 15.4, 0.9, 15.9, 0.9, 24.6, 56.2, 57.4, 1657, 32, 59.0, 1155766.152, 1938701.409, 41.9875954, -87.70244871, 2, 'WEST RIDGE', 40, 20, '(41.9875954, -87.70244871)') ,\n(610191, 'Stone Elementary Scholastic Academy', 'ES', '6239 N Leavitt St', 'Chicago', 'IL', 60659, '(773) 534-2045', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610191.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 78.0, nan, nan, 'Average', 55.0, 'Average', 43.0, 'Weak', nan, nan, nan, 'Strong', 55.0, 'Average', 47.0, '96.40%', 1.9, '96.00%', '100.00%', 83.2, 46.2, 62.8, 67.5, 59.9, 56.7, 62.0, 68.8, 59.9, 59.4, 40.0, 47.3, 45.7, 44.7, 0.3, 1.7, 'Yellow', 'Green', 57.1, 56.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 646, 32, nan, 1160554.673, 1941569.632, 41.99536777, -87.68475636, 2, 'WEST RIDGE', 50, 24, '(41.99536777, -87.68475636)') ,\n(610405, 'Suder Montessori Magnet Elementary School', 'ES', '2022 W Washington Blvd', 'Chicago', 'IL', 60612, '(773) 534-7685', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610405.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, nan, nan, nan, nan, '96.60%', 6.5, '95.70%', '100.00%', 82.2, 60.7, 61.3, 50.9, 58.0, 59.5, nan, nan, nan, nan, nan, nan, 40.7, 29.2, 0.7, -0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 351, 38, nan, 1162799.297, 1900725.344, 41.88324176, -87.67764871, 28, 'NEAR WEST SIDE', 2, 13, '(41.88324176, -87.67764871)') ,\n(610249, 'Talman Elementary School', 'ES', '5450 S Talman Ave', 'Chicago', 'IL', 60632, '(773) 535-7850', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610249.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Weak', 85.0, 'Very Strong', 81.0, 'Strong', 57.0, 'Average', 48.0, '96.20%', 9.2, '95.50%', '100.00%', 50.6, nan, 54.7, 22.6, 71.6, 75.0, 56.0, 42.2, 76.7, 62.8, 11.1, 25.9, 27.3, 16.5, 0.5, 0.7, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 376, 43, nan, 1159597.93, 1868206.8290000001, 41.79407382, -87.6902983, 63, 'GAGE PARK', 14, 9, '(41.79407382, -87.6902983)') ,\n(610396, 'Tarkington School of Excellence Elementary School', 'ES', '3330 W 71st St', 'Chicago', 'IL', 60629, '(773) 535-4700', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610396.pdf', 'AUSL Schools', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 48.0, nan, nan, 'Weak', 39.0, 'Average', 40.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Strong', 56.0, '95.90%', 10.5, '96.60%', '100.00%', nan, nan, 36.3, 34.5, 56.0, 53.7, 30.6, 36.7, 53.4, 56.1, 6.3, 10.7, 13.4, 9.7, -0.9, -0.2, 'Red', 'Yellow', 26.1, 53.3, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1070, 44, nan, 1155280.2, 1857374.1369999999, 41.76443483, -87.70642093, 66, 'CHICAGO LAWN', 18, 8, '(41.76443483, -87.70642093)') ,\n(610506, 'TEAM Englewood Community Academy High School', 'HS', '6201 S Stewart Ave', 'Chicago', 'IL', 60621, '(773) 535-3530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610506.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 45.0, 'Average', 48.0, 'Average', 55.0, 'Average', 53.0, 'Weak', 74.0, 'Strong', 68.0, 'Strong', 58.0, 'Average', 49.0, '87.30%', 8.2, '96.50%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.0, 12.4, 14.3, 13.3, 1.3, 15.3, 1.0, 7.9, nan, nan, 440, 45, 77.4, 1174730.35, 1863740.125, 41.7814927, -87.63494163, 68, 'ENGLEWOOD', 20, 7, '(41.7814927, -87.63494163)') ,\n(609921, 'Telpochcalli Elementary School', 'ES', '2832 W 24th Blvd', 'Chicago', 'IL', 60623, '(773) 534-1402', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609921.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 42.0, 'Average', 48.0, 'Very Weak', 19.0, 'Weak', 22.0, 'Weak', 82.0, 'Strong', 69.0, 'Strong', 55.0, 'Average', 51.0, '95.40%', 2.7, '94.70%', '96.40%', 31.3, nan, 31.8, 13.6, 50.8, 66.2, 30.8, 25.4, 51.6, 54.4, 4.5, 18.2, 8.4, 7.3, -0.1, 0.9, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 285, 39, nan, 1157725.352, 1887836.509, 41.84797834, -87.6966316, 30, 'SOUTH LAWNDALE', 12, 10, '(41.84797834, -87.6966316)') ,\n(609991, 'Theodore Herzl Elementary School', 'ES', '3711 W Douglas Blvd', 'Chicago', 'IL', 60623, '(773) 534-1480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609991.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 41.0, nan, nan, 'Average', 53.0, 'Strong', 65.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '90.80%', 23.9, '97.00%', '96.00%', 62.6, 35.6, 13.5, 16.3, 63.3, 50.4, 22.1, 22.1, 65.7, 67.6, 0.0, 10.5, 4.6, 3.8, 0.1, 0.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 512, 36, nan, 1151795.4, 1892995.835, 41.86225473, -87.71825927, 29, 'NORTH LAWNDALE', 24, 10, '(41.86225473, -87.71825927)') ,\n(609728, 'Theodore Roosevelt High School', 'HS', '3436 W Wilson Ave', 'Chicago', 'IL', 60625, '(773) 534-5000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609728.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 42.0, nan, nan, 'Average', 42.0, 'Weak', 37.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '82.30%', 9.7, '96.10%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.9, 13.4, 14.3, 14.0, 1.1, 15.7, 1.4, 13.7, 59.4, 42.9, 1490, 31, 61.2, 1152459.778, 1930390.229, 41.96485522, -87.71483051, 14, 'ALBANY PARK', 33, 17, '(41.96485522, -87.71483051)') ,\n(610178, 'Theophilus Schmid Elementary School', 'ES', '9755 S Greenwood Ave', 'Chicago', 'IL', 60628, '(773) 535-6235', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610178.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 46.0, nan, nan, 'Weak', 31.0, 'Average', 53.0, 'Weak', nan, nan, nan, 'Weak', 41.0, 'Average', 52.0, '92.10%', 67.1, '96.30%', '100.00%', 62.8, 45.2, 35.8, 22.6, 51.9, 58.5, 47.1, 43.1, 56.3, 54.2, 13.3, 20.0, 17.1, 6.7, -0.4, -0.7, 'Yellow', 'Yellow', 43.8, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 185, 48, nan, 1185308.063, 1840332.575, 41.71701772, -87.59689622, 50, 'PULLMAN', 8, 5, '(41.71701772, -87.59689622)') ,\n(609794, 'Thomas A Edison Regional Gifted Center Elementary School', 'ES', '4929 N Sawyer Ave', 'Chicago', 'IL', 60625, '(773) 534-0540', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609794.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 91.0, nan, nan, 'Strong', 64.0, 'Average', 56.0, 'Weak', nan, nan, nan, 'Strong', 55.0, 'Strong', 57.0, '96.60%', 1.9, '96.30%', '100.00%', nan, nan, 92.0, 97.8, 52.8, 57.6, 93.1, 98.9, 55.2, 60.4, 80.0, 96.7, 79.0, 88.4, 1.8, 1.6, 'Green', 'Green', 31.8, 78.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 269, 31, nan, 1153858.196, 1932691.8909999998, 41.971143299999994, -87.70962725, 14, 'ALBANY PARK', 39, 17, '(41.9711433, -87.70962725)') ,\n(609987, 'Thomas A Hendricks Elementary Community Academy', 'ES', '4316 S Princeton Ave', 'Chicago', 'IL', 60609, '(773) 535-1696', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609987.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 33.0, nan, nan, 'Strong', 67.0, 'Strong', 78.0, 'Weak', nan, nan, nan, 'Average', 51.0, 'Average', 52.0, '91.20%', 61.8, '95.60%', '100.00%', 58.7, 60.0, 31.6, 12.1, 41.1, 50.0, 48.5, 16.7, 71.9, 54.9, 7.5, 32.5, 8.5, 1.0, 0.9, -2.3, 'Green', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 307, 42, nan, 1174966.903, 1876241.753, 41.81579314, -87.63370133, 37, 'FULLER PARK', 3, 9, '(41.81579314, -87.63370133)') ,\n(609851, 'Thomas Chalmers Specialty Elementary School', 'ES', '2745 W Roosevelt Rd', 'Chicago', 'IL', 60608, '(773) 534-1720', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609851.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 34.0, 'Strong', 76.0, 'Weak', 39.0, 'Average', 45.0, 'Weak', 88.0, 'Strong', 78.0, nan, nan, nan, nan, '94.40%', 3.4, '97.30%', '100.00%', 37.5, nan, 7.5, 10.3, 35.8, 12.8, 17.2, 21.9, 31.4, 57.6, 0.0, 12.5, 2.1, 4.9, -0.7, 1.2, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 288, 39, nan, 1158048.936, 1894554.926, 41.8664078, -87.69526079, 29, 'NORTH LAWNDALE', 28, 10, '(41.8664078, -87.69526079)') ,\n(609896, 'Thomas Drummond Elementary School', 'ES', '1845 W Cortland St', 'Chicago', 'IL', 60622, '(773) 534-4120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609896.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 72.0, nan, nan, 'Average', 59.0, 'Average', 46.0, 'Weak', nan, nan, nan, 'Strong', 58.0, 'Average', 48.0, '95.20%', 0.3, '96.70%', '97.50%', 71.3, nan, 68.6, 69.3, 66.3, 75.3, 37.7, 42.9, 43.3, 61.3, 0.0, 0.0, 34.0, 34.0, 0.0, 0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 352, 35, nan, 1163566.562, 1912627.608, 41.91588634, -87.67449563, 22, 'LOGAN SQUARE', 32, 14, '(41.91588634, -87.67449563)') ,\n(610002, 'Thomas Hoyne Elementary School', 'ES', '8905 S Crandon Ave', 'Chicago', 'IL', 60617, '(773) 535-6425', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610002.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, 'Strong', 66.0, 'Strong', 77.0, 'Strong', 73.0, 'Weak', 67.0, 'Strong', 66.0, nan, nan, nan, nan, '94.80%', 0.0, '95.40%', '100.00%', 40.2, nan, 36.2, 42.7, 56.3, 61.7, 54.0, 65.0, 50.0, 50.0, 14.6, 41.5, 23.4, 14.0, 0.7, 0.0, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 288, 47, nan, 1193092.436, 1846336.5459999999, 41.73330693, -87.5681905, 48, 'CALUMET HEIGHTS', 7, 4, '(41.73330693, -87.5681905)') ,\n(610295, 'Thomas J Higgins Elementary Community Academy', 'ES', '11710 S Morgan St', 'Chicago', 'IL', 60643, '(773) 535-5625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610295.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 49.0, 'Average', 53.0, '95.20%', 0.5, '95.20%', '100.00%', 62.0, 30.4, 35.7, 25.2, 58.3, 63.7, 43.6, 35.3, 63.8, 57.0, 9.1, 20.5, 19.1, 9.8, 1.7, 0.4, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 369, 49, nan, 1171725.3329999999, 1827097.573, 41.68100687, -87.64702985, 53, 'WEST PULLMAN', 34, 5, '(41.68100687, -87.64702985)') ,\n(610220, 'Thomas J Waters Elementary School', 'ES', '4540 N Campbell Ave', 'Chicago', 'IL', 60625, '(773) 534-5090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610220.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 85.0, nan, nan, 'Average', 56.0, 'Average', 54.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Average', 49.0, '96.10%', 0.4, '95.10%', '100.00%', 70.4, 31.2, 51.1, 59.0, 42.0, 44.2, 49.6, 54.7, 61.6, 50.8, 22.7, 61.4, 27.2, 28.5, 0.2, 0.6, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 590, 31, nan, 1158841.899, 1930252.459, 41.96434835, -87.69136881, 4, 'LINCOLN SQUARE', 47, 19, '(41.96434835, -87.69136881)') ,\n(609715, 'Thomas Kelly High School', 'HS', '4136 S California Ave', 'Chicago', 'IL', 60632, '(773) 535-4900', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609715.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 36.0, 'Average', 53.0, 'Weak', 36.0, 'Weak', 35.0, 'Weak', 61.0, 'Average', 57.0, nan, nan, nan, nan, '89.30%', 5.8, '95.30%', '99.70%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.4, 13.4, 15.1, 14.6, 1.2, 17.1, 2.0, 22.1, 67.4, 46.4, 2883, 39, 65.6, 1158337.7859999998, 1877175.576, 41.81871101, -87.69467483, 58, 'BRIGHTON PARK', 14, 9, '(41.81871101, -87.69467483)') ,\n(610321, 'Thurgood Marshall Middle School', 'MS', '3900 N Lawndale Ave', 'Chicago', 'IL', 60618, '(773) 534-5200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610321.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 39.0, nan, nan, 'Very Weak', 11.0, 'Very Weak', 1.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '94.00%', 18.1, '96.00%', '97.20%', nan, nan, nan, nan, nan, nan, 44.8, 35.7, 47.2, 50.9, 20.5, 19.9, 18.6, 4.0, -1.7, -2.2, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 435, 31, nan, 1150989.856, 1925703.541, 41.952023600000004, -87.72035833, 16, 'IRVING PARK', 39, 17, '(41.9520236, -87.72035833)') ,\n(609895, 'Turner-Drew Elementary Language Academy', 'ES', '9300 S Princeton Ave', 'Chicago', 'IL', 60620, '(773) 535-5720', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609895.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 56.0, 'Average', 56.0, 'Average', 48.0, 'Average', 51.0, 'Weak', 75.0, 'Strong', 66.0, nan, nan, nan, nan, '95.80%', 24.0, '95.20%', '100.00%', 80.2, 54.2, 46.3, 46.0, 54.5, 60.5, 68.8, 55.3, 57.9, 52.2, 18.4, 44.9, 24.2, 23.2, 0.1, 0.9, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 373, 48, nan, 1176188.712, 1843386.9319999998, 41.72560849, -87.6302048, 49, 'ROSELAND', 21, 6, '(41.72560849, -87.6302048)') ,\n(610394, 'Uplift Community High School', 'HS', '900 W Wilson Ave', 'Chicago', 'IL', 60640, '(773) 534-2875', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610394.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 50.0, 'Average', 50.0, 'Strong', 61.0, 'Strong', 62.0, 'Weak', 48.0, 'Average', 53.0, nan, nan, nan, nan, '88.70%', 8.1, '95.90%', '98.30%', nan, nan, nan, nan, nan, nan, 51.2, 34.9, 70.0, 29.3, 14.8, 14.8, 23.3, 2.3, 0.8, -0.2, 'Yellow', 'Yellow', nan, nan, 13.6, 13.9, 14.9, 14.7, 1.1, 16.9, 2.0, 19.6, 62.4, 65.0, 473, 32, 91.4, 1169403.4009999998, 1930780.685, 41.96557412, -87.65252191, 3, 'UPTOWN', 46, 19, '(41.96557412, -87.65252191)') ,\n(610504, 'Velma F Thomas Early Childhood Center', 'ES', '3625 S Hoyne Ave', 'Chicago', 'IL', 60609, '(773) 535-4088', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610504.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', nan, 'Standard', nan, nan, 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Strong', 55.0, 'Strong', 57.0, nan, 0.0, '95.60%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 211, 39, nan, 1162992.8229999999, 1880511.57, 41.82776914, -87.67750501, 59, 'MCKINLEY PARK', 11, 9, '(41.82776914, -87.67750501)') ,\n(609989, 'Victor Herbert Elementary School', 'ES', '2131 W Monroe St', 'Chicago', 'IL', 60612, '(773) 534-7806', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609989.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 50.0, 'Average', 52.0, 'Strong', 64.0, 'Strong', 72.0, 'Weak', 48.0, 'Average', 50.0, 'Average', 50.0, 'Average', 52.0, '92.80%', 10.3, '96.20%', '100.00%', 58.4, 26.0, 53.3, 34.9, 63.7, 83.5, 53.8, 29.0, 73.9, 58.1, 25.0, 25.0, 30.0, 10.3, 2.1, -0.8, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 398, 38, nan, 1162083.381, 1899481.0980000002, 41.87984242, -87.68031236, 28, 'NEAR WEST SIDE', 2, 12, '(41.87984242, -87.68031236)') ,\n(609944, 'Virgil Grissom Elementary School', 'ES', '12810 S Escanaba Ave', 'Chicago', 'IL', 60633, '(773) 535-5380', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609944.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 62.0, 'Strong', 66.0, 'Average', 51.0, 'Strong', 64.0, 'Weak', 69.0, 'Very Strong', 87.0, nan, nan, nan, nan, '95.50%', 2.5, '96.40%', '100.00%', 84.4, 51.6, 53.4, 47.7, 45.5, 60.2, 50.0, 45.7, 62.0, 54.3, 23.5, 44.1, 22.0, 17.7, -1.1, -1.5, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 330, 47, nan, 1197378.736, 1820556.064, 41.66245712, -87.55334334, 55, 'HEGEWISCH', 10, 4, '(41.66245712, -87.55334334)') ,\n(610518, 'VOISE Academy High School', 'HS', '231 N Pine Ave', 'Chicago', 'IL', 60644, '(773) 534-0660', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610518.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 42.0, nan, nan, 'Average', 52.0, 'Average', 52.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '84.50%', 25.0, '96.50%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.3, 12.5, 13.6, 13.6, 1.3, 14.4, 0.8, nan, nan, nan, 410, 36, 82.0, 1139494.763, 1901274.2580000001, 41.88520477, -87.76321191, 25, 'AUSTIN', 28, 15, '(41.88520477, -87.76321191)') ,\n(610363, 'Walt Disney Magnet Elementary School', 'ES', '4140 N Marine Dr', 'Chicago', 'IL', 60613, '(773) 534-5840', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610363.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, nan, nan, nan, nan, '96.40%', 7.7, '96.40%', '100.00%', nan, 53.8, 63.9, 64.9, 54.7, 64.1, 72.8, 69.6, 59.9, 51.0, 38.7, 53.3, 46.0, 35.7, 0.4, 0.6, 'Green', 'Green', 20.0, 74.1, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1651, 32, nan, 1171170.774, 1927599.86, 41.95680712, -87.6461175, 3, 'UPTOWN', 46, 19, '(41.95680712, -87.6461175)') ,\n(610095, 'Walter L Newberry Math & Science Academy Elementary School', 'ES', '700 W Willow St', 'Chicago', 'IL', 60614, '(773) 534-8000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610095.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 71.0, 'Strong', 74.0, 'Average', 52.0, 'Strong', 66.0, 'Weak', 41.0, 'Average', 44.0, 'Strong', 54.0, 'Average', 47.0, '95.10%', 2.1, '96.30%', '100.00%', nan, nan, 70.2, 61.5, 57.5, 66.9, 68.0, 65.5, 61.1, 59.3, 40.3, 43.5, 46.2, 35.9, 1.0, 1.3, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 558, 33, nan, 1171323.787, 1911991.711, 41.91397435, -87.64601501, 7, 'LINCOLN PARK', 43, 18, '(41.91397435, -87.64601501)') ,\n(609680, 'Walter Payton College Preparatory High School', 'HS', '1034 N Wells St', 'Chicago', 'IL', 60610, '(773) 534-0034', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609680.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 98.0, nan, nan, 'Very Strong', 80.0, 'Strong', 77.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '93.40%', 0.7, '96.10%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 21.2, 21.8, 23.1, 23.2, 2.0, 27.0, 3.9, 96.4, 96.9, 82.4, 881, 33, 90.7, 1174484.638, 1907490.082, 41.90155157, -87.63453744, 8, 'NEAR NORTH SIDE', 27, 18, '(41.90155157, -87.63453744)') ,\n(609955, 'Walter Q Gresham Elementary School', 'ES', '8524 S Green St', 'Chicago', 'IL', 60620, '(773) 535-3350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609955.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 25.0, 'Average', 49.0, 'Very Weak', 7.0, 'Very Weak', 11.0, 'Weak', 68.0, 'Average', 57.0, 'Average', 48.0, 'Weak', 46.0, '95.50%', 35.4, '95.00%', '100.00%', 61.4, 54.0, 30.7, 24.0, 47.7, 57.3, 13.9, 21.6, 35.2, 34.6, 5.4, 16.2, 9.9, 7.4, 1.4, 0.0, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 359, 45, nan, 1172094.314, 1848202.3080000002, 41.73891334, -87.64506169, 71, 'AUBURN GRESHAM', 21, 6, '(41.73891334, -87.64506169)') ,\n(610264, 'Walter Reed Elementary School', 'ES', '6350 S Stewart Ave', 'Chicago', 'IL', 60621, '(773) 535-3075', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610264.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 29.0, nan, nan, 'Strong', 62.0, 'Strong', 68.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 51.0, '92.40%', 75.5, '93.20%', '100.00%', nan, nan, 43.2, 34.1, 65.9, 65.9, 66.0, 20.0, 93.6, 75.6, 0.0, 5.3, 28.6, 9.9, 2.0, -0.3, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 44, 45, nan, 1174681.574, 1862611.0359999998, 41.77839545, -87.63515406, 68, 'ENGLEWOOD', 20, 7, '(41.77839545, -87.63515406)') ,\n(609855, 'Walter S Christopher Elementary School', 'ES', '5042 S Artesian Ave', 'Chicago', 'IL', 60632, '(773) 535-9375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609855.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', nan, nan, 'Average', 48.0, nan, nan, nan, nan, 'Weak', 72.0, 'Strong', 64.0, 'Average', 52.0, 'Average', 53.0, '86.10%', 4.0, '94.10%', '100.00%', 42.9, 15.8, 28.8, 19.0, 66.1, 45.5, 61.7, 44.7, 63.0, 54.3, 7.7, 38.5, 13.3, 5.7, 0.7, 0.7, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 279, 43, nan, 1160846.21, 1870985.96, 41.8016744, -87.68564411, 63, 'GAGE PARK', 14, 9, '(41.8016744, -87.68564411)') ,\n(610179, 'Washington D Smyser Elementary School', 'ES', '4310 N Melvina Ave', 'Chicago', 'IL', 60634, '(773) 534-3711', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610179.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 55.0, nan, nan, 'Average', 43.0, 'Average', 45.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.50%', 4.9, '97.20%', '96.60%', 72.6, nan, 53.9, 58.4, 61.1, 65.7, 59.2, 60.2, 64.7, 51.5, 27.0, 49.5, 36.5, 27.5, 0.6, 0.1, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1047, 30, nan, 1134202.834, 1928091.892, 41.95889037, -87.78201198, 15, 'PORTAGE PARK', 38, 16, '(41.95889037, -87.78201198)') ,\n(610121, 'Washington Irving Elementary School', 'ES', '749 S Oakley Blvd', 'Chicago', 'IL', 60612, '(773) 534-7295', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610121.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 43.0, nan, nan, 'Weak', 34.0, 'Average', 44.0, 'Weak', nan, nan, nan, 'Average', 48.0, 'Average', 49.0, '95.60%', 15.1, '95.90%', '89.00%', 71.8, 64.1, 36.8, 34.0, 45.5, 52.2, 57.1, 46.8, 68.8, 64.2, 17.0, 27.7, 22.6, 20.9, -0.8, 0.7, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 482, 38, nan, 1161216.3909999998, 1896475.332, 41.87161238, -87.68357939, 28, 'NEAR WEST SIDE', 25, 12, '(41.87161238, -87.68357939)') ,\n(609740, 'Wells Community Academy High School', 'HS', '936 N Ashland Ave', 'Chicago', 'IL', 60622, '(773) 534-7010', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609740.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 51.0, 'Average', 49.0, 'Average', 41.0, 'Weak', 38.0, 'Weak', 65.0, 'Average', 49.0, nan, nan, nan, nan, '74.10%', 16.9, '96.20%', '96.70%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.8, 12.4, 14.2, 13.6, 0.8, 14.9, 0.7, 7.2, 59.3, 43.8, 633, 35, 61.3, 1165512.916, 1906490.72, 41.89900506, -87.66751989, 24, 'WEST TOWN', 1, 13, '(41.89900506, -87.66751989)') ,\n(610032, 'Wendell E Green Elementary School', 'ES', '1150 W 96th St', 'Chicago', 'IL', 60643, '(773) 535-2575', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610032.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Weak', 32.0, nan, nan, 'Average', 47.0, 'Average', 53.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Average', 47.0, '93.90%', 20.1, '95.70%', '100.00%', 59.4, 46.3, 53.0, 39.2, 72.6, 63.5, 55.2, 37.5, 59.8, 67.5, 12.5, 12.5, 25.8, 15.7, 2.2, 1.0, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 254, 49, nan, 1170124.549, 1841046.4109999998, 41.71931953, -87.65248591, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.71931953, -87.65248591)') ,\n(609727, 'Wendell Phillips Academy High School', 'HS', '244 E Pershing Rd', 'Chicago', 'IL', 60653, '(773) 535-1603', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609727.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', nan, nan, 'Strong', 71.0, nan, nan, nan, nan, 'Weak', 80.0, 'Strong', 79.0, nan, nan, nan, nan, '63.00%', 22.0, '96.10%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 11.5, 12.8, 13.4, 12.9, 1.4, 15.0, 1.6, 2.5, 40.3, 42.6, 590, 40, 32.1, 1178735.106, 1879229.78, 41.82390751, -87.61978794, 35, 'DOUGLAS', 3, 2, '(41.82390751, -87.61978794)') ,\n(609978, 'Wendell Smith Elementary School', 'ES', '744 E 103rd St', 'Chicago', 'IL', 60628, '(773) 535-5689', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609978.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 22.0, 'Weak', 24.0, 'Very Weak', 1.0, 'Very Weak', 1.0, 'Weak', 31.0, 'Weak', 29.0, 'Weak', 42.0, 'Weak', 46.0, '92.10%', 90.0, '94.10%', '95.20%', 64.8, 42.6, 25.0, 20.2, 53.1, 49.0, 34.6, 22.3, 30.7, 45.9, 8.6, 22.9, 9.4, 4.2, -0.5, -1.0, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 363, 48, nan, 1183210.11, 1836877.16, 41.70758462, -87.60468711, 50, 'PULLMAN', 8, 5, '(41.70758462, -87.60468711)') ,\n(610100, 'West Park Elementary Academy', 'ES', '1425 N Tripp Ave', 'Chicago', 'IL', 60651, '(773) 534-4940', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610100.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 29.0, nan, nan, 'Weak', 36.0, 'Average', 40.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Average', 47.0, '93.70%', 16.3, '96.70%', '100.00%', 60.7, 44.4, 22.0, 17.6, 39.2, 37.3, 29.0, 26.4, 61.2, 51.2, 9.2, 15.4, 10.8, 6.9, -0.3, -0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 686, 34, nan, 1147827.76, 1909240.9519999998, 41.90691024, -87.73240646, 23, 'HUMBOLDT PARK', 37, 25, '(41.90691024, -87.73240646)') ,\n(610224, 'West Pullman Elementary School', 'ES', '11941 S Parnell Ave', 'Chicago', 'IL', 60628, '(773) 535-5500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610224.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 23.0, 'Average', 46.0, 'Average', 55.0, 'Average', 47.0, 'Weak', 57.0, 'Average', 50.0, 'Average', 47.0, 'Weak', 44.0, '91.20%', 7.6, '93.80%', '100.00%', 31.0, 28.8, 12.6, 15.5, 45.9, 43.5, 25.7, 22.5, 32.0, 38.8, 9.7, 22.6, 5.7, 5.7, -0.9, -1.3, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 343, 48, nan, 1174831.25, 1825538.503, 41.67666008, -87.63570683, 53, 'WEST PULLMAN', 34, 5, '(41.67666008, -87.63570683)') ,\n(610542, 'West Ridge Elementary School', 'ES', '6700 N Whipple St', 'Chicago', 'IL', 60645, '(773) 534-8250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610542.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Strong', 67.0, nan, nan, 'Average', 41.0, 'Weak', 28.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Strong', 58.0, '94.90%', 6.8, '0.00%', '100.00%', 69.0, 46.8, 43.3, 32.9, 46.3, 60.9, 44.0, 34.4, 48.7, 43.2, 18.2, 26.2, 16.9, 13.0, -2.2, -1.0, 'Red', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 659, 32, nan, 1154772.178, 1944306.386, 42.00299575, -87.70595357, 2, 'WEST RIDGE', 50, 24, '(42.00299575, -87.70595357)') ,\n(609755, 'Whitney M Young Magnet High School', 'HS', '211 S Laflin St', 'Chicago', 'IL', 60607, '(773) 534-7500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609755.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 95.0, 'Very Strong', 80.0, 'Strong', 69.0, 'Strong', 67.0, 'Weak', 52.0, 'Average', 46.0, 'Average', 53.0, 'Average', 50.0, '94.60%', 1.2, '94.90%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 100.0, 99.0, 99.5, 77.0, -1.2, -0.8, 'Red', 'Yellow', 56.3, 89.2, 20.8, 21.0, 22.6, 22.8, 2.0, 26.6, 4.0, 89.0, 93.9, 88.3, 2166, 38, 96.7, 1166475.742, 1899058.6269999999, 41.87859028, -87.66419634, 28, 'NEAR WEST SIDE', 2, 12, '(41.87859028, -87.66419634)') ,\n(610230, 'Wildwood Elementary School', 'ES', '6950 N Hiawatha Ave', 'Chicago', 'IL', 60646, '(773) 534-1188', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610230.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 90.0, 'Strong', 77.0, 'Strong', 67.0, 'Weak', 55.0, 'Average', 45.0, 'Strong', 55.0, 'Weak', 37.0, '96.20%', 1.0, '95.70%', '92.30%', 88.8, nan, 71.5, 67.2, 47.4, 59.9, 79.5, 79.5, 82.0, 63.9, 50.9, 68.4, 52.9, 33.2, 0.7, -0.4, 'Green', 'Yellow', 22.8, 84.6, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 414, 30, nan, 1135078.471, 1945800.949, 42.00747007, -87.77837089, 12, 'FOREST GLEN', 41, 16, '(42.00747007, -87.77837089)') ,\n(610251, 'Willa Cather Elementary School', 'ES', '2908 W Washington Blvd', 'Chicago', 'IL', 60612, '(773) 534-6780', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610251.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 69.0, 'Strong', 70.0, 'Strong', 78.0, 'Very Strong', 81.0, 'Weak', 62.0, 'Very Strong', 83.0, 'Strong', 56.0, 'Average', 53.0, '94.90%', 29.5, '96.40%', '100.00%', 62.7, 44.0, 42.9, 27.1, 54.3, 72.5, 36.1, 21.3, 45.5, 38.2, 0.0, 5.3, 20.3, 20.9, 2.7, 2.3, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 234, 34, nan, 1156925.053, 1900584.719, 41.882977000000004, -87.69922315, 27, 'EAST GARFIELD PARK', 2, 13, '(41.882977, -87.69922315)') ,\n(610299, 'William A Hinton Elementary School', 'ES', '644 W 71st St', 'Chicago', 'IL', 60621, '(773) 535-3875', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610299.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 18.0, nan, nan, 'Weak', 30.0, 'Weak', 29.0, 'Weak', nan, nan, nan, 'Weak', 45.0, 'Average', 50.0, '91.70%', 28.9, '95.10%', '100.00%', nan, nan, 28.9, 15.2, 52.6, 59.7, 31.1, 23.0, 40.2, 45.2, 10.0, 12.5, 9.2, 3.8, 0.0, -1.4, 'Yellow', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 321, 45, nan, 1173035.972, 1857848.3390000002, 41.76536255, -87.64132740000001, 68, 'ENGLEWOOD', 6, 7, '(41.76536255, -87.6413274)') ,\n(610109, 'William Bishop Owen Scholastic Academy Elementary School', 'ES', '8247 S Christiana Ave', 'Chicago', 'IL', 60652, '(773) 535-9330', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610109.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 49.0, 'Strong', 61.0, 'Weak', 39.0, 'Average', 56.0, 'Weak', 51.0, 'Average', 47.0, 'Strong', 58.0, 'Average', 53.0, '96.90%', 1.2, '95.00%', '100.00%', 88.2, 77.6, 69.5, 73.2, 72.0, 70.7, 67.5, 61.3, 62.5, 53.8, 19.2, 30.8, 28.4, 29.0, 0.7, 0.1, 'Yellow', 'Yellow', 44.4, 54.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 243, 44, nan, 1155530.649, 1849518.217, 41.74287191, -87.70571301, 70, 'ASHBURN', 18, 8, '(41.74287191, -87.70571301)') ,\n(610101, 'William B Ogden Elementary School', 'ES', '24 W Walton St', 'Chicago', 'IL', 60610, '(773) 534-8110', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610101.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, nan, nan, nan, nan, '95.10%', 4.5, '97.00%', '100.00%', 40.0, nan, 67.3, 61.3, 57.8, 66.8, nan, nan, nan, nan, nan, nan, 40.8, 39.9, 0.3, 0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 722, 33, nan, 1175917.983, 1906947.8590000002, 41.90003152, -87.62928901, 8, 'NEAR NORTH SIDE', 42, 18, '(41.90003152, -87.62928901)') ,\n(609945, 'William C Goudy Elementary School', 'ES', '5120 N Winthrop Ave', 'Chicago', 'IL', 60640, '(773) 534-2480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609945.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 70.0, 'Average', 52.0, 'Strong', 74.0, 'Strong', 61.0, 'Weak', 65.0, 'Average', 52.0, nan, nan, nan, nan, '96.10%', 6.0, '95.80%', '100.00%', 64.0, 62.9, 46.1, 32.8, 52.5, 59.5, 61.5, 48.3, 65.6, 60.7, 37.5, 51.8, 30.8, 23.6, 0.9, 1.0, 'Green', 'Green', 34.5, 100.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 805, 32, nan, 1167897.992, 1934499.505, 41.97581137, -87.65794915, 3, 'UPTOWN', 48, 20, '(41.97581137, -87.65794915)') ,\n(610143, 'William C Reavis Math & Science Specialty Elementary School', 'ES', '834 E 50th St', 'Chicago', 'IL', 60615, '(773) 535-1060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610143.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 48.0, 'Very Weak', 6.0, 'Weak', 37.0, 'Weak', 26.0, 'Weak', 10.0, 'Very Weak', 15.0, 'Weak', 45.0, 'Average', 47.0, '93.50%', 37.2, '94.80%', '98.20%', 47.2, nan, 31.0, 29.5, 43.5, 51.9, 19.1, 23.4, 38.5, 48.4, 3.2, 16.1, 5.8, 10.1, -0.7, 0.0, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 311, 42, nan, 1182841.804, 1872082.7719999999, 41.80420101, -87.6049443, 39, 'KENWOOD', 4, 2, '(41.80420101, -87.6049443)') ,\n(610364, 'William E B Dubois Elementary School', 'ES', '330 E 133rd St', 'Chicago', 'IL', 60827, '(773) 535-5582', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610364.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 47.0, nan, nan, 'Weak', 33.0, 'Average', 42.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Weak', 45.0, '93.30%', 48.5, '94.40%', '100.00%', 73.2, 64.8, 17.9, 11.8, 37.5, 44.4, 23.2, 20.9, 64.5, 55.7, 0.0, 12.5, 7.9, 5.0, 0.4, 0.4, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 184, 48, nan, 1180737.575, 1817242.1530000002, 41.65376037, -87.61434156, 54, 'RIVERDALE', 9, 5, '(41.65376037, -87.61434156)') ,\n(609884, 'William E Dever Elementary School', 'ES', '3436 N Osceola Ave', 'Chicago', 'IL', 60634, '(773) 534-3090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609884.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, 'Strong', 66.0, 'Strong', 60.0, 'Strong', 78.0, 'Weak', 37.0, 'Weak', 36.0, 'Average', 50.0, 'Average', 48.0, '95.50%', 2.1, '95.90%', '100.00%', 71.1, 34.9, 60.8, 51.1, 62.9, 66.3, 73.5, 57.6, 68.6, 63.8, 20.0, 46.7, 31.5, 27.9, 0.4, 1.3, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 843, 30, nan, 1125764.896, 1922082.0219999999, 41.94254335, -87.813168, 17, 'DUNNING', 36, 16, '(41.94254335, -87.813168)') ,\n(609967, 'William F Finkl Elementary School', 'ES', '2332 S Western Ave', 'Chicago', 'IL', 60608, '(773) 535-5850', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609967.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 56.0, 'Strong', 60.0, 'Average', 50.0, 'Average', 52.0, 'Weak', 41.0, 'Average', 56.0, 'Average', 49.0, 'Strong', 54.0, '95.20%', 6.7, '96.90%', '100.00%', 58.6, nan, 33.1, 25.2, 53.6, 47.8, 44.9, 34.8, 48.4, 50.8, 17.1, 25.7, 19.2, 9.6, 0.4, 0.7, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 588, 39, nan, 1160699.3059999999, 1888303.5219999999, 41.84919885, -87.68570412, 31, 'LOWER WEST SIDE', 25, 10, '(41.84919885, -87.68570412)') ,\n(609994, 'William G Hibbard Elementary School', 'ES', '3244 W Ainslie St', 'Chicago', 'IL', 60625, '(773) 534-5191', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609994.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 49.0, 'Average', 50.0, '96.50%', 3.0, '96.90%', '99.30%', 70.7, 20.7, 38.3, 27.2, 40.0, 46.7, 38.8, 34.7, 33.9, 38.3, nan, nan, 19.3, 15.3, 0.5, -0.1, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1239, 31, nan, 1153660.42, 1932405.9719999998, 41.97036267, -87.71036214, 14, 'ALBANY PARK', 39, 17, '(41.97036267, -87.71036214)') ,\n(609812, 'William H Brown Elementary School', 'ES', '54 N Hermitage Ave', 'Chicago', 'IL', 60612, '(773) 534-7250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609812.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 36.0, 'Strong', 73.0, 'Average', 58.0, 'Strong', 78.0, 'Weak', 74.0, 'Strong', 73.0, 'Weak', 46.0, 'Average', 49.0, '92.80%', 59.0, '94.70%', '100.00%', 67.6, 51.4, 28.3, 15.1, 32.0, 60.0, 41.9, 20.5, 58.0, 36.2, 4.2, 4.2, 10.2, 3.2, 1.1, -0.9, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 217, 38, nan, 1164687.1090000002, 1900646.996, 41.88298698, -87.67071879, 28, 'NEAR WEST SIDE', 27, 13, '(41.88298698, -87.67071879)') ,\n(610023, 'William H King Elementary School', 'ES', '740 S Campbell Ave', 'Chicago', 'IL', 60612, '(773) 534-7898', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610023.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 47.0, nan, nan, 'Average', 44.0, 'Average', 47.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '92.40%', 17.7, '95.10%', '100.00%', 42.1, nan, 30.1, 29.4, 57.5, 53.8, 36.8, 27.5, 65.9, 69.0, 10.0, 16.7, 10.6, 8.4, 0.3, -0.4, 'Yellow', 'Yellow', 40.6, 7.7, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 264, 38, nan, 1159805.717, 1896555.6930000002, 41.87186209, -87.68875632, 28, 'NEAR WEST SIDE', 2, 11, '(41.87186209, -87.68875632)') ,\n(609734, 'William Howard Taft High School', 'HS', '6530 W Bryn Mawr Ave', 'Chicago', 'IL', 60631, '(773) 534-1000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609734.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 61.0, nan, nan, 'Weak', 37.0, 'Weak', 32.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '89.10%', 9.8, '96.10%', '99.70%', nan, nan, nan, nan, nan, nan, 91.8, 93.6, 41.3, 59.8, 71.9, 88.4, 80.2, 50.6, -3.1, -1.8, 'Red', 'Red', 74.4, 60.0, 14.8, 15.7, 15.9, 16.4, 1.6, 17.9, 2.0, 32.3, 67.4, 62.2, 2922, 30, 77.3, 1131556.247, 1936856.822, 41.98298852, -87.79153829999998, 10, 'NORWOOD PARK', 41, 16, '(41.98298852, -87.7915383)') ,\n(610136, 'William H Prescott Elementary School', 'ES', '1632 W Wrightwood Ave', 'Chicago', 'IL', 60614, '(773) 534-5505', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610136.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Very Strong', 90.0, nan, nan, 'Average', 51.0, 'Weak', 24.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 48.0, '93.50%', 73.9, '97.60%', '95.00%', nan, nan, 24.4, 31.7, 63.2, 46.3, 42.6, 43.5, 61.7, 65.9, nan, nan, 16.0, 8.9, 0.4, 0.5, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 240, 33, nan, 1164875.342, 1917368.608, 41.92886824, -87.66955245, 7, 'LINCOLN PARK', 32, 19, '(41.92886824, -87.66955245)') ,\n(610142, 'William H Ray Elementary School', 'ES', '5631 S Kimbark Ave', 'Chicago', 'IL', 60637, '(773) 535-0970', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610142.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 69.0, nan, nan, 'Weak', 22.0, 'Very Weak', 12.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Weak', 46.0, '95.10%', 6.7, '94.90%', '98.80%', 75.1, 46.4, 51.5, 51.5, 53.6, 44.3, 48.4, 63.1, 50.0, 50.8, nan, nan, 31.8, 35.2, -0.8, 1.5, 'Red', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 654, 46, nan, 1185667.78, 1867788.013, 41.79234967, -87.59471544, 41, 'HYDE PARK', 5, 2, '(41.79234967, -87.59471544)') ,\n(610153, 'William H Ryder Math & Science Specialty Elementary School', 'ES', '8716 S Wallace St', 'Chicago', 'IL', 60620, '(773) 535-3843', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610153.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 34.0, nan, nan, 'Average', 42.0, 'Average', 50.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '93.00%', 22.5, '94.30%', '96.40%', 73.6, 56.9, 21.6, 6.8, 32.7, 37.5, 37.7, 17.7, 44.6, 37.0, 20.5, 33.3, 10.6, 5.6, -0.5, -1.6, 'Yellow', 'Red', 27.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 302, 49, nan, 1173792.13, 1846923.856, 41.73536767, -87.63887904, 71, 'AUBURN GRESHAM', 21, 22, '(41.73536767, -87.63887904)') ,\n(610167, 'William H Seward Communication Arts Academy Elementary School', 'ES', '4600 S Hermitage Ave', 'Chicago', 'IL', 60609, '(773) 535-4890', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610167.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 40.0, 'Average', 43.0, 'Weak', 38.0, 'Average', 41.0, 'Weak', 66.0, 'Strong', 63.0, 'Average', 49.0, 'Average', 50.0, '95.70%', 14.4, '95.70%', '100.00%', 58.6, nan, 25.2, 28.2, 49.1, 58.8, 45.3, 33.9, 57.8, 51.3, 12.5, 25.0, 12.2, 7.9, -1.1, -0.5, 'Red', 'Yellow', 43.3, 46.4, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 896, 42, nan, 1165412.465, 1874162.7769999998, 41.81029632, -87.66880778, 61, 'NEW CITY', 20, 9, '(41.81029632, -87.66880778)') ,\n(609698, 'William J Bogan High School', 'HS', '3939 W 79th St', 'Chicago', 'IL', 60652, '(773) 535-2180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609698.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 20.0, nan, nan, 'Very Weak', 18.0, 'Weak', 29.0, 'Weak', nan, nan, nan, 'Weak', 41.0, 'Weak', 38.0, '75.20%', 5.8, '94.30%', '99.60%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 13.4, 13.3, 14.3, 14.3, 0.9, 15.2, 0.9, 13.1, 52.5, 50.0, 1656, 44, 67.1, 1151315.548, 1851847.8590000002, 41.74934817, -87.72109673, 70, 'ASHBURN', 18, 8, '(41.74934817, -87.72109673)') ,\n(610061, 'William J & Charles H Mayo Elementary School', 'ES', '249 E 37th St', 'Chicago', 'IL', 60653, '(773) 535-1260', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610061.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 52.0, 'Average', 55.0, 'Strong', 62.0, 'Strong', 65.0, 'Weak', 52.0, 'Average', 47.0, 'Average', 50.0, 'Strong', 55.0, '95.30%', 26.0, '95.20%', '100.00%', 48.3, 24.1, 28.2, 25.2, 45.9, 58.0, 40.0, 36.6, 57.4, 53.7, 10.0, 30.0, 15.5, 6.0, 0.7, -0.3, 'Green', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 453, 40, nan, 1178726.235, 1880476.124, 41.82732778, -87.61978251, 35, 'DOUGLAS', 3, 2, '(41.82732778, -87.61978251)') ,\n(610104, 'William J Onahan Elementary School', 'ES', '6634 W Raven St', 'Chicago', 'IL', 60631, '(773) 534-1180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610104.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, 'Average', 59.0, 'Weak', 29.0, 'Weak', 27.0, 'Weak', 32.0, 'Weak', 39.0, 'Average', 47.0, 'Weak', 44.0, '95.70%', 6.9, '96.10%', '98.80%', 76.8, 56.2, 62.7, 61.2, 40.8, 53.9, 60.2, 63.8, 49.7, 44.8, 44.0, 54.0, 35.1, 29.6, 0.2, 0.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 687, 30, nan, 1131111.88, 1940800.44, 41.99381787, -87.79308113, 10, 'NORWOOD PARK', 41, 16, '(41.99381787, -87.79308113)') ,\n(609678, 'William Jones College Preparatory High School', 'HS', '606 S State St', 'Chicago', 'IL', 60605, '(773) 534-8600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609678.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 92.0, nan, nan, 'Strong', 64.0, 'Strong', 67.0, 'Weak', nan, nan, nan, nan, nan, nan, nan, '93.80%', 4.5, '95.30%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 19.6, 20.3, 21.3, 21.1, 1.5, 24.8, 3.5, 83.3, 90.1, 86.9, 871, 38, 92.8, 1176412.354, 1897618.0880000002, 41.87441898, -87.62775497, 32, 'LOOP', 2, 1, '(41.87441898, -87.62775497)') ,\n(610193, 'William K New Sullivan Elementary School', 'ES', '8331 S Mackinaw', 'Chicago', 'IL', 60617, '(773) 535-6585', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610193.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 27.0, 'Average', 42.0, 'Weak', 35.0, 'Average', 40.0, 'Weak', 58.0, 'Weak', 33.0, 'Average', 47.0, 'Strong', 54.0, '90.30%', 27.3, '94.00%', '97.80%', 34.1, 17.1, 17.8, 14.1, 42.9, 51.5, 10.2, 21.0, 56.6, 52.5, 0.0, 6.7, 2.9, 5.8, -0.9, 0.2, 'Red', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 527, 47, nan, 1199962.656, 1850233.5369999998, 41.743830200000005, -87.542891, 46, 'SOUTH CHICAGO', 10, 4, '(41.7438302, -87.542891)') ,\n(610123, 'William Penn Elementary School', 'ES', '1616 S Avers Ave', 'Chicago', 'IL', 60623, '(773) 534-1665', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610123.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Strong', 78.0, nan, nan, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Weak', nan, nan, nan, 'Average', 52.0, 'Average', 53.0, '91.80%', 5.7, '95.20%', '92.90%', 42.7, nan, 22.4, 17.6, 53.0, 51.5, 16.5, 15.7, 42.5, 51.3, 8.7, 17.4, 8.1, 5.5, 0.3, -0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 423, 37, nan, 1150967.365, 1891574.2419999999, 41.85836995, -87.72133609, 29, 'NORTH LAWNDALE', 24, 10, '(41.85836995, -87.72133609)') ,\n(609949, 'William P Gray Elementary School', 'ES', '3730 N Laramie Ave', 'Chicago', 'IL', 60641, '(773) 534-3520', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609949.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 57.0, nan, nan, 'Average', 44.0, 'Weak', 35.0, 'Weak', nan, nan, nan, 'Average', 47.0, 'Average', 50.0, '95.60%', 5.9, '95.90%', '99.40%', 74.1, nan, 53.5, 51.4, 69.1, 77.5, 43.0, 53.5, 61.0, 70.1, 16.3, 28.7, 26.1, 19.4, 0.5, 0.9, 'Green', 'Green', 46.1, 34.9, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1345, 29, nan, 1140988.981, 1924452.515, 41.94878107, -87.75715297, 15, 'PORTAGE PARK', 38, 16, '(41.94878107, -87.75715297)') ,\n(610097, 'William P Nixon Elementary School', 'ES', '2121 N Keeler Ave', 'Chicago', 'IL', 60639, '(773) 534-4375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610097.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 52.0, nan, nan, 'Strong', 70.0, 'Strong', 62.0, 'Weak', nan, nan, nan, 'Weak', 46.0, 'Average', 47.0, '95.60%', 3.5, '95.80%', '100.00%', 66.9, 25.0, 31.8, 27.8, 52.4, 57.3, 35.5, 20.7, 57.0, 50.0, nan, nan, 11.2, 9.6, -0.1, -0.2, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1061, 34, nan, 1148073.735, 1913864.6630000002, 41.91959342, -87.73138368, 20, 'HERMOSA', 30, 25, '(41.91959342, -87.73138368)') ,\n(609711, 'William Rainey Harper High School', 'HS', '6520 S Wood St', 'Chicago', 'IL', 60636, '(773) 535-9150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609711.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 22.0, 'Weak', 32.0, 'Weak', 39.0, 'Average', 42.0, 'Weak', 31.0, 'Weak', 31.0, 'Average', 49.0, 'Average', 53.0, '73.00%', 63.6, '94.70%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.2, 11.9, 13.1, 13.4, 1.2, 14.4, 1.3, 5.9, 37.0, 40.3, 621, 43, 64.2, 1165424.867, 1861341.255, 41.77511222, -87.66912563, 67, 'WEST ENGLEWOOD', 15, 7, '(41.77511222, -87.66912563)') ,\n(610232, 'Williams Multiplex Elementary School', 'ES', '2710 S Dearborn St', 'Chicago', 'IL', 60616, '(773) 534-9245', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610232.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', nan, nan, nan, nan, nan, nan, nan, nan, 'Weak', nan, nan, nan, 'Average', 50.0, 'Average', 50.0, '91.60%', 17.0, '95.90%', '100.00%', 45.3, 31.1, 22.3, 25.9, 48.1, 58.1, nan, nan, nan, nan, nan, nan, 24.3, 27.9, 1.1, 2.5, 'Green', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 266, 40, nan, 1176275.784, 1886545.486, 41.84403809, -87.62859008, 35, 'DOUGLAS', 3, 1, '(41.84403809, -87.62859008)') ,\n(610336, 'Williams Preparatory Academy Middle School', 'MS', '2710 S Dearborn St', 'Chicago', 'IL', 60616, '(773) 534-9235', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610336.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 49.0, nan, nan, 'Average', 42.0, 'Average', 48.0, 'Weak', nan, nan, nan, 'Average', 53.0, 'Strong', 56.0, '93.30%', 14.8, '92.40%', '96.80%', nan, nan, nan, nan, nan, nan, 16.2, 26.1, 56.3, 50.0, 2.8, 11.1, 7.8, 15.5, 0.4, 3.3, 'Yellow', 'Green', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 125, 40, nan, 1176275.784, 1886545.486, 41.84403809, -87.62859008, 35, 'DOUGLAS', 3, 1, '(41.84403809, -87.62859008)') ,\n(610172, 'William T Sherman Elementary School', 'ES', '1000 W 52nd St', 'Chicago', 'IL', 60609, '(773) 535-1757', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610172.pdf', 'AUSL Schools', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 32.0, nan, nan, 'Average', 46.0, 'Average', 55.0, 'Weak', nan, nan, nan, 'Average', 49.0, 'Average', 52.0, '92.30%', 230.6, '95.00%', '100.00%', nan, nan, 21.8, 26.8, 41.1, 42.0, 30.1, 19.7, 46.3, 35.2, 7.5, 15.1, 8.1, 1.7, 0.7, -1.3, 'Green', 'Red', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 462, 45, nan, 1170500.817, 1870373.159, 41.79978772, -87.65025483, 61, 'NEW CITY', 16, 9, '(41.79978772, -87.65025483)') ,\n(609844, 'William W Carter Elementary School', 'ES', '5740 S Michigan Ave', 'Chicago', 'IL', 60637, '(773) 535-0860', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609844.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 13.0, 'Average', 49.0, 'Weak', 33.0, 'Weak', 35.0, 'Weak', 56.0, 'Strong', 62.0, 'Weak', 46.0, 'Weak', 46.0, '91.20%', 27.0, '95.90%', '100.00%', 72.3, 46.6, 21.8, 17.8, 50.5, 57.6, 24.8, 24.8, 44.9, 48.6, 13.3, 33.3, 6.8, 5.5, -0.2, -1.2, 'Yellow', 'Red', 42.4, 50.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 371, 42, nan, 1178101.365, 1866810.1230000001, 41.78984129, -87.62248974, 40, 'WASHINGTON PARK', 20, 2, '(41.78984129, -87.62248974)') ,\n(610088, 'Wolfgang A Mozart Elementary School', 'ES', '2200 N Hamlin Ave', 'Chicago', 'IL', 60647, '(773) 534-4160', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610088.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 41.0, nan, nan, 'Average', 56.0, 'Weak', 32.0, 'Weak', nan, nan, nan, 'Average', 50.0, 'Strong', 54.0, '95.20%', 3.6, '96.40%', '100.00%', 39.8, 10.1, 28.7, 34.2, 51.1, 48.2, 46.7, 32.6, 74.7, 64.0, nan, nan, 12.0, 11.0, 0.4, 0.3, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 748, 34, nan, 1150644.396, 1914368.955, 41.92092734, -87.72192541, 22, 'LOGAN SQUARE', 35, 25, '(41.92092734, -87.72192541)') ,\n(609977, 'Woodlawn Community Elementary School', 'ES', '6657 S Kimbark Ave', 'Chicago', 'IL', 60637, '(773) 535-0801', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609977.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 70.0, nan, nan, 'Very Strong', 80.0, 'Strong', 66.0, 'Weak', nan, nan, nan, 'Strong', 59.0, 'Strong', 54.0, '93.90%', 12.4, '94.30%', '100.00%', 57.7, nan, 43.9, 45.8, 60.0, 62.4, 75.0, 30.0, 88.9, 55.6, nan, nan, 17.5, 22.2, 0.1, 0.0, 'Yellow', 'Yellow', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 238, 46, nan, 1185825.188, 1860883.5790000001, 41.77339962, -87.59435584, 42, 'WOODLAWN', 5, 3, '(41.77339962, -87.59435584)') ,\n(610392, 'World Language Academy High School', 'HS', '3120 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 535-4334', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610392.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 51.0, 'Weak', 33.0, 'Average', 49.0, 'Average', 47.0, 'Weak', 49.0, 'Average', 49.0, 'Average', 50.0, 'Weak', 46.0, '91.60%', 4.0, '96.00%', '100.00%', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 12.9, 13.4, 14.6, 14.1, 1.2, 16.2, 1.6, 14.6, 69.4, 48.1, 382, 37, 76.0, 1147521.3020000001, 1883405.128, 41.83601953, -87.73419465, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83601953, -87.73419465)')"""

print(d[:4000])


INSERT INTO PUBLIC_SCHOOLS (School_ID, NAME_OF_SCHOOL, Elementary_Middle_or_High_School, Street_Address, City, State, ZIP_Code, Phone_Number, Link, Network_Manager, Collaborative_Name, Adequate_Yearly_Progress_Made_, Track_Schedule, CPS_Performance_Policy_Status, CPS_Performance_Policy_Level, HEALTHY_SCHOOL_CERTIFIED, Safety_Icon, SAFETY_SCORE, Family_Involvement_Icon, Family_Involvement_Score, Environment_Icon, Environment_Score, Instruction_Icon, Instruction_Score, Leaders_Icon, Leaders_Score, Teachers_Icon, Teachers_Score, Parent_Engagement_Icon, Parent_Engagement_Score, Parent_Environment_Icon, Parent_Environment_Score, AVERAGE_STUDENT_ATTENDANCE, Rate_of_Misconducts__per_100_students_, Average_Teacher_Attendance, Individualized_Education_Program_Compliance_Rate, Pk_2_Literacy__, Pk_2_Math__, Gr3_5_Grade_Level_Math__, Gr3_5_Grade_Level_Read__, Gr3_5_Keep_Pace_Read__, Gr3_5_Keep_Pace_Math__, Gr6_8_Grade_Level_Math__, Gr6_8_Grade_Level_Read__, Gr6_8_Keep_Pace_Math_, Gr6_8_Keep_Pace_R

In [26]:
%%sql
INSERT INTO BVR08213.PUBLIC_SCHOOLS (
                                    School_ID,
                                    NAME_OF_SCHOOL,
                                    Elementary_Middle_or_High_School,                       
                                    Street_Address,                                                
                                    City,                                                
                                    State,                                                
                                    ZIP_Code,                                                
                                    Phone_Number,                                                
                                    Link,                                                
                                    Network_Manager,                                                
                                    Collaborative_Name,                                                
                                    Adequate_Yearly_Progress_Made_,                                                
                                    Track_Schedule,                                                
                                    CPS_Performance_Policy_Status ,                                                
                                    CPS_Performance_Policy_Level ,                                                
                                    HEALTHY_SCHOOL_CERTIFIED ,                                                
                                    Safety_Icon,                                                
                                    SAFETY_SCORE,                                                
                                    Family_Involvement_Icon,                                                
                                    Family_Involvement_Score,                                                
                                    Environment_Icon ,                                                
                                    Environment_Score,                                                
                                    Instruction_Icon ,                                                
                                    Instruction_Score,                                                
                                    Leaders_Icon ,                                                
                                    Leaders_Score,                                                
                                    Teachers_Icon ,                                                
                                    Teachers_Score,                                                
                                    Parent_Engagement_Icon ,                                                
                                    Parent_Engagement_Score,                                                
                                    Parent_Environment_Icon ,                                                
                                    Parent_Environment_Score,                                                
                                    AVERAGE_STUDENT_ATTENDANCE,                                                
                                    Rate_of_Misconducts__per_100_students_,                                                
                                    Average_Teacher_Attendance,                                                
                                    Individualized_Education_Program_Compliance_Rate,                                                
                                    Pk_2_Literacy__,                                                
                                    Pk_2_Math__,                                                
                                    Gr3_5_Grade_Level_Math__,                                                
                                    Gr3_5_Grade_Level_Read__,                                                
                                    Gr3_5_Keep_Pace_Read__,                                                
                                    Gr3_5_Keep_Pace_Math__,                                                
                                    Gr6_8_Grade_Level_Math__,                                                
                                    Gr6_8_Grade_Level_Read__,                                                
                                    Gr6_8_Keep_Pace_Math_,                                                
                                    Gr6_8_Keep_Pace_Read__,                                                
                                    Gr_8_Explore_Math__,                                                
                                    Gr_8_Explore_Read__,                                                
                                    ISAT_Exceeding_Math__,                                                
                                    ISAT_Exceeding_Reading__,                                                
                                    ISAT_Value_Add_Math,                                                
                                    ISAT_Value_Add_Read,                                                
                                    ISAT_Value_Add_Color_Math ,                                                
                                    ISAT_Value_Add_Color_Read ,                                                
                                    Students_Taking__Algebra__,                                                
                                    Students_Passing__Algebra__,                                                
                                    Gr_9_EXPLORE_2009,                                                
                                    Gr_9_EXPLORE_2010,                                                
                                    Gr_10_PLAN_2009,                                                
                                    Gr_10_PLAN_2010,                                                
                                    Net_Change_EXPLORE_and_PLAN,                                                
                                    Gr_11_Average_ACT_2011,                                                
                                    Net_Change_PLAN_and_ACT,                                                
                                    College_Eligibility__,                                                
                                    Graduation_Rate__,                                                
                                    College_Enrollment_Rate__,                                                
                                    COLLEGE_ENROLLMENT,                                                
                                    General_Services_Route,                                                
                                    Freshman_on_Track_Rate__,                                                
                                    X_COORDINATE ,                                                
                                    Y_COORDINATE ,                                                
                                    LATITUDE,                                                
                                    LONGITUDE,                                         
                                    COMMUNITY_AREA_NUMBER,                                                
                                    COMMUNITY_AREA_NAME,                                                
                                    WARD,                                                
                                    Police_District,                                                
                                    LOCATION
                                    ) 
VALUES 
(610038, 'Abraham Lincoln Elementary School', 'ES', '615 W Kemper Pl', 'Chicago', 'IL', 60614, '(773) 534-5720', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'Yes', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Strong', 74.0, 'Strong', 66.0, 'Weak', 65.0, 'Strong', 70.0, 'Strong', 56.0, 'Average', 47.0, '96.00%', 2.0, '96.40%', '95.80%', 80.1, 43.3, 89.6, 84.9, 60.7, 62.6, 81.9, 85.2, 52.0, 62.4, 66.3, 77.9, 69.7, 64.4, 0.2, 0.9, 'Yellow', 'Green', 67.1, 54.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 813, 33, NULL, 1171699.4579999999, 1915829.428, 41.92449696, -87.64452163, 7, 'LINCOLN PARK', 43, 18, '(41.92449696, -87.64452163)') ,
(610281, 'Adam Clayton Powell Paideia Community Academy Elementary School', 'ES', '7511 S South Shore Dr', 'Chicago', 'IL', 60649, '(773) 535-6650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 54.0, 'Strong', 66.0, 'Strong', 74.0, 'Very Strong', 84.0, 'Weak', 63.0, 'Strong', 76.0, 'Weak', 46.0, 'Average', 50.0, '95.60%', 15.7, '95.30%', '100.00%', 62.4, 51.7, 21.9, 15.1, 29.0, 42.8, 38.5, 27.4, 44.8, 42.7, 14.1, 34.4, 16.8, 16.5, 0.7, 1.4, 'Green', 'Green', 17.2, 27.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 521, 46, NULL, 1196129.985, 1856209.466, 41.76032435, -87.55673627, 43, 'SOUTH SHORE', 7, 4, '(41.76032435, -87.55673627)') ,
(610185, 'Adlai E Stevenson Elementary School', 'ES', '8010 S Kostner Ave', 'Chicago', 'IL', 60652, '(773) 535-2280', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610185.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 61.0, NULL, NULL, 'Average', 50.0, 'Weak', 36.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Weak', 41.0, '95.70%', 2.3, '94.70%', '98.30%', 53.7, 26.6, 38.3, 34.7, 43.7, 57.3, 48.8, 39.2, 46.8, 44.0, 7.5, 21.9, 18.3, 15.5, -0.9, -1.0, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1324, 44, NULL, 1148427.165, 1851012.215, 41.74711093, -87.73170248, 70, 'ASHBURN', 13, 8, '(41.74711093, -87.73170248)') ,
(609993, 'Agustin Lara Elementary Academy', 'ES', '4619 S Wolcott Ave', 'Chicago', 'IL', 60609, '(773) 535-4389', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609993.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 56.0, 'Average', 44.0, 'Average', 45.0, 'Weak', 37.0, 'Weak', 65.0, 'Average', 48.0, 'Average', 53.0, 'Strong', 58.0, '95.50%', 10.4, '95.80%', '100.00%', 76.9, NULL, 26.0, 24.7, 61.8, 49.7, 39.2, 27.2, 69.7, 60.6, 9.1, 18.2, 11.1, 9.6, 0.9, 2.4, 'Green', 'Green', 42.9, 25.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 556, 42, NULL, 1164504.29, 1873959.199, 41.8097569, -87.6721446, 61, 'NEW CITY', 20, 9, '(41.8097569, -87.6721446)') ,
(610513, 'Air Force Academy High School', 'HS', '3630 S Wells St', 'Chicago', 'IL', 60609, '(773) 535-1590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610513.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', NULL, 'Standard', 'Not on Probation', 'Not Enough Data', 'Yes', 'Average', 49.0, 'Strong', 60.0, 'Strong', 60.0, 'Average', 55.0, 'Weak', 45.0, 'Average', 54.0, 'Average', 53.0, 'Average', 49.0, '93.30%', 15.6, '96.90%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.6, 14.8, NULL, 16.0, 1.4, NULL, NULL, NULL, NULL, NULL, 302, 40, 91.8, 1175177.622, 1880745.126, 41.82814609, -87.63279369, 34, 'ARMOUR SQUARE', 11, 9, '(41.82814609, -87.63279369)') ,
(610212, 'Albany Park Multicultural Academy', 'MS', '4929 N Sawyer Ave', 'Chicago', 'IL', 60625, '(773) 534-5108', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610212.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, 'Weak', 37.0, 'Strong', 66.0, 'Strong', 71.0, 'Weak', 43.0, 'Average', 50.0, 'Weak', 46.0, 'Average', 51.0, '97.00%', 2.3, '96.90%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 60.7, 39.8, 53.7, 59.8, 17.5, 20.8, 34.5, 15.6, 0.2, 0.3, 'Yellow', 'Yellow', 29.2, 50.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 266, 31, NULL, 1153858.196, 1932691.8909999998, 41.971143299999994, -87.70962725, 14, 'ALBANY PARK', 39, 17, '(41.9711433, -87.70962725)') ,
(609720, 'Albert G Lane Technical High School', 'HS', '2501 W Addison St', 'Chicago', 'IL', 60618, '(773) 534-5400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609720.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 88.0, NULL, NULL, 'Strong', 62.0, 'Average', 52.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '96.30%', 2.1, '96.20%', '99.40%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 19.1, 19.5, 19.9, 20.1, 1.0, 23.4, 3.5, 67.9, 92.2, 79.8, 4368, 35, 90.7, 1158975.392, 1923791.705, 41.94661693, -87.69105603, 5, 'NORTH CENTER', 47, 19, '(41.94661693, -87.69105603)') ,
(610342, 'Albert R Sabin Elementary Magnet School', 'ES', '2216 W Hirsch St', 'Chicago', 'IL', 60622, '(773) 534-4491', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610342.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 67.0, NULL, NULL, 'Weak', 30.0, 'Very Weak', 18.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.70%', 28.1, '95.00%', '100.00%', 56.3, NULL, 33.5, 33.5, 49.5, 49.5, 27.4, 39.3, 35.5, 44.4, 25.0, 23.4, 18.0, 12.8, -1.8, 0.1, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 620, 35, NULL, 1161265.299, 1909314.592, 41.90684338, -87.68304259, 24, 'WEST TOWN', 1, 14, '(41.90684338, -87.68304259)') ,
(610524, 'Alcott High School for the Humanities', 'HS', '2957 N Hoyne Ave', 'Chicago', 'IL', 60618, '(773) 534-5979', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610524.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', NULL, 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Strong', 70.0, NULL, NULL, 'Strong', 67.0, 'Average', 51.0, 'Weak', NULL, NULL, NULL, 'Strong', 57.0, 'Weak', 43.0, '92.70%', 7.1, '96.90%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.7, 13.7, NULL, 16.0, 1.3, NULL, NULL, NULL, NULL, NULL, 232, 33, 87.6, 1161870.5559999999, 1919857.44, 41.93576106, -87.68052441, 5, 'NORTH CENTER', 1, 19, '(41.93576106, -87.68052441)') ,
(610209, 'Alessandro Volta Elementary School', 'ES', '4950 N Avers Ave', 'Chicago', 'IL', 60625, '(773) 534-5080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610209.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 43.0, 'Strong', 61.0, 'Weak', 28.0, 'Weak', 37.0, 'Weak', 62.0, 'Average', 56.0, 'Average', 51.0, 'Average', 53.0, '96.40%', 22.5, '95.90%', '100.00%', 63.9, 43.2, 51.3, 32.9, 50.7, 70.0, 52.5, 31.3, 59.8, 57.6, 16.5, 24.7, 19.9, 14.2, 0.3, -0.4, 'Yellow', 'Yellow', 31.6, 65.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1023, 31, NULL, 1149774.095, 1932831.1509999998, 41.97160605, -87.72464139, 14, 'ALBANY PARK', 39, 17, '(41.97160605, -87.72464139)') ,
(609799, 'Alexander Graham Bell Elementary School', 'ES', '3730 N Oakley Ave', 'Chicago', 'IL', 60618, '(773) 534-5150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609799.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 88.0, 'Strong', 64.0, 'Average', 46.0, 'Weak', 51.0, 'Average', 51.0, NULL, NULL, NULL, NULL, '96.30%', 6.3, '95.90%', '99.30%', 91.9, 67.3, 79.2, 77.4, 53.3, 54.7, 84.2, 83.0, 49.8, 53.6, 62.6, 71.7, 64.0, 57.9, 0.0, 0.3, 'Yellow', 'Yellow', 58.1, 65.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 998, 35, NULL, 1160327.881, 1924862.7219999998, 41.94952795, -87.68605496, 5, 'NORTH CENTER', 47, 19, '(41.94952795, -87.68605496)') ,
(609947, 'Alexander Graham Elementary School', 'ES', '4436 S Union Ave', 'Chicago', 'IL', 60609, '(773) 535-1308', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609947.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 49.0, 'Weak', 30.0, 'Weak', 31.0, 'Weak', 33.0, 'Weak', 30.0, 'Average', 41.0, 'Average', 48.0, 'Average', 51.0, '92.50%', 27.4, '95.00%', '92.10%', 76.7, 47.4, 59.4, 38.8, 51.6, 76.2, 44.4, 31.3, 66.7, 49.3, 5.9, 35.3, 20.8, 11.1, 1.7, -0.5, 'Green', 'Yellow', 45.3, 31.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 476, 42, NULL, 1172335.702, 1875308.997, 41.81329195, -87.64338051, 61, 'NEW CITY', 11, 9, '(41.81329195, -87.64338051)') ,
(609963, 'Alexander Hamilton Elementary School', 'ES', '1650 W Cornelia Ave', 'Chicago', 'IL', 60657, '(773) 534-5484', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609963.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 73.0, 'Strong', 64.0, 'Strong', 60.0, 'Average', 59.0, 'Weak', 59.0, 'Average', 48.0, 'Strong', 57.0, 'Strong', 54.0, '95.30%', 12.5, '97.40%', '97.40%', 70.3, 47.2, 34.4, 48.4, 56.5, 62.9, 31.9, 46.8, 48.9, 51.1, 14.3, 35.7, 13.8, 21.3, 0.6, 0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 307, 33, NULL, 1164474.274, 1923334.414, 41.94524725, -87.67085683, 6, 'LAKE VIEW', 32, 19, '(41.94524725, -87.67085683)') ,
(610210, 'Alexander von Humboldt Elementary School', 'ES', '2620 W Hirsch St', 'Chicago', 'IL', 60622, '(773) 534-4480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610210.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 31.0, NULL, NULL, 'Weak', 32.0, 'Average', 45.0, 'Weak', NULL, NULL, NULL, 'Weak', 43.0, 'Weak', 44.0, '92.50%', 185.5, '96.00%', '100.00%', NULL, 32.6, 23.6, 24.0, 56.3, 54.0, 31.8, 19.7, 60.7, 39.7, 9.1, 20.5, 8.6, 5.7, -0.4, -1.2, 'Yellow', 'Red', 14.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 398, 34, NULL, 1158554.232, 1909250.021, 41.90672216, -87.69300325, 24, 'WEST TOWN', 26, 14, '(41.90672216, -87.69300325)') ,
(609808, 'Alex Haley Elementary Academy', 'ES', '11411 S Eggleston Ave', 'Chicago', 'IL', 60628, '(773) 535-5340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609808.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 28.0, NULL, NULL, 'Average', 58.0, 'Strong', 60.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 49.0, '94.90%', 2.9, '94.70%', '94.70%', 51.6, 31.4, 17.3, 13.6, 43.7, 41.4, 36.3, 24.5, 64.8, 53.9, 1.8, 16.1, 11.6, 1.9, 0.5, -1.5, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 525, 48, NULL, 1175378.7140000002, 1829178.573, 41.68663682, -87.63359476, 49, 'ROSELAND', 34, 22, '(41.68663682, -87.63359476)') ,
(610028, 'Alfred David Kohn Elementary School', 'ES', '10414 S State St', 'Chicago', 'IL', 60628, '(773) 535-5489', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610028.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 19.0, 'Weak', 26.0, 'Weak', 22.0, 'Very Weak', 13.0, 'Weak', 21.0, 'Very Weak', 19.0, 'Average', 47.0, 'Weak', 45.0, '90.10%', 31.2, '94.20%', '96.40%', 41.3, 31.9, 34.0, 22.2, 42.8, 56.2, 29.6, 22.8, 80.0, 61.7, 4.3, 19.6, 5.0, 5.6, 0.5, -0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 428, 48, NULL, 1178053.498, 1835837.822, 41.70485072, -87.62360199, 49, 'ROSELAND', 34, 5, '(41.70485072, -87.62360199)') ,
(610098, 'Alfred Nobel Elementary School', 'ES', '4127 W Hirsch St', 'Chicago', 'IL', 60651, '(773) 534-4365', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610098.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 37.0, 'Weak', 34.0, 'Weak', 37.0, 'Weak', 35.0, 'Weak', 48.0, 'Weak', 36.0, 'Average', 48.0, 'Average', 51.0, '94.60%', 24.8, '95.20%', '100.00%', 55.4, 33.2, 26.7, 23.2, 47.3, 50.0, 39.2, 28.1, 59.5, 51.0, 15.2, 16.7, 9.7, 7.1, -0.3, 0.7, 'Yellow', 'Yellow', 23.4, 44.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 801, 34, NULL, 1148503.002, 1908960.619, 41.90612797, -87.72993324, 23, 'HUMBOLDT PARK', 37, 25, '(41.90612797, -87.72993324)') ,
(609788, 'Alice L Barnard Computer Math & Science Center Elementary School', 'ES', '10354 S Charles St', 'Chicago', 'IL', 60643, '(773) 535-2625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609788.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 46.0, NULL, NULL, 'Weak', 39.0, 'Average', 58.0, 'Weak', NULL, NULL, NULL, 'Strong', 69.0, 'Average', 51.0, '95.60%', 28.5, '95.00%', '100.00%', 62.1, 38.5, 43.0, 38.8, 47.4, 61.8, 45.6, 40.4, 49.4, 61.4, 18.5, 29.6, 18.0, 15.7, 1.9, 1.9, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 274, 49, NULL, 1168627.899, 1835867.534, 41.70514024, -87.65811642, 72, 'BEVERLY', 19, 22, '(41.70514024, -87.65811642)') ,
(610334, 'Al Raby High School', 'HS', '3545 W Fulton Blvd', 'Chicago', 'IL', 60624, '(773) 534-6755', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610334.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 52.0, NULL, NULL, 'Average', 51.0, 'Average', 53.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '80.30%', 14.0, '95.10%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.8, 13.0, 14.1, 14.0, 1.2, 15.2, 1.1, 13.5, 72.2, 64.2, 551, 34, 74.3, 1152512.295, 1901720.85, 41.886183, -87.71539705, 27, 'EAST GARFIELD PARK', 28, 11, '(41.886183, -87.71539705)') ,
(610131, 'Ambrose Plamondon Elementary School', 'ES', '2642 W 15th Pl', 'Chicago', 'IL', 60608, '(773) 534-1789', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610131.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 45.0, NULL, NULL, 'Weak', 32.0, 'Weak', 28.0, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Weak', 41.0, '96.90%', 0.6, '96.60%', '100.00%', 61.9, 14.3, 27.3, 37.9, 35.0, 54.8, 43.5, 25.0, 74.4, 40.0, 16.7, 25.0, 12.3, 11.4, -1.5, -1.3, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 171, 39, NULL, 1158816.3020000001, 1892329.996, 41.86028668, -87.69250468, 29, 'NORTH LAWNDALE', 28, 10, '(41.86028668, -87.69250468)') ,
(610316, 'Amelia Earhart Options for Knowledge Elementary School', 'ES', '1710 E 93rd St', 'Chicago', 'IL', 60617, '(773) 535-6416', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610316.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Very Strong', 81.0, 'Strong', 63.0, 'Strong', 60.0, 'Strong', 73.0, 'Weak', 34.0, 'Average', 54.0, NULL, NULL, NULL, NULL, '96.30%', 33.4, '95.40%', '100.00%', 76.7, NULL, 58.1, 59.1, 66.7, 58.1, 48.8, 67.5, 51.3, 62.5, NULL, NULL, 27.5, 26.4, 0.1, 1.0, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 264, 47, NULL, 1189297.5890000002, 1843689.5269999998, 41.72613498, -87.58217727, 48, 'CALUMET HEIGHTS', 8, 4, '(41.72613498, -87.58217727)') ,
(609780, 'Ames Middle School', 'MS', '1920 N Hamlin Ave', 'Chicago', 'IL', 60647, '(773) 534-4970', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609780.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 38.0, 'Very Weak', 17.0, 'Weak', 27.0, 'Weak', 35.0, 'Weak', 26.0, 'Weak', 26.0, 'Average', 50.0, 'Average', 50.0, '94.30%', 5.3, '96.00%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 35.1, 27.9, 76.8, 51.1, 9.2, 18.4, 14.5, 3.0, -1.9, -1.5, 'Red', 'Red', 28.8, 55.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 586, 34, NULL, 1150696.779, 1912593.823, 41.91605519, -87.72177945, 22, 'LOGAN SQUARE', 26, 25, '(41.91605519, -87.72177945)') ,
(610339, 'Amos Alonzo Stagg Elementary School', 'ES', '7424 S Morgan St', 'Chicago', 'IL', 60621, '(773) 535-3565', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610339.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 23.0, 'Weak', 35.0, 'Weak', 35.0, 'Average', 42.0, 'Weak', 25.0, 'Average', 42.0, 'Weak', 44.0, 'Average', 50.0, '91.40%', 34.3, '95.50%', '100.00%', NULL, NULL, 35.3, 22.4, 47.6, 58.2, 20.1, 26.8, 49.2, 43.9, 8.0, 23.5, 7.7, 7.8, -0.4, -0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 539, 45, NULL, 1170892.32, 1855485.8669999999, 41.75892669, -87.6492534, 68, 'ENGLEWOOD', 17, 7, '(41.75892669, -87.6492534)') ,
(610320, 'Ana Roque de Duprey Elementary School', 'ES', '2620 W Hirsch St', 'Chicago', 'IL', 60622, '(773) 534-4230', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610320.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'Yes', 'Average', 57.0, NULL, NULL, 'Very Weak', 12.0, 'Very Weak', 14.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Strong', 57.0, '95.40%', 77.5, '95.60%', '100.00%', 32.1, 23.3, 56.8, 32.4, 75.8, 75.8, 34.9, 34.1, 53.8, 52.5, 7.1, 28.6, 11.1, 6.2, 0.7, 0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 114, 34, NULL, 1158554.232, 1909250.021, 41.90672216, -87.69300325, 24, 'WEST TOWN', 26, 14, '(41.90672216, -87.69300325)') ,
(609837, 'Andrew Carnegie Elementary School', 'ES', '1414 E 61st Pl', 'Chicago', 'IL', 60637, '(773) 535-0530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609837.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 48.0, 'Average', 51.0, 'Weak', 37.0, 'Strong', 63.0, 'Weak', 42.0, 'Weak', 31.0, 'Average', 49.0, 'Average', 49.0, '94.60%', 6.2, '94.40%', '100.00%', NULL, NULL, 40.2, 48.3, 40.8, 52.3, 29.8, 44.7, 32.6, 36.2, 18.4, 38.8, 21.4, 20.4, -1.2, 0.1, 'Red', 'Yellow', 48.1, 8.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 730, 46, NULL, 1186764.646, 1864534.4730000002, 41.78339577, -87.59079649, 42, 'WOODLAWN', 20, 3, '(41.78339577, -87.59079649)') ,
(610060, 'Andrew Jackson Elementary Language Academy', 'ES', '1340 W Harrison St', 'Chicago', 'IL', 60607, '(773) 534-7000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610060.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 86.0, NULL, NULL, 'Average', 57.0, 'Average', 46.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '97.40%', 0.7, '96.10%', '100.00%', 88.7, NULL, 90.3, 90.9, 70.6, 81.2, 85.7, 90.7, 64.8, 62.1, 72.6, 87.1, 70.4, 62.7, 0.6, 0.8, 'Green', 'Green', 48.4, 80.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 556, 38, NULL, 1167364.912, 1897512.032, 41.87432724, -87.66097599, 28, 'NEAR WEST SIDE', 2, 12, '(41.87432724, -87.66097599)') ,
(609869, 'Anna R. Langford Community Academy', 'ES', '6010 S Throop St', 'Chicago', 'IL', 60636, '(773) 535-9180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609869.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 31.0, 'Weak', 31.0, 'Average', 54.0, 'Weak', 32.0, 'Weak', 49.0, 'Average', 50.0, 'Average', 53.0, 'Average', 52.0, '91.80%', 29.0, '95.40%', '100.00%', 58.8, NULL, 40.0, 14.0, 52.8, 59.8, 31.9, 17.0, 51.3, 44.9, 7.7, 11.5, 12.6, 7.5, 0.5, 0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 339, 43, NULL, 1168649.029, 1864844.4630000002, 41.78465652, -87.6572053, 67, 'WEST ENGLEWOOD', 16, 7, '(41.78465652, -87.6572053)') ,
(610084, 'Annie Keller Elementary Gifted Magnet School', 'ES', '3020 W 108th St', 'Chicago', 'IL', 60655, '(773) 535-2636', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610084.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 97.0, 'Very Strong', 85.0, 'Very Strong', 82.0, 'Weak', 94.0, 'Very Strong', 82.0, 'Strong', 68.0, 'Strong', 60.0, '97.50%', 4.9, '96.50%', '100.00%', 100.0, NULL, 100.0, 100.0, 63.4, 74.7, 96.5, 97.7, 61.2, 76.7, 89.3, 100.0, 92.8, 92.3, 1.6, 2.3, 'Green', 'Green', 51.5, 70.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 245, 49, NULL, 1157959.455, 1832892.0669999998, 41.69719792, -87.69726379999999, 74, 'MOUNT GREENWOOD', 19, 22, '(41.69719792, -87.6972638)') ,
(610229, 'A.N. Pritzker School', 'ES', '2009 W Schiller St', 'Chicago', 'IL', 60622, '(773) 534-4415', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610229.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 59.0, NULL, NULL, 'Average', 58.0, 'Strong', 61.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.60%', 36.5, '95.60%', '98.50%', 56.8, 11.8, 58.2, 58.5, 55.0, 55.6, 62.9, 54.9, 57.6, 55.1, 38.7, 50.0, 37.2, 34.5, -0.2, 0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 669, 35, NULL, 1162693.096, 1909391.648, 41.907025, -87.67779555, 24, 'WEST TOWN', 1, 14, '(41.907025, -87.67779555)') ,
(610277, 'Anthony Overton Elementary School', 'ES', '221 E 49th St', 'Chicago', 'IL', 60615, '(773) 535-1430', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610277.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 32.0, NULL, NULL, 'Strong', 64.0, 'Strong', 76.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Strong', 55.0, '90.50%', 24.9, '94.90%', '100.00%', 52.0, 54.8, 19.6, 16.2, 41.1, 40.0, 20.4, 25.0, 45.1, 46.2, 0.0, 8.3, 8.2, 4.3, 1.2, 0.7, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 436, 42, NULL, 1178699.353, 1872549.0259999998, 41.80557578, -87.62012252, 38, 'GRAND BOULEVARD', 3, 2, '(41.80557578, -87.62012252)') ,
(609951, 'Ariel Elementary Community Academy', 'ES', '1119 E 46th St', 'Chicago', 'IL', 60653, '(773) 535-1996', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609951.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Average', 50.0, '95.50%', 5.4, '96.00%', '100.00%', 70.8, 34.3, 48.4, 43.0, 62.0, 58.6, 40.0, 47.1, 58.6, 49.3, 14.6, 36.6, 24.8, 14.4, 0.0, -1.6, 'Yellow', 'Red', 35.0, 28.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 550, 42, NULL, 1184576.517, 1874697.119, 41.81133445, -87.5985003, 39, 'KENWOOD', 4, 2, '(41.81133445, -87.5985003)') ,
(610161, 'Arna Wendell Bontemps Elementary School', 'ES', '1241 W 58th St', 'Chicago', 'IL', 60636, '(773) 535-9175', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610161.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 33.0, NULL, NULL, 'Average', 48.0, 'Average', 59.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Average', 52.0, '94.20%', 31.2, '95.80%', '100.00%', 36.4, 26.9, 25.7, 20.8, 64.3, 64.8, 28.7, 22.1, 54.3, 51.9, 0.0, 18.8, 7.5, 4.2, -1.1, -2.5, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 345, 43, NULL, 1168872.4209999999, 1866273.4440000001, 41.78857298, -87.65634499, 67, 'WEST ENGLEWOOD', 16, 7, '(41.78857298, -87.65634499)') ,
(610171, 'Arnold Mireles Elementary Academy', 'ES', '9000 S Exchange Ave', 'Chicago', 'IL', 60617, '(773) 535-6360', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610171.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 27.0, NULL, NULL, 'Average', 43.0, 'Average', 54.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 51.0, '92.80%', 11.8, '95.20%', '98.90%', 44.0, 27.5, 21.9, 15.2, 45.4, 45.7, 23.0, 19.4, 46.9, 51.0, 6.4, 18.2, 7.6, 5.4, -1.1, -0.7, 'Red', 'Red', 9.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 905, 47, NULL, 1197292.129, 1845782.158, 41.73168217, -87.55282373, 46, 'SOUTH CHICAGO', 10, 4, '(41.73168217, -87.55282373)') ,
(610037, 'Arthur A Libby Elementary School', 'ES', '5300 S Loomis Blvd', 'Chicago', 'IL', 60609, '(773) 535-9050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610037.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, NULL, NULL, 'Strong', 60.0, 'Strong', 60.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 52.0, '90.80%', 21.6, '95.10%', '100.00%', 31.1, 37.3, 24.7, 12.0, 43.2, 48.3, 32.0, 17.7, 55.3, 48.3, 4.2, 14.6, 4.8, 3.9, -0.4, -2.8, 'Yellow', 'Red', 18.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 539, 43, NULL, 1167858.322, 1869572.5019999999, 41.79764784, -87.65996858, 61, 'NEW CITY', 16, 9, '(41.79764784, -87.65996858)') ,
(609887, 'Arthur Dixon Elementary School', 'ES', '8306 S Saint Lawrence Ave', 'Chicago', 'IL', 60619, '(773) 535-3834', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609887.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 47.0, NULL, NULL, 'Average', 47.0, 'Average', 52.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.80%', 32.2, '95.60%', '100.00%', 69.7, 35.5, 42.8, 37.8, 51.8, 70.8, 43.5, 44.5, 51.3, 49.4, 22.1, 31.6, 22.0, 15.7, -0.3, 0.7, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 652, 45, NULL, 1181644.805, 1850009.5769999998, 41.74365776, -87.61001515, 44, 'CHATHAM', 6, 6, '(41.74365776, -87.61001515)') ,
(609836, 'Arthur E Canty Elementary School', 'ES', '3740 N Panama Ave', 'Chicago', 'IL', 60634, '(773) 534-1238', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609836.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, 'Strong', 64.0, 'Strong', 70.0, 'Strong', 64.0, 'Weak', 49.0, 'Average', 47.0, 'Strong', 66.0, 'Average', 53.0, '96.10%', 2.3, '96.10%', '99.10%', 75.3, 60.6, 77.2, 58.3, 52.9, 62.0, 81.3, 71.3, 61.4, 59.7, 41.3, 60.0, 47.4, 34.2, 0.3, 0.1, 'Yellow', 'Yellow', 29.8, 100.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 784, 30, NULL, 1121355.6979999999, 1924041.4119999998, 41.94799245, -87.8293319, 17, 'DUNNING', 36, 16, '(41.94799245, -87.8293319)') ,
(610268, 'Arthur R Ashe Elementary School', 'ES', '8505 S Ingleside Ave', 'Chicago', 'IL', 60619, '(773) 535-3550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610268.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Weak', 39.0, 'Weak', 46.0, '87.90%', 41.6, '93.40%', '100.00%', 30.1, 25.5, 20.3, 14.3, 47.4, 54.8, 43.4, 23.6, 61.0, 58.7, 7.9, 15.8, 9.2, 2.8, 1.1, -0.9, 'Green', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 459, 45, NULL, 1184078.6609999998, 1848752.991, 41.74015307, -87.60113659, 44, 'CHATHAM', 8, 6, '(41.74015307, -87.60113659)') ,
(609941, 'Asa Philip Randolph Elementary School', 'ES', '7316 S Hoyne Ave', 'Chicago', 'IL', 60636, '(773) 535-9015', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609941.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 15.0, NULL, NULL, 'Average', 41.0, 'Average', 48.0, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Average', 49.0, '92.10%', 43.5, '95.30%', '95.60%', 52.9, 52.6, 20.2, 11.3, 40.1, 48.1, 44.2, 21.3, 63.0, 50.8, 12.1, 12.1, 13.8, 4.5, 0.3, -1.3, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 573, 43, NULL, 1163589.689, 1856037.765, 41.76059741, -87.67600175, 67, 'WEST ENGLEWOOD', 17, 7, '(41.76059741, -87.67600175)') ,
(610287, 'Ashburn Community Elementary School', 'ES', '8300 S Saint Louis Ave', 'Chicago', 'IL', 60652, '(773) 535-7860', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610287.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 48.0, 'Strong', 76.0, 'Average', 52.0, 'Average', 40.0, 'Weak', 68.0, 'Strong', 70.0, 'Average', 51.0, 'Average', 50.0, '95.50%', 28.5, '97.10%', '100.00%', 70.6, 61.7, 40.4, 33.3, 55.5, 56.2, 45.8, 36.8, 69.9, 45.5, 22.4, 30.6, 15.1, 9.5, 0.4, -0.6, 'Yellow', 'Yellow', 40.8, 40.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 490, 44, NULL, 1154458.76, 1849313.602, 41.74233179, -87.70964592, 70, 'ASHBURN', 18, 8, '(41.74233179, -87.70964592)') ,
(609820, 'Augustus H Burley Elementary School', 'ES', '1630 W Barry Ave', 'Chicago', 'IL', 60657, '(773) 534-5475', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609820.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, NULL, NULL, 'Strong', 78.0, 'Strong', 65.0, 'Weak', NULL, NULL, NULL, 'Strong', 59.0, 'Average', 49.0, '96.50%', 0.7, '95.00%', '97.90%', 69.4, 47.0, 64.5, 70.2, 51.2, 66.7, 69.4, 75.0, 62.9, 67.3, 50.0, 69.6, 54.2, 53.3, 1.3, 2.1, 'Green', 'Green', 35.7, 80.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 572, 33, NULL, 1164768.164, 1920682.831, 41.93796493, -87.66985204, 6, 'LAKE VIEW', 32, 19, '(41.93796493, -87.66985204)') ,
(400018, 'Austin Business and Entrepreneurship Academy High School', 'HS', '231 N Pine Ave', 'Chicago', 'IL', 60644, '(773) 534-6316', NULL, 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Non_Standard', 'Probation', 'Level 3', 'No', 'Weak', 38.0, NULL, NULL, 'Average', 41.0, 'Weak', 33.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '76.00%', 12.4, '0.00%', '98.20%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.0, 12.0, 14.3, 12.9, 0.9, 14.4, 0.1, 12.2, 65.6, 51.3, 316, 36, NULL, 1139494.763, 1901274.2580000001, 41.88520477, -87.76321191, 25, 'AUSTIN', 28, 15, '(41.88520477, -87.76321191)') ,
(610169, 'Austin O Sexton Elementary School', 'ES', '6020 S Langley Ave', 'Chicago', 'IL', 60637, '(773) 535-0640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610169.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 23.0, NULL, NULL, 'Weak', 38.0, 'Weak', 34.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Weak', 44.0, '90.50%', 8.6, '94.10%', '100.00%', 48.8, NULL, 12.7, 13.5, 56.6, 51.9, 26.4, 33.6, 53.1, 66.7, 3.7, 11.1, 6.1, 5.2, 0.2, -0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 337, 42, NULL, 1181884.66, 1865139.761, 41.78517099, -87.60866927, 42, 'WOODLAWN', 20, 3, '(41.78517099, -87.60866927)') ,
(610501, 'Austin Polytechnical Academy High School', 'HS', '231 N Pine Ave', 'Chicago', 'IL', 60644, '(773) 534-6300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610501.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 38.0, NULL, NULL, 'Average', 45.0, 'Average', 53.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '70.10%', 21.5, '95.20%', '98.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.0, 12.1, 13.6, 13.1, 1.1, 14.8, 1.2, 8.5, NULL, NULL, 250, 36, 56.1, 1139494.763, 1901274.2580000001, 41.88520477, -87.76321191, 25, 'AUSTIN', 28, 15, '(41.88520477, -87.76321191)') ,
(609786, 'Avalon Park Elementary School', 'ES', '8045 S Kenwood Ave', 'Chicago', 'IL', 60619, '(773) 535-6615', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609786.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 32.0, 'Weak', 24.0, 'Weak', 38.0, 'Weak', 32.0, 'Weak', 32.0, 'Very Weak', 16.0, 'Weak', 46.0, 'Weak', 45.0, '92.10%', 52.1, '96.40%', '97.70%', 47.8, 15.4, 11.9, 22.3, 41.7, 40.9, 14.8, 20.6, 42.7, 44.1, 2.2, 24.4, 7.9, 5.1, 1.1, 1.2, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 398, 47, NULL, 1186651.081, 1851731.65, 41.74826636, -87.59161761, 45, 'AVALON PARK', 8, 4, '(41.74826636, -87.59161761)') ,
(609871, 'Barbara Vick Early Childhood & Family Center', 'ES', '2554 W 113th St', 'Chicago', 'IL', 60655, '(773) 535-2671', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609871.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', NULL, 'Track_E', NULL, NULL, 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Strong', 66.0, 'Strong', 60.0, '60.90%', 0.0, '95.50%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 204, 49, NULL, 1161152.12, 1829643.669, 41.68821843, -87.68566340000001, 75, 'MORGAN PARK', 19, 22, '(41.68821843, -87.6856634)') ,
(609922, 'Belmont-Cragin Elementary School', 'ES', '2456 N Mango Ave', 'Chicago', 'IL', 60639, '(773) 534-2900', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609922.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 75.0, 'Very Strong', 81.0, 'Strong', 67.0, 'Strong', 66.0, 'Weak', 45.0, 'Average', 53.0, 'Strong', 55.0, 'Strong', 54.0, '95.30%', 1.3, '96.80%', '98.90%', 60.7, 31.8, 22.5, 21.3, 51.8, 55.3, 42.9, 39.0, 61.7, 53.2, 0.0, 6.9, 11.0, 11.0, 0.2, 1.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 634, 29, NULL, 1137592.32, 1915980.764, 41.92559567, -87.76984337, 19, 'BELMONT CRAGIN', 30, 25, '(41.92559567, -87.76984337)') ,
(609764, 'Benito Juarez Community Academy High School', 'HS', '2150 S Laflin St', 'Chicago', 'IL', 60608, '(773) 534-7030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609764.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 46.0, 'Average', 53.0, 'Average', 44.0, 'Average', 45.0, 'Weak', 49.0, 'Average', 48.0, NULL, NULL, NULL, NULL, '83.70%', 17.2, '95.80%', '99.50%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.2, 13.7, 14.6, 14.5, 1.3, 15.8, 1.2, 14.8, 56.2, 48.8, 1683, 39, 71.1, 1166665.696, 1889615.3480000002, 41.85267306, -87.66376931, 31, 'LOWER WEST SIDE', 25, 12, '(41.85267306, -87.66376931)') ,
(610265, 'Benjamin Banneker Elementary School', 'ES', '6656 S Normal Blvd', 'Chicago', 'IL', 60621, '(773) 535-3020', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610265.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, NULL, NULL, 'Weak', 35.0, 'Average', 58.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '91.30%', 26.9, '93.90%', '100.00%', NULL, 31.1, 10.9, 11.8, 40.0, 34.4, 34.4, 31.1, 55.7, 60.5, 25.9, 30.8, 7.9, 1.5, 0.7, 0.9, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 302, 45, NULL, 1174114.357, 1860550.7240000002, 41.77275433, -87.63729464, 68, 'ENGLEWOOD', 6, 7, '(41.77275433, -87.63729464)') ,
(610290, 'Benjamin E Mays Elementary Academy', 'ES', '838 W Marquette Rd', 'Chicago', 'IL', 60621, '(773) 535-3892', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610290.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 27.0, 'Weak', 23.0, 'Average', 55.0, 'Average', 48.0, 'Weak', 41.0, 'Weak', 37.0, NULL, NULL, NULL, NULL, '95.50%', 0.3, '96.60%', '100.00%', NULL, 60.7, 45.9, 26.8, 47.2, 60.2, 37.2, 24.7, 60.7, 57.8, 17.1, 31.4, 17.4, 9.6, 0.1, -0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 301, 45, NULL, 1171705.53, 1860460.8280000002, 41.77256081, -87.64612737, 68, 'ENGLEWOOD', 16, 7, '(41.77256081, -87.64612737)') ,
(610076, 'Bernhard Moos Elementary School', 'ES', '1711 N California Ave', 'Chicago', 'IL', 60647, '(773) 534-4340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610076.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 50.0, 'Weak', 34.0, 'Average', 47.0, 'Weak', 32.0, 'Weak', 34.0, 'Weak', 31.0, NULL, NULL, NULL, NULL, '93.00%', 45.5, '96.30%', '98.50%', 52.7, 52.4, 43.0, 26.5, 60.8, 61.6, 60.6, 30.5, 70.8, 61.3, 10.9, 12.7, 24.0, 7.5, 1.4, 0.2, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 429, 34, NULL, 1157476.303, 1911358.8969999999, 41.912531099999995, -87.69690545, 24, 'WEST TOWN', 1, 14, '(41.9125311, -87.69690545)') ,
(610150, 'Betsy Ross Elementary School', 'ES', '6059 S Wabash Ave', 'Chicago', 'IL', 60637, '(773) 535-0650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610150.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 39.0, NULL, NULL, 'Average', 51.0, 'Average', 50.0, 'Weak', NULL, NULL, NULL, 'Weak', 37.0, 'Weak', 44.0, '88.90%', 64.1, '93.90%', '97.90%', 25.3, 31.0, 10.3, 8.2, 31.5, 22.8, 24.4, 20.2, 56.3, 38.1, 0.0, 5.3, 2.5, 2.0, -0.1, -1.1, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 350, 42, NULL, 1177801.979, 1864618.42, 41.78383382, -87.62365383, 40, 'WASHINGTON PARK', 20, 3, '(41.78383382, -87.62365383)') ,
(610175, 'Beulah Shoesmith Elementary School', 'ES', '1330 E 50th St', 'Chicago', 'IL', 60615, '(773) 535-1764', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610175.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 45.0, 'Average', 54.0, 'Average', 40.0, 'Weak', 39.0, 'Weak', 37.0, 'Average', 41.0, 'Average', 49.0, 'Weak', 45.0, '94.70%', 0.3, '94.80%', '100.00%', 65.0, 12.0, 32.5, 28.4, 54.4, 61.3, 56.4, 43.6, 83.3, 66.7, NULL, NULL, 10.8, 19.6, 0.0, 1.6, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 325, 46, NULL, 1185846.553, 1872152.239, 41.80432122, -87.59392226, 39, 'KENWOOD', 4, 2, '(41.80432122, -87.59392226)') ,
(610087, 'Blair Early Childhood Center', 'ES', '6751 W 63rd Pl', 'Chicago', 'IL', 60638, '(773) 535-2076', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610087.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', NULL, 'Track_E', NULL, NULL, 'No', NULL, NULL, 'Strong', 64.0, NULL, NULL, NULL, NULL, 'Weak', 61.0, 'Strong', 66.0, 'Strong', 58.0, 'Strong', 65.0, '80.20%', 0.8, '94.80%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 137, 44, NULL, 1132295.745, 1861625.19, 41.77652935, -87.79056836, 64, 'CLEARING', 23, 8, '(41.77652935, -87.79056836)') ,
(609969, 'Bret Harte Elementary School', 'ES', '1556 E 56th St', 'Chicago', 'IL', 60637, '(773) 535-0870', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609969.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 63.0, NULL, NULL, 'Strong', 77.0, 'Strong', 76.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Weak', 44.0, '94.20%', 17.9, '95.40%', '98.50%', 57.1, NULL, 52.8, 60.0, 66.1, 60.8, 43.8, 60.4, 60.4, 64.6, NULL, NULL, 27.5, 21.9, 0.4, 0.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 340, 46, NULL, 1187805.4840000002, 1868216.424, 41.7934746, -87.58686328, 41, 'HYDE PARK', 5, 2, '(41.7934746, -87.58686328)') ,
(610106, 'Brian Piccolo Elementary Specialty School', 'ES', '1040 N Keeler Ave', 'Chicago', 'IL', 60651, '(773) 534-4425', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610106.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 27.0, NULL, NULL, 'Weak', 29.0, 'Weak', 31.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '90.60%', 65.8, '95.00%', '92.90%', 29.2, 19.6, 9.7, 15.2, 40.9, 43.0, 20.6, 16.5, 51.3, 48.1, 6.8, 5.1, 3.4, 1.6, -0.2, -1.4, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 549, 34, NULL, 1148146.922, 1906778.22, 41.9001461, -87.73129753, 23, 'HUMBOLDT PARK', 37, 11, '(41.9001461, -87.73129753)') ,
(610317, 'Brighton Park Elementary School', 'ES', '3825 S Washtenaw Ave', 'Chicago', 'IL', 60632, '(773) 535-7237', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610317.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 55.0, NULL, NULL, 'Average', 49.0, 'Average', 43.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '96.30%', 2.6, '95.90%', '100.00%', 64.5, 50.0, 37.7, 32.2, 51.2, 58.8, 46.1, 39.7, 64.7, 53.9, 6.0, 20.9, 18.6, 11.4, 0.8, 0.2, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 691, 39, NULL, 1159041.6609999998, 1879076.7769999998, 41.82391375, -87.69204069, 58, 'BRIGHTON PARK', 12, 9, '(41.82391375, -87.69204069)') ,
(610381, 'Bronzeville Scholastic Academy High School', 'HS', '4934 S Wabash Ave', 'Chicago', 'IL', 60615, '(773) 535-1150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610381.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 41.0, 'Weak', 38.0, 'Average', 42.0, 'Average', 43.0, 'Weak', 32.0, 'Very Weak', 15.0, 'Average', 51.0, 'Weak', 43.0, '92.30%', 16.2, '95.60%', '97.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.3, 13.8, 15.3, 15.7, 1.4, 16.3, 1.0, 14.7, 85.3, 73.2, 550, 42, 71.1, 1177513.255, 1872170.4440000001, 41.80456384, -87.62448408, 38, 'GRAND BOULEVARD', 3, 2, '(41.80456384, -87.62448408)') ,
(609821, 'Burnham Elementary Inclusive Academy', 'ES', '1903 E 96th St', 'Chicago', 'IL', 60617, '(773) 535-6530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609821.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 61.0, 'Strong', 77.0, 'Very Strong', 85.0, 'Very Strong', 99.0, 'Weak', 65.0, 'Strong', 71.0, 'Strong', 54.0, 'Average', 47.0, '95.50%', 29.4, '96.90%', '96.00%', 75.9, 65.1, 59.1, 43.6, 61.1, 79.6, 77.2, 58.7, 57.5, 65.5, 8.7, 26.1, 36.5, 22.0, 1.3, 0.6, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 286, 47, NULL, 1190566.496, 1841647.9530000002, 41.72050221, -87.57759489, 51, 'SOUTH DEERING', 7, 4, '(41.72050221, -87.57759489)') ,
(609827, 'Burnside Elementary Scholastic Academy', 'ES', '650 E 91st Pl', 'Chicago', 'IL', 60619, '(773) 535-3300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609827.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 51.0, NULL, NULL, 'Average', 58.0, 'Strong', 70.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 47.0, '94.90%', 8.9, '94.90%', '100.00%', 81.7, 55.8, 31.1, 38.5, 49.3, 39.9, 40.6, 33.0, 52.4, 49.4, 18.6, 35.6, 15.5, 14.3, 0.0, -0.4, 'Yellow', 'Yellow', 20.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 766, 45, NULL, 1182385.6509999998, 1844494.844, 41.72850759, -87.60747108, 44, 'CHATHAM', 6, 6, '(41.72850759, -87.60747108)') ,
(610353, 'Calmeca Academy of Fine Arts and Dual Language', 'ES', '3456 W 38th St', 'Chicago', 'IL', 60632, '(773) 535-7000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610353.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', 'Average', 58.0, 'Strong', 67.0, 'Average', 59.0, 'Strong', 72.0, 'Weak', 64.0, 'Average', 55.0, 'Average', 51.0, 'Strong', 57.0, '96.50%', 1.9, '96.90%', '100.00%', 47.0, NULL, 44.2, 23.9, 52.8, 64.5, 35.8, 26.6, 49.8, 49.3, 16.1, 21.0, 13.9, 10.3, -0.7, 1.5, 'Red', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 792, 39, NULL, 1153706.842, 1879216.756, 41.82440547, -87.71160875, 58, 'BRIGHTON PARK', 12, 9, '(41.82440547, -87.71160875)') ,
(609729, 'Carl Schurz High School', 'HS', '3601 N Milwaukee Ave', 'Chicago', 'IL', 60641, '(773) 534-3420', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609729.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 48.0, 'Weak', 36.0, 'Weak', 36.0, 'Weak', 29.0, 'Weak', 30.0, 'Weak', 31.0, NULL, NULL, NULL, NULL, '81.20%', 13.8, '96.00%', '98.90%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.1, 12.9, 14.3, 14.0, 0.9, 15.6, 1.3, 9.7, 62.1, 40.4, 2366, 29, 61.6, 1146851.1979999999, 1923627.843, 41.94640794, -87.73562542, 16, 'IRVING PARK', 38, 17, '(41.94640794, -87.73562542)') ,
(610039, 'Carl von Linne Elementary School', 'ES', '3221 N Sacramento Ave', 'Chicago', 'IL', 60618, '(773) 534-5262', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610039.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 68.0, 'Average', 49.0, 'Strong', 66.0, 'Strong', 75.0, 'Weak', 37.0, 'Average', 44.0, 'Average', 50.0, 'Strong', 55.0, '95.80%', 1.8, '96.30%', '100.00%', 66.1, 41.2, 40.2, 29.7, 51.3, 72.0, 49.4, 44.2, 64.4, 67.6, 20.4, 40.7, 24.8, 15.1, 1.2, 0.9, 'Green', 'Green', 48.2, 24.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 644, 31, NULL, 1155853.119, 1921364.031, 41.94001885, -87.70259825, 21, 'AVONDALE', 33, 17, '(41.94001885, -87.70259825)') ,
(610238, 'Carrie Jacobs Bond Elementary School', 'ES', '7050 S May St', 'Chicago', 'IL', 60621, '(773) 535-3480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610238.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 54.0, 'Average', 49.0, 'Strong', 67.0, 'Strong', 63.0, 'Weak', 39.0, 'Weak', 32.0, 'Average', 53.0, 'Average', 52.0, '91.70%', 18.8, '94.30%', '96.90%', NULL, 56.5, 34.0, 16.1, 41.7, 48.3, 41.3, 23.8, 65.8, 52.8, 7.7, 11.5, 7.9, 5.6, -0.2, 0.9, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 343, 45, NULL, 1169836.4440000001, 1857831.503, 41.76538639, -87.65305514, 68, 'ENGLEWOOD', 17, 7, '(41.76538639, -87.65305514)') ,
(609839, 'Carroll-Rosenwald Specialty Elementary School', 'ES', '2929 W 83rd St', 'Chicago', 'IL', 60652, '(773) 535-9414', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609839.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, NULL, NULL, 'Very Weak', 12.0, 'Very Weak', 12.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 47.0, '94.80%', 27.8, '95.60%', '100.00%', 63.1, 47.5, 26.7, 29.4, 52.0, 38.4, 37.1, 27.7, 45.1, 43.5, 13.8, 30.0, 11.8, 9.1, -0.6, 0.6, 'Red', 'Yellow', 10.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 537, 43, NULL, 1158181.605, 1849398.28, 41.74248931, -87.696003, 70, 'ASHBURN', 18, 8, '(41.74248931, -87.696003)') ,
(610345, 'Carter G Woodson South Elementary School', 'ES', '4414 S Evans', 'Chicago', 'IL', 60653, '(773) 535-1280', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610345.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 26.0, 'Weak', 27.0, 'Very Weak', 2.0, 'Very Weak', 1.0, 'Weak', 30.0, 'Average', 41.0, 'Weak', 44.0, 'Average', 47.0, '91.90%', 70.5, '94.70%', '98.40%', 65.3, 50.8, 9.8, 6.3, 31.7, 33.3, 9.1, 18.2, 33.7, 40.4, 8.0, 16.0, 4.3, 2.4, -0.7, -2.4, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 329, 40, NULL, 1181853.018, 1875820.129, 41.81447955, -87.60845505, 38, 'GRAND BOULEVARD', 3, 2, '(41.81447955, -87.60845505)') ,
(610148, 'Cesar E Chavez Multicultural Academic Center Elementary School', 'ES', '4747 S Marshfield Ave', 'Chicago', 'IL', 60609, '(773) 535-4600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610148.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 54.0, 'Average', 52.0, 'Strong', 63.0, 'Average', 52.0, 'Weak', 57.0, 'Average', 50.0, 'Weak', 45.0, 'Average', 51.0, '95.10%', 11.1, '96.10%', '100.00%', 59.6, 37.8, 37.7, 16.5, 49.8, 74.2, 62.7, 28.0, 80.7, 63.3, 9.2, 10.8, 19.8, 8.4, 2.0, 0.0, 'Green', 'Yellow', 44.6, 18.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 915, 42, NULL, 1166186.662, 1873024.0790000001, 41.80715514, -87.66600055, 61, 'NEW CITY', 20, 9, '(41.80715514, -87.66600055)') ,
(609679, 'Charles Allen Prosser Career Academy High School', 'HS', '2148 N Long Ave', 'Chicago', 'IL', 60639, '(773) 534-3200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609679.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, 'Average', 47.0, 'Average', 53.0, 'Average', 51.0, 'Weak', 37.0, 'Average', 44.0, 'Weak', 46.0, 'Weak', 45.0, '86.00%', 9.7, '94.40%', '99.40%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 15.5, 15.1, 16.3, 16.2, 0.7, 17.9, 1.6, 27.7, 78.7, 58.0, 1415, 29, 79.4, 1139989.443, 1913970.649, 41.92003613, -87.76108432, 19, 'BELMONT CRAGIN', 37, 25, '(41.92003613, -87.76108432)') ,
(610005, 'Charles Evans Hughes Elementary School', 'ES', '4247 W 15th St', 'Chicago', 'IL', 60623, '(773) 534-1762', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610005.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, NULL, NULL, 'Very Strong', 82.0, 'Strong', 72.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.40%', 0.0, '94.70%', '100.00%', 69.2, 63.3, 32.6, 16.9, 40.3, 76.0, 54.8, 20.8, 69.6, 45.8, 5.9, 23.5, 12.2, 4.7, 0.5, -1.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 315, 36, NULL, 1148118.259, 1892326.805, 41.86049033, -87.73177486, 29, 'NORTH LAWNDALE', 24, 10, '(41.86049033, -87.73177486)') ,
(609879, 'Charles Gates Dawes Elementary School', 'ES', '3810 W 81st Pl', 'Chicago', 'IL', 60652, '(773) 535-2350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609879.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 44.0, 'Weak', 38.0, 'Weak', 27.0, 'Weak', 30.0, 'Weak', 60.0, 'Average', 54.0, 'Average', 50.0, 'Average', 48.0, '95.70%', 35.6, '95.60%', '99.50%', 54.3, 24.0, 39.9, 42.9, 66.9, 76.5, 52.3, 56.2, 70.1, 65.3, 19.8, 35.5, 23.1, 13.6, 0.5, 0.3, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1032, 44, NULL, 1152339.16, 1850308.795, 41.74510467, -87.71738613, 70, 'ASHBURN', 18, 8, '(41.74510467, -87.71738613)') ,
(609966, 'Charles G Hammond Elementary School', 'ES', '2819 W 21st Pl', 'Chicago', 'IL', 60623, '(773) 535-4580', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609966.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 40.0, NULL, NULL, 'Average', 41.0, 'Average', 43.0, 'Weak', NULL, NULL, NULL, 'Weak', 43.0, 'Weak', 45.0, '95.30%', 8.5, '95.20%', '92.00%', 77.4, NULL, 27.4, 24.8, 62.4, 53.7, 38.1, 28.8, 60.2, 59.8, 12.8, 5.3, 10.1, 5.8, 0.5, 0.2, 'Yellow', 'Yellow', 25.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 446, 39, NULL, 1157809.015, 1889554.6230000001, 41.85269133, -87.69627777, 30, 'SOUTH LAWNDALE', 12, 10, '(41.85269133, -87.69627777)') ,
(610366, 'Charles H Wacker Elementary School', 'ES', '9746 S Morgan St', 'Chicago', 'IL', 60643, '(773) 535-2821', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610366.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 32.0, NULL, NULL, 'Weak', 34.0, 'Weak', 32.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Weak', 41.0, '93.40%', 11.5, '93.40%', '100.00%', 64.5, 53.9, 20.8, 31.0, 34.8, 40.6, 66.7, 60.0, 85.7, 67.2, 7.7, 23.1, 17.4, 15.9, 1.8, 0.0, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 246, 49, NULL, 1171336.009, 1839924.624, 41.716214799999996, -87.6480814, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.7162148, -87.6480814)') ,
(610030, 'Charles Kozminski Elementary Community Academy', 'ES', '936 E 54th St', 'Chicago', 'IL', 60615, '(773) 535-0980', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610030.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, NULL, NULL, 'Weak', 35.0, 'Average', 40.0, 'Weak', NULL, NULL, NULL, 'Weak', 39.0, 'Weak', 42.0, '92.00%', 55.6, '94.40%', '100.00%', 60.0, 34.9, 27.5, 24.6, 36.9, 33.9, 14.7, 23.8, 37.9, 43.4, 6.5, 18.8, 5.0, 6.1, -1.2, -1.1, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 403, 46, NULL, 1183537.482, 1869839.6509999998, 41.7980295, -87.60246287, 41, 'HYDE PARK', 4, 2, '(41.7980295, -87.60246287)') ,
(609996, 'Charles N Holden Elementary School', 'ES', '1104 W 31st St', 'Chicago', 'IL', 60608, '(773) 535-7200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609996.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 53.0, 'Average', 44.0, 'Average', 50.0, 'Weak', 35.0, 'Weak', 53.0, 'Average', 51.0, 'Strong', 54.0, 'Average', 50.0, '94.90%', 4.7, '96.00%', '100.00%', 53.2, NULL, 33.9, 26.1, 49.2, 45.7, 56.8, 37.4, 57.3, 58.2, 28.0, 37.5, 16.7, 11.5, -0.8, 0.0, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 600, 40, NULL, 1169423.887, 1884309.813, 41.83805473, -87.65380002, 60, 'BRIDGEPORT', 11, 9, '(41.83805473, -87.65380002)') ,
(609833, 'Charles P Caldwell Academy of Math & Science Elementary School', 'ES', '8546 S Cregier', 'Chicago', 'IL', 60617, '(773) 535-6300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609833.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 25.0, 'Weak', 31.0, 'Average', 40.0, 'Average', 44.0, 'Weak', 28.0, 'Weak', 25.0, 'Weak', 44.0, 'Average', 47.0, '92.70%', 30.7, '94.70%', '100.00%', 60.6, 67.6, 25.3, 23.4, 54.0, 38.2, 42.7, 26.5, 52.8, 48.1, 4.8, 14.3, 10.9, 6.8, 0.9, -0.1, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 291, 47, NULL, 1189641.205, 1848463.6269999999, 41.73922735, -87.58076569, 45, 'AVALON PARK', 8, 4, '(41.73922735, -87.58076569)') ,
(609732, 'Charles P Steinmetz Academic Centre High School', 'HS', '3030 N Mobile Ave', 'Chicago', 'IL', 60634, '(773) 534-3030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609732.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 26.0, 'Weak', 35.0, 'Weak', 29.0, 'Weak', 34.0, 'Weak', 36.0, 'Weak', 32.0, NULL, NULL, NULL, NULL, '79.20%', 30.4, '96.20%', '99.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.0, 13.8, 13.8, 14.3, 1.3, 16.1, 2.3, 16.3, 56.7, 47.0, 1890, 30, 70.3, 1133838.348, 1919613.6940000001, 41.93563171, -87.78355194, 19, 'BELMONT CRAGIN', 36, 25, '(41.93563171, -87.78355194)') ,
(609875, 'Charles R Darwin Elementary School', 'ES', '3116 W Belden Ave', 'Chicago', 'IL', 60647, '(773) 534-4110', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609875.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, 'Average', 40.0, 'Average', 44.0, 'Weak', 34.0, 'Weak', 31.0, 'Average', 46.0, 'Average', 49.0, 'Strong', 54.0, '95.60%', 33.8, '96.20%', '100.00%', 50.3, 35.6, 37.7, 33.5, 59.6, 68.5, 29.6, 36.1, 60.6, 51.2, 16.7, 19.2, 16.0, 6.9, -0.2, -0.8, 'Yellow', 'Red', 21.0, 50.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 622, 35, NULL, 1155126.902, 1915284.2030000002, 41.92334998, -87.70543099, 22, 'LOGAN SQUARE', 26, 14, '(41.92334998, -87.70543099)') ,
(609986, 'Charles R Henderson Elementary School', 'ES', '5650 S Wolcott Ave', 'Chicago', 'IL', 60636, '(773) 535-9080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609986.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 16.0, NULL, NULL, 'Weak', 32.0, 'Weak', 38.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Weak', 44.0, '89.20%', 80.6, '93.60%', '100.00%', 38.8, 23.1, 21.6, 18.0, 52.8, 55.9, 18.8, 14.4, 60.6, 45.3, 4.4, 17.8, 4.8, 2.6, -0.1, -0.6, 'Yellow', 'Yellow', 8.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 407, 43, NULL, 1164617.918, 1866985.2580000001, 41.79061716, -87.67192466, 67, 'WEST ENGLEWOOD', 15, 7, '(41.79061716, -87.67192466)') ,
(609813, 'Charles S Brownell Elementary School', 'ES', '6741 S Michigan Ave', 'Chicago', 'IL', 60637, '(773) 535-3030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609813.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 20.0, 'Very Weak', 16.0, 'Strong', 79.0, 'Very Strong', 82.0, 'Weak', 20.0, 'Very Weak', 19.0, 'Average', 48.0, 'Weak', 46.0, '92.80%', 14.1, '95.50%', '100.00%', NULL, NULL, 24.3, 17.0, 38.8, 63.1, 36.8, 32.4, 67.6, 47.2, NULL, NULL, 12.9, 8.8, 1.5, -0.4, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 310, 42, NULL, 1178368.986, 1860177.542, 41.77163474, -87.62170972, 69, 'GREATER GRAND CROSSING', 20, 3, '(41.77163474, -87.62170972)') ,
(609883, 'Charles S Deneen Elementary School', 'ES', '7240 S Wabash Ave', 'Chicago', 'IL', 60619, '(773) 535-3035', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609883.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 55.0, 'Weak', 33.0, 'Average', 49.0, 'Average', 51.0, 'Weak', 15.0, 'Average', 49.0, NULL, NULL, NULL, NULL, '93.80%', 39.5, '97.10%', '100.00%', 45.9, NULL, 20.6, 18.4, 51.8, 44.3, 20.0, 26.8, 61.3, 54.4, 2.2, 4.4, 9.4, 6.8, 2.8, -0.5, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 551, 46, NULL, 1177940.398, 1856846.4530000002, 41.76250359, -87.62338156, 69, 'GREATER GRAND CROSSING', 6, 3, '(41.76250359, -87.62338156)') ,
(610194, 'Charles Sumner  Math & Science Community Acad Elementary School', 'ES', '4320 W 5th Ave', 'Chicago', 'IL', 60624, '(773) 534-6730', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610194.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 41.0, 'Weak', 27.0, 'Average', 44.0, 'Average', 48.0, 'Weak', 30.0, 'Weak', 34.0, 'Average', 48.0, 'Weak', 46.0, '93.90%', 12.3, '96.90%', '100.00%', 49.0, NULL, 28.4, 33.1, 50.0, 54.7, 31.9, 36.3, 59.3, 47.4, 15.6, 11.1, 13.1, 9.7, -1.0, 0.1, 'Red', 'Yellow', 48.9, 23.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 430, 36, NULL, 1147607.2, 1896098.17, 41.87084922, -87.73355414, 26, 'WEST GARFIELD PARK', 24, 11, '(41.87084922, -87.73355414)') ,
(609897, 'Charles W Earle Elementary School', 'ES', '6121 S Hermitage Ave', 'Chicago', 'IL', 60636, '(773) 535-9130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609897.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 24.0, NULL, NULL, 'Strong', 62.0, 'Strong', 71.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Average', 48.0, '93.80%', 16.9, '94.60%', '100.00%', 37.5, 36.8, 44.1, 20.8, 56.1, 73.8, 38.6, 25.0, 59.4, 50.7, 12.0, 20.0, 9.8, 5.6, 1.1, -0.6, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 331, 43, NULL, 1165772.423, 1863964.0380000002, 41.78230211, -87.66777712, 67, 'WEST ENGLEWOOD', 15, 7, '(41.78230211, -87.66777712)') ,
(610248, 'Chicago Academy Elementary School', 'ES', '3400 N Austin Ave', 'Chicago', 'IL', 60634, '(773) 534-0146', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610248.pdf', 'AUSL Schools', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, NULL, NULL, 'Average', 40.0, 'Weak', 22.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 51.0, '96.30%', 6.0, '96.10%', '100.00%', NULL, NULL, 66.3, 62.6, 66.7, 68.8, 62.9, 64.9, 57.5, 62.4, 26.7, 46.7, 41.9, 30.3, -0.1, 1.8, 'Yellow', 'Green', 50.0, 53.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 601, 30, NULL, 1135740.091, 1922002.9409999999, 41.94215439, -87.77650575, 17, 'DUNNING', 38, 16, '(41.94215439, -87.77650575)') ,
(610340, 'Chicago Academy High School', 'HS', '3400 N Austin Ave', 'Chicago', 'IL', 60634, '(773) 534-0146', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610340.pdf', 'AUSL Schools', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, 'Weak', 35.0, 'Average', 57.0, 'Average', 53.0, 'Weak', 56.0, 'Average', 57.0, NULL, NULL, NULL, NULL, '93.10%', 6.3, '96.10%', '97.10%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.6, 14.4, 16.3, 16.1, 1.5, 18.4, 2.1, 23.9, 70.1, 64.2, 502, 30, 80.6, 1135740.091, 1922002.9409999999, 41.94215439, -87.77650575, 17, 'DUNNING', 38, 16, '(41.94215439, -87.77650575)') ,
(609753, 'Chicago High School for Agricultural Sciences', 'HS', '3857 W 111th St', 'Chicago', 'IL', 60655, '(773) 535-2500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609753.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, 'Strong', 77.0, 'Average', 49.0, 'Average', 47.0, 'Weak', 57.0, 'Average', 40.0, 'Average', 52.0, 'Strong', 55.0, '92.80%', 5.2, '95.20%', '96.80%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 16.1, 16.5, 16.9, 17.4, 1.3, 20.4, 3.5, 30.7, 70.7, 79.6, 579, 49, 88.1, 1152383.425, 1830663.6469999999, 41.69119388, -87.71773863, 74, 'MOUNT GREENWOOD', 19, 22, '(41.69119388, -87.71773863)') ,
(609754, 'Chicago Military Academy High School', 'HS', '3519 S Giles Ave', 'Chicago', 'IL', 60653, '(773) 534-9750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609754.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 32.0, NULL, NULL, 'Weak', 23.0, 'Weak', 35.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '90.30%', 28.2, '95.60%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.8, 14.4, 15.2, 15.6, 0.8, 17.4, 2.2, 24.4, 83.6, 69.3, 483, 40, 88.1, 1178881.169, 1881647.166, 41.83053768, -87.61917837, 35, 'DOUGLAS', 2, 2, '(41.83053768, -87.61917837)') ,
(609674, 'Chicago Vocational Career Academy High School', 'HS', '2100 E 87th St', 'Chicago', 'IL', 60617, '(773) 535-6100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609674.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 27.0, 'Average', 43.0, 'Weak', 21.0, 'Weak', 30.0, 'Weak', 34.0, 'Weak', 39.0, NULL, NULL, NULL, NULL, '68.80%', 33.3, '94.60%', '99.10%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.5, 12.6, 13.8, 13.2, 0.7, 14.3, 0.5, 7.0, 49.5, 54.0, 833, 47, 44.1, 1191700.5929999999, 1847743.551, 41.73720173, -87.57324389, 45, 'AVALON PARK', 8, 4, '(41.73720173, -87.57324389)') ,
(609899, 'Christian Ebinger Elementary School', 'ES', '7350 W Pratt Ave', 'Chicago', 'IL', 60631, '(773) 534-1070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609899.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 78.0, 'Strong', 72.0, 'Strong', 66.0, 'Average', 50.0, 'Weak', 26.0, 'Strong', 62.0, 'Average', 52.0, 'Weak', 43.0, '95.90%', 9.3, '95.20%', '97.00%', 84.7, 61.5, 81.2, 68.5, 63.3, 82.2, 77.4, 69.4, 56.8, 70.1, 34.5, 53.4, 50.6, 40.2, 0.5, 1.4, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 650, 30, NULL, 1126172.537, 1944639.095, 42.00443557, -87.81116435, 9, 'EDISON PARK', 41, 16, '(42.00443557, -87.81116435)') ,
(609705, 'Christian Fenger Academy High School', 'HS', '11220 S Wallace St', 'Chicago', 'IL', 60628, '(773) 535-5430', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609705.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, 'Weak', 28.0, 'Average', 41.0, 'Weak', 32.0, 'Weak', 46.0, 'Average', 42.0, NULL, NULL, NULL, NULL, '74.40%', 49.8, '95.40%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.0, 12.1, 12.9, 13.3, 1.3, 14.0, 1.1, 11.3, 34.8, 36.7, 688, 48, 68.2, 1174272.711, 1830372.91, 41.68993885, -87.63760833, 49, 'ROSELAND', 34, 22, '(41.68993885, -87.63760833)') ,
(609863, 'Christopher Columbus Elementary School', 'ES', '1003 N Leavitt St', 'Chicago', 'IL', 60622, '(773) 534-4350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609863.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, NULL, NULL, 'Average', 57.0, 'Average', 55.0, 'Weak', NULL, NULL, NULL, 'Strong', 54.0, 'Average', 48.0, '95.50%', 2.4, '96.90%', '92.60%', 77.0, 72.3, 59.2, 48.7, 57.7, 62.5, 76.2, 56.9, 78.8, 63.0, 40.0, 45.0, 42.0, 30.5, 1.1, 1.6, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 309, 35, NULL, 1161595.965, 1906670.676, 41.89958139, -87.68190167, 24, 'WEST TOWN', 32, 13, '(41.89958139, -87.68190167)') ,
(609790, 'Clara Barton Elementary School', 'ES', '7650 S Wolcott Ave', 'Chicago', 'IL', 60620, '(773) 535-3260', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609790.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 25.0, 'Average', 42.0, 'Weak', 28.0, 'Weak', 34.0, 'Weak', 77.0, 'Average', 59.0, 'Average', 51.0, 'Average', 49.0, '91.70%', 25.3, '96.40%', '100.00%', 64.3, NULL, 27.0, 22.8, 51.8, 55.3, 27.4, 22.4, 45.3, 52.9, 6.5, 13.0, 8.1, 5.7, -0.5, -0.3, 'Yellow', 'Yellow', 60.0, 3.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 602, 43, NULL, 1164981.896, 1853724.2969999998, 41.75421962, -87.67096453, 71, 'AUBURN GRESHAM', 18, 6, '(41.75421962, -87.67096453)') ,
(610347, 'Claremont Academy Elementary School', 'ES', '2300 W 64th St', 'Chicago', 'IL', 60636, '(773) 535-8110', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610347.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 36.0, 'Weak', 33.0, 'Average', 50.0, 'Average', 51.0, 'Weak', 62.0, 'Average', 59.0, 'Weak', 46.0, 'Average', 52.0, '92.50%', 64.2, '96.20%', '100.00%', 58.7, 60.5, 40.9, 22.5, 52.3, 66.9, 46.0, 31.7, 72.5, 61.0, 14.9, 14.9, 12.1, 8.1, 0.3, 0.7, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 531, 43, NULL, 1162155.645, 1862200.052, 41.77753754, -87.68108634, 66, 'CHICAGO LAWN', 15, 8, '(41.77753754, -87.68108634)') ,
(610499, 'Collins Academy High School', 'HS', '1313 S Sacramento Dr', 'Chicago', 'IL', 60623, '(773) 534-1840', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610499.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 60.0, 'Average', 42.0, 'Average', 51.0, 'Average', 46.0, 'Weak', 47.0, 'Average', 48.0, 'Strong', 54.0, 'Strong', 56.0, '91.40%', 2.9, '96.60%', '98.70%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.5, 12.5, 14.2, 14.1, 1.6, 15.6, 1.4, 20.5, NULL, NULL, 409, 37, 70.4, 1156217.24, 1893722.861, 41.86416169, -87.70200761, 29, 'NORTH LAWNDALE', 24, 10, '(41.86416169, -87.70200761)') ,
(610170, 'Columbia Explorers Elementary Academy', 'ES', '4520 S Kedzie Ave', 'Chicago', 'IL', 60632, '(773) 535-4050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610170.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, 'Strong', 64.0, 'Average', 59.0, 'Strong', 63.0, 'Weak', 65.0, 'Strong', 72.0, 'Average', 52.0, 'Strong', 55.0, '95.40%', 8.9, '97.10%', '100.00%', 67.6, 30.5, 27.4, 32.5, 44.4, 48.7, 41.6, 49.3, 52.8, 59.6, 17.7, 27.8, 17.1, 13.8, 0.8, 1.5, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1038, 39, NULL, 1155752.301, 1874367.2340000002, 41.81105689, -87.70423481, 58, 'BRIGHTON PARK', 14, 8, '(41.81105689, -87.70423481)') ,
(610004, 'Countee Cullen Elementary School', 'ES', '10650 S Eberhart Ave', 'Chicago', 'IL', 60628, '(773) 535-5375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610004.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 14.0, NULL, NULL, 'Weak', 28.0, 'Weak', 34.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.10%', 36.9, '96.30%', '100.00%', 71.9, 71.4, 24.4, 28.9, 55.3, 68.2, 26.8, 25.9, 56.2, 45.8, 9.5, 4.8, 9.7, 8.1, -1.8, -1.7, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 233, 48, NULL, 1181431.47, 1834229.814, 41.70036107, -87.61128176, 49, 'ROSELAND', 9, 5, '(41.70036107, -87.61128176)') ,
(609781, 'Crispus Attucks Elementary School', 'ES', '5055 S State St', 'Chicago', 'IL', 60609, '(773) 535-1270', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609781.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 45.0, 'Strong', 76.0, 'Strong', 64.0, 'Average', 57.0, 'Weak', 81.0, 'Very Strong', 85.0, NULL, NULL, NULL, NULL, '95.20%', 12.5, '93.50%', '100.00%', 64.7, 66.7, 25.3, 9.9, 42.4, 53.5, 16.0, 16.5, 53.3, 32.4, 3.1, 9.4, 7.6, 10.0, -0.3, -2.0, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 274, 42, NULL, 1177176.077, 1871285.644, 41.80214349, -87.62574740000001, 38, 'GRAND BOULEVARD', 3, 2, '(41.80214349, -87.6257474)') ,
(609873, 'Crown Community Academy of Fine Arts Center Elementary School', 'ES', '2128 S Saint Louis Ave', 'Chicago', 'IL', 60623, '(773) 534-1680', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609873.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 27.0, 'Weak', 39.0, 'Average', 51.0, 'Average', 52.0, 'Weak', 45.0, 'Weak', 31.0, 'Average', 49.0, 'Average', 51.0, '92.50%', 100.5, '96.70%', '100.00%', 51.6, 38.0, 12.5, 16.1, 35.2, 38.7, 36.7, 28.1, 64.3, 56.6, 3.4, 6.9, 3.9, 8.8, -0.3, 1.0, 'Yellow', 'Yellow', 13.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 298, 37, NULL, 1153387.649, 1889511.3869999999, 41.85266155, -87.7125068, 29, 'NORTH LAWNDALE', 24, 10, '(41.85266155, -87.7125068)') ,
(610063, 'Cyrus H McCormick Elementary School', 'ES', '2712 S Sawyer Ave', 'Chicago', 'IL', 60623, '(773) 535-7252', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610063.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'Yes', NULL, NULL, 'Strong', 60.0, NULL, NULL, NULL, NULL, 'Weak', 61.0, 'Average', 54.0, 'Average', 52.0, 'Strong', 56.0, '95.90%', 4.7, '95.90%', '100.00%', 55.3, 39.4, 42.1, 22.6, 43.9, 54.2, NULL, NULL, NULL, NULL, NULL, NULL, 25.2, 10.3, -0.2, 0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 894, 37, NULL, 1155096.537, 1885727.878, 41.84224508, -87.706336, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84224508, -87.706336)') ,
(609804, 'Daniel Boone Elementary School', 'ES', '6710 N Washtenaw Ave', 'Chicago', 'IL', 60645, '(773) 534-2160', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609804.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 52.0, 'Average', 50.0, 'Average', 47.0, 'Average', 48.0, 'Weak', 45.0, 'Average', 53.0, 'Average', 50.0, 'Average', 49.0, '95.50%', 10.5, '95.40%', '97.60%', 65.0, 43.0, 39.7, 33.2, 55.4, 61.8, 43.1, 35.2, 56.3, 42.3, 22.5, 35.0, 22.0, 17.6, 1.1, 1.9, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 752, 32, NULL, 1157068.6570000001, 1944483.4719999998, 42.00343519, -87.69750011, 2, 'WEST RIDGE', 50, 24, '(42.00343519, -87.69750011)') ,
(610083, 'Daniel C Beard Elementary School', 'ES', '6445 W Strong St', 'Chicago', 'IL', 60656, '(773) 534-1228', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610083.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', NULL, 'Track_E', 'Not Applicable', 'Not Enough Data', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Strong', 57.0, 'Strong', 58.0, '91.90%', 0.0, '94.30%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 160, 30, NULL, 1132103.024, 1932162.514, 41.97009738, -87.78963691, 10, 'NORWOOD PARK', 41, 16, '(41.97009738, -87.78963691)') ,
(610380, 'Daniel Hale Williams Prep School of Medicine', 'HS', '4934 S Wabash Ave', 'Chicago', 'IL', 60615, '(773) 535-1120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610380.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 50.0, NULL, NULL, 'Average', 45.0, 'Average', 54.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '91.30%', 5.4, '96.70%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 54.5, 63.8, 55.8, 58.5, 21.4, 33.3, 23.7, 8.5, -0.9, -1.8, 'Yellow', 'Red', 64.1, 25.0, 15.0, 15.1, 15.8, 15.4, 0.4, 17.5, 1.7, 32.7, NULL, NULL, 324, 42, 90.1, 1177513.255, 1872170.4440000001, 41.80456384, -87.62448408, 38, 'GRAND BOULEVARD', 3, 2, '(41.80456384, -87.62448408)') ,
(609870, 'Daniel J Corkery Elementary School', 'ES', '2510 S Kildare Ave', 'Chicago', 'IL', 60623, '(773) 534-1650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609870.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 39.0, NULL, NULL, 'Weak', 31.0, 'Weak', 29.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.70%', 18.1, '97.20%', '100.00%', 64.1, 35.3, 35.7, 22.6, 48.4, 61.1, 45.8, 28.8, 47.5, 53.0, 21.6, 19.6, 13.6, 8.2, 0.1, 0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 631, 37, NULL, 1148304.879, 1886901.414, 41.84559878, -87.73122946, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84559878, -87.73122946)') ,
(609835, 'Daniel R Cameron Elementary School', 'ES', '1234 N Monticello Ave', 'Chicago', 'IL', 60651, '(773) 534-4290', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609835.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, NULL, NULL, 'Average', 49.0, 'Average', 50.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Weak', 46.0, '92.60%', 153.9, '96.20%', '98.40%', 47.8, 21.1, 16.4, 16.1, 31.3, 34.7, 19.2, 15.6, 33.5, 35.0, 9.2, 7.1, 10.3, 5.9, -1.0, -1.4, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 828, 34, NULL, 1151767.5459999999, 1908130.0680000002, 41.90378521, -87.71796315, 23, 'HUMBOLDT PARK', 26, 25, '(41.90378521, -87.71796315)') ,
(610223, 'Daniel S Wentworth Elementary School', 'ES', '6950 S Sangamon St', 'Chicago', 'IL', 60621, '(773) 535-3394', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610223.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 30.0, 'Weak', 32.0, 'Weak', 34.0, 'Weak', 36.0, 'Weak', 31.0, 'Weak', 32.0, NULL, NULL, NULL, NULL, '91.60%', 73.8, '95.90%', '96.20%', NULL, 24.7, 17.9, 14.2, 48.5, 51.0, 40.2, 18.8, 74.4, 38.8, 10.5, 10.5, 5.5, 1.5, 1.0, -0.6, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 365, 45, NULL, 1171140.355, 1858532.5580000002, 41.76728177, -87.64825544, 68, 'ENGLEWOOD', 17, 7, '(41.76728177, -87.64825544)') ,
(610221, 'Daniel Webster Elementary School', 'ES', '4055 W Arthington St', 'Chicago', 'IL', 60624, '(773) 534-6925', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610221.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 48.0, 'Average', 56.0, 'Very Strong', 81.0, 'Strong', 66.0, 'Weak', 46.0, 'Average', 44.0, 'Weak', 45.0, 'Strong', 55.0, '94.80%', 5.7, '97.20%', '100.00%', 81.6, 45.9, 37.0, 37.0, 56.6, 63.3, 44.2, 26.9, 57.8, 58.8, 2.6, 37.5, 25.9, 21.0, 2.0, 0.7, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 370, 36, NULL, 1149263.6090000002, 1895759.896, 41.86988904, -87.72748159, 26, 'WEST GARFIELD PARK', 24, 11, '(41.86988904, -87.72748159)') ,
(609704, 'David G Farragut Career Academy High School', 'HS', '2345 S Christiana Ave', 'Chicago', 'IL', 60623, '(773) 534-1300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609704.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 43.0, NULL, NULL, 'Average', 42.0, 'Weak', 32.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '86.10%', 18.6, '95.20%', '99.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.6, 12.9, 14.4, 13.9, 1.3, 16.0, 1.6, 13.4, 43.2, 31.3, 1456, 37, 68.6, 1154443.895, 1888031.41, 41.8485793, -87.70866954, 30, 'SOUTH LAWNDALE', 22, 10, '(41.8485793, -87.70866954)') ,
(610402, 'DeVry University Advantage Academy High School', 'HS', '3300 N Campbell', 'Chicago', 'IL', 60618, '(773) 697-2216', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610402.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Strong', 58.0, 'Strong', 55.0, '96.70%', 0.0, '94.50%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 20.7, NULL, 40.6, NULL, 63.3, 200, 35, NULL, 1159059.4109999998, 1921900.745, 41.94142629, -87.6907993, 5, 'NORTH CENTER', 47, 19, '(41.94142629, -87.6907993)') ,
(609885, 'Dewey Elementary Academy of Fine Arts', 'ES', '5415 S Union Ave', 'Chicago', 'IL', 60609, '(773) 535-1666', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609885.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 41.0, NULL, NULL, 'Average', 42.0, 'Weak', 35.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Strong', 58.0, '90.70%', 26.3, '95.50%', '100.00%', 64.7, 56.1, 12.2, 11.8, 30.9, 44.2, 10.4, 4.3, 40.3, 23.1, 3.6, 0.0, 3.3, 0.5, 0.4, -1.2, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 357, 45, NULL, 1172596.05, 1868881.791, 41.795649299999994, -87.642615, 61, 'NEW CITY', 3, 9, '(41.7956493, -87.642615)') ,
(609859, 'DeWitt Clinton Elementary School', 'ES', '6110 N Fairfield Ave', 'Chicago', 'IL', 60659, '(773) 534-2025', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609859.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 47.0, 'Average', 41.0, 'Average', 47.0, 'Average', 52.0, 'Weak', 41.0, 'Weak', 36.0, NULL, NULL, NULL, NULL, '95.30%', 14.8, '95.40%', '100.00%', 68.8, 44.9, 45.8, 40.1, 55.6, 63.7, 64.7, 39.7, 70.6, 54.2, 24.3, 25.2, 33.5, 21.8, 0.6, -0.5, 'Green', 'Yellow', 23.6, 46.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1139, 32, NULL, 1156854.125, 1940520.2119999998, 41.99256421, -87.69839748, 2, 'WEST RIDGE', 50, 24, '(41.99256421, -87.69839748)') ,
(610515, 'Disney II Magnet School', 'ES', '3815 N Kedvale Ave', 'Chicago', 'IL', 60641, '(773) 534-3750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610515.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', NULL, NULL, 'Very Strong', 99.0, NULL, NULL, NULL, NULL, 'Weak', 75.0, 'Average', 53.0, 'Strong', 62.0, 'Strong', 58.0, '97.30%', 0.0, '96.50%', '100.00%', 84.6, NULL, 76.5, 79.4, 63.4, 63.7, NULL, NULL, NULL, NULL, NULL, NULL, 57.8, 52.0, 0.6, 2.1, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 363, 29, NULL, 1148206.405, 1925121.479, 41.95048055, -87.73060545, 16, 'IRVING PARK', 38, 17, '(41.95048055, -87.73060545)') ,
(610077, 'Donald Morrill Math & Science Elementary School', 'ES', '6011 S Rockwell St', 'Chicago', 'IL', 60629, '(773) 535-9288', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610077.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Weak', 25.0, 'Weak', 32.0, 'Weak', 39.0, 'Weak', 35.0, 'Weak', 32.0, 'Weak', 44.0, 'Average', 49.0, '93.40%', 19.6, '94.80%', '97.30%', 64.6, 46.3, 20.0, 15.3, 46.6, 42.6, 30.9, 22.9, 54.5, 50.3, 12.1, 26.2, 10.0, 5.7, -0.8, -1.2, 'Red', 'Red', 38.9, 50.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 823, 43, NULL, 1160111.7959999999, 1864645.2980000002, 41.78428993, -87.68851192, 66, 'CHICAGO LAWN', 15, 8, '(41.78428993, -87.68851192)') ,
(610198, 'Douglas Taylor Elementary School', 'ES', '9912 S Avenue H', 'Chicago', 'IL', 60617, '(773) 535-6240', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610198.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 34.0, NULL, NULL, 'Weak', 20.0, 'Very Weak', 1.0, 'Weak', NULL, NULL, NULL, 'Weak', 42.0, 'Weak', 45.0, '95.10%', 16.6, '94.30%', '100.00%', 48.9, 34.4, 28.5, 22.3, 44.4, 44.9, 39.7, 33.5, 56.6, 48.9, 15.7, 18.6, 12.4, 9.9, -0.5, -0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 632, 47, NULL, 1202756.0929999999, 1839915.4880000001, 41.71544589, -87.5330074, 52, 'EAST SIDE', 10, 4, '(41.71544589, -87.5330074)') ,
(610533, 'Dr Jorge Prieto Math and Science', 'ES', '2231 N Central Ave', 'Chicago', 'IL', 60639, '(773) 534-0210', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610533.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Strong', 58.0, '94.70%', 7.0, '96.90%', '100.00%', 48.4, 19.5, 33.8, 19.9, 43.1, 55.0, 44.2, 30.0, 56.9, 47.8, 12.1, 28.9, 14.6, 8.6, -0.7, -0.7, 'Red', 'Red', 17.5, 52.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1001, 29, NULL, 1138718.257, 1914429.033, 41.92131717, -87.7657438, 19, 'BELMONT CRAGIN', 37, 25, '(41.92131717, -87.7657438)') ,
(609751, 'Dr  Martin Luther King  Jr  College Prep High School', 'HS', '4445 S Drexel Blvd', 'Chicago', 'IL', 60653, '(773) 535-1180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609751.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Strong', 56.0, 'Average', 52.0, '92.90%', 4.4, '96.30%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 17.4, 17.7, 17.7, 18.1, 0.7, 20.5, 2.8, 31.9, 75.4, 85.1, 915, 40, 91.1, 1183114.175, 1875544.7869999998, 41.81369471, -87.60383761, 39, 'KENWOOD', 4, 2, '(41.81369471, -87.60383761)') ,
(610266, 'Dumas Technology Academy', 'ES', '6650 S Ellis Ave', 'Chicago', 'IL', 60637, '(773) 535-0750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610266.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 19.0, NULL, NULL, 'Weak', 25.0, 'Average', 49.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 53.0, '92.50%', 38.4, '95.60%', '98.10%', 63.6, 12.0, 22.0, 18.8, 61.6, 63.7, 20.0, 22.4, 61.0, 53.7, 15.9, 40.9, 8.3, 5.8, -0.2, -0.7, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 381, 46, NULL, 1183997.399, 1860905.9440000001, 41.77350389, -87.60105531, 42, 'WOODLAWN', 5, 3, '(41.77350389, -87.60105531)') ,
(610188, 'Dunne Technology Academy', 'ES', '10845 S Union Ave', 'Chicago', 'IL', 60628, '(773) 535-5517', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610188.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 46.0, 'Average', 53.0, 'Strong', 65.0, 'Strong', 71.0, 'Weak', 51.0, 'Average', 59.0, 'Average', 49.0, 'Average', 47.0, '94.60%', 22.8, '96.40%', '100.00%', 53.6, 38.8, 36.3, 31.4, 55.8, 71.1, 39.4, 37.7, 57.0, 49.5, 9.6, 25.0, 24.7, 17.2, 2.8, 1.8, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 310, 49, NULL, 1173617.217, 1832764.2859999998, 41.69651565, -87.63993763, 49, 'ROSELAND', 34, 22, '(41.69651565, -87.63993763)') ,
(610352, 'Durkin Park Elementary School', 'ES', '8445 S Kolin Ave', 'Chicago', 'IL', 60652, '(773) 535-2322', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610352.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 61.0, 'Strong', 74.0, 'Average', 57.0, 'Average', 58.0, 'Weak', 88.0, 'Strong', 74.0, 'Average', 53.0, 'Weak', 38.0, '95.40%', 0.7, '95.80%', '100.00%', 75.6, 58.1, 36.6, 32.3, 55.8, 65.6, 53.4, 40.4, 61.8, 61.5, 22.4, 40.8, 22.0, 16.3, 0.8, 1.5, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 543, 44, NULL, 1148934.857, 1848010.352, 41.73886352, -87.72991927, 70, 'ASHBURN', 18, 8, '(41.73886352, -87.72991927)') ,
(610254, 'Dvorak Technology Academy', 'ES', '3615 W 16th St', 'Chicago', 'IL', 60623, '(773) 534-1690', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610254.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 46.0, 'Weak', 22.0, 'Strong', 66.0, 'Strong', 69.0, 'Weak', 4.0, 'Very Weak', 13.0, NULL, NULL, NULL, NULL, '94.20%', 4.4, '95.40%', '100.00%', 49.0, 42.3, 20.5, 22.4, 45.8, 43.1, 25.1, 26.3, 38.9, 53.6, 3.6, 30.4, 11.0, 10.4, -1.5, 0.4, 'Red', 'Yellow', 20.3, 16.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 556, 37, NULL, 1152488.851, 1891762.257, 41.85885599, -87.71574626, 29, 'NORTH LAWNDALE', 24, 10, '(41.85885599, -87.71574626)') ,
(609736, 'Dyett High School', 'HS', '555 E 51st St', 'Chicago', 'IL', 60615, '(773) 535-1825', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609736.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 27.0, 'Very Weak', 18.0, 'Weak', 35.0, 'Average', 47.0, 'Weak', 35.0, 'Weak', 25.0, NULL, NULL, NULL, NULL, '62.50%', 24.4, '93.50%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 11.6, 11.7, 13.0, 12.9, 1.3, 14.0, 1.0, 3.3, 33.7, 51.0, 318, 42, 37.1, 1180944.201, 1871282.832, 41.80204982, -87.61192836, 40, 'WASHINGTON PARK', 4, 2, '(41.80204982, -87.61192836)') ,
(610132, 'Edgar Allan Poe Elementary Classical School', 'ES', '10538 S Langley Ave', 'Chicago', 'IL', 60628, '(773) 535-5525', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610132.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Weak', 33.0, 'Strong', 66.0, 'Very Strong', 88.0, 'Weak', 27.0, 'Average', 41.0, 'Strong', 55.0, 'Average', 47.0, '97.60%', 0.0, '97.20%', '100.00%', 100.0, 78.3, 94.0, 91.6, 51.8, 65.5, 78.6, 92.9, 53.6, 60.7, NULL, NULL, 78.8, 79.6, 1.7, -0.2, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 192, 48, NULL, 1182742.505, 1835063.9, 41.70261965, -87.60645552, 50, 'PULLMAN', 9, 5, '(41.70261965, -87.60645552)') ,
(609901, 'Edgebrook Elementary School', 'ES', '6525 N Hiawatha Ave', 'Chicago', 'IL', 60646, '(773) 534-1194', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609901.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Average', 51.0, 'Average', 53.0, 'Weak', 66.0, 'Strong', 70.0, 'Strong', 56.0, 'Weak', 39.0, '96.90%', 3.4, '96.60%', '100.00%', 96.3, 47.6, 88.1, 74.1, 55.2, 75.2, 80.0, 79.1, 54.5, 60.0, 50.0, 71.4, 60.2, 56.8, 0.0, 0.5, 'Yellow', 'Yellow', 83.3, 64.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 468, 30, NULL, 1139595.896, 1942911.685, 41.99946016, -87.76182109999999, 12, 'FOREST GLEN', 41, 16, '(41.99946016, -87.7618211)') ,
(610523, 'Edison Park Elementary School', 'ES', '6220 N Olcott Ave', 'Chicago', 'IL', 60631, '(773) 534-0960', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610523.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 95.0, NULL, NULL, 'Average', 52.0, 'Average', 49.0, 'Weak', NULL, NULL, NULL, 'Strong', 56.0, 'Weak', 46.0, '96.10%', 6.9, '96.60%', '98.60%', 78.1, NULL, 64.2, 62.6, 54.7, 49.1, 59.4, 61.4, 38.8, 48.5, 40.0, 55.0, 27.7, 32.6, -0.6, -0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 431, 30, NULL, 1125476.537, 1940915.071, 41.99422812, -87.81380814, 10, 'NORWOOD PARK', 41, 16, '(41.99422812, -87.81380814)') ,
(609819, 'Edmond Burke Elementary School', 'ES', '5356 S King Dr', 'Chicago', 'IL', 60615, '(773) 535-1325', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609819.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 1.0, NULL, NULL, 'Very Weak', 13.0, 'Weak', 22.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 48.0, '89.30%', 82.0, '95.50%', '96.70%', 44.4, 57.4, 30.3, 16.2, 40.7, 40.4, 37.6, 24.7, 63.3, 45.5, 3.6, 14.3, 7.9, 4.3, -1.4, -1.9, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 244, 42, NULL, 1179801.469, 1869357.523, 41.79679285, -87.61617809, 40, 'WASHINGTON PARK', 3, 2, '(41.79679285, -87.61617809)') ,
(609815, 'Edward A Bouchet Math & Science Academy Elementary School', 'ES', '7355 S Jeffery Blvd', 'Chicago', 'IL', 60649, '(773) 535-0501', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609815.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 33.0, 'Average', 57.0, 'Average', 55.0, 'Average', 50.0, 'Weak', 53.0, 'Average', 58.0, 'Average', 50.0, 'Average', 48.0, '92.90%', 28.0, '95.90%', '98.30%', 57.6, 42.4, 13.7, 21.7, 51.6, 44.8, 15.7, 34.7, 48.2, 66.5, 0.0, 23.7, 5.6, 7.6, -0.2, 0.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 783, 46, NULL, 1190861.978, 1856358.0380000002, 41.76086093, -87.57603849, 43, 'SOUTH SHORE', 5, 3, '(41.76086093, -87.57603849)') ,
(610246, 'Edward Beasley Elementary Magnet Academic Center', 'ES', '5255 S State St', 'Chicago', 'IL', 60609, '(773) 535-1230', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610246.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 47.0, NULL, NULL, 'Weak', 34.0, 'Average', 48.0, 'Weak', NULL, NULL, NULL, 'Weak', 42.0, 'Weak', 45.0, '95.00%', 10.3, '94.70%', '100.00%', 74.0, 54.1, 39.8, 38.7, 43.6, 47.5, 53.8, 55.4, 57.3, 54.2, 27.0, 43.0, 26.4, 21.3, 0.3, 0.3, 'Yellow', 'Yellow', 38.6, 76.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1365, 42, NULL, 1177212.26, 1869948.081, 41.79847228, -87.62565509, 40, 'WASHINGTON PARK', 3, 2, '(41.79847228, -87.62565509)') ,
(609881, 'Edward C Delano Elementary School', 'ES', '3937 W Wilcox St', 'Chicago', 'IL', 60624, '(773) 534-6620', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609881.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 38.0, 'Strong', 64.0, 'Strong', 66.0, 'Average', 52.0, 'Weak', 63.0, 'Strong', 79.0, 'Strong', 55.0, 'Strong', 58.0, '93.40%', 20.3, '94.70%', '94.70%', 88.4, 76.7, 26.7, 18.0, 62.6, 62.0, 41.5, 32.0, 67.1, 69.3, 3.4, 6.9, 6.8, 10.9, 1.0, 1.7, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 414, 34, NULL, 1150012.228, 1899002.166, 41.87877166, -87.72464875, 26, 'WEST GARFIELD PARK', 28, 11, '(41.87877166, -87.72464875)') ,
(609862, 'Edward Coles Elementary Language Academy', 'ES', '8441 S Yates Blvd', 'Chicago', 'IL', 60617, '(773) 535-6550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609862.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 22.0, 'Average', 43.0, 'Average', 40.0, 'Average', 54.0, 'Weak', 52.0, 'Average', 53.0, 'Average', 50.0, 'Average', 51.0, '95.10%', 24.9, '95.00%', '98.70%', 76.2, 60.0, 44.6, 26.4, 54.7, 72.5, 45.5, 22.4, 72.1, 52.6, 0.0, 11.4, 16.7, 5.7, 1.0, -0.1, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 554, 47, NULL, 1193682.146, 1849300.8630000001, 41.74142687, -87.56593332, 46, 'SOUTH CHICAGO', 7, 4, '(41.74142687, -87.56593332)') ,
(609909, 'Edward Everett Elementary School', 'ES', '3419 S Bell Ave', 'Chicago', 'IL', 60608, '(773) 535-4550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609909.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', NULL, NULL, 'Strong', 66.0, NULL, NULL, NULL, NULL, 'Weak', 47.0, 'Average', 45.0, NULL, NULL, NULL, NULL, '95.30%', 2.4, '97.70%', '93.80%', 64.4, 47.0, 37.2, 35.0, 56.9, 52.2, NULL, NULL, NULL, NULL, NULL, NULL, 13.6, 13.6, 0.6, 1.3, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 321, 39, NULL, 1161955.375, 1881878.553, 41.83154196, -87.68127322, 59, 'MCKINLEY PARK', 11, 9, '(41.83154196, -87.68127322)') ,
(610012, 'Edward Jenner Elementary Academy of the Arts', 'ES', '1119 N Cleveland Ave', 'Chicago', 'IL', 60610, '(773) 534-8440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610012.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 50.0, 'Weak', 34.0, 'Strong', 62.0, 'Average', 41.0, 'Weak', 20.0, 'Weak', 26.0, 'Average', 49.0, 'Average', 53.0, '88.90%', 16.0, '94.80%', '98.70%', 34.8, 21.3, 21.9, 10.7, 42.3, 57.7, 37.1, 14.5, 69.4, 46.7, 4.3, 25.5, 8.2, 1.3, 2.0, 0.2, 'Green', 'Yellow', 45.8, 31.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 314, 33, NULL, 1172718.668, 1907660.3390000002, 41.90205805, -87.64101894, 8, 'NEAR NORTH SIDE', 27, 18, '(41.90205805, -87.64101894)') ,
(609904, 'Edward K Ellington Elementary School', 'ES', '243 N Parkside Ave', 'Chicago', 'IL', 60644, '(773) 534-6361', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609904.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 30.0, NULL, NULL, 'Weak', 37.0, 'Average', 55.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Average', 50.0, '93.30%', 17.4, '97.30%', '98.90%', 56.6, 46.9, 29.5, 33.3, 60.6, 64.5, 43.6, 29.9, 69.3, 60.8, 11.1, 14.8, 16.1, 8.0, 1.9, 0.4, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 377, 36, NULL, 1138584.642, 1901378.0019999999, 41.885506, -87.76655157, 25, 'AUSTIN', 29, 15, '(41.885506, -87.76655157)') ,
(610006, 'Edward N Hurley Elementary School', 'ES', '3849 W 69th Pl', 'Chicago', 'IL', 60629, '(773) 535-2068', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610006.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 77.0, 'Average', 56.0, 'Average', 50.0, 'Average', 49.0, 'Weak', 43.0, 'Average', 54.0, 'Average', 49.0, 'Weak', 46.0, '95.40%', 12.2, '94.90%', '96.60%', 74.8, 51.7, 47.6, 32.1, 56.6, 58.4, 47.0, 39.6, 54.0, 60.7, 19.0, 26.7, 16.4, 11.7, 0.0, 0.8, 'Yellow', 'Green', NULL, 30.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 830, 44, NULL, 1151703.8020000001, 1858195.0380000002, 41.7667583, -87.71950804, 65, 'WEST LAWN', 13, 8, '(41.7667583, -87.71950804)') ,
(609735, 'Edward Tilden Career Community Academy High School', 'HS', '4747 S Union Ave', 'Chicago', 'IL', 60609, '(773) 535-1625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609735.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 34.0, 'Average', 57.0, 'Average', 45.0, 'Weak', 37.0, 'Weak', 54.0, 'Weak', 35.0, 'Average', 51.0, 'Average', 47.0, '78.40%', 13.0, '95.20%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 11.6, 12.0, 12.9, 12.9, 1.3, 13.5, 0.6, 10.5, 41.1, 37.0, 461, 42, 86.5, 1172475.4740000002, 1873203.826, 41.80751207, -87.64292985, 61, 'NEW CITY', 11, 9, '(41.80751207, -87.64292985)') ,
(610315, 'Edward White Elementary Career Academy', 'ES', '1136 W 122nd St', 'Chicago', 'IL', 60643, '(773) 535-5672', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610315.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 27.0, NULL, NULL, 'Average', 44.0, 'Average', 48.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Weak', 46.0, '93.80%', 31.0, '96.70%', '100.00%', 64.9, 35.1, 16.3, 32.6, 67.6, 47.4, 31.6, 34.2, 57.6, 81.8, 14.3, 28.6, 7.4, 7.4, 1.9, 1.2, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 138, 49, NULL, 1170787.9, 1823930.602, 41.67233661, -87.65055321, 53, 'WEST PULLMAN', 34, 5, '(41.67233661, -87.65055321)') ,
(609708, 'Edwin G Foreman High School', 'HS', '3235 N LeClaire Ave', 'Chicago', 'IL', 60641, '(773) 534-3400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609708.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 26.0, NULL, NULL, 'Weak', 28.0, 'Weak', 23.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '84.30%', 10.3, '94.80%', '99.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.0, 12.7, 14.1, 13.7, 0.7, 15.3, 1.2, 12.5, 59.7, 40.9, 1716, 29, 75.6, 1141833.3590000002, 1921189.138, 41.93981043, -87.7541302, 15, 'PORTAGE PARK', 30, 16, '(41.93981043, -87.7541302)') ,
(609913, 'Elaine O Goodlow Elementary Magnet School', 'ES', '2040 W 62nd St', 'Chicago', 'IL', 60636, '(773) 535-9365', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609913.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 44.0, 'Average', 42.0, 'Average', 52.0, 'Average', 56.0, 'Weak', 35.0, 'Weak', 28.0, 'Weak', 46.0, 'Weak', 46.0, '91.60%', 47.5, '96.00%', '100.00%', 77.8, 67.9, 40.5, 26.0, 50.4, 67.7, 35.4, 29.2, 61.0, 50.0, 4.0, 8.0, 9.8, 8.6, -0.2, -0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 419, 43, NULL, 1163653.744, 1863565.274, 41.78125258, -87.67555599, 67, 'WEST ENGLEWOOD', 15, 7, '(41.78125258, -87.67555599)') ,
(610233, 'Elihu Yale Elementary School', 'ES', '7025 S Princeton Ave', 'Chicago', 'IL', 60621, '(773) 535-3190', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610233.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 29.0, 'Weak', 35.0, 'Average', 43.0, 'Average', 50.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Average', 47.0, '89.70%', 67.2, '97.00%', '100.00%', 16.4, 6.9, 23.3, 19.1, 39.1, 38.2, 31.9, 15.7, 62.5, 50.0, 0.0, 12.9, 4.5, 3.2, -0.1, -1.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 182, 45, NULL, 1175551.272, 1858262.1940000001, 41.76644232, -87.63209571, 69, 'GREATER GRAND CROSSING', 6, 7, '(41.76644232, -87.63209571)') ,
(610227, 'Eli Whitney Elementary School', 'ES', '2815 S Komensky Ave', 'Chicago', 'IL', 60623, '(773) 534-1560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610227.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', 'Average', 44.0, NULL, NULL, 'Average', 49.0, 'Average', 53.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.80%', 11.6, '95.20%', '97.60%', 68.7, 26.5, 30.3, 26.1, 54.5, 60.8, 41.5, 29.2, 55.3, 54.3, 6.8, 12.6, 15.0, 9.4, -0.1, 0.7, 'Yellow', 'Yellow', 9.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1133, 37, NULL, 1149877.98, 1884894.804, 41.84006197, -87.72550840000001, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84006197, -87.7255084)') ,
(610195, 'Elizabeth H Sutherland Elementary School', 'ES', '10015 S Leavitt St', 'Chicago', 'IL', 60643, '(773) 535-2580', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610195.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Very Strong', 86.0, NULL, NULL, 'Strong', 64.0, 'Average', 59.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Weak', 40.0, '95.60%', 6.8, '96.20%', '96.00%', 87.7, NULL, 56.6, 72.8, 66.5, 67.6, 66.9, 84.7, 70.2, 68.1, 29.3, 60.9, 37.2, 35.7, 0.0, -0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 762, 49, NULL, 1163512.956, 1838118.8909999998, 41.71142687, -87.67678398, 72, 'BEVERLY', 19, 22, '(41.71142687, -87.67678398)') ,
(610119, 'Elizabeth Peabody Elementary School', 'ES', '1444 W Augusta Blvd', 'Chicago', 'IL', 60622, '(773) 534-4170', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610119.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 51.0, 'Average', 58.0, 'Average', 50.0, 'Strong', 61.0, 'Weak', 57.0, 'Average', 55.0, 'Average', 52.0, 'Average', 49.0, '95.20%', 7.6, '97.00%', '100.00%', 66.2, NULL, 18.4, 10.5, 32.4, 43.5, 45.0, 30.4, 56.4, 46.7, 14.3, 28.6, 9.1, 5.9, -1.1, -0.8, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 289, 35, NULL, 1166416.965, 1906812.638, 41.89986912, -87.66419014, 24, 'WEST TOWN', 27, 13, '(41.89986912, -87.66419014)') ,
(609852, 'Eliza Chappell Elementary School', 'ES', '2135 W Foster Ave', 'Chicago', 'IL', 60625, '(773) 534-2390', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609852.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 70.0, 'Strong', 65.0, 'Average', 53.0, 'Average', 51.0, 'Weak', 56.0, 'Average', 48.0, 'Average', 50.0, 'Average', 52.0, '95.10%', 2.9, '96.70%', '98.90%', 72.6, 57.0, 56.0, 54.4, 55.2, 69.0, 53.2, 63.9, 60.0, 59.8, 12.8, 41.0, 19.6, 17.6, 0.4, 1.0, 'Yellow', 'Green', 27.5, 63.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 451, 31, NULL, 1161016.902, 1934466.545, 41.975867, -87.68325438, 4, 'LINCOLN SQUARE', 47, 20, '(41.975867, -87.68325438)') ,
(610235, 'Ella Flagg Young Elementary School', 'ES', '1434 N Parkside Ave', 'Chicago', 'IL', 60651, '(773) 534-6200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610235.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 31.0, NULL, NULL, 'Average', 44.0, 'Average', 50.0, 'Weak', NULL, NULL, NULL, 'Weak', 44.0, 'Average', 47.0, '94.30%', 3.0, '95.40%', '100.00%', 55.4, 29.6, 21.3, 24.9, 44.4, 44.3, 32.9, 28.2, 61.5, 54.9, 11.5, 16.1, 9.1, 8.0, 0.1, 0.8, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1111, 36, NULL, 1138362.5359999998, 1909130.405, 41.90678358, -87.76717934, 25, 'AUSTIN', 29, 25, '(41.90678358, -87.76717934)') ,
(609682, 'Ellen H Richards Career Academy High School', 'HS', '5009 S Laflin St', 'Chicago', 'IL', 60609, '(773) 535-4945', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609682.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 30.0, 'Weak', 23.0, 'Weak', 32.0, 'Very Weak', 19.0, 'Weak', 42.0, 'Weak', 34.0, 'Weak', 43.0, 'Average', 47.0, '90.60%', 17.3, '95.70%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 11.8, 12.0, 13.1, 12.8, 1.0, 14.0, 0.9, 5.5, 57.3, 43.1, 473, 42, 63.5, 1167224.006, 1871465.305, 41.80285552, -87.66224052, 61, 'NEW CITY', 16, 9, '(41.80285552, -87.66224052)') ,
(610073, 'Ellen Mitchell Elementary School', 'ES', '2233 W Ohio St', 'Chicago', 'IL', 60612, '(773) 534-7655', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610073.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Strong', 64.0, 'Very Strong', 95.0, 'Very Strong', 95.0, 'Weak', 81.0, 'Very Strong', 90.0, 'Strong', 56.0, 'Strong', 54.0, '95.50%', 12.2, '97.70%', '100.00%', 46.7, 41.9, 53.1, 43.9, 72.4, 75.5, 35.5, 51.3, 55.3, 55.3, 21.1, 26.3, 30.1, 27.3, -0.5, -0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 344, 35, NULL, 1161269.097, 1903925.361, 41.89205482, -87.68317867, 24, 'WEST TOWN', 26, 13, '(41.89205482, -87.68317867)') ,
(609712, 'Emil G Hirsch Metropolitan High School', 'HS', '7740 S Ingleside Ave', 'Chicago', 'IL', 60619, '(773) 535-3100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609712.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 13.0, 'Average', 46.0, 'Weak', 28.0, 'Weak', 28.0, 'Weak', 52.0, 'Average', 50.0, NULL, NULL, NULL, NULL, '84.80%', 47.1, '95.40%', '88.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 11.5, 11.2, 12.9, 12.4, 0.9, 14.1, 1.2, 10.8, 36.2, 45.1, 458, 46, 69.5, 1183865.127, 1853710.216, 41.7537612, -87.60176452, 69, 'GREATER GRAND CROSSING', 8, 6, '(41.7537612, -87.60176452)') ,
(609973, 'Emiliano Zapata Elementary Academy', 'ES', '2728 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 534-1390', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609973.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 60.0, 'Average', 52.0, 'Average', 43.0, 'Average', 42.0, 'Weak', 73.0, 'Strong', 79.0, 'Weak', 46.0, 'Average', 49.0, '96.60%', 2.8, '96.80%', '100.00%', NULL, NULL, 51.6, 40.3, 68.9, 76.2, 44.5, 45.5, 60.5, 64.3, 15.5, 35.7, 21.6, 13.7, -0.3, 1.2, 'Yellow', 'Green', 20.2, 52.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 988, 37, NULL, 1147465.325, 1885362.468, 41.84139181, -87.73434997, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84139181, -87.73434997)') ,
(610065, 'Emmett Louis Till Math and Science Academy', 'ES', '6543 S Champlain Ave', 'Chicago', 'IL', 60637, '(773) 535-0570', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610065.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Weak', 31.0, 'Average', 47.0, 'Average', 40.0, 'Weak', 49.0, 'Average', 42.0, 'Average', 47.0, 'Average', 48.0, '88.00%', 95.7, '93.30%', '90.50%', 34.3, NULL, 15.8, 7.9, 39.4, 43.1, 15.1, 15.5, 43.6, 52.5, 4.5, 23.3, 3.2, 3.5, -0.4, -1.6, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 471, 46, NULL, 1181736.7820000001, 1861592.586, 41.77544062, -87.60932095, 42, 'WOODLAWN', 20, 3, '(41.77544062, -87.60932095)') ,
(609916, 'Enrico Fermi Elementary School', 'ES', '1415 E 70th St', 'Chicago', 'IL', 60637, '(773) 535-0540', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609916.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 30.0, 'Average', 42.0, 'Strong', 65.0, 'Strong', 63.0, 'Weak', 42.0, 'Average', 47.0, NULL, NULL, NULL, NULL, '89.90%', 101.2, '95.40%', '100.00%', 43.8, 8.3, 10.7, 18.1, 35.9, 26.6, 6.3, 7.9, 32.7, 32.7, 0.0, 11.5, 0.6, 2.6, -1.0, -1.0, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 238, 46, NULL, 1186924.7859999998, 1858826.102, 41.76772772, -87.59039017, 43, 'SOUTH SHORE', 5, 3, '(41.76772772, -87.59039017)') ,
(610203, 'Enrico Tonti Elementary School', 'ES', '5815 S Homan Ave', 'Chicago', 'IL', 60629, '(773) 535-9280', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610203.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', NULL, NULL, 'Average', 44.0, NULL, NULL, NULL, NULL, 'Weak', 47.0, 'Average', 54.0, 'Average', 49.0, 'Average', 49.0, '95.80%', 2.1, '97.20%', '100.00%', 73.4, 35.7, 37.9, 24.8, 47.9, 59.5, NULL, NULL, NULL, NULL, NULL, NULL, 17.5, 13.0, 0.7, 0.3, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1021, 44, NULL, 1154749.55, 1865777.301, 41.78750499, -87.70814201, 63, 'GAGE PARK', 14, 8, '(41.78750499, -87.70814201)') ,
(610543, 'Eric Solorio Academy High School', 'HS', '5400 S St Louis Ave', 'Chicago', 'IL', 60632, '(773) 535-9070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610543.pdf', 'AUSL Schools', 'SOUTHWEST SIDE COLLABORATIVE', NULL, 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Strong', 67.0, 'Average', 54.0, 'Average', 53.0, 'Strong', 63.0, 'Weak', 62.0, 'Strong', 64.0, 'Average', 53.0, 'Strong', 56.0, '92.20%', 3.0, '0.00%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 581, 44, 85.6, 1153924.117, 1868568.611, 41.79518118, -87.71109447, 63, 'GAGE PARK', 14, 8, '(41.79518118, -87.71109447)') ,
(610137, 'Ernst Prussing Elementary School', 'ES', '4650 N Menard Ave', 'Chicago', 'IL', 60630, '(773) 534-3460', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610137.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 71.0, NULL, NULL, 'Average', 49.0, 'Average', 42.0, 'Weak', NULL, NULL, NULL, 'Weak', 43.0, 'Weak', 44.0, '95.40%', 23.4, '96.00%', '98.50%', 84.6, NULL, 55.4, 54.0, 48.3, 58.2, 56.0, 57.4, 52.3, 46.3, 47.2, 75.5, 31.4, 24.5, -0.8, -0.1, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 672, 30, NULL, 1136787.381, 1930557.622, 41.96561054, -87.77245066, 15, 'PORTAGE PARK', 45, 16, '(41.96561054, -87.77245066)') ,
(609908, 'Esmond Elementary School', 'ES', '1865 W Montvale', 'Chicago', 'IL', 60643, '(773) 535-2650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609908.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 36.0, NULL, NULL, 'Average', 49.0, 'Average', 43.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Average', 50.0, '92.90%', 34.0, '95.70%', '100.00%', 77.6, 38.8, 33.3, 26.3, 52.0, 47.5, 32.0, 22.1, 55.2, 37.5, 8.6, 17.1, 8.9, 7.7, 0.9, -0.4, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 342, 49, NULL, 1165656.136, 1830105.142, 41.68939069, -87.66916143, 75, 'MORGAN PARK', 19, 22, '(41.68939069, -87.66916143)') ,
(609918, 'Eugene Field Elementary School', 'ES', '7019 N Ashland Ave', 'Chicago', 'IL', 60626, '(773) 534-2030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609918.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 42.0, 'Weak', 33.0, 'Average', 47.0, 'Weak', 37.0, 'Weak', 21.0, 'Very Weak', 8.0, 'Weak', 44.0, 'Average', 53.0, '96.80%', 10.0, '96.70%', '100.00%', NULL, NULL, 23.9, 24.3, 49.4, 51.4, 48.4, 36.4, 70.3, 53.2, 7.4, 13.0, 9.8, 7.9, -0.2, 0.1, 'Yellow', 'Yellow', 45.6, 20.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 375, 32, NULL, 1164539.561, 1946810.3380000002, 42.00966462, -87.66994862, 1, 'ROGERS PARK', 49, 24, '(42.00966462, -87.66994862)') ,
(609874, 'Everett McKinley Dirksen Elementary School', 'ES', '8601 W Foster Ave', 'Chicago', 'IL', 60656, '(773) 534-1090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609874.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 44.0, NULL, NULL, 'Very Weak', 12.0, 'Very Weak', 1.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.90%', 2.7, '96.20%', '93.80%', 69.9, 45.8, 65.1, 56.9, 55.5, 55.2, 51.9, 45.1, 45.6, 40.7, 22.5, 35.2, 32.8, 25.5, -0.5, -0.1, 'Yellow', 'Yellow', 6.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 786, 30, NULL, 1118113.731, 1933386.1509999998, 41.97368728, -87.84105162, 76, 'OHARE', 41, 16, '(41.97368728, -87.84105162)') ,
(610319, 'Evergreen Academy Middle School', 'MS', '3537 S Paulina St', 'Chicago', 'IL', 60609, '(773) 535-4836', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610319.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 60.0, 'Average', 58.0, 'Average', 47.0, 'Average', 44.0, 'Weak', 59.0, 'Strong', 65.0, 'Weak', 43.0, 'Average', 48.0, '96.10%', 5.5, '97.40%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 37.6, 31.6, 54.4, 57.5, 13.3, 21.7, 11.5, 9.3, -0.3, -0.5, 'Yellow', 'Yellow', 37.8, 43.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 389, 39, NULL, 1165638.817, 1881113.479, 41.829365, -87.66778007, 59, 'MCKINLEY PARK', 11, 9, '(41.829365, -87.66778007)') ,
(610057, 'Fairfield Elementary Academy', 'ES', '6201 S Fairfield Ave', 'Chicago', 'IL', 60629, '(773) 535-9500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610057.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Weak', 24.0, 'Average', 41.0, 'Weak', 34.0, 'Weak', 45.0, 'Weak', 25.0, NULL, NULL, NULL, NULL, '93.40%', 43.4, '93.30%', '98.60%', 49.0, 36.8, 15.9, 11.1, 27.3, 42.0, 20.5, 17.8, 44.7, 35.8, 0.0, 12.5, 4.1, 4.3, 0.1, -0.7, 'Yellow', 'Yellow', 16.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 604, 43, NULL, 1159148.629, 1863391.0569999998, 41.78086789, -87.69207758, 66, 'CHICAGO LAWN', 15, 8, '(41.78086789, -87.69207758)') ,
(610541, 'Federico Garcia Lorca Elementary School', 'ES', '3231 N Springfield Ave', 'Chicago', 'IL', 60618, '(773) 534-0950', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610541.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 53.0, 'Average', 58.0, 'Average', 53.0, 'Average', 58.0, 'Weak', 74.0, 'Very Strong', 82.0, 'Average', 51.0, 'Strong', 59.0, '96.80%', 43.8, '0.00%', '100.00%', 63.0, 41.7, 39.1, 25.6, 51.5, 63.8, 37.8, 27.3, 72.2, 52.3, 7.7, 25.0, 8.9, 7.5, -1.0, -0.1, 'Red', 'Yellow', 39.4, 42.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 860, 29, NULL, 1149828.0590000001, 1921320.477, 41.94001885, -87.72474354, 21, 'AVONDALE', 30, 17, '(41.94001885, -87.72474354)') ,
(610308, 'Wilma Rudolph Elementary Learning Center', 'ES', '110 N Paulina St', 'Chicago', 'IL', 60612, '(773) 534-7460', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610308.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', NULL, 'Standard', 'Not Applicable', 'Not Enough Data', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Strong', 57.0, 'Strong', 60.0, '80.00%', 0.0, '95.60%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 100, 38, NULL, 1165013.632, 1900863.727, 41.88357478, -87.66951363, 28, 'NEAR WEST SIDE', 27, 13, '(41.88357478, -87.66951363)') ,
(610120, 'Ferdinand Peck Elementary School', 'ES', '3826 W 58th St', 'Chicago', 'IL', 60629, '(773) 535-2450', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610120.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 51.0, 'Average', 43.0, 'Average', 43.0, 'Average', 47.0, 'Weak', 32.0, 'Weak', 37.0, 'Weak', 44.0, 'Weak', 43.0, '96.30%', 1.2, '97.20%', '96.90%', 74.4, 59.0, 50.8, 31.3, 53.3, 69.5, 44.0, 40.2, 71.4, 64.1, 24.1, 38.6, 23.2, 13.8, 0.7, 1.1, 'Green', 'Green', 17.8, 87.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1560, 44, NULL, 1151726.72, 1865909.055, 41.78792632, -87.71922211, 62, 'WEST ELSDON', 13, 8, '(41.78792632, -87.71922211)') ,
(609917, 'Fernwood Elementary School', 'ES', '10041 S Union Ave', 'Chicago', 'IL', 60628, '(773) 535-2700', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609917.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 29.0, 'Weak', 37.0, 'Very Weak', 17.0, 'Average', 43.0, 'Weak', 39.0, 'Average', 50.0, 'Average', 48.0, 'Average', 53.0, '93.40%', 29.0, '96.10%', '100.00%', 70.5, 43.3, 22.5, 12.7, 35.2, 55.4, 31.1, 20.4, 38.7, 34.8, 15.6, 34.4, 12.4, 7.8, -0.7, -1.7, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 344, 49, NULL, 1173454.065, 1838113.389, 41.71119799, -87.6403774, 73, 'WASHINGTON HEIGHTS', 34, 22, '(41.71119799, -87.6403774)') ,
(610258, 'Florence B Price Elementary School', 'ES', '4351 S Drexel Blvd', 'Chicago', 'IL', 60653, '(773) 535-1300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610258.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 20.0, NULL, NULL, 'Weak', 37.0, 'Average', 46.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Average', 47.0, '90.00%', 156.6, '95.20%', '100.00%', NULL, NULL, 10.0, 13.5, 46.9, 41.7, 7.8, 27.3, 44.6, 50.7, 0.0, 6.9, 0.0, 0.7, -1.6, -0.2, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 112, 40, NULL, 1183099.293, 1876145.197, 41.81534262, -87.60387349999999, 39, 'KENWOOD', 4, 2, '(41.81534262, -87.6038735)') ,
(610096, 'Florence Nightingale Elementary School', 'ES', '5250 S Rockwell St', 'Chicago', 'IL', 60632, '(773) 535-9270', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610096.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, 'Average', 54.0, 'Average', 40.0, 'Average', 48.0, 'Weak', 68.0, 'Strong', 72.0, 'Average', 48.0, 'Average', 48.0, '95.60%', 9.4, '95.80%', '99.40%', 57.1, 35.1, 38.7, 25.6, 48.7, 61.6, 32.4, 29.1, 47.7, 54.1, 10.7, 17.4, 11.3, 8.2, -0.8, -1.2, 'Red', 'Red', 23.3, 32.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1382, 43, NULL, 1159893.892, 1869545.12, 41.79774019, -87.68917624, 63, 'GAGE PARK', 14, 9, '(41.79774019, -87.68917624)') ,
(609924, 'Fort Dearborn Elementary School', 'ES', '9025 S Throop St', 'Chicago', 'IL', 60620, '(773) 535-2680', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609924.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 34.0, NULL, NULL, 'Strong', 61.0, 'Strong', 71.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 50.0, '93.40%', 24.5, '95.90%', '100.00%', 60.1, 48.3, 36.6, 35.8, 52.0, 64.5, 69.4, 44.2, 74.7, 70.2, 7.0, 33.3, 25.2, 14.1, 1.4, 0.1, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 478, 49, NULL, 1169287.183, 1844810.5380000002, 41.72966698, -87.65544435, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.72966698, -87.65544435)') ,
(609927, 'Foster Park Elementary School', 'ES', '8530 S Wood St', 'Chicago', 'IL', 60620, '(773) 535-2725', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609927.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 11.0, 'Average', 41.0, 'Weak', 20.0, 'Weak', 22.0, 'Weak', 50.0, 'Average', 46.0, 'Weak', 46.0, 'Average', 48.0, '93.70%', 33.6, '94.70%', '97.10%', NULL, 59.1, 23.4, 17.1, 41.2, 40.3, 29.2, 15.9, 45.1, 29.3, 4.0, 25.5, 8.0, 5.7, -1.1, -2.1, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 418, 43, NULL, 1165798.179, 1847975.2780000002, 41.738426200000006, -87.66813597, 71, 'AUBURN GRESHAM', 18, 6, '(41.7384262, -87.66813597)') ,
(610215, 'Francisco I Madero Middle School', 'MS', '3202 W 28th St', 'Chicago', 'IL', 60623, '(773) 535-4466', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610215.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'Yes', 'Average', 49.0, 'Weak', 32.0, 'Average', 48.0, 'Average', 47.0, 'Weak', 49.0, 'Weak', 34.0, 'Average', 47.0, 'Average', 52.0, '95.10%', 7.1, '97.00%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 28.5, 28.4, 34.3, 54.5, 10.5, 25.3, 15.1, 4.9, -1.1, -0.7, 'Red', 'Yellow', 16.5, 50.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 314, 37, NULL, 1155440.547, 1885270.824, 41.84098398, -87.70508585, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84098398, -87.70508585)') ,
(610067, 'Francis M McKay Elementary School', 'ES', '6901 S Fairfield Ave', 'Chicago', 'IL', 60629, '(773) 535-9340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610067.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 23.0, 'Average', 48.0, 'Weak', 37.0, 'Average', 54.0, 'Weak', 67.0, 'Strong', 61.0, 'Average', 48.0, 'Average', 51.0, '93.50%', 9.1, '95.10%', '100.00%', 51.4, 34.2, 16.0, 13.0, 33.2, 47.6, 23.4, 21.6, 59.1, 47.5, 5.1, 13.9, 5.7, 5.7, -0.6, 0.2, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 891, 43, NULL, 1159272.821, 1858741.621, 41.76810662, -87.69174944, 66, 'CHICAGO LAWN', 18, 8, '(41.76810662, -87.69174944)') ,
(610114, 'Francis Parkman Elementary School', 'ES', '245 W 51st St', 'Chicago', 'IL', 60609, '(773) 535-1739', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610114.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 38.0, 'Weak', 25.0, 'Average', 45.0, 'Weak', 39.0, 'Weak', 20.0, 'Weak', 28.0, NULL, NULL, NULL, NULL, '90.60%', 20.7, '95.30%', '100.00%', 62.2, 35.9, 15.1, 5.9, 48.8, 34.1, 14.8, 15.8, 39.3, 43.4, 6.3, 12.5, 4.6, 3.1, -0.3, -0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 224, 42, NULL, 1175207.047, 1871107.604, 41.80169919, -87.63297392, 37, 'FULLER PARK', 3, 9, '(41.80169919, -87.63297392)') ,
(610020, 'Francis Scott Key Elementary School', 'ES', '517 N Parkside Ave', 'Chicago', 'IL', 60644, '(773) 534-6230', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610020.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 37.0, NULL, NULL, 'Average', 49.0, 'Strong', 64.0, 'Weak', NULL, NULL, NULL, 'Weak', 44.0, 'Weak', 45.0, '91.90%', 39.6, '94.00%', '100.00%', 54.5, 44.4, 27.2, 26.3, 59.8, 55.1, 26.9, 25.9, 69.4, 63.3, 7.5, 7.5, 11.3, 5.9, 0.9, 1.5, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 301, 36, NULL, 1138633.137, 1902956.198, 41.88983589, -87.76633519, 25, 'AUSTIN', 29, 15, '(41.88983589, -87.76633519)') ,
(610112, 'Francis W Parker Elementary Community Academy', 'ES', '6800 S Stewart Ave', 'Chicago', 'IL', 60621, '(773) 535-3375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610112.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 30.0, NULL, NULL, 'Average', 53.0, 'Average', 57.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 51.0, '91.90%', 39.0, '94.90%', '98.60%', NULL, 30.9, 25.6, 14.2, 39.6, 55.7, 25.3, 27.3, 60.0, 60.0, 7.7, 14.1, 8.4, 6.5, 0.4, 0.0, 'Yellow', 'Yellow', 10.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 808, 45, NULL, 1174661.5459999999, 1859841.533, 41.77079606, -87.63530990000001, 68, 'ENGLEWOOD', 6, 7, '(41.77079606, -87.6353099)') ,
(609800, 'Frank I Bennett Elementary School', 'ES', '10115 S Prairie Ave', 'Chicago', 'IL', 60628, '(773) 535-5460', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609800.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 46.0, 'Average', 46.0, 'Average', 55.0, 'Strong', 66.0, 'Weak', 55.0, 'Average', 56.0, 'Average', 47.0, 'Weak', 45.0, '92.50%', 6.5, '95.90%', '100.00%', 57.2, 41.9, 35.3, 27.4, 56.2, 56.7, 55.2, 34.2, 60.5, 58.6, 10.0, 40.0, 13.7, 8.4, 0.7, 0.1, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 504, 48, NULL, 1179742.916, 1837880.076, 41.71041656, -87.61735342, 49, 'ROSELAND', 9, 5, '(41.71041656, -87.61735342)') ,
(609939, 'Frank L Gillespie Elementary School', 'ES', '9301 S State St', 'Chicago', 'IL', 60619, '(773) 535-5065', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609939.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 27.0, NULL, NULL, 'Weak', 30.0, 'Average', 45.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '91.90%', 2.1, '96.30%', '98.80%', 55.1, 33.6, 24.7, 24.0, 50.7, 58.7, 36.8, 33.8, 53.2, 52.8, 8.1, 16.2, 7.6, 5.6, 0.2, 0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 609, 48, NULL, 1177970.337, 1843287.805, 41.72529635, -87.62368165, 49, 'ROSELAND', 6, 6, '(41.72529635, -87.62368165)') ,
(609926, 'Franklin Elementary Fine Arts Center', 'ES', '225 W Evergreen Ave', 'Chicago', 'IL', 60610, '(773) 534-8510', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609926.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, NULL, NULL, 'Strong', 60.0, 'Average', 55.0, 'Weak', NULL, NULL, NULL, 'Strong', 58.0, 'Average', 48.0, '95.90%', 5.6, '95.30%', '100.00%', 72.7, 42.6, 74.1, 66.9, 50.0, 53.1, 70.2, 71.3, 56.4, 54.8, 33.3, 63.6, 55.1, 50.2, 0.8, 1.4, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 363, 33, NULL, 1174204.012, 1909317.563, 41.90657253, -87.6355136, 8, 'NEAR NORTH SIDE', 43, 18, '(41.90657253, -87.6355136)') ,
(609958, 'Frank W Gunsaulus Elementary Scholastic Academy', 'ES', '4420 S Sacramento Ave', 'Chicago', 'IL', 60632, '(773) 535-7215', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609958.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 46.0, 'Weak', 38.0, 'Weak', 30.0, 'Average', 45.0, 'Weak', 34.0, 'Very Weak', 14.0, 'Average', 49.0, 'Average', 48.0, '96.60%', 87.3, '96.00%', '100.00%', 72.8, 61.1, 44.0, 32.7, 48.2, 59.2, 56.3, 51.1, 56.5, 59.6, 27.4, 42.4, 24.0, 18.1, 0.6, 1.7, 'Green', 'Green', 22.4, 100.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 867, 39, NULL, 1157065.0259999998, 1875083.27, 41.81299531, -87.69940042, 58, 'BRIGHTON PARK', 14, 9, '(41.81299531, -87.69940042)') ,
(610144, 'Frank W Reilly Elementary School', 'ES', '3650 W School St', 'Chicago', 'IL', 60618, '(773) 534-5250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610144.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 48.0, NULL, NULL, 'Weak', 33.0, 'Weak', 34.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 51.0, '97.00%', 8.7, '96.70%', '100.00%', 77.8, 40.1, 39.5, 27.0, 63.2, 62.7, 48.7, 33.2, 62.7, 59.2, 11.6, 30.3, 15.5, 9.8, 0.2, 0.5, 'Yellow', 'Yellow', 22.5, 69.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1195, 29, NULL, 1151243.469, 1921732.4130000002, 41.94112155, -87.71953058, 21, 'AVONDALE', 35, 17, '(41.94112155, -87.71953058)') ,
(610165, 'Franz Peter Schubert Elementary School', 'ES', '2727 N Long Ave', 'Chicago', 'IL', 60639, '(773) 534-3080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610165.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, 'Weak', 31.0, NULL, NULL, NULL, NULL, 'Weak', 32.0, 'Weak', 26.0, 'Average', 47.0, 'Average', 49.0, '95.80%', 2.1, '97.00%', '100.00%', 73.2, 35.9, 37.4, 27.3, 49.5, 56.4, NULL, NULL, NULL, NULL, NULL, NULL, 10.6, 10.2, -0.2, -1.1, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 949, 29, NULL, 1139945.571, 1917742.0019999999, 41.93038593, -87.76115305, 19, 'BELMONT CRAGIN', 31, 25, '(41.93038593, -87.76115305)') ,
(610503, 'Frazier Prospective IB Magnet Elementary School', 'ES', '4027 W Grenshaw St', 'Chicago', 'IL', 60624, '(773) 534-6880', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610503.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 60.0, 'Average', 55.0, 'Very Strong', 84.0, 'Strong', 72.0, 'Weak', 42.0, 'Average', 55.0, 'Strong', 66.0, 'Strong', 65.0, '96.80%', 7.2, '96.60%', '100.00%', 81.3, 68.8, 71.6, 48.6, 64.9, 71.6, 76.2, 61.9, 87.3, 82.5, 5.0, 30.0, 37.6, 25.4, 1.6, 1.9, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 220, 36, NULL, 1149588.241, 1894694.332, 41.86695871, -87.72631742, 29, 'NORTH LAWNDALE', 24, 11, '(41.86695871, -87.72631742)') ,
(609854, 'Frederic Chopin Elementary School', 'ES', '2450 W Rice St', 'Chicago', 'IL', 60622, '(773) 534-4080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609854.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 86.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Weak', 97.0, 'Very Strong', 99.0, 'Average', 49.0, 'Average', 49.0, '96.90%', 1.7, '96.00%', '100.00%', 67.5, 59.7, 59.8, 41.5, 59.8, 72.0, 59.3, 56.3, 67.4, 70.1, 60.6, 45.5, 57.6, 47.6, 3.1, 4.9, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 269, 35, NULL, 1159698.085, 1905621.0559999999, 41.89674048, -87.68890154, 24, 'WEST TOWN', 1, 13, '(41.89674048, -87.68890154)') ,
(610245, 'Frederick A Douglass Academy High School', 'HS', '543 N Waller Ave', 'Chicago', 'IL', 60644, '(773) 534-6176', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610245.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 44.0, NULL, NULL, 'Weak', 39.0, 'Weak', 37.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Weak', 44.0, '79.20%', 28.0, '94.70%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.0, 12.3, 13.8, 13.5, 0.5, 14.8, 1.0, 5.1, 52.8, 39.5, 354, 36, 77.1, 1138278.715, 1903151.591, 41.89037849, -87.76763207, 25, 'AUSTIN', 29, 15, '(41.89037849, -87.76763207)') ,
(609930, 'Frederick Funston Elementary School', 'ES', '2010 N Central Park', 'Chicago', 'IL', 60647, '(773) 534-4125', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609930.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 16.0, NULL, NULL, 'Very Weak', 14.0, 'Very Weak', 16.0, 'Weak', NULL, NULL, NULL, 'Weak', 40.0, 'Average', 47.0, '94.70%', 16.8, '95.40%', '100.00%', 65.1, 40.2, 18.2, 17.4, 32.6, 48.3, 26.1, 26.8, 46.4, 41.8, 18.2, 31.8, 12.2, 5.8, -2.1, -1.1, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 619, 34, NULL, 1152012.963, 1913173.956, 41.91762127, -87.71692852, 22, 'LOGAN SQUARE', 26, 25, '(41.91762127, -87.71692852)') ,
(610163, 'Frederick Stock Elementary School', 'ES', '7507 W Birchwood Ave', 'Chicago', 'IL', 60631, '(773) 534-1215', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610163.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', NULL, 'Standard', NULL, NULL, 'No', NULL, NULL, 'Very Strong', 99.0, NULL, NULL, NULL, NULL, 'Weak', 84.0, 'Very Strong', 81.0, 'Strong', 69.0, 'Strong', 63.0, '72.40%', 0.0, '95.70%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 260, 30, NULL, 1125318.193, 1949163.0259999998, 42.0168639, -87.81420664, 9, 'EDISON PARK', 41, 16, '(42.0168639, -87.81420664)') ,
(610010, 'Friedrich Ludwig Jahn Elementary School', 'ES', '3149 N Wolcott Ave', 'Chicago', 'IL', 60657, '(773) 534-5500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610010.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 72.0, NULL, NULL, 'Strong', 60.0, 'Average', 59.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Weak', 46.0, '94.70%', 19.9, '94.70%', '100.00%', 59.4, 36.2, 41.4, 40.6, 47.9, 48.9, 30.2, 45.5, 40.6, 58.7, 13.5, 32.4, 23.9, 15.2, 0.3, 0.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 495, 33, NULL, 1163153.6709999999, 1921136.53, 41.93924404, -87.67577282, 5, 'NORTH CENTER', 32, 19, '(41.93924404, -87.67577282)') ,
(609737, 'Friedrich W von Steuben Metropolitan Science High School', 'HS', '5039 N Kimball Ave', 'Chicago', 'IL', 60625, '(773) 534-5100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609737.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 70.0, 'Weak', 39.0, 'Average', 42.0, 'Average', 51.0, 'Weak', 26.0, 'Weak', 29.0, NULL, NULL, NULL, NULL, '87.00%', 4.0, '95.60%', '99.20%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 15.9, 15.7, 17.3, 16.7, 0.8, 20.4, 3.1, 34.6, 88.2, 78.5, 1636, 31, 76.8, 1152840.522, 1933431.315, 41.973192600000004, -87.71334969, 13, 'NORTH PARK', 39, 17, '(41.9731926, -87.71334969)') ,
(609709, 'Gage Park High School', 'HS', '5630 S Rockwell St', 'Chicago', 'IL', 60629, '(773) 535-9230', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609709.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 14.0, 'Average', 43.0, 'Weak', 29.0, 'Weak', 37.0, 'Weak', 38.0, 'Weak', 35.0, NULL, NULL, NULL, NULL, '72.50%', 19.5, '94.00%', '99.10%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.4, 12.0, 13.8, 13.6, 1.2, 14.1, 0.3, 7.9, 52.8, 40.3, 1052, 43, 46.0, 1159962.875, 1867094.3469999998, 41.79101352, -87.68899064, 63, 'GAGE PARK', 16, 8, '(41.79101352, -87.68899064)') ,
(610009, 'Galileo Math & Science Scholastic Academy Elementary School', 'ES', '820 S Carpenter St', 'Chicago', 'IL', 60607, '(773) 534-7070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610009.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 65.0, 'Average', 52.0, 'Average', 40.0, 'Average', 42.0, 'Weak', 56.0, 'Weak', 33.0, 'Average', 49.0, 'Average', 50.0, '95.00%', 3.1, '97.00%', '100.00%', 77.3, 60.5, 54.7, 55.4, 68.7, 68.2, 55.6, 59.5, 66.3, 56.6, 33.3, 41.7, 26.5, 22.3, -1.0, -0.8, 'Red', 'Red', 33.3, 78.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 606, 38, NULL, 1169436.562, 1896405.606, 41.87124634, -87.65340205, 28, 'NEAR WEST SIDE', 25, 12, '(41.87124634, -87.65340205)') ,
(610072, 'Garrett A Morgan Elementary School', 'ES', '8407 S Kerfoot Ave', 'Chicago', 'IL', 60620, '(773) 535-3366', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610072.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Weak', 38.0, 'Average', 58.0, 'Average', 55.0, 'Weak', 46.0, 'Weak', 36.0, 'Average', 47.0, 'Weak', 46.0, '92.00%', 10.0, '94.80%', '96.80%', 71.2, 39.6, 28.3, 26.2, 56.3, 64.6, 19.2, 30.9, 53.1, 55.8, 8.3, 21.7, 7.9, 3.2, -0.7, 0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 237, 45, NULL, 1173449.365, 1849063.053, 41.74124549, -87.64007166, 71, 'AUBURN GRESHAM', 21, 6, '(41.74124549, -87.64007166)') ,
(610293, 'Genevieve Melody Elementary School', 'ES', '412 S Keeler Ave', 'Chicago', 'IL', 60624, '(773) 534-6850', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610293.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Strong', 61.0, NULL, NULL, 'Strong', 65.0, 'Strong', 77.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '92.10%', 26.0, '96.00%', '94.50%', 56.0, NULL, 11.1, 19.2, 43.3, 37.1, 26.9, 29.6, 55.8, 76.9, 6.3, 12.5, 4.2, 6.3, -0.3, 0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 315, 36, NULL, 1148424.226, 1897525.231, 41.87474954, -87.73051772, 26, 'WEST GARFIELD PARK', 24, 11, '(41.87474954, -87.73051772)') ,
(609779, 'George Armstrong International Studies Elementary School', 'ES', '2110 W Greenleaf Ave', 'Chicago', 'IL', 60645, '(773) 534-2150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609779.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 59.0, 'Average', 57.0, 'Average', 59.0, 'Strong', 64.0, 'Weak', 53.0, 'Average', 45.0, NULL, NULL, NULL, NULL, '96.30%', 8.1, '96.30%', '99.40%', 66.5, 45.5, 47.5, 38.8, 55.7, 65.5, 59.7, 51.5, 72.9, 54.4, 28.2, 43.5, 30.4, 19.5, 0.7, -0.2, 'Green', 'Yellow', 30.8, 60.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1509, 32, NULL, 1160591.979, 1946966.544, 42.01017628, -87.68446868, 2, 'WEST RIDGE', 50, 24, '(42.01017628, -87.68446868)') ,
(610062, 'George B McClellan Elementary School', 'ES', '3527 S Wallace St', 'Chicago', 'IL', 60609, '(773) 535-1732', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610062.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 49.0, NULL, NULL, 'Weak', 33.0, 'Average', 40.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Average', 47.0, '93.20%', 7.8, '96.00%', '98.60%', 40.5, 17.4, 35.2, 26.8, 65.2, 55.1, 33.9, 39.7, 57.6, 58.3, 27.8, 22.2, 22.7, 21.3, 0.0, 0.9, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 283, 40, NULL, 1172917.689, 1881398.625, 41.82998962, -87.64106579, 60, 'BRIDGEPORT', 11, 9, '(41.82998962, -87.64106579)') ,
(610196, 'George B Swift Elementary Specialty School', 'ES', '5900 N Winthrop Ave', 'Chicago', 'IL', 60660, '(773) 534-2695', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610196.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 61.0, 'Average', 53.0, 'Strong', 62.0, 'Strong', 60.0, 'Weak', 61.0, 'Average', 59.0, 'Average', 49.0, 'Strong', 54.0, '96.30%', 6.0, '95.80%', '98.80%', 69.2, 50.4, 49.4, 33.9, 55.5, 63.3, 52.9, 36.3, 58.2, 51.2, 23.9, 23.9, 25.4, 16.2, -0.8, 1.1, 'Red', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 709, 32, NULL, 1167755.176, 1939362.0269999998, 41.98915734, -87.65833342, 77, 'EDGEWATER', 48, 20, '(41.98915734, -87.65833342)') ,
(609849, 'George F Cassell Elementary School', 'ES', '11314 S Spaulding Ave', 'Chicago', 'IL', 60655, '(773) 535-2640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609849.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 76.0, 'Strong', 67.0, 'Average', 57.0, 'Weak', 36.0, 'Weak', 66.0, 'Strong', 61.0, 'Average', 52.0, 'Weak', 46.0, '94.40%', 13.8, '95.10%', '98.90%', 62.6, 40.0, 48.5, 54.7, 57.5, 66.3, 43.6, 61.0, 57.3, 63.6, 20.5, 54.5, 24.9, 24.5, -0.6, -0.8, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 358, 49, NULL, 1156350.416, 1829287.52, 41.68733881, -87.70325179, 74, 'MOUNT GREENWOOD', 19, 22, '(41.68733881, -87.70325179)') ,
(609761, 'George H Corliss High School', 'HS', '821 E 103rd St', 'Chicago', 'IL', 60628, '(773) 535-5115', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609761.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 33.0, 'Average', 52.0, 'Average', 40.0, 'Average', 48.0, 'Weak', 48.0, 'Average', 54.0, NULL, NULL, NULL, NULL, '71.30%', 23.4, '95.20%', '98.50%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.0, 11.4, 13.0, 12.9, 0.9, 14.3, 1.3, 10.6, 42.8, 44.0, 621, 48, 76.2, 1183649.971, 1836810.314, 41.70739096, -87.60307842, 50, 'PULLMAN', 9, 5, '(41.70739096, -87.60307842)') ,
(610305, 'George Leland Elementary School', 'ES', '5221 W Congress Pkwy', 'Chicago', 'IL', 60644, '(773) 534-6340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610305.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, 'Strong', 71.0, NULL, NULL, NULL, NULL, 'Weak', 68.0, 'Strong', 77.0, 'Strong', 60.0, 'Average', 53.0, '92.60%', 3.3, '94.90%', '100.00%', 67.8, 59.1, 30.8, 23.1, 46.2, 61.5, NULL, NULL, NULL, NULL, NULL, NULL, 46.2, 20.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 182, 36, NULL, 1141563.256, 1897140.916, 41.87382439, -87.75571809, 25, 'AUSTIN', 29, 15, '(41.87382439, -87.75571809)') ,
(610048, 'George Manierre Elementary School', 'ES', '1420 N Hudson Ave', 'Chicago', 'IL', 60610, '(773) 534-8456', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610048.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 43.0, NULL, NULL, 'Average', 49.0, 'Average', 40.0, 'Weak', NULL, NULL, NULL, 'Weak', 42.0, 'Weak', 44.0, '91.00%', 44.0, '94.80%', '100.00%', 44.4, NULL, 23.7, 13.9, 44.5, 52.8, 26.5, 17.1, 49.1, 41.7, 0.0, 11.1, 4.6, 4.7, -1.1, -1.2, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 396, 33, NULL, 1173014.972, 1909920.5680000002, 41.90825367, -87.63986347, 8, 'NEAR NORTH SIDE', 27, 18, '(41.90825367, -87.63986347)') ,
(610139, 'George M Pullman Elementary School', 'ES', '11311 S Forrestville Ave', 'Chicago', 'IL', 60628, '(773) 535-5395', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610139.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, NULL, NULL, 'Strong', 66.0, 'Strong', 75.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 49.0, '93.50%', 14.6, '96.20%', '100.00%', 63.8, 49.4, 32.2, 41.7, 69.5, 67.2, 39.7, 32.9, 67.2, 54.3, 4.0, 16.0, 14.8, 17.8, 0.7, 0.7, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 259, 48, NULL, 1181982.459, 1830019.85, 41.68879566, -87.60939383, 50, 'PULLMAN', 9, 5, '(41.68879566, -87.60939383)') ,
(609795, 'George Rogers Clark Elementary School', 'ES', '1045 S Monitor Ave', 'Chicago', 'IL', 60644, '(773) 534-6225', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609795.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Weak', 39.0, NULL, NULL, 'Weak', 32.0, 'Weak', 36.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 51.0, '93.90%', 14.4, '96.40%', '100.00%', 80.0, 74.1, 56.8, 37.5, 60.7, 71.1, 60.5, 46.6, 57.5, 56.1, 30.0, 33.3, 22.0, 15.3, 1.2, 0.7, 'Green', 'Yellow', 29.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 297, 36, NULL, 1137583.9679999999, 1894930.8769999999, 41.867832299999996, -87.77038158, 25, 'AUSTIN', 29, 15, '(41.8678323, -87.77038158)') ,
(609760, 'George Washington Carver Military Academy High School', 'HS', '13100 S Doty Ave', 'Chicago', 'IL', 60627, '(773) 535-5250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609760.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 53.0, NULL, NULL, 'Weak', 37.0, 'Weak', 34.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '91.60%', 8.9, '96.40%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.5, 14.2, 15.0, 14.9, 0.4, 16.8, 1.8, 21.2, 78.7, 60.5, 532, 48, 79.5, 1187202.39, 1818214.083, 41.65627709, -87.5906566, 54, 'RIVERDALE', 9, 5, '(41.65627709, -87.5906566)') ,
(609845, 'George Washington Carver Primary School', 'ES', '901 E 133rd Pl', 'Chicago', 'IL', 60827, '(773) 535-5674', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609845.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 25.0, NULL, NULL, 'Weak', 33.0, 'Weak', 30.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Strong', 57.0, '90.90%', 15.6, '94.70%', '100.00%', 37.4, 36.6, 23.8, 19.7, 38.8, 45.8, 22.0, 17.1, 27.9, 43.1, 4.5, 22.7, 9.4, 8.0, -0.8, -0.6, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 546, 48, NULL, 1184786.467, 1817244.896, 41.65367428, -87.59952683, 54, 'RIVERDALE', 9, 5, '(41.65367428, -87.59952683)') ,
(610219, 'George Washington Elementary School', 'ES', '3611 E 114th St', 'Chicago', 'IL', 60617, '(773) 535-5010', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610219.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 58.0, 'Strong', 61.0, 'Weak', 33.0, 'Average', 43.0, 'Weak', 80.0, 'Strong', 76.0, 'Average', 52.0, 'Average', 51.0, '95.50%', 5.6, '95.90%', '99.00%', 69.3, 56.4, 53.8, 59.6, 70.0, 76.7, 54.2, 58.1, 69.6, 68.3, 23.0, 37.5, 30.4, 24.7, 0.4, 0.8, 'Yellow', 'Green', 36.8, 25.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 813, 47, NULL, 1202028.594, 1829944.81, 41.68810403, -87.53600985, 52, 'EAST SIDE', 10, 4, '(41.68810403, -87.53600985)') ,
(609739, 'George Washington High School', 'HS', '3535 E 114th St', 'Chicago', 'IL', 60617, '(773) 535-5725', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609739.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 46.0, 'Average', 53.0, 'Weak', 36.0, 'Weak', 36.0, 'Weak', 52.0, 'Average', 46.0, NULL, NULL, NULL, NULL, '80.20%', 12.7, '95.10%', '99.50%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.4, 13.6, 14.3, 14.1, 0.7, 15.5, 1.2, 14.0, 71.6, 50.9, 1524, 47, 78.6, 1201607.873, 1829942.879, 41.688109399999995, -87.53755009999999, 52, 'EAST SIDE', 10, 4, '(41.6881094, -87.5375501)') ,
(609900, 'George W Curtis Elementary School', 'ES', '32 E 115th St', 'Chicago', 'IL', 60628, '(773) 535-5050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609900.pdf', 'AUSL Schools', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 23.0, NULL, NULL, 'Average', 43.0, 'Average', 58.0, 'Weak', NULL, NULL, NULL, 'Strong', 68.0, 'Strong', 70.0, '93.80%', 18.2, '98.10%', '97.90%', 51.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 9.1, 18.8, 10.1, 3.8, 0.9, 0.9, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 504, 48, NULL, 1178641.811, 1828771.2619999999, 41.68544575, -87.62166148, 49, 'ROSELAND', 9, 5, '(41.68544575, -87.62166148)') ,
(609693, 'George Westinghouse High School', 'HS', '3223 W Franklin Blvd', 'Chicago', 'IL', 60624, '(773) 534-6400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609693.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', NULL, 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Strong', 74.0, 'Strong', 72.0, 'Strong', 77.0, 'Very Strong', 80.0, 'Weak', 86.0, 'Strong', 71.0, 'Strong', 54.0, 'Strong', 60.0, '93.60%', 2.0, '96.80%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 16.1, 16.9, NULL, 17.1, 1.0, NULL, NULL, NULL, NULL, NULL, 828, 34, 90.5, 1154695.243, 1903109.304, 41.88994963, -87.70734353, 23, 'HUMBOLDT PARK', 28, 11, '(41.88994963, -87.70734353)') ,
(610202, 'George W Tilton Elementary School', 'ES', '223 N Keeler Ave', 'Chicago', 'IL', 60624, '(773) 534-6746', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610202.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 6.0, 'Weak', 25.0, 'Weak', 30.0, 'Average', 41.0, 'Weak', 25.0, 'Average', 44.0, 'Average', 51.0, 'Average', 49.0, '91.40%', 65.5, '95.10%', '100.00%', 51.1, 39.6, 22.5, 10.8, 29.0, 36.8, 29.1, 12.0, 46.2, 21.4, 14.3, 8.8, 10.0, 3.5, -0.7, -2.6, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 321, 36, NULL, 1148397.097, 1900931.417, 41.88409703, -87.73052949, 26, 'WEST GARFIELD PARK', 28, 11, '(41.88409703, -87.73052949)') ,
(609920, 'Gerald Delgado Kanoon Elementary Magnet School', 'ES', '2233 S Kedzie Ave', 'Chicago', 'IL', 60623, '(773) 534-1736', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609920.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 44.0, 'Weak', 37.0, 'Average', 48.0, 'Average', 48.0, 'Weak', 61.0, 'Average', 43.0, 'Average', 47.0, 'Average', 49.0, '95.90%', 3.3, '95.60%', '100.00%', 76.2, 35.2, 16.9, 18.3, 51.7, 51.2, 25.7, 28.6, 51.5, 60.0, 16.0, 22.0, 8.0, 6.6, -0.4, -0.6, 'Yellow', 'Yellow', 21.6, 45.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 713, 37, NULL, 1155417.273, 1888838.9309999999, 41.85077575, -87.70507547, 30, 'SOUTH LAWNDALE', 24, 10, '(41.85077575, -87.70507547)') ,
(610187, 'Graeme Stewart Elementary School', 'ES', '4525 N Kenmore Ave', 'Chicago', 'IL', 60640, '(773) 534-2640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610187.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 49.0, NULL, NULL, 'Average', 46.0, 'Average', 57.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 52.0, '95.20%', 77.0, '96.10%', '100.00%', 47.4, 44.2, 23.6, 25.8, 66.3, 52.9, 31.7, 46.0, 59.3, 55.9, 9.1, 40.9, 11.9, 9.7, 0.5, 1.1, 'Yellow', 'Yellow', 30.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 280, 32, NULL, 1168446.748, 1930325.571, 41.96434608, -87.65605252, 3, 'UPTOWN', 46, 19, '(41.96434608, -87.65605252)') ,
(610285, 'Granville T Woods Math & Science Academy Elementary School', 'ES', '6206 S Racine Ave', 'Chicago', 'IL', 60636, '(773) 535-9250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610285.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, NULL, NULL, 'Average', 43.0, 'Weak', 31.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 49.0, '91.90%', 41.5, '95.20%', '100.00%', NULL, 61.7, 25.8, 11.4, 35.1, 53.8, 24.2, 21.6, 50.9, 46.3, 5.4, 17.5, 11.1, 6.5, 0.0, -0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 402, 43, NULL, 1169345.488, 1863577.552, 41.7811649, -87.65468845, 67, 'WEST ENGLEWOOD', 16, 7, '(41.7811649, -87.65468845)') ,
(610383, 'Greater Lawndale High School For Social Justice', 'HS', '3120 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 535-4300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610383.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 43.0, 'Strong', 72.0, 'Average', 46.0, 'Average', 44.0, 'Weak', 81.0, 'Strong', 65.0, 'Average', 53.0, 'Weak', 46.0, '87.50%', 16.4, '95.10%', '95.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.6, 14.0, 14.5, 14.4, 0.8, 15.7, 1.2, 14.9, 67.0, 57.4, 371, 37, 87.4, 1147521.3020000001, 1883405.128, 41.83601953, -87.73419465, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83601953, -87.73419465)') ,
(609857, 'Grover Cleveland Elementary School', 'ES', '3121 W Byron St', 'Chicago', 'IL', 60618, '(773) 534-5130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609857.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, 'Average', 43.0, 'Average', 58.0, 'Average', 47.0, 'Weak', 51.0, 'Average', 44.0, 'Average', 49.0, 'Average', 48.0, '95.00%', 6.1, '96.30%', '100.00%', 72.5, 38.7, 53.6, 37.3, 49.7, 49.5, 47.9, 45.7, 50.5, 58.3, 21.2, 36.5, 23.4, 15.8, -0.3, -0.5, 'Yellow', 'Yellow', 27.6, 58.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 688, 31, NULL, 1154778.842, 1925727.705, 41.95201470000001, -87.70642927, 16, 'IRVING PARK', 33, 17, '(41.9520147, -87.70642927)') ,
(610241, 'Guglielmo Marconi Elementary Community Academy', 'ES', '230 N Kolmar Ave', 'Chicago', 'IL', 60624, '(773) 534-6210', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610241.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 26.0, NULL, NULL, 'Strong', 65.0, 'Strong', 76.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Average', 52.0, '91.50%', 58.8, '96.50%', '100.00%', 64.7, 65.3, 28.8, 23.4, 51.7, 70.0, 37.7, 31.5, 64.6, 65.7, 0.0, 21.1, 4.0, 7.3, -1.6, -1.3, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 245, 36, NULL, 1145991.517, 1900994.657, 41.88431659, -87.73936153, 26, 'WEST GARFIELD PARK', 28, 11, '(41.88431659, -87.73936153)') ,
(609741, 'Gurdon S Hubbard High School', 'HS', '6200 S Hamlin Ave', 'Chicago', 'IL', 60629, '(773) 535-2200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609741.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 42.0, NULL, NULL, 'Weak', 34.0, 'Weak', 29.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '89.30%', 4.3, '96.40%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.4, 14.2, 15.3, 15.3, 0.9, 17.3, 2.0, 30.1, 57.2, 51.3, 1672, 44, 64.5, 1152065.741, 1863208.025, 41.78050763, -87.71804991, 65, 'WEST LAWN', 13, 8, '(41.78050763, -87.71804991)') ,
(609726, 'Gwendolyn Brooks College Preparatory Academy High School', 'HS', '250 E 111th St', 'Chicago', 'IL', 60628, '(773) 535-9930', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609726.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, NULL, NULL, 'Average', 44.0, 'Average', 49.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 49.0, '87.10%', 0.1, '95.20%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 18.0, 18.2, 18.7, 18.7, 0.7, 21.5, 2.8, 51.1, 88.1, 87.9, 783, 48, 90.3, 1180061.781, 1831459.2130000002, 41.69278956, -87.61638144, 49, 'ROSELAND', 9, 5, '(41.69278956, -87.61638144)') ,
(610182, 'Hannah G Solomon Elementary School', 'ES', '6206 N Hamlin Ave', 'Chicago', 'IL', 60659, '(773) 534-5226', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610182.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 78.0, 'Very Strong', 96.0, 'Strong', 67.0, 'Strong', 62.0, 'Weak', 75.0, 'Strong', 62.0, 'Average', 52.0, 'Average', 51.0, '95.80%', 0.8, '95.70%', '99.00%', 71.2, 51.1, 74.4, 63.6, 59.1, 70.0, 73.9, 72.8, 75.0, 67.4, 45.5, 63.6, 52.4, 42.8, 0.6, -0.2, 'Yellow', 'Yellow', 45.5, 93.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 363, 31, NULL, 1149868.813, 1940981.795, 41.99397007, -87.72407991, 13, 'NORTH PARK', 39, 17, '(41.99397007, -87.72407991)') ,
(610068, 'Hanson Park Elementary School', 'ES', '5411 W Fullerton Ave', 'Chicago', 'IL', 60639, '(773) 534-3100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610068.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, NULL, NULL, 'Weak', 34.0, 'Weak', 33.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 47.0, '95.00%', 24.4, '94.90%', '100.00%', 48.5, 42.6, 31.0, 20.6, 45.9, 51.5, 42.0, 29.2, 50.7, 41.2, 12.4, 22.1, 14.4, 11.0, -0.8, -0.5, 'Red', 'Yellow', 14.0, 88.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1499, 29, NULL, 1139861.158, 1915387.949, 41.92392771, -87.76152094, 19, 'BELMONT CRAGIN', 37, 25, '(41.92392771, -87.76152094)') ,
(610124, 'Harold Washington Elementary School', 'ES', '9130 S University Ave', 'Chicago', 'IL', 60619, '(773) 535-6225', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610124.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 28.0, NULL, NULL, 'Average', 45.0, 'Strong', 63.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 49.0, '96.10%', 3.1, '96.20%', '100.00%', 67.6, 55.6, 27.8, 39.0, 53.2, 53.6, 32.5, 37.6, 45.0, 46.6, 9.5, 28.6, 16.0, 13.1, -0.2, 0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 549, 48, NULL, 1185436.746, 1844530.731, 41.72853492, -87.59629327, 47, 'BURNSIDE', 8, 4, '(41.72853492, -87.59629327)') ,
(610192, 'Harriet Beecher Stowe Elementary School', 'ES', '3444 W Wabansia Ave', 'Chicago', 'IL', 60647, '(773) 534-4175', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610192.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 28.0, NULL, NULL, 'Weak', 26.0, 'Weak', 31.0, 'Weak', NULL, NULL, NULL, 'Weak', 44.0, 'Strong', 54.0, '93.80%', 13.7, '94.70%', '98.70%', 39.9, 32.2, 11.1, 12.0, 36.9, 48.6, 16.7, 18.0, 49.4, 48.6, 2.1, 9.5, 3.1, 2.5, -0.2, -0.8, 'Yellow', 'Red', 19.1, 11.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 893, 34, NULL, 1153019.365, 1911120.2580000001, 41.91196584, -87.7132855, 23, 'HUMBOLDT PARK', 26, 14, '(41.91196584, -87.7132855)') ,
(610158, 'Harriet E Sayre Elementary Language Academy', 'ES', '1850 N Newland Ave', 'Chicago', 'IL', 60707, '(773) 534-3351', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610158.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 68.0, NULL, NULL, 'Average', 41.0, 'Average', 50.0, 'Weak', NULL, NULL, NULL, 'Weak', 44.0, 'Average', 49.0, '96.20%', 2.6, '96.60%', '100.00%', 69.1, NULL, 37.6, 42.5, 44.8, 37.1, 41.7, 48.1, 47.6, 42.8, 20.0, 38.3, 19.6, 17.3, -0.5, 0.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 605, 29, NULL, 1129750.187, 1911749.959, 41.9141236, -87.79875667, 25, 'AUSTIN', 36, 25, '(41.9141236, -87.79875667)') ,
(609974, 'Hawthorne Elementary Scholastic Academy', 'ES', '3319 N Clifton Ave', 'Chicago', 'IL', 60657, '(773) 534-5550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609974.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'Yes', 'Very Strong', 92.0, 'Very Strong', 88.0, 'Strong', 61.0, 'Average', 56.0, 'Weak', 90.0, 'Very Strong', 84.0, 'Strong', 58.0, 'Average', 52.0, '96.90%', 1.2, '96.50%', '100.00%', 85.3, 56.9, 82.7, 83.3, 58.6, 62.5, 89.1, 87.4, 65.1, 62.9, 79.0, 80.6, 75.1, 66.0, 1.8, 2.2, 'Green', 'Green', 98.4, 91.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 569, 33, NULL, 1168069.5259999998, 1922284.759, 41.94228994, -87.65767253, 6, 'LAKE VIEW', 44, 19, '(41.94228994, -87.65767253)') ,
(609985, 'Helen M Hefferan Elementary School', 'ES', '4409 W Wilcox St', 'Chicago', 'IL', 60624, '(773) 534-6192', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609985.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 29.0, 'Weak', 39.0, 'Weak', 26.0, 'Strong', 63.0, 'Weak', 58.0, 'Average', 52.0, 'Average', 51.0, 'Strong', 56.0, '93.80%', 13.4, '95.90%', '100.00%', 72.4, 64.9, 36.5, 34.1, 45.7, 37.3, 61.4, 46.7, 61.6, 51.1, 28.0, 44.0, 25.6, 19.2, 0.5, 0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 259, 36, NULL, 1146965.713, 1898931.63, 41.87863685, -87.73583687, 26, 'WEST GARFIELD PARK', 28, 11, '(41.87863685, -87.73583687)') ,
(610122, 'Helen Peirce International Studies Elementary School', 'ES', '1423 W Bryn Mawr Ave', 'Chicago', 'IL', 60660, '(773) 534-2440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610122.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, NULL, NULL, 'Weak', 32.0, 'Weak', 22.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 47.0, '96.70%', 5.8, '95.80%', '100.00%', 57.2, 58.2, 56.4, 47.2, 52.4, 63.3, 52.8, 49.1, 49.1, 57.3, 25.9, 35.2, 25.5, 20.5, -1.9, -1.4, 'Red', 'Red', 100.0, 5.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 977, 32, NULL, 1165723.804, 1937258.243, 41.98342819, -87.66586537, 77, 'EDGEWATER', 48, 20, '(41.98342819, -87.66586537)') ,
(609972, 'Helge A Haugan Elementary School', 'ES', '4540 N Hamlin Ave', 'Chicago', 'IL', 60625, '(773) 534-5040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609972.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 43.0, 'Weak', 39.0, 'Average', 57.0, 'Average', 49.0, 'Weak', 33.0, 'Weak', 30.0, 'Weak', 46.0, 'Average', 51.0, '96.20%', 23.4, '94.60%', '100.00%', 68.9, 37.2, 31.5, 24.3, 53.4, 58.5, 32.5, 29.3, 48.0, 32.9, NULL, NULL, 15.2, 10.2, -0.3, 0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1299, 31, NULL, 1150195.236, 1930075.176, 41.96403525, -87.72316496, 14, 'ALBANY PARK', 39, 17, '(41.96403525, -87.72316496)') ,
(609856, 'Henry Clay Elementary School', 'ES', '13231 S Burley Ave', 'Chicago', 'IL', 60633, '(773) 535-5600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609856.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 38.0, 'Weak', 39.0, 'Weak', 30.0, 'Weak', 30.0, 'Weak', 24.0, 'Weak', 31.0, NULL, NULL, NULL, NULL, '95.20%', 6.0, '96.10%', '100.00%', 52.4, 47.0, 34.4, 27.1, 42.0, 48.0, 39.3, 29.6, 54.8, 37.1, 10.2, 25.4, 17.1, 12.3, -0.3, -0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 633, 47, NULL, 1199800.451, 1817700.031, 41.65455937, -87.544577, 55, 'HEGEWISCH', 10, 4, '(41.65455937, -87.544577)') ,
(610040, 'Henry D Lloyd Elementary School', 'ES', '2103 N Lamon Ave', 'Chicago', 'IL', 60639, '(773) 534-3070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610040.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 51.0, '96.20%', 0.0, '97.20%', '97.00%', NULL, 32.9, 34.1, 18.3, 46.5, 53.3, NULL, NULL, NULL, NULL, NULL, NULL, 23.5, 10.5, -0.8, -0.3, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1334, 29, NULL, 1143423.139, 1913554.3590000002, 41.91883022, -87.74847861, 19, 'BELMONT CRAGIN', 31, 25, '(41.91883022, -87.74847861)') ,
(610092, 'Henry H Nash Elementary School', 'ES', '4837 W Erie St', 'Chicago', 'IL', 60644, '(773) 534-6125', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610092.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 41.0, NULL, NULL, 'Strong', 68.0, 'Strong', 76.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.10%', 27.9, '95.80%', '100.00%', 48.6, 46.4, 22.4, 18.9, 60.4, 59.1, 27.1, 19.5, 74.6, 60.7, 5.1, 7.5, 7.1, 6.0, 0.6, -0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 393, 36, NULL, 1143836.768, 1903833.4530000002, 41.89214724, -87.74720292, 25, 'AUSTIN', 28, 15, '(41.89214724, -87.74720292)') ,
(610279, 'Henry O Tanner Elementary School', 'ES', '7350 S Evans Ave', 'Chicago', 'IL', 60619, '(773) 535-3870', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610279.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 33.0, NULL, NULL, 'Average', 42.0, 'Average', 55.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '91.80%', 43.1, '95.60%', '98.00%', 61.3, 35.3, 19.5, 13.3, 52.6, 52.3, 22.1, 21.1, 56.6, 57.4, 5.3, 8.1, 6.9, 4.6, 1.9, 0.3, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 448, 46, NULL, 1182462.062, 1856226.416, 41.76069854, -87.60682834, 69, 'GREATER GRAND CROSSING', 6, 3, '(41.76069854, -87.60682834)') ,
(609861, 'Henry R Clissold Elementary School', 'ES', '2350 W 110th Pl', 'Chicago', 'IL', 60643, '(773) 535-2560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609861.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 45.0, 'Strong', 61.0, 'Weak', 39.0, 'Strong', 61.0, 'Weak', 38.0, 'Average', 51.0, 'Average', 52.0, 'Weak', 46.0, '95.10%', 8.4, '96.90%', '94.80%', 75.5, NULL, 58.5, 64.0, 68.8, 62.8, 46.7, 63.0, 53.3, 58.7, 33.8, 49.2, 27.7, 26.7, -0.7, 0.5, 'Red', 'Yellow', 42.2, 44.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 588, 49, NULL, 1162450.244, 1831415.5990000002, 41.69305404, -87.68086190000001, 75, 'MORGAN PARK', 19, 22, '(41.69305404, -87.6808619)') ,
(610297, 'High School of Leadership at South Shore', 'HS', '7627 S Constance Ave', 'Chicago', 'IL', 60649, '(773) 535-6190', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610297.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', NULL, NULL, 'Weak', 30.0, NULL, NULL, NULL, NULL, 'Weak', 16.0, 'Weak', 28.0, 'Weak', 42.0, 'Weak', 31.0, '72.00%', 43.4, '95.20%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 11.7, 11.7, 13.1, 13.1, 1.4, 14.4, 1.3, 12.2, 36.6, 68.8, 731, 46, 48.8, 1189903.206, 1854648.757, 41.75619362, -87.57960727, 43, 'SOUTH SHORE', 8, 4, '(41.75619362, -87.57960727)') ,
(609798, 'Hiram H Belding Elementary School', 'ES', '4257 N Tripp Ave', 'Chicago', 'IL', 60641, '(773) 534-3590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609798.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 64.0, 'Strong', 63.0, 'Strong', 67.0, 'Strong', 69.0, 'Weak', 49.0, 'Average', 51.0, 'Average', 53.0, 'Average', 52.0, '95.00%', 21.3, '96.20%', '98.60%', 80.0, 78.6, 46.2, 40.4, 50.3, 45.7, 55.6, 54.6, 54.5, 54.2, 36.8, 49.1, 27.3, 21.4, -0.8, -0.5, 'Red', 'Yellow', 46.4, 57.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 672, 31, NULL, 1147266.366, 1928184.928, 41.95890499, -87.73398223, 16, 'IRVING PARK', 38, 17, '(41.95890499, -87.73398223)') ,
(609768, 'Hope College Preparatory High School', 'HS', '5515 S Lowe Ave', 'Chicago', 'IL', 60621, '(773) 535-3160', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609768.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Average', 51.0, 'Weak', 37.0, 'Average', 44.0, 'Weak', 56.0, 'Weak', 35.0, NULL, NULL, NULL, NULL, '72.20%', 24.0, '94.30%', '99.10%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.3, 13.0, 13.7, 13.5, 1.2, 15.0, 1.3, 16.7, 55.2, 58.2, 561, 45, 63.6, 1172948.54, 1868120.666, 41.79355292, -87.64134488, 68, 'ENGLEWOOD', 3, 7, '(41.79355292, -87.64134488)') ,
(609850, 'Horace Greeley Elementary School', 'ES', '832 W Sheridan Rd', 'Chicago', 'IL', 60613, '(773) 534-5800', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609850.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 65.0, NULL, NULL, 'Average', 48.0, 'Weak', 38.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Average', 52.0, '95.90%', 6.0, '96.00%', '100.00%', 64.5, 35.5, 46.7, 37.4, 48.8, 52.8, 65.6, 47.9, 53.1, 49.7, 43.1, 50.0, 26.9, 16.3, 0.2, -0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 611, 32, NULL, 1169860.926, 1926142.5780000002, 41.95283701, -87.65097549, 6, 'LAKE VIEW', 46, 19, '(41.95283701, -87.65097549)') ,
(610052, 'Horace Mann Elementary School', 'ES', '8050 S Chappel Ave', 'Chicago', 'IL', 60617, '(773) 535-6640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610052.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Weak', 31.0, 'Weak', 31.0, 'Weak', 30.0, 'Weak', 31.0, 'Weak', 24.0, 'Weak', 44.0, 'Weak', 46.0, '92.60%', 44.4, '94.50%', '100.00%', 46.9, 30.2, 12.8, 16.5, 49.7, 46.2, 23.3, 23.3, 57.2, 51.0, 6.4, 27.7, 3.4, 3.4, -0.7, 0.1, 'Red', 'Yellow', 19.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 472, 47, NULL, 1191225.84, 1851783.833, 41.74830012, -87.57485274, 46, 'SOUTH CHICAGO', 8, 4, '(41.74830012, -87.57485274)') ,
(610058, 'Horatio May Elementary Community Academy', 'ES', '512 S Lavergne Ave', 'Chicago', 'IL', 60644, '(773) 534-6140', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610058.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, NULL, NULL, 'Average', 47.0, 'Average', 56.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '91.90%', 40.0, '96.40%', '98.20%', 63.3, 58.8, 17.5, 17.5, 54.0, 51.4, 27.1, 25.0, 58.6, 58.6, 4.6, 13.8, 9.4, 5.6, -0.4, -0.8, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 458, 36, NULL, 1143308.514, 1897064.832, 41.87358321, -87.74931216, 25, 'AUSTIN', 29, 15, '(41.87358321, -87.74931216)') ,
(609713, 'Hyde Park Academy High School', 'HS', '6220 S Stony Island Ave', 'Chicago', 'IL', 60637, '(773) 535-0880', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609713.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 29.0, 'Weak', 39.0, 'Weak', 34.0, 'Average', 43.0, 'Weak', 35.0, 'Average', 41.0, NULL, NULL, NULL, NULL, '70.50%', 18.5, '94.20%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.0, 12.8, 14.4, 14.1, 1.1, 15.6, 1.2, 11.1, 51.1, 57.6, 1518, 46, 63.7, 1187898.897, 1864121.427, 41.78223539, -87.58665115, 42, 'WOODLAWN', 5, 3, '(41.78223539, -87.58665115)') ,
(610390, 'Hyman G Rickover Naval Academy High School', 'HS', '5900 N Glenwood Ave', 'Chicago', 'IL', 60660, '(773) 534-2890', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610390.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 64.0, NULL, NULL, 'Strong', 64.0, 'Average', 42.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Average', 48.0, '92.90%', 15.4, '95.90%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 15.2, 15.4, 16.5, 15.8, 0.6, 18.7, 2.2, 23.4, 63.0, 63.5, 423, 32, 90.7, 1165871.81, 1939308.369, 41.98905063, -87.66526222, 77, 'EDGEWATER', 48, 20, '(41.98905063, -87.66526222)') ,
(610110, 'Ida B Wells Preparatory Elementary Academy', 'ES', '244 E Pershing Rd', 'Chicago', 'IL', 60653, '(773) 535-1204', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610110.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 49.0, NULL, NULL, 'Strong', 64.0, 'Very Strong', 93.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '93.20%', 9.5, '92.80%', '100.00%', 80.9, NULL, 24.7, 26.3, 41.3, 60.5, 49.3, 42.3, 58.8, 57.4, 8.3, 37.5, 15.3, 10.1, 0.3, -0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 215, 40, NULL, 1178735.106, 1879229.78, 41.82390751, -87.61978794, 35, 'DOUGLAS', 3, 2, '(41.82390751, -87.61978794)') ,
(610273, 'Ignance Paderewski Elementary Learning Academy', 'ES', '2221 S Lawndale Ave', 'Chicago', 'IL', 60623, '(773) 534-1821', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610273.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 31.0, NULL, NULL, 'Strong', 63.0, 'Very Strong', 98.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Average', 47.0, '92.10%', 20.9, '95.20%', '100.00%', 42.3, 40.9, 19.1, 19.1, 50.0, 64.3, 27.5, 17.0, 44.2, 58.7, 0.0, 0.0, 2.0, 2.0, 0.2, -0.8, 'Yellow', 'Yellow', 93.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 130, 37, NULL, 1152095.48, 1888877.695, 41.85094816, -87.71726618, 30, 'SOUTH LAWNDALE', 22, 10, '(41.85094816, -87.71726618)') ,
(610384, 'Infinity Math Science and Technology High School', 'HS', '3120 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 535-4225', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610384.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, NULL, NULL, 'Strong', 66.0, 'Strong', 65.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Average', 48.0, '92.20%', 7.5, '96.00%', '85.40%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.1, 15.0, 14.8, 15.1, 1.0, 17.2, 2.4, 29.0, 84.4, 55.7, 388, 37, 98.1, 1147521.3020000001, 1883405.128, 41.83601953, -87.73419465, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83601953, -87.73419465)') ,
(610078, 'Inter-American Elementary Magnet School', 'ES', '851 W Waveland Ave', 'Chicago', 'IL', 60613, '(773) 534-5490', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610078.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, NULL, NULL, 'Average', 50.0, 'Average', 51.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Average', 47.0, '96.20%', 3.0, '96.20%', '100.00%', 68.2, 31.2, 46.6, 38.7, 51.3, 49.5, 52.3, 50.0, 54.8, 54.1, 34.5, 29.6, 27.6, 21.3, 1.1, 1.9, 'Green', 'Green', 32.1, 83.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 690, 33, NULL, 1169719.256, 1924734.562, 41.94897646, -87.65153745, 6, 'LAKE VIEW', 44, 19, '(41.94897646, -87.65153745)') ,
(609848, 'Ira F Aldridge Elementary School', 'ES', '630 E 131st St', 'Chicago', 'IL', 60827, '(773) 535-5614', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609848.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 39.0, NULL, NULL, 'Weak', 36.0, 'Weak', 39.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Weak', 46.0, '92.90%', 49.0, '96.30%', '100.00%', 62.7, 60.9, 20.7, 20.2, 46.7, 40.9, 21.2, 29.8, 37.4, 60.2, 2.5, 20.0, 7.7, 7.2, -1.5, -0.8, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 285, 48, NULL, 1182870.294, 1818598.178, 41.65743243, -87.60649623, 54, 'RIVERDALE', 9, 5, '(41.65743243, -87.60649623)') ,
(610532, 'Irene C. Hernandez Middle School for the Advancement of Science', 'MS', '3510 W 55th St', 'Chicago', 'IL', 60632, '(773) 535-8850', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610532.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Not Enough Data', 'No', 'Weak', 34.0, 'Weak', 34.0, 'Weak', 27.0, 'Weak', 27.0, 'Weak', 28.0, 'Weak', 27.0, NULL, NULL, NULL, NULL, '95.50%', 14.8, '96.20%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 33.7, 28.9, 34.2, 46.6, 12.5, 18.0, 11.6, 9.2, -1.8, -0.8, 'Red', 'Red', 18.2, 40.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 970, 45, NULL, 1153845.758, 1867957.685, 41.79350627, -87.71139803, 63, 'GAGE PARK', 14, 8, '(41.79350627, -87.71139803)') ,
(610125, 'Irma C Ruiz Elementary School', 'ES', '2410 S Leavitt St', 'Chicago', 'IL', 60608, '(773) 535-4825', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610125.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'Yes', 'Average', 57.0, 'Average', 59.0, 'Average', 57.0, 'Strong', 62.0, 'Weak', 51.0, 'Average', 56.0, 'Average', 50.0, 'Average', 51.0, '95.50%', 15.9, '95.00%', '95.20%', 55.8, 40.7, 19.4, 26.4, 44.0, 41.9, 38.9, 33.6, 57.5, 48.5, 10.7, 27.2, 10.7, 12.8, -0.4, 0.4, 'Red', 'Yellow', 28.8, 16.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 916, 39, NULL, 1162043.649, 1887912.321, 41.84809742, -87.68078114, 31, 'LOWER WEST SIDE', 25, 10, '(41.84809742, -87.68078114)') ,
(610276, 'Irvin C Mollison Elementary School', 'ES', '4415 S Dr Martin L King Jr Dr', 'Chicago', 'IL', 60653, '(773) 535-1804', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610276.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 44.0, NULL, NULL, 'Average', 46.0, 'Weak', 38.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Average', 50.0, '91.10%', 36.1, '94.30%', '94.40%', 49.3, 36.8, 27.4, 19.8, 48.2, 41.9, 34.4, 26.4, 57.9, 41.9, 11.1, 25.9, 4.8, 4.3, 0.2, 0.2, 'Yellow', 'Yellow', 20.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 244, 40, NULL, 1179638.34, 1875750.718, 41.81434006, -87.61658079, 38, 'GRAND BOULEVARD', 3, 2, '(41.81434006, -87.61658079)') ,
(610103, 'Isabelle C O Keeffe Elementary School', 'ES', '6940 S Merrill Ave', 'Chicago', 'IL', 60649, '(773) 535-0600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610103.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 17.0, NULL, NULL, 'Weak', 28.0, 'Weak', 29.0, 'Weak', NULL, NULL, NULL, 'Weak', 40.0, 'Weak', 43.0, '92.10%', 24.6, '94.60%', '100.00%', 52.0, 54.0, 13.8, 16.4, 42.8, 29.1, 17.9, 28.6, 43.3, 60.0, 1.7, 19.0, 3.4, 6.7, -0.4, 0.7, 'Yellow', 'Yellow', 17.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 575, 46, NULL, 1191708.826, 1859191.4640000002, 41.76861557, -87.57284299, 43, 'SOUTH SHORE', 5, 3, '(41.76861557, -87.57284299)') ,
(610256, 'Jackie Robinson Elementary School', 'ES', '4225 S Lake Park Ave', 'Chicago', 'IL', 60653, '(773) 535-1777', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610256.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Average', 47.0, '90.50%', 47.9, '93.80%', '100.00%', 52.0, 30.6, 10.6, 21.3, 39.1, 26.1, NULL, NULL, NULL, NULL, NULL, NULL, 6.5, 6.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 140, 40, NULL, 1184412.88, 1876890.9740000002, 41.81735839, -87.59903174, 36, 'OAKLAND', 4, 2, '(41.81735839, -87.59903174)') ,
(609797, 'Jacob Beidler Elementary School', 'ES', '3151 W Walnut St', 'Chicago', 'IL', 60612, '(773) 534-6811', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609797.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 63.0, NULL, NULL, 'Strong', 70.0, 'Average', 58.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.40%', 0.7, '96.60%', '100.00%', 70.7, 54.1, 30.0, 17.3, 55.4, 70.7, 24.8, 24.3, 50.7, 56.2, 3.8, 7.5, 14.3, 5.1, 0.5, 0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 411, 34, NULL, 1155122.862, 1901415.832, 41.88529401, -87.70581859, 27, 'EAST GARFIELD PARK', 27, 13, '(41.88529401, -87.70581859)') ,
(609766, 'Jacqueline B Vaughn Occupational High School', 'HS', '4355 N Linder Ave', 'Chicago', 'IL', 60641, '(773) 534-3600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609766.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', NULL, 'Track_E', 'Not Applicable', 'Not Enough Data', 'No', 'Average', 57.0, 'Strong', 72.0, 'Weak', 33.0, 'Weak', 20.0, 'Weak', 67.0, 'Strong', 69.0, 'Strong', 63.0, 'Strong', 57.0, '87.60%', 3.7, '95.50%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 55.6, 65.3, NULL, 203, 30, NULL, 1138928.091, 1928655.205, 41.96035141, -87.76462602, 15, 'PORTAGE PARK', 38, 16, '(41.96035141, -87.76462602)') ,
(609912, 'James B Farnsworth Elementary School', 'ES', '5414 N Linder Ave', 'Chicago', 'IL', 60630, '(773) 534-3535', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609912.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, 'Strong', 62.0, 'Average', 40.0, 'Weak', 38.0, 'Weak', 63.0, 'Average', 59.0, 'Strong', 55.0, 'Average', 50.0, '94.20%', 1.7, '95.60%', '100.00%', 68.4, 33.1, 50.3, 60.8, 62.7, 55.0, 52.1, 57.6, 59.2, 60.4, 30.2, 41.8, 30.7, 25.6, 1.0, 0.3, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 574, 30, NULL, 1138478.05, 1935743.025, 41.9798092, -87.76610809, 11, 'JEFFERSON PARK', 45, 16, '(41.9798092, -87.76610809)') ,
(610070, 'James B McPherson Elementary School', 'ES', '4728 N Wolcott Ave', 'Chicago', 'IL', 60640, '(773) 534-2625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610070.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 71.0, NULL, NULL, 'Average', 48.0, 'Weak', 38.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Strong', 54.0, '96.50%', 1.2, '96.30%', '100.00%', 69.7, 41.9, 36.7, 39.7, 47.4, 51.7, 30.4, 42.0, 34.1, 43.4, 10.2, 37.3, 15.4, 15.9, -0.8, -0.6, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 695, 31, NULL, 1162923.519, 1931558.605, 41.96784758, -87.67632505, 4, 'LINCOLN SQUARE', 47, 19, '(41.96784758, -87.67632505)') ,
(610066, 'James E McDade Elementary Classical School', 'ES', '8801 S Indiana Ave', 'Chicago', 'IL', 60619, '(773) 535-3669', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610066.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, NULL, NULL, 'Average', 57.0, 'Average', 52.0, 'Weak', NULL, NULL, NULL, 'Strong', 61.0, 'Average', 52.0, '96.20%', 0.0, '94.30%', '100.00%', NULL, NULL, 87.7, 82.2, 50.7, 48.1, NULL, 90.0, NULL, 45.0, NULL, NULL, 73.3, 74.1, -1.1, -0.7, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 199, 45, NULL, 1179172.217, 1846656.745, 41.73451387, -87.61917677, 44, 'CHATHAM', 6, 6, '(41.73451387, -87.61917677)') ,
(609803, 'James G Blaine Elementary School', 'ES', '1420 W Grace St', 'Chicago', 'IL', 60613, '(773) 534-5750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609803.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, NULL, NULL, 'Strong', 76.0, 'Strong', 74.0, 'Weak', NULL, NULL, NULL, 'Weak', 40.0, 'Weak', 46.0, '96.40%', 2.1, '96.00%', '88.40%', 87.5, 62.6, 88.6, 78.5, 61.7, 62.8, 81.9, 79.9, 56.3, 63.6, 53.6, 73.2, 61.3, 50.1, 0.9, -0.4, 'Green', 'Yellow', 50.0, 63.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 923, 33, NULL, 1166063.1670000001, 1925373.2680000002, 41.95080812, -87.66495825, 6, 'LAKE VIEW', 44, 19, '(41.95080812, -87.66495825)') ,
(609983, 'James Hedges Elementary School', 'ES', '4747 S Winchester Ave', 'Chicago', 'IL', 60609, '(773) 535-7360', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609983.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 22.0, NULL, NULL, 'Weak', 22.0, 'Weak', 24.0, 'Weak', NULL, NULL, NULL, 'Weak', 41.0, 'Weak', 45.0, '95.40%', 48.8, '96.50%', '100.00%', 66.3, 41.1, 34.0, 19.6, 46.3, 50.4, 25.1, 25.2, 57.0, 55.1, 13.2, 17.1, 12.6, 6.8, -0.1, 0.5, 'Yellow', 'Yellow', 16.5, 61.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 845, 42, NULL, 1164199.04, 1873001.0469999998, 41.80713406, -87.67329121, 61, 'NEW CITY', 20, 9, '(41.80713406, -87.67329121)') ,
(610047, 'James Madison Elementary School', 'ES', '7433 S Dorchester Ave', 'Chicago', 'IL', 60619, '(773) 535-0551', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610047.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 39.0, NULL, NULL, 'Weak', 32.0, 'Weak', 28.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.90%', 12.4, '97.70%', '100.00%', 60.0, 60.7, 21.8, 24.3, 52.0, 49.0, 33.3, 19.3, 48.8, 38.6, 3.1, 28.1, 14.1, 5.1, 0.7, -0.9, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 317, 46, NULL, 1186872.582, 1855846.3469999998, 41.75955224, -87.59067582, 43, 'SOUTH SHORE', 5, 3, '(41.75955224, -87.59067582)') ,
(610074, 'James Monroe Elementary School', 'ES', '3651 W Schubert Ave', 'Chicago', 'IL', 60647, '(773) 534-4155', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610074.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 45.0, NULL, NULL, 'Weak', 26.0, 'Weak', 22.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 47.0, '95.10%', 22.1, '95.80%', '98.10%', 66.5, 25.8, 37.9, 27.0, 46.5, 55.1, 31.6, 32.2, 50.3, 45.3, 12.5, 24.0, 15.5, 11.6, -0.6, -0.4, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1125, 29, NULL, 1151359.1509999998, 1917666.2340000002, 41.92996135, -87.71921246, 22, 'LOGAN SQUARE', 35, 25, '(41.92996135, -87.71921246)') ,
(610200, 'James N Thorp Elementary School', 'ES', '8914 S Buffalo Ave', 'Chicago', 'IL', 60617, '(773) 535-6250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610200.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 32.0, NULL, NULL, 'Weak', 39.0, 'Weak', 39.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '91.80%', 45.4, '95.90%', '100.00%', 51.0, 24.3, 9.1, 10.4, 33.6, 31.0, 27.3, 30.5, 52.9, 43.8, 3.6, 20.0, 8.7, 3.6, -0.4, -1.2, 'Yellow', 'Red', 14.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 449, 47, NULL, 1199554.7, 1846396.13, 41.73331035, -87.54451462, 46, 'SOUTH CHICAGO', 10, 4, '(41.73331035, -87.54451462)') ,
(610107, 'James Otis Elementary School', 'ES', '525 N Armour St', 'Chicago', 'IL', 60622, '(773) 534-7665', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610107.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 1', 'No', 'Very Strong', 83.0, 'Very Strong', 92.0, 'Strong', 67.0, 'Average', 58.0, 'Weak', 92.0, 'Very Strong', 94.0, 'Strong', 54.0, 'Strong', 54.0, '95.50%', 3.2, '96.50%', '100.00%', 50.9, 47.1, 35.9, 46.6, 53.4, 63.1, 51.4, 55.0, 61.3, 57.7, 25.8, 19.4, 21.8, 11.1, 0.7, -0.2, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 491, 35, NULL, 1166113.4109999998, 1903868.626, 41.89179703, -87.66538928, 24, 'WEST TOWN', 27, 13, '(41.89179703, -87.66538928)') ,
(609891, 'James R Doolittle Jr Elementary School', 'ES', '535 E 35th St', 'Chicago', 'IL', 60616, '(773) 535-1040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609891.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 30.0, NULL, NULL, 'Weak', 25.0, 'Average', 46.0, 'Weak', NULL, NULL, NULL, 'Weak', 44.0, 'Average', 47.0, '91.40%', 134.8, '94.70%', '97.70%', 55.6, 50.0, 15.9, 3.7, 40.6, 32.4, 7.4, 24.5, 39.8, 54.9, 11.8, 35.3, 2.2, 1.1, -1.4, -0.2, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 306, 40, NULL, 1180586.11, 1881863.446, 41.831092100000006, -87.61291629, 35, 'DOUGLAS', 4, 2, '(41.8310921, -87.61291629)') ,
(610044, 'James Russell Lowell Elementary School', 'ES', '3320 W Hirsch St', 'Chicago', 'IL', 60651, '(773) 534-4300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610044.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 39.0, NULL, NULL, 'Weak', 39.0, 'Average', 40.0, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Average', 52.0, '93.20%', 11.2, '95.70%', '100.00%', 52.7, 40.9, 20.6, 18.5, 39.5, 50.3, 42.3, 28.4, 66.4, 52.2, 8.0, 26.0, 10.2, 8.2, 0.2, 0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 593, 34, NULL, 1153864.623, 1909151.595, 41.90654684, -87.71023279, 23, 'HUMBOLDT PARK', 26, 14, '(41.90654684, -87.71023279)') ,
(610174, 'James Shields Elementary School', 'ES', '4250 S Rockwell St', 'Chicago', 'IL', 60632, '(773) 535-7285', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610174.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 38.0, NULL, NULL, 'Average', 57.0, 'Average', 58.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 50.0, '97.00%', 13.9, '97.00%', '99.60%', 54.3, NULL, 37.2, 35.1, 59.8, 70.8, 41.9, 44.9, 63.2, 66.8, 23.3, 25.6, 18.3, 12.7, 1.1, 1.5, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1834, 39, NULL, 1159705.642, 1876174.2680000002, 41.81593528, -87.68968453, 58, 'BRIGHTON PARK', 12, 9, '(41.81593528, -87.68968453)') ,
(610213, 'James Wadsworth Elementary School', 'ES', '6420 S University Ave', 'Chicago', 'IL', 60637, '(773) 535-0730', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610213.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 49.0, '92.20%', 8.0, '95.50%', '100.00%', 78.0, 45.1, 13.4, 22.3, 60.6, 35.6, NULL, NULL, NULL, NULL, NULL, NULL, 5.4, 5.4, -1.3, -0.6, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 277, 46, NULL, 1184773.873, 1862597.144, 41.77812652, -87.59815597, 42, 'WOODLAWN', 20, 3, '(41.77812652, -87.59815597)') ,
(610217, 'James Ward Elementary School', 'ES', '2701 S Shields Ave', 'Chicago', 'IL', 60616, '(773) 534-9050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610217.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 49.0, 'Strong', 65.0, 'Average', 50.0, 'Average', 46.0, 'Weak', 63.0, 'Average', 56.0, 'Average', 48.0, 'Weak', 43.0, '97.80%', 1.3, '96.90%', '100.00%', 74.8, 63.8, 71.6, 57.0, 68.1, 80.6, 77.8, 53.3, 79.3, 51.9, 44.2, 34.6, 51.1, 23.8, -0.1, -1.0, 'Yellow', 'Red', 43.6, 50.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 492, 40, NULL, 1174442.789, 1886499.005, 41.84395162, -87.63531815, 34, 'ARMOUR SQUARE', 11, 9, '(41.84395162, -87.63531815)') ,
(610274, 'James Weldon Johnson Elementary School', 'ES', '1420 S Albany Ave', 'Chicago', 'IL', 60623, '(773) 534-1829', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610274.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Strong', 67.0, 'Weak', 38.0, 'Strong', 61.0, 'Strong', 75.0, 'Weak', 63.0, 'Strong', 66.0, 'Average', 51.0, 'Strong', 54.0, '94.70%', 1.4, '97.50%', '100.00%', 64.3, NULL, 21.3, 19.8, 54.3, 48.8, 17.3, 28.0, 59.3, 50.0, 3.8, 30.8, 8.9, 5.0, -0.4, -1.0, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 365, 37, NULL, 1155905.3020000001, 1892874.4430000002, 41.86183983, -87.7031756, 29, 'NORTH LAWNDALE', 24, 10, '(41.86183983, -87.7031756)') ,
(609772, 'Jane Addams Elementary School', 'ES', '10810 S Avenue H', 'Chicago', 'IL', 60617, '(773) 535-6210', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609772.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 56.0, 'Average', 50.0, 'Average', 44.0, 'Average', 46.0, 'Weak', 47.0, 'Average', 44.0, 'Average', 53.0, 'Average', 53.0, '96.00%', 5.0, '96.80%', '100.00%', 71.4, NULL, 32.5, 40.6, 60.7, 63.6, 53.9, 57.7, 77.4, 66.8, 19.5, 36.8, 19.3, 15.9, 0.5, -0.4, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 928, 47, NULL, 1202811.36, 1833818.906, 41.69871497, -87.53301251, 52, 'EAST SIDE', 10, 4, '(41.69871497, -87.53301251)') ,
(610093, 'Jane A Neil Elementary School', 'ES', '8555 S Michigan Ave', 'Chicago', 'IL', 60619, '(773) 535-3000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610093.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 42.0, 'Average', 41.0, 'Weak', 27.0, 'Weak', 28.0, 'Weak', 40.0, 'Weak', 36.0, 'Average', 52.0, 'Average', 53.0, '88.90%', 5.0, '94.60%', '100.00%', NULL, 47.4, 25.0, 32.3, 51.7, 60.4, 38.1, 41.7, 62.7, 69.9, 3.4, 13.8, 11.4, 7.1, -0.7, 1.0, 'Red', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 306, 45, NULL, 1178690.672, 1848082.986, 41.7384386, -87.62089767, 44, 'CHATHAM', 6, 6, '(41.7384386, -87.62089767)') ,
(609796, 'Jean Baptiste Beaubien Elementary School', 'ES', '5025 N Laramie Ave', 'Chicago', 'IL', 60630, '(773) 534-3500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609796.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 63.0, NULL, NULL, 'Average', 49.0, 'Average', 47.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.70%', 7.4, '96.30%', '98.40%', 81.6, NULL, 66.8, 68.9, 67.4, 69.1, 77.0, 73.4, 70.6, 60.1, 45.1, 56.6, 49.1, 41.8, 0.2, 0.2, 'Yellow', 'Yellow', 36.8, 72.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1181, 31, NULL, 1140829.242, 1933051.044, 41.97237914, -87.75752776, 11, 'JEFFERSON PARK', 45, 16, '(41.97237914, -87.75752776)') ,
(610031, 'Jean D Lafayette Elementary School', 'ES', '2714 W Augusta Blvd', 'Chicago', 'IL', 60622, '(773) 534-4326', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610031.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 44.0, 'Average', 53.0, 'Average', 43.0, 'Weak', 38.0, 'Weak', 80.0, 'Strong', 72.0, 'Average', 52.0, 'Average', 52.0, '94.10%', 14.3, '96.30%', '98.60%', 54.6, 12.0, 25.3, 23.3, 44.9, 45.6, 26.0, 22.7, 48.3, 46.7, 11.3, 26.4, 8.3, 6.2, -0.3, -1.3, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 474, 34, NULL, 1158045.392, 1906579.3480000002, 41.89940402, -87.69494543, 24, 'WEST TOWN', 1, 13, '(41.89940402, -87.69494543)') ,
(610271, 'Jensen Elementary Scholastic Academy', 'ES', '3030 W Harrison St', 'Chicago', 'IL', 60612, '(773) 534-6840', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610271.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 35.0, 'Average', 41.0, 'Average', 42.0, 'Strong', 67.0, 'Weak', 43.0, 'Average', 40.0, 'Weak', 46.0, 'Average', 47.0, '93.10%', 17.2, '95.30%', '100.00%', 81.3, 75.5, 61.4, 37.3, 58.9, 78.6, 48.8, 37.3, 63.4, 63.2, 11.6, 23.3, 20.2, 11.7, 2.9, 0.9, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 364, 37, NULL, 1156127.601, 1897250.02, 41.87384239, -87.70224149, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87384239, -87.70224149)') ,
(609932, 'Jesse Owens Elementary Community Academy', 'ES', '12450 S State St', 'Chicago', 'IL', 60628, '(773) 535-5661', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609932.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Strong', 55.0, '91.30%', 4.9, '93.50%', '100.00%', 57.1, 37.9, 11.9, 28.4, 42.2, 42.2, NULL, NULL, NULL, NULL, NULL, NULL, 50.0, 13.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 340, 48, NULL, 1178488.2140000002, 1822184.0590000001, 41.66737294, -87.62242272, 53, 'WEST PULLMAN', 9, 5, '(41.66737294, -87.62242272)') ,
(610173, 'Jesse Sherwood Elementary School', 'ES', '245 W 57th St', 'Chicago', 'IL', 60621, '(773) 535-0829', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610173.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 38.0, NULL, NULL, 'Average', 47.0, 'Average', 58.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Strong', 56.0, '92.00%', 3.9, '95.50%', '100.00%', 50.9, 41.5, 17.6, 17.1, 51.5, 47.5, 55.2, 40.0, 79.6, 65.2, NULL, NULL, 25.7, 12.9, 3.6, 3.2, 'Green', 'Green', 43.1, 71.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 355, 42, NULL, 1175428.3909999998, 1867127.635, 41.79077279999999, -87.63228120000001, 68, 'ENGLEWOOD', 20, 7, '(41.7907728, -87.6322812)') ,
(609942, 'Johann W von Goethe Elementary School', 'ES', '2236 N Rockwell St', 'Chicago', 'IL', 60647, '(773) 534-4135', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609942.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 75.0, 'Strong', 61.0, 'Strong', 68.0, 'Average', 52.0, 'Weak', 62.0, 'Strong', 69.0, 'Strong', 57.0, 'Average', 53.0, '95.80%', 5.6, '97.00%', '99.10%', 72.7, 55.1, 40.6, 42.4, 68.6, 80.2, 57.6, 52.7, 71.2, 59.6, 21.4, 31.4, 24.9, 17.6, -0.8, -0.7, 'Red', 'Red', 20.5, 29.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 786, 35, NULL, 1158528.612, 1915119.2130000002, 41.9228282, -87.69293637, 22, 'LOGAN SQUARE', 1, 14, '(41.9228282, -87.69293637)') ,
(610216, 'John A Walsh Elementary School', 'ES', '2015 S Peoria St', 'Chicago', 'IL', 60608, '(773) 534-7950', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610216.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', 'Strong', 60.0, NULL, NULL, 'Strong', 70.0, 'Very Strong', 91.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 53.0, '95.10%', 2.9, '95.30%', '97.00%', 64.4, NULL, 37.6, 37.0, 49.6, 61.5, 47.1, 44.5, 60.4, 63.9, 12.1, 24.2, 17.4, 12.5, 0.2, 0.3, 'Yellow', 'Yellow', 40.0, 42.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 501, 39, NULL, 1170942.07, 1890439.543, 41.85484214, -87.64804964, 31, 'LOWER WEST SIDE', 25, 12, '(41.85484214, -87.64804964)') ,
(609789, 'John Barry Elementary School', 'ES', '2828 N Kilbourn Ave', 'Chicago', 'IL', 60641, '(773) 534-3455', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609789.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, 'Average', 49.0, 'Strong', 70.0, 'Very Strong', 81.0, 'Weak', 34.0, 'Average', 41.0, 'Average', 51.0, 'Average', 49.0, '95.10%', 5.6, '96.10%', '97.70%', 76.4, 55.0, 41.2, 28.2, 44.9, 49.2, 63.6, 32.5, 81.2, 64.8, NULL, NULL, 15.2, 12.4, 0.7, 0.5, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 800, 29, NULL, 1145852.597, 1918556.6519999998, 41.93251118, -87.73942514, 20, 'HERMOSA', 31, 25, '(41.93251118, -87.73942514)') ,
(609894, 'John B Drake Elementary School', 'ES', '2722 S King Dr', 'Chicago', 'IL', 60616, '(773) 534-9129', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609894.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 59.0, 'Weak', 20.0, 'Average', 57.0, 'Average', 46.0, 'Weak', 43.0, 'Very Weak', 16.0, 'Average', 50.0, 'Average', 49.0, '93.80%', 43.3, '95.90%', '100.00%', 63.0, NULL, 15.7, 29.8, 67.1, 60.0, 41.8, 50.0, 83.1, 78.5, 7.7, 26.9, 7.7, 9.5, -0.2, 0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 249, 40, NULL, 1179225.388, 1886481.0890000002, 41.84379445, -87.61776768, 35, 'DOUGLAS', 4, 1, '(41.84379445, -87.61776768)') ,
(610089, 'John B Murphy Elementary School', 'ES', '3539 W Grace St', 'Chicago', 'IL', 60618, '(773) 534-5223', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610089.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 80.0, NULL, NULL, 'Strong', 75.0, 'Average', 59.0, 'Weak', NULL, NULL, NULL, 'Strong', 58.0, 'Strong', 54.0, '95.60%', 1.9, '96.30%', '100.00%', 61.3, NULL, 43.4, 30.5, 40.6, 45.9, 64.3, 44.8, 46.7, 40.7, NULL, NULL, 22.0, 20.7, -0.5, -0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 638, 31, NULL, 1151954.94, 1924991.364, 41.95005034, -87.71682949, 16, 'IRVING PARK', 35, 17, '(41.95005034, -87.71682949)') ,
(610243, 'John Calhoun North Elementary School', 'ES', '2833 W Adams St', 'Chicago', 'IL', 60612, '(773) 534-6940', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610243.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 52.0, 'Average', 51.0, 'Average', 44.0, 'Average', 54.0, 'Weak', 79.0, 'Very Strong', 88.0, 'Weak', 43.0, 'Average', 48.0, '95.90%', 4.6, '96.50%', '98.10%', 71.3, 48.8, 37.4, 28.3, 52.2, 61.5, 50.5, 37.9, 62.5, 69.3, 12.5, 25.0, 32.8, 14.4, 3.5, 3.5, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 328, 38, NULL, 1157392.716, 1898855.0559999999, 41.87822115, -87.69755292, 27, 'EAST GARFIELD PARK', 2, 11, '(41.87822115, -87.69755292)') ,
(609829, 'John C Burroughs Elementary School', 'ES', '3542 S Washtenaw Ave', 'Chicago', 'IL', 60632, '(773) 535-7226', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609829.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', 'Average', 50.0, 'Average', 55.0, 'Weak', 39.0, 'Weak', 38.0, 'Weak', 71.0, 'Average', 53.0, 'Average', 50.0, 'Weak', 45.0, '95.60%', 0.2, '96.10%', '100.00%', 67.3, 27.9, 47.7, 43.4, 55.9, 45.1, 44.2, 57.4, 51.9, 64.3, 25.0, 15.9, 25.9, 16.0, 0.1, -0.1, 'Yellow', 'Yellow', 42.2, 21.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 510, 39, NULL, 1158904.877, 1880879.379, 41.8288631, -87.69249318, 58, 'BRIGHTON PARK', 12, 9, '(41.8288631, -87.69249318)') ,
(609866, 'John C Coonley Elementary School', 'ES', '4046 N Leavitt St', 'Chicago', 'IL', 60618, '(773) 534-5140', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609866.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, 'Very Strong', 90.0, 'Strong', 65.0, 'Weak', 39.0, 'Weak', 65.0, 'Average', 56.0, 'Strong', 58.0, 'Strong', 55.0, '95.90%', 1.4, '96.50%', '100.00%', 90.1, 52.9, 72.6, 66.7, 61.4, 65.8, 64.8, 64.8, 53.8, 63.5, 27.8, 50.0, 45.3, 39.0, -0.7, 0.9, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 668, 31, NULL, 1160924.137, 1927061.47, 41.95554905, -87.68380195, 5, 'NORTH CENTER', 47, 19, '(41.95554905, -87.68380195)') ,
(609893, 'John C Dore Elementary School', 'ES', '6108 S Natoma Ave', 'Chicago', 'IL', 60638, '(773) 535-2080', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609893.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 74.0, 'Strong', 67.0, 'Strong', 69.0, 'Strong', 79.0, 'Weak', 57.0, 'Average', 51.0, 'Strong', 58.0, 'Weak', 46.0, '95.60%', 10.8, '96.70%', '96.20%', 81.8, 52.9, 58.4, 62.7, 70.5, 68.1, 76.9, 74.0, 85.0, 69.4, 58.0, 60.0, 43.8, 25.3, 2.6, 0.5, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 590, 44, NULL, 1133084.3, 1863232.77, 41.78092716, -87.78764009, 64, 'CLEARING', 23, 8, '(41.78092716, -87.78764009)') ,
(609959, 'John Charles Haines Elementary School', 'ES', '247 W 23rd Pl', 'Chicago', 'IL', 60616, '(773) 534-9200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609959.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Weak', 32.0, 'Average', 47.0, 'Weak', 37.0, 'Weak', 37.0, 'Weak', 42.0, 'Average', 48.0, 'Weak', 43.0, 'Weak', 43.0, '98.40%', 0.3, '96.20%', '96.90%', 70.2, 47.6, 69.3, 57.6, 69.0, 69.8, 73.2, 53.6, 74.3, 63.6, 35.9, 32.0, 45.0, 25.7, 1.0, 0.5, 'Green', 'Yellow', 25.9, 93.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 664, 40, NULL, 1174842.529, 1888681.3280000002, 41.84993116, -87.63378596, 34, 'ARMOUR SQUARE', 25, 9, '(41.84993116, -87.63378596)') ,
(610176, 'John D Shoop Math-Science Technical Academy Elementary School', 'ES', '11140 S Bishop St', 'Chicago', 'IL', 60643, '(773) 535-2715', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610176.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 32.0, 'Average', 47.0, 'Average', 43.0, 'Average', 51.0, 'Weak', 53.0, 'Average', 42.0, 'Average', 52.0, 'Strong', 54.0, '93.20%', 40.8, '96.30%', '86.90%', 75.9, 67.7, 34.5, 27.9, 50.0, 52.2, 44.2, 34.2, 50.6, 48.3, 13.8, 13.6, 12.9, 8.3, 0.6, -0.2, 'Green', 'Yellow', 4.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 553, 49, NULL, 1168507.416, 1830689.468, 41.69093337, -87.65870614, 75, 'MORGAN PARK', 34, 22, '(41.69093337, -87.65870614)') ,
(609898, 'John F Eberhart Elementary School', 'ES', '3400 W 65th Pl', 'Chicago', 'IL', 60629, '(773) 535-9190', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609898.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 42.0, 'Average', 41.0, 'Weak', 33.0, 'Weak', 34.0, 'Weak', 42.0, 'Weak', 32.0, NULL, NULL, NULL, NULL, '95.30%', 13.1, '95.60%', '100.00%', 48.2, 27.3, 28.2, 31.4, 52.4, 49.5, 32.8, 39.9, 50.4, 51.3, 10.2, 20.9, 8.9, 9.6, -1.5, -0.7, 'Red', 'Red', 15.9, 17.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1777, 44, NULL, 1154822.365, 1861019.3830000001, 41.7744471, -87.70800189, 66, 'CHICAGO LAWN', 15, 8, '(41.7744471, -87.70800189)') ,
(609919, 'John Fiske Elementary School', 'ES', '6145 S Ingleside Ave', 'Chicago', 'IL', 60637, '(773) 535-0990', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609919.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 13.0, 'Weak', 24.0, 'Weak', 37.0, 'Average', 55.0, 'Weak', 40.0, 'Weak', 35.0, 'Average', 47.0, 'Average', 50.0, '92.90%', 37.1, '91.70%', '91.40%', 87.1, NULL, 15.7, 14.6, 58.0, 54.2, 30.1, 30.1, 69.7, 61.8, 32.1, 28.6, 4.0, 4.0, -1.3, -1.5, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 254, 46, NULL, 1183547.778, 1864329.648, 41.78290934, -87.60259685, 42, 'WOODLAWN', 20, 3, '(41.78290934, -87.60259685)') ,
(609718, 'John F Kennedy High School', 'HS', '6325 W 56th St', 'Chicago', 'IL', 60638, '(773) 535-2325', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609718.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 53.0, 'Weak', 30.0, 'Weak', 32.0, 'Weak', 27.0, 'Weak', 22.0, 'Weak', 24.0, NULL, NULL, NULL, NULL, '83.00%', 22.0, '94.70%', '99.30%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.8, 14.1, 15.1, 15.6, 0.8, 16.4, 1.3, 24.4, 53.3, 52.9, 1589, 44, 59.7, 1135044.301, 1866707.447, 41.79042796, -87.78037224, 56, 'GARFIELD RIDGE', 23, 8, '(41.79042796, -87.78037224)') ,
(610263, 'John Foster Dulles Elementary School', 'ES', '6311 S Calumet Ave', 'Chicago', 'IL', 60637, '(773) 535-0690', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610263.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 42.0, 'Weak', 22.0, 'Average', 53.0, 'Average', 42.0, 'Weak', 32.0, 'Weak', 21.0, 'Average', 51.0, 'Strong', 55.0, '94.10%', 28.2, '97.30%', '100.00%', 91.7, NULL, 16.9, 19.9, 42.3, 47.4, 15.5, 13.1, 61.9, 48.8, 2.5, 5.1, 9.1, 3.8, 1.4, 0.1, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 537, 42, NULL, 1179581.237, 1863162.082, 41.779796999999995, -87.61717494, 69, 'GREATER GRAND CROSSING', 20, 3, '(41.779797, -87.61717494)') ,
(610228, 'John Greenleaf Whittier Elementary School', 'ES', '1900 W 23rd St', 'Chicago', 'IL', 60608, '(773) 535-4590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610228.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 65.0, NULL, NULL, 'Average', 45.0, 'Weak', 24.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.90%', 3.8, '96.50%', '100.00%', 50.4, NULL, 13.3, 17.3, 35.9, 36.2, 22.3, 21.5, 50.0, 45.8, 6.4, 25.5, 6.3, 3.9, -0.9, -0.2, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 409, 39, NULL, 1164042.174, 1888805.999, 41.85050786, -87.67342126, 31, 'LOWER WEST SIDE', 25, 10, '(41.85050786, -87.67342126)') ,
(609694, 'John Hancock College Preparatory High School', 'HS', '4034 W 56th St', 'Chicago', 'IL', 60629, '(773) 535-2410', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609694.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 51.0, 'Average', 44.0, 'Average', 44.0, 'Average', 41.0, 'Weak', 37.0, 'Average', 42.0, 'Average', 50.0, 'Weak', 44.0, '78.20%', 7.5, '95.90%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.6, 13.8, 14.7, 15.0, 0.4, 15.8, 1.1, 22.6, 56.1, 45.9, 967, 44, 87.4, 1150280.45, 1867197.8269999998, 41.79149114, -87.72449161, 62, 'WEST ELSDON', 13, 8, '(41.79149114, -87.72449161)') ,
(609971, 'John Harvard Elementary School of Excellence', 'ES', '7525 S Harvard', 'Chicago', 'IL', 60620, '(773) 535-3045', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609971.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 51.0, 'Weak', 24.0, 'Strong', 61.0, 'Strong', 61.0, 'Weak', 43.0, 'Weak', 36.0, 'Average', 47.0, 'Strong', 55.0, '93.00%', 8.7, '96.00%', '97.80%', NULL, NULL, 25.6, 28.2, 42.7, 46.8, 50.0, 38.9, 58.8, 53.1, 14.6, 23.1, 13.8, 7.7, 1.2, 0.4, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 476, 45, NULL, 1175413.4270000001, 1854942.1169999999, 41.75733472, -87.63270008, 69, 'GREATER GRAND CROSSING', 17, 6, '(41.75733472, -87.63270008)') ,
(609975, 'John Hay Elementary Community Academy', 'ES', '1018 N Laramie Ave', 'Chicago', 'IL', 60651, '(773) 534-6000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609975.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 44.0, NULL, NULL, 'Average', 49.0, 'Strong', 61.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Strong', 55.0, '92.90%', 30.8, '97.10%', '100.00%', 53.8, 51.5, 45.8, 26.8, 58.0, 68.4, 34.3, 34.1, 68.7, 65.2, 0.0, 5.4, 11.9, 10.0, 0.0, 0.9, 'Yellow', 'Green', 10.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 554, 36, NULL, 1141465.048, 1906381.947, 41.89918473, -87.75585034, 25, 'AUSTIN', 37, 15, '(41.89918473, -87.75585034)') ,
(609964, 'John H Hamline Elementary School', 'ES', '4747 S Bishop St', 'Chicago', 'IL', 60609, '(773) 535-4565', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609964.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 33.0, 'Weak', 24.0, 'Weak', 29.0, 'Average', 43.0, 'Weak', 17.0, 'Weak', 33.0, 'Weak', 44.0, 'Average', 47.0, '94.40%', 7.5, '95.70%', '100.00%', 61.7, 48.9, 39.1, 19.5, 50.8, 67.9, 58.0, 26.5, 64.5, 56.5, 9.1, 18.2, 20.1, 6.1, 1.0, -0.2, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 679, 42, NULL, 1167511.897, 1873073.8830000001, 41.80726347, -87.66113856, 61, 'NEW CITY', 20, 9, '(41.80726347, -87.66113856)') ,
(610026, 'John H Kinzie Elementary School', 'ES', '5625 S Mobile Ave', 'Chicago', 'IL', 60638, '(773) 535-2425', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610026.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, NULL, NULL, 'Average', 49.0, 'Average', 54.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 48.0, '95.40%', 14.9, '95.30%', '99.10%', 62.0, NULL, 62.6, 42.6, 51.6, 67.6, 68.3, 50.2, 60.0, 56.4, 36.7, 40.5, 32.6, 19.7, 0.4, -0.3, 'Yellow', 'Yellow', 26.5, 63.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 733, 44, NULL, 1135391.774, 1866445.86, 41.78970398, -87.7791043, 56, 'GARFIELD RIDGE', 23, 8, '(41.78970398, -87.7791043)') ,
(610207, 'John H Vanderpoel Elementary Magnet School', 'ES', '9510 S Prospect Ave', 'Chicago', 'IL', 60643, '(773) 535-2690', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610207.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 74.0, NULL, NULL, 'Weak', 24.0, 'Weak', 31.0, 'Weak', NULL, NULL, NULL, 'Strong', 55.0, 'Average', 50.0, '97.20%', 0.3, '96.20%', '100.00%', 81.6, 69.8, 51.7, 47.7, 53.5, 59.8, 42.7, 54.5, 35.7, 55.9, 11.1, 38.9, 17.7, 24.1, -0.3, 1.0, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 317, 49, NULL, 1166544.213, 1841551.135, 41.72078152, -87.66558526, 72, 'BEVERLY', 19, 22, '(41.72078152, -87.66558526)') ,
(609782, 'John J Audubon Elementary School', 'ES', '3500 N Hoyne Ave', 'Chicago', 'IL', 60618, '(773) 534-5470', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609782.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 95.0, 'Strong', 73.0, 'Strong', 75.0, 'Strong', 60.0, 'Weak', 60.0, 'Strong', 65.0, 'Strong', 61.0, 'Average', 47.0, '96.20%', 4.5, '96.70%', '98.60%', NULL, 54.3, 72.1, 68.9, 65.5, 76.7, 67.5, 70.3, 82.6, 65.3, 31.6, 36.8, 45.3, 34.8, 1.2, 1.1, 'Green', 'Green', 47.4, 61.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 580, 33, NULL, 1161693.811, 1923258.727, 41.94509808, -87.68107885, 5, 'NORTH CENTER', 47, 19, '(41.94509808, -87.68107885)') ,
(610126, 'John J Pershing Elementary Humanities Magnet', 'ES', '3113 S Rhodes Ave', 'Chicago', 'IL', 60616, '(773) 534-9272', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610126.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', NULL, NULL, 'Very Strong', 90.0, NULL, NULL, NULL, NULL, 'Weak', 70.0, 'Strong', 78.0, NULL, NULL, NULL, NULL, '95.50%', 19.2, '96.70%', '100.00%', NULL, 92.3, 56.3, 56.3, 77.1, 80.9, NULL, NULL, NULL, NULL, NULL, NULL, 54.9, 39.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 258, 40, NULL, 1180449.2040000001, 1884392.3159999999, 41.83803465, -87.61334083, 35, 'DOUGLAS', 4, 2, '(41.83803465, -87.61334083)') ,
(610054, 'John L Marsh Elementary School', 'ES', '9822 S Exchange Ave', 'Chicago', 'IL', 60617, '(773) 535-6430', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610054.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Strong', 72.0, 'Average', 53.0, 'Average', 53.0, 'Weak', 39.0, 'Weak', 55.0, 'Average', 48.0, 'Average', 50.0, 'Strong', 55.0, '95.70%', 5.4, '95.50%', '99.10%', 55.8, 37.6, 34.5, 34.9, 53.2, 58.7, 33.5, 44.9, 50.8, 52.2, 11.1, 31.1, 18.9, 14.5, -0.9, -0.8, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 841, 47, NULL, 1197424.97, 1840279.493, 41.71657908, -87.55251993, 51, 'SOUTH DEERING', 10, 4, '(41.71657908, -87.55251993)') ,
(609723, 'John Marshall Metropolitan High School', 'HS', '3250 W Adams St', 'Chicago', 'IL', 60624, '(773) 534-6455', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609723.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 51.0, 'Average', 52.0, 'Average', 45.0, 'Average', 46.0, 'Weak', 49.0, 'Average', 42.0, NULL, NULL, NULL, NULL, '74.10%', 42.9, '95.80%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 11.7, 11.8, 13.4, 13.2, 1.5, 14.6, 1.2, 6.5, 36.8, 45.2, 726, 38, 55.2, 1154542.5829999999, 1898925.511, 41.87847194, -87.70801612, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87847194, -87.70801612)') ,
(609710, 'John M Harlan Community Academy High School', 'HS', '9652 S Michigan Ave', 'Chicago', 'IL', 60628, '(773) 535-5400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609710.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '81.20%', 21.6, '94.40%', '95.70%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 16.0, 40.0, 17.8, 15.6, -2.9, -0.8, 'Red', 'Yellow', 100.0, 40.0, 13.2, 12.9, 14.1, 13.7, 0.5, 14.7, 0.6, 5.9, 47.9, 55.7, 1245, 48, 64.4, 1178815.804, 1840781.935, 41.71840075, -87.62066069, 49, 'ROSELAND', 6, 5, '(41.71840075, -87.62066069)') ,
(609954, 'John Milton Gregory Elementary School', 'ES', '3715 W Polk St', 'Chicago', 'IL', 60624, '(773) 534-6820', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609954.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 44.0, 'Average', 43.0, 'Weak', 39.0, 'Average', 43.0, 'Weak', 66.0, 'Average', 59.0, 'Average', 49.0, 'Strong', 55.0, '95.20%', 3.2, '94.60%', '100.00%', 61.6, 45.5, 36.8, 17.9, 62.3, 70.8, 44.3, 37.7, 69.1, 77.9, 6.3, 41.7, 24.2, 19.5, 1.5, 2.1, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 323, 37, NULL, 1151660.565, 1896077.812, 41.87071468, -87.71867323, 27, 'EAST GARFIELD PARK', 24, 11, '(41.87071468, -87.71867323)') ,
(610180, 'John M Smyth Elementary School', 'ES', '1059 W 13th St', 'Chicago', 'IL', 60608, '(773) 534-7180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610180.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 37.0, NULL, NULL, 'Average', 54.0, 'Strong', 64.0, 'Weak', NULL, NULL, NULL, 'Weak', 43.0, 'Average', 52.0, '92.40%', 20.1, '93.30%', '100.00%', 67.8, NULL, 17.6, 10.6, 38.1, 46.0, 24.8, 12.3, 54.2, 38.8, 1.4, 1.4, 6.3, 3.2, 1.1, -0.9, 'Green', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 475, 38, NULL, 1169435.063, 1894247.175, 41.86532347, -87.65347031, 28, 'NEAR WEST SIDE', 2, 12, '(41.86532347, -87.65347031)') ,
(610199, 'Johnnie Colemon Elementary Academy', 'ES', '1441 W 119th St', 'Chicago', 'IL', 60643, '(773) 535-3975', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610199.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 60.0, NULL, NULL, 'Average', 56.0, 'Strong', 63.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Strong', 57.0, '95.90%', 23.4, '95.40%', '100.00%', 75.3, 53.4, 24.4, 22.1, 47.6, 46.9, 47.2, 44.4, 37.7, 50.7, 12.5, 16.7, 14.6, 18.4, 0.2, 0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 266, 49, NULL, 1168705.904, 1825787.71, 41.67747786, -87.65812011, 53, 'WEST PULLMAN', 34, 5, '(41.67747786, -87.65812011)') ,
(610111, 'John Palmer Elementary School', 'ES', '5051 N Kenneth Ave', 'Chicago', 'IL', 60630, '(773) 534-3704', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610111.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 75.0, NULL, NULL, 'Strong', 61.0, 'Strong', 68.0, 'Weak', NULL, NULL, NULL, 'Strong', 65.0, 'Strong', 61.0, '96.30%', 12.8, '94.80%', '99.20%', 72.2, 42.4, 47.7, 38.0, 58.9, 58.5, 48.9, 45.3, 52.5, 57.3, 24.8, 37.6, 25.3, 15.1, 0.5, 0.3, 'Green', 'Yellow', 26.5, 74.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 919, 31, NULL, 1145817.577, 1933407.476, 41.97326374, -87.73917547, 14, 'ALBANY PARK', 39, 17, '(41.97326374, -87.73917547)') ,
(609775, 'John P Altgeld Elementary School', 'ES', '1340 W 71st St', 'Chicago', 'IL', 60636, '(773) 535-3250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609775.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 48.0, 'Weak', 37.0, 'Strong', 64.0, 'Average', 58.0, 'Weak', 27.0, 'Weak', 35.0, 'Weak', 46.0, 'Average', 48.0, '91.90%', 32.9, '95.50%', '100.00%', 42.1, NULL, 30.1, 34.4, 67.1, 52.4, 37.2, 37.2, 56.0, 56.8, 8.8, 40.4, 16.4, 20.9, -0.8, 3.0, 'Red', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 527, 43, NULL, 1168430.269, 1857717.233, 41.76510322, -87.6582125, 67, 'WEST ENGLEWOOD', 17, 7, '(41.76510322, -87.6582125)') ,
(610184, 'John Spry Elementary Community School', 'ES', '2400 S Marshall Blvd', 'Chicago', 'IL', 60623, '(773) 534-1700', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610184.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 66.0, 'Average', 59.0, 'Strong', 70.0, 'Strong', 67.0, 'Weak', 52.0, 'Average', 43.0, 'Weak', 46.0, 'Average', 48.0, '96.20%', 5.9, '97.40%', '99.00%', 44.4, 12.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 18.7, 13.3, 17.9, 7.3, 1.7, 1.1, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 809, 39, NULL, 1157044.9109999998, 1887895.675, 41.84815451, -87.69912725, 30, 'SOUTH LAWNDALE', 12, 10, '(41.84815451, -87.69912725)') ,
(610269, 'John T McCutcheon Elementary School', 'ES', '4865 N Sheridan Rd', 'Chicago', 'IL', 60640, '(773) 534-2680', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610269.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 56.0, 'Weak', 36.0, 'Strong', 67.0, 'Average', 41.0, 'Weak', 32.0, 'Average', 42.0, 'Average', 52.0, 'Average', 53.0, '94.00%', 8.8, '94.80%', '98.10%', 62.4, 66.3, 31.3, 30.3, 46.7, 48.3, 34.1, 40.5, 48.1, 58.4, 9.1, 13.6, 19.5, 10.8, 1.0, -0.2, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 355, 32, NULL, 1168789.535, 1932738.4619999998, 41.97095968, -87.65472193, 3, 'UPTOWN', 48, 20, '(41.97095968, -87.65472193)') ,
(610130, 'John T Pirie Fine Arts & Academic Center Elementary School', 'ES', '650 E 85th St', 'Chicago', 'IL', 60619, '(773) 535-3435', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610130.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 26.0, 'Average', 44.0, 'Strong', 63.0, 'Strong', 62.0, 'Weak', 43.0, 'Average', 46.0, NULL, NULL, NULL, NULL, '93.50%', 20.8, '95.40%', '100.00%', 74.5, 49.6, 31.1, 37.3, 55.8, 49.7, 44.2, 50.0, 71.8, 73.7, NULL, NULL, 10.7, 20.9, 0.1, 1.6, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 384, 45, NULL, 1182225.898, 1848815.302, 41.74036711, -87.60792289, 44, 'CHATHAM', 6, 6, '(41.74036711, -87.60792289)') ,
(609864, 'John W Cook Elementary School', 'ES', '8150 S Bishop St', 'Chicago', 'IL', 60620, '(773) 535-3315', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609864.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Average', 47.0, '94.00%', 26.0, '94.00%', '100.00%', 57.1, 36.0, 21.5, 12.7, 42.4, 55.4, 41.2, 31.8, 68.7, 68.2, 1.7, 6.8, 6.4, 4.6, 0.8, 0.6, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 482, 49, NULL, 1168050.5320000001, 1850484.1530000002, 41.74526284, -87.65981190000001, 71, 'AUBURN GRESHAM', 21, 6, '(41.74526284, -87.6598119)') ,
(609937, 'John W Garvy Elementary School', 'ES', '5225 N Oak Park Ave', 'Chicago', 'IL', 60656, '(773) 534-1185', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609937.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 60.0, 'Strong', 70.0, 'Average', 50.0, 'Average', 51.0, 'Weak', 51.0, 'Average', 55.0, 'Strong', 56.0, 'Average', 49.0, '95.30%', 10.2, '96.50%', '97.30%', 83.4, 43.6, 63.9, 57.8, 58.1, 63.9, 60.6, 59.4, 51.4, 55.3, 38.4, 49.3, 34.4, 33.5, -0.1, 0.2, 'Yellow', 'Yellow', 44.6, 45.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 720, 30, NULL, 1130304.414, 1934430.6940000001, 41.97635262, -87.79619832, 10, 'NORWOOD PARK', 41, 16, '(41.97635262, -87.79619832)') ,
(610225, 'John Whistler Elementary School', 'ES', '11533 S Ada St', 'Chicago', 'IL', 60643, '(773) 535-5560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610225.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 45.0, 'Weak', 32.0, 'Average', 41.0, 'Weak', 28.0, 'Weak', 20.0, 'Weak', 21.0, 'Average', 52.0, 'Strong', 55.0, '92.40%', 11.1, '97.00%', '100.00%', 74.7, 60.2, 34.0, 27.4, 49.0, 61.2, 31.5, 28.0, 50.4, 51.2, 6.0, 8.2, 5.4, 4.1, -0.9, 0.5, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 375, 49, NULL, 1169444.9340000001, 1828137.9440000001, 41.68391137, -87.65534733, 53, 'WEST PULLMAN', 34, 5, '(41.68391137, -87.65534733)') ,
(609828, 'Jonathan Burr Elementary School', 'ES', '1621 W Wabansia Ave', 'Chicago', 'IL', 60622, '(773) 534-4090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609828.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 71.0, 'Strong', 66.0, 'Strong', 68.0, 'Average', 51.0, 'Weak', 53.0, 'Strong', 63.0, 'Strong', 56.0, 'Average', 49.0, '95.90%', 4.3, '96.90%', '100.00%', 78.5, 53.1, 53.8, 43.8, 51.9, 51.9, 71.3, 57.7, 69.2, 58.4, 14.3, 25.0, 29.1, 22.2, 0.4, 0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 348, 35, NULL, 1165168.4640000002, 1911344.1809999999, 41.91233061, -87.66864691, 24, 'WEST TOWN', 1, 14, '(41.91233061, -87.66864691)') ,
(610159, 'Jonathan Y Scammon Elementary School', 'ES', '4201 W Henderson St', 'Chicago', 'IL', 60641, '(773) 534-3475', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610159.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 44.0, NULL, NULL, 'Weak', 21.0, 'Weak', 20.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Weak', 44.0, '95.90%', 8.0, '95.30%', '100.00%', 69.6, 39.3, 32.4, 27.7, 50.9, 52.2, 39.1, 33.0, 61.5, 60.5, 14.7, 33.7, 14.8, 11.9, -0.6, 0.2, 'Red', 'Yellow', 19.4, 100.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 937, 29, NULL, 1147787.325, 1921900.2959999999, 41.94164944, -87.73222901, 16, 'IRVING PARK', 30, 17, '(41.94164944, -87.73222901)') ,
(609865, 'Jordan Elementary Community School', 'ES', '7414 N Wolcott Ave', 'Chicago', 'IL', 60626, '(773) 534-2220', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609865.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 61.0, 'Weak', 31.0, 'Average', 57.0, 'Average', 49.0, 'Weak', 17.0, 'Very Weak', 15.0, 'Average', 52.0, 'Average', 51.0, '95.60%', 16.4, '95.10%', '100.00%', 60.1, 42.0, 36.0, 21.1, 41.4, 54.1, 26.3, 19.1, 43.2, 34.7, 4.8, 15.9, 16.9, 7.1, 0.3, -1.2, 'Yellow', 'Red', 37.7, 23.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 753, 32, NULL, 1162385.2240000002, 1949501.716, 42.01709536, -87.67779928, 1, 'ROGERS PARK', 49, 24, '(42.01709536, -87.67779928)') ,
(610313, 'Jose De Diego Elementary Community Academy', 'ES', '1313 N Claremont Ave', 'Chicago', 'IL', 60622, '(773) 534-4451', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610313.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'Yes', 'Average', 57.0, 'Average', 53.0, 'Weak', 37.0, 'Average', 40.0, 'Weak', 60.0, 'Strong', 64.0, 'Average', 53.0, 'Strong', 54.0, '95.00%', 29.0, '96.80%', '100.00%', 75.0, 51.4, 29.9, 32.4, 49.2, 44.4, 45.5, 45.1, 49.1, 51.3, 17.5, 33.3, 16.7, 11.3, -0.3, -1.1, 'Yellow', 'Red', 49.2, 23.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 869, 35, NULL, 1160540.669, 1908752.314, 41.90531549, -87.68572005, 24, 'WEST TOWN', 1, 14, '(41.90531549, -87.68572005)') ,
(609950, 'Josefa Ortiz De Dominguez Elementary School', 'ES', '3000 S Lawndale Ave', 'Chicago', 'IL', 60623, '(773) 534-1600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609950.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', NULL, NULL, 'No', NULL, NULL, 'Strong', 69.0, NULL, NULL, NULL, NULL, 'Weak', 48.0, 'Average', 54.0, 'Average', 51.0, 'Strong', 55.0, '95.80%', 3.6, '97.00%', '99.00%', 72.4, 51.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 793, 37, NULL, 1152138.268, 1884434.021, 41.83875333, -87.71722614, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83875333, -87.71722614)') ,
(610242, 'Joseph Brennemann Elementary School', 'ES', '4251 N Clarendon Ave', 'Chicago', 'IL', 60613, '(773) 534-5766', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610242.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 50.0, 'Average', 56.0, 'Average', 55.0, 'Average', 59.0, 'Weak', 41.0, 'Average', 54.0, 'Average', 53.0, 'Strong', 59.0, '96.50%', 13.7, '97.70%', '96.70%', 79.8, 47.2, 32.4, 30.8, 50.5, 65.7, 48.8, 30.0, 66.2, 50.0, 8.3, 25.0, 18.1, 10.9, 1.0, 1.0, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 362, 32, NULL, 1170183.793, 1928747.67, 41.95997841, -87.64971223, 3, 'UPTOWN', 46, 19, '(41.95997841, -87.64971223)') ,
(609938, 'Joseph E Gary Elementary School', 'ES', '3740 W 31st St', 'Chicago', 'IL', 60623, '(773) 534-1455', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609938.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 32.0, 'Weak', 27.0, 'Weak', 29.0, 'Weak', 28.0, 'Weak', 29.0, 'Very Weak', 17.0, 'Weak', 45.0, 'Average', 48.0, '96.10%', 16.8, '96.20%', '100.00%', NULL, NULL, 34.5, 21.8, 50.5, 55.3, 45.5, 27.7, 58.4, 41.7, 12.3, 21.3, 14.3, 7.6, 0.4, 0.1, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1204, 37, NULL, 1151739.739, 1883825.92, 41.83709246, -87.71870452, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83709246, -87.71870452)') ,
(610041, 'Josephine C Locke Elementary School', 'ES', '2828 N Oak Park Ave', 'Chicago', 'IL', 60634, '(773) 534-3300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610041.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 57.0, 'Average', 47.0, 'Average', 49.0, 'Average', 56.0, 'Weak', 44.0, 'Average', 45.0, 'Average', 47.0, 'Average', 50.0, '95.50%', 2.2, '95.10%', '100.00%', 63.4, 39.5, 36.1, 30.5, 55.3, 63.0, 58.7, 46.0, 64.0, 70.5, 26.1, 33.8, 21.1, 15.6, 0.7, 1.7, 'Green', 'Green', 23.9, 45.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1317, 29, NULL, 1130557.942, 1918183.07, 41.93176301, -87.79564085, 18, 'MONTCLARE', 36, 25, '(41.93176301, -87.79564085)') ,
(610015, 'Joseph Jungman Elementary School', 'ES', '1746 S Miller St', 'Chicago', 'IL', 60608, '(773) 534-7375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610015.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 36.0, 'Weak', 34.0, 'Average', 56.0, 'Average', 53.0, 'Weak', 20.0, 'Average', 43.0, 'Average', 48.0, 'Average', 49.0, '96.40%', 5.3, '96.30%', '93.80%', 75.5, 34.7, 32.5, 20.0, 59.2, 64.4, 25.0, 34.9, 51.3, 71.3, 6.9, 34.5, 7.8, 6.6, 0.1, -0.7, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 286, 39, NULL, 1169764.005, 1891617.595, 41.85810053, -87.65233933, 31, 'LOWER WEST SIDE', 25, 12, '(41.85810053, -87.65233933)') ,
(609925, 'Joseph Kellman Corporate Community Elementary School', 'ES', '751 S Sacramento Blvd', 'Chicago', 'IL', 60612, '(773) 534-6602', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609925.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, 'Strong', 68.0, 'Average', 42.0, 'Average', 45.0, 'Weak', 37.0, 'Average', 50.0, 'Strong', 59.0, 'Strong', 57.0, '95.20%', 4.4, '95.80%', '100.00%', 84.7, 55.3, 37.0, 32.3, 48.9, 56.0, 34.5, 43.2, 56.5, 55.2, 0.0, 11.5, 13.3, 10.5, -0.3, -0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 293, 37, NULL, 1156531.906, 1896287.544, 41.87119309, -87.70078311, 27, 'EAST GARFIELD PARK', 2, 11, '(41.87119309, -87.70078311)') ,
(610043, 'Joseph Lovett Elementary School', 'ES', '6333 W Bloomingdale Ave', 'Chicago', 'IL', 60639, '(773) 534-3130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610043.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 43.0, NULL, NULL, 'Weak', 32.0, 'Weak', 22.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Weak', 45.0, '95.40%', 16.1, '95.40%', '100.00%', 68.5, 62.7, 23.5, 33.1, 51.2, 51.2, 32.9, 37.3, 40.3, 65.5, 7.8, 27.5, 9.1, 8.3, -0.2, -0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 490, 29, NULL, 1133787.949, 1911253.2, 41.91269042, -87.78393402, 25, 'AUSTIN', 29, 25, '(41.91269042, -87.78393402)') ,
(610189, 'Joseph Stockton Elementary School', 'ES', '4420 N Beacon St', 'Chicago', 'IL', 60640, '(773) 534-2450', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610189.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 37.0, NULL, NULL, 'Average', 51.0, 'Strong', 68.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.90%', 11.0, '95.30%', '100.00%', 53.2, 47.2, 35.3, 28.2, 41.2, 50.0, 54.7, 56.4, 51.1, 50.5, 21.1, 44.7, 18.3, 12.4, -0.5, -0.1, 'Yellow', 'Yellow', 30.8, 75.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 462, 32, NULL, 1166321.035, 1929570.108, 41.9623189, -87.66388983, 3, 'UPTOWN', 46, 19, '(41.9623189, -87.66388983)') ,
(610218, 'Joseph Warren Elementary School', 'ES', '9239 S Jeffery Ave', 'Chicago', 'IL', 60617, '(773) 535-6625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610218.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 33.0, 'Average', 55.0, 'Average', 49.0, 'Average', 40.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Average', 50.0, '95.00%', 21.6, '95.30%', '100.00%', 38.6, 20.0, 26.1, 21.5, 55.3, 46.4, 33.3, 27.0, 45.6, 35.0, 8.6, 25.7, 13.7, 14.7, -1.0, -0.4, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 328, 47, NULL, 1191168.455, 1843939.0669999998, 41.72677474, -87.5753162, 48, 'CALUMET HEIGHTS', 8, 4, '(41.72677474, -87.5753162)') ,
(610019, 'Joshua D Kershaw Elementary School', 'ES', '6450 S Lowe Ave', 'Chicago', 'IL', 60621, '(773) 535-3050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610019.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 62.0, 'Strong', 63.0, 'Strong', 69.0, 'Average', 56.0, 'Weak', 50.0, 'Average', 59.0, 'Strong', 58.0, 'Strong', 63.0, '94.80%', 8.2, '96.00%', '100.00%', NULL, NULL, 55.2, 46.3, 71.6, 71.6, 67.2, 58.1, 83.3, 78.7, 5.6, 38.9, 22.3, 16.2, 1.9, 1.1, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 262, 45, NULL, 1173111.97, 1861899.898, 41.77647883, -87.64092929, 68, 'ENGLEWOOD', 20, 7, '(41.77647883, -87.64092929)') ,
(610129, 'Josiah Pickard Elementary School', 'ES', '2301 W 21st Pl', 'Chicago', 'IL', 60608, '(773) 535-7280', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610129.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 50.0, 'Weak', 36.0, 'Weak', 35.0, 'Weak', 37.0, 'Weak', 32.0, 'Weak', 34.0, 'Average', 48.0, 'Strong', 54.0, '95.80%', 5.1, '96.40%', '100.00%', 76.4, NULL, 48.9, 25.6, 55.0, 78.6, 42.5, 30.2, 70.0, 69.8, 6.6, 8.2, 16.1, 11.0, 1.7, 0.0, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 656, 39, NULL, 1161333.371, 1889647.219, 41.85287295, -87.6833397, 31, 'LOWER WEST SIDE', 25, 12, '(41.85287295, -87.6833397)') ,
(610022, 'Joyce Kilmer Elementary School', 'ES', '6700 N Greenview Ave', 'Chicago', 'IL', 60626, '(773) 534-2115', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610022.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', NULL, NULL, 'Average', 43.0, NULL, NULL, NULL, NULL, 'Weak', 43.0, 'Average', 42.0, 'Average', 49.0, 'Average', 51.0, '96.00%', 15.6, '96.90%', '100.00%', 70.0, 51.2, 40.5, 28.6, 53.4, 61.2, 39.4, 33.7, 57.8, 52.8, 22.7, 27.3, 22.8, 15.0, -0.2, -0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 886, 32, NULL, 1165015.365, 1944631.6819999998, 42.00367622, -87.66826026, 1, 'ROGERS PARK', 40, 24, '(42.00367622, -87.66826026)') ,
(610253, 'Julia C Lathrop Elementary School', 'ES', '1440 S Christiana Ave', 'Chicago', 'IL', 60623, '(773) 534-1812', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610253.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 67.0, NULL, NULL, 'Average', 58.0, 'Weak', 26.0, 'Weak', NULL, NULL, NULL, 'Weak', 40.0, 'Average', 50.0, '90.50%', 113.5, '95.50%', '100.00%', 38.5, NULL, 19.5, 13.2, 45.5, 34.1, 18.4, 34.7, 35.0, 61.1, 7.7, 3.8, 5.9, 4.0, -1.7, -0.6, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 83, 37, NULL, 1154232.147, 1892627.1719999998, 41.86119482, -87.70932409999999, 29, 'NORTH LAWNDALE', 24, 10, '(41.86119482, -87.7093241)') ,
(610000, 'Julia Ward Howe Elementary School of Excellence', 'ES', '720 N Lorel Ave', 'Chicago', 'IL', 60644, '(773) 534-6060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610000.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 1', 'No', 'Strong', 67.0, 'Average', 46.0, 'Very Strong', 90.0, 'Strong', 69.0, 'Weak', 59.0, 'Strong', 67.0, 'Strong', 57.0, 'Strong', 64.0, '96.00%', 7.7, '97.10%', '100.00%', NULL, NULL, 36.9, 35.6, 63.1, 75.2, 38.5, 39.1, 67.5, 56.4, 7.4, 14.8, 20.6, 9.3, 0.8, -0.8, 'Green', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 576, 36, NULL, 1140502.548, 1904349.511, 41.89362521, -87.75943557, 25, 'AUSTIN', 37, 15, '(41.89362521, -87.75943557)') ,
(610280, 'Kate S Buckingham Special Education Center', 'ES', '9207 S Phillips Ave', 'Chicago', 'IL', 60617, '(773) 535-6422', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610280.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', NULL, 'Standard', 'Not Applicable', 'Level 3', 'No', 'Weak', 31.0, NULL, NULL, 'Weak', 24.0, 'Weak', 20.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '83.40%', 116.9, '94.40%', '100.00%', NULL, NULL, 20.0, 20.0, 80.0, 70.0, 0.0, 4.3, 45.0, 59.1, NULL, NULL, 0.0, 0.0, -0.4, -0.8, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 26, 47, NULL, 1194146.741, 1844346.869, 41.72782129, -87.56439326, 48, 'CALUMET HEIGHTS', 7, 4, '(41.72782129, -87.56439326)') ,
(610016, 'Kate S Kellogg Elementary School', 'ES', '9241 S Leavitt St', 'Chicago', 'IL', 60620, '(773) 535-2590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610016.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 76.0, NULL, NULL, 'Average', 57.0, 'Strong', 63.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Weak', 46.0, '96.30%', 12.0, '95.40%', '91.80%', 78.8, 10.0, 49.4, 67.8, 65.5, 65.5, 52.0, 77.5, 49.0, 64.7, 17.5, 47.5, 24.2, 30.9, -0.4, 0.7, 'Yellow', 'Yellow', 28.2, 36.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 283, 49, NULL, 1163362.1840000001, 1843156.174, 41.72525315, -87.67719555, 72, 'BEVERLY', 19, 22, '(41.72525315, -87.67719555)') ,
(609716, 'Kelvyn Park High School', 'HS', '4343 W Wrightwood Ave', 'Chicago', 'IL', 60639, '(773) 534-4200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609716.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 32.0, 'Weak', 28.0, 'Weak', 32.0, 'Weak', 28.0, 'Weak', 19.0, 'Average', 43.0, NULL, NULL, NULL, NULL, '74.40%', 12.9, '94.50%', '99.50%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.8, 12.2, 13.8, 13.8, 1.0, 14.9, 1.1, 16.7, 54.1, 35.0, 1252, 29, 56.9, 1146795.845, 1916879.493, 41.92789092, -87.73600171, 20, 'HERMOSA', 31, 25, '(41.92789092, -87.73600171)') ,
(609746, 'Kenwood Academy High School', 'HS', '5015 S Blackstone Ave', 'Chicago', 'IL', 60615, '(773) 535-1350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609746.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 63.0, NULL, NULL, 'Average', 48.0, 'Average', 56.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '88.40%', 5.9, '95.30%', '99.40%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 60.2, 75.0, 69.5, 31.0, -1.1, -0.5, 'Red', 'Yellow', 81.2, 49.0, 15.6, 15.1, 16.8, 16.9, 1.3, 19.1, 2.3, 26.8, 69.5, 79.7, 1852, 46, 70.3, 1186813.047, 1871954.1519999998, 41.80375479, -87.59038395, 39, 'KENWOOD', 4, 2, '(41.80375479, -87.59038395)') ,
(609719, 'Lake View High School', 'HS', '4015 N Ashland Ave', 'Chicago', 'IL', 60613, '(773) 534-5440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609719.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 64.0, NULL, NULL, 'Average', 52.0, 'Average', 43.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Weak', 43.0, '87.70%', 8.5, '96.20%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 15.3, 15.8, 16.1, 16.7, 1.4, 17.8, 1.7, 36.6, 81.4, 62.8, 1500, 33, 87.3, 1164975.217, 1926813.581, 41.95478361, -87.66891643, 6, 'LAKE VIEW', 47, 19, '(41.95478361, -87.66891643)') ,
(610368, 'Langston Hughes Elementary School', 'ES', '240 W 104th St', 'Chicago', 'IL', 60628, '(773) 535-5075', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610368.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Average', 54.0, 'Very Weak', 16.0, 'Weak', 30.0, 'Weak', 62.0, 'Average', 48.0, 'Weak', 46.0, 'Strong', 56.0, '90.00%', 27.5, '95.50%', '100.00%', 44.7, 67.0, 19.3, 16.8, 46.8, 54.5, 22.7, 18.7, 57.0, 43.7, 0.0, 9.5, 8.2, 6.1, 0.3, -0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 424, 48, NULL, 1176345.568, 1836013.039, 41.70536999, -87.62985093, 49, 'ROSELAND', 34, 5, '(41.70536999, -87.62985093)') ,
(610033, 'LaSalle Elementary Language Academy', 'ES', '1734 N Orleans St', 'Chicago', 'IL', 60614, '(773) 534-8470', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610033.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Strong', 79.0, 'Strong', 62.0, 'Average', 52.0, 'Weak', 39.0, 'Average', 43.0, 'Average', 53.0, 'Average', 48.0, '96.80%', 7.0, '97.60%', '100.00%', 86.0, 47.4, 81.4, 78.3, 67.7, 76.4, 79.2, 84.7, 73.2, 76.5, 61.0, 71.2, 55.5, 49.5, 0.3, -0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 576, 33, NULL, 1173614.17, 1911976.635, 41.9138823, -87.63760107, 7, 'LINCOLN PARK', 43, 18, '(41.9138823, -87.63760107)') ,
(610520, 'LaSalle II Magnet Elementary School', 'ES', '1148 N Honore St', 'Chicago', 'IL', 60622, '(773) 534-0490', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610520.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, 'Very Strong', 81.0, 'Weak', 36.0, 'Average', 52.0, 'Weak', 83.0, 'Strong', 79.0, 'Strong', 55.0, 'Average', 51.0, '95.10%', 5.6, '94.80%', '100.00%', NULL, NULL, 61.1, 69.9, 82.5, 80.2, 61.1, 76.4, 70.4, 85.5, NULL, NULL, 33.8, 38.0, 1.1, 2.0, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 609, 35, NULL, 1163810.597, 1907900.166, 41.90290876, -87.67373263, 24, 'WEST TOWN', 1, 13, '(41.90290876, -87.67373263)') ,
(609910, 'Laughlin Falconer Elementary School', 'ES', '3020 N Lamon Ave', 'Chicago', 'IL', 60641, '(773) 534-3560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609910.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 54.0, 'Weak', 24.0, 'Average', 48.0, 'Weak', 27.0, 'Weak', 22.0, 'Weak', 30.0, 'Average', 49.0, 'Average', 53.0, '96.10%', 5.7, '97.00%', '99.50%', 71.7, 42.9, 47.7, 28.5, 46.9, 63.4, 72.0, 33.3, 77.2, 59.1, NULL, NULL, 21.0, 14.2, 1.1, 0.8, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1520, 29, NULL, 1143139.285, 1919727.764, 41.93577597, -87.74936705, 19, 'BELMONT CRAGIN', 31, 25, '(41.93577597, -87.74936705)') ,
(610133, 'Laura S Ward Elementary School', 'ES', '410 N Monticello Ave', 'Chicago', 'IL', 60624, '(773) 534-6440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610133.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '92.90%', 3.9, '97.50%', '100.00%', 72.6, 54.0, 35.2, 13.0, 43.5, 60.7, 41.7, 27.5, 44.0, 50.4, 14.3, 47.1, 14.2, 6.9, -3.5, -5.0, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 429, 34, NULL, 1151927.188, 1902542.7869999998, 41.88845002, -87.71752405, 23, 'HUMBOLDT PARK', 27, 11, '(41.88845002, -87.71752405)') ,
(610034, 'Lawndale Elementary Community Academy', 'ES', '3500 W Douglas Blvd', 'Chicago', 'IL', 60623, '(773) 534-1635', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610034.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '91.00%', 18.8, '94.80%', '98.00%', 43.4, NULL, 17.2, 10.0, 34.0, 44.0, 21.1, 15.9, 45.2, 47.0, 0.0, 2.0, 3.3, 4.0, -2.1, -2.2, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 508, 37, NULL, 1153224.4209999999, 1893272.908, 41.86298683, -87.71300615, 29, 'NORTH LAWNDALE', 24, 10, '(41.86298683, -87.71300615)') ,
(610024, 'Lazaro Cardenas Elementary School', 'ES', '2345 S Millard Ave', 'Chicago', 'IL', 60623, '(773) 534-1465', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610024.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', NULL, NULL, 'Average', 58.0, NULL, NULL, NULL, NULL, 'Weak', 62.0, 'Average', 58.0, 'Average', 52.0, 'Strong', 57.0, '96.30%', 3.2, '98.50%', '100.00%', 77.2, 59.6, 31.9, 26.5, 41.4, 57.7, NULL, NULL, NULL, NULL, NULL, NULL, 26.3, 15.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 654, 37, NULL, 1152451.967, 1887990.6930000002, 41.8485071, -87.71598119, 30, 'SOUTH LAWNDALE', 22, 10, '(41.8485071, -87.71598119)') ,
(609907, 'Leif Ericson Elementary Scholastic Academy', 'ES', '3600 W 5th Ave', 'Chicago', 'IL', 60624, '(773) 534-6660', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609907.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 33.0, NULL, NULL, 'Average', 49.0, 'Average', 52.0, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Weak', 45.0, '94.70%', 20.8, '95.10%', '100.00%', 76.1, 70.9, 41.3, 22.4, 43.1, 58.5, 43.8, 25.4, 60.9, 40.9, 10.4, 20.8, 13.3, 7.7, 2.0, 1.0, 'Green', 'Green', 51.1, 4.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 518, 34, NULL, 1152417.4440000001, 1897922.54, 41.87576191, -87.71584573, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87576191, -87.71584573)') ,
(610298, 'Lenart Elementary Regional Gifted Center', 'ES', '8101 S LaSalle St', 'Chicago', 'IL', 60620, '(773) 535-0040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610298.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 79.0, 'Average', 58.0, 'Average', 51.0, 'Strong', 67.0, 'Weak', 50.0, 'Strong', 63.0, 'Average', 52.0, 'Average', 52.0, '97.40%', 0.3, '97.80%', '100.00%', NULL, NULL, 93.9, 96.9, 69.4, 46.9, 93.5, 96.8, 57.6, 55.9, 86.7, 93.3, 88.0, 86.4, 1.0, 0.5, 'Green', 'Yellow', 100.0, 70.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 328, 45, NULL, 1176725.617, 1851241.455, 41.74715024, -87.62800231, 44, 'CHATHAM', 21, 6, '(41.74715024, -87.62800231)') ,
(610036, 'Leslie Lewis Elementary School', 'ES', '1431 N Leamington Ave', 'Chicago', 'IL', 60651, '(773) 534-3060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610036.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 21.0, 'Weak', 37.0, 'Weak', 36.0, 'Strong', 64.0, 'Weak', 30.0, 'Average', 48.0, 'Weak', 42.0, 'Weak', 43.0, '91.80%', 37.7, '95.30%', '98.60%', 67.8, 58.4, 20.3, 19.8, 51.3, 40.1, 15.8, 22.5, 53.5, 58.3, 2.3, 4.5, 4.0, 5.0, -0.5, -0.1, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 565, 36, NULL, 1141810.2920000001, 1909168.612, 41.90682528, -87.75451319, 25, 'AUSTIN', 37, 25, '(41.90682528, -87.75451319)') ,
(609738, 'Lincoln Park High School', 'HS', '2001 N Orchard St', 'Chicago', 'IL', 60614, '(773) 534-8130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609738.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 65.0, 'Strong', 61.0, 'Average', 44.0, 'Average', 49.0, 'Weak', 45.0, 'Average', 45.0, NULL, NULL, NULL, NULL, '84.60%', 9.2, '96.00%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 17.7, 18.0, 18.6, 19.0, 1.3, 21.6, 3.0, 53.6, 77.9, 75.5, 2342, 33, 74.4, 1171322.336, 1913569.392, 41.91830362, -87.64597389, 7, 'LINCOLN PARK', 43, 18, '(41.91830362, -87.64597389)') ,
(609807, 'Lionel Hampton Fine & Performing Arts Elementary School', 'ES', '3434 W 77th St', 'Chicago', 'IL', 60652, '(773) 535-4030', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609807.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 42.0, NULL, NULL, 'Weak', 34.0, 'Weak', 28.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Average', 51.0, '95.00%', 94.9, '96.60%', '100.00%', 59.6, 40.0, 35.3, 28.6, 57.1, 56.1, 37.6, 33.8, 52.8, 54.6, 9.0, 37.2, 9.4, 9.4, -1.0, 0.0, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 658, 44, NULL, 1154672.983, 1853379.243, 41.75348432, -87.70875287, 70, 'ASHBURN', 18, 8, '(41.75348432, -87.70875287)') ,
(609834, 'Little Village Elementary School', 'ES', '2620 S Lawndale Ave', 'Chicago', 'IL', 60623, '(773) 534-1880', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609834.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 45.0, 'Average', 43.0, 'Average', 52.0, 'Strong', 66.0, 'Weak', 45.0, 'Average', 43.0, 'Average', 47.0, 'Average', 50.0, '96.70%', 3.2, '96.70%', '93.90%', 69.0, 56.0, 26.0, 25.1, 49.8, 52.2, 42.1, 32.6, 63.2, 56.3, 17.6, 19.4, 11.3, 11.4, 0.6, 1.5, 'Green', 'Green', 30.9, 66.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 842, 37, NULL, 1152090.217, 1886231.1730000002, 41.84368588, -87.71735516, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84368588, -87.71735516)') ,
(610325, 'Logandale Middle School', 'MS', '3212 W George St', 'Chicago', 'IL', 60618, '(773) 534-5350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610325.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 56.0, NULL, NULL, 'Weak', 32.0, 'Weak', 28.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 51.0, '95.10%', 5.1, '95.60%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 39.1, 31.0, 56.9, 55.1, 20.0, 22.6, 11.6, 7.8, -0.6, -0.3, 'Yellow', 'Yellow', 34.8, 37.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 941, 29, NULL, 1154387.612, 1919159.019, 41.933997600000005, -87.70804359, 21, 'AVONDALE', 35, 14, '(41.9339976, -87.70804359)') ,
(609809, 'Lorenz Brentano Math & Science Academy Elementary School', 'ES', '2723 N Fairfield Ave', 'Chicago', 'IL', 60647, '(773) 534-4100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609809.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 57.0, 'Average', 47.0, 'Average', 43.0, 'Average', 46.0, 'Weak', 30.0, 'Weak', 32.0, NULL, NULL, NULL, NULL, '94.70%', 9.2, '97.70%', '100.00%', 48.5, 42.6, 44.2, 30.8, 50.9, 56.1, 54.4, 30.9, 48.9, 46.6, 20.8, 28.3, 18.4, 8.2, -1.6, -1.0, 'Red', 'Red', 48.1, 3.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 455, 31, NULL, 1157602.725, 1918079.5369999998, 41.93097047, -87.69625758, 22, 'LOGAN SQUARE', 35, 14, '(41.93097047, -87.69625758)') ,
(609773, 'Louis A Agassiz Elementary School', 'ES', '2851 N Seminary Ave', 'Chicago', 'IL', 60657, '(773) 534-5725', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609773.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 53.0, NULL, NULL, 'Weak', 23.0, 'Weak', 21.0, 'Weak', NULL, NULL, NULL, 'Strong', 56.0, 'Average', 47.0, '95.20%', 32.6, '96.80%', '98.10%', 68.5, 39.0, 44.1, 49.5, 44.9, 49.5, 43.6, 50.0, 43.6, 43.6, 30.0, 53.3, 27.5, 27.5, 0.2, -0.9, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 432, 33, NULL, 1168495.925, 1919310.804, 41.93412004, -87.65619167, 6, 'LAKE VIEW', 44, 19, '(41.93412004, -87.65619167)') ,
(609774, 'Louisa May Alcott Elementary School', 'ES', '2625 N Orchard St', 'Chicago', 'IL', 60614, '(773) 534-5460', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609774.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', NULL, NULL, 'Strong', 66.0, NULL, NULL, NULL, NULL, 'Weak', 56.0, 'Average', 57.0, 'Strong', 60.0, 'Weak', 41.0, '96.60%', 3.7, '97.10%', '93.20%', 85.1, 75.1, 72.6, 76.9, 56.4, 55.2, 73.1, 70.7, 69.2, 59.8, 43.5, 47.8, 52.6, 46.1, 0.4, 0.3, 'Yellow', 'Yellow', 50.0, 73.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 481, 35, NULL, 1171189.134, 1917774.756, 41.92984626, -87.64633949, 7, 'LINCOLN PARK', 43, 19, '(41.92984626, -87.64633949)') ,
(610156, 'Louis Armstrong Math & Science Elementary School', 'ES', '5345 W Congress Pkwy', 'Chicago', 'IL', 60644, '(773) 534-6365', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610156.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, NULL, NULL, 'Average', 49.0, 'Strong', 62.0, 'Weak', NULL, NULL, NULL, 'Strong', 59.0, 'Average', 53.0, '92.60%', 83.4, '97.90%', '100.00%', NULL, NULL, 20.0, 16.5, 36.4, 31.6, 41.4, 34.5, 57.1, 60.7, NULL, NULL, 1.7, 7.8, -0.8, 0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 93, 36, NULL, 1140614.251, 1897119.257, 41.87378243, -87.75920297, 25, 'AUSTIN', 29, 15, '(41.87378243, -87.75920297)') ,
(610094, 'Louis Nettelhorst Elementary School', 'ES', '3252 N Broadway St', 'Chicago', 'IL', 60657, '(773) 534-5810', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610094.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 58.0, NULL, NULL, 'Weak', 34.0, 'Weak', 30.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.50%', 5.3, '95.90%', '97.90%', 85.4, 52.2, 67.6, 71.2, 61.6, 61.4, 56.8, 66.7, 45.1, 51.0, 29.6, 29.6, 38.2, 42.4, 0.2, 0.0, 'Yellow', 'Yellow', 53.6, 33.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 729, 33, NULL, 1171628.585, 1922072.58, 41.94162999, -87.64459776, 6, 'LAKE VIEW', 44, 19, '(41.94162999, -87.64459776)') ,
(610117, 'Louis Pasteur Elementary School', 'ES', '5825 S Kostner Ave', 'Chicago', 'IL', 60629, '(773) 535-2270', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610117.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 50.0, NULL, NULL, 'Average', 42.0, 'Weak', 38.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.00%', 5.0, '95.80%', '97.30%', 45.0, 19.2, 39.1, 43.4, 53.6, 57.4, 39.8, 50.8, 46.0, 51.5, 17.0, 28.1, 15.9, 13.7, -3.1, -0.6, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1173, 44, NULL, 1148072.928, 1865505.479, 41.78688968, -87.73262962, 62, 'WEST ELSDON', 13, 8, '(41.78688968, -87.73262962)') ,
(610237, 'Ludwig Van Beethoven Elementary School', 'ES', '25 W 47th St', 'Chicago', 'IL', 60609, '(773) 535-1480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610237.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 40.0, NULL, NULL, 'Average', 58.0, 'Average', 59.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Average', 52.0, '93.70%', 11.4, '94.70%', '98.70%', 58.3, 72.6, 18.7, 18.2, 43.1, 44.2, 33.8, 12.5, 54.5, 39.2, NULL, NULL, 16.5, 4.0, 0.7, -0.8, 'Green', 'Yellow', 39.1, 70.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 411, 42, NULL, 1176782.61, 1873820.331, 41.80910779, -87.62711401, 38, 'GRAND BOULEVARD', 3, 2, '(41.80910779, -87.62711401)') ,
(610108, 'Luke O Toole Elementary School', 'ES', '6550 S Seeley Ave', 'Chicago', 'IL', 60636, '(773) 535-9040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610108.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 5.0, NULL, NULL, 'Weak', 33.0, 'Average', 40.0, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Average', 47.0, '89.20%', 26.7, '94.60%', '100.00%', 63.5, NULL, 21.2, 17.1, 39.4, 49.6, 26.0, 21.1, 55.1, 50.3, 2.9, 13.4, 7.8, 4.6, -0.3, -0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 511, 43, NULL, 1163788.5520000001, 1860992.429, 41.77418952, -87.67513395, 67, 'WEST ENGLEWOOD', 15, 7, '(41.77418952, -87.67513395)') ,
(609818, 'Luther Burbank Elementary School', 'ES', '2035 N Mobile Ave', 'Chicago', 'IL', 60639, '(773) 534-3000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609818.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 37.0, NULL, NULL, 'Average', 42.0, 'Weak', 34.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 49.0, '95.20%', 9.8, '95.50%', '100.00%', 60.8, 45.2, 42.8, 34.9, 61.2, 63.8, 41.0, 34.4, 55.0, 54.5, 13.1, 17.2, 20.6, 11.3, -0.2, 0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1139, 29, NULL, 1134122.521, 1913042.0459999999, 41.91759334, -87.78266267, 19, 'BELMONT CRAGIN', 29, 25, '(41.91759334, -87.78266267)') ,
(609817, 'Lyman A Budlong Elementary School', 'ES', '2701 W Foster Ave', 'Chicago', 'IL', 60625, '(773) 534-2591', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609817.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, 'Average', 56.0, 'Weak', 33.0, 'Weak', 38.0, 'Weak', 44.0, 'Average', 41.0, 'Average', 50.0, 'Average', 49.0, '95.40%', 3.9, '95.70%', '98.70%', 77.1, 54.4, 44.9, 53.5, 48.1, 57.9, 36.3, 44.0, 44.7, 36.4, 9.1, 23.6, 26.6, 23.7, -1.7, -0.4, 'Red', 'Yellow', 48.1, 16.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 864, 31, NULL, 1157387.173, 1934388.0869999998, 41.97572652, -87.69660443, 4, 'LINCOLN SQUARE', 40, 20, '(41.97572652, -87.69660443)') ,
(610205, 'Lyman Trumbull Elementary School', 'ES', '5200 N Ashland Ave', 'Chicago', 'IL', 60640, '(773) 534-2430', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610205.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 56.0, NULL, NULL, 'Average', 47.0, 'Weak', 25.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Strong', 55.0, '94.30%', 17.8, '95.20%', '97.70%', 55.1, 25.7, 21.6, 29.0, 37.5, 38.0, 39.0, 42.2, 30.4, 31.8, 21.1, 36.8, 14.2, 13.5, -1.8, -1.5, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 438, 32, NULL, 1164684.835, 1934622.0119999999, 41.97621644, -87.66976159999999, 77, 'EDGEWATER', 40, 20, '(41.97621644, -87.6697616)') ,
(610369, 'Mahalia Jackson Elementary School', 'ES', '917 W 88th St', 'Chicago', 'IL', 60620, '(773) 535-3341', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610369.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 38.0, NULL, NULL, 'Average', 54.0, 'Strong', 66.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Average', 51.0, '92.50%', 35.7, '94.60%', '96.60%', 57.8, 44.4, 14.8, 24.4, 47.8, 44.6, 24.4, 31.1, 54.2, 48.1, 6.7, 6.9, 7.6, 9.2, -0.3, 0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 323, 49, NULL, 1171645.6809999999, 1846444.0159999998, 41.73409818, -87.64675679, 71, 'AUBURN GRESHAM', 21, 22, '(41.73409818, -87.64675679)') ,
(610197, 'Mancel Talcott Elementary School', 'ES', '1840 W Ohio St', 'Chicago', 'IL', 60622, '(773) 534-7130', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610197.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 63.0, 'Average', 58.0, 'Average', 59.0, 'Strong', 71.0, 'Weak', 53.0, 'Average', 55.0, 'Average', 51.0, 'Average', 49.0, '95.80%', 3.2, '96.90%', '100.00%', 68.2, NULL, 43.2, 27.0, 49.6, 55.0, 70.3, 53.5, 68.6, 62.7, 17.0, 34.0, 25.6, 20.8, 0.2, 0.8, 'Yellow', 'Yellow', 37.3, 57.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 580, 35, NULL, 1163847.473, 1904082.62, 41.89243235, -87.67370499, 24, 'WEST TOWN', 1, 13, '(41.89243235, -87.67370499)') ,
(609722, 'Manley Career Academy High School', 'HS', '2935 W Polk St', 'Chicago', 'IL', 60612, '(773) 534-6900', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609722.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 41.0, 'Weak', 39.0, 'Average', 43.0, 'Weak', 31.0, 'Weak', 19.0, 'Weak', 32.0, NULL, NULL, NULL, NULL, '66.80%', 19.7, '95.40%', '98.40%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.2, 11.9, 13.3, 13.0, 0.8, 13.8, 0.5, 6.7, 49.0, 51.9, 599, 37, 59.3, 1156776.858, 1896186.78, 41.87091163, -87.69988652, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87091163, -87.69988652)') ,
(609872, 'Manuel Perez Elementary School', 'ES', '1241 W 19th St', 'Chicago', 'IL', 60608, '(773) 534-7650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609872.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'Yes', 'Average', 45.0, NULL, NULL, 'Average', 42.0, 'Average', 45.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Average', 50.0, '95.90%', 7.3, '96.60%', '100.00%', 67.7, NULL, 37.9, 33.6, 61.1, 62.3, 45.9, 26.7, 71.8, 51.8, 14.0, 14.0, 14.8, 8.7, 0.0, 1.1, 'Yellow', 'Green', 35.6, 18.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 426, 39, NULL, 1168231.034, 1890840.4219999998, 41.85600113, -87.6579887, 31, 'LOWER WEST SIDE', 25, 12, '(41.85600113, -87.6579887)') ,
(610128, 'Marcus Moziah Garvey Elementary School', 'ES', '10309 S Morgan St', 'Chicago', 'IL', 60643, '(773) 535-2763', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610128.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 50.0, NULL, NULL, 'Average', 50.0, 'Average', 48.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Average', 51.0, '94.00%', 20.5, '96.00%', '100.00%', 69.3, 62.7, 40.4, 47.4, 60.2, 59.8, 55.4, 52.9, 73.1, 63.9, 10.3, 48.7, 22.4, 18.5, 0.1, 0.9, 'Yellow', 'Green', 30.8, 69.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 356, 49, NULL, 1171511.646, 1836420.6709999999, 41.7065956, -87.64754034, 73, 'WASHINGTON HEIGHTS', 34, 22, '(41.7065956, -87.64754034)') ,
(610544, 'Mariano Azuela Elementary School', 'ES', '4707 W Marquette Rd', 'Chicago', 'IL', 60629, '(773) 535-7395', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610544.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 54.0, 'Very Strong', 96.0, 'Strong', 61.0, 'Very Strong', 81.0, 'Weak', 73.0, 'Very Strong', 84.0, 'Average', 47.0, 'Strong', 58.0, '95.50%', 19.7, '0.00%', '98.70%', 69.5, 42.9, 28.7, 26.1, 48.2, 32.1, 46.9, 34.9, 55.2, 57.4, NULL, NULL, 13.0, 11.9, -1.3, 0.2, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 873, 44, NULL, 1146123.64, 1859720.388, 41.77105158, -87.73992325, 65, 'WEST LAWN', 13, 8, '(41.77105158, -87.73992325)') ,
(610017, 'Maria Saucedo Elementary Scholastic Academy', 'ES', '2850 W 24th Blvd', 'Chicago', 'IL', 60623, '(773) 534-1770', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610017.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 60.0, NULL, NULL, 'Average', 55.0, 'Strong', 64.0, 'Weak', NULL, NULL, NULL, 'Weak', 43.0, 'Weak', 42.0, '96.80%', 19.4, '96.90%', '98.90%', 72.3, 35.3, 38.6, 35.8, 54.9, 57.5, 51.8, 39.8, 56.3, 52.8, 15.3, 27.0, 24.3, 16.0, 2.0, 1.7, 'Green', 'Green', 10.8, 91.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1306, 39, NULL, 1157546.774, 1887831.632, 41.84796859, -87.69728712, 30, 'SOUTH LAWNDALE', 12, 10, '(41.84796859, -87.69728712)') ,
(609756, 'Marie Sklodowska Curie Metropolitan High School', 'HS', '4959 S Archer Ave', 'Chicago', 'IL', 60632, '(773) 535-2100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609756.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 43.0, 'Weak', 35.0, 'Weak', 37.0, 'Average', 45.0, 'Weak', 33.0, 'Weak', 39.0, NULL, NULL, NULL, NULL, '85.00%', 9.1, '95.00%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.2, 14.2, 15.6, 15.1, 0.9, 16.9, 1.3, 22.9, 76.1, 61.3, 3320, 37, 61.0, 1150928.132, 1871413.2030000002, 41.80304615, -87.72200673, 57, 'ARCHER HEIGHTS', 14, 8, '(41.80304615, -87.72200673)') ,
(610502, 'Marine Military Math and Science Academy', 'HS', '145 S Campbell Ave', 'Chicago', 'IL', 60612, '(773) 534-7818', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610502.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 41.0, 'Average', 49.0, 'Weak', 38.0, 'Weak', 34.0, 'Weak', 53.0, 'Weak', 35.0, 'Strong', 54.0, 'Average', 49.0, '91.30%', 4.5, '95.30%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.0, 15.0, 15.9, 15.3, 1.3, 17.7, 1.8, 37.7, NULL, NULL, 366, 38, 86.3, 1159815.389, 1899102.4919999999, 41.87885054, -87.68865058, 28, 'NEAR WEST SIDE', 2, 11, '(41.87885054, -87.68865058)') ,
(610081, 'Mark Sheridan Elementary Math & Science Academy', 'ES', '533 W 27th St', 'Chicago', 'IL', 60616, '(773) 534-9120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610081.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 74.0, 'Average', 59.0, 'Average', 53.0, 'Strong', 63.0, 'Weak', 59.0, 'Strong', 63.0, 'Strong', 55.0, 'Average', 51.0, '96.80%', 1.3, '97.00%', '95.90%', 85.5, NULL, 62.7, 74.0, 72.3, 77.4, 59.4, 62.4, 67.8, 61.3, 35.9, 42.2, 47.2, 38.3, 1.5, 1.5, 'Green', 'Green', 50.0, 65.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 535, 40, NULL, 1172807.818, 1886639.3080000002, 41.84437293, -87.64131399, 60, 'BRIDGEPORT', 11, 9, '(41.84437293, -87.64131399)') ,
(610177, 'Mark Skinner Elementary School', 'ES', '1260 W Adams St', 'Chicago', 'IL', 60607, '(773) 534-7790', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610177.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 87.0, NULL, NULL, 'Average', 48.0, 'Average', 53.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '97.10%', 0.0, '94.90%', '96.70%', NULL, NULL, 92.5, 96.8, 61.5, 59.1, 89.4, 97.8, 56.5, 56.8, 81.6, 91.8, 77.2, 73.0, 1.0, 0.6, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 784, 38, NULL, 1167788.281, 1899265.832, 41.87913069, -87.65937102, 28, 'NEAR WEST SIDE', 27, 12, '(41.87913069, -87.65937102)') ,
(610206, 'Mark Twain Elementary School', 'ES', '5134 S Lotus Ave', 'Chicago', 'IL', 60638, '(773) 535-2290', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610206.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, 'Strong', 76.0, 'Average', 46.0, 'Average', 44.0, 'Weak', 84.0, 'Strong', 71.0, 'Strong', 60.0, 'Strong', 60.0, '96.30%', 5.3, '97.80%', '98.30%', 84.6, 61.1, 44.4, 48.5, 53.2, 63.3, 49.1, 58.1, 62.2, 55.8, 32.1, 43.8, 32.3, 19.8, 0.2, -0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1161, 44, NULL, 1140861.0829999999, 1869888.283, 41.79905206, -87.75896524, 56, 'GARFIELD RIDGE', 23, 8, '(41.79905206, -87.75896524)') ,
(610053, 'Marquette Elementary School', 'ES', '6550 S Richmond St', 'Chicago', 'IL', 60629, '(773) 535-9260', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610053.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 15.0, NULL, NULL, 'Very Weak', 17.0, 'Very Weak', 17.0, 'Weak', NULL, NULL, NULL, 'Weak', 43.0, 'Weak', 46.0, '93.10%', 64.6, '95.30%', '99.30%', 54.4, 21.8, 22.6, 16.2, 42.0, 40.0, 14.2, 16.2, 31.2, 40.2, 4.2, 13.2, 6.8, 4.6, -1.4, -0.6, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1390, 44, NULL, 1157806.476, 1860844.388, 41.77390682, -87.6970673, 66, 'CHICAGO LAWN', 15, 8, '(41.77390682, -87.6970673)') ,
(610152, 'Martha Ruggles Elementary School', 'ES', '7831 S Prairie Ave', 'Chicago', 'IL', 60619, '(773) 535-3085', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610152.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 43.0, NULL, NULL, 'Weak', 37.0, 'Average', 41.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Weak', 46.0, '93.90%', 30.9, '96.70%', '98.00%', NULL, 49.3, 23.6, 21.9, 55.0, 57.6, 26.8, 24.4, 51.3, 56.5, 8.8, 26.5, 8.7, 5.5, 0.2, 1.0, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 417, 46, NULL, 1179432.46, 1853011.9219999998, 41.75194729, -87.61802983, 69, 'GREATER GRAND CROSSING', 6, 6, '(41.75194729, -87.61802983)') ,
(610154, 'Martin A Ryerson Elementary School', 'ES', '646 N Lawndale Ave', 'Chicago', 'IL', 60624, '(773) 534-6700', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610154.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 40.0, 'Weak', 32.0, 'Average', 48.0, 'Average', 43.0, 'Weak', 41.0, 'Weak', 37.0, 'Weak', 45.0, 'Average', 48.0, '95.50%', 95.1, '96.80%', '100.00%', 63.6, 48.6, 33.3, 19.2, 48.9, 72.9, 59.2, 26.4, 93.3, 67.5, 13.6, 11.4, 23.1, 17.1, 2.8, 0.2, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 445, 34, NULL, 1151558.259, 1904275.283, 41.89321142, -87.71883334, 23, 'HUMBOLDT PARK', 27, 11, '(41.89321142, -87.71883334)') ,
(610539, 'Marvin Camras Elementary School', 'ES', '3000 N Mango Ave', 'Chicago', 'IL', 60634, '(773) 534-2960', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610539.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 54.0, 'Average', 58.0, 'Weak', 37.0, 'Average', 41.0, 'Weak', 83.0, 'Very Strong', 88.0, 'Average', 51.0, 'Strong', 55.0, '95.10%', 4.3, '0.00%', '97.70%', 47.5, 39.9, 30.8, 27.8, 47.8, 38.6, 41.1, 38.5, 49.2, 55.4, 15.3, 30.5, 11.2, 12.0, -1.8, -0.3, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 826, 30, NULL, 1137482.2959999999, 1919394.9, 41.93496641, -87.77016525, 19, 'BELMONT CRAGIN', 30, 25, '(41.93496641, -87.77016525)') ,
(610355, 'Mary E Courtenay Elementary Language Arts Center', 'ES', '1726 W Berteau Ave', 'Chicago', 'IL', 60613, '(773) 534-5790', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610355.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Very Strong', 99.0, NULL, NULL, 'Very Strong', 95.0, 'Very Strong', 80.0, 'Weak', NULL, NULL, NULL, 'Strong', 59.0, 'Average', 49.0, '94.90%', 10.8, '94.20%', '100.00%', 90.5, 92.9, 51.0, 38.5, 62.0, 59.2, 47.7, 57.8, 45.5, 37.8, 16.7, 55.6, 21.2, 28.6, -1.2, -0.8, 'Red', 'Yellow', 19.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 257, 33, NULL, 1164156.393, 1927985.5380000002, 41.9580169, -87.6718933, 6, 'LAKE VIEW', 47, 19, '(41.9580169, -87.6718933)') ,
(610312, 'Mary E McDowell Elementary School', 'ES', '1419 E 89th St', 'Chicago', 'IL', 60619, '(773) 535-6404', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610312.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', NULL, NULL, 'Average', 49.0, NULL, NULL, NULL, NULL, 'Weak', 74.0, 'Average', 53.0, NULL, NULL, NULL, NULL, '95.50%', 3.1, '96.10%', '100.00%', 79.3, 53.8, 39.0, 54.2, 50.8, 64.4, NULL, NULL, NULL, NULL, NULL, NULL, 21.0, 16.1, 0.0, -0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 188, 47, NULL, 1187249.1409999998, 1846211.629, 41.733104700000005, -87.58960096, 48, 'CALUMET HEIGHTS', 8, 4, '(41.7331047, -87.58960096)') ,
(610127, 'Mary Gage Peterson Elementary School', 'ES', '5510 N Christiana Ave', 'Chicago', 'IL', 60625, '(773) 534-5070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610127.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 62.0, NULL, NULL, 'Average', 49.0, 'Average', 58.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Average', 48.0, '95.50%', 10.3, '96.70%', '100.00%', 72.7, 37.7, 53.1, 47.1, 51.6, 52.1, 59.7, 52.2, 49.2, 51.9, 34.0, 44.7, 35.5, 27.5, -0.3, 0.0, 'Yellow', 'Yellow', 33.3, 83.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 860, 31, NULL, 1153016.828, 1936489.312, 41.98158041, -87.71261988, 13, 'NORTH PARK', 40, 17, '(41.98158041, -87.71261988)') ,
(610046, 'Mary Lyon Elementary School', 'ES', '2941 N McVicker Ave', 'Chicago', 'IL', 60634, '(773) 534-3120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610046.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, NULL, NULL, 'Average', 42.0, 'Weak', 38.0, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Average', 51.0, '95.90%', 3.7, '97.00%', '99.30%', 72.0, NULL, 42.9, 51.1, 68.1, 70.2, 61.6, 59.0, 62.9, 55.6, 37.6, 36.4, 29.5, 23.7, -0.3, 0.4, 'Yellow', 'Yellow', 24.3, 100.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1423, 30, NULL, 1135587.211, 1919095.574, 41.93417898, -87.77713699, 19, 'BELMONT CRAGIN', 29, 25, '(41.93417898, -87.77713699)') ,
(609888, 'Mary Mapes Dodge Elementary Renaissance Academy', 'ES', '2651 W Washington Blvd', 'Chicago', 'IL', 60612, '(773) 534-6640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609888.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 57.0, 'Strong', 61.0, 'Strong', 68.0, 'Strong', 73.0, 'Weak', 77.0, 'Very Strong', 83.0, 'Strong', 56.0, 'Strong', 57.0, '94.40%', 12.7, '97.10%', '100.00%', NULL, NULL, 51.4, 49.7, 53.1, 55.2, 53.2, 54.0, 59.0, 51.1, 12.0, 22.0, 29.4, 16.0, 0.4, -1.6, 'Yellow', 'Red', 43.1, 19.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 434, 38, NULL, 1158615.705, 1900540.3059999999, 41.88282068, -87.6930162, 27, 'EAST GARFIELD PARK', 2, 13, '(41.88282068, -87.6930162)') ,
(610365, 'Mary McLeod Bethune Elementary School', 'ES', '3030 W Arthington St', 'Chicago', 'IL', 60612, '(773) 534-6890', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610365.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 53.0, 'Average', 47.0, 'Average', 53.0, 'Weak', 39.0, 'Weak', 31.0, 'Average', 46.0, 'Strong', 56.0, 'Strong', 58.0, '93.30%', 25.8, '97.60%', '98.00%', 50.0, NULL, 13.8, 13.8, 44.0, 49.1, 22.1, 24.1, 57.0, 46.0, 6.7, 16.7, 6.1, 4.4, 1.3, -0.4, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 362, 37, NULL, 1156159.096, 1895921.1430000002, 41.87019518, -87.70216173, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87019518, -87.70216173)') ,
(610535, 'Mason High School', 'HS', '4217 W 18th St', 'Chicago', 'IL', 60623, '(773) 534-1530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610535.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', NULL, 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 50.0, NULL, NULL, 'Average', 59.0, 'Average', 59.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 47.0, '90.40%', 44.3, '0.00%', '72.70%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.6, 13.5, NULL, 13.9, 1.3, NULL, NULL, NULL, NULL, NULL, 48, 36, 40.9, 1148477.255, 1891003.4409999999, 41.85685194, -87.73049117, 29, 'NORTH LAWNDALE', 24, 10, '(41.85685194, -87.73049117)') ,
(610240, 'Matthew A Henson Elementary School', 'ES', '1326 S Avers Ave', 'Chicago', 'IL', 60623, '(773) 534-1804', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610240.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 37.0, NULL, NULL, 'Weak', 34.0, 'Weak', 22.0, 'Weak', NULL, NULL, NULL, 'Strong', 55.0, 'Strong', 63.0, '90.90%', 13.2, '95.70%', '97.90%', 70.1, 46.8, 19.0, 8.6, 48.6, 52.1, 15.7, 17.1, 30.3, 34.7, 3.6, 32.1, 3.5, 1.7, -0.8, -1.9, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 271, 36, NULL, 1150885.71, 1893465.136, 41.86356038, -87.72158636, 29, 'NORTH LAWNDALE', 24, 10, '(41.86356038, -87.72158636)') ,
(609935, 'Matthew Gallistel Elementary Language Academy', 'ES', '10347 S Ewing Ave', 'Chicago', 'IL', 60617, '(773) 535-6540', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609935.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 49.0, 'Average', 45.0, 'Weak', 37.0, 'Weak', 32.0, 'Weak', 49.0, 'Average', 40.0, 'Average', 50.0, 'Weak', 44.0, '95.10%', 5.9, '96.30%', '100.00%', 66.7, 53.6, 40.1, 23.7, 48.7, 62.2, 46.7, 34.0, 62.8, 58.6, 17.4, 27.7, 17.8, 12.9, -0.6, 0.9, 'Red', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1408, 47, NULL, 1202203.353, 1836767.747, 41.70682232, -87.53513861, 52, 'EAST SIDE', 10, 4, '(41.70682232, -87.53513861)') ,
(610362, 'Medgar Evers Elementary School', 'ES', '9811 S Lowe Ave', 'Chicago', 'IL', 60628, '(773) 535-2565', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610362.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Weak', 27.0, NULL, NULL, 'Weak', 35.0, 'Average', 49.0, 'Weak', NULL, NULL, NULL, 'Strong', 56.0, 'Strong', 55.0, '94.90%', 18.1, '96.10%', '100.00%', 66.4, 46.7, 48.3, 39.7, 59.1, 71.9, 67.2, 43.9, 54.1, 51.2, 17.6, 21.6, 17.3, 14.8, 1.0, 0.4, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 367, 49, NULL, 1173743.636, 1839743.2980000002, 41.7156643, -87.63926881, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.7156643, -87.63926881)') ,
(609928, 'Melville W Fuller Elementary School', 'ES', '4214 S Saint Lawrence Ave', 'Chicago', 'IL', 60653, '(773) 535-1687', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609928.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 22.0, NULL, NULL, 'Weak', 32.0, 'Weak', 33.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Strong', 55.0, '91.80%', 40.2, '93.90%', '100.00%', 40.9, 21.4, 10.4, 2.4, 50.0, 40.0, 6.8, 10.9, 22.4, 34.8, 6.3, 12.5, 3.6, 1.4, -1.3, -1.3, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 241, 40, NULL, 1180924.721, 1877097.4640000002, 41.8180061, -87.61182077, 38, 'GRAND BOULEVARD', 4, 2, '(41.8180061, -87.61182077)') ,
(610055, 'Michael Faraday Elementary School', 'ES', '3250 W Monroe St', 'Chicago', 'IL', 60624, '(773) 534-6670', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610055.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, NULL, NULL, 'Strong', 65.0, 'Strong', 66.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 50.0, '93.30%', 39.4, '94.60%', '100.00%', 56.4, 34.0, 34.4, 17.2, 53.2, 73.3, 59.8, 22.9, 80.6, 63.5, 17.1, 17.1, 17.9, 7.0, 1.7, -1.1, 'Green', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 194, 38, NULL, 1154521.277, 1899387.886, 41.87974117, -87.70808199, 27, 'EAST GARFIELD PARK', 28, 11, '(41.87974117, -87.70808199)') ,
(609832, 'Michael M Byrne Elementary School', 'ES', '5329 S Oak Park Ave', 'Chicago', 'IL', 60638, '(773) 535-2170', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609832.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Probation', 'Level 3', 'No', 'Strong', 63.0, 'Strong', 61.0, 'Weak', 35.0, 'Weak', 38.0, 'Weak', 59.0, 'Average', 57.0, 'Average', 49.0, 'Weak', 45.0, '94.60%', 51.8, '95.80%', '94.00%', 74.3, 69.6, 50.2, 42.9, 43.7, 67.6, 44.9, 48.1, 45.8, 36.5, 21.5, 34.2, 23.1, 24.8, -1.2, -1.1, 'Red', 'Red', 19.3, 56.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 705, 44, NULL, 1131997.021, 1868365.169, 41.79503029, -87.79150778, 56, 'GARFIELD RIDGE', 23, 8, '(41.79503029, -87.79150778)') ,
(610244, 'Michele Clark Academic Prep Magnet High School', 'HS', '5101 W Harrison St', 'Chicago', 'IL', 60644, '(773) 534-6250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610244.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Weak', 36.0, '79.10%', 24.4, '95.40%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 19.4, 22.8, 36.5, 52.7, 3.6, 10.7, 1.7, 2.6, -1.2, -0.7, 'Red', 'Yellow', NULL, NULL, 14.0, 13.3, 14.4, 14.5, 0.5, 15.7, 1.3, 13.0, 87.4, 60.4, 726, 36, 78.2, 1142209.317, 1896792.791, 41.87285714, -87.75335467, 25, 'AUSTIN', 24, 15, '(41.87285714, -87.75335467)') ,
(610208, 'Mildred I Lavizzo Elementary School', 'ES', '138 W 109th St', 'Chicago', 'IL', 60628, '(773) 535-5300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610208.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 15.0, 'Weak', 35.0, 'Average', 41.0, 'Average', 46.0, 'Weak', 39.0, 'Weak', 30.0, 'Average', 48.0, 'Average', 51.0, '91.50%', 39.1, '95.00%', '95.70%', 49.6, 33.3, 5.7, 15.6, 35.7, 39.8, 24.3, 30.1, 65.6, 68.9, 4.3, 17.0, 6.1, 4.6, 0.6, 0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 394, 48, NULL, 1177114.736, 1832710.465, 41.69628998, -87.62713349, 49, 'ROSELAND', 34, 5, '(41.69628998, -87.62713349)') ,
(609830, 'Milton Brunson Math & Science Specialty Elementary School', 'ES', '932 N Central Ave', 'Chicago', 'IL', 60651, '(773) 534-6025', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609830.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 30.0, NULL, NULL, 'Weak', 30.0, 'Average', 45.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Strong', 55.0, '91.30%', 16.6, '95.00%', '100.00%', 64.4, 43.9, 21.1, 18.7, 44.4, 36.4, 22.6, 19.0, 33.6, 45.2, 3.4, 8.6, 8.8, 5.4, -0.3, -0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 658, 36, NULL, 1138802.71, 1905814.957, 41.89767761, -87.76564297, 25, 'AUSTIN', 29, 15, '(41.89767761, -87.76564297)') ,
(610011, 'Minnie Mars Jamieson Elementary School', 'ES', '5650 N Mozart St', 'Chicago', 'IL', 60659, '(773) 534-2395', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610011.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, 'Average', 50.0, 'Strong', 62.0, 'Strong', 66.0, 'Weak', 55.0, 'Average', 41.0, 'Average', 52.0, 'Average', 53.0, '95.70%', 8.1, '96.20%', '100.00%', 79.1, NULL, 63.8, 58.0, 70.1, 72.1, 57.6, 67.2, 66.1, 62.7, 30.5, 35.4, 37.5, 23.7, 0.3, 0.6, 'Yellow', 'Yellow', 38.1, 42.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 811, 32, NULL, 1156286.231, 1937618.645, 41.98461372, -87.70056525, 2, 'WEST RIDGE', 40, 20, '(41.98461372, -87.70056525)') ,
(610018, 'Miriam G Canter Middle School', 'MS', '4959 S Blackstone Ave', 'Chicago', 'IL', 60615, '(773) 535-1410', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610018.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 57.0, 'Average', 55.0, 'Weak', 31.0, 'Weak', 37.0, 'Weak', 61.0, 'Strong', 62.0, NULL, NULL, NULL, NULL, '91.40%', 3.4, '95.70%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 33.3, 39.6, 53.9, 46.5, 17.6, 31.5, 21.6, 7.8, -0.2, -1.2, 'Yellow', 'Red', 24.1, 76.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 222, 46, NULL, 1186808.1909999999, 1872169.101, 41.80434474, -87.59039494, 39, 'KENWOOD', 4, 2, '(41.80434474, -87.59039494)') ,
(609725, 'Morgan Park High School', 'HS', '1744 W Pryor Ave', 'Chicago', 'IL', 60643, '(773) 535-2550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609725.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 40.0, NULL, NULL, 'Weak', 25.0, 'Weak', 38.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '84.20%', 11.1, '94.60%', '99.50%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 60.0, 80.0, 52.0, 38.2, -1.8, -1.0, 'Red', 'Yellow', 98.5, 64.1, 14.7, 14.6, 15.6, 15.9, 1.2, 17.8, 2.2, 33.6, 80.0, 78.5, 1584, 49, 73.3, 1166609.57, 1831085.382, 41.69206041, -87.66564309, 75, 'MORGAN PARK', 19, 22, '(41.69206041, -87.66564309)') ,
(610257, 'Morton School of Excellence', 'ES', '431 N Troy St', 'Chicago', 'IL', 60612, '(773) 534-6791', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610257.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 1', 'No', 'Average', 53.0, 'Average', 49.0, 'Strong', 66.0, 'Strong', 63.0, 'Weak', 86.0, 'Very Strong', 95.0, 'Average', 52.0, 'Strong', 59.0, '93.60%', 73.9, '97.10%', '100.00%', NULL, NULL, 43.2, 36.8, 57.9, 62.2, 38.6, 36.0, 65.3, 49.0, 7.1, 7.1, 16.3, 8.4, 2.8, 0.5, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 349, 34, NULL, 1155338.246, 1902872.4030000002, 41.88928666, -87.7049885, 23, 'HUMBOLDT PARK', 27, 13, '(41.88928666, -87.7049885)') ,
(610075, 'Moses Montefiore Special Elementary School', 'ES', '1310 S Ashland Ave', 'Chicago', 'IL', 60608, '(773) 534-7825', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610075.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', NULL, 'Track_E', 'Not Applicable', 'Level 3', 'No', 'Weak', 28.0, NULL, NULL, 'Average', 42.0, 'Very Weak', 14.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Strong', 54.0, '80.40%', 251.6, '93.50%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 0.0, 2.9, 35.5, 32.3, 0.0, 0.0, 0.0, 0.0, -0.6, -0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 21, 38, NULL, 1165892.869, 1893999.3069999998, 41.86471954, -87.66648084, 28, 'NEAR WEST SIDE', 2, 12, '(41.86471954, -87.66648084)') ,
(610082, 'Mount Greenwood Elementary School', 'ES', '10841 S Homan Ave', 'Chicago', 'IL', 60655, '(773) 535-2786', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610082.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 84.0, NULL, NULL, 'Average', 58.0, 'Weak', 38.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '96.00%', 2.9, '96.10%', '98.50%', 85.8, NULL, 64.6, 71.2, 66.9, 63.1, 72.0, 71.6, 63.1, 51.6, 41.3, 65.3, 39.5, 39.2, -0.1, 0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 909, 49, NULL, 1155639.216, 1832346.852, 41.69574845, -87.70577383, 74, 'MOUNT GREENWOOD', 19, 22, '(41.69574845, -87.70577383)') ,
(610086, 'Mount Vernon Elementary School', 'ES', '10540 S Morgan St', 'Chicago', 'IL', 60643, '(773) 535-2825', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610086.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 31.0, 'Weak', 26.0, 'Average', 42.0, 'Average', 44.0, 'Weak', 14.0, 'Very Weak', 19.0, 'Average', 47.0, 'Average', 52.0, '93.30%', 76.6, '94.10%', '100.00%', 41.2, 36.5, 18.8, 17.1, 37.5, 52.5, 29.3, 40.5, 74.5, 75.0, 0.0, 4.3, 14.6, 9.8, 1.0, 0.7, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 290, 49, NULL, 1171484.32, 1834733.756, 41.70196704, -87.64768959, 73, 'WASHINGTON HEIGHTS', 34, 22, '(41.70196704, -87.64768959)') ,
(610385, 'Multicultural Academy of Scholarship', 'HS', '3120 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 535-4242', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610385.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 48.0, NULL, NULL, 'Average', 47.0, 'Average', 42.0, 'Weak', NULL, NULL, NULL, 'Strong', 56.0, 'Average', 52.0, '89.20%', 22.4, '96.70%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.0, 13.0, 14.4, 14.1, 1.1, 15.6, 1.2, 21.1, 58.0, 58.5, 324, 37, 78.7, 1147521.3020000001, 1883405.128, 41.83601953, -87.73419465, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83601953, -87.73419465)') ,
(609806, 'Myra Bradwell Communications Arts & Sciences Elementary School', 'ES', '7736 S Burnham Ave', 'Chicago', 'IL', 60649, '(773) 535-6600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609806.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Weak', 35.0, 'Average', 41.0, 'Average', 50.0, 'Weak', 52.0, 'Average', 54.0, 'Average', 49.0, 'Strong', 54.0, '92.10%', 75.9, '96.40%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1.4, 10.8, 6.3, 4.3, 1.2, -1.2, 'Green', 'Red', NULL, NULL, 11.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 775, 46, NULL, 1196028.811, 1854039.898, 41.7543734, -87.55717881, 43, 'SOUTH SHORE', 7, 4, '(41.7543734, -87.55717881)') ,
(609952, 'Nathanael Greene Elementary School', 'ES', '3525 S Honore', 'Chicago', 'IL', 60609, '(773) 535-4560', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609952.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'Yes', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Strong', 54.0, '95.50%', 3.8, '96.20%', '98.60%', NULL, NULL, 43.2, 44.7, 58.3, 49.7, NULL, NULL, NULL, NULL, NULL, NULL, 23.2, 20.2, -1.2, 0.2, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 631, 39, NULL, 1164636.2759999998, 1881214.535, 41.82966356, -87.67145551, 59, 'MCKINLEY PARK', 11, 9, '(41.82966356, -87.67145551)') ,
(609960, 'Nathan Hale Elementary School', 'ES', '6140 S Melvina', 'Chicago', 'IL', 60638, '(773) 535-2265', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609960.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, 'Strong', 63.0, 'Average', 51.0, 'Average', 56.0, 'Weak', 75.0, 'Strong', 65.0, 'Strong', 55.0, 'Average', 52.0, '94.50%', 9.8, '96.20%', '96.50%', 53.8, 34.4, 43.3, 49.0, 55.2, 56.2, 35.1, 54.6, 53.6, 55.1, 12.5, 29.5, 19.3, 15.6, -0.5, 0.4, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 872, 44, NULL, 1136092.7920000001, 1862988.841, 41.78020486, -87.77661589, 64, 'CLEARING', 23, 8, '(41.78020486, -87.77661589)') ,
(610134, 'Nathaniel Pope Elementary School', 'ES', '1852 S Albany Ave', 'Chicago', 'IL', 60623, '(773) 534-1795', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610134.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 48.0, NULL, NULL, 'Strong', 73.0, 'Very Strong', 83.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '91.60%', 54.2, '95.80%', '93.50%', 50.0, NULL, 19.3, 17.6, 40.0, 60.9, 33.3, 21.9, 64.8, 70.2, 0.0, 10.5, 4.5, 6.8, 0.2, -0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 200, 37, NULL, 1155971.872, 1890640.0680000002, 41.85570712, -87.70299144, 29, 'NORTH LAWNDALE', 24, 10, '(41.85570712, -87.70299144)') ,
(610348, 'Nathan R Goldblatt Elementary School', 'ES', '4257 W Adams St', 'Chicago', 'IL', 60624, '(773) 534-6860', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610348.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 49.0, NULL, NULL, 'Very Strong', 80.0, 'Very Strong', 99.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Strong', 58.0, '92.90%', 57.7, '95.60%', '100.00%', 66.7, 56.0, 44.7, 17.2, 56.8, 69.3, 55.1, 23.5, 67.7, 31.3, NULL, NULL, 11.7, 7.5, 0.7, -1.1, 'Green', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 268, 36, NULL, 1147813.932, 1898625.5969999998, 41.87778081, -87.73273021, 26, 'WEST GARFIELD PARK', 28, 11, '(41.87778081, -87.73273021)') ,
(609876, 'Nathan S Davis Elementary School', 'ES', '3014 W 39th Pl', 'Chicago', 'IL', 60632, '(773) 535-4540', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609876.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 43.0, NULL, NULL, 'Average', 41.0, 'Average', 46.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 50.0, '95.10%', 2.4, '95.90%', '94.00%', 42.1, NULL, 35.4, 20.7, 44.4, 55.3, 39.2, 24.3, 61.4, 48.2, 18.6, 26.7, 16.2, 9.2, 0.5, 1.0, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1032, 39, NULL, 1156815.412, 1878318.629, 41.821878600000005, -87.70022859, 58, 'BRIGHTON PARK', 14, 9, '(41.8218786, -87.70022859)') ,
(610231, 'National Teachers Elementary Academy', 'ES', '55 W Cermak Rd', 'Chicago', 'IL', 60616, '(773) 534-9970', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610231.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, 'Strong', 70.0, 'Strong', 65.0, 'Strong', 60.0, 'Weak', 59.0, 'Average', 46.0, 'Strong', 56.0, 'Strong', 61.0, '94.70%', 9.2, '96.30%', '100.00%', 56.8, 36.7, 45.6, 48.5, 62.5, 70.4, 25.6, 42.9, 57.8, 61.5, 10.0, 10.0, 16.5, 12.7, 0.4, 1.1, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 446, 40, NULL, 1176061.392, 1889726.757, 41.85277255, -87.62928114, 33, 'NEAR SOUTH SIDE', 3, 1, '(41.85277255, -87.62928114)') ,
(609692, 'Neal F Simeon Career Academy High School', 'HS', '8147 S Vincennes Ave', 'Chicago', 'IL', 60620, '(773) 535-3200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609692.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 52.0, NULL, NULL, 'Weak', 33.0, 'Average', 42.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '78.60%', 2.1, '94.30%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.3, 14.1, 15.3, 14.8, 0.5, 16.2, 0.9, 14.8, 75.2, 74.0, 1535, 45, 68.5, 1174719.074, 1850730.1919999998, 41.74579217, -87.63537, 44, 'CHATHAM', 21, 6, '(41.74579217, -87.63537)') ,
(610085, 'Near North Elementary School', 'ES', '739 N Ada St', 'Chicago', 'IL', 60642, '(773) 534-7845', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610085.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', NULL, 'Standard', 'Not Applicable', 'Level 3', 'No', 'Average', 50.0, NULL, NULL, 'Weak', 27.0, 'Weak', 23.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '80.20%', 58.3, '94.80%', '98.10%', NULL, NULL, 0.0, 0.0, 41.2, 21.7, 0.0, 3.8, 39.6, 31.3, 0.0, 0.0, 1.3, 0.0, -1.8, -1.6, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 91, 35, NULL, 1167232.603, 1905230.9330000002, 41.89551132, -87.66123981, 24, 'WEST TOWN', 27, 13, '(41.89551132, -87.66123981)') ,
(610284, 'New Field Elementary School', 'ES', '1707 W Morse Ave', 'Chicago', 'IL', 60626, '(773) 534-2760', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610284.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Strong', 56.0, '96.20%', 3.1, '95.60%', '99.00%', 74.2, 52.7, 32.1, 25.2, 40.4, 55.9, NULL, NULL, NULL, NULL, NULL, NULL, 23.2, 10.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 696, 32, NULL, 1163673.054, 1946078.6819999998, 42.00767533, -87.67315753, 1, 'ROGERS PARK', 49, 24, '(42.00767533, -87.67315753)') ,
(610323, 'New Millennium High School of Health at Bowen', 'HS', '2710 E 89th St', 'Chicago', 'IL', 60617, '(773) 535-7650', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610323.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 17.0, 'Weak', 23.0, 'Weak', 37.0, 'Weak', 25.0, 'Weak', 29.0, 'Very Weak', 6.0, NULL, NULL, NULL, NULL, '75.60%', 61.8, '94.70%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.1, 12.3, 13.9, 13.2, 1.1, 14.7, 0.8, 16.7, 67.2, 52.4, 701, 47, 62.1, 1195939.937, 1846527.5280000002, 41.73376107, -87.55775263, 46, 'SOUTH CHICAGO', 7, 4, '(41.73376107, -87.55775263)') ,
(609792, 'Newton Bateman Elementary School', 'ES', '4220 N Richmond St', 'Chicago', 'IL', 60618, '(773) 534-5055', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609792.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, 'Weak', 36.0, 'Average', 50.0, 'Average', 47.0, 'Weak', 25.0, 'Weak', 34.0, 'Average', 52.0, 'Average', 52.0, '96.20%', 0.2, '96.70%', '99.10%', 68.0, 33.4, 35.8, 36.6, 54.2, 49.6, 52.6, 47.1, 65.1, 57.6, 12.5, 34.5, 21.7, 16.1, -0.8, -0.1, 'Red', 'Yellow', 27.4, 67.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1015, 31, NULL, 1155915.8490000002, 1928001.2119999998, 41.95823045, -87.70218806, 16, 'IRVING PARK', 33, 17, '(41.95823045, -87.70218806)') ,
(609730, 'Nicholas Senn High School', 'HS', '5900 N Glenwood Ave', 'Chicago', 'IL', 60660, '(773) 534-2365', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609730.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 48.0, NULL, NULL, 'Average', 45.0, 'Average', 42.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '81.70%', 41.3, '96.40%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.8, 14.3, 14.3, 13.9, 1.1, 15.8, 1.5, 17.5, 40.1, 54.4, 1053, 32, 79.8, 1165871.81, 1939308.369, 41.98905063, -87.66526222, 77, 'EDGEWATER', 48, 20, '(41.98905063, -87.66526222)') ,
(609793, 'Nicholson Technology Academy', 'ES', '6006 S Peoria St', 'Chicago', 'IL', 60621, '(773) 535-3285', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609793.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 48.0, 'Weak', 37.0, 'Strong', 74.0, 'Strong', 73.0, 'Weak', 42.0, 'Weak', 39.0, NULL, NULL, NULL, NULL, '93.40%', 6.0, '97.00%', '100.00%', NULL, NULL, 39.1, 18.6, 54.1, 58.5, 64.3, 34.5, 69.4, 56.7, 21.2, 21.2, 22.6, 8.9, 1.9, -0.2, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 464, 45, NULL, 1171298.247, 1864964.2680000002, 41.7849277, -87.64748868, 68, 'ENGLEWOOD', 16, 7, '(41.7849277, -87.64748868)') ,
(609961, 'Ninos Heroes Elementary Academic Center', 'ES', '8344 S Commercial Ave', 'Chicago', 'IL', 60617, '(773) 535-6694', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609961.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 36.0, NULL, NULL, 'Strong', 61.0, 'Average', 55.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 49.0, '93.90%', 25.9, '95.00%', '100.00%', 52.5, 35.7, 33.9, 28.1, 52.8, 62.1, 44.2, 27.9, 66.3, 59.5, 3.6, 16.4, 14.7, 9.4, 0.9, 0.5, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 435, 47, NULL, 1197504.847, 1850181.7480000001, 41.74374969, -87.55189813, 46, 'SOUTH CHICAGO', 10, 4, '(41.74374969, -87.55189813)') ,
(609810, 'Norman A Bridge Elementary School', 'ES', '3800 N New England Ave', 'Chicago', 'IL', 60634, '(773) 534-3718', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609810.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 67.0, NULL, NULL, 'Average', 49.0, 'Weak', 35.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Average', 48.0, '95.60%', 6.6, '96.60%', '98.40%', 73.0, 43.0, 56.5, 45.8, 55.9, 74.0, NULL, NULL, NULL, NULL, 33.0, 38.6, 27.8, 18.9, -0.3, -0.9, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1004, 30, NULL, 1129684.0559999999, 1924513.9319999998, 41.94915063, -87.79870722, 17, 'DUNNING', 38, 16, '(41.94915063, -87.79870722)') ,
(609691, 'North-Grand High School', 'HS', '4338 W Wabansia Ave', 'Chicago', 'IL', 60639, '(773) 534-8520', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609691.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 42.0, NULL, NULL, 'Weak', 39.0, 'Weak', 25.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '88.60%', 15.3, '95.60%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.3, 12.8, 14.1, 14.2, 0.9, 15.4, 1.3, 15.8, 69.7, 41.3, 883, 34, 81.3, 1147041.135, 1910981.274, 41.91170094, -87.73525151, 23, 'HUMBOLDT PARK', 31, 25, '(41.91170094, -87.73525151)') ,
(610354, 'North River Elementary School', 'ES', '4416 N Troy St', 'Chicago', 'IL', 60625, '(773) 534-0590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610354.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 67.0, 'Strong', 60.0, 'Average', 53.0, 'Average', 51.0, 'Weak', 47.0, 'Average', 47.0, 'Average', 50.0, 'Average', 51.0, '95.60%', 20.1, '96.00%', '100.00%', 46.2, 55.8, 47.9, 39.4, 56.3, 64.1, 27.8, 35.1, 35.7, 54.9, 22.2, 40.7, 19.6, 13.7, -0.2, -0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 359, 31, NULL, 1154546.547, 1929268.9209999999, 41.96173669, -87.70718808, 14, 'ALBANY PARK', 33, 17, '(41.96173669, -87.70718808)') ,
(609749, 'Northside College Preparatory High School', 'HS', '5501 N Kedzie Ave', 'Chicago', 'IL', 60625, '(773) 534-3954', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609749.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, NULL, NULL, 'Very Strong', 99.0, 'Very Strong', 88.0, 'Weak', NULL, NULL, NULL, 'Strong', 57.0, 'Strong', 62.0, '95.70%', 2.8, '96.80%', '98.40%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 22.4, 22.2, 24.5, 24.7, 2.3, 28.8, 4.3, 98.0, 97.6, 90.7, 1053, 31, 95.9, 1154090.8590000002, 1936413.866, 41.98135196, -87.70867192, 13, 'NORTH PARK', 40, 17, '(41.98135196, -87.70867192)') ,
(609762, 'Percy L Julian High School', 'HS', '10330 S Elizabeth St', 'Chicago', 'IL', 60643, '(773) 535-5170', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609762.pdf', 'Far South Side High School Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Average', 50.0, 'Weak', 30.0, 'Weak', 35.0, 'Weak', 53.0, 'Average', 51.0, NULL, NULL, NULL, NULL, '74.60%', 33.4, '94.20%', '98.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.1, 12.8, 14.2, 13.9, 0.8, 14.8, 0.6, 8.3, 60.3, 54.9, 1226, 49, 65.3, 1169808.363, 1836139.004, 41.70585971, -87.65378585, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.70585971, -87.65378585)') ,
(609744, 'Northside Learning Center High School', 'HS', '3730 W Bryn Mawr Ave', 'Chicago', 'IL', 60659, '(773) 534-5180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609744.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', NULL, 'Standard', 'Not Applicable', 'Not Enough Data', 'No', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 81.0, 'Weak', 34.0, 'Weak', 75.0, 'Very Strong', 84.0, 'Strong', 62.0, 'Strong', 59.0, '89.80%', 5.3, '95.60%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 66.7, 0.0, NULL, 298, 31, NULL, 1150357.037, 1936987.496, 41.98299994, -87.72238881, 13, 'NORTH PARK', 39, 17, '(41.98299994, -87.72238881)') ,
(610051, 'Northwest Middle School', 'MS', '5252 W Palmer St', 'Chicago', 'IL', 60639, '(773) 534-3250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610051.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 34.0, NULL, NULL, 'Very Weak', 17.0, 'Very Weak', 13.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '93.40%', 2.2, '95.10%', '98.90%', NULL, NULL, NULL, NULL, NULL, NULL, 29.5, 25.1, 41.4, 41.1, 10.3, 14.8, 11.3, 5.9, -0.9, -1.4, 'Red', 'Red', 10.5, 62.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 756, 29, NULL, 1140773.188, 1914158.9440000001, 41.92053845, -87.75820003, 19, 'BELMONT CRAGIN', 37, 25, '(41.92053845, -87.75820003)') ,
(610099, 'Norwood Park Elementary School', 'ES', '5900 N Nina Ave', 'Chicago', 'IL', 60631, '(773) 534-1198', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610099.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 86.0, 'Strong', 69.0, 'Strong', 68.0, 'Weak', 31.0, 'Weak', 32.0, 'Strong', 57.0, 'Average', 52.0, '95.50%', 0.3, '95.60%', '100.00%', 75.2, 57.8, 69.3, 65.8, 56.3, 52.7, 90.3, 77.7, 57.3, 61.2, 64.1, 69.2, 50.5, 39.0, -1.8, -0.9, 'Red', 'Red', 94.7, 69.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 359, 30, NULL, 1128430.406, 1938729.626, 41.98818136, -87.80299207, 10, 'NORWOOD PARK', 41, 16, '(41.98818136, -87.80299207)') ,
(610529, 'Ogden International High School', 'HS', '1250 W Erie St', 'Chicago', 'IL', 60622, '(773) 534-0866', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610529.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Average', 56.0, NULL, NULL, 'Weak', 39.0, 'Weak', 24.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.20%', 13.7, '0.00%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, 61.9, 71.6, 48.1, 61.6, 47.9, 69.2, 42.6, 38.9, -0.6, -0.6, 'Red', 'Yellow', 58.7, 29.0, 14.5, 14.7, NULL, 16.0, 1.5, NULL, NULL, NULL, NULL, NULL, 558, 35, 93.3, 1167748.1, 1904529.595, 41.89357570000001, -87.65936676, 24, 'WEST TOWN', 27, 13, '(41.8935757, -87.65936676)') ,
(610201, 'Ole A Thorp Elementary Scholastic Academy', 'ES', '6024 W Warwick Ave', 'Chicago', 'IL', 60634, '(773) 534-3640', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610201.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 70.0, 'Strong', 61.0, 'Average', 47.0, 'Average', 40.0, 'Weak', 47.0, 'Average', 57.0, 'Average', 50.0, 'Average', 50.0, '96.40%', 25.8, '95.80%', '100.00%', 80.6, 62.3, 62.3, 65.0, 56.8, 55.9, 73.9, 66.5, 64.8, 61.5, 46.8, 58.5, 48.8, 38.7, 0.8, -0.5, 'Green', 'Yellow', 33.7, 100.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 834, 30, NULL, 1135416.207, 1924349.243, 41.94859865, -87.77764028, 17, 'DUNNING', 38, 16, '(41.94859865, -87.77764028)') ,
(610300, 'Oliver S Westcott Elementary School', 'ES', '409 W 80th St', 'Chicago', 'IL', 60620, '(773) 535-3090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610300.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 30.0, 'Average', 41.0, 'Average', 48.0, 'Average', 54.0, 'Weak', 47.0, 'Average', 47.0, 'Weak', 46.0, 'Average', 48.0, '92.60%', 31.5, '94.80%', '100.00%', NULL, 14.7, 36.4, 27.9, 50.4, 48.5, 27.9, 31.3, 43.5, 51.5, 6.0, 20.0, 7.0, 6.0, 0.3, 0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 413, 45, NULL, 1174868.475, 1851842.144, 41.74884018, -87.63478949, 44, 'CHATHAM', 21, 6, '(41.74884018, -87.63478949)') ,
(609997, 'Oliver Wendell Holmes Elementary School', 'ES', '955 W Garfield Blvd', 'Chicago', 'IL', 60621, '(773) 535-9025', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609997.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 21.0, NULL, NULL, 'Weak', 23.0, 'Average', 41.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 48.0, '87.00%', 139.8, '92.40%', '97.90%', 40.0, 30.5, 17.7, 8.0, 28.8, 33.1, 20.8, 8.8, 41.8, 20.2, 3.1, 9.4, 5.0, 3.0, -0.8, -2.3, 'Red', 'Red', 9.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 334, 45, NULL, 1170648.336, 1868177.8909999998, 41.79376044, -87.64977785, 68, 'ENGLEWOOD', 16, 7, '(41.79376044, -87.64977785)') ,
(610105, 'Oriole Park Elementary School', 'ES', '5424 N Oketo Ave', 'Chicago', 'IL', 60656, '(773) 534-1201', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610105.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, NULL, NULL, 'Very Strong', 88.0, 'Strong', 71.0, 'Weak', NULL, NULL, NULL, 'Strong', 60.0, 'Weak', 46.0, '96.50%', 2.3, '95.50%', '98.40%', 84.4, NULL, 86.0, 78.1, 59.6, 77.5, 91.4, 85.1, 81.1, 67.0, 69.6, 72.9, 75.1, 54.7, 1.1, -0.2, 'Green', 'Yellow', 87.1, 53.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 648, 30, NULL, 1125978.285, 1935191.9819999998, 41.97851501, -87.81209057, 10, 'NORWOOD PARK', 41, 16, '(41.97851501, -87.81209057)') ,
(610329, 'Orozco Fine Arts & Sciences Elementary School', 'ES', '1940 W 18th St', 'Chicago', 'IL', 60608, '(773) 534-7215', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610329.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 55.0, NULL, NULL, 'Weak', 34.0, 'Average', 44.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 53.0, '97.60%', 1.4, '96.40%', '100.00%', 92.9, 78.6, 82.8, 67.0, 73.9, 77.0, 50.1, 45.0, 51.0, 52.0, 18.8, 33.8, 30.2, 17.4, -0.6, -0.1, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 636, 39, NULL, 1163539.274, 1891453.1469999999, 41.85778248, -87.67519253, 31, 'LOWER WEST SIDE', 25, 12, '(41.85778248, -87.67519253)') ,
(610389, 'Orr Academy High School', 'HS', '730 N Pulaski Rd', 'Chicago', 'IL', 60624, '(773) 534-6500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610389.pdf', 'AUSL Schools', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '66.30%', 10.2, '95.00%', '99.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 11.7, 12.0, 13.6, 13.3, 1.6, 15.1, 1.5, 5.8, NULL, 51.9, 831, 34, 59.5, 1149548.53, 1904711.708, 41.89444828, -87.72620305, 23, 'HUMBOLDT PARK', 28, 11, '(41.89444828, -87.72620305)') ,
(609811, 'Orville T Bright Elementary School', 'ES', '10740 S Calhoun Ave', 'Chicago', 'IL', 60617, '(773) 535-6215', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609811.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 18.0, NULL, NULL, 'Weak', 28.0, 'Weak', 27.0, 'Weak', NULL, NULL, NULL, 'Weak', 41.0, 'Weak', 42.0, '92.20%', 55.7, '95.30%', '100.00%', 50.5, 19.0, 30.4, 28.2, 65.1, 64.8, 35.8, 32.3, 74.1, 61.3, 0.0, 17.9, 8.2, 5.9, 2.4, 0.6, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 361, 47, NULL, 1194858.644, 1834119.7780000002, 41.69973969, -87.56212104, 51, 'SOUTH DEERING', 10, 4, '(41.69973969, -87.56212104)') ,
(610367, 'Oscar DePriest Elementary School', 'ES', '139 S Parkside Ave', 'Chicago', 'IL', 60644, '(773) 534-6800', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610367.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 35.0, 'Weak', 39.0, 'Average', 47.0, 'Average', 53.0, 'Weak', 51.0, 'Weak', 39.0, 'Average', 47.0, 'Strong', 54.0, '92.70%', 20.0, '95.60%', '94.10%', 66.4, 62.0, 26.8, 18.4, 42.9, 43.2, 35.8, 45.9, 78.2, 76.9, 0.0, 9.1, 4.8, 9.8, 0.9, 2.3, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 524, 36, NULL, 1138681.703, 1898643.0369999998, 41.87799914, -87.7662615, 25, 'AUSTIN', 29, 15, '(41.87799914, -87.7662615)') ,
(610059, 'Oscar F Mayer Elementary School', 'ES', '2250 N Clifton Ave', 'Chicago', 'IL', 60614, '(773) 534-5535', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610059.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 67.0, NULL, NULL, 'Weak', 35.0, 'Average', 46.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.00%', 11.4, '95.20%', '100.00%', 79.6, NULL, 47.9, 36.8, 53.1, 59.8, 26.9, 37.4, 37.8, 46.8, 16.4, 32.7, 18.1, 17.0, -0.6, 0.1, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 605, 33, NULL, 1168219.452, 1915320.55, 41.92317657, -87.65732331, 7, 'LINCOLN PARK', 32, 18, '(41.92317657, -87.65732331)') ,
(610021, 'Pablo Casals Elementary School', 'ES', '3501 W Potomac Ave', 'Chicago', 'IL', 60651, '(773) 534-4444', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610021.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Weak', 42.0, 'Weak', 43.0, '92.00%', 47.1, '95.90%', '97.20%', 45.1, 32.1, 23.6, 14.6, 46.7, 39.7, 24.6, 25.8, 45.4, 67.2, 8.2, 18.4, 9.6, 5.0, -1.1, -0.3, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 505, 34, NULL, 1152770.808, 1908388.615, 41.90447488, -87.71427105, 23, 'HUMBOLDT PARK', 26, 14, '(41.90447488, -87.71427105)') ,
(610115, 'Park Manor Elementary School', 'ES', '7037 S Rhodes Ave', 'Chicago', 'IL', 60637, '(773) 535-3070', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610115.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Average', 42.0, 'Average', 42.0, 'Average', 48.0, 'Weak', 69.0, 'Strong', 67.0, 'Strong', 55.0, 'Strong', 54.0, '94.90%', 16.6, '96.30%', '100.00%', NULL, NULL, 17.6, 16.0, 43.3, 45.3, 28.9, 26.5, 75.0, 66.2, 3.6, 3.6, 6.2, 5.7, 1.7, 1.2, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 352, 46, NULL, 1181167.289, 1858324.511, 41.76648584, -87.61150917, 69, 'GREATER GRAND CROSSING', 6, 3, '(41.76648584, -87.61150917)') ,
(610116, 'Parkside Elementary Community Academy', 'ES', '6938 S East End Ave', 'Chicago', 'IL', 60649, '(773) 535-0940', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610116.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Weak', 42.0, 'Weak', 43.0, '89.80%', 90.7, '94.90%', '100.00%', 57.3, 33.6, 17.3, 7.8, 45.2, 44.0, 19.0, 22.6, 41.7, 53.3, 9.5, 4.8, 5.6, 3.5, -0.2, -0.1, 'Yellow', 'Yellow', NULL, NULL, 11.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 348, 46, NULL, 1188940.5420000001, 1859149.081, 41.76856599, -87.58299131, 43, 'SOUTH SHORE', 5, 3, '(41.76856599, -87.58299131)') ,
(609988, 'Patrick Henry Elementary School', 'ES', '4250 N Saint Louis Ave', 'Chicago', 'IL', 60618, '(773) 534-5060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609988.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 45.0, 'Average', 56.0, 'Average', 46.0, 'Average', 43.0, 'Weak', 68.0, 'Average', 45.0, 'Average', 52.0, 'Average', 49.0, '95.10%', 4.3, '95.90%', '100.00%', 56.6, 35.5, 42.1, 38.1, 68.2, 61.3, 30.4, 48.2, 44.6, 75.0, NULL, NULL, 12.5, 12.3, 0.6, 1.0, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 650, 31, NULL, 1152247.412, 1928240.9980000001, 41.95896179, -87.71566829, 16, 'IRVING PARK', 33, 17, '(41.95896179, -87.71566829)') ,
(610003, 'Paul Cuffe Math-Science Technology Academy Elementary School', 'ES', '8324 S Racine Ave', 'Chicago', 'IL', 60620, '(773) 535-8250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610003.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 51.0, 'Strong', 60.0, 'Average', 45.0, 'Average', 54.0, 'Weak', 66.0, 'Very Strong', 83.0, 'Average', 48.0, 'Average', 51.0, '93.60%', 46.9, '95.40%', '98.50%', 72.4, 72.1, 44.3, 44.9, 63.0, 68.8, 54.2, 44.3, 54.8, 52.4, 23.1, 40.0, 20.6, 15.6, -0.5, -1.1, 'Red', 'Red', 28.8, 21.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 512, 45, NULL, 1169734.759, 1849470.5, 41.74244489, -87.65366994, 71, 'AUBURN GRESHAM', 21, 6, '(41.74244489, -87.65366994)') ,
(609676, 'Paul Laurence Dunbar Career Academy High School', 'HS', '3000 S King Dr', 'Chicago', 'IL', 60616, '(773) 534-9000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609676.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 29.0, 'Average', 57.0, 'Weak', 31.0, 'Weak', 37.0, 'Weak', 57.0, 'Average', 49.0, NULL, NULL, NULL, NULL, '76.90%', 21.5, '96.00%', '98.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.8, 12.7, 13.9, 13.8, 1.0, 14.7, 0.8, 6.9, 61.7, 51.5, 1456, 40, 65.7, 1179262.097, 1885167.032, 41.84018775, -87.61767315, 35, 'DOUGLAS', 2, 1, '(41.84018775, -87.61767315)') ,
(610146, 'Paul Revere Elementary School', 'ES', '1010 E 72nd St', 'Chicago', 'IL', 60619, '(773) 535-0618', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610146.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 36.0, NULL, NULL, 'Average', 42.0, 'Average', 56.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Average', 47.0, '92.40%', 33.3, '94.60%', '89.30%', 46.5, 38.4, 10.7, 16.4, 55.0, 49.1, 15.7, 25.7, 50.6, 61.1, 6.7, 16.7, 3.5, 3.4, -2.4, -0.4, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 320, 46, NULL, 1184267.268, 1857526.2380000001, 41.76422335, -87.60017165, 69, 'GREATER GRAND CROSSING', 5, 3, '(41.76422335, -87.60017165)') ,
(609707, 'Paul Robeson High School', 'HS', '6835 S Normal Blvd', 'Chicago', 'IL', 60621, '(773) 535-3800', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609707.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 29.0, NULL, NULL, 'Weak', 33.0, 'Average', 40.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '73.50%', 26.4, '94.70%', '97.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 11.6, 11.0, 12.7, 12.7, 1.1, 13.6, 0.9, 2.9, 40.6, 42.2, 733, 45, 53.3, 1174171.884, 1859450.314, 41.76973341, -87.63711641, 68, 'ENGLEWOOD', 6, 7, '(41.76973341, -87.63711641)') ,
(609791, 'Perkins Bass Elementary School', 'ES', '1140 W 66th St', 'Chicago', 'IL', 60621, '(773) 535-3275', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609791.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 28.0, 'Average', 50.0, 'Average', 47.0, 'Average', 57.0, 'Weak', 52.0, 'Average', 42.0, 'Average', 47.0, 'Average', 49.0, '92.10%', 28.9, '96.40%', '100.00%', 61.7, 42.2, 15.5, 15.9, 45.3, 31.5, 23.9, 18.6, 53.6, 38.8, 7.3, 11.9, 5.0, 6.0, -0.1, 0.9, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 369, 45, NULL, 1169677.933, 1861068.9430000002, 41.77427377, -87.65354234, 68, 'ENGLEWOOD', 17, 7, '(41.77427377, -87.65354234)') ,
(610395, 'Pershing West Middle School', 'MS', '3200 S Calumet Ave', 'Chicago', 'IL', 60616, '(773) 534-9240', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610395.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 46.0, 'Average', 53.0, 'Weak', 34.0, 'Average', 55.0, 'Weak', 76.0, 'Average', 58.0, NULL, NULL, NULL, NULL, '96.20%', 3.7, '97.60%', '100.00%', NULL, NULL, 52.7, 45.0, 40.6, 60.0, 43.7, 54.1, 48.7, 52.9, 15.4, 28.8, 17.5, 21.4, -0.9, -0.5, 'Red', 'Yellow', 40.7, 50.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 269, 40, NULL, 1179082.41, 1883818.0119999999, 41.83649005, -87.61837372, 35, 'DOUGLAS', 2, 2, '(41.83649005, -87.61837372)') ,
(610145, 'Peter A Reinberg Elementary School', 'ES', '3425 N Major Ave', 'Chicago', 'IL', 60634, '(773) 534-3465', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610145.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 60.0, 'Average', 51.0, 'Weak', 35.0, 'Weak', 30.0, 'Weak', 60.0, 'Average', 54.0, 'Average', 47.0, 'Weak', 45.0, '95.80%', 8.2, '95.70%', '99.50%', 70.9, 46.3, 50.6, 39.6, 51.3, 66.0, 45.0, 39.1, 56.3, 49.6, 19.8, 25.7, 24.0, 14.7, -0.3, 0.5, 'Yellow', 'Yellow', 25.0, 80.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 832, 30, NULL, 1137794.92, 1922326.021, 41.94300407, -87.76894541, 15, 'PORTAGE PARK', 38, 16, '(41.94300407, -87.76894541)') ,
(609867, 'Peter Cooper Elementary Dual Language Academy', 'ES', '1624 W 19th St', 'Chicago', 'IL', 60608, '(773) 534-7205', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609867.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '96.20%', 5.7, '96.30%', '100.00%', 62.7, 28.7, 37.1, 26.1, 43.5, 44.6, NULL, NULL, NULL, NULL, NULL, NULL, 9.6, 10.7, 0.5, 1.3, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 709, 39, NULL, 1165708.517, 1890841.583, 41.85605839, -87.66724753, 31, 'LOWER WEST SIDE', 25, 12, '(41.85605839, -87.66724753)') ,
(610147, 'Philip Rogers Elementary School', 'ES', '7345 N Washtenaw Ave', 'Chicago', 'IL', 60645, '(773) 534-2125', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610147.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 81.0, 'Strong', 76.0, 'Strong', 68.0, 'Strong', 62.0, 'Weak', 75.0, 'Strong', 64.0, 'Average', 53.0, 'Average', 51.0, '96.40%', 3.0, '95.10%', '100.00%', 77.3, 50.9, 49.8, 47.2, 60.9, 61.9, 73.0, 52.7, 75.1, 53.6, 37.3, 52.5, 34.3, 28.3, 0.7, 0.7, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 735, 32, NULL, 1157132.164, 1948795.4740000002, 42.01526618, -87.69714858, 2, 'WEST RIDGE', 50, 24, '(42.01526618, -87.69714858)') ,
(609777, 'Phillip D Armour Elementary School', 'ES', '950 W 33rd Pl', 'Chicago', 'IL', 60608, '(773) 535-4530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609777.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 36.0, NULL, NULL, 'Weak', 34.0, 'Very Weak', 14.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Average', 51.0, '93.90%', 16.0, '95.80%', '98.00%', 67.7, 26.5, 16.5, 24.0, 37.8, 43.6, 29.8, 24.0, 51.5, 44.9, 3.8, 11.5, 10.1, 8.7, -0.5, 0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 365, 40, NULL, 1170302.817, 1882796.1940000001, 41.8338821, -87.65061898, 60, 'BRIDGEPORT', 11, 9, '(41.8338821, -87.65061898)') ,
(610090, 'Phillip Murray Elementary Language Academy', 'ES', '5335 S Kenwood Ave', 'Chicago', 'IL', 60615, '(773) 535-0585', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610090.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 66.0, NULL, NULL, 'Weak', 34.0, 'Weak', 37.0, 'Weak', NULL, NULL, NULL, 'Strong', 56.0, 'Average', 53.0, '95.60%', 3.5, '95.90%', '100.00%', 70.2, NULL, 66.9, 67.4, 55.1, 61.8, 43.6, 58.0, 53.0, 42.0, NULL, NULL, 35.0, 36.0, -0.4, 0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 533, 46, NULL, 1186033.436, 1870041.58, 41.79852501, -87.59330352, 41, 'HYDE PARK', 4, 2, '(41.79852501, -87.59330352)') ,
(609981, 'Phobe Apperson Hearst Elementary School', 'ES', '4640 S Lamon Ave', 'Chicago', 'IL', 60638, '(773) 535-2376', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609981.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 33.0, NULL, NULL, 'Weak', 31.0, 'Weak', 27.0, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Weak', 44.0, '92.10%', 25.5, '95.80%', '97.90%', 52.2, 48.8, 14.9, 10.7, 29.7, 40.0, 29.5, 22.2, 50.0, 47.1, 7.1, 10.7, 4.9, 6.7, -0.2, -1.6, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 364, 44, NULL, 1144438.111, 1873241.7659999998, 41.80818821, -87.74576315, 56, 'GARFIELD RIDGE', 23, 8, '(41.80818821, -87.74576315)') ,
(610304, 'Phoenix Military Academy High School', 'HS', '145 S Campbell Ave', 'Chicago', 'IL', 60612, '(773) 534-7275', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610304.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 53.0, NULL, NULL, 'Average', 59.0, 'Strong', 78.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Average', 48.0, '92.10%', 4.3, '97.40%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.7, 14.9, 15.9, 15.9, 1.2, 18.6, 2.7, 33.9, 74.8, 51.3, 461, 38, 74.1, 1159815.389, 1899102.4919999999, 41.87885054, -87.68865058, 28, 'NEAR WEST SIDE', 2, 11, '(41.87885054, -87.68865058)') ,
(610013, 'Pilsen Elementary Community Academy', 'ES', '1420 W 17th St', 'Chicago', 'IL', 60608, '(773) 534-7675', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610013.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 43.0, 'Average', 40.0, 'Average', 49.0, 'Average', 55.0, 'Weak', 16.0, 'Weak', 37.0, NULL, NULL, NULL, NULL, '96.00%', 9.4, '96.70%', '100.00%', 58.1, 36.1, 23.7, 19.4, 46.3, 46.7, 35.7, 33.3, 50.0, 51.2, NULL, NULL, 18.0, 12.8, -1.0, 0.3, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 447, 39, NULL, 1167068.653, 1891878.989, 41.85887606, -87.66222540000001, 31, 'LOWER WEST SIDE', 25, 12, '(41.85887606, -87.6622254)') ,
(610135, 'Portage Park Elementary School', 'ES', '5330 W Berteau Ave', 'Chicago', 'IL', 60641, '(773) 534-3576', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610135.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 62.0, NULL, NULL, 'Average', 46.0, 'Weak', 32.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 53.0, '94.20%', 3.7, '95.90%', '100.00%', 67.3, 33.8, 53.7, 41.6, 45.9, 61.1, 62.8, 55.3, 51.2, 44.1, 27.1, 37.4, 22.9, 21.1, -0.2, 0.8, 'Yellow', 'Green', 22.0, 66.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1139, 30, NULL, 1139920.815, 1927445.996, 41.9570151, -87.76100593, 15, 'PORTAGE PARK', 38, 16, '(41.9570151, -87.76100593)') ,
(610138, 'Pulaski International Academy Elmentary School', 'ES', '2230 W McLean Ave', 'Chicago', 'IL', 60647, '(773) 534-4391', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610138.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 68.0, 'Average', 45.0, 'Average', 54.0, 'Average', 57.0, 'Weak', 26.0, 'Weak', 23.0, 'Average', 47.0, 'Average', 47.0, '96.10%', 1.7, '96.20%', '100.00%', 75.2, 48.5, 38.3, 38.5, 54.5, 58.3, 56.7, 46.0, 71.1, 56.7, 33.3, 44.0, 24.2, 19.4, 0.5, 1.2, 'Green', 'Green', 37.3, 53.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 864, 35, NULL, 1161026.753, 1913565.719, 41.91851374, -87.68380061, 22, 'LOGAN SQUARE', 32, 14, '(41.91851374, -87.68380061)') ,
(609842, 'Rachel Carson Elementary School', 'ES', '5516 S Maplewood Ave', 'Chicago', 'IL', 60629, '(773) 535-9222', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609842.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 38.0, 'Weak', 36.0, 'Average', 45.0, 'Average', 56.0, 'Weak', 26.0, 'Weak', 36.0, 'Average', 50.0, 'Average', 51.0, '97.60%', 3.9, '97.70%', '100.00%', 68.5, NULL, 44.0, 35.0, 63.2, 75.8, 40.8, 45.5, 59.8, 59.5, 18.1, 24.4, 19.4, 12.2, -0.4, 0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1222, 43, NULL, 1160271.679, 1867913.54, 41.79325514, -87.68783576, 63, 'GAGE PARK', 16, 8, '(41.79325514, -87.68783576)') ,
(609902, 'Ralph H Metcalfe Elementary Community Academy', 'ES', '12339 S Normal Ave', 'Chicago', 'IL', 60628, '(773) 535-5590', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609902.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 50.0, '94.90%', 22.3, '95.50%', '100.00%', 43.2, 20.9, 27.5, 26.2, 47.3, 42.7, 51.1, 34.6, 60.5, 57.6, 13.6, 40.7, 18.2, 12.6, -0.7, -0.3, 'Red', 'Yellow', 24.6, 84.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 489, 48, NULL, 1175245.267, 1822916.582, 41.6694559, -87.63426928, 53, 'WEST PULLMAN', 34, 5, '(41.6694559, -87.63426928)') ,
(610141, 'Ravenswood Elementary School', 'ES', '4332 N Paulina St', 'Chicago', 'IL', 60613, '(773) 534-5525', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610141.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 46.0, NULL, NULL, 'Average', 42.0, 'Average', 41.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Weak', 44.0, '94.90%', 8.1, '95.70%', '100.00%', 78.7, NULL, 46.9, 45.9, 62.2, 74.5, 39.8, 51.9, 67.6, 59.4, 4.3, 34.8, 18.9, 13.7, -0.4, -0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 465, 33, NULL, 1164402.827, 1928980.2780000002, 41.96074129, -87.67095905, 6, 'LAKE VIEW', 47, 19, '(41.96074129, -87.67095905)') ,
(609769, 'Ray Graham Training Center High School', 'HS', '2347 S Wabash Ave', 'Chicago', 'IL', 60616, '(773) 534-9257', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609769.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', NULL, 'Standard', 'Not Applicable', 'Not Enough Data', 'No', 'Very Strong', 90.0, 'Very Strong', 82.0, 'Average', 52.0, 'Weak', 39.0, 'Weak', 63.0, 'Average', 52.0, 'Average', 53.0, 'Strong', 58.0, '86.30%', 3.7, '94.30%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 18.8, 10.3, NULL, 137, 40, NULL, 1177161.058, 1888615.131, 41.84969736, -87.6252787, 33, 'NEAR SOUTH SIDE', 2, 1, '(41.84969736, -87.6252787)') ,
(609903, 'Richard Edwards Elementary School', 'ES', '4815 S Karlov Ave', 'Chicago', 'IL', 60632, '(773) 535-4875', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609903.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 48.0, 'Average', 48.0, 'Weak', 38.0, 'Average', 41.0, 'Weak', 49.0, 'Average', 49.0, 'Average', 49.0, 'Average', 47.0, '95.30%', 10.3, '96.80%', '96.10%', 73.3, 42.8, 46.5, 26.3, 52.2, 62.5, 32.7, 27.6, 44.1, 50.8, 11.0, 28.4, 17.6, 11.4, -0.7, -0.1, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1503, 44, NULL, 1149882.354, 1872308.4740000002, 41.80552325, -87.72581893, 57, 'ARCHER HEIGHTS', 14, 8, '(41.80552325, -87.72581893)') ,
(610291, 'Richard Henry Lee Elementary School', 'ES', '6448 S Tripp Ave', 'Chicago', 'IL', 60629, '(773) 535-2255', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610291.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 59.0, NULL, NULL, 'Average', 46.0, 'Weak', 35.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.00%', 21.8, '95.30%', '96.10%', 55.7, 29.4, 46.9, 28.5, 51.2, 60.6, 56.3, 30.7, 60.1, 43.7, 30.1, 22.1, 15.6, 7.6, 0.0, -0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 832, 44, NULL, 1149113.752, 1861299.6930000002, 41.77532831, -87.72892174, 65, 'WEST LAWN', 13, 8, '(41.77532831, -87.72892174)') ,
(610239, 'Richard J Daley Elementary Academy', 'ES', '5024 S Wolcott Ave', 'Chicago', 'IL', 60609, '(773) 535-9091', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610239.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 29.0, 'Average', 40.0, 'Weak', 37.0, 'Average', 46.0, 'Weak', 50.0, 'Average', 42.0, 'Weak', 46.0, 'Average', 47.0, '94.90%', 44.2, '96.00%', '98.40%', 50.8, 26.4, 36.5, 13.3, 52.6, 63.9, 43.4, 25.5, 70.4, 55.6, 16.7, 5.6, 14.4, 5.3, 1.0, 0.5, 'Green', 'Yellow', 13.2, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 797, 42, NULL, 1164498.2179999999, 1871257.706, 41.80234381, -87.67224309, 61, 'NEW CITY', 16, 9, '(41.80234381, -87.67224309)') ,
(610102, 'Richard J Oglesby Elementary School', 'ES', '7646 S Green St', 'Chicago', 'IL', 60620, '(773) 535-3060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610102.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 44.0, 'Average', 40.0, 'Average', 49.0, 'Average', 49.0, 'Weak', 43.0, 'Weak', 32.0, 'Weak', 46.0, 'Weak', 46.0, '94.40%', 25.4, '96.90%', '100.00%', NULL, NULL, 26.9, 18.8, 46.4, 48.2, 32.5, 30.7, 58.1, 56.7, 14.3, 22.9, 5.1, 7.5, 0.7, 0.3, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 402, 45, NULL, 1171926.914, 1853950.5359999998, 41.75469091, -87.64550666, 71, 'AUBURN GRESHAM', 17, 6, '(41.75469091, -87.64550666)') ,
(609702, 'Richard T Crane Technical Preparatory High School', 'HS', '2245 W Jackson Blvd', 'Chicago', 'IL', 60612, '(773) 534-7550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609702.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 43.0, NULL, NULL, 'Average', 50.0, 'Average', 48.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Weak', 46.0, '57.90%', 19.9, '93.90%', '99.10%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.0, 11.9, 13.2, 13.6, 1.6, 14.0, 0.8, 3.7, 48.0, 32.6, 478, 38, 40.9, 1161290.138, 1898611.918, 41.87747384, -87.68324922, 28, 'NEAR WEST SIDE', 2, 12, '(41.87747384, -87.68324922)') ,
(610234, 'Richard Yates Elementary School', 'ES', '1839 N Richmond St', 'Chicago', 'IL', 60647, '(773) 534-4550', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610234.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 42.0, NULL, NULL, 'Strong', 66.0, 'Strong', 73.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Strong', 55.0, '93.70%', 5.6, '96.20%', '98.60%', 60.4, 55.8, 41.3, 18.0, 42.9, 53.5, 42.2, 20.2, 41.3, 51.7, 21.8, 9.1, 15.2, 5.4, -0.2, -1.0, 'Yellow', 'Red', 28.6, 77.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 690, 34, NULL, 1156561.0829999999, 1912262.2580000001, 41.91502858, -87.70024324, 22, 'LOGAN SQUARE', 35, 14, '(41.91502858, -87.70024324)') ,
(609695, 'Roald Amundsen High School', 'HS', '5110 N Damen Ave', 'Chicago', 'IL', 60625, '(773) 534-2320', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609695.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 51.0, 'Average', 44.0, 'Average', 43.0, 'Average', 42.0, 'Weak', 27.0, 'Average', 41.0, 'Weak', 43.0, 'Weak', 41.0, '81.10%', 8.7, '95.90%', '99.20%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.8, 13.9, 14.9, 14.8, 1.0, 16.7, 1.8, 16.0, 64.6, 49.1, 1532, 31, 72.2, 1162034.1570000001, 1934187.196, 41.97507922, -87.67952139, 4, 'LINCOLN SQUARE', 47, 20, '(41.97507922, -87.67952139)') ,
(610350, 'Robert A Black Magnet Elementary School', 'ES', '9101 S Euclid Ave', 'Chicago', 'IL', 60619, '(773) 535-6390', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610350.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 68.0, NULL, NULL, 'Strong', 63.0, 'Average', 58.0, 'Weak', NULL, NULL, NULL, 'Strong', 55.0, 'Weak', 46.0, '96.00%', 0.4, '93.70%', '100.00%', NULL, NULL, 46.1, 59.9, 61.7, 73.1, 37.4, 58.4, 54.0, 55.4, 18.5, 33.3, 20.1, 21.0, 0.0, 0.0, 'Yellow', 'Yellow', 20.4, 36.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 474, 47, NULL, 1190807.321, 1844986.0380000002, 41.72965645, -87.57660534, 48, 'CALUMET HEIGHTS', 8, 4, '(41.72965645, -87.57660534)') ,
(609906, 'Robert Emmet Elementary School', 'ES', '5500 W Madison St', 'Chicago', 'IL', 60644, '(773) 534-6050', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609906.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 37.0, NULL, NULL, 'Strong', 64.0, 'Strong', 71.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Strong', 54.0, '89.80%', 30.1, '94.80%', '100.00%', 68.1, 67.4, 28.8, 17.0, 48.0, 67.3, 15.1, 27.4, 56.9, 63.4, 8.9, 20.0, 12.9, 13.3, 1.0, 1.2, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 460, 36, NULL, 1139489.278, 1899514.7030000002, 41.88037642, -87.76327497, 25, 'AUSTIN', 29, 15, '(41.88037642, -87.76327497)') ,
(609929, 'Robert Fulton Elementary School', 'ES', '5300 S Hermitage Ave', 'Chicago', 'IL', 60609, '(773) 535-9000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609929.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 46.0, 'Average', 41.0, 'Strong', 63.0, 'Strong', 63.0, 'Weak', 53.0, 'Weak', 39.0, 'Weak', 45.0, 'Average', 49.0, '95.10%', 10.1, '96.80%', '100.00%', 40.7, NULL, 17.9, 10.3, 41.2, 40.9, 50.4, 7.9, 70.8, 49.6, 10.6, 6.4, 9.6, 1.1, 0.9, -0.9, 'Green', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 466, 43, NULL, 1165540.244, 1869516.208, 41.79754288, -87.66847093, 61, 'NEW CITY', 16, 9, '(41.79754288, -87.66847093)') ,
(609979, 'Robert Healy Elementary School', 'ES', '3010 S Parnell Ave', 'Chicago', 'IL', 60616, '(773) 534-9190', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609979.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 63.0, NULL, NULL, 'Average', 54.0, 'Weak', 36.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '96.60%', 11.3, '97.10%', '99.10%', 84.0, NULL, 72.1, 62.8, 63.4, 80.0, 64.3, 68.5, 61.4, 69.3, 38.6, 52.9, 45.1, 29.9, -0.3, 0.6, 'Yellow', 'Green', 18.9, 80.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1384, 40, NULL, 1173077.2920000001, 1884891.36, 41.83957045, -87.64037683, 60, 'BRIDGEPORT', 11, 9, '(41.83957045, -87.64037683)') ,
(610045, 'Robert H Lawrence Elementary School', 'ES', '9928 S Crandon Ave', 'Chicago', 'IL', 60617, '(773) 535-6320', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610045.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 19.0, 'Weak', 29.0, 'Average', 40.0, 'Average', 51.0, 'Weak', 35.0, 'Weak', 30.0, NULL, NULL, NULL, NULL, '90.70%', 14.0, '95.30%', '100.00%', 56.7, 35.9, 9.0, 14.6, 41.2, 31.0, 8.0, 15.8, 37.3, 40.6, 5.3, 15.8, 2.7, 3.4, -1.4, -1.3, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 371, 47, NULL, 1193433.0520000001, 1839450.176, 41.71440176, -87.56716721, 51, 'SOUTH DEERING', 7, 4, '(41.71440176, -87.56716721)') ,
(609956, 'Robert L Grimes Elementary School', 'ES', '5450 W 64th Pl', 'Chicago', 'IL', 60638, '(773) 535-2364', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609956.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 79.0, 'Strong', 64.0, 'Average', 56.0, 'Average', 59.0, 'Weak', 72.0, 'Strong', 71.0, 'Average', 53.0, 'Average', 49.0, '95.20%', 5.0, '95.60%', '100.00%', 58.8, 39.4, 37.4, 39.7, 50.7, 50.3, 45.5, 57.3, 51.0, 58.7, 17.4, 32.6, 22.1, 18.8, -0.4, 0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 486, 44, NULL, 1140971.91, 1861303.67, 41.77549238, -87.75876919, 64, 'CLEARING', 13, 8, '(41.77549238, -87.75876919)') ,
(610391, 'Robert Lindblom Math & Science Academy High School', 'HS', '6130 S Wolcott Ave', 'Chicago', 'IL', 60636, '(773) 535-9300', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610391.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 66.0, NULL, NULL, 'Strong', 75.0, 'Very Strong', 85.0, 'Weak', NULL, NULL, NULL, 'Strong', 55.0, 'Average', 51.0, '92.70%', 2.1, '95.90%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 49.1, 70.2, 61.7, 37.6, -1.2, -0.8, 'Red', 'Yellow', 61.1, 78.2, 18.0, 18.5, 19.1, 19.0, 1.0, 22.2, 3.1, 29.1, 68.9, 85.9, 916, 43, 83.3, 1164703.614, 1863861.294, 41.78204279999999, -87.67169858, 67, 'WEST ENGLEWOOD', 15, 7, '(41.7820428, -87.67169858)') ,
(610252, 'Robert Nathaniel Dett Elementary School', 'ES', '2306 W Maypole Ave', 'Chicago', 'IL', 60612, '(773) 534-7160', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610252.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 48.0, NULL, NULL, 'Average', 51.0, 'Strong', 65.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Strong', 57.0, '94.40%', 6.9, '94.80%', '100.00%', 84.3, 58.8, 29.4, 16.2, 37.9, 55.4, 31.4, 26.7, 56.3, 29.6, 8.0, 12.0, 24.4, 15.4, 0.4, 1.0, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 250, 35, NULL, 1160870.308, 1900992.919, 41.88401623, -87.68472465, 28, 'NEAR WEST SIDE', 2, 13, '(41.88401623, -87.68472465)') ,
(609759, 'Roberto Clemente Community Academy High School', 'HS', '1147 N Western Ave', 'Chicago', 'IL', 60622, '(773) 534-4000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609759.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 44.0, 'Weak', 31.0, 'Weak', 39.0, 'Weak', 34.0, 'Weak', 28.0, 'Weak', 25.0, NULL, NULL, NULL, NULL, '69.60%', 20.6, '94.20%', '99.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.6, 12.3, 14.0, 13.3, 0.7, 15.4, 1.4, 12.5, 49.5, 41.5, 1016, 35, 58.9, 1160235.206, 1907768.8769999999, 41.90262318, -87.68686934, 24, 'WEST TOWN', 1, 13, '(41.90262318, -87.68686934)') ,
(610029, 'Rodolfo Lozano Bilingual & International Center Elementary School', 'ES', '1501 N Greenview Ave', 'Chicago', 'IL', 60622, '(773) 534-4750', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610029.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, NULL, NULL, 'Weak', 39.0, 'Weak', 37.0, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Average', 53.0, '94.80%', 0.9, '96.20%', '100.00%', 70.0, 38.7, 31.0, 29.5, 44.7, 54.3, 38.9, 34.7, 55.6, 50.0, 15.0, 20.0, 9.3, 11.3, -0.4, -0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 319, 35, NULL, 1166155.136, 1910106.654, 41.90891372, -87.66505758, 24, 'WEST TOWN', 32, 14, '(41.90891372, -87.66505758)') ,
(609733, 'Roger C Sullivan High School', 'HS', '6631 N Bosworth Ave', 'Chicago', 'IL', 60626, '(773) 534-2000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609733.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 30.0, 'Average', 44.0, 'Weak', 34.0, 'Weak', 34.0, 'Weak', 42.0, 'Weak', 33.0, 'Weak', 44.0, 'Weak', 45.0, '81.60%', 14.7, '95.80%', '94.40%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.6, 13.2, 13.8, 13.5, 0.9, 15.4, 1.6, 14.8, 39.3, 52.1, 826, 32, 68.6, 1164774.883, 1944269.292, 42.00268694, -87.66915533, 1, 'ROGERS PARK', 40, 24, '(42.00268694, -87.66915533)') ,
(610091, 'Ronald Brown Elementary Community Academy', 'ES', '12607 S Union Ave', 'Chicago', 'IL', 60628, '(773) 535-5385', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610091.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 31.0, 'Weak', 26.0, 'Average', 52.0, 'Weak', 33.0, 'Weak', 48.0, 'Average', 45.0, 'Average', 52.0, 'Strong', 55.0, '93.80%', 35.6, '96.30%', '98.10%', 62.7, 32.0, 28.0, 33.7, 57.1, 47.4, 37.9, 33.0, 62.4, 66.7, 4.0, 32.0, 13.4, 12.5, 0.6, 1.4, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 283, 48, NULL, 1173988.072, 1821226.195, 41.66484509, -87.63892027, 53, 'WEST PULLMAN', 9, 5, '(41.66484509, -87.63892027)') ,
(610282, 'Ronald E McNair Elementary School', 'ES', '4820 W Walton St', 'Chicago', 'IL', 60651, '(773) 534-8980', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610282.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 31.0, 'Weak', 30.0, 'Average', 56.0, 'Average', 59.0, 'Weak', 32.0, 'Weak', 20.0, 'Average', 47.0, 'Average', 53.0, '90.70%', 52.8, '96.50%', '100.00%', 65.4, 47.7, 36.6, 27.0, 47.0, 55.1, 36.1, 26.8, 45.7, 54.9, 4.5, 9.1, 9.8, 5.6, -1.2, -1.7, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 422, 36, NULL, 1143946.854, 1905911.072, 41.8978464, -87.74674645, 25, 'AUSTIN', 37, 15, '(41.8978464, -87.74674645)') ,
(609826, 'Rosario Castellanos Elementary School', 'ES', '2524 S Central Park Ave', 'Chicago', 'IL', 60623, '(773) 534-1620', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609826.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 36.0, 'Weak', 24.0, 'Weak', 34.0, 'Weak', 20.0, 'Weak', 17.0, 'Weak', 31.0, 'Weak', 46.0, 'Average', 48.0, '96.30%', 24.3, '96.90%', '100.00%', NULL, NULL, 34.8, 18.8, 49.8, 58.5, 33.3, 17.2, 61.0, 48.5, 7.5, 15.1, 8.0, 6.2, -0.8, -0.6, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 525, 37, NULL, 1152737.279, 1886876.128, 41.84544296, -87.71496352, 30, 'SOUTH LAWNDALE', 22, 10, '(41.84544296, -87.71496352)') ,
(610056, 'Roswell B Mason Elementary School', 'ES', '4217 W 18th St', 'Chicago', 'IL', 60623, '(773) 534-1530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610056.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 37.0, 'Average', 47.0, 'Average', 44.0, 'Average', 58.0, 'Weak', 50.0, 'Weak', 36.0, 'Average', 51.0, 'Strong', 54.0, '92.40%', 31.0, '95.00%', '94.50%', 48.4, NULL, 8.1, 20.3, 34.2, 31.2, 12.9, 28.4, 24.3, 39.8, 5.7, 26.9, 4.2, 5.8, -2.1, -0.8, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 479, 36, NULL, 1148477.255, 1891003.4409999999, 41.85685194, -87.73049117, 29, 'NORTH LAWNDALE', 24, 10, '(41.85685194, -87.73049117)') ,
(610027, 'Rudyard Kipling Elementary School', 'ES', '9351 S Lowe Ave', 'Chicago', 'IL', 60620, '(773) 535-3151', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610027.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 52.0, NULL, NULL, 'Average', 42.0, 'Average', 42.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Average', 51.0, '94.70%', 23.4, '96.00%', '97.80%', 76.2, 52.5, 49.7, 44.4, 64.5, 58.2, 51.6, 46.3, 57.1, 50.3, 13.7, 31.4, 20.9, 21.2, -0.6, 0.2, 'Red', 'Yellow', 16.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 445, 49, NULL, 1173661.215, 1842654.148, 41.72365391, -87.63948475, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.72365391, -87.63948475)') ,
(610250, 'Rueben Salazar Elementary Bilingual Center', 'ES', '160 W Wendell St', 'Chicago', 'IL', 60610, '(773) 534-8310', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610250.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Strong', 64.0, NULL, NULL, 'Average', 54.0, 'Average', 53.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Average', 47.0, '95.10%', 44.6, '95.80%', '100.00%', 68.3, 33.1, 61.7, 50.8, 53.3, 75.7, 43.6, 60.2, 54.3, 65.6, 22.6, 64.5, 20.5, 15.5, 0.9, 0.3, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 444, 33, NULL, 1174702.288, 1907452.8190000001, 41.90144445, -87.63373912, 8, 'NEAR NORTH SIDE', 43, 18, '(41.90144445, -87.63373912)') ,
(609995, 'Rufus M Hitch Elementary School', 'ES', '5625 N McVicker Ave', 'Chicago', 'IL', 60646, '(773) 534-1189', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609995.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'Yes', 'Strong', 64.0, 'Strong', 77.0, 'Average', 55.0, 'Average', 58.0, 'Weak', 65.0, 'Strong', 70.0, 'Strong', 55.0, 'Strong', 54.0, '95.10%', 3.7, '96.60%', '98.60%', 73.7, 27.6, 62.2, 51.7, 50.0, 67.8, 56.5, 54.8, 52.0, 63.3, 22.9, 37.5, 29.5, 24.7, -0.1, 0.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 542, 30, NULL, 1134940.195, 1937129.571, 41.98367756, -87.77908614, 10, 'NORWOOD PARK', 45, 16, '(41.98367756, -87.77908614)') ,
(609853, 'Salmon P Chase Elementary School', 'ES', '2021 N Point St', 'Chicago', 'IL', 60647, '(773) 534-4185', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609853.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 55.0, 'Weak', 28.0, 'Average', 59.0, 'Weak', 39.0, 'Weak', 25.0, 'Weak', 27.0, NULL, NULL, NULL, NULL, '95.40%', 18.5, '95.80%', '98.60%', 63.0, 26.7, 40.6, 25.0, 57.3, 72.1, 47.1, 36.8, 70.2, 56.8, 23.1, 23.1, 17.1, 11.0, -0.6, 0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 504, 35, NULL, 1158104.87, 1913362.1619999998, 41.91801539, -87.69454144, 22, 'LOGAN SQUARE', 1, 14, '(41.91801539, -87.69454144)') ,
(609943, 'Samuel Gompers Fine Arts Options Elementary School', 'ES', '12302 S State St', 'Chicago', 'IL', 60628, '(773) 535-5475', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609943.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 34.0, 'Average', 42.0, 'Average', 48.0, 'Average', 52.0, 'Weak', 57.0, 'Average', 59.0, 'Average', 49.0, 'Average', 47.0, '93.60%', 54.2, '96.00%', '100.00%', NULL, NULL, 36.9, 30.1, 57.9, 56.5, 36.8, 29.5, 67.5, 56.3, 3.9, 15.7, 13.5, 8.5, 0.4, -0.7, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 285, 48, NULL, 1178435.326, 1823353.395, 41.67058298, -87.62258098, 53, 'WEST PULLMAN', 9, 5, '(41.67058298, -87.62258098)') ,
(610155, 'Sauganash Elementary School', 'ES', '6040 N Kilpatrick Ave', 'Chicago', 'IL', 60646, '(773) 534-3470', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610155.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, NULL, NULL, 'Strong', 76.0, 'Strong', 61.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '96.30%', 1.5, '95.40%', '92.90%', 85.6, 50.4, 68.5, 75.7, 66.9, 70.2, 81.4, 80.1, 58.7, 57.8, 57.1, 69.6, 49.1, 34.5, 2.2, 0.3, 'Green', 'Yellow', 35.7, 78.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 549, 31, NULL, 1143919.392, 1939921.031, 41.99117332, -87.74599137, 12, 'FOREST GLEN', 39, 17, '(41.99117332, -87.74599137)') ,
(609805, 'Scott Joplin Elementary School', 'ES', '7931 S Honore St', 'Chicago', 'IL', 60620, '(773) 535-3425', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609805.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 35.0, 'Weak', 29.0, 'Average', 44.0, 'Average', 53.0, 'Weak', 45.0, 'Weak', 34.0, 'Average', 52.0, 'Average', 50.0, '95.00%', 44.5, '94.60%', '97.70%', NULL, NULL, 27.8, 27.4, 66.9, 49.4, 33.1, 29.6, 49.7, 61.6, 11.8, 31.4, 9.8, 12.2, 0.2, 0.7, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 538, 43, NULL, 1165441.875, 1851949.948, 41.74934082, -87.66932903, 71, 'AUBURN GRESHAM', 18, 6, '(41.74934082, -87.66932903)') ,
(609968, 'Sharon Christa McAuliffe Elementary School', 'ES', '1841 N Springfield Ave', 'Chicago', 'IL', 60647, '(773) 534-4400', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609968.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'Yes', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Strong', 56.0, '93.90%', 14.2, '95.70%', '98.90%', 64.7, 25.1, 34.4, 24.4, 39.5, 56.1, 27.5, 21.6, 37.5, 44.7, NULL, NULL, 10.4, 11.0, -1.0, -0.9, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 862, 34, NULL, 1150124.467, 1912135.0019999999, 41.91480732, -87.72389409, 20, 'HERMOSA', 30, 25, '(41.91480732, -87.72389409)') ,
(610157, 'Sidney Sawyer Elementary School', 'ES', '5248 S Sawyer Ave', 'Chicago', 'IL', 60632, '(773) 535-0440', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610157.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 57.0, 'Weak', 30.0, 'Average', 41.0, 'Weak', 39.0, 'Weak', 24.0, 'Weak', 29.0, 'Average', 49.0, 'Average', 52.0, '96.80%', 7.4, '96.40%', '100.00%', 67.8, NULL, 37.0, 34.9, 58.5, 65.8, 36.8, 44.2, 57.5, 54.1, 19.1, 27.0, 18.7, 12.7, -0.4, 0.4, 'Red', 'Yellow', 17.6, 82.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1846, 44, NULL, 1155567.996, 1869433.427, 41.79752155, -87.70504311, 63, 'GAGE PARK', 14, 8, '(41.79752155, -87.70504311)') ,
(610283, 'Simon Guggenheim Elementary School', 'ES', '7141 S Morgan St', 'Chicago', 'IL', 60621, '(773) 535-3587', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610283.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 30.0, NULL, NULL, 'Weak', 38.0, 'Weak', 22.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 47.0, '90.90%', 29.5, '93.10%', '100.00%', NULL, NULL, 3.5, 9.8, 33.3, 31.6, 16.2, 15.3, 38.6, 40.9, 4.5, 13.6, 4.2, 6.6, -1.7, 0.9, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 291, 45, NULL, 1170922.983, 1857302.22, 41.76391032, -87.64908807, 68, 'ENGLEWOOD', 17, 7, '(41.76391032, -87.64908807)') ,
(610521, 'Sir Miles Davis Magnet Elementary Academy', 'ES', '6730 S Paulina St', 'Chicago', 'IL', 60636, '(773) 535-9120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610521.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 35.0, NULL, NULL, 'Average', 51.0, 'Average', 54.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '93.00%', 57.8, '94.50%', '100.00%', 63.9, 62.3, 24.8, 14.1, 43.1, 52.0, 22.6, 17.1, 49.1, 45.5, 4.4, 13.3, 5.9, 3.2, 0.0, -0.9, 'Yellow', 'Red', 25.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 354, 43, NULL, 1166133.893, 1859929.0180000002, 41.77122181, -87.66656657, 67, 'WEST ENGLEWOOD', 15, 7, '(41.77122181, -87.66656657)') ,
(610534, 'Skinner North', 'ES', '640 W Scott St', 'Chicago', 'IL', 60610, '(773) 534-8500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610534.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Strong', 64.0, 'Average', 52.0, '96.40%', 0.0, '97.80%', '100.00%', 83.3, 83.0, 100.0, 100.0, 74.1, 76.0, NULL, NULL, NULL, NULL, NULL, NULL, 100.0, 100.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 242, 33, NULL, 1171670.9740000002, 1908773.84, 41.9051367, -87.64483441, 8, 'NEAR NORTH SIDE', 27, 18, '(41.9051367, -87.64483441)') ,
(610226, 'Socorro Sandoval Elementary School', 'ES', '5534 S Saint Louis Ave', 'Chicago', 'IL', 60629, '(773) 535-0457', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610226.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Weak', 44.0, 'Average', 50.0, '95.80%', 3.7, '96.50%', '98.90%', 77.3, 47.9, 44.1, 23.0, 53.0, 61.5, NULL, NULL, NULL, NULL, NULL, NULL, 15.9, 9.1, -0.5, -0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1186, 44, NULL, 1153950.752, 1867537.733, 41.79235177, -87.71102417, 63, 'GAGE PARK', 14, 8, '(41.79235177, -87.71102417)') ,
(610160, 'Songhai Elementary Learning Institute', 'ES', '11725 S Perry Ave', 'Chicago', 'IL', 60628, '(773) 535-5547', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610160.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 17.0, NULL, NULL, 'Average', 47.0, 'Average', 54.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Weak', 46.0, '89.50%', 84.7, '95.40%', '100.00%', 24.8, NULL, 13.8, 16.3, 35.8, 36.8, 25.6, 22.5, 70.0, 67.2, 7.4, 25.9, 3.6, 3.6, 0.1, -1.4, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 352, 48, NULL, 1177727.869, 1827095.66, 41.68086831, -87.62505762, 53, 'WEST PULLMAN', 34, 5, '(41.68086831, -87.62505762)') ,
(609990, 'South Loop Elementary School', 'ES', '1212 S Plymouth Ct', 'Chicago', 'IL', 60605, '(773) 534-8690', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609990.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 84.0, 'Strong', 66.0, 'Weak', 34.0, 'Weak', 36.0, 'Weak', 57.0, 'Average', 46.0, 'Strong', 54.0, 'Weak', 46.0, '95.30%', 12.7, '97.20%', '100.00%', 89.0, 70.1, 81.7, 80.1, 63.5, 66.8, 72.0, 68.7, 70.9, 71.6, 27.3, 40.9, 54.2, 48.8, 1.0, 1.5, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 795, 38, NULL, 1176190.878, 1894860.267, 41.86685634, -87.62865124, 33, 'NEAR SOUTH SIDE', 2, 1, '(41.86685634, -87.62865124)') ,
(610530, 'South Shore Fine Arts Academy', 'ES', '1415 E 70th St', 'Chicago', 'IL', 60637, '(773) 535-8340', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610530.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Strong', 56.0, 'Strong', 55.0, '93.60%', 22.4, '96.30%', '100.00%', 77.2, 46.5, 29.2, 20.8, 40.0, 54.5, NULL, NULL, NULL, NULL, NULL, NULL, 20.8, 0.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 255, 46, NULL, 1186924.7859999998, 1858826.102, 41.76772772, -87.59039017, 43, 'SOUTH SHORE', 5, 3, '(41.76772772, -87.59039017)') ,
(609745, 'Southside Occupational Academy High School', 'HS', '7342 S Hoyne Ave', 'Chicago', 'IL', 60636, '(773) 535-9100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609745.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', NULL, 'Track_E', 'Not Applicable', 'Not Enough Data', 'No', NULL, NULL, 'Average', 52.0, NULL, NULL, NULL, NULL, 'Weak', 47.0, 'Strong', 60.0, 'Strong', 63.0, 'Strong', 60.0, '85.50%', 13.1, '95.20%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 40.0, 71.7, NULL, 201, 43, NULL, 1163596.617, 1855772.0590000001, 41.75986813, -87.6759838, 67, 'WEST ENGLEWOOD', 17, 7, '(41.75986813, -87.6759838)') ,
(610183, 'Spencer Technology Academy', 'ES', '214 N Lavergne Ave', 'Chicago', 'IL', 60644, '(773) 534-6150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610183.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Weak', 35.0, 'Average', 50.0, 'Average', 57.0, 'Strong', 71.0, 'Weak', 66.0, 'Average', 56.0, 'Average', 50.0, 'Strong', 55.0, '92.50%', 24.1, '96.00%', '100.00%', 55.3, 44.9, 26.1, 21.5, 56.0, 50.0, 36.0, 29.4, 56.6, 61.4, 9.4, 21.9, 10.7, 11.0, 0.7, 1.7, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 811, 36, NULL, 1142962.029, 1901180.1369999999, 41.88488257, -87.75048169, 25, 'AUSTIN', 28, 15, '(41.88488257, -87.75048169)') ,
(610357, 'Spry Community Links High School', 'HS', '2400 S Marshall Blvd', 'Chicago', 'IL', 60623, '(773) 534-1997', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610357.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 70.0, NULL, NULL, 'Very Strong', 92.0, 'Very Strong', 87.0, 'Weak', NULL, NULL, NULL, 'Strong', 56.0, 'Average', 51.0, '95.60%', 9.7, '95.70%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.3, 14.6, 15.5, 15.9, 1.6, 16.9, 1.4, 35.0, 73.7, 65.2, 205, 39, 92.8, 1157044.9109999998, 1887895.675, 41.84815451, -87.69912725, 30, 'SOUTH LAWNDALE', 12, 10, '(41.84815451, -87.69912725)') ,
(609880, 'Stephen Decatur Classical Elementary School', 'ES', '7030 N Sacramento Ave', 'Chicago', 'IL', 60645, '(773) 534-2201', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609880.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Weak', 99.0, 'Very Strong', 99.0, 'Average', 52.0, 'Weak', 46.0, '96.20%', 2.8, '95.40%', '100.00%', 100.0, 97.2, 98.6, 98.6, 50.7, 61.3, 100.0, 100.0, 48.1, 78.6, NULL, NULL, 93.6, 93.6, 0.2, -1.0, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 289, 32, NULL, 1155108.239, 1946608.822, 42.00930696, -87.70465502, 2, 'WEST RIDGE', 50, 24, '(42.00930696, -87.70465502)') ,
(609933, 'Stephen F Gale Elementary Community Academy', 'ES', '1631 W Jonquil Ter', 'Chicago', 'IL', 60626, '(773) 534-2100', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609933.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 39.0, 'Weak', 31.0, 'Weak', 35.0, 'Weak', 27.0, 'Weak', 33.0, 'Weak', 29.0, 'Weak', 45.0, 'Weak', 43.0, '92.10%', 34.8, '95.20%', '100.00%', 40.1, 15.5, 18.7, 17.9, 49.3, 39.3, 32.4, 24.8, 46.2, 48.9, 7.5, 22.5, 7.6, 6.9, -0.5, 0.0, 'Yellow', 'Yellow', 20.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 532, 32, NULL, 1163959.517, 1950960.205, 42.02106425, -87.67196481, 1, 'ROGERS PARK', 49, 24, '(42.02106425, -87.67196481)') ,
(609976, 'Stephen K Hayt Elementary School', 'ES', '1518 W Granville Ave', 'Chicago', 'IL', 60660, '(773) 534-2040', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609976.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 72.0, 'Strong', 61.0, 'Average', 52.0, 'Average', 44.0, 'Weak', 48.0, 'Strong', 66.0, NULL, NULL, NULL, NULL, '95.60%', 55.6, '96.00%', '99.30%', 78.2, 59.2, 50.8, 46.9, 56.9, 66.5, 55.5, 46.3, 61.5, 57.5, 26.7, 39.5, 28.6, 23.2, 0.3, 0.1, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1000, 32, NULL, 1164969.8020000001, 1941312.085, 41.99456814, -87.66852270000001, 77, 'EDGEWATER', 40, 24, '(41.99456814, -87.6685227)') ,
(609724, 'Stephen T Mather High School', 'HS', '5835 N Lincoln Ave', 'Chicago', 'IL', 60659, '(773) 534-2350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609724.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 58.0, 'Average', 44.0, 'Average', 48.0, 'Average', 47.0, 'Weak', 50.0, 'Average', 56.0, NULL, NULL, NULL, NULL, '87.60%', 7.6, '96.50%', '97.90%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 14.5, 14.4, 15.0, 15.4, 0.9, 15.9, 0.9, 24.6, 56.2, 57.4, 1657, 32, 59.0, 1155766.152, 1938701.409, 41.9875954, -87.70244871, 2, 'WEST RIDGE', 40, 20, '(41.9875954, -87.70244871)') ,
(610191, 'Stone Elementary Scholastic Academy', 'ES', '6239 N Leavitt St', 'Chicago', 'IL', 60659, '(773) 534-2045', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610191.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 78.0, NULL, NULL, 'Average', 55.0, 'Average', 43.0, 'Weak', NULL, NULL, NULL, 'Strong', 55.0, 'Average', 47.0, '96.40%', 1.9, '96.00%', '100.00%', 83.2, 46.2, 62.8, 67.5, 59.9, 56.7, 62.0, 68.8, 59.9, 59.4, 40.0, 47.3, 45.7, 44.7, 0.3, 1.7, 'Yellow', 'Green', 57.1, 56.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 646, 32, NULL, 1160554.673, 1941569.632, 41.99536777, -87.68475636, 2, 'WEST RIDGE', 50, 24, '(41.99536777, -87.68475636)') ,
(610405, 'Suder Montessori Magnet Elementary School', 'ES', '2022 W Washington Blvd', 'Chicago', 'IL', 60612, '(773) 534-7685', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610405.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '96.60%', 6.5, '95.70%', '100.00%', 82.2, 60.7, 61.3, 50.9, 58.0, 59.5, NULL, NULL, NULL, NULL, NULL, NULL, 40.7, 29.2, 0.7, -0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 351, 38, NULL, 1162799.297, 1900725.344, 41.88324176, -87.67764871, 28, 'NEAR WEST SIDE', 2, 13, '(41.88324176, -87.67764871)') ,
(610249, 'Talman Elementary School', 'ES', '5450 S Talman Ave', 'Chicago', 'IL', 60632, '(773) 535-7850', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610249.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Weak', 85.0, 'Very Strong', 81.0, 'Strong', 57.0, 'Average', 48.0, '96.20%', 9.2, '95.50%', '100.00%', 50.6, NULL, 54.7, 22.6, 71.6, 75.0, 56.0, 42.2, 76.7, 62.8, 11.1, 25.9, 27.3, 16.5, 0.5, 0.7, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 376, 43, NULL, 1159597.93, 1868206.8290000001, 41.79407382, -87.6902983, 63, 'GAGE PARK', 14, 9, '(41.79407382, -87.6902983)') ,
(610396, 'Tarkington School of Excellence Elementary School', 'ES', '3330 W 71st St', 'Chicago', 'IL', 60629, '(773) 535-4700', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610396.pdf', 'AUSL Schools', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 48.0, NULL, NULL, 'Weak', 39.0, 'Average', 40.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Strong', 56.0, '95.90%', 10.5, '96.60%', '100.00%', NULL, NULL, 36.3, 34.5, 56.0, 53.7, 30.6, 36.7, 53.4, 56.1, 6.3, 10.7, 13.4, 9.7, -0.9, -0.2, 'Red', 'Yellow', 26.1, 53.3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1070, 44, NULL, 1155280.2, 1857374.1369999999, 41.76443483, -87.70642093, 66, 'CHICAGO LAWN', 18, 8, '(41.76443483, -87.70642093)') ,
(610506, 'TEAM Englewood Community Academy High School', 'HS', '6201 S Stewart Ave', 'Chicago', 'IL', 60621, '(773) 535-3530', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610506.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 45.0, 'Average', 48.0, 'Average', 55.0, 'Average', 53.0, 'Weak', 74.0, 'Strong', 68.0, 'Strong', 58.0, 'Average', 49.0, '87.30%', 8.2, '96.50%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.0, 12.4, 14.3, 13.3, 1.3, 15.3, 1.0, 7.9, NULL, NULL, 440, 45, 77.4, 1174730.35, 1863740.125, 41.7814927, -87.63494163, 68, 'ENGLEWOOD', 20, 7, '(41.7814927, -87.63494163)') ,
(609921, 'Telpochcalli Elementary School', 'ES', '2832 W 24th Blvd', 'Chicago', 'IL', 60623, '(773) 534-1402', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609921.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 42.0, 'Average', 48.0, 'Very Weak', 19.0, 'Weak', 22.0, 'Weak', 82.0, 'Strong', 69.0, 'Strong', 55.0, 'Average', 51.0, '95.40%', 2.7, '94.70%', '96.40%', 31.3, NULL, 31.8, 13.6, 50.8, 66.2, 30.8, 25.4, 51.6, 54.4, 4.5, 18.2, 8.4, 7.3, -0.1, 0.9, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 285, 39, NULL, 1157725.352, 1887836.509, 41.84797834, -87.6966316, 30, 'SOUTH LAWNDALE', 12, 10, '(41.84797834, -87.6966316)') ,
(609991, 'Theodore Herzl Elementary School', 'ES', '3711 W Douglas Blvd', 'Chicago', 'IL', 60623, '(773) 534-1480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609991.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 41.0, NULL, NULL, 'Average', 53.0, 'Strong', 65.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '90.80%', 23.9, '97.00%', '96.00%', 62.6, 35.6, 13.5, 16.3, 63.3, 50.4, 22.1, 22.1, 65.7, 67.6, 0.0, 10.5, 4.6, 3.8, 0.1, 0.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 512, 36, NULL, 1151795.4, 1892995.835, 41.86225473, -87.71825927, 29, 'NORTH LAWNDALE', 24, 10, '(41.86225473, -87.71825927)') ,
(609728, 'Theodore Roosevelt High School', 'HS', '3436 W Wilson Ave', 'Chicago', 'IL', 60625, '(773) 534-5000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609728.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 42.0, NULL, NULL, 'Average', 42.0, 'Weak', 37.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '82.30%', 9.7, '96.10%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.9, 13.4, 14.3, 14.0, 1.1, 15.7, 1.4, 13.7, 59.4, 42.9, 1490, 31, 61.2, 1152459.778, 1930390.229, 41.96485522, -87.71483051, 14, 'ALBANY PARK', 33, 17, '(41.96485522, -87.71483051)') ,
(610178, 'Theophilus Schmid Elementary School', 'ES', '9755 S Greenwood Ave', 'Chicago', 'IL', 60628, '(773) 535-6235', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610178.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 46.0, NULL, NULL, 'Weak', 31.0, 'Average', 53.0, 'Weak', NULL, NULL, NULL, 'Weak', 41.0, 'Average', 52.0, '92.10%', 67.1, '96.30%', '100.00%', 62.8, 45.2, 35.8, 22.6, 51.9, 58.5, 47.1, 43.1, 56.3, 54.2, 13.3, 20.0, 17.1, 6.7, -0.4, -0.7, 'Yellow', 'Yellow', 43.8, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 185, 48, NULL, 1185308.063, 1840332.575, 41.71701772, -87.59689622, 50, 'PULLMAN', 8, 5, '(41.71701772, -87.59689622)') ,
(609794, 'Thomas A Edison Regional Gifted Center Elementary School', 'ES', '4929 N Sawyer Ave', 'Chicago', 'IL', 60625, '(773) 534-0540', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609794.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 91.0, NULL, NULL, 'Strong', 64.0, 'Average', 56.0, 'Weak', NULL, NULL, NULL, 'Strong', 55.0, 'Strong', 57.0, '96.60%', 1.9, '96.30%', '100.00%', NULL, NULL, 92.0, 97.8, 52.8, 57.6, 93.1, 98.9, 55.2, 60.4, 80.0, 96.7, 79.0, 88.4, 1.8, 1.6, 'Green', 'Green', 31.8, 78.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 269, 31, NULL, 1153858.196, 1932691.8909999998, 41.971143299999994, -87.70962725, 14, 'ALBANY PARK', 39, 17, '(41.9711433, -87.70962725)') ,
(609987, 'Thomas A Hendricks Elementary Community Academy', 'ES', '4316 S Princeton Ave', 'Chicago', 'IL', 60609, '(773) 535-1696', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609987.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 33.0, NULL, NULL, 'Strong', 67.0, 'Strong', 78.0, 'Weak', NULL, NULL, NULL, 'Average', 51.0, 'Average', 52.0, '91.20%', 61.8, '95.60%', '100.00%', 58.7, 60.0, 31.6, 12.1, 41.1, 50.0, 48.5, 16.7, 71.9, 54.9, 7.5, 32.5, 8.5, 1.0, 0.9, -2.3, 'Green', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 307, 42, NULL, 1174966.903, 1876241.753, 41.81579314, -87.63370133, 37, 'FULLER PARK', 3, 9, '(41.81579314, -87.63370133)') ,
(609851, 'Thomas Chalmers Specialty Elementary School', 'ES', '2745 W Roosevelt Rd', 'Chicago', 'IL', 60608, '(773) 534-1720', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609851.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 34.0, 'Strong', 76.0, 'Weak', 39.0, 'Average', 45.0, 'Weak', 88.0, 'Strong', 78.0, NULL, NULL, NULL, NULL, '94.40%', 3.4, '97.30%', '100.00%', 37.5, NULL, 7.5, 10.3, 35.8, 12.8, 17.2, 21.9, 31.4, 57.6, 0.0, 12.5, 2.1, 4.9, -0.7, 1.2, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 288, 39, NULL, 1158048.936, 1894554.926, 41.8664078, -87.69526079, 29, 'NORTH LAWNDALE', 28, 10, '(41.8664078, -87.69526079)') ,
(609896, 'Thomas Drummond Elementary School', 'ES', '1845 W Cortland St', 'Chicago', 'IL', 60622, '(773) 534-4120', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609896.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 72.0, NULL, NULL, 'Average', 59.0, 'Average', 46.0, 'Weak', NULL, NULL, NULL, 'Strong', 58.0, 'Average', 48.0, '95.20%', 0.3, '96.70%', '97.50%', 71.3, NULL, 68.6, 69.3, 66.3, 75.3, 37.7, 42.9, 43.3, 61.3, 0.0, 0.0, 34.0, 34.0, 0.0, 0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 352, 35, NULL, 1163566.562, 1912627.608, 41.91588634, -87.67449563, 22, 'LOGAN SQUARE', 32, 14, '(41.91588634, -87.67449563)') ,
(610002, 'Thomas Hoyne Elementary School', 'ES', '8905 S Crandon Ave', 'Chicago', 'IL', 60617, '(773) 535-6425', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610002.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, 'Strong', 66.0, 'Strong', 77.0, 'Strong', 73.0, 'Weak', 67.0, 'Strong', 66.0, NULL, NULL, NULL, NULL, '94.80%', 0.0, '95.40%', '100.00%', 40.2, NULL, 36.2, 42.7, 56.3, 61.7, 54.0, 65.0, 50.0, 50.0, 14.6, 41.5, 23.4, 14.0, 0.7, 0.0, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 288, 47, NULL, 1193092.436, 1846336.5459999999, 41.73330693, -87.5681905, 48, 'CALUMET HEIGHTS', 7, 4, '(41.73330693, -87.5681905)') ,
(610295, 'Thomas J Higgins Elementary Community Academy', 'ES', '11710 S Morgan St', 'Chicago', 'IL', 60643, '(773) 535-5625', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610295.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Average', 53.0, '95.20%', 0.5, '95.20%', '100.00%', 62.0, 30.4, 35.7, 25.2, 58.3, 63.7, 43.6, 35.3, 63.8, 57.0, 9.1, 20.5, 19.1, 9.8, 1.7, 0.4, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 369, 49, NULL, 1171725.3329999999, 1827097.573, 41.68100687, -87.64702985, 53, 'WEST PULLMAN', 34, 5, '(41.68100687, -87.64702985)') ,
(610220, 'Thomas J Waters Elementary School', 'ES', '4540 N Campbell Ave', 'Chicago', 'IL', 60625, '(773) 534-5090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610220.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 85.0, NULL, NULL, 'Average', 56.0, 'Average', 54.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Average', 49.0, '96.10%', 0.4, '95.10%', '100.00%', 70.4, 31.2, 51.1, 59.0, 42.0, 44.2, 49.6, 54.7, 61.6, 50.8, 22.7, 61.4, 27.2, 28.5, 0.2, 0.6, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 590, 31, NULL, 1158841.899, 1930252.459, 41.96434835, -87.69136881, 4, 'LINCOLN SQUARE', 47, 19, '(41.96434835, -87.69136881)') ,
(609715, 'Thomas Kelly High School', 'HS', '4136 S California Ave', 'Chicago', 'IL', 60632, '(773) 535-4900', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609715.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Weak', 36.0, 'Average', 53.0, 'Weak', 36.0, 'Weak', 35.0, 'Weak', 61.0, 'Average', 57.0, NULL, NULL, NULL, NULL, '89.30%', 5.8, '95.30%', '99.70%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.4, 13.4, 15.1, 14.6, 1.2, 17.1, 2.0, 22.1, 67.4, 46.4, 2883, 39, 65.6, 1158337.7859999998, 1877175.576, 41.81871101, -87.69467483, 58, 'BRIGHTON PARK', 14, 9, '(41.81871101, -87.69467483)') ,
(610321, 'Thurgood Marshall Middle School', 'MS', '3900 N Lawndale Ave', 'Chicago', 'IL', 60618, '(773) 534-5200', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610321.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 39.0, NULL, NULL, 'Very Weak', 11.0, 'Very Weak', 1.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '94.00%', 18.1, '96.00%', '97.20%', NULL, NULL, NULL, NULL, NULL, NULL, 44.8, 35.7, 47.2, 50.9, 20.5, 19.9, 18.6, 4.0, -1.7, -2.2, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 435, 31, NULL, 1150989.856, 1925703.541, 41.952023600000004, -87.72035833, 16, 'IRVING PARK', 39, 17, '(41.9520236, -87.72035833)') ,
(609895, 'Turner-Drew Elementary Language Academy', 'ES', '9300 S Princeton Ave', 'Chicago', 'IL', 60620, '(773) 535-5720', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609895.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 56.0, 'Average', 56.0, 'Average', 48.0, 'Average', 51.0, 'Weak', 75.0, 'Strong', 66.0, NULL, NULL, NULL, NULL, '95.80%', 24.0, '95.20%', '100.00%', 80.2, 54.2, 46.3, 46.0, 54.5, 60.5, 68.8, 55.3, 57.9, 52.2, 18.4, 44.9, 24.2, 23.2, 0.1, 0.9, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 373, 48, NULL, 1176188.712, 1843386.9319999998, 41.72560849, -87.6302048, 49, 'ROSELAND', 21, 6, '(41.72560849, -87.6302048)') ,
(610394, 'Uplift Community High School', 'HS', '900 W Wilson Ave', 'Chicago', 'IL', 60640, '(773) 534-2875', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610394.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 50.0, 'Average', 50.0, 'Strong', 61.0, 'Strong', 62.0, 'Weak', 48.0, 'Average', 53.0, NULL, NULL, NULL, NULL, '88.70%', 8.1, '95.90%', '98.30%', NULL, NULL, NULL, NULL, NULL, NULL, 51.2, 34.9, 70.0, 29.3, 14.8, 14.8, 23.3, 2.3, 0.8, -0.2, 'Yellow', 'Yellow', NULL, NULL, 13.6, 13.9, 14.9, 14.7, 1.1, 16.9, 2.0, 19.6, 62.4, 65.0, 473, 32, 91.4, 1169403.4009999998, 1930780.685, 41.96557412, -87.65252191, 3, 'UPTOWN', 46, 19, '(41.96557412, -87.65252191)') ,
(610504, 'Velma F Thomas Early Childhood Center', 'ES', '3625 S Hoyne Ave', 'Chicago', 'IL', 60609, '(773) 535-4088', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610504.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', NULL, 'Standard', NULL, NULL, 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Strong', 55.0, 'Strong', 57.0, NULL, 0.0, '95.60%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 211, 39, NULL, 1162992.8229999999, 1880511.57, 41.82776914, -87.67750501, 59, 'MCKINLEY PARK', 11, 9, '(41.82776914, -87.67750501)') ,
(609989, 'Victor Herbert Elementary School', 'ES', '2131 W Monroe St', 'Chicago', 'IL', 60612, '(773) 534-7806', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609989.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Average', 50.0, 'Average', 52.0, 'Strong', 64.0, 'Strong', 72.0, 'Weak', 48.0, 'Average', 50.0, 'Average', 50.0, 'Average', 52.0, '92.80%', 10.3, '96.20%', '100.00%', 58.4, 26.0, 53.3, 34.9, 63.7, 83.5, 53.8, 29.0, 73.9, 58.1, 25.0, 25.0, 30.0, 10.3, 2.1, -0.8, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 398, 38, NULL, 1162083.381, 1899481.0980000002, 41.87984242, -87.68031236, 28, 'NEAR WEST SIDE', 2, 12, '(41.87984242, -87.68031236)') ,
(609944, 'Virgil Grissom Elementary School', 'ES', '12810 S Escanaba Ave', 'Chicago', 'IL', 60633, '(773) 535-5380', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609944.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 62.0, 'Strong', 66.0, 'Average', 51.0, 'Strong', 64.0, 'Weak', 69.0, 'Very Strong', 87.0, NULL, NULL, NULL, NULL, '95.50%', 2.5, '96.40%', '100.00%', 84.4, 51.6, 53.4, 47.7, 45.5, 60.2, 50.0, 45.7, 62.0, 54.3, 23.5, 44.1, 22.0, 17.7, -1.1, -1.5, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 330, 47, NULL, 1197378.736, 1820556.064, 41.66245712, -87.55334334, 55, 'HEGEWISCH', 10, 4, '(41.66245712, -87.55334334)') ,
(610518, 'VOISE Academy High School', 'HS', '231 N Pine Ave', 'Chicago', 'IL', 60644, '(773) 534-0660', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610518.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 42.0, NULL, NULL, 'Average', 52.0, 'Average', 52.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '84.50%', 25.0, '96.50%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.3, 12.5, 13.6, 13.6, 1.3, 14.4, 0.8, NULL, NULL, NULL, 410, 36, 82.0, 1139494.763, 1901274.2580000001, 41.88520477, -87.76321191, 25, 'AUSTIN', 28, 15, '(41.88520477, -87.76321191)') ,
(610363, 'Walt Disney Magnet Elementary School', 'ES', '4140 N Marine Dr', 'Chicago', 'IL', 60613, '(773) 534-5840', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610363.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '96.40%', 7.7, '96.40%', '100.00%', NULL, 53.8, 63.9, 64.9, 54.7, 64.1, 72.8, 69.6, 59.9, 51.0, 38.7, 53.3, 46.0, 35.7, 0.4, 0.6, 'Green', 'Green', 20.0, 74.1, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1651, 32, NULL, 1171170.774, 1927599.86, 41.95680712, -87.6461175, 3, 'UPTOWN', 46, 19, '(41.95680712, -87.6461175)') ,
(610095, 'Walter L Newberry Math & Science Academy Elementary School', 'ES', '700 W Willow St', 'Chicago', 'IL', 60614, '(773) 534-8000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610095.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 71.0, 'Strong', 74.0, 'Average', 52.0, 'Strong', 66.0, 'Weak', 41.0, 'Average', 44.0, 'Strong', 54.0, 'Average', 47.0, '95.10%', 2.1, '96.30%', '100.00%', NULL, NULL, 70.2, 61.5, 57.5, 66.9, 68.0, 65.5, 61.1, 59.3, 40.3, 43.5, 46.2, 35.9, 1.0, 1.3, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 558, 33, NULL, 1171323.787, 1911991.711, 41.91397435, -87.64601501, 7, 'LINCOLN PARK', 43, 18, '(41.91397435, -87.64601501)') ,
(609680, 'Walter Payton College Preparatory High School', 'HS', '1034 N Wells St', 'Chicago', 'IL', 60610, '(773) 534-0034', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609680.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 98.0, NULL, NULL, 'Very Strong', 80.0, 'Strong', 77.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '93.40%', 0.7, '96.10%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 21.2, 21.8, 23.1, 23.2, 2.0, 27.0, 3.9, 96.4, 96.9, 82.4, 881, 33, 90.7, 1174484.638, 1907490.082, 41.90155157, -87.63453744, 8, 'NEAR NORTH SIDE', 27, 18, '(41.90155157, -87.63453744)') ,
(609955, 'Walter Q Gresham Elementary School', 'ES', '8524 S Green St', 'Chicago', 'IL', 60620, '(773) 535-3350', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609955.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 25.0, 'Average', 49.0, 'Very Weak', 7.0, 'Very Weak', 11.0, 'Weak', 68.0, 'Average', 57.0, 'Average', 48.0, 'Weak', 46.0, '95.50%', 35.4, '95.00%', '100.00%', 61.4, 54.0, 30.7, 24.0, 47.7, 57.3, 13.9, 21.6, 35.2, 34.6, 5.4, 16.2, 9.9, 7.4, 1.4, 0.0, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 359, 45, NULL, 1172094.314, 1848202.3080000002, 41.73891334, -87.64506169, 71, 'AUBURN GRESHAM', 21, 6, '(41.73891334, -87.64506169)') ,
(610264, 'Walter Reed Elementary School', 'ES', '6350 S Stewart Ave', 'Chicago', 'IL', 60621, '(773) 535-3075', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610264.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 29.0, NULL, NULL, 'Strong', 62.0, 'Strong', 68.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 51.0, '92.40%', 75.5, '93.20%', '100.00%', NULL, NULL, 43.2, 34.1, 65.9, 65.9, 66.0, 20.0, 93.6, 75.6, 0.0, 5.3, 28.6, 9.9, 2.0, -0.3, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 44, 45, NULL, 1174681.574, 1862611.0359999998, 41.77839545, -87.63515406, 68, 'ENGLEWOOD', 20, 7, '(41.77839545, -87.63515406)') ,
(609855, 'Walter S Christopher Elementary School', 'ES', '5042 S Artesian Ave', 'Chicago', 'IL', 60632, '(773) 535-9375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609855.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', NULL, NULL, 'Average', 48.0, NULL, NULL, NULL, NULL, 'Weak', 72.0, 'Strong', 64.0, 'Average', 52.0, 'Average', 53.0, '86.10%', 4.0, '94.10%', '100.00%', 42.9, 15.8, 28.8, 19.0, 66.1, 45.5, 61.7, 44.7, 63.0, 54.3, 7.7, 38.5, 13.3, 5.7, 0.7, 0.7, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 279, 43, NULL, 1160846.21, 1870985.96, 41.8016744, -87.68564411, 63, 'GAGE PARK', 14, 9, '(41.8016744, -87.68564411)') ,
(610179, 'Washington D Smyser Elementary School', 'ES', '4310 N Melvina Ave', 'Chicago', 'IL', 60634, '(773) 534-3711', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610179.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 55.0, NULL, NULL, 'Average', 43.0, 'Average', 45.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.50%', 4.9, '97.20%', '96.60%', 72.6, NULL, 53.9, 58.4, 61.1, 65.7, 59.2, 60.2, 64.7, 51.5, 27.0, 49.5, 36.5, 27.5, 0.6, 0.1, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1047, 30, NULL, 1134202.834, 1928091.892, 41.95889037, -87.78201198, 15, 'PORTAGE PARK', 38, 16, '(41.95889037, -87.78201198)') ,
(610121, 'Washington Irving Elementary School', 'ES', '749 S Oakley Blvd', 'Chicago', 'IL', 60612, '(773) 534-7295', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610121.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 43.0, NULL, NULL, 'Weak', 34.0, 'Average', 44.0, 'Weak', NULL, NULL, NULL, 'Average', 48.0, 'Average', 49.0, '95.60%', 15.1, '95.90%', '89.00%', 71.8, 64.1, 36.8, 34.0, 45.5, 52.2, 57.1, 46.8, 68.8, 64.2, 17.0, 27.7, 22.6, 20.9, -0.8, 0.7, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 482, 38, NULL, 1161216.3909999998, 1896475.332, 41.87161238, -87.68357939, 28, 'NEAR WEST SIDE', 25, 12, '(41.87161238, -87.68357939)') ,
(609740, 'Wells Community Academy High School', 'HS', '936 N Ashland Ave', 'Chicago', 'IL', 60622, '(773) 534-7010', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609740.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Average', 51.0, 'Average', 49.0, 'Average', 41.0, 'Weak', 38.0, 'Weak', 65.0, 'Average', 49.0, NULL, NULL, NULL, NULL, '74.10%', 16.9, '96.20%', '96.70%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.8, 12.4, 14.2, 13.6, 0.8, 14.9, 0.7, 7.2, 59.3, 43.8, 633, 35, 61.3, 1165512.916, 1906490.72, 41.89900506, -87.66751989, 24, 'WEST TOWN', 1, 13, '(41.89900506, -87.66751989)') ,
(610032, 'Wendell E Green Elementary School', 'ES', '1150 W 96th St', 'Chicago', 'IL', 60643, '(773) 535-2575', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610032.pdf', 'Rock Island Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Weak', 32.0, NULL, NULL, 'Average', 47.0, 'Average', 53.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Average', 47.0, '93.90%', 20.1, '95.70%', '100.00%', 59.4, 46.3, 53.0, 39.2, 72.6, 63.5, 55.2, 37.5, 59.8, 67.5, 12.5, 12.5, 25.8, 15.7, 2.2, 1.0, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 254, 49, NULL, 1170124.549, 1841046.4109999998, 41.71931953, -87.65248591, 73, 'WASHINGTON HEIGHTS', 21, 22, '(41.71931953, -87.65248591)') ,
(609727, 'Wendell Phillips Academy High School', 'HS', '244 E Pershing Rd', 'Chicago', 'IL', 60653, '(773) 535-1603', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609727.pdf', 'AUSL Schools', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', NULL, NULL, 'Strong', 71.0, NULL, NULL, NULL, NULL, 'Weak', 80.0, 'Strong', 79.0, NULL, NULL, NULL, NULL, '63.00%', 22.0, '96.10%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 11.5, 12.8, 13.4, 12.9, 1.4, 15.0, 1.6, 2.5, 40.3, 42.6, 590, 40, 32.1, 1178735.106, 1879229.78, 41.82390751, -87.61978794, 35, 'DOUGLAS', 3, 2, '(41.82390751, -87.61978794)') ,
(609978, 'Wendell Smith Elementary School', 'ES', '744 E 103rd St', 'Chicago', 'IL', 60628, '(773) 535-5689', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609978.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 22.0, 'Weak', 24.0, 'Very Weak', 1.0, 'Very Weak', 1.0, 'Weak', 31.0, 'Weak', 29.0, 'Weak', 42.0, 'Weak', 46.0, '92.10%', 90.0, '94.10%', '95.20%', 64.8, 42.6, 25.0, 20.2, 53.1, 49.0, 34.6, 22.3, 30.7, 45.9, 8.6, 22.9, 9.4, 4.2, -0.5, -1.0, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 363, 48, NULL, 1183210.11, 1836877.16, 41.70758462, -87.60468711, 50, 'PULLMAN', 8, 5, '(41.70758462, -87.60468711)') ,
(610100, 'West Park Elementary Academy', 'ES', '1425 N Tripp Ave', 'Chicago', 'IL', 60651, '(773) 534-4940', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610100.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 29.0, NULL, NULL, 'Weak', 36.0, 'Average', 40.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Average', 47.0, '93.70%', 16.3, '96.70%', '100.00%', 60.7, 44.4, 22.0, 17.6, 39.2, 37.3, 29.0, 26.4, 61.2, 51.2, 9.2, 15.4, 10.8, 6.9, -0.3, -0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 686, 34, NULL, 1147827.76, 1909240.9519999998, 41.90691024, -87.73240646, 23, 'HUMBOLDT PARK', 37, 25, '(41.90691024, -87.73240646)') ,
(610224, 'West Pullman Elementary School', 'ES', '11941 S Parnell Ave', 'Chicago', 'IL', 60628, '(773) 535-5500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610224.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 23.0, 'Average', 46.0, 'Average', 55.0, 'Average', 47.0, 'Weak', 57.0, 'Average', 50.0, 'Average', 47.0, 'Weak', 44.0, '91.20%', 7.6, '93.80%', '100.00%', 31.0, 28.8, 12.6, 15.5, 45.9, 43.5, 25.7, 22.5, 32.0, 38.8, 9.7, 22.6, 5.7, 5.7, -0.9, -1.3, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 343, 48, NULL, 1174831.25, 1825538.503, 41.67666008, -87.63570683, 53, 'WEST PULLMAN', 34, 5, '(41.67666008, -87.63570683)') ,
(610542, 'West Ridge Elementary School', 'ES', '6700 N Whipple St', 'Chicago', 'IL', 60645, '(773) 534-8250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610542.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Not Enough Data', 'No', 'Strong', 67.0, NULL, NULL, 'Average', 41.0, 'Weak', 28.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Strong', 58.0, '94.90%', 6.8, '0.00%', '100.00%', 69.0, 46.8, 43.3, 32.9, 46.3, 60.9, 44.0, 34.4, 48.7, 43.2, 18.2, 26.2, 16.9, 13.0, -2.2, -1.0, 'Red', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 659, 32, NULL, 1154772.178, 1944306.386, 42.00299575, -87.70595357, 2, 'WEST RIDGE', 50, 24, '(42.00299575, -87.70595357)') ,
(609755, 'Whitney M Young Magnet High School', 'HS', '211 S Laflin St', 'Chicago', 'IL', 60607, '(773) 534-7500', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609755.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 95.0, 'Very Strong', 80.0, 'Strong', 69.0, 'Strong', 67.0, 'Weak', 52.0, 'Average', 46.0, 'Average', 53.0, 'Average', 50.0, '94.60%', 1.2, '94.90%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 100.0, 99.0, 99.5, 77.0, -1.2, -0.8, 'Red', 'Yellow', 56.3, 89.2, 20.8, 21.0, 22.6, 22.8, 2.0, 26.6, 4.0, 89.0, 93.9, 88.3, 2166, 38, 96.7, 1166475.742, 1899058.6269999999, 41.87859028, -87.66419634, 28, 'NEAR WEST SIDE', 2, 12, '(41.87859028, -87.66419634)') ,
(610230, 'Wildwood Elementary School', 'ES', '6950 N Hiawatha Ave', 'Chicago', 'IL', 60646, '(773) 534-1188', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610230.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 99.0, 'Very Strong', 90.0, 'Strong', 77.0, 'Strong', 67.0, 'Weak', 55.0, 'Average', 45.0, 'Strong', 55.0, 'Weak', 37.0, '96.20%', 1.0, '95.70%', '92.30%', 88.8, NULL, 71.5, 67.2, 47.4, 59.9, 79.5, 79.5, 82.0, 63.9, 50.9, 68.4, 52.9, 33.2, 0.7, -0.4, 'Green', 'Yellow', 22.8, 84.6, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 414, 30, NULL, 1135078.471, 1945800.949, 42.00747007, -87.77837089, 12, 'FOREST GLEN', 41, 16, '(42.00747007, -87.77837089)') ,
(610251, 'Willa Cather Elementary School', 'ES', '2908 W Washington Blvd', 'Chicago', 'IL', 60612, '(773) 534-6780', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610251.pdf', 'Garfield-Humboldt Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 1', 'No', 'Strong', 69.0, 'Strong', 70.0, 'Strong', 78.0, 'Very Strong', 81.0, 'Weak', 62.0, 'Very Strong', 83.0, 'Strong', 56.0, 'Average', 53.0, '94.90%', 29.5, '96.40%', '100.00%', 62.7, 44.0, 42.9, 27.1, 54.3, 72.5, 36.1, 21.3, 45.5, 38.2, 0.0, 5.3, 20.3, 20.9, 2.7, 2.3, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 234, 34, NULL, 1156925.053, 1900584.719, 41.882977000000004, -87.69922315, 27, 'EAST GARFIELD PARK', 2, 13, '(41.882977, -87.69922315)') ,
(610299, 'William A Hinton Elementary School', 'ES', '644 W 71st St', 'Chicago', 'IL', 60621, '(773) 535-3875', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610299.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Very Weak', 18.0, NULL, NULL, 'Weak', 30.0, 'Weak', 29.0, 'Weak', NULL, NULL, NULL, 'Weak', 45.0, 'Average', 50.0, '91.70%', 28.9, '95.10%', '100.00%', NULL, NULL, 28.9, 15.2, 52.6, 59.7, 31.1, 23.0, 40.2, 45.2, 10.0, 12.5, 9.2, 3.8, 0.0, -1.4, 'Yellow', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 321, 45, NULL, 1173035.972, 1857848.3390000002, 41.76536255, -87.64132740000001, 68, 'ENGLEWOOD', 6, 7, '(41.76536255, -87.6413274)') ,
(610109, 'William Bishop Owen Scholastic Academy Elementary School', 'ES', '8247 S Christiana Ave', 'Chicago', 'IL', 60652, '(773) 535-9330', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610109.pdf', 'Midway Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 49.0, 'Strong', 61.0, 'Weak', 39.0, 'Average', 56.0, 'Weak', 51.0, 'Average', 47.0, 'Strong', 58.0, 'Average', 53.0, '96.90%', 1.2, '95.00%', '100.00%', 88.2, 77.6, 69.5, 73.2, 72.0, 70.7, 67.5, 61.3, 62.5, 53.8, 19.2, 30.8, 28.4, 29.0, 0.7, 0.1, 'Yellow', 'Yellow', 44.4, 54.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 243, 44, NULL, 1155530.649, 1849518.217, 41.74287191, -87.70571301, 70, 'ASHBURN', 18, 8, '(41.74287191, -87.70571301)') ,
(610101, 'William B Ogden Elementary School', 'ES', '24 W Walton St', 'Chicago', 'IL', 60610, '(773) 534-8110', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610101.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '95.10%', 4.5, '97.00%', '100.00%', 40.0, NULL, 67.3, 61.3, 57.8, 66.8, NULL, NULL, NULL, NULL, NULL, NULL, 40.8, 39.9, 0.3, 0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 722, 33, NULL, 1175917.983, 1906947.8590000002, 41.90003152, -87.62928901, 8, 'NEAR NORTH SIDE', 42, 18, '(41.90003152, -87.62928901)') ,
(609945, 'William C Goudy Elementary School', 'ES', '5120 N Winthrop Ave', 'Chicago', 'IL', 60640, '(773) 534-2480', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609945.pdf', 'Ravenswood-Ridge Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 70.0, 'Average', 52.0, 'Strong', 74.0, 'Strong', 61.0, 'Weak', 65.0, 'Average', 52.0, NULL, NULL, NULL, NULL, '96.10%', 6.0, '95.80%', '100.00%', 64.0, 62.9, 46.1, 32.8, 52.5, 59.5, 61.5, 48.3, 65.6, 60.7, 37.5, 51.8, 30.8, 23.6, 0.9, 1.0, 'Green', 'Green', 34.5, 100.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 805, 32, NULL, 1167897.992, 1934499.505, 41.97581137, -87.65794915, 3, 'UPTOWN', 48, 20, '(41.97581137, -87.65794915)') ,
(610143, 'William C Reavis Math & Science Specialty Elementary School', 'ES', '834 E 50th St', 'Chicago', 'IL', 60615, '(773) 535-1060', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610143.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 48.0, 'Very Weak', 6.0, 'Weak', 37.0, 'Weak', 26.0, 'Weak', 10.0, 'Very Weak', 15.0, 'Weak', 45.0, 'Average', 47.0, '93.50%', 37.2, '94.80%', '98.20%', 47.2, NULL, 31.0, 29.5, 43.5, 51.9, 19.1, 23.4, 38.5, 48.4, 3.2, 16.1, 5.8, 10.1, -0.7, 0.0, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 311, 42, NULL, 1182841.804, 1872082.7719999999, 41.80420101, -87.6049443, 39, 'KENWOOD', 4, 2, '(41.80420101, -87.6049443)') ,
(610364, 'William E B Dubois Elementary School', 'ES', '330 E 133rd St', 'Chicago', 'IL', 60827, '(773) 535-5582', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610364.pdf', 'Lake Calumet Elementary Network', 'FAR SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 47.0, NULL, NULL, 'Weak', 33.0, 'Average', 42.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Weak', 45.0, '93.30%', 48.5, '94.40%', '100.00%', 73.2, 64.8, 17.9, 11.8, 37.5, 44.4, 23.2, 20.9, 64.5, 55.7, 0.0, 12.5, 7.9, 5.0, 0.4, 0.4, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 184, 48, NULL, 1180737.575, 1817242.1530000002, 41.65376037, -87.61434156, 54, 'RIVERDALE', 9, 5, '(41.65376037, -87.61434156)') ,
(609884, 'William E Dever Elementary School', 'ES', '3436 N Osceola Ave', 'Chicago', 'IL', 60634, '(773) 534-3090', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609884.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 67.0, 'Strong', 66.0, 'Strong', 60.0, 'Strong', 78.0, 'Weak', 37.0, 'Weak', 36.0, 'Average', 50.0, 'Average', 48.0, '95.50%', 2.1, '95.90%', '100.00%', 71.1, 34.9, 60.8, 51.1, 62.9, 66.3, 73.5, 57.6, 68.6, 63.8, 20.0, 46.7, 31.5, 27.9, 0.4, 1.3, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 843, 30, NULL, 1125764.896, 1922082.0219999999, 41.94254335, -87.813168, 17, 'DUNNING', 36, 16, '(41.94254335, -87.813168)') ,
(609967, 'William F Finkl Elementary School', 'ES', '2332 S Western Ave', 'Chicago', 'IL', 60608, '(773) 535-5850', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609967.pdf', 'Pilsen-Little Village Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Average', 56.0, 'Strong', 60.0, 'Average', 50.0, 'Average', 52.0, 'Weak', 41.0, 'Average', 56.0, 'Average', 49.0, 'Strong', 54.0, '95.20%', 6.7, '96.90%', '100.00%', 58.6, NULL, 33.1, 25.2, 53.6, 47.8, 44.9, 34.8, 48.4, 50.8, 17.1, 25.7, 19.2, 9.6, 0.4, 0.7, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 588, 39, NULL, 1160699.3059999999, 1888303.5219999999, 41.84919885, -87.68570412, 31, 'LOWER WEST SIDE', 25, 10, '(41.84919885, -87.68570412)') ,
(609994, 'William G Hibbard Elementary School', 'ES', '3244 W Ainslie St', 'Chicago', 'IL', 60625, '(773) 534-5191', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609994.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Average', 50.0, '96.50%', 3.0, '96.90%', '99.30%', 70.7, 20.7, 38.3, 27.2, 40.0, 46.7, 38.8, 34.7, 33.9, 38.3, NULL, NULL, 19.3, 15.3, 0.5, -0.1, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1239, 31, NULL, 1153660.42, 1932405.9719999998, 41.97036267, -87.71036214, 14, 'ALBANY PARK', 39, 17, '(41.97036267, -87.71036214)') ,
(609812, 'William H Brown Elementary School', 'ES', '54 N Hermitage Ave', 'Chicago', 'IL', 60612, '(773) 534-7250', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609812.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 2', 'No', 'Weak', 36.0, 'Strong', 73.0, 'Average', 58.0, 'Strong', 78.0, 'Weak', 74.0, 'Strong', 73.0, 'Weak', 46.0, 'Average', 49.0, '92.80%', 59.0, '94.70%', '100.00%', 67.6, 51.4, 28.3, 15.1, 32.0, 60.0, 41.9, 20.5, 58.0, 36.2, 4.2, 4.2, 10.2, 3.2, 1.1, -0.9, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 217, 38, NULL, 1164687.1090000002, 1900646.996, 41.88298698, -87.67071879, 28, 'NEAR WEST SIDE', 27, 13, '(41.88298698, -87.67071879)') ,
(610023, 'William H King Elementary School', 'ES', '740 S Campbell Ave', 'Chicago', 'IL', 60612, '(773) 534-7898', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610023.pdf', 'Fulton Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Average', 47.0, NULL, NULL, 'Average', 44.0, 'Average', 47.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '92.40%', 17.7, '95.10%', '100.00%', 42.1, NULL, 30.1, 29.4, 57.5, 53.8, 36.8, 27.5, 65.9, 69.0, 10.0, 16.7, 10.6, 8.4, 0.3, -0.4, 'Yellow', 'Yellow', 40.6, 7.7, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 264, 38, NULL, 1159805.717, 1896555.6930000002, 41.87186209, -87.68875632, 28, 'NEAR WEST SIDE', 2, 11, '(41.87186209, -87.68875632)') ,
(609734, 'William Howard Taft High School', 'HS', '6530 W Bryn Mawr Ave', 'Chicago', 'IL', 60631, '(773) 534-1000', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609734.pdf', 'North-Northwest Side High School Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 61.0, NULL, NULL, 'Weak', 37.0, 'Weak', 32.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '89.10%', 9.8, '96.10%', '99.70%', NULL, NULL, NULL, NULL, NULL, NULL, 91.8, 93.6, 41.3, 59.8, 71.9, 88.4, 80.2, 50.6, -3.1, -1.8, 'Red', 'Red', 74.4, 60.0, 14.8, 15.7, 15.9, 16.4, 1.6, 17.9, 2.0, 32.3, 67.4, 62.2, 2922, 30, 77.3, 1131556.247, 1936856.822, 41.98298852, -87.79153829999998, 10, 'NORWOOD PARK', 41, 16, '(41.98298852, -87.7915383)') ,
(610136, 'William H Prescott Elementary School', 'ES', '1632 W Wrightwood Ave', 'Chicago', 'IL', 60614, '(773) 534-5505', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610136.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Very Strong', 90.0, NULL, NULL, 'Average', 51.0, 'Weak', 24.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 48.0, '93.50%', 73.9, '97.60%', '95.00%', NULL, NULL, 24.4, 31.7, 63.2, 46.3, 42.6, 43.5, 61.7, 65.9, NULL, NULL, 16.0, 8.9, 0.4, 0.5, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 240, 33, NULL, 1164875.342, 1917368.608, 41.92886824, -87.66955245, 7, 'LINCOLN PARK', 32, 19, '(41.92886824, -87.66955245)') ,
(610142, 'William H Ray Elementary School', 'ES', '5631 S Kimbark Ave', 'Chicago', 'IL', 60637, '(773) 535-0970', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610142.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 69.0, NULL, NULL, 'Weak', 22.0, 'Very Weak', 12.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Weak', 46.0, '95.10%', 6.7, '94.90%', '98.80%', 75.1, 46.4, 51.5, 51.5, 53.6, 44.3, 48.4, 63.1, 50.0, 50.8, NULL, NULL, 31.8, 35.2, -0.8, 1.5, 'Red', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 654, 46, NULL, 1185667.78, 1867788.013, 41.79234967, -87.59471544, 41, 'HYDE PARK', 5, 2, '(41.79234967, -87.59471544)') ,
(610153, 'William H Ryder Math & Science Specialty Elementary School', 'ES', '8716 S Wallace St', 'Chicago', 'IL', 60620, '(773) 535-3843', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610153.pdf', 'Englewood-Gresham Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 34.0, NULL, NULL, 'Average', 42.0, 'Average', 50.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '93.00%', 22.5, '94.30%', '96.40%', 73.6, 56.9, 21.6, 6.8, 32.7, 37.5, 37.7, 17.7, 44.6, 37.0, 20.5, 33.3, 10.6, 5.6, -0.5, -1.6, 'Yellow', 'Red', 27.5, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 302, 49, NULL, 1173792.13, 1846923.856, 41.73536767, -87.63887904, 71, 'AUBURN GRESHAM', 21, 22, '(41.73536767, -87.63887904)') ,
(610167, 'William H Seward Communication Arts Academy Elementary School', 'ES', '4600 S Hermitage Ave', 'Chicago', 'IL', 60609, '(773) 535-4890', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610167.pdf', 'Pershing Elementary Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 40.0, 'Average', 43.0, 'Weak', 38.0, 'Average', 41.0, 'Weak', 66.0, 'Strong', 63.0, 'Average', 49.0, 'Average', 50.0, '95.70%', 14.4, '95.70%', '100.00%', 58.6, NULL, 25.2, 28.2, 49.1, 58.8, 45.3, 33.9, 57.8, 51.3, 12.5, 25.0, 12.2, 7.9, -1.1, -0.5, 'Red', 'Yellow', 43.3, 46.4, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 896, 42, NULL, 1165412.465, 1874162.7769999998, 41.81029632, -87.66880778, 61, 'NEW CITY', 20, 9, '(41.81029632, -87.66880778)') ,
(609698, 'William J Bogan High School', 'HS', '3939 W 79th St', 'Chicago', 'IL', 60652, '(773) 535-2180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609698.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 20.0, NULL, NULL, 'Very Weak', 18.0, 'Weak', 29.0, 'Weak', NULL, NULL, NULL, 'Weak', 41.0, 'Weak', 38.0, '75.20%', 5.8, '94.30%', '99.60%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 13.4, 13.3, 14.3, 14.3, 0.9, 15.2, 0.9, 13.1, 52.5, 50.0, 1656, 44, 67.1, 1151315.548, 1851847.8590000002, 41.74934817, -87.72109673, 70, 'ASHBURN', 18, 8, '(41.74934817, -87.72109673)') ,
(610061, 'William J & Charles H Mayo Elementary School', 'ES', '249 E 37th St', 'Chicago', 'IL', 60653, '(773) 535-1260', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610061.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 52.0, 'Average', 55.0, 'Strong', 62.0, 'Strong', 65.0, 'Weak', 52.0, 'Average', 47.0, 'Average', 50.0, 'Strong', 55.0, '95.30%', 26.0, '95.20%', '100.00%', 48.3, 24.1, 28.2, 25.2, 45.9, 58.0, 40.0, 36.6, 57.4, 53.7, 10.0, 30.0, 15.5, 6.0, 0.7, -0.3, 'Green', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 453, 40, NULL, 1178726.235, 1880476.124, 41.82732778, -87.61978251, 35, 'DOUGLAS', 3, 2, '(41.82732778, -87.61978251)') ,
(610104, 'William J Onahan Elementary School', 'ES', '6634 W Raven St', 'Chicago', 'IL', 60631, '(773) 534-1180', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610104.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Strong', 64.0, 'Average', 59.0, 'Weak', 29.0, 'Weak', 27.0, 'Weak', 32.0, 'Weak', 39.0, 'Average', 47.0, 'Weak', 44.0, '95.70%', 6.9, '96.10%', '98.80%', 76.8, 56.2, 62.7, 61.2, 40.8, 53.9, 60.2, 63.8, 49.7, 44.8, 44.0, 54.0, 35.1, 29.6, 0.2, 0.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 687, 30, NULL, 1131111.88, 1940800.44, 41.99381787, -87.79308113, 10, 'NORWOOD PARK', 41, 16, '(41.99381787, -87.79308113)') ,
(609678, 'William Jones College Preparatory High School', 'HS', '606 S State St', 'Chicago', 'IL', 60605, '(773) 534-8600', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609678.pdf', 'South Side High School Network', 'SOUTH SIDE COLLABORATIVE', 'Yes', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Very Strong', 92.0, NULL, NULL, 'Strong', 64.0, 'Strong', 67.0, 'Weak', NULL, NULL, NULL, NULL, NULL, NULL, NULL, '93.80%', 4.5, '95.30%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 19.6, 20.3, 21.3, 21.1, 1.5, 24.8, 3.5, 83.3, 90.1, 86.9, 871, 38, 92.8, 1176412.354, 1897618.0880000002, 41.87441898, -87.62775497, 32, 'LOOP', 2, 1, '(41.87441898, -87.62775497)') ,
(610193, 'William K New Sullivan Elementary School', 'ES', '8331 S Mackinaw', 'Chicago', 'IL', 60617, '(773) 535-6585', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610193.pdf', 'Skyway Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Weak', 27.0, 'Average', 42.0, 'Weak', 35.0, 'Average', 40.0, 'Weak', 58.0, 'Weak', 33.0, 'Average', 47.0, 'Strong', 54.0, '90.30%', 27.3, '94.00%', '97.80%', 34.1, 17.1, 17.8, 14.1, 42.9, 51.5, 10.2, 21.0, 56.6, 52.5, 0.0, 6.7, 2.9, 5.8, -0.9, 0.2, 'Red', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 527, 47, NULL, 1199962.656, 1850233.5369999998, 41.743830200000005, -87.542891, 46, 'SOUTH CHICAGO', 10, 4, '(41.7438302, -87.542891)') ,
(610123, 'William Penn Elementary School', 'ES', '1616 S Avers Ave', 'Chicago', 'IL', 60623, '(773) 534-1665', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610123.pdf', 'Austin-North Lawndale Elementary Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Strong', 78.0, NULL, NULL, 'Very Strong', 99.0, 'Very Strong', 99.0, 'Weak', NULL, NULL, NULL, 'Average', 52.0, 'Average', 53.0, '91.80%', 5.7, '95.20%', '92.90%', 42.7, NULL, 22.4, 17.6, 53.0, 51.5, 16.5, 15.7, 42.5, 51.3, 8.7, 17.4, 8.1, 5.5, 0.3, -0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 423, 37, NULL, 1150967.365, 1891574.2419999999, 41.85836995, -87.72133609, 29, 'NORTH LAWNDALE', 24, 10, '(41.85836995, -87.72133609)') ,
(609949, 'William P Gray Elementary School', 'ES', '3730 N Laramie Ave', 'Chicago', 'IL', 60641, '(773) 534-3520', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609949.pdf', 'O Hare Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 1', 'No', 'Average', 57.0, NULL, NULL, 'Average', 44.0, 'Weak', 35.0, 'Weak', NULL, NULL, NULL, 'Average', 47.0, 'Average', 50.0, '95.60%', 5.9, '95.90%', '99.40%', 74.1, NULL, 53.5, 51.4, 69.1, 77.5, 43.0, 53.5, 61.0, 70.1, 16.3, 28.7, 26.1, 19.4, 0.5, 0.9, 'Green', 'Green', 46.1, 34.9, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1345, 29, NULL, 1140988.981, 1924452.515, 41.94878107, -87.75715297, 15, 'PORTAGE PARK', 38, 16, '(41.94878107, -87.75715297)') ,
(610097, 'William P Nixon Elementary School', 'ES', '2121 N Keeler Ave', 'Chicago', 'IL', 60639, '(773) 534-4375', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610097.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 52.0, NULL, NULL, 'Strong', 70.0, 'Strong', 62.0, 'Weak', NULL, NULL, NULL, 'Weak', 46.0, 'Average', 47.0, '95.60%', 3.5, '95.80%', '100.00%', 66.9, 25.0, 31.8, 27.8, 52.4, 57.3, 35.5, 20.7, 57.0, 50.0, NULL, NULL, 11.2, 9.6, -0.1, -0.2, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 1061, 34, NULL, 1148073.735, 1913864.6630000002, 41.91959342, -87.73138368, 20, 'HERMOSA', 30, 25, '(41.91959342, -87.73138368)') ,
(609711, 'William Rainey Harper High School', 'HS', '6520 S Wood St', 'Chicago', 'IL', 60636, '(773) 535-9150', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609711.pdf', 'Southwest Side High School Network', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', 'Weak', 22.0, 'Weak', 32.0, 'Weak', 39.0, 'Average', 42.0, 'Weak', 31.0, 'Weak', 31.0, 'Average', 49.0, 'Average', 53.0, '73.00%', 63.6, '94.70%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.2, 11.9, 13.1, 13.4, 1.2, 14.4, 1.3, 5.9, 37.0, 40.3, 621, 43, 64.2, 1165424.867, 1861341.255, 41.77511222, -87.66912563, 67, 'WEST ENGLEWOOD', 15, 7, '(41.77511222, -87.66912563)') ,
(610232, 'Williams Multiplex Elementary School', 'ES', '2710 S Dearborn St', 'Chicago', 'IL', 60616, '(773) 534-9245', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610232.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 2', 'No', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Average', 50.0, '91.60%', 17.0, '95.90%', '100.00%', 45.3, 31.1, 22.3, 25.9, 48.1, 58.1, NULL, NULL, NULL, NULL, NULL, NULL, 24.3, 27.9, 1.1, 2.5, 'Green', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 266, 40, NULL, 1176275.784, 1886545.486, 41.84403809, -87.62859008, 35, 'DOUGLAS', 3, 1, '(41.84403809, -87.62859008)') ,
(610336, 'Williams Preparatory Academy Middle School', 'MS', '2710 S Dearborn St', 'Chicago', 'IL', 60616, '(773) 534-9235', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610336.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Average', 49.0, NULL, NULL, 'Average', 42.0, 'Average', 48.0, 'Weak', NULL, NULL, NULL, 'Average', 53.0, 'Strong', 56.0, '93.30%', 14.8, '92.40%', '96.80%', NULL, NULL, NULL, NULL, NULL, NULL, 16.2, 26.1, 56.3, 50.0, 2.8, 11.1, 7.8, 15.5, 0.4, 3.3, 'Yellow', 'Green', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 125, 40, NULL, 1176275.784, 1886545.486, 41.84403809, -87.62859008, 35, 'DOUGLAS', 3, 1, '(41.84403809, -87.62859008)') ,
(610172, 'William T Sherman Elementary School', 'ES', '1000 W 52nd St', 'Chicago', 'IL', 60609, '(773) 535-1757', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610172.pdf', 'AUSL Schools', 'SOUTHWEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Probation', 'Level 3', 'No', 'Weak', 32.0, NULL, NULL, 'Average', 46.0, 'Average', 55.0, 'Weak', NULL, NULL, NULL, 'Average', 49.0, 'Average', 52.0, '92.30%', 230.6, '95.00%', '100.00%', NULL, NULL, 21.8, 26.8, 41.1, 42.0, 30.1, 19.7, 46.3, 35.2, 7.5, 15.1, 8.1, 1.7, 0.7, -1.3, 'Green', 'Red', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 462, 45, NULL, 1170500.817, 1870373.159, 41.79978772, -87.65025483, 61, 'NEW CITY', 16, 9, '(41.79978772, -87.65025483)') ,
(609844, 'William W Carter Elementary School', 'ES', '5740 S Michigan Ave', 'Chicago', 'IL', 60637, '(773) 535-0860', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609844.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Probation', 'Level 3', 'No', 'Very Weak', 13.0, 'Average', 49.0, 'Weak', 33.0, 'Weak', 35.0, 'Weak', 56.0, 'Strong', 62.0, 'Weak', 46.0, 'Weak', 46.0, '91.20%', 27.0, '95.90%', '100.00%', 72.3, 46.6, 21.8, 17.8, 50.5, 57.6, 24.8, 24.8, 44.9, 48.6, 13.3, 33.3, 6.8, 5.5, -0.2, -1.2, 'Yellow', 'Red', 42.4, 50.0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 371, 42, NULL, 1178101.365, 1866810.1230000001, 41.78984129, -87.62248974, 40, 'WASHINGTON PARK', 20, 2, '(41.78984129, -87.62248974)') ,
(610088, 'Wolfgang A Mozart Elementary School', 'ES', '2200 N Hamlin Ave', 'Chicago', 'IL', 60647, '(773) 534-4160', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610088.pdf', 'Fullerton Elementary Network', 'NORTH-NORTHWEST SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Average', 41.0, NULL, NULL, 'Average', 56.0, 'Weak', 32.0, 'Weak', NULL, NULL, NULL, 'Average', 50.0, 'Strong', 54.0, '95.20%', 3.6, '96.40%', '100.00%', 39.8, 10.1, 28.7, 34.2, 51.1, 48.2, 46.7, 32.6, 74.7, 64.0, NULL, NULL, 12.0, 11.0, 0.4, 0.3, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 748, 34, NULL, 1150644.396, 1914368.955, 41.92092734, -87.72192541, 22, 'LOGAN SQUARE', 35, 25, '(41.92092734, -87.72192541)') ,
(609977, 'Woodlawn Community Elementary School', 'ES', '6657 S Kimbark Ave', 'Chicago', 'IL', 60637, '(773) 535-0801', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609977.pdf', 'Burnham Park Elementary Network', 'SOUTH SIDE COLLABORATIVE', 'No', 'Standard', 'Not on Probation', 'Level 2', 'No', 'Strong', 70.0, NULL, NULL, 'Very Strong', 80.0, 'Strong', 66.0, 'Weak', NULL, NULL, NULL, 'Strong', 59.0, 'Strong', 54.0, '93.90%', 12.4, '94.30%', '100.00%', 57.7, NULL, 43.9, 45.8, 60.0, 62.4, 75.0, 30.0, 88.9, 55.6, NULL, NULL, 17.5, 22.2, 0.1, 0.0, 'Yellow', 'Yellow', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 238, 46, NULL, 1185825.188, 1860883.5790000001, 41.77339962, -87.59435584, 42, 'WOODLAWN', 5, 3, '(41.77339962, -87.59435584)') ,
(610392, 'World Language Academy High School', 'HS', '3120 S Kostner Ave', 'Chicago', 'IL', 60623, '(773) 535-4334', 'http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610392.pdf', 'West Side High School Network', 'WEST SIDE COLLABORATIVE', 'No', 'Track_E', 'Not on Probation', 'Level 2', 'No', 'Average', 51.0, 'Weak', 33.0, 'Average', 49.0, 'Average', 47.0, 'Weak', 49.0, 'Average', 49.0, 'Average', 50.0, 'Weak', 46.0, '91.60%', 4.0, '96.00%', '100.00%', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, 12.9, 13.4, 14.6, 14.1, 1.2, 16.2, 1.6, 14.6, 69.4, 48.1, 382, 37, 76.0, 1147521.3020000001, 1883405.128, 41.83601953, -87.73419465, 30, 'SOUTH LAWNDALE', 22, 10, '(41.83601953, -87.73419465)')
;

 * ibm_db_sa://bvr08213:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
566 rows affected.


[]

# Closing Connection

In [27]:
# closing the connection

ibm_db.close(conn)

True